In [67]:
import pandas as pd
import requests
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import re
import time
import json
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.support.ui import Select

%matplotlib inline

In [68]:
df=pd.read_csv('county_income.csv')

In [69]:
df=df.drop(columns='GeographicArea')

In [70]:
df.head(3)

,county,facility_counts,MedianHouseholdIncome,MarginofError(+/-)
0,Denver,2071,60098,950
1,El Paso,666,62535,844
2,Pueblo,509,42386,1201


In [71]:
# df['string']=df.apply(lambda x:'%s county has a Median Household Income of %s (with a margin of error of %s) and %s registered retail Marijuana facilities'% (df['county'],df['MedianHouseholdIncome'],df['MarginofError(+/-)'],df['facility_counts']),axis=1)

In [72]:
df['string']=df['county'].astype(str)+' county has a Median Household Income of ' +'$' +df['MedianHouseholdIncome'].astype(str) + ' and ' +df['facility_counts'].astype(str)+ ' licensed retail marijuana facilities.'

In [52]:
df['MedianHouseholdIncome'].describe()

count        51.000000
mean      55704.196078
std       16562.781341
min       29000.000000
25%       43577.500000
50%       53010.000000
75%       66496.000000
max      111154.000000
Name: MedianHouseholdIncome, dtype: float64

In [94]:
df.corr()

,facility_counts,MedianHouseholdIncome,MarginofError(+/-)
facility_counts,1.000000,0.101272,-0.344775
MedianHouseholdIncome,0.101272,1.000000,-0.139215
MarginofError(+/-),-0.344775,-0.139215,1.000000


In [73]:
#Sorting Counties into Income Groups
df.loc[df.MedianHouseholdIncome <= 43577.500000, 'group'] = '1' 
df.loc[(df.MedianHouseholdIncome > 43577.500000) & (df.MedianHouseholdIncome <=53010.00000), 'group'] = '2' 
df.loc[(df.MedianHouseholdIncome > 45301.00000) & (df.MedianHouseholdIncome <=66496.000000), 'group'] = '3'
df.loc[(df.MedianHouseholdIncome >=66496.000000), 'group'] = '4'
df.loc[df.group=='1', 'income_quartile'] = 'Bottom Income Quartile' 
df.loc[df.group=='2', 'income_quartile'] = 'Third Income Quartile'
df.loc[df.group=='3', 'income_quartile'] = 'Second Income Quartile' 
df.loc[df.group=='4', 'income_quartile'] = 'Top Income Quartile' 

In [74]:
output = df.groupby('county')['string'].apply(lambda x: "<div id='county'><P>%s</P></div>" % '</p><p> '.join(x)).reset_index(name='properties.article')
output

,county,properties.article
0,Adams,<div id='county'><P>Adams county has a Median ...
1,Alamosa,<div id='county'><P>Alamosa county has a Media...
2,Arapahoe,<div id='county'><P>Arapahoe county has a Medi...
3,Archuleta,<div id='county'><P>Archuleta county has a Med...
4,Bent,<div id='county'><P>Bent county has a Median H...
5,Boulder,<div id='county'><P>Boulder county has a Media...
6,Chaffee,<div id='county'><P>Chaffee county has a Media...
7,Clear Creek,<div id='county'><P>Clear Creek county has a M...
8,Conejos,<div id='county'><P>Conejos county has a Media...
9,Costilla,<div id='county'><P>Costilla county has a Medi...


In [75]:
facilities = df[['county','facility_counts','income_quartile']]
output = output.merge(facilities, how='left', on='county')
group = df[['county','group']]
output = output.merge(group, how='left', on='county')
output = output.rename(index=str, columns={"county": "properties.name", "facility_counts": "properties.headline","group":"properties.group_id","income_quartile":"properties.group_name"})
output['properties.headline'] = output['properties.headline'].map(str) + " licensed marijuana businesses"
output['properties.name'] = output['properties.name'].map(str) + " County"

In [76]:
geometry=[{"type":"Feature","geometry":{"type":"Polygon","coordinates":[[[-103.70570099999992,40.00136600000013],[-103.70569499999999,39.98511399999995],[-103.70601999999997,39.973374999999976],[-103.70624399999991,39.95888000000025],[-103.70617099999993,39.95397300000013],[-103.70617099999993,39.93264399999998],[-103.70610299999998,39.908538000000135],[-103.70629499999995,39.888999000000126],[-103.70630299999999,39.857679000000076],[-103.70626299999998,39.82854800000007],[-103.70637999999997,39.79605900000007],[-103.70651900000001,39.78349100000008],[-103.70657699999992,39.76555300000007],[-103.70654699999994,39.739894000000106],[-103.72393599999998,39.739785000000154],[-103.79395699999992,39.74004500000001],[-103.80019900000002,39.7402010000003],[-103.828213,39.74024600000013],[-103.86606299999994,39.74023599999998],[-103.8755099999999,39.74029799999994],[-103.90751599999999,39.74065400000029],[-103.91928799999994,39.74067000000019],[-103.94049999999999,39.740521000000115],[-103.96551599999992,39.740518000000066],[-103.9723029999999,39.74040100000013],[-103.97434099999992,39.73998400000028],[-103.97763599999979,39.740267000000244],[-104.00348300000002,39.740170000000205],[-104.0407249999999,39.739981],[-104.05940299999997,39.739971999999966],[-104.11506899999989,39.739774000000125],[-104.13428199999993,39.73960400000027],[-104.15313400000002,39.73949700000014],[-104.20952499999987,39.7390180000001],[-104.26498699999985,39.738877],[-104.29796399999998,39.738680000000045],[-104.30356999999975,39.738515000000234],[-104.34008900000003,39.73824800000017],[-104.3587599999999,39.738197000000014],[-104.39186199999989,39.73804000000018],[-104.42669299999994,39.73809099999994],[-104.4591309999999,39.738178999999946],[-104.50737799999996,39.738249999999994],[-104.55888400000003,39.739330000000166],[-104.59880900000002,39.739844999999946],[-104.62464299999988,39.740077000000156],[-104.63042200000001,39.73950000000019],[-104.64072299999992,39.739885999999956],[-104.64415199999996,39.73972000000009],[-104.65280299999978,39.739778000000285],[-104.66026999999997,39.74048200000027],[-104.67830499999997,39.739995000000135],[-104.71063299999986,39.74012399999998],[-104.71066699999994,39.740254999999934],[-104.725637,39.740267000000244],[-104.73195499999997,39.74082200000015],[-104.73564499999992,39.74025400000005],[-104.75412699999987,39.74022800000006],[-104.80140399999993,39.740286000000026],[-104.80884500000002,39.74014200000005],[-104.82168999999993,39.74029200000018],[-104.843254,39.74030699999997],[-104.84709099999992,39.740207000000055],[-104.88420699999995,39.74021199999993],[-104.88464599999998,39.7401560000003],[-104.88466399999993,39.74746700000003],[-104.87997099999995,39.74743200000012],[-104.87990100000002,39.75473500000004],[-104.85872499999994,39.75475900000009],[-104.84693999999996,39.75468699999999],[-104.84822999999994,39.75648500000011],[-104.8512199999999,39.756981999999994],[-104.85643099999993,39.758387000000255],[-104.85663099999994,39.76028600000018],[-104.85653100000002,39.767818999999974],[-104.8563769999999,39.76867300000026],[-104.8375959999999,39.76726700000029],[-104.83296100000001,39.76619500000004],[-104.83300899999989,39.76917700000007],[-104.82832299999995,39.769156000000066],[-104.82822899999991,39.77317399999998],[-104.80667399999999,39.773190000000056],[-104.78172699999999,39.773387000000014],[-104.78181799999987,39.77648500000004],[-104.77215699999999,39.776408999999944],[-104.77216599999991,39.77146600000026],[-104.76762699999989,39.77168700000004],[-104.76455999999996,39.77159799999998],[-104.76457600000003,39.769390999999985],[-104.76077499999997,39.769488000000194],[-104.73665499999998,39.76939300000015],[-104.73460099999994,39.769181],[-104.73442,39.78968900000007],[-104.73439899999994,39.79844200000008],[-104.74458400000003,39.79850800000003],[-104.762606,39.798433000000045],[-104.76283399999988,39.79987800000009],[-104.76263199999994,39.80367800000005],[-104.762901,39.813038000000006],[-104.76246999999995,39.82297999999997],[-104.71709599999986,39.82321900000005],[-104.6799529999999,39.822869000000026],[-104.66627699999998,39.822850000000074],[-104.66619600000001,39.81400500000012],[-104.65545500000002,39.81404800000013],[-104.65551099999993,39.82247000000007],[-104.63967300000002,39.82255199999992],[-104.63964599999991,39.82689799999997],[-104.630221,39.82678500000003],[-104.62743799999993,39.826888000000224],[-104.61990599999996,39.82663300000007],[-104.619709,39.84323100000029],[-104.61981100000003,39.855781000000036],[-104.61984100000001,39.87376700000016],[-104.60579099999995,39.87382699999995],[-104.60585300000002,39.882244999999955],[-104.600302,39.88228900000024],[-104.600435,39.89945899999998],[-104.62584400000003,39.89929599999999],[-104.63396899999987,39.899289000000124],[-104.63385799999992,39.90606699999995],[-104.63409200000001,39.90619100000009],[-104.68021299999992,39.906599000000085],[-104.68026099999997,39.91424699999999],[-104.69293399999987,39.91418300000021],[-104.69260600000001,39.906614000000104],[-104.70537499999995,39.90641800000003],[-104.70556799999991,39.898490000000095],[-104.73154999999986,39.89861099999996],[-104.73190299999993,39.87533300000024],[-104.73185899999993,39.855828999999915],[-104.7321379999999,39.845085999999924],[-104.75051299999978,39.845434000000125],[-104.76213899999993,39.8454440000001],[-104.762317,39.841874000000075],[-104.7908589999999,39.842009000000076],[-104.79070799999994,39.8272140000002],[-104.79105399999997,39.81046200000003],[-104.79089499999998,39.79825100000028],[-104.80950999999999,39.798317000000225],[-104.827922,39.798208999999986],[-104.82913100000002,39.798059000000194],[-104.84230699999995,39.79815700000012],[-104.84934199999992,39.798347000000035],[-104.86604099999988,39.79838600000005],[-104.86607500000002,39.81298599999997],[-104.88458499999996,39.81297499999994],[-104.88474599999978,39.7983220000001],[-104.89690499999989,39.79836400000022],[-104.89693299999999,39.801158999999984],[-104.900394,39.80113500000016],[-104.90045199999992,39.798377000000016],[-104.90356099999985,39.79839099999998],[-104.90343799999988,39.78380800000002],[-104.92700200000002,39.78377800000021],[-104.92703399999999,39.79105400000003],[-104.94048699999996,39.79100199999999],[-104.94047599999993,39.79460100000006],[-104.955356,39.79468600000018],[-104.96026799999999,39.790966000000026],[-104.968861,39.790974000000176],[-104.96775799999995,39.791721000000166],[-104.966611,39.793836],[-104.96876499999996,39.79383999999999],[-104.9687159999998,39.798396000000196],[-104.973143,39.79829200000029],[-104.97324300000002,39.79630199999997],[-104.97484699999995,39.79561000000007],[-104.97483799999992,39.79422599999998],[-104.97324500000002,39.79413100000011],[-104.97323499999987,39.79247000000015],[-104.97605699999997,39.79230100000001],[-104.97595000000001,39.79103800000013],[-104.990434,39.79108300000013],[-104.99080900000001,39.792139000000134],[-104.99333999999999,39.79208199999999],[-104.99343599999986,39.79118599999998],[-104.99998600000004,39.79103500000025],[-105.02078699999998,39.791089000000284],[-105.02083700000003,39.79278599999998],[-105.02514899999994,39.794002999999975],[-105.02518999999995,39.790933000000166],[-105.03003799999999,39.79108600000001],[-105.02998199999996,39.79187999999999],[-105.0346419999999,39.791870000000245],[-105.03461799999985,39.791142000000036],[-105.05324599999989,39.791063000000065],[-105.05316199999987,39.799239000000114],[-105.05282899999986,39.80387400000029],[-105.05325600000003,39.80649700000026],[-105.05328199999997,39.821125000000166],[-105.05318799999998,39.849649],[-105.0528349999999,39.85254500000002],[-105.05333799999977,39.85648500000002],[-105.05313999999993,39.87521600000014],[-105.053042,39.896838],[-105.05289199999993,39.91422400000022],[-105.03434700000003,39.91418100000021],[-105.03439499999996,39.921305000000075],[-105.023551,39.92124800000016],[-105.02354500000001,39.9192250000001],[-105.02155199999987,39.91920700000003],[-105.02159099999989,39.92136700000009],[-105.01546899999988,39.921402000000114],[-105.01565699999998,39.94313099999994],[-104.99709999999988,39.943239000000005],[-104.99715200000003,39.94677100000001],[-105.00626499999998,39.94680499999998],[-105.00629800000002,39.9504390000003],[-105.01222499999989,39.95044600000017],[-105.015039,39.951457000000005],[-105.01711699999998,39.95281399999999],[-105.01556499999992,39.955188000000305],[-105.01569099999995,39.97043700000012],[-105.01342299999999,39.97044799999992],[-105.01340899999997,39.97226499999999],[-105.01572199999987,39.97227600000019],[-105.01584500000001,39.981193000000076],[-105.00696299999998,39.98126700000017],[-104.99742299999997,39.98121300000014],[-104.99739799999986,39.97842000000014],[-104.99840399999994,39.97758600000026],[-104.99838899999997,39.97580500000015],[-104.99737899999997,39.97578500000009],[-104.99724999999995,39.96852899999999],[-104.98810800000001,39.96846800000026],[-104.98810800000001,39.97217499999999],[-104.99706500000002,39.97214900000017],[-104.99707999999993,39.973954000000106],[-104.99437999999998,39.97395300000022],[-104.99441099999996,39.975802999999985],[-104.98778400000003,39.97574600000024],[-104.9877459999999,39.98648300000008],[-104.987052,39.988529000000085],[-104.98244299999999,39.997090000000014],[-104.98094400000002,40.000324999999975],[-104.961408,40.000337000000286],[-104.90478899999994,40.00032299999998],[-104.84879799999987,40.00038300000017],[-104.8003169999999,40.000430000000165],[-104.78853100000003,40.00040700000005],[-104.74592899999993,40.000496999999996],[-104.71066099999996,40.0005000000001],[-104.665437,40.00053099999997],[-104.65098499999999,40.00058200000029],[-104.63155199999989,40.00053099999997],[-104.62440099999992,40.00057000000004],[-104.60185999999987,40.00052899999997],[-104.59234800000002,40.00059299999998],[-104.54679599999986,40.000602000000185],[-104.52724199999994,40.000628000000006],[-104.51110199999988,40.00058400000029],[-104.49997899999988,40.0006250000003],[-104.45226000000002,40.0006250000003],[-104.44010999999995,40.00071900000006],[-104.40365999999995,40.00074200000006],[-104.39537799999994,40.000822000000255],[-104.38137999999992,40.00069000000002],[-104.30149599999993,40.000765],[-104.26738599999993,40.00092100000029],[-104.21926500000001,40.000829000000124],[-104.16933399999988,40.000781000000245],[-104.15033199999999,40.000856000000226],[-104.077024,40.000974999999926],[-104.07557600000001,40.001031000000125],[-104.03905399999985,40.00106700000009],[-104.03714999999994,40.001125000000286],[-104.0005049999998,40.001059],[-103.99767799999995,40.00115100000011],[-103.95460099999991,40.001132000000155],[-103.94653499999998,40.00117200000011],[-103.89091000000002,40.001221000000044],[-103.86774300000002,40.001198999999986],[-103.86364399999991,40.00147000000004],[-103.86146699999989,40.001215000000116],[-103.78762399999988,40.00132000000025],[-103.73379299999999,40.00133600000015],[-103.729803,40.00121200000001],[-103.71679699999999,40.00140699999997],[-103.70570099999992,40.00136600000013]]]},"properties":{"OBJECTID":1,"COUNTYFP":"001","Shape_Leng":4.32102125651,"Shape_Area":0.322757892201,"low":132545,"lowmod":231255,"LMMI":334950,"lowmoduniv":467200,"lowmod_pct":0.494980736301,"NAMELSAD10":"Adams County"}},
{"type":"Feature","geometry":{"type":"Polygon","coordinates":[[[-105.45725499999992,37.751464],[-105.45696399999991,37.748646000000065],[-105.45734199999993,37.74702400000001],[-105.45668899999998,37.74599500000011],[-105.45426099999997,37.744462999999996],[-105.45276399999995,37.74040499999995],[-105.45504799999992,37.73658700000027],[-105.455104,37.733601000000135],[-105.45358499999992,37.73198400000007],[-105.45361600000001,37.729813000000036],[-105.45436599999988,37.72606500000006],[-105.45424500000001,37.72395099999994],[-105.45510999999999,37.72315200000014],[-105.45715599999988,37.7198640000002],[-105.45963099999994,37.71792600000026],[-105.46165000000002,37.717419000000234],[-105.46325999999982,37.71599800000007],[-105.46392599999996,37.71380099999999],[-105.46659299999999,37.71184499999998],[-105.46797499999985,37.711260000000266],[-105.47111599999988,37.71124800000001],[-105.47212799999994,37.70946800000013],[-105.47449299999994,37.70760100000001],[-105.47730200000001,37.70768600000014],[-105.47807099999989,37.70577800000001],[-105.47792099999998,37.704162999999994],[-105.479062,37.700155000000166],[-105.48047600000001,37.69807600000013],[-105.48016599999994,37.69663800000012],[-105.48137399999996,37.69375500000001],[-105.48086899999993,37.69185600000026],[-105.48089900000002,37.68718000000024],[-105.47875699999997,37.68613900000008],[-105.47770699999995,37.68461100000013],[-105.47738900000002,37.681202000000155],[-105.47665099999989,37.67916400000013],[-105.476899,37.675850000000025],[-105.478519,37.67334500000021],[-105.47878699999995,37.67210200000022],[-105.47856799999988,37.668476000000226],[-105.4772769999999,37.667158000000086],[-105.477351,37.66574700000001],[-105.48111999999992,37.66170300000027],[-105.48145099999994,37.658525000000225],[-105.48540200000002,37.654730000000086],[-105.48697600000003,37.65348299999994],[-105.48685799999987,37.64966199999998],[-105.4878159999999,37.644521999999995],[-105.49020699999994,37.640386000000035],[-105.49035100000003,37.638667999999996],[-105.492388,37.634783000000084],[-105.49183699999998,37.63381700000008],[-105.49219799999992,37.62637400000011],[-105.491513,37.62500300000005],[-105.49260700000002,37.62306700000005],[-105.49638600000003,37.619185000000016],[-105.497299,37.616749000000084],[-105.49829199999999,37.61571600000025],[-105.498602,37.61413700000014],[-105.49792400000001,37.61306800000017],[-105.49609299999997,37.612348999999995],[-105.49483099999998,37.60849899999994],[-105.49304099999995,37.60528699999992],[-105.49349299999994,37.604449000000045],[-105.49327299999976,37.600919000000204],[-105.50128599999994,37.59860300000025],[-105.501755,37.5975710000003],[-105.50091299999997,37.59609700000004],[-105.495635,37.59244699999999],[-105.49459100000001,37.59235200000012],[-105.49413500000003,37.58907700000003],[-105.49468300000001,37.585541999999975],[-105.49215499999991,37.58268100000021],[-105.49057399999992,37.582620000000304],[-105.48594899999995,37.57927900000027],[-105.48551600000002,37.577899000000116],[-105.48834099999999,37.57691400000016],[-105.49066799999997,37.57546200000013],[-105.49278299999992,37.572948999999994],[-105.49422700000002,37.572173000000134],[-105.49605999999989,37.57006000000007],[-105.49730499999998,37.56685999999996],[-105.49630999999988,37.56385199999994],[-105.50056699999993,37.560141000000044],[-105.53596199999976,37.530558000000156],[-105.54240600000003,37.52531800000003],[-105.55409599999996,37.515598000000125],[-105.56101799999999,37.50973400000004],[-105.57287899999994,37.49989700000026],[-105.57548999999989,37.497919000000195],[-105.59701899999988,37.4801750000002],[-105.60232499999995,37.475031],[-105.62163399999991,37.45944300000008],[-105.6365889999999,37.44630600000005],[-105.69005900000002,37.401638000000105],[-105.72195299999998,37.37500799999998],[-105.73396000000002,37.364874999999984],[-105.74333899999993,37.35679800000031],[-105.75057400000003,37.3569770000002],[-105.75931100000003,37.35668700000019],[-105.79596300000003,37.356359000000225],[-105.80530299999987,37.3562160000003],[-105.82038999999992,37.35620000000006],[-105.83720499999987,37.356363000000215],[-105.83837599999993,37.356263000000126],[-105.847692,37.35645299999999],[-105.85144300000002,37.356273000000215],[-105.85758499999991,37.35636100000022],[-105.87481799999989,37.356313],[-105.89812899999987,37.35642800000005],[-105.90996799999988,37.35630800000007],[-105.927324,37.3563640000001],[-105.929664,37.356224999999995],[-105.93826499999994,37.356527000000085],[-105.94731300000001,37.356615999999974],[-105.94980899999996,37.35647999999992],[-105.96999099999994,37.35628299999996],[-105.978341,37.35634700000014],[-105.98352799999992,37.35624600000011],[-106.00214099999977,37.35653700000006],[-106.01028600000001,37.35659700000002],[-106.03891199999987,37.356952999999976],[-106.03918499999997,37.37155999999993],[-106.03933099999989,37.40085200000027],[-106.03939299999996,37.41350800000009],[-106.03934199999992,37.4314950000001],[-106.03934399999997,37.464533000000074],[-106.03928699999989,37.48452300000025],[-106.03926899999999,37.52922100000001],[-106.03927499999998,37.564571000000115],[-106.03931499999999,37.57482200000027],[-106.03921699999995,37.58367100000004],[-106.03916500000003,37.61783800000012],[-106.03908899999993,37.64548599999995],[-106.03886799999987,37.675849000000085],[-106.03857099999988,37.7242940000001],[-106.03841299999988,37.74835300000018],[-106.01959099999988,37.74841700000013],[-105.99258299999997,37.74868600000019],[-105.96253899999994,37.74878600000011],[-105.92791599999987,37.74884900000018],[-105.91008099999999,37.749135000000024],[-105.872481,37.74959799999999],[-105.81944499999997,37.7502970000001],[-105.809327,37.750451000000055],[-105.79026799999991,37.75063500000016],[-105.75057300000003,37.75092200000029],[-105.73682500000001,37.75128500000011],[-105.70149499999997,37.751458000000014],[-105.64356099999998,37.75174800000008],[-105.62556899999998,37.751888000000065],[-105.61726899999991,37.75230600000003],[-105.596295,37.75209500000017],[-105.51404600000001,37.7512730000002],[-105.50056599999994,37.75117799999998],[-105.49797899999987,37.751003000000026],[-105.45725499999992,37.751464]]]},"properties":{"OBJECTID":2,"COUNTYFP":"003","Shape_Leng":1.83561891436,"Shape_Area":0.191063932732,"low":6095,"lowmod":8315,"LMMI":11645,"lowmoduniv":15195,"lowmod_pct":0.547219480092,"NAMELSAD10":"Alamosa County"}},
{"type":"Feature","geometry":{"type":"MultiPolygon","coordinates":[[[[-104.9294349999999,39.67238700000013],[-104.92733099999998,39.669484000000125],[-104.92954199999997,39.6693810000001],[-104.931622,39.66747299999997],[-104.93169499999982,39.67056400000013],[-104.93412899999998,39.67128100000008],[-104.93421799999987,39.67338799999999],[-104.9316409999999,39.67337400000008],[-104.9294349999999,39.67238700000013]]],[[[-104.92446499999988,39.67664800000006],[-104.92225000000002,39.674833000000035],[-104.919218,39.673880000000054],[-104.91755899999987,39.672983000000045],[-104.91754400000002,39.66757999999993],[-104.91637800000001,39.66758500000003],[-104.91594999999995,39.666152000000125],[-104.91593899999992,39.66347100000013],[-104.91296999999986,39.663133000000244],[-104.91298499999994,39.65620900000022],[-104.9165719999998,39.65611900000005],[-104.91589599999992,39.657149000000004],[-104.91675700000002,39.65900999999997],[-104.91824699999995,39.65934700000014],[-104.91887500000001,39.66033800000008],[-104.92101600000001,39.66033800000008],[-104.92223999999982,39.66376000000025],[-104.92222099999987,39.666947000000164],[-104.92291899999998,39.66788400000007],[-104.92642899999987,39.669394000000295],[-104.93024199999996,39.674832000000094],[-104.92757599999999,39.67482599999994],[-104.92572499999989,39.67528099999993],[-104.92446499999988,39.67664800000006]]],[[[-104.91103199999992,39.68207300000006],[-104.90820499999995,39.68207000000018],[-104.90820299999996,39.679883000000075],[-104.91103999999996,39.67988700000001],[-104.91103199999992,39.68207300000006]]],[[[-104.89878599999992,39.6857040000001],[-104.89878299999992,39.68340200000006],[-104.902671,39.68571200000008],[-104.89878599999992,39.6857040000001]]],[[[-104.92898299999996,39.70847500000019],[-104.92911799999996,39.70678700000013],[-104.926963,39.70669900000013],[-104.92692399999999,39.70360000000005],[-104.931423,39.703789000000256],[-104.93163400000003,39.70218700000021],[-104.93286799999998,39.70218199999999],[-104.93283899999989,39.7037940000003],[-104.93402699999996,39.70389800000021],[-104.93399599999998,39.70138000000003],[-104.93283099999996,39.70028600000012],[-104.92857799999985,39.700221000000056],[-104.92856399999994,39.69842900000015],[-104.92786399999994,39.696654000000024],[-104.93591700000002,39.69659900000005],[-104.9362129999999,39.69478200000003],[-104.93953899999997,39.69468800000004],[-104.94061199999999,39.69656999999995],[-104.93863499999992,39.696605000000204],[-104.93871000000001,39.698181999999974],[-104.94073799999984,39.69795600000026],[-104.94068699999997,39.711221000000194],[-104.93164799999994,39.71109799999999],[-104.93152999999995,39.70908800000029],[-104.92898299999996,39.70847500000019]],[[-104.93389699999994,39.698481000000015],[-104.93165099999993,39.698398000000054],[-104.93171399999989,39.70020800000009],[-104.93413499999991,39.70018100000016],[-104.93389699999994,39.698481000000015]]],[[[-103.70654699999994,39.739894000000106],[-103.70667399999996,39.7254410000001],[-103.70662299999992,39.697811],[-103.70661699999988,39.65838900000011],[-103.70661299999989,39.624985000000265],[-103.70696399999997,39.58757400000002],[-103.70707199999987,39.56629500000008],[-103.71435500000001,39.56633100000022],[-103.75192500000003,39.56656100000009],[-103.82649199999992,39.566597],[-103.84194599999995,39.56657100000018],[-103.87551199999979,39.56662300000022],[-103.90856100000002,39.56635800000015],[-103.95075600000001,39.56623000000002],[-103.97863899999993,39.566360000000145],[-103.98697599999997,39.56615100000005],[-104.02768600000002,39.56594999999999],[-104.05656999999997,39.56595399999992],[-104.06732099999999,39.56582000000003],[-104.10238099999998,39.565698999999995],[-104.11129799999998,39.56557700000019],[-104.15622799999994,39.56532800000019],[-104.18120899999991,39.565313],[-104.20723499999991,39.56517299999996],[-104.22040400000003,39.56514600000003],[-104.25420500000001,39.56494299999997],[-104.26482299999998,39.56471799999997],[-104.28760799999998,39.56471700000026],[-104.32344999999992,39.56438600000024],[-104.343301,39.564429000000246],[-104.36169799999999,39.564335000000085],[-104.37597999999997,39.56394899999998],[-104.38113099999993,39.56409700000012],[-104.4174349999999,39.563764000000276],[-104.42745199999996,39.56360100000029],[-104.44770399999999,39.563741000000164],[-104.45736699999998,39.56397600000008],[-104.4693769999999,39.563946000000044],[-104.47823800000003,39.56406500000031],[-104.49831399999988,39.564143],[-104.5161839999999,39.56438600000024],[-104.52376800000002,39.56457300000005],[-104.55505399999993,39.56500799999998],[-104.55995000000001,39.56514800000002],[-104.60444399999989,39.56590899999992],[-104.6456829999999,39.56588799999997],[-104.66062599999987,39.56590899999992],[-104.68143800000001,39.56598800000006],[-104.72092800000001,39.56598800000006],[-104.72542799999985,39.56588799999997],[-104.73912799999988,39.56588799999997],[-104.74922899999996,39.566088000000036],[-104.79354399999994,39.566091000000256],[-104.80137099999996,39.56588700000026],[-104.81023099999976,39.566088000000036],[-104.813131,39.56598800000006],[-104.85533299999986,39.56598800000006],[-104.86066499999987,39.566050000000246],[-104.88282600000002,39.56595100000004],[-104.890221,39.56600700000024],[-104.94170500000001,39.56602499999997],[-104.96097199999997,39.56615500000004],[-104.97178099999985,39.566064000000154],[-105.000268,39.566109000000154],[-105.0020679999999,39.566233000000125],[-105.01031699999993,39.56608600000004],[-105.02899400000001,39.56618700000001],[-105.04028499999998,39.56619000000012],[-105.04874000000001,39.566088000000036],[-105.05413999999996,39.566088000000036],[-105.05424,39.569788000000074],[-105.05394100000001,39.58059000000014],[-105.05383099999989,39.59187800000012],[-105.05367199999989,39.59780999999998],[-105.05383599999976,39.600885000000005],[-105.05362200000002,39.60972800000002],[-105.05364499999996,39.6170800000001],[-105.05343899999997,39.62142300000005],[-105.05343899999997,39.62438700000018],[-105.05333000000002,39.62798900000013],[-105.05345699999987,39.62952600000011],[-105.05341099999998,39.63158600000003],[-105.04433899999992,39.63158699999991],[-105.04404799999992,39.62801600000023],[-105.040618,39.628272000000095],[-105.038005,39.62683600000014],[-105.03680199999991,39.62812900000017],[-105.02912700000002,39.627838000000054],[-105.02536899999996,39.62859800000007],[-105.02535999999981,39.63144500000027],[-105.02956299999994,39.632355000000246],[-105.03005699999977,39.63349199999999],[-105.03475200000003,39.63359000000008],[-105.03470700000003,39.638593000000014],[-105.02999799999998,39.63857500000029],[-105.02997899999991,39.64224500000017],[-105.031338,39.64308700000021],[-105.03123799999997,39.643987000000095],[-105.03465899999998,39.64399100000003],[-105.03463099999988,39.65142600000024],[-105.03401699999995,39.65346500000021],[-105.03216199999997,39.653276000000005],[-105.032038,39.65708700000022],[-105.03083800000002,39.657187000000135],[-105.02300300000002,39.65683000000007],[-105.02298199999996,39.660526000000004],[-105.01580000000001,39.66050800000011],[-105.01569599999993,39.678684000000146],[-105.00866399999995,39.67862700000023],[-105.00863700000002,39.67678000000001],[-105.00657599999994,39.67677800000018],[-105.00642099999999,39.67318],[-105.00663700000001,39.667986000000155],[-105.00647100000003,39.66691300000002],[-105.00771700000001,39.664220000000114],[-105.01066499999996,39.66050700000005],[-105.00488200000001,39.660455000000184],[-105.00449299999997,39.6607860000002],[-105.00486199999995,39.664462999999955],[-105.00437399999998,39.66630600000008],[-104.9999059999999,39.66788400000007],[-104.992299,39.667679000000135],[-104.97348599999998,39.66768400000001],[-104.97348899999986,39.66586600000005],[-104.96880499999997,39.66583900000012],[-104.96878500000003,39.66041700000022],[-104.96322399999997,39.66037000000006],[-104.96314899999999,39.66156000000029],[-104.95944299999996,39.66152600000015],[-104.95948499999997,39.65669100000031],[-104.95378800000003,39.65668299999999],[-104.95495199999988,39.655797000000234],[-104.95590299999998,39.65299800000008],[-104.94540099999995,39.6529680000001],[-104.93170299999997,39.65304400000002],[-104.931873,39.64582600000017],[-104.92246,39.64584900000011],[-104.91325699999999,39.63869100000028],[-104.91340699999995,39.62408300000004],[-104.906969,39.62407200000007],[-104.90086999999994,39.62372299999993],[-104.89471900000001,39.62402800000012],[-104.885265,39.62408300000004],[-104.88511199999988,39.63926200000009],[-104.8803289999999,39.641491000000144],[-104.88013999999998,39.65305699999999],[-104.87749699999989,39.653052000000116],[-104.876306,39.65229500000021],[-104.87622399999998,39.6443010000001],[-104.85675300000003,39.653456000000006],[-104.85603800000001,39.65417800000023],[-104.85294399999998,39.65570600000001],[-104.84773299999995,39.656788000000006],[-104.84833200000003,39.65916100000004],[-104.85306699999984,39.659914000000185],[-104.85543699999994,39.66080400000027],[-104.85644099999996,39.66034000000025],[-104.86633399999977,39.660308999999984],[-104.8660509999998,39.670847000000094],[-104.87088199999994,39.67476399999998],[-104.87552700000003,39.67476399999998],[-104.87531000000001,39.67278399999992],[-104.87823300000002,39.6727870000002],[-104.87583199999978,39.66978699999993],[-104.87623300000001,39.66398700000002],[-104.879433,39.66388700000027],[-104.88413299999996,39.66748700000005],[-104.89013299999999,39.66758699999997],[-104.88963299999983,39.66658700000005],[-104.8880329999999,39.66578700000008],[-104.89018799999985,39.66439300000002],[-104.89394600000003,39.66746400000011],[-104.89614,39.66761700000018],[-104.90350899999999,39.66762400000022],[-104.90357899999992,39.67667499999999],[-104.90478199999995,39.67781800000006],[-104.90698399999991,39.67879599999998],[-104.90698199999991,39.679737000000046],[-104.903888,39.68033200000025],[-104.90343999999988,39.67859800000031],[-104.89405799999997,39.678448],[-104.89404599999995,39.682060999999976],[-104.88930800000003,39.6820560000001],[-104.88932799999992,39.68568400000004],[-104.89405799999997,39.68569400000001],[-104.894092,39.692874000000074],[-104.89876099999998,39.69299200000023],[-104.89881000000003,39.6893410000003],[-104.90351199999998,39.68935399999998],[-104.90351099999998,39.69297],[-104.90011499999991,39.69298500000002],[-104.90350699999999,39.69662199999999],[-104.90354399999995,39.69853500000005],[-104.8992849999998,39.69697600000018],[-104.900304,39.696254000000295],[-104.89880599999987,39.69509099999999],[-104.89678200000003,39.696587000000136],[-104.877611,39.696506],[-104.87769800000001,39.69798800000024],[-104.875316,39.69799599999999],[-104.87530899999985,39.696505000000286],[-104.86596199999991,39.69655899999998],[-104.8659639999999,39.699086000000136],[-104.8683009999998,39.69853100000023],[-104.87066199999975,39.69847500000003],[-104.87064800000002,39.700535000000116],[-104.869393,39.70108900000031],[-104.86893199999997,39.70298699999995],[-104.86596699999996,39.704317999999944],[-104.86597099999994,39.71105299999999],[-104.870653,39.71016900000001],[-104.8708309999999,39.718175999999914],[-104.87404100000003,39.71817799999991],[-104.87534199999993,39.71894400000025],[-104.8753299999999,39.725576000000274],[-104.87973499999998,39.72562300000004],[-104.881934,39.724960000000124],[-104.88502599999987,39.72813700000006],[-104.88661300000001,39.72921900000006],[-104.88473499999986,39.73083100000002],[-104.88464599999998,39.7401560000003],[-104.88420699999995,39.74021199999993],[-104.84709099999992,39.740207000000055],[-104.843254,39.74030699999997],[-104.82168999999993,39.74029200000018],[-104.80884500000002,39.74014200000005],[-104.80140399999993,39.740286000000026],[-104.75412699999987,39.74022800000006],[-104.73564499999992,39.74025400000005],[-104.73195499999997,39.74082200000015],[-104.725637,39.740267000000244],[-104.71066699999994,39.740254999999934],[-104.71063299999986,39.74012399999998],[-104.67830499999997,39.739995000000135],[-104.66026999999997,39.74048200000027],[-104.65280299999978,39.739778000000285],[-104.64415199999996,39.73972000000009],[-104.64072299999992,39.739885999999956],[-104.63042200000001,39.73950000000019],[-104.62464299999988,39.740077000000156],[-104.59880900000002,39.739844999999946],[-104.55888400000003,39.739330000000166],[-104.50737799999996,39.738249999999994],[-104.4591309999999,39.738178999999946],[-104.42669299999994,39.73809099999994],[-104.39186199999989,39.73804000000018],[-104.3587599999999,39.738197000000014],[-104.34008900000003,39.73824800000017],[-104.30356999999975,39.738515000000234],[-104.29796399999998,39.738680000000045],[-104.26498699999985,39.738877],[-104.20952499999987,39.7390180000001],[-104.15313400000002,39.73949700000014],[-104.13428199999993,39.73960400000027],[-104.11506899999989,39.739774000000125],[-104.05940299999997,39.739971999999966],[-104.0407249999999,39.739981],[-104.00348300000002,39.740170000000205],[-103.97763599999979,39.740267000000244],[-103.97434099999992,39.73998400000028],[-103.9723029999999,39.74040100000013],[-103.96551599999992,39.740518000000066],[-103.94049999999999,39.740521000000115],[-103.91928799999994,39.74067000000019],[-103.90751599999999,39.74065400000029],[-103.8755099999999,39.74029799999994],[-103.86606299999994,39.74023599999998],[-103.828213,39.74024600000013],[-103.80019900000002,39.7402010000003],[-103.79395699999992,39.74004500000001],[-103.72393599999998,39.739785000000154],[-103.70654699999994,39.739894000000106]]]]},"properties":{"OBJECTID":3,"COUNTYFP":"005","Shape_Leng":3.7139791237,"Shape_Area":0.218915819332,"low":145159,"lowmod":250415,"LMMI":371560,"lowmoduniv":603745,"lowmod_pct":0.414769480493,"NAMELSAD10":"Arapahoe County"}},
{"type":"Feature","geometry":{"type":"Polygon","coordinates":[[[-107.300318,37.42302000000001],[-107.28034699999989,37.42299799999995],[-107.25159100000002,37.42296799999997],[-107.242006,37.422970000000134],[-107.23717699999997,37.42297200000013],[-107.21776999999997,37.42298200000005],[-107.21748400000001,37.42298200000005],[-107.21622399999995,37.422979],[-107.20982400000003,37.422959000000105],[-107.19833800000004,37.42295300000001],[-107.19655999999998,37.42295300000001],[-107.19393699999995,37.42295300000001],[-107.193578,37.42295300000001],[-107.187003,37.422952000000066],[-107.17532799999998,37.42295000000007],[-107.16643999999991,37.422948000000076],[-107.1656079999999,37.422948000000076],[-107.13341699999995,37.42294300000003],[-107.12867999999997,37.42294200000015],[-107.12868199999997,37.421783000000005],[-107.12871399999995,37.40394900000001],[-107.128737,37.39240900000016],[-107.12559999999996,37.39248600000013],[-107.11358699999994,37.39249100000001],[-107.09869099999997,37.392434000000094],[-107.09117200000003,37.39242500000006],[-107.08919900000001,37.39245500000004],[-107.07752099999993,37.392403],[-107.07197899999994,37.392379000000005],[-107.06698399999993,37.39235700000012],[-107.05816099999993,37.39232900000013],[-107.04936799999996,37.392304000000024],[-107.04777899999993,37.39230500000008],[-107.04699299999993,37.39230599999996],[-107.04085600000002,37.392381],[-107.02717999999999,37.39262200000002],[-107.02242599999994,37.39270699999997],[-107.00059699999991,37.393136000000084],[-106.98503699999998,37.39324599999998],[-106.96389599999992,37.39341200000001],[-106.960961,37.39343800000006],[-106.94340399999987,37.3935570000001],[-106.89976399999995,37.393853999999976],[-106.89836700000001,37.39386300000001],[-106.87559499999986,37.39402300000012],[-106.87118299999992,37.39366500000017],[-106.85217899999998,37.394010000000094],[-106.801196,37.394975000000045],[-106.79042400000003,37.39520800000014],[-106.77658400000001,37.3954720000001],[-106.773125,37.395538000000045],[-106.75045099999994,37.395974999999964],[-106.72936900000002,37.39604600000018],[-106.72094500000003,37.39613300000008],[-106.71076699999998,37.396238000000096],[-106.71077099999997,37.399782000000016],[-106.71077299999996,37.40262900000022],[-106.71077499999984,37.40422799999999],[-106.70586800000001,37.404171000000076],[-106.67837299999997,37.40359600000011],[-106.678246,37.3965730000001],[-106.67830300000003,37.37500700000004],[-106.67810199999991,37.36346100000003],[-106.67842200000001,37.30310700000001],[-106.67870499999998,37.2500060000001],[-106.6783539999999,37.22856600000017],[-106.62558999999993,37.20669200000009],[-106.61440600000003,37.20214300000026],[-106.603406,37.19740100000013],[-106.59738400000003,37.19497400000023],[-106.58917799999989,37.14018799999997],[-106.57698699999992,37.12500799999998],[-106.5509019999999,37.09237400000012],[-106.52130899999997,37.05536100000006],[-106.50058899999988,37.02944500000018],[-106.48906199999993,37.01508000000018],[-106.479963,37.00001000000009],[-106.47974399999998,36.99936100000008],[-106.47622899999988,36.99375800000007],[-106.49859400000003,36.99370500000015],[-106.50038599999976,36.99375500000019],[-106.5479949999999,36.99336100000028],[-106.59117800000001,36.992923000000076],[-106.61628200000001,36.992907],[-106.6292069999999,36.993162000000154],[-106.66154699999998,36.993196000000125],[-106.675253,36.99294800000001],[-106.68912399999999,36.993015999999955],[-106.75058999999999,36.99246100000005],[-106.77488299999987,36.99252800000028],[-106.79791799999987,36.992485999999985],[-106.8503,36.99242800000002],[-106.869798,36.9924240000002],[-106.87729300000001,37.0001410000001],[-106.94068199999998,37.00010600000007],[-106.97094699999997,37.000112],[-107.00032699999997,37.00014599999997],[-107.00059199999993,37.000009000000205],[-107.05990199999991,37.000009000000205],[-107.10723999999988,37.00000799999992],[-107.18597199999994,37.000005000000044],[-107.25621899999999,37.00000300000005],[-107.310022,37.000005000000044],[-107.359871,37.000005000000044],[-107.39629300000001,37.000005000000044],[-107.44820699999997,37.000005000000044],[-107.48173699999995,37.000005000000044],[-107.48155099999991,37.055678],[-107.48142399999989,37.093861000000004],[-107.48210699999998,37.12349900000021],[-107.4820959999999,37.158187000000055],[-107.4820529999999,37.213230000000294],[-107.48225899999989,37.25000299999999],[-107.482211,37.270880000000034],[-107.482079,37.30602800000014],[-107.48213299999992,37.32175300000023],[-107.48240900000002,37.32443500000011],[-107.48208399999999,37.33716300000003],[-107.48202499999996,37.371271000000206],[-107.482238,37.37727200000023],[-107.48213099999987,37.42267300000026],[-107.46177299999994,37.42285500000003],[-107.409559,37.42331900000005],[-107.38116500000001,37.42357299999992],[-107.37560699999995,37.42362300000002],[-107.37307499999997,37.42355199999997],[-107.35602899999992,37.42308000000003],[-107.35321699999997,37.42307699999992],[-107.34469899999993,37.42306800000006],[-107.300318,37.42302000000001]]]},"properties":{"OBJECTID":4,"COUNTYFP":"007","Shape_Leng":2.77755869667,"Shape_Area":0.356288662668,"low":3105,"lowmod":5170,"LMMI":7250,"lowmoduniv":12005,"lowmod_pct":0.430653894211,"NAMELSAD10":"Archuleta County"}},
{"type":"Feature","geometry":{"type":"Polygon","coordinates":[[[-102.04158499999994,37.64428200000003],[-102.04161799999991,37.60786800000017],[-102.04179299999993,37.60123400000015],[-102.04193199999997,37.57904700000006],[-102.04168299999998,37.578555999999935],[-102.04189400000001,37.55797700000005],[-102.041899,37.54118600000004],[-102.04201599999999,37.535261000000105],[-102.04178799999988,37.505855999999994],[-102.04182999999989,37.485133000000076],[-102.04177600000003,37.4680010000003],[-102.04179899999997,37.45629800000029],[-102.04160999999999,37.447601000000134],[-102.04175499999997,37.43485499999997],[-102.04167099999995,37.42753399999992],[-102.04182000000003,37.41014899999999],[-102.04152199999999,37.37502099999995],[-102.04208899999992,37.35283000000027],[-102.04192299999994,37.35265899999996],[-102.04192399999994,37.337655999999924],[-102.04182500000002,37.29646999999994],[-102.04198600000001,37.2793310000003],[-102.04196299999995,37.25816400000008],[-102.04212399999994,37.25002100000006],[-102.04217999999997,37.225661],[-102.04196199999996,37.22190999999992],[-102.04197199999999,37.19958300000019],[-102.04202499999997,37.185429000000056],[-102.04199499999993,37.15594400000009],[-102.041812,37.13736800000021],[-102.04195799999991,37.132209000000216],[-102.04194699999988,37.10695399999997],[-102.04203199999989,37.10514000000012],[-102.04208599999993,37.04734400000001],[-102.04191999999995,37.03508300000021],[-102.04174899999992,37.034397000000126],[-102.04205399999995,37.028334000000086],[-102.04205499999995,37.010508999999956],[-102.04198999999988,36.99917900000008],[-102.04208899999992,36.993015999999955],[-102.05438699999996,36.99313000000018],[-102.07719299999991,36.993084000000124],[-102.08472299999988,36.99323400000026],[-102.125452,36.9933470000002],[-102.17511499999995,36.99356799999998],[-102.20831599999997,36.993730000000255],[-102.23513599999995,36.99401199999994],[-102.24847699999998,36.9941980000001],[-102.25219900000002,36.994170000000054],[-102.26493099999988,36.99438600000019],[-102.28500700000001,36.994151999999985],[-102.299327,36.994249000000025],[-102.34377799999987,36.9944090000003],[-102.35536699999994,36.994575],[-102.37546099999997,36.99458000000004],[-102.38857999999999,36.994776000000115],[-102.4342509999999,36.9948270000001],[-102.44628999999992,36.99474400000014],[-102.5004669999999,36.99496699999992],[-102.52468299999998,36.99487100000016],[-102.54181699999987,36.99490500000013],[-102.57090999999997,36.995134000000064],[-102.62547099999995,36.99503600000031],[-102.651431,36.995047],[-102.66212199999995,36.99527100000023],[-102.69814199999996,36.995149000000254],[-102.70590199999998,36.99557299999998],[-102.74205999999987,36.997689000000264],[-102.75214099999988,36.99834100000021],[-102.79362700000001,36.999374000000046],[-102.81507399999998,36.999774000000116],[-102.82946499999997,36.99973100000011],[-102.84192099999996,36.9996000000001],[-102.87548099999992,36.99962800000014],[-102.93627200000003,36.9989920000001],[-102.97555499999993,36.99861299999998],[-102.98409400000003,36.99864000000008],[-102.98697599999991,36.99852400000026],[-103.00048600000002,37.00001900000012],[-103.01355899999999,37.00021199999992],[-103.08610599999997,37.00026399999996],[-103.08653400000003,37.03905900000001],[-103.08727599999997,37.102678000000026],[-103.0869689999999,37.13508700000011],[-103.08632,37.14342599999998],[-103.08655799999991,37.176128000000006],[-103.08664299999992,37.180689000000086],[-103.08607599999999,37.200359000000276],[-103.08510699999994,37.236590999999976],[-103.083777,37.28439300000008],[-103.08352699999995,37.29429100000016],[-103.08367299999992,37.30806799999999],[-103.08323199999995,37.32270900000009],[-103.08335999999997,37.33719100000019],[-103.08302499999996,37.346745000000055],[-103.0830509999999,37.35173900000024],[-103.08276599999994,37.35839900000008],[-103.08278200000001,37.3663360000001],[-103.082537,37.38161300000007],[-103.07466299999999,37.38175400000017],[-103.07480499999991,37.398437000000115],[-103.075243,37.415738999999974],[-103.07502099999994,37.436776000000066],[-103.07514200000003,37.45094300000022],[-103.07500099999987,37.45574899999997],[-103.07515699999999,37.460738000000106],[-103.07514100000003,37.48448900000028],[-103.07527499999998,37.500037000000134],[-103.07522099999989,37.50158900000008],[-103.07538299999987,37.55663200000015],[-103.075581,37.624601000000155],[-103.07574199999999,37.62501099999997],[-103.07593799999995,37.643420000000276],[-103.05638599999997,37.64349200000004],[-103.04549599999996,37.643781000000104],[-103.02606800000001,37.64383299999997],[-103.02394800000002,37.64435800000007],[-103.02120300000001,37.64365400000008],[-103.003243,37.64366500000011],[-103.00048099999998,37.643754],[-102.94451900000001,37.643861000000015],[-102.93893100000003,37.64385000000016],[-102.90249599999987,37.64399300000025],[-102.87536899999998,37.643856000000085],[-102.85793100000001,37.644062000000076],[-102.85240599999992,37.64422400000018],[-102.83864900000003,37.644159000000116],[-102.82055699999995,37.643583999999976],[-102.77682099999998,37.643569000000014],[-102.76610700000003,37.643417],[-102.74761499999994,37.64364200000017],[-102.71266600000001,37.6435900000003],[-102.69447899999994,37.643595000000005],[-102.65845099999996,37.64368000000013],[-102.62547199999995,37.64362100000005],[-102.62051699999995,37.64348600000022],[-102.580667,37.64362299999999],[-102.56484799999998,37.64377899999994],[-102.55954700000001,37.64360900000008],[-102.54892699999999,37.64371299999999],[-102.5004689999999,37.6438280000001],[-102.49928699999998,37.643857000000196],[-102.46166499999998,37.643669999999986],[-102.41940199999999,37.643912000000114],[-102.39621899999992,37.64411700000005],[-102.374008,37.64413400000001],[-102.36317300000002,37.6443690000001],[-102.35503099999994,37.644165999999984],[-102.32867599999997,37.64421800000002],[-102.317656,37.64412800000008],[-102.300997,37.6441040000002],[-102.27395199999995,37.643961000000274],[-102.26151800000002,37.644178000000124],[-102.23794199999998,37.644113999999945],[-102.22271799999993,37.64424000000008],[-102.21955399999996,37.64416200000005],[-102.201281,37.644181000000174],[-102.18333100000001,37.644062000000076],[-102.16491299999996,37.64428700000025],[-102.15835900000002,37.644240999999965],[-102.15634499999999,37.644424000000015],[-102.148864,37.64420200000012],[-102.09089299999994,37.64424600000007],[-102.07368799999989,37.644204000000116],[-102.05576300000001,37.64408200000014],[-102.04158499999994,37.64428200000003]]]},"properties":{"OBJECTID":5,"COUNTYFP":"009","Shape_Leng":3.38193876785,"Shape_Area":0.673298118518,"low":1250,"lowmod":1920,"LMMI":2665,"lowmoduniv":3600,"lowmod_pct":0.533333333333,"NAMELSAD10":"Baca County"}},
{"type":"Feature","geometry":{"type":"Polygon","coordinates":[[[-102.74232999999992,38.26697100000007],[-102.74189799999999,38.250000000000114],[-102.74218899999988,38.23902500000008],[-102.74238500000001,38.235377000000085],[-102.7424059999999,38.215886000000296],[-102.74229699999995,38.20157400000005],[-102.74241499999994,38.18678000000011],[-102.742434,38.17214000000024],[-102.7423649999999,38.15746800000005],[-102.741847,38.12500799999998],[-102.74167599999998,38.09656000000001],[-102.74149999999997,38.08596800000009],[-102.74140999999992,38.04282800000027],[-102.74136199999992,38.012517],[-102.741308,38.00000700000004],[-102.74104799999998,37.99789900000002],[-102.74096099999997,37.98289100000011],[-102.74090399999989,37.954938000000084],[-102.7406509999999,37.883501000000194],[-102.74035700000002,37.863177000000064],[-102.74034599999999,37.851853000000006],[-102.74046399999997,37.82406400000002],[-102.74103000000002,37.823534999999936],[-102.74770799999993,37.82345100000009],[-102.7479879999999,37.823216],[-102.74783299999996,37.80387400000001],[-102.747839,37.76721299999997],[-102.74774399999995,37.73287900000025],[-102.74779999999998,37.72563000000014],[-102.74774499999995,37.70136400000001],[-102.74766399999993,37.687216000000205],[-102.747683,37.67481299999997],[-102.74761499999994,37.64364200000017],[-102.76610700000003,37.643417],[-102.77682099999998,37.643569000000014],[-102.82055699999995,37.643583999999976],[-102.83864900000003,37.644159000000116],[-102.85240599999992,37.64422400000018],[-102.85793100000001,37.644062000000076],[-102.87536899999998,37.643856000000085],[-102.90249599999987,37.64399300000025],[-102.93893100000003,37.64385000000016],[-102.94451900000001,37.643861000000015],[-103.00048099999998,37.643754],[-103.003243,37.64366500000011],[-103.02120300000001,37.64365400000008],[-103.02394800000002,37.64435800000007],[-103.02606800000001,37.64383299999997],[-103.04549599999996,37.643781000000104],[-103.05638599999997,37.64349200000004],[-103.07593799999995,37.643420000000276],[-103.114734,37.64341500000006],[-103.11764099999999,37.64329900000001],[-103.12548500000003,37.64358100000027],[-103.13148799999988,37.64366200000006],[-103.14667400000002,37.6435920000003],[-103.16323299999993,37.643599000000165],[-103.17517299999986,37.64351499999998],[-103.18749200000002,37.643669999999986],[-103.20549099999988,37.64408600000013],[-103.22204099999993,37.64401900000007],[-103.25882199999995,37.643980000000056],[-103.2675999999999,37.64414500000004],[-103.29057899999998,37.64385100000004],[-103.30813599999993,37.64393999999993],[-103.32690899999989,37.64376800000014],[-103.34180400000002,37.643513000000155],[-103.36126000000002,37.64333499999998],[-103.375493,37.64353500000004],[-103.38676999999996,37.64351499999998],[-103.40433899999994,37.64357600000005],[-103.40442599999994,37.65820300000007],[-103.40396299999992,37.66296300000005],[-103.40391699999992,37.673168000000146],[-103.40438499999993,37.68689000000023],[-103.40379599999994,37.70166500000005],[-103.40365199999997,37.73433799999998],[-103.40343399999995,37.75001100000003],[-103.40349600000002,37.75907400000011],[-103.40368199999995,37.822792000000106],[-103.399854,37.82281200000017],[-103.39983599999994,37.86652700000013],[-103.3998739999999,37.875063000000125],[-103.39978100000002,37.89961600000004],[-103.39911999999987,37.94014900000002],[-103.398236,37.959118000000274],[-103.39810799999998,37.960627000000045],[-103.39837299999988,37.979783],[-103.39833399999992,37.98175099999992],[-103.39894199999998,37.98952200000008],[-103.39884399999994,37.99430600000011],[-103.39882399999999,38.013904000000196],[-103.39867499999991,38.02239500000002],[-103.39866599999988,38.038392999999985],[-103.3985459999999,38.04626600000017],[-103.39865299999997,38.082745000000045],[-103.39885699999985,38.098945000000015],[-103.39874399999985,38.10146700000013],[-103.39857999999987,38.114491000000214],[-103.39897199999996,38.121727000000305],[-103.39877899999999,38.125010000000145],[-103.39923499999998,38.18820200000016],[-103.39959999999996,38.24114899999995],[-103.39987699999989,38.25000400000005],[-103.39993799999996,38.26543499999997],[-103.38504399999988,38.265293000000156],[-103.37548899999996,38.26550000000003],[-103.31794500000001,38.265596000000016],[-103.26694299999997,38.2656800000002],[-103.19971499999991,38.26580100000007],[-103.16357999999997,38.26602600000007],[-103.16310899999996,38.26597000000021],[-103.14004799999998,38.26597700000025],[-103.13109600000001,38.26639200000011],[-103.12547799999999,38.266464999999926],[-103.09691299999986,38.26632500000028],[-103.07586900000001,38.266364000000294],[-103.05772799999988,38.26659300000006],[-103.04906099999988,38.26649400000002],[-103.02298599999989,38.26640299999997],[-103.01146199999994,38.26646699999992],[-103.00047899999998,38.26637300000016],[-103.00047899999998,38.2667170000002],[-102.95866099999995,38.26669000000027],[-102.95007299999992,38.26692600000007],[-102.94106499999987,38.26675800000021],[-102.92304499999995,38.26668600000011],[-102.904155,38.266722999999956],[-102.84910400000001,38.26705400000003],[-102.84232099999991,38.26706600000011],[-102.83908499999995,38.26664100000011],[-102.83448099999998,38.26646100000016],[-102.82204399999995,38.266529000000276],[-102.82167499999997,38.26695000000029],[-102.798991,38.26779599999992],[-102.79301499999991,38.26766200000003],[-102.76133699999997,38.26667900000007],[-102.74232999999992,38.26697100000007]]]},"properties":{"OBJECTID":6,"COUNTYFP":"011","Shape_Leng":2.57239091196,"Shape_Area":0.409167757323,"low":1280,"lowmod":1990,"LMMI":2690,"lowmoduniv":3580,"lowmod_pct":0.555865921788,"NAMELSAD10":"Bent County"}},
{"type":"Feature","geometry":{"type":"MultiPolygon","coordinates":[[[[-105.05771300000004,39.97412100000008],[-105.05774699999989,39.96863600000029],[-105.05420200000003,39.96587000000022],[-105.05282799999986,39.96589200000028],[-105.05288899999994,39.95782600000007],[-105.07166299999989,39.957786],[-105.08004799999998,39.95792799999998],[-105.08256599999987,39.95927700000004],[-105.07679599999994,39.96230600000001],[-105.07473500000003,39.96393499999999],[-105.06980099999981,39.968482000000165],[-105.06726900000001,39.97036899999995],[-105.06507499999987,39.97155400000014],[-105.05770899999976,39.97466900000012],[-105.05771300000004,39.97412100000008]]],[[[-105.0550889999999,40.26179300000024],[-105.05510499999997,40.24901100000011],[-105.05484999999982,40.23971299999994],[-105.05507599999999,40.232778999999994],[-105.05519399999986,40.222852000000046],[-105.05519299999997,40.18106300000005],[-105.05534199999994,40.16036200000019],[-105.05540200000002,40.14087799999993],[-105.05535699999996,40.12751700000024],[-105.05543899999998,40.1118690000003],[-105.05541900000003,40.09018400000019],[-105.05550499999993,40.04399600000005],[-105.05536899999987,40.0291850000001],[-105.05540999999988,40.01771000000031],[-105.05564199999998,40.000249999999994],[-105.05282299999999,40.00026000000014],[-105.05279699999977,39.97767400000009],[-105.06492599999996,39.97257700000023],[-105.06961100000001,39.96985700000005],[-105.07570399999992,39.964331999999956],[-105.07863199999997,39.9621820000001],[-105.08203000000003,39.96084999999999],[-105.088099,39.959753000000035],[-105.0882309999999,39.95792799999998],[-105.08412599999997,39.95848700000005],[-105.0831169999999,39.95779600000009],[-105.07200199999994,39.95766999999995],[-105.07189900000003,39.95019800000006],[-105.07656800000001,39.950537999999995],[-105.07663699999983,39.943329000000176],[-105.09107199999977,39.9432810000003],[-105.09085299999987,39.93605100000002],[-105.10923599999995,39.936135000000036],[-105.1092349999999,39.93540800000011],[-105.11229600000001,39.933447],[-105.11344799999995,39.931432000000086],[-105.111873,39.92805000000021],[-105.1137619999999,39.92861500000009],[-105.11448399999995,39.93012600000003],[-105.11865699999998,39.93296500000008],[-105.11856199999994,39.94332300000002],[-105.10935299999989,39.943336000000215],[-105.10933299999999,39.95064000000025],[-105.1205139999999,39.95052700000031],[-105.120837,39.95129300000008],[-105.11597199999989,39.953907000000186],[-105.10953699999999,39.95610500000015],[-105.10951699999987,39.956996000000004],[-105.1143889999999,39.95545800000008],[-105.11794999999995,39.953944000000035],[-105.12115999999997,39.95213799999999],[-105.12194299999993,39.95416],[-105.12800199999987,39.95417299999997],[-105.12816499999997,39.93961800000005],[-105.13757800000002,39.93971200000021],[-105.13760399999995,39.92893100000009],[-105.14704099999989,39.92893800000013],[-105.14714299999997,39.91388900000021],[-105.16656499999999,39.91389300000003],[-105.19092899999998,39.91408200000001],[-105.19664699999987,39.914186000000086],[-105.20024099999995,39.91398600000025],[-105.21247499999976,39.91423600000013],[-105.23134299999992,39.914085000000114],[-105.23854399999988,39.91438599999992],[-105.24526999999995,39.914084],[-105.26394500000004,39.91398600000025],[-105.26494500000001,39.914186000000086],[-105.26944500000002,39.91398600000025],[-105.28265599999992,39.91368700000021],[-105.30684500000001,39.91398600000025],[-105.31414699999993,39.91388599999999],[-105.33954699999992,39.913786000000016],[-105.37394299999988,39.913181000000066],[-105.3978489999999,39.912886000000185],[-105.398167,39.92954299999991],[-105.39797099999998,39.93486700000011],[-105.40084300000001,39.935032000000035],[-105.40603899999996,39.93174300000004],[-105.40798599999994,39.93185399999999],[-105.41152499999993,39.932690000000264],[-105.41397099999989,39.93440900000002],[-105.41502699999995,39.93324500000017],[-105.41550599999988,39.931570000000136],[-105.417326,39.930669000000194],[-105.41902399999992,39.92873600000013],[-105.42081399999995,39.930901000000006],[-105.42206299999992,39.931626000000165],[-105.42588599999988,39.9310780000003],[-105.429325,39.93120899999991],[-105.43083100000001,39.931764000000044],[-105.43254999999988,39.93340599999999],[-105.435989,39.93565800000016],[-105.4372939999999,39.93609200000003],[-105.43847199999993,39.937531000000035],[-105.44003399999997,39.93838800000009],[-105.4434159999999,39.9379600000002],[-105.44459299999994,39.93729000000019],[-105.447069,39.93691000000001],[-105.44948299999999,39.93520600000005],[-105.45344699999998,39.93419100000028],[-105.45565099999999,39.93228600000003],[-105.45735099999996,39.932186000000115],[-105.46747099999999,39.93228400000004],[-105.50086599999997,39.932387999999946],[-105.54175399999991,39.93238700000023],[-105.55905499999994,39.932286999999974],[-105.57655499999998,39.932286999999974],[-105.5938559999999,39.93238700000023],[-105.62566800000002,39.93238599999995],[-105.62876999999997,39.9320860000002],[-105.64618299999995,39.93228500000015],[-105.65308699999991,39.93228500000015],[-105.67579799999999,39.93244500000009],[-105.67526499999997,39.93332300000003],[-105.67894100000001,39.93286000000029],[-105.68294300000002,39.93378900000005],[-105.68661599999996,39.93746900000008],[-105.68962699999997,39.93863400000021],[-105.69029999999998,39.939525],[-105.69319999999993,39.94123900000011],[-105.69345299999998,39.94401700000026],[-105.69179600000001,39.94624100000004],[-105.69281499999988,39.94816300000008],[-105.69436199999996,39.94968399999999],[-105.69366199999996,39.951153999999974],[-105.69119899999993,39.952991000000054],[-105.68939999999992,39.95350600000023],[-105.68850800000001,39.95483100000007],[-105.68838299999999,39.95653000000016],[-105.68510800000001,39.95874099999992],[-105.68818499999986,39.959663000000205],[-105.68833099999995,39.9628800000001],[-105.689255,39.96473000000003],[-105.68865399999993,39.96643300000028],[-105.68882699999995,39.96776200000028],[-105.69034499999998,39.96985500000005],[-105.68967800000001,39.97100100000023],[-105.68841399999997,39.975426000000084],[-105.68493799999987,39.97933000000012],[-105.68546199999997,39.981387000000154],[-105.68512699999997,39.98310000000015],[-105.68747999999988,39.985418000000266],[-105.68604799999997,39.98747700000007],[-105.685968,39.989742999999976],[-105.68390599999992,39.99163200000015],[-105.68316399999998,39.992881000000295],[-105.68320999999997,39.99465000000009],[-105.68501300000003,39.99451699999992],[-105.687929,39.995062000000075],[-105.69018499999987,39.99593399999992],[-105.69119399999994,39.99731300000019],[-105.69117,39.99952100000007],[-105.69049699999994,40.00086900000008],[-105.69105200000001,40.00376399999999],[-105.68976700000002,40.005955000000085],[-105.688469,40.00694599999997],[-105.69001500000002,40.00931700000024],[-105.68966599999999,40.01554100000004],[-105.6878259999998,40.01597900000024],[-105.68634700000001,40.01509499999992],[-105.68305199999998,40.014141999999936],[-105.680431,40.01401999999996],[-105.67797699999988,40.01454100000012],[-105.67451,40.01618599999995],[-105.67013599999996,40.01687900000013],[-105.66709800000001,40.01631700000013],[-105.661606,40.01646600000004],[-105.66019,40.01834900000023],[-105.65740800000003,40.019528000000264],[-105.65307100000001,40.020425000000046],[-105.65279299999992,40.023772000000235],[-105.64957400000003,40.0277420000001],[-105.64393799999999,40.03330099999994],[-105.64119700000003,40.03417200000007],[-105.64019300000001,40.03725500000007],[-105.6387969999999,40.038787000000184],[-105.63914899999997,40.04046900000009],[-105.63829199999986,40.04432600000018],[-105.63748599999997,40.0458450000001],[-105.63850699999989,40.04673900000017],[-105.64233799999994,40.048120999999924],[-105.64509899999996,40.050670000000025],[-105.64558199999988,40.052660000000174],[-105.6464519999999,40.05346499999996],[-105.64842499999992,40.0539370000003],[-105.65106099999997,40.05844500000006],[-105.64941899999991,40.05999200000019],[-105.64753300000001,40.06428600000021],[-105.64669199999986,40.06522000000001],[-105.64143299999995,40.06788399999999],[-105.63887299999988,40.06703099999993],[-105.640739,40.06987100000009],[-105.641121,40.07299999999998],[-105.63883799999991,40.07340499999992],[-105.63731299999995,40.07445400000023],[-105.63542899999993,40.07494700000001],[-105.6346769999999,40.077035999999964],[-105.6348359999999,40.07921499999998],[-105.63322899999997,40.08118500000006],[-105.63563299999993,40.082871000000125],[-105.6360039999999,40.08455300000003],[-105.63514699999996,40.08497600000004],[-105.63521100000003,40.08718300000004],[-105.63375999999994,40.08837900000009],[-105.63430199999993,40.09016400000013],[-105.6347189999999,40.09384799999998],[-105.63453900000002,40.095485000000224],[-105.63326599999994,40.09545400000013],[-105.63208099999986,40.09673799999996],[-105.63538599999993,40.1004430000001],[-105.63660999999985,40.10253600000027],[-105.63610399999999,40.104741000000274],[-105.63668099999995,40.106993999999986],[-105.63570500000003,40.10830800000019],[-105.63542299999989,40.11117200000018],[-105.63465199999996,40.113232000000096],[-105.63162999999992,40.11348900000007],[-105.62997299999995,40.114157999999975],[-105.629121,40.1170360000001],[-105.62768899999998,40.11843699999997],[-105.62820999999985,40.12035300000025],[-105.62757399999998,40.122062000000255],[-105.628782,40.12351199999995],[-105.62818499999992,40.124986999999976],[-105.62608999999992,40.1273460000001],[-105.62913300000002,40.1291580000003],[-105.62998299999998,40.13043399999998],[-105.63004799999987,40.13458800000001],[-105.62938799999995,40.136150999999984],[-105.630156,40.13809900000024],[-105.63254099999989,40.13843099999997],[-105.6341779999999,40.139125000000206],[-105.63546799999995,40.140417000000184],[-105.63509699999992,40.144190000000094],[-105.63535399999995,40.14556600000003],[-105.63655099999994,40.146375000000205],[-105.64108899999991,40.14754899999997],[-105.64480899999995,40.14807600000006],[-105.64734099999993,40.14921300000003],[-105.64730999999995,40.15080700000027],[-105.65053899999992,40.15057100000013],[-105.6559959999999,40.15128100000004],[-105.65686199999988,40.15293700000029],[-105.65819099999999,40.15399600000001],[-105.66024799999991,40.15473500000002],[-105.66134299999999,40.15652400000005],[-105.66468399999997,40.15674300000006],[-105.66772900000001,40.15827600000006],[-105.66800399999994,40.159886000000256],[-105.66753599999993,40.16118600000016],[-105.67057999999992,40.162691000000166],[-105.67147299999999,40.16542199999998],[-105.67133599999994,40.166497000000106],[-105.66959099999997,40.16748500000017],[-105.66950599999996,40.16862499999996],[-105.66833399999996,40.16961500000002],[-105.66811499999989,40.17085800000018],[-105.66857299999992,40.17357999999996],[-105.6697329999999,40.17425800000029],[-105.66908799999982,40.1762590000003],[-105.66828099999992,40.176914000000124],[-105.670771,40.17848900000007],[-105.67179099999993,40.180028999999934],[-105.67168199999986,40.18200400000029],[-105.67031500000002,40.183417000000134],[-105.67062599999997,40.18539300000003],[-105.67379299999993,40.18574200000012],[-105.67546899999985,40.18639200000007],[-105.67755399999999,40.1885070000003],[-105.67775099999994,40.192984000000024],[-105.67764399999999,40.19721100000015],[-105.67640499999993,40.19950300000011],[-105.67764499999998,40.2023310000003],[-105.67525199999994,40.20321300000012],[-105.6736959999999,40.204421000000025],[-105.67055900000003,40.207802000000186],[-105.67119400000001,40.209692000000246],[-105.67111899999992,40.21216300000003],[-105.67019199999987,40.21360800000025],[-105.6709459999999,40.21460200000024],[-105.67125499999997,40.21707900000001],[-105.67030299999999,40.21935700000006],[-105.6676599999999,40.22037],[-105.66453200000001,40.21981599999998],[-105.66368799999998,40.2203540000001],[-105.66859399999998,40.223285000000146],[-105.66904599999998,40.22417900000022],[-105.66776700000003,40.22671900000023],[-105.66844400000002,40.22814000000005],[-105.66726699999998,40.22981800000014],[-105.66735399999988,40.23095900000004],[-105.66608999999994,40.23146600000007],[-105.66437799999994,40.233213000000205],[-105.66447900000003,40.23510100000027],[-105.665437,40.23726999999997],[-105.66494999999992,40.23855500000025],[-105.66209499999997,40.24252200000001],[-105.66212799999994,40.243224],[-105.66020400000002,40.245177000000126],[-105.6601619999999,40.248351000000014],[-105.658932,40.249413000000004],[-105.650396,40.24937900000003],[-105.64806799999997,40.24907700000006],[-105.64401999999995,40.249939000000154],[-105.64742899999993,40.25136900000018],[-105.64978899999988,40.25287700000024],[-105.65119900000002,40.25424700000008],[-105.65119700000002,40.257257000000266],[-105.651656,40.2585170000001],[-105.653321,40.26045700000003],[-105.635693,40.26048200000014],[-105.61776099999992,40.26067300000011],[-105.60939499999978,40.26068300000003],[-105.58105499999999,40.261299000000065],[-105.55958499999991,40.26153100000005],[-105.53535399999998,40.26128400000022],[-105.52724999999998,40.26099299999993],[-105.50048099999992,40.26027700000003],[-105.49532099999993,40.259726000000114],[-105.48617899999999,40.259708000000046],[-105.46275199999991,40.25938400000007],[-105.44389999999999,40.25872300000009],[-105.43255199999993,40.25838400000026],[-105.41666599999996,40.25801200000001],[-105.4023489999999,40.257784000000015],[-105.39099599999997,40.257884000000274],[-105.375226,40.261291000000085],[-105.35474699999992,40.261784000000205],[-105.34700299999992,40.261685],[-105.33624800000001,40.261685],[-105.32674500000002,40.26198499999998],[-105.31331099999989,40.26218600000004],[-105.28944499999989,40.262785000000065],[-105.27344399999987,40.26248499999991],[-105.2600509999998,40.262384999999995],[-105.23731899999996,40.26189500000015],[-105.22644199999996,40.26148499999999],[-105.21394199999992,40.26128499999993],[-105.21311899999995,40.26118500000001],[-105.19647600000002,40.261085000000094],[-105.17964599999988,40.2610830000001],[-105.15903500000002,40.26121899999998],[-105.13105899999994,40.26107899999994],[-105.13017899999988,40.26158399999997],[-105.110138,40.26128400000022],[-105.10753799999998,40.261384000000135],[-105.08734800000002,40.261585999999966],[-105.08303699999993,40.26153500000021],[-105.0550889999999,40.26179300000024]],[[-105.107374,39.957915000000014],[-105.10917899999987,39.9571030000003],[-105.10917899999987,39.9562140000001],[-105.10580400000003,39.95717400000012],[-105.099965,39.957854999999995],[-105.09995999999995,39.95897900000011],[-105.1038519999999,39.958651000000145],[-105.107374,39.957915000000014]]]]},"properties":{"OBJECTID":7,"COUNTYFP":"013","Shape_Leng":2.38337339873,"Shape_Area":0.202534657243,"low":94520,"lowmod":121700,"LMMI":167445,"lowmoduniv":299850,"lowmod_pct":0.405869601467,"NAMELSAD10":"Boulder County"}},
{"type":"Feature","geometry":{"type":"MultiPolygon","coordinates":[[[[-105.107374,39.957915000000014],[-105.1038519999999,39.958651000000145],[-105.09995999999995,39.95897900000011],[-105.099965,39.957854999999995],[-105.10580400000003,39.95717400000012],[-105.10917899999987,39.9562140000001],[-105.10917899999987,39.9571030000003],[-105.107374,39.957915000000014]]],[[[-104.961408,40.000337000000286],[-104.98094400000002,40.000324999999975],[-104.98244299999999,39.997090000000014],[-104.987052,39.988529000000085],[-104.9877459999999,39.98648300000008],[-104.98778400000003,39.97574600000024],[-104.99441099999996,39.975802999999985],[-104.99437999999998,39.97395300000022],[-104.99707999999993,39.973954000000106],[-104.99706500000002,39.97214900000017],[-104.98810800000001,39.97217499999999],[-104.98810800000001,39.96846800000026],[-104.99724999999995,39.96852899999999],[-104.99737899999997,39.97578500000009],[-104.99838899999997,39.97580500000015],[-104.99840399999994,39.97758600000026],[-104.99739799999986,39.97842000000014],[-104.99742299999997,39.98121300000014],[-105.00696299999998,39.98126700000017],[-105.01584500000001,39.981193000000076],[-105.01572199999987,39.97227600000019],[-105.01340899999997,39.97226499999999],[-105.01342299999999,39.97044799999992],[-105.01569099999995,39.97043700000012],[-105.01556499999992,39.955188000000305],[-105.01711699999998,39.95281399999999],[-105.015039,39.951457000000005],[-105.01222499999989,39.95044600000017],[-105.00629800000002,39.9504390000003],[-105.00626499999998,39.94680499999998],[-104.99715200000003,39.94677100000001],[-104.99709999999988,39.943239000000005],[-105.01565699999998,39.94313099999994],[-105.01546899999988,39.921402000000114],[-105.02159099999989,39.92136700000009],[-105.02155199999987,39.91920700000003],[-105.02354500000001,39.9192250000001],[-105.023551,39.92124800000016],[-105.03439499999996,39.921305000000075],[-105.03434700000003,39.91418100000021],[-105.05289199999993,39.91422400000022],[-105.055319,39.91372499999994],[-105.05529399999989,39.91242599999998],[-105.05767599999996,39.9124290000002],[-105.05767599999996,39.913702],[-105.06238399999995,39.913987000000134],[-105.06239199999999,39.90702500000003],[-105.071775,39.90701799999994],[-105.071688,39.899833],[-105.08422999999988,39.89983699999999],[-105.08425099999994,39.898956999999996],[-105.09062399999993,39.89890299999996],[-105.09062599999993,39.899677000000054],[-105.0966919999999,39.899788],[-105.09668099999999,39.900787000000264],[-105.10010199999999,39.90220600000009],[-105.100641,39.90863800000005],[-105.10695099999981,39.91172900000004],[-105.10817800000001,39.91281000000009],[-105.10860899999994,39.914136999999926],[-105.12791099999998,39.914015000000006],[-105.14711699999998,39.91400200000015],[-105.14739599999996,39.90009399999997],[-105.13144999999997,39.90009900000018],[-105.12867799999992,39.89998499999996],[-105.12889499999994,39.89035300000006],[-105.13031899999999,39.89070600000014],[-105.13088500000003,39.892697000000226],[-105.13310499999994,39.89201200000002],[-105.13630799999993,39.89202],[-105.13645700000001,39.8909460000001],[-105.13485199999991,39.88929200000007],[-105.14698900000002,39.889211999999986],[-105.14743999999996,39.889449999999954],[-105.16516699999994,39.891486000000214],[-105.165571,39.89249500000005],[-105.16568999999998,39.91051499999992],[-105.16205699999995,39.91143399999993],[-105.15845799999988,39.9115960000002],[-105.15331299999997,39.913039000000026],[-105.15044599999999,39.91323999999997],[-105.14714299999997,39.91388900000021],[-105.14704099999989,39.92893800000013],[-105.13760399999995,39.92893100000009],[-105.13757800000002,39.93971200000021],[-105.12816499999997,39.93961800000005],[-105.12800199999987,39.95417299999997],[-105.12194299999993,39.95416],[-105.12115999999997,39.95213799999999],[-105.11794999999995,39.953944000000035],[-105.1143889999999,39.95545800000008],[-105.10951699999987,39.956996000000004],[-105.10953699999999,39.95610500000015],[-105.11597199999989,39.953907000000186],[-105.120837,39.95129300000008],[-105.1205139999999,39.95052700000031],[-105.10933299999999,39.95064000000025],[-105.10935299999989,39.943336000000215],[-105.11856199999994,39.94332300000002],[-105.11865699999998,39.93296500000008],[-105.11448399999995,39.93012600000003],[-105.1137619999999,39.92861500000009],[-105.111873,39.92805000000021],[-105.11344799999995,39.931432000000086],[-105.11229600000001,39.933447],[-105.1092349999999,39.93540800000011],[-105.10923599999995,39.936135000000036],[-105.09085299999987,39.93605100000002],[-105.09107199999977,39.9432810000003],[-105.07663699999983,39.943329000000176],[-105.07656800000001,39.950537999999995],[-105.07189900000003,39.95019800000006],[-105.07200199999994,39.95766999999995],[-105.0831169999999,39.95779600000009],[-105.08412599999997,39.95848700000005],[-105.0882309999999,39.95792799999998],[-105.088099,39.959753000000035],[-105.08203000000003,39.96084999999999],[-105.07863199999997,39.9621820000001],[-105.07570399999992,39.964331999999956],[-105.06961100000001,39.96985700000005],[-105.06492599999996,39.97257700000023],[-105.05279699999977,39.97767400000009],[-105.05282299999999,40.00026000000014],[-105.04623199999997,40.0004660000003],[-105.00790599999993,40.00048300000009],[-105.00799499999994,40.0075810000003],[-104.98964699999999,40.00767900000005],[-104.9896139999999,40.01489000000021],[-104.99906499999992,40.01488699999993],[-104.9990699999999,40.01613100000003],[-104.99617599999999,40.01943199999994],[-104.99678399999993,40.02082000000007],[-104.99674199999993,40.02350899999993],[-104.99900499999984,40.02350899999993],[-104.999007,40.02577600000012],[-104.98654299999993,40.02579900000006],[-104.98649999999992,40.02930900000024],[-104.98006299999997,40.02932500000014],[-104.98004099999991,40.04031600000002],[-104.97990500000003,40.042111000000034],[-104.97852599999999,40.04388799999998],[-104.961071,40.044144000000074],[-104.96124199999986,40.032687000000124],[-104.96132499999999,40.01515100000012],[-104.96148299999999,40.014979000000096],[-104.96153800000002,40.00040400000012],[-104.961408,40.000337000000286]],[[-104.97944499999988,40.033058000000096],[-104.97991699999994,40.029486000000134],[-104.973074,40.029511999999954],[-104.97304999999994,40.03303700000015],[-104.97944499999988,40.033058000000096]],[[-105.05771300000004,39.97412100000008],[-105.05770899999976,39.97466900000012],[-105.06507499999987,39.97155400000014],[-105.06726900000001,39.97036899999995],[-105.06980099999981,39.968482000000165],[-105.07473500000003,39.96393499999999],[-105.07679599999994,39.96230600000001],[-105.08256599999987,39.95927700000004],[-105.08004799999998,39.95792799999998],[-105.07166299999989,39.957786],[-105.05288899999994,39.95782600000007],[-105.05282799999986,39.96589200000028],[-105.05420200000003,39.96587000000022],[-105.05774699999989,39.96863600000029],[-105.05771300000004,39.97412100000008]]]]},"properties":{"OBJECTID":8,"COUNTYFP":"014","Shape_Leng":1.17463452414,"Shape_Area":0.009157969946,"low":9045,"lowmod":17315,"LMMI":29050,"lowmoduniv":60355,"lowmod_pct":0.286885924944,"NAMELSAD10":"Broomfield County"}},
{"type":"Feature","geometry":{"type":"Polygon","coordinates":[[[-106.19100600000002,39.05607300000014],[-106.19009299999988,39.05452400000007],[-106.18636199999992,39.052201000000025],[-106.1848149999999,39.050837],[-106.18492800000001,39.04911000000027],[-106.18570199999982,39.04676100000006],[-106.1853529999999,39.0440200000001],[-106.1827819999998,39.04011600000001],[-106.182501,39.03855100000004],[-106.18356199999988,39.03703100000007],[-106.18497000000002,39.036235999999974],[-106.185879,39.03403800000001],[-106.18690699999996,39.033195999999975],[-106.18872499999992,39.03323500000005],[-106.19032699999997,39.030570000000125],[-106.19207699999998,39.02928700000001],[-106.19127699999996,39.02583300000009],[-106.18980899999991,39.02311300000025],[-106.18947300000002,39.02127900000005],[-106.19310499999995,39.01968099999999],[-106.19295399999999,39.01770900000008],[-106.191416,39.016549],[-106.18825299999986,39.01538099999999],[-106.18607699999995,39.0155950000003],[-106.18494899999996,39.012581000000125],[-106.18305199999998,39.011835000000076],[-106.17946999999987,39.01124900000025],[-106.17792299999996,39.00910099999999],[-106.17802499999993,39.0076890000002],[-106.177009,39.005341999999985],[-106.176962,39.00194900000008],[-106.17591799999997,38.99964700000004],[-106.17491599999994,38.99959600000011],[-106.17316499999998,38.99740300000019],[-106.17006700000002,38.994644000000164],[-106.169804,38.99253299999992],[-106.16853399999991,38.991249000000096],[-106.16566099999994,38.99243900000016],[-106.16341999999997,38.993908000000204],[-106.16051399999998,38.994808000000035],[-106.157804,38.99426399999999],[-106.15407800000003,38.99616700000024],[-106.15066599999994,38.99709300000029],[-106.14956000000001,38.99702600000006],[-106.1454139999999,38.99789900000019],[-106.14265899999987,38.995428000000004],[-106.1412509999999,38.99496599999998],[-106.13777800000003,38.994642],[-106.13256799999999,38.995519000000115],[-106.13171699999992,38.9949180000001],[-106.12751799999995,38.994017000000156],[-106.12246799999991,38.990816000000166],[-106.11677099999997,38.98808400000024],[-106.11323899999996,38.98781000000014],[-106.11187100000001,38.986953000000085],[-106.10895299999999,38.98737700000004],[-106.10950999999989,38.98593900000003],[-106.10947899999996,38.9816330000001],[-106.10760699999997,38.9797470000002],[-106.10648999999984,38.976494],[-106.10558900000001,38.976046000000224],[-106.104379,38.974140999999975],[-106.10496799999987,38.97002600000002],[-106.10558300000002,38.96941800000002],[-106.10828499999985,38.96346300000022],[-106.11061699999999,38.96203400000013],[-106.11219299999999,38.96018800000013],[-106.11105999999995,38.95766900000007],[-106.11375699999991,38.955327000000125],[-106.11343999999997,38.95278800000017],[-106.11204299999986,38.95090800000003],[-106.113114,38.9450240000001],[-106.11266499999999,38.94136600000013],[-106.110995,38.94009900000009],[-106.10575799999987,38.94036399999999],[-106.1009709999999,38.93869999999998],[-106.09823399999993,38.938171000000125],[-106.09646099999998,38.938243],[-106.09486199999992,38.93677500000024],[-106.09354999999988,38.93795800000004],[-106.09065299999992,38.93654300000003],[-106.08531999999991,38.93946000000017],[-106.08358699999997,38.937935000000095],[-106.07862799999998,38.93525500000004],[-106.07618899999989,38.934555000000046],[-106.07396299999999,38.93475799999999],[-106.07148099999989,38.93377600000025],[-106.07047999999986,38.93211099999996],[-106.06782499999991,38.93287200000009],[-106.06590999999992,38.93189599999994],[-106.06496699999997,38.93269000000009],[-106.06475099999994,38.93443300000007],[-106.0635319999999,38.93568000000005],[-106.06014900000002,38.93715300000025],[-106.05844899999988,38.93915099999998],[-106.05824999999993,38.941805000000215],[-106.05170099999992,38.94345200000009],[-106.04861299999988,38.94282700000002],[-106.04577899999987,38.94154800000007],[-106.04378999999989,38.94192599999991],[-106.03887299999991,38.940792000000044],[-106.03683000000001,38.940646999999956],[-106.032287,38.941599000000224],[-106.0307969999999,38.94250799999992],[-106.02813799999996,38.94284900000008],[-106.02627099999995,38.94109000000003],[-106.02461700000003,38.93883000000028],[-106.02052900000001,38.93462800000003],[-106.01806299999981,38.93445099999997],[-106.0150099999999,38.93271200000015],[-106.0125999999999,38.929665000000284],[-106.01047399999993,38.92781900000011],[-106.00821899999988,38.9264800000002],[-106.00639999999993,38.92408399999999],[-105.99918300000002,38.91840900000011],[-105.9993429999999,38.91633700000028],[-105.99817000000002,38.91515899999996],[-105.99444899999997,38.914857999999924],[-105.98610000000002,38.91810600000008],[-105.98381599999993,38.91652300000021],[-105.97844799999979,38.91465500000004],[-105.97653099999991,38.91441900000001],[-105.97686499999992,38.912968000000035],[-105.97603299999992,38.9118390000001],[-105.97664399999985,38.9106220000001],[-105.97272900000002,38.90956100000005],[-105.97188599999998,38.908923000000016],[-105.96946399999996,38.90860600000008],[-105.96775699999984,38.906693000000075],[-105.96577000000002,38.90587599999998],[-105.96744699999988,38.90493900000007],[-105.96807499999994,38.9037830000002],[-105.96866099999994,38.90072000000009],[-105.97240699999992,38.89925699999998],[-105.97407299999992,38.89668000000029],[-105.97941799999995,38.89453300000008],[-105.97974499999981,38.891728],[-105.979175,38.888912999999945],[-105.97823999999991,38.887905000000046],[-105.9771439999999,38.88514600000002],[-105.97510799999992,38.883740999999986],[-105.97429699999998,38.88245800000021],[-105.97063700000001,38.88319600000017],[-105.96496499999995,38.885977000000025],[-105.96356000000003,38.885855000000106],[-105.959969,38.88440600000007],[-105.95564300000001,38.88369100000011],[-105.95381500000002,38.88419799999991],[-105.94913899999995,38.883104],[-105.94796399999996,38.88327100000015],[-105.94277299999999,38.885505999999964],[-105.941238,38.885591999999974],[-105.93887499999988,38.88332300000019],[-105.93602599999997,38.88144900000003],[-105.93297499999994,38.878219000000115],[-105.93115899999998,38.87470899999994],[-105.92684599999995,38.874235],[-105.92512299999999,38.87217799999996],[-105.92709000000002,38.8714820000003],[-105.92841099999998,38.869751000000065],[-105.927819,38.86884500000025],[-105.92913199999992,38.865945000000295],[-105.93144799999999,38.8665270000003],[-105.93251799999996,38.86450100000019],[-105.93205399999994,38.86323900000002],[-105.93300699999992,38.8610510000002],[-105.93501499999996,38.85910400000006],[-105.93711400000001,38.858407000000284],[-105.93951499999997,38.85659300000009],[-105.94317399999994,38.85230700000022],[-105.94288699999993,38.850503000000174],[-105.94056899999998,38.8492270000001],[-105.939685,38.84811400000001],[-105.93914699999999,38.84604600000017],[-105.940156,38.84352799999999],[-105.94184899999999,38.841268999999954],[-105.94162499999993,38.84017400000016],[-105.939662,38.83913600000005],[-105.936398,38.836571000000106],[-105.93505399999992,38.83430100000004],[-105.9322489999999,38.832731000000024],[-105.928067,38.828125000000284],[-105.92601299999995,38.82649100000026],[-105.92424599999987,38.82419300000004],[-105.92067299999991,38.82163000000003],[-105.91723199999996,38.81752500000022],[-105.913997,38.81606300000004],[-105.91258399999987,38.81389900000005],[-105.91072699999995,38.81236800000022],[-105.91068699999994,38.81048100000004],[-105.906587,38.806962],[-105.91045400000002,38.804876999999976],[-105.91149300000001,38.802931000000285],[-105.91055199999988,38.80197200000015],[-105.91179399999999,38.799878999999976],[-105.91268000000002,38.797354000000155],[-105.91290799999985,38.795513000000085],[-105.9144639999999,38.79373100000004],[-105.913704,38.79245899999995],[-105.9151389999999,38.790437],[-105.91804999999988,38.790245000000084],[-105.92112299999985,38.789360000000045],[-105.92163700000003,38.788105000000144],[-105.92368599999998,38.786246000000006],[-105.9267089999999,38.78550800000005],[-105.92529200000001,38.783787000000075],[-105.925161,38.78266699999995],[-105.92770599999989,38.782256000000245],[-105.92956299999997,38.78138999999999],[-105.92953499999999,38.77972199999999],[-105.93054999999993,38.77854100000002],[-105.92964199999994,38.777236000000016],[-105.92928899999987,38.7745470000001],[-105.93142799999993,38.7727490000002],[-105.93421199999989,38.77184300000022],[-105.936287,38.771622000000036],[-105.9372919999999,38.772088000000224],[-105.93974899999989,38.77191100000016],[-105.93989299999998,38.77067500000021],[-105.94251099999997,38.7685120000001],[-105.9445399999999,38.76810499999999],[-105.93970200000001,38.76476399999996],[-105.93875799999995,38.76302200000026],[-105.937994,38.75603600000011],[-105.93830599999995,38.753685000000246],[-105.94143299999996,38.748687000000245],[-105.94133699999992,38.745604999999955],[-105.94210699999996,38.74478800000003],[-105.94785999999999,38.743823000000134],[-105.947947,38.74168899999995],[-105.94426799999997,38.73706500000003],[-105.94475199999994,38.73576600000001],[-105.943017,38.73453200000006],[-105.94349299999988,38.73081800000023],[-105.94433300000003,38.730533000000264],[-105.94483000000002,38.72836500000011],[-105.94610799999998,38.72743500000007],[-105.94346899999994,38.72314399999999],[-105.944434,38.72174699999999],[-105.94610599999999,38.7216440000002],[-105.947565,38.72018900000006],[-105.94618499999984,38.716724000000056],[-105.949614,38.71501300000011],[-105.95016399999992,38.714375000000246],[-105.95303899999993,38.71365500000002],[-105.95544699999988,38.71066700000006],[-105.9572839999999,38.70897800000006],[-105.96182299999992,38.70605499999999],[-105.96364699999987,38.7013290000001],[-105.96537499999994,38.69897900000001],[-105.96772299999998,38.697135],[-105.96899599999995,38.69718600000016],[-105.96998599999989,38.69490300000007],[-105.96974999999998,38.69355100000013],[-105.96475099999981,38.692427000000066],[-105.96247299999999,38.69119500000005],[-105.95890199999985,38.691619],[-105.95751899999999,38.690562000000114],[-105.9549179999999,38.69016099999993],[-105.95262000000002,38.6902980000001],[-105.9510019999999,38.69091600000007],[-105.94834299999997,38.691013999999996],[-105.94530899999995,38.69022799999999],[-105.94538599999981,38.688078000000075],[-105.94416699999994,38.68556100000001],[-105.944275,38.68485199999998],[-105.94223599999998,38.6839820000003],[-105.93949299999986,38.68372600000009],[-105.93671399999994,38.68264900000014],[-105.932324,38.68187500000022],[-105.93079699999993,38.680761000000246],[-105.9262129999999,38.67990700000013],[-105.92599000000001,38.67886900000019],[-105.92458499999987,38.67751300000026],[-105.92264099999994,38.67700499999995],[-105.92231999999984,38.67508099999992],[-105.92497199999991,38.67256800000001],[-105.92410499999994,38.670388000000116],[-105.92575099999988,38.669384000000036],[-105.92548399999998,38.666365000000155],[-105.92023499999999,38.664314000000275],[-105.91974299999998,38.66280200000023],[-105.92090299999995,38.65979400000003],[-105.91713500000003,38.65956500000027],[-105.91410399999995,38.656000000000176],[-105.9129299999999,38.65379700000017],[-105.90917200000001,38.65364100000022],[-105.90792399999987,38.65267000000017],[-105.90791200000001,38.65093900000011],[-105.90945899999997,38.64916199999999],[-105.91096199999987,38.64835899999997],[-105.911202,38.64701500000001],[-105.91317400000003,38.64558300000016],[-105.91128900000001,38.64321400000006],[-105.91055099999988,38.64169400000026],[-105.91114999999996,38.639163999999994],[-105.91356399999995,38.63805999999994],[-105.915528,38.635637999999915],[-105.917327,38.63452100000006],[-105.91773899999998,38.633255000000304],[-105.91566499999999,38.63004800000016],[-105.91394599999995,38.629281000000105],[-105.91197499999993,38.62907799999999],[-105.90964599999995,38.62741900000009],[-105.90666799999997,38.627140000000054],[-105.90498999999983,38.62614600000006],[-105.90429499999988,38.62483099999997],[-105.90255100000002,38.62527400000022],[-105.90055499999988,38.62473800000009],[-105.89967899999988,38.62197300000014],[-105.89634799999988,38.622653999999955],[-105.8943579999999,38.623723000000155],[-105.88714900000002,38.62132900000012],[-105.88597199999987,38.62062600000007],[-105.88490999999993,38.61873100000008],[-105.88568099999986,38.6175310000001],[-105.88574699999998,38.6155490000001],[-105.88417500000003,38.61286300000006],[-105.88186399999995,38.61004700000012],[-105.87917599999986,38.60731199999998],[-105.87854499999997,38.60523200000017],[-105.87803299999979,38.60058100000009],[-105.87886699999996,38.598301000000276],[-105.8799049999999,38.59691699999996],[-105.88059599999997,38.59486300000003],[-105.88216699999998,38.59314800000004],[-105.88383899999997,38.59224800000021],[-105.88539000000003,38.590106000000105],[-105.88981999999993,38.58709800000008],[-105.89113800000001,38.583957000000055],[-105.89051699999987,38.581965999999966],[-105.89095299999997,38.579088999999954],[-105.89052599999991,38.57735600000029],[-105.891299,38.57488299999994],[-105.89248999999995,38.573289000000045],[-105.88846899999999,38.571978000000115],[-105.88805000000002,38.571220000000096],[-105.88552400000003,38.569782000000146],[-105.885876,38.56821899999994],[-105.88420199999996,38.56550099999998],[-105.88463899999999,38.562270000000126],[-105.88542899999987,38.5613120000001],[-105.89060999999992,38.561053000000015],[-105.89322500000003,38.55870300000004],[-105.89335999999992,38.55801000000008],[-105.89217899999989,38.554576],[-105.89244999999994,38.55213400000008],[-105.89352699999995,38.55093500000021],[-105.89714100000003,38.55079400000005],[-105.90024499999998,38.54944400000011],[-105.903411,38.548554000000195],[-105.90376000000003,38.547982000000104],[-105.90334899999976,38.54304900000017],[-105.90392899999989,38.53935100000007],[-105.90505000000002,38.53672500000005],[-105.90459399999986,38.534154000000115],[-105.90573299999994,38.532358000000215],[-105.90385200000003,38.53142900000006],[-105.90341699999999,38.52852699999994],[-105.90441099999998,38.52763400000015],[-105.90437299999996,38.52615100000003],[-105.9073459999999,38.52308400000027],[-105.9083399999999,38.522489000000235],[-105.90822999999995,38.520998000000134],[-105.90875900000003,38.516882000000294],[-105.907872,38.51489000000015],[-105.905666,38.51291299999997],[-105.9050539999999,38.51166300000011],[-105.905887,38.509577000000036],[-105.90607399999999,38.50759499999998],[-105.90685399999995,38.50626600000015],[-105.90871700000002,38.505630999999994],[-105.93803999999989,38.50347000000028],[-105.95229499999994,38.49237999999997],[-106.00056899999993,38.45464300000009],[-106.01075100000003,38.446566000000246],[-106.01597699999996,38.44762400000002],[-106.01767999999993,38.44869300000005],[-106.01691700000003,38.45179600000006],[-106.01904300000001,38.45442300000019],[-106.02148899999997,38.454469000000074],[-106.02354000000003,38.45391700000005],[-106.030665,38.45353800000015],[-106.03600999999992,38.45511500000009],[-106.04010499999998,38.45696300000003],[-106.04323399999998,38.457671000000175],[-106.05207399999989,38.45422100000019],[-106.05568399999999,38.45489000000009],[-106.05857100000003,38.45694500000013],[-106.06031000000002,38.45496400000019],[-106.06004299999995,38.45273700000013],[-106.06224399999991,38.45033000000029],[-106.05911100000003,38.44836399999997],[-106.05798199999987,38.444691000000034],[-106.06052599999987,38.441075000000126],[-106.06260199999997,38.439200000000255],[-106.06404599999996,38.43665400000009],[-106.06514400000003,38.43587400000001],[-106.06640199999975,38.43379600000003],[-106.06883199999993,38.43293700000021],[-106.07022599999988,38.432005000000004],[-106.06935399999986,38.43018000000018],[-106.07121199999995,38.429159000000254],[-106.07212999999996,38.42738700000001],[-106.07428199999993,38.42650300000008],[-106.07455500000003,38.424979000000064],[-106.07653499999992,38.42365200000006],[-106.08013699999987,38.423538000000235],[-106.08407799999998,38.422875999999974],[-106.08606799999978,38.424113000000034],[-106.08812999999998,38.42228000000023],[-106.12557099999998,38.422092000000305],[-106.17297500000001,38.422394000000054],[-106.17619699999995,38.4223860000003],[-106.22461599999997,38.422692999999924],[-106.24694,38.42276800000013],[-106.24790999999976,38.42395900000008],[-106.24861799999991,38.42604299999999],[-106.24837000000002,38.42863999999997],[-106.24948399999988,38.43324400000006],[-106.24814099999998,38.43602900000025],[-106.245903,38.43930300000005],[-106.25010299999997,38.43955600000004],[-106.25308699999994,38.44100300000002],[-106.26133299999992,38.44235500000002],[-106.2660709999999,38.44173500000022],[-106.26855499999999,38.443543000000034],[-106.2704839999999,38.445520000000215],[-106.27215199999989,38.44638900000018],[-106.27296799999999,38.447630000000174],[-106.27224699999994,38.44877600000012],[-106.27178999999995,38.45244200000025],[-106.27265499999993,38.455196],[-106.27724599999982,38.45428200000026],[-106.28060899999991,38.45433700000018],[-106.28380700000002,38.45679400000006],[-106.28853199999998,38.46137300000004],[-106.28928099999996,38.46275600000007],[-106.28913799999987,38.46432800000031],[-106.29002599999995,38.46685700000012],[-106.28966700000001,38.47008799999992],[-106.29049399999997,38.471802000000025],[-106.29054300000001,38.47352799999999],[-106.29299099999997,38.47480200000024],[-106.29559799999993,38.47558600000008],[-106.29981999999995,38.47571800000003],[-106.3030149999999,38.4769570000002],[-106.3045469999999,38.478707999999926],[-106.30412100000001,38.480052000000114],[-106.30648399999995,38.48234099999996],[-106.30680899999993,38.48414100000025],[-106.30824999999987,38.48586799999998],[-106.30929700000002,38.48796300000009],[-106.3096329999999,38.4905610000003],[-106.31053500000002,38.49122800000009],[-106.31138099999987,38.49465200000003],[-106.31554599999976,38.4944770000003],[-106.31831999999997,38.49515500000007],[-106.32052899999996,38.497645000000034],[-106.32302399999998,38.496943999999985],[-106.32784399999991,38.49732900000021],[-106.3301689999999,38.493690000000015],[-106.33246499999996,38.494530000000054],[-106.33630799999997,38.49665000000016],[-106.3388789999999,38.49926000000028],[-106.33875999999998,38.50056999999998],[-106.34135900000001,38.502411999999936],[-106.3428239999999,38.50288000000012],[-106.34422999999987,38.504590000000064],[-106.34542499999998,38.50713400000001],[-106.3454549999999,38.50823000000014],[-106.34659999999997,38.50966100000022],[-106.34810800000002,38.51249900000005],[-106.34581300000002,38.51496000000026],[-106.34504399999992,38.51667100000009],[-106.34370299999989,38.51746000000014],[-106.34259699999996,38.52153699999997],[-106.344063,38.52258499999999],[-106.34640799999988,38.52625300000028],[-106.34713099999993,38.528761000000145],[-106.35042299999998,38.53145000000006],[-106.35418500000003,38.53338400000001],[-106.35454299999992,38.53614600000009],[-106.35370599999993,38.53922000000006],[-106.35496399999982,38.54286800000028],[-106.35358399999996,38.54713300000003],[-106.35353299999991,38.54997300000002],[-106.35203399999989,38.55169100000023],[-106.35054299999996,38.552667000000156],[-106.35082999999992,38.55377699999997],[-106.350144,38.55558300000001],[-106.34664099999998,38.558864000000085],[-106.34745099999992,38.561041000000046],[-106.34668099999999,38.56278300000014],[-106.34686699999992,38.56510400000013],[-106.34461899999991,38.568564000000265],[-106.34738299999998,38.572598000000255],[-106.34987699999994,38.57476400000024],[-106.35167299999995,38.576905000000124],[-106.35197499999998,38.57883700000008],[-106.35341999999997,38.58146700000009],[-106.35129999999998,38.58348999999998],[-106.353139,38.58517999999998],[-106.35544900000002,38.5879360000003],[-106.35595099999989,38.59010000000029],[-106.35724199999993,38.5910370000002],[-106.35818799999987,38.59271300000012],[-106.36102399999987,38.594487000000186],[-106.36123499999991,38.595879999999966],[-106.35931700000003,38.59939300000002],[-106.3598219999999,38.60060499999997],[-106.362052,38.602346000000296],[-106.36260800000002,38.603493000000014],[-106.36469699999992,38.60588899999999],[-106.36364600000002,38.61022400000002],[-106.36384299999997,38.61362500000007],[-106.36525099999994,38.61589900000007],[-106.369126,38.61867800000016],[-106.37557999999996,38.621223999999984],[-106.37914899999993,38.620118000000105],[-106.38212499999997,38.62171900000004],[-106.38264100000004,38.623416000000134],[-106.384208,38.62386900000013],[-106.38536799999997,38.62499500000024],[-106.38626899999991,38.627782999999965],[-106.385174,38.632362000000114],[-106.38417700000002,38.633827],[-106.38191899999998,38.63541500000014],[-106.38036199999993,38.637057000000084],[-106.37989899999991,38.63848299999995],[-106.38316399999997,38.638922000000036],[-106.38578799999999,38.638627000000156],[-106.38663200000002,38.63965600000006],[-106.38898999999998,38.64089700000022],[-106.39016399999997,38.64103399999999],[-106.39282499999996,38.640085],[-106.39726699999994,38.640297000000146],[-106.39846799999987,38.639748],[-106.40118899999993,38.639702000000284],[-106.40223799999995,38.64216299999998],[-106.40150699999987,38.64307600000001],[-106.4017189999999,38.645782999999994],[-106.40418399999993,38.64673200000004],[-106.40794399999999,38.64767900000021],[-106.41175199999986,38.64702900000026],[-106.41467599999993,38.64540699999998],[-106.41722299999992,38.6449190000003],[-106.42017899999996,38.64670000000001],[-106.42180399999995,38.6467890000003],[-106.424037,38.648845999999935],[-106.42713899999995,38.65067299999998],[-106.4295449999999,38.65123500000027],[-106.430159,38.65346799999992],[-106.42908799999992,38.65762000000029],[-106.43131699999986,38.66328900000002],[-106.43113999999997,38.66506600000014],[-106.43220899999977,38.66792099999992],[-106.4340899999998,38.66887800000006],[-106.43590699999987,38.671150000000296],[-106.43617899999998,38.673735000000306],[-106.43807200000003,38.67663500000009],[-106.43705499999999,38.67806900000011],[-106.43703199999987,38.679374999999936],[-106.437861,38.68250000000006],[-106.44017599999995,38.68687],[-106.44595899999996,38.68898800000005],[-106.44618300000002,38.68983200000008],[-106.44835799999998,38.692945000000066],[-106.451054,38.695364999999924],[-106.45212599999996,38.69877700000018],[-106.45013499999999,38.702660000000094],[-106.44831299999998,38.704927999999995],[-106.4455299999999,38.705532999999946],[-106.44455299999998,38.706926000000294],[-106.44468,38.70803700000022],[-106.44349299999988,38.709231000000045],[-106.43814199999997,38.70863099999997],[-106.43458799999979,38.70927000000006],[-106.43165599999986,38.708708000000115],[-106.42946799999993,38.70981100000006],[-106.425275,38.710307000000114],[-106.4247089999999,38.712693],[-106.42322199999995,38.71616600000027],[-106.42278599999986,38.718857000000185],[-106.42346699999979,38.72084700000016],[-106.41918499999997,38.723989000000245],[-106.41743099999997,38.72567500000031],[-106.41666499999985,38.727814000000194],[-106.41675199999992,38.73375599999997],[-106.41849300000001,38.735303999999985],[-106.42004999999995,38.73880500000007],[-106.418658,38.73963200000014],[-106.41801099999992,38.74202800000012],[-106.41840099999985,38.749149999999986],[-106.4196369999998,38.7499950000003],[-106.41995299999985,38.751529000000005],[-106.41728,38.75542600000006],[-106.41509899999994,38.757094999999936],[-106.41614899999996,38.7592370000001],[-106.41626499999995,38.76172800000012],[-106.41714299999995,38.764435000000105],[-106.4192589999999,38.766000000000076],[-106.41805799999997,38.76827000000014],[-106.41745200000003,38.77047400000026],[-106.41549800000001,38.7724410000003],[-106.41549800000001,38.7744170000002],[-106.413816,38.779093000000046],[-106.414737,38.780186000000015],[-106.41663499999993,38.785196000000155],[-106.41658599999988,38.787623000000224],[-106.41451799999993,38.78944500000017],[-106.4130909999999,38.78999900000002],[-106.41288099999991,38.791161000000216],[-106.41436299999998,38.795203000000015],[-106.41505499999994,38.799636000000135],[-106.417911,38.80069100000003],[-106.41858100000002,38.80295400000023],[-106.42066599999987,38.804149000000166],[-106.42271099999977,38.807633000000294],[-106.42243199999996,38.809296000000074],[-106.42101799999995,38.81041399999998],[-106.42055099999993,38.81160899999992],[-106.41602499999999,38.81378700000005],[-106.4150449999999,38.81528200000008],[-106.41533400000003,38.81944200000021],[-106.41626299999996,38.82385000000005],[-106.41188799999992,38.82573599999995],[-106.40977799999996,38.827541999999994],[-106.40698699999996,38.8293340000003],[-106.405284,38.828397999999936],[-106.403505,38.82659699999999],[-106.39902099999995,38.825298000000146],[-106.39752899999996,38.82313400000015],[-106.39502399999992,38.82239299999992],[-106.39126899999991,38.82276000000013],[-106.3878479999999,38.823804000000166],[-106.38363200000003,38.82470500000028],[-106.38358900000003,38.82850400000001],[-106.38282199999992,38.83157900000026],[-106.37901999999991,38.835132000000044],[-106.37881899999991,38.83612500000015],[-106.37601000000001,38.8364160000001],[-106.37457899999998,38.83738600000004],[-106.37020399999994,38.8387190000002],[-106.37012599999997,38.840485000000115],[-106.36835699999989,38.84208700000016],[-106.36644199999989,38.84468100000021],[-106.36445799999996,38.846185000000105],[-106.36537099999993,38.8476060000001],[-106.36537799999996,38.85171900000006],[-106.36430000000001,38.85341900000003],[-106.36056599999989,38.85379999999992],[-106.35689100000002,38.855318000000125],[-106.35269299999987,38.85563400000001],[-106.34955099999996,38.85490300000026],[-106.34890599999989,38.856337999999994],[-106.34897999999998,38.86191000000002],[-106.34924999999993,38.86438600000008],[-106.34698200000003,38.86721300000022],[-106.34478000000001,38.86860500000029],[-106.34267399999993,38.869350999999995],[-106.34085399999998,38.87078300000002],[-106.338819,38.871722999999974],[-106.33584599999995,38.87229000000019],[-106.33677899999998,38.87499400000007],[-106.33524499999987,38.87927000000002],[-106.33406500000001,38.88130899999999],[-106.33248300000002,38.88069600000006],[-106.33052199999997,38.881772000000296],[-106.32750499999992,38.88229700000022],[-106.32644600000003,38.8841010000001],[-106.32645600000001,38.88686800000005],[-106.32603899999987,38.88892700000008],[-106.32470999999993,38.89051499999999],[-106.32131299999998,38.8925990000003],[-106.32055499999996,38.89470800000004],[-106.32086299999986,38.895859999999914],[-106.32288899999992,38.89801900000003],[-106.32471999999996,38.90292900000003],[-106.324115,38.904795999999976],[-106.32609899999994,38.911004000000275],[-106.33105599999982,38.911695000000066],[-106.33498400000002,38.91166599999997],[-106.33812799999993,38.9087200000003],[-106.34404899999993,38.90760599999999],[-106.34524099999987,38.90706699999993],[-106.34733399999993,38.908467000000144],[-106.35150099999993,38.90988800000031],[-106.35303499999992,38.91093599999999],[-106.3570729999999,38.91294500000009],[-106.36154399999998,38.91233900000009],[-106.36577299999988,38.90987100000012],[-106.36748199999988,38.906917000000135],[-106.372211,38.90639699999991],[-106.3734399999999,38.906608000000006],[-106.37622199999998,38.90953900000005],[-106.37574499999994,38.91303900000025],[-106.37716799999998,38.917981000000054],[-106.37698199999988,38.92267400000003],[-106.38035999999994,38.92487100000028],[-106.38083599999999,38.92568499999993],[-106.38299899999998,38.923932000000036],[-106.38345099999998,38.922964000000036],[-106.386257,38.922968999999966],[-106.38691299999999,38.92112099999997],[-106.38899399999997,38.91998100000001],[-106.39234399999987,38.91877500000021],[-106.392991,38.91771800000015],[-106.393959,38.91224900000026],[-106.399497,38.91194300000001],[-106.40212400000001,38.91061700000023],[-106.403208,38.910864999999944],[-106.40497699999992,38.912668999999994],[-106.406071,38.912982],[-106.40760199999994,38.91476200000017],[-106.41209100000003,38.915665000000274],[-106.41316599999988,38.91626800000006],[-106.41427399999998,38.91812900000002],[-106.41562699999992,38.91853000000003],[-106.41717999999992,38.92089800000019],[-106.41824400000002,38.925396000000205],[-106.41822699999994,38.92663000000016],[-106.41935399999983,38.92767700000013],[-106.421831,38.92806800000011],[-106.42495200000002,38.926094999999975],[-106.42744800000003,38.925922000000014],[-106.43026199999997,38.92490900000007],[-106.43206199999997,38.92304300000001],[-106.43433299999992,38.919999000000075],[-106.43486399999989,38.918184],[-106.43651999999997,38.91644699999995],[-106.43717499999997,38.91490600000003],[-106.43933300000003,38.915458],[-106.44214999999986,38.91557400000022],[-106.44606499999998,38.9138660000001],[-106.44800499999997,38.9143620000001],[-106.44995099999977,38.91301200000015],[-106.45380799999998,38.912755000000004],[-106.45781099999999,38.910078000000226],[-106.46194100000002,38.911303000000146],[-106.46503199999995,38.910353999999984],[-106.46748300000002,38.91226000000029],[-106.47004399999997,38.91305300000016],[-106.47220899999996,38.915927000000124],[-106.47491199999996,38.91612299999997],[-106.48090499999995,38.91864299999992],[-106.48211999999995,38.92093000000017],[-106.48565199999996,38.922108000000094],[-106.4873589999998,38.92350700000003],[-106.48993799999994,38.9263160000001],[-106.49071400000003,38.929685000000006],[-106.49385599999994,38.931089000000156],[-106.49625800000001,38.93347000000017],[-106.49875999999989,38.934690000000046],[-106.500583,38.93474800000001],[-106.50269499999996,38.936445000000106],[-106.50437099999999,38.93684899999994],[-106.505584,38.93945700000029],[-106.50583899999975,38.94292000000013],[-106.50531299999994,38.94439999999997],[-106.507162,38.94609200000019],[-106.50807700000001,38.94589000000002],[-106.513195,38.94748700000014],[-106.515738,38.94866400000001],[-106.51670000000001,38.95034700000002],[-106.518284,38.95151699999997],[-106.5198769999999,38.95396900000003],[-106.52200999999991,38.95387500000004],[-106.52312199999989,38.954445000000135],[-106.52628600000003,38.95505200000008],[-106.52851099999981,38.959882999999934],[-106.52584499999995,38.96312400000005],[-106.52405599999997,38.96420499999999],[-106.523147,38.965535000000045],[-106.521861,38.96866599999993],[-106.52442299999979,38.971093000000224],[-106.5246259999999,38.97231600000015],[-106.52628700000002,38.976588000000106],[-106.52838899999995,38.980727000000286],[-106.53201999999999,38.98138600000027],[-106.53605299999998,38.983491000000186],[-106.54214999999999,38.98827700000004],[-106.54426699999999,38.98914400000001],[-106.546762,38.991078000000016],[-106.55016299999994,38.99154500000026],[-106.55218300000001,38.99347600000016],[-106.55421099999995,38.99871100000007],[-106.55685499999987,38.99814600000002],[-106.55910099999988,38.99862400000012],[-106.560427,38.99823200000003],[-106.56612899999999,38.99379200000004],[-106.56852200000003,38.9944780000003],[-106.57472100000001,38.99282099999999],[-106.57716099999999,38.99044400000014],[-106.57952299999994,38.99170200000009],[-106.58093999999994,38.991970000000265],[-106.58276899999993,38.991476000000034],[-106.58559500000001,38.99269700000019],[-106.58680200000003,38.99269400000014],[-106.5904329999999,38.991271000000154],[-106.59415299999995,38.99093600000015],[-106.59550899999988,38.99127699999991],[-106.5992139999999,38.99799400000023],[-106.59542999999991,39.00213200000013],[-106.59445499999998,39.00418500000018],[-106.59516200000002,39.01030500000013],[-106.59495399999986,39.01375200000001],[-106.59517900000003,39.01685999999995],[-106.594921,39.02080900000004],[-106.59169199999997,39.024923000000115],[-106.5925959999999,39.027626999999995],[-106.59437700000001,39.02995900000002],[-106.59794099999993,39.03166600000003],[-106.59951599999994,39.0337880000003],[-106.59893599999992,39.03676799999994],[-106.59779700000001,39.03924900000004],[-106.59723299999996,39.04482100000013],[-106.594964,39.04539699999998],[-106.58913299999989,39.045582000000024],[-106.58424499999984,39.04718700000012],[-106.5807299999999,39.04871700000007],[-106.57755099999986,39.053210000000036],[-106.577966,39.05790800000011],[-106.500583,39.057475000000125],[-106.49506599999995,39.057252000000176],[-106.45739400000002,39.056911000000014],[-106.38906799999995,39.056295000000205],[-106.37557999999996,39.05630499999995],[-106.36863,39.0564720000001],[-106.34715499999987,39.05646200000001],[-106.30868599999997,39.05615000000029],[-106.29677499999997,39.056437000000074],[-106.24643500000002,39.05638300000021],[-106.23164700000001,39.056459000000075],[-106.19100600000002,39.05607300000014]]]},"properties":{"OBJECTID":9,"COUNTYFP":"015","Shape_Leng":3.12582189647,"Shape_Area":0.272389119235,"low":3270,"lowmod":6980,"LMMI":10325,"lowmoduniv":17180,"lowmod_pct":0.406286379511,"NAMELSAD10":"Chaffee County"}},
{"type":"Feature","geometry":{"type":"Polygon","coordinates":[[[-102.04657099999986,39.04703800000004],[-102.04628200000002,39.000004000000274],[-102.04604599999993,38.981776000000025],[-102.04604199999994,38.97321400000021],[-102.04586799999993,38.93872500000009],[-102.04559999999998,38.886261000000275],[-102.04545799999994,38.875007000000096],[-102.04557699999992,38.873296000000096],[-102.04535099999998,38.85689700000006],[-102.04540899999989,38.84278499999999],[-102.04539499999998,38.82597199999998],[-102.04547299999996,38.813666000000126],[-102.04533399999991,38.79946300000006],[-102.04538499999995,38.78335199999998],[-102.04528700000003,38.75532199999998],[-102.04539499999998,38.74221699999998],[-102.04520999999994,38.70452000000017],[-102.04516000000001,38.675220999999965],[-102.04507499999988,38.66956100000027],[-102.04520699999995,38.65855700000003],[-102.04510900000002,38.65433999999999],[-102.04511500000001,38.62973199999993],[-102.04479199999992,38.61534],[-102.04508199999992,38.61517200000014],[-102.06382999999994,38.61515600000007],[-102.06553499999995,38.615688000000034],[-102.06883399999998,38.615176000000304],[-102.07432399999993,38.6150770000001],[-102.096993,38.61501700000031],[-102.10623399999997,38.615065000000186],[-102.12532899999991,38.6148860000003],[-102.14391399999994,38.61511200000001],[-102.17069100000003,38.615296000000285],[-102.19963399999989,38.61521700000014],[-102.21753999999987,38.61500600000028],[-102.2379489999999,38.61514500000004],[-102.25438600000001,38.615196000000196],[-102.29068499999994,38.614979000000176],[-102.32567299999994,38.61532500000021],[-102.32783799999993,38.615151000000196],[-102.34288900000001,38.61524499999996],[-102.36448799999994,38.61525599999999],[-102.39078099999995,38.6151780000003],[-102.39930699999991,38.61509700000016],[-102.42143599999997,38.615027000000055],[-102.43498399999999,38.61482199999995],[-102.45624399999991,38.61482600000011],[-102.47908100000001,38.61433499999998],[-102.50221599999992,38.61394900000022],[-102.51096699999994,38.61369899999994],[-102.54652199999998,38.613438999999914],[-102.56559500000003,38.613372000000254],[-102.57900999999987,38.613268000000176],[-102.58690099999995,38.61332700000008],[-102.61160899999999,38.61324399999995],[-102.62049699999989,38.612914000000046],[-102.62547099999995,38.61325099999999],[-102.63297399999999,38.61325400000027],[-102.66143399999987,38.61347000000001],[-102.66303799999997,38.61335600000018],[-102.68588999999992,38.61365399999994],[-102.70902899999999,38.61381100000017],[-102.76628199999993,38.61413299999998],[-102.78594699999996,38.614400000000046],[-102.80659999999995,38.61450200000013],[-102.82325999999995,38.61445400000025],[-102.87547799999993,38.614597],[-102.93345699999992,38.61510099999998],[-102.98800099999988,38.615026],[-103.01199999999989,38.61481600000019],[-103.02563699999996,38.61499399999997],[-103.05424099999993,38.615068000000065],[-103.06222799999989,38.615020000000186],[-103.09905299999991,38.614314000000036],[-103.13641699999994,38.61323900000008],[-103.17294299999998,38.612450000000024],[-103.17261199999996,38.62891600000029],[-103.17247399999991,38.69005700000008],[-103.16415899999998,38.69013300000012],[-103.16406099999989,38.72555600000004],[-103.16378499999996,38.749998000000005],[-103.16380300000003,38.79136200000016],[-103.163835,38.83524299999999],[-103.16370799999999,38.83666900000003],[-103.163636,38.84595300000029],[-103.16373099999993,38.8495640000001],[-103.163745,38.86492700000008],[-103.16345999999999,38.87496100000021],[-103.16335599999991,38.927080000000274],[-103.163253,38.935704999999984],[-103.16308500000002,38.99999399999996],[-103.16298699999993,39.00814900000006],[-103.16302499999995,39.03760999999997],[-103.12535300000002,39.038601000000085],[-103.11562899999996,39.03845300000029],[-103.09438299999994,39.03893899999997],[-103.07119999999998,39.03925000000015],[-103.05137199999996,39.039561000000106],[-103.04915999999997,39.03943899999996],[-102.99332900000002,39.03932600000002],[-102.917934,39.0392470000001],[-102.87766599999998,39.03937900000017],[-102.85794399999992,39.039337000000046],[-102.84918500000003,39.03949299999999],[-102.81717799999996,39.03979500000014],[-102.798787,39.039710000000014],[-102.75047999999987,39.039968000000044],[-102.72411,39.04019900000014],[-102.71576599999997,39.03867500000018],[-102.68731099999991,39.039038000000005],[-102.66920399999998,39.039427000000046],[-102.65052899999989,39.03964499999995],[-102.63174300000003,39.039747000000034],[-102.59910499999995,39.04020700000012],[-102.58558900000003,39.040547000000174],[-102.57685100000003,39.04067600000019],[-102.55807699999997,39.040782999999976],[-102.52075100000002,39.04135600000012],[-102.51178500000003,39.04139300000003],[-102.499326,39.04166900000013],[-102.47520199999997,39.04240100000027],[-102.46655399999997,39.04271600000004],[-102.45445699999993,39.04277500000012],[-102.44729999999998,39.04293500000023],[-102.43397099999999,39.04364200000009],[-102.42037199999987,39.04416900000018],[-102.4092599999999,39.044167000000016],[-102.37546800000001,39.04484899999994],[-102.36773199999993,39.04562500000003],[-102.352438,39.04564900000025],[-102.33004599999992,39.04591600000015],[-102.32576699999998,39.045887999999934],[-102.28180199999986,39.04634800000002],[-102.24421899999999,39.04634200000021],[-102.22723100000002,39.04636800000003],[-102.21752900000001,39.04626700000023],[-102.20429999999988,39.04627900000014],[-102.17730299999988,39.04644000000013],[-102.13047899999998,39.046535000000006],[-102.12528399999985,39.04662700000017],[-102.11485199999993,39.04658100000012],[-102.08731099999989,39.04676100000006],[-102.06064100000003,39.04708200000016],[-102.04657099999986,39.04703800000004]]]},"properties":{"OBJECTID":10,"COUNTYFP":"017","Shape_Leng":3.11125876445,"Shape_Area":0.478627643406,"low":305,"lowmod":770,"LMMI":1290,"lowmoduniv":1955,"lowmod_pct":0.393861892583,"NAMELSAD10":"Cheyenne County"}},
{"type":"Feature","geometry":{"type":"Polygon","coordinates":[[[-105.39794899999993,39.74708700000019],[-105.39814899999993,39.733282000000145],[-105.39824899999996,39.71788700000019],[-105.39854500000001,39.7010190000002],[-105.39855,39.68598900000006],[-105.39834000000002,39.676184000000035],[-105.39834899999988,39.6652880000002],[-105.39823999999993,39.65838000000008],[-105.39824999999996,39.63658800000019],[-105.39844999999997,39.634607000000074],[-105.39844999999997,39.628693999999996],[-105.39895099999995,39.60478599999999],[-105.39894999999996,39.585087999999985],[-105.3991499999999,39.581588000000124],[-105.3991499999999,39.574388],[-105.39898599999992,39.57307400000019],[-105.39894899999996,39.566056],[-105.401051,39.56598800000006],[-105.42725100000001,39.56598800000006],[-105.42815999999999,39.56592800000027],[-105.4449219999999,39.56598800000006],[-105.45920899999993,39.56594200000001],[-105.46795299999991,39.566088000000036],[-105.48511699999995,39.566188000000295],[-105.500292,39.566134999999974],[-105.50782199999998,39.56625200000008],[-105.54777999999999,39.56721600000026],[-105.58798599999989,39.568197000000055],[-105.623875,39.56703600000009],[-105.63587499999988,39.566707000000065],[-105.65795000000003,39.56635],[-105.6951489999999,39.56564400000002],[-105.7216009999998,39.565269000000285],[-105.72405399999991,39.56537500000002],[-105.750561,39.56489700000003],[-105.77498000000003,39.56492300000025],[-105.82966199999998,39.564865000000054],[-105.83061499999997,39.566460000000234],[-105.83659899999998,39.56879000000009],[-105.83839799999998,39.57145300000002],[-105.83836899999989,39.572952000000214],[-105.8336779999998,39.576135000000136],[-105.83154200000001,39.57807999999994],[-105.82783399999988,39.57906900000006],[-105.82654400000001,39.58020000000022],[-105.82271899999989,39.580093999999974],[-105.82062300000001,39.58092499999998],[-105.82006299999995,39.5821390000001],[-105.81809099999987,39.58084000000008],[-105.81379300000003,39.582405999999935],[-105.80808300000001,39.58339500000005],[-105.80596200000002,39.58583800000008],[-105.803222,39.58981899999992],[-105.80137099999996,39.591244000000245],[-105.80043299999988,39.59321100000028],[-105.7961049999999,39.59393800000004],[-105.794374,39.594830000000115],[-105.79317300000002,39.59667800000011],[-105.79166399999997,39.59789100000006],[-105.7827449999998,39.600145000000055],[-105.78138799999999,39.60089699999992],[-105.77756699999992,39.60441700000001],[-105.77596899999998,39.605139000000065],[-105.77570199999991,39.60652800000008],[-105.77624099999991,39.60833800000012],[-105.77614299999988,39.61108000000013],[-105.77732699999996,39.61275700000016],[-105.78120599999994,39.61596000000014],[-105.78270599999996,39.61795699999999],[-105.78283699999997,39.62265600000012],[-105.78222800000003,39.6243300000001],[-105.78267399999999,39.62902400000013],[-105.78460299999989,39.63114700000011],[-105.78801699999985,39.632393000000036],[-105.79119100000003,39.634236999999985],[-105.79305799999986,39.63632800000016],[-105.79827499999993,39.63635399999998],[-105.80065699999994,39.635571000000084],[-105.80622700000004,39.634947000000295],[-105.81093899999996,39.63534800000008],[-105.81664599999993,39.633546000000194],[-105.81863599999991,39.634910000000104],[-105.81848399999996,39.63638700000007],[-105.819163,39.63965900000011],[-105.82017699999989,39.64053500000011],[-105.82099199999999,39.64256499999999],[-105.82269600000001,39.64232900000002],[-105.82626499999992,39.64092800000009],[-105.82873999999998,39.64194900000001],[-105.83395000000002,39.64222200000023],[-105.83669099999997,39.64295000000004],[-105.838863,39.64457800000014],[-105.83947399999994,39.64559500000024],[-105.84154599999994,39.64618400000012],[-105.84610199999997,39.64408700000001],[-105.84866499999993,39.64415600000001],[-105.85122799999999,39.646641999999986],[-105.85434099999998,39.649081000000194],[-105.8556559999999,39.652233000000024],[-105.8567809999999,39.65588800000012],[-105.85677299999998,39.65783000000005],[-105.85947499999997,39.65870200000006],[-105.86202600000001,39.66227600000002],[-105.86296799999997,39.664957000000015],[-105.86476700000003,39.66721799999999],[-105.86810500000001,39.66690599999998],[-105.87364000000002,39.66586100000018],[-105.87961299999978,39.663638000000276],[-105.88486099999994,39.65708399999994],[-105.888215,39.658427000000245],[-105.89394499999992,39.65941800000002],[-105.8968119999999,39.66466400000007],[-105.90369099999992,39.66443500000014],[-105.910753,39.66229400000009],[-105.91486599999996,39.66578000000004],[-105.91995500000002,39.669434000000024],[-105.92228499999999,39.67012300000022],[-105.92629,39.67298500000004],[-105.92703,39.6767030000002],[-105.92465299999998,39.68015600000007],[-105.92132900000001,39.68431000000004],[-105.919581,39.69028700000024],[-105.9204729999999,39.69253800000013],[-105.92004199999985,39.69388800000013],[-105.92461800000001,39.69897200000014],[-105.92406099999994,39.70157400000011],[-105.9245459999999,39.70342300000016],[-105.92185999999998,39.708124999999995],[-105.92140699999999,39.709936000000084],[-105.91806499999996,39.712084000000004],[-105.91676499999994,39.71357599999999],[-105.91177499999998,39.71529499999997],[-105.91201399999989,39.71615500000007],[-105.91049900000002,39.720713000000046],[-105.90884499999976,39.72294299999999],[-105.90746000000001,39.72404500000005],[-105.90827999999999,39.72624600000029],[-105.90472899999992,39.72877900000026],[-105.89995899999997,39.72878100000008],[-105.89175499999999,39.730387000000064],[-105.88943999999992,39.7315480000002],[-105.88694999999996,39.73417600000022],[-105.883714,39.73701200000022],[-105.88237499999997,39.74032799999998],[-105.88326599999988,39.74380700000006],[-105.88234299999999,39.745579000000134],[-105.88316599999996,39.74763500000023],[-105.88500099999999,39.749986000000035],[-105.88458800000001,39.752859000000285],[-105.88212099999981,39.75576899999999],[-105.88048100000003,39.75930299999999],[-105.88152400000001,39.76051700000028],[-105.88255400000003,39.762989000000175],[-105.8851059999999,39.76415500000019],[-105.88671599999998,39.766002999999955],[-105.88754399999999,39.77047700000031],[-105.88963799999999,39.77365800000007],[-105.88956699999994,39.77567299999998],[-105.89002899999991,39.778639999999996],[-105.89209,39.7808],[-105.89265699999993,39.78263800000002],[-105.89234599999986,39.784108000000174],[-105.89073899999994,39.786384],[-105.88769499999995,39.78902199999999],[-105.88790199999994,39.79013400000014],[-105.88786499999992,39.79582400000004],[-105.88739499999991,39.79699599999998],[-105.88335799999993,39.79800900000015],[-105.87474599999996,39.79903500000012],[-105.87230599999998,39.796692000000064],[-105.86873500000002,39.796628999999996],[-105.8601149999999,39.79694399999994],[-105.85735,39.79578800000024],[-105.85481099999998,39.795590000000004],[-105.85335700000002,39.79227400000008],[-105.85207500000001,39.791127000000074],[-105.85090100000002,39.789090999999985],[-105.84992599999981,39.788564000000065],[-105.84782299999989,39.78919400000001],[-105.84408799999994,39.78951800000016],[-105.84072299999997,39.78893700000003],[-105.83860900000002,39.78813200000002],[-105.83341899999988,39.78788900000001],[-105.82999699999988,39.78821900000031],[-105.82713899999993,39.78699799999998],[-105.82568699999996,39.78685900000005],[-105.82295399999998,39.78560500000003],[-105.821416,39.78806400000008],[-105.81987100000003,39.789491000000226],[-105.81841299999996,39.790013000000044],[-105.81417699999992,39.79029800000001],[-105.81051100000002,39.79147500000005],[-105.80971999999991,39.794962999999996],[-105.8075169999999,39.79805399999998],[-105.80486099999996,39.79903300000012],[-105.80193999999995,39.79933599999998],[-105.80126499999989,39.80145200000027],[-105.79901299999989,39.80343000000016],[-105.79580399999986,39.802919000000145],[-105.79373099999992,39.801944000000105],[-105.79093999999992,39.80310099999991],[-105.78670799999998,39.802474000000075],[-105.78466199999997,39.802434000000176],[-105.78231399999987,39.80117700000005],[-105.77971199999996,39.79905000000031],[-105.77701200000001,39.7981020000002],[-105.77337399999993,39.79723700000022],[-105.76945999999981,39.79570400000006],[-105.76595799999996,39.79485499999993],[-105.76388199999997,39.79498399999994],[-105.76095999999995,39.797101],[-105.75678099999988,39.79822200000018],[-105.75266699999992,39.79994000000022],[-105.750561,39.80130300000002],[-105.74968199999995,39.80094399999996],[-105.747074,39.80132500000025],[-105.74448999999987,39.80337500000002],[-105.74141999999989,39.804489000000046],[-105.73855999999995,39.805015000000196],[-105.73642599999994,39.804801000000055],[-105.73498099999995,39.8102310000001],[-105.73345699999987,39.81254200000001],[-105.73142599999994,39.814319000000125],[-105.72517600000003,39.81732800000009],[-105.72388399999988,39.819332000000145],[-105.72455799999994,39.82198599999998],[-105.724245,39.824073],[-105.72327899999999,39.825353000000234],[-105.719243,39.82754399999999],[-105.71896500000003,39.82896200000005],[-105.71933999999987,39.83105100000006],[-105.71698099999992,39.835472000000095],[-105.713369,39.838212],[-105.71233899999987,39.837710000000186],[-105.7081419999999,39.83672400000012],[-105.70327099999992,39.836712000000205],[-105.70126600000003,39.83712600000018],[-105.69634500000001,39.8389190000002],[-105.69574699999998,39.83948200000009],[-105.69461799999993,39.843711999999925],[-105.69379499999985,39.845475000000135],[-105.69354199999998,39.84785300000027],[-105.69131099999987,39.85133799999994],[-105.69034799999986,39.85199600000004],[-105.688849,39.851404000000116],[-105.686462,39.849229000000264],[-105.68299399999995,39.84836500000017],[-105.68144499999977,39.84749000000005],[-105.677347,39.84743099999997],[-105.67658599999999,39.84897600000028],[-105.67533999999984,39.84976300000005],[-105.67175199999997,39.84798000000029],[-105.66740500000003,39.84758000000005],[-105.66331100000002,39.850036000000046],[-105.66047099999997,39.85056800000001],[-105.65463799999998,39.84922799999998],[-105.65175599999992,39.84826100000009],[-105.64995999999991,39.84677199999999],[-105.64802299999997,39.846315000000004],[-105.64390200000003,39.84647200000006],[-105.64012600000001,39.84493700000007],[-105.63617599999992,39.84455399999996],[-105.63306499999999,39.84334299999995],[-105.63250299999987,39.841575000000034],[-105.63115099999999,39.839338000000055],[-105.631485,39.83767800000021],[-105.63024100000001,39.83656200000007],[-105.62812400000001,39.83347599999996],[-105.627569,39.832087],[-105.62356899999997,39.82989100000003],[-105.62126499999994,39.82896200000005],[-105.61759799999993,39.826168000000166],[-105.61632800000001,39.82554100000016],[-105.61362999999989,39.82268900000025],[-105.61156299999993,39.82473399999998],[-105.607755,39.823030000000074],[-105.606043,39.82282600000002],[-105.60344999999995,39.82076500000022],[-105.60197199999988,39.8179190000003],[-105.59744999999981,39.818813000000034],[-105.59466399999997,39.82013600000005],[-105.59202099999987,39.822694000000126],[-105.588638,39.82302800000025],[-105.5860689999999,39.82192600000019],[-105.58399599999996,39.82276699999994],[-105.57928099999992,39.822533000000135],[-105.57574399999993,39.82072000000005],[-105.57417799999996,39.81901300000021],[-105.57454199999995,39.81745000000001],[-105.5760709999999,39.81432900000027],[-105.57883599999997,39.81371500000006],[-105.57971599999996,39.812833000000296],[-105.57854099999997,39.81100000000015],[-105.57804999999996,39.80556200000001],[-105.57191999999998,39.802362000000244],[-105.57135299999987,39.80123200000003],[-105.5664119999999,39.8003360000003],[-105.5641,39.80036300000023],[-105.56157000000002,39.79586100000023],[-105.56041099999976,39.79486900000006],[-105.55832199999986,39.795322000000226],[-105.55526599999996,39.79424900000026],[-105.55433999999991,39.792028000000016],[-105.55112899999995,39.79023600000022],[-105.54597999999999,39.78698300000019],[-105.54876199999995,39.784635000000094],[-105.54863399999994,39.782812000000035],[-105.54787099999987,39.781057000000146],[-105.54467499999993,39.777990000000216],[-105.54425499999991,39.77692000000013],[-105.545076,39.77363500000013],[-105.54130799999996,39.773810000000026],[-105.53860499999996,39.77162000000021],[-105.533909,39.77015499999999],[-105.5311329999999,39.76969200000002],[-105.53020800000002,39.769078000000206],[-105.52558899999991,39.768686000000116],[-105.52128399999998,39.766094000000066],[-105.51850100000001,39.76511399999998],[-105.51650899999987,39.76498400000003],[-105.5122209999999,39.76598600000017],[-105.51018799999991,39.76581500000003],[-105.5083899999999,39.76444800000007],[-105.50408299999998,39.76318900000001],[-105.50138900000002,39.76300800000013],[-105.49425299999996,39.76379800000012],[-105.49386099999992,39.76271800000012],[-105.49593400000003,39.76138400000025],[-105.49668300000002,39.759650000000306],[-105.49536699999993,39.75767500000012],[-105.49296700000002,39.75690600000013],[-105.49070999999998,39.757189000000096],[-105.489511,39.75816199999991],[-105.48636499999992,39.75841200000019],[-105.4840299999999,39.759205000000065],[-105.47893499999998,39.75908300000009],[-105.47240799999992,39.757509000000255],[-105.46978100000001,39.75653600000004],[-105.46569,39.756359000000145],[-105.46265399999987,39.75701800000013],[-105.46129799999994,39.755612000000156],[-105.45462899999978,39.755789999999934],[-105.45104200000003,39.75379000000021],[-105.45032800000001,39.75288200000023],[-105.44801499999994,39.75320900000008],[-105.44138999999996,39.755517999999995],[-105.43797799999993,39.755855],[-105.43020000000001,39.75462700000003],[-105.42734999999993,39.755798000000084],[-105.42199399999998,39.75501800000001],[-105.42253399999998,39.753385000000094],[-105.42243499999995,39.7508600000001],[-105.42355199999992,39.749915000000044],[-105.42325800000003,39.74821599999996],[-105.42166900000001,39.74589099999997],[-105.4182669999999,39.74723499999999],[-105.41498199999995,39.749501000000066],[-105.409672,39.74988900000005],[-105.40818799999994,39.751037],[-105.40657599999997,39.75139500000029],[-105.4032299999999,39.750214000000085],[-105.40216499999997,39.7489020000001],[-105.39794899999993,39.74708700000019]]]},"properties":{"OBJECTID":11,"COUNTYFP":"019","Shape_Leng":1.7200934352,"Shape_Area":0.107815436282,"low":1670,"lowmod":2980,"LMMI":4820,"lowmoduniv":9005,"lowmod_pct":0.330927262632,"NAMELSAD10":"Clear Creek County"}},
{"type":"Feature","geometry":{"type":"Polygon","coordinates":[[[-106.03933099999989,37.40085200000027],[-106.03918499999997,37.37155999999993],[-106.03891199999987,37.356952999999976],[-106.01028600000001,37.35659700000002],[-106.00214099999977,37.35653700000006],[-105.98352799999992,37.35624600000011],[-105.978341,37.35634700000014],[-105.96999099999994,37.35628299999996],[-105.94980899999996,37.35647999999992],[-105.94731300000001,37.356615999999974],[-105.93826499999994,37.356527000000085],[-105.929664,37.356224999999995],[-105.927324,37.3563640000001],[-105.90996799999988,37.35630800000007],[-105.89812899999987,37.35642800000005],[-105.87481799999989,37.356313],[-105.85758499999991,37.35636100000022],[-105.85144300000002,37.356273000000215],[-105.847692,37.35645299999999],[-105.83837599999993,37.356263000000126],[-105.83720499999987,37.356363000000215],[-105.82038999999992,37.35620000000006],[-105.80530299999987,37.3562160000003],[-105.79596300000003,37.356359000000225],[-105.75931100000003,37.35668700000019],[-105.75057400000003,37.3569770000002],[-105.74333899999993,37.35679800000031],[-105.75004999999993,37.35115399999995],[-105.74977799999999,37.349334],[-105.747526,37.34950300000014],[-105.74800299999993,37.34802400000001],[-105.75252999999987,37.34555899999998],[-105.75319100000002,37.34365699999995],[-105.75223,37.34195100000022],[-105.75067299999995,37.34146000000027],[-105.74943399999995,37.33904900000027],[-105.74956500000002,37.337653000000046],[-105.75071499999996,37.33596699999998],[-105.75058999999993,37.33360100000016],[-105.7489589999999,37.33277000000015],[-105.74685199999999,37.33092500000026],[-105.74759399999994,37.32964900000002],[-105.74758699999978,37.327041000000236],[-105.74584899999996,37.32386100000002],[-105.74336199999999,37.32194300000015],[-105.73893499999997,37.320687000000305],[-105.73821799999979,37.31885700000004],[-105.73904400000004,37.31815600000016],[-105.74160999999998,37.31746300000003],[-105.74247399999996,37.315894000000014],[-105.74110199999996,37.31450700000005],[-105.73819299999985,37.31423800000016],[-105.73712599999988,37.31327400000015],[-105.73790200000002,37.31095499999992],[-105.736535,37.309567000000015],[-105.73656599999998,37.308084000000065],[-105.73792799999995,37.3076980000003],[-105.73810699999984,37.30624899999992],[-105.73667699999993,37.30565800000005],[-105.73582699999986,37.30384900000013],[-105.73456299999998,37.30269900000002],[-105.73224399999987,37.303111],[-105.73145699999986,37.30463499999996],[-105.72916499999985,37.30487100000016],[-105.7279999999999,37.30146300000007],[-105.72784000000001,37.2988850000001],[-105.72556599999996,37.29550200000017],[-105.72245099999998,37.29473900000028],[-105.722285,37.29241400000012],[-105.72049500000003,37.29112900000018],[-105.72009200000002,37.2898390000002],[-105.72053800000003,37.28849300000002],[-105.720036,37.28695400000004],[-105.72030799999993,37.28561400000001],[-105.72236399999997,37.28533600000014],[-105.72416199999998,37.28635399999996],[-105.72612500000002,37.286509000000194],[-105.72751399999993,37.28586600000028],[-105.72834599999993,37.28353099999998],[-105.72528,37.281026000000054],[-105.72538799999978,37.28007500000001],[-105.7234709999999,37.27667500000007],[-105.72391399999987,37.27461100000005],[-105.72967699999992,37.27134100000018],[-105.7328839999999,37.26637600000009],[-105.73618499999998,37.266447000000255],[-105.73765300000002,37.26554600000014],[-105.73877900000002,37.263563000000204],[-105.73667299999994,37.26014100000003],[-105.73757899999987,37.25766699999997],[-105.7408769999999,37.255353000000184],[-105.74278200000003,37.255384000000106],[-105.74352299999998,37.253714000000286],[-105.74547799999999,37.25104399999998],[-105.7462099999999,37.24897599999997],[-105.74609899999996,37.24777399999999],[-105.74422799999996,37.24655800000005],[-105.74383699999993,37.24393900000007],[-105.7408779999999,37.24092500000029],[-105.74077799999998,37.2362040000001],[-105.74171899999993,37.23391200000003],[-105.74605399999996,37.23133400000023],[-105.746693,37.230367],[-105.74604499999992,37.22657700000008],[-105.74376799999999,37.22354800000011],[-105.74359299999992,37.22148900000013],[-105.74558699999994,37.21889600000014],[-105.74576499999995,37.218131000000085],[-105.74424999999991,37.21499400000005],[-105.74105499999996,37.21332800000005],[-105.73955199999995,37.2118430000001],[-105.73989899999998,37.21054899999996],[-105.73834299999993,37.20733699999994],[-105.73864099999997,37.20633199999992],[-105.73801099999991,37.20389799999998],[-105.7392539999999,37.202779000000305],[-105.738271,37.200807000000054],[-105.73644200000001,37.20021900000023],[-105.73597699999993,37.19931500000007],[-105.73655599999995,37.19717300000008],[-105.73806099999996,37.19608000000005],[-105.73900100000003,37.194699000000014],[-105.73892599999994,37.191314999999975],[-105.73815400000001,37.19014400000009],[-105.7358769999999,37.189422000000036],[-105.73464099999995,37.189620000000104],[-105.73284599999994,37.18822200000022],[-105.732235,37.18455300000005],[-105.7297549999999,37.18050600000004],[-105.72974599999998,37.17945900000029],[-105.7309229999999,37.17602500000004],[-105.73514199999994,37.172941000000094],[-105.73690799999997,37.17071499999997],[-105.73678899999987,37.16853100000026],[-105.73803499999985,37.16548400000022],[-105.73685199999994,37.16370100000012],[-105.7358349999999,37.16333700000001],[-105.7341669999999,37.160997000000066],[-105.73225000000002,37.159368000000256],[-105.73153500000001,37.157985000000224],[-105.731404,37.156249000000116],[-105.732121,37.15329700000029],[-105.73413499999992,37.149888000000146],[-105.73725499999989,37.147736000000066],[-105.74062000000004,37.14504499999998],[-105.74409599999996,37.14349100000021],[-105.74575799999991,37.1400730000002],[-105.74786499999988,37.137471000000005],[-105.74883399999993,37.1350010000001],[-105.74818299999998,37.1332680000001],[-105.7477899999999,37.12859000000026],[-105.74872399999998,37.12241000000017],[-105.74987099999987,37.11967900000013],[-105.74978099999998,37.11464699999999],[-105.7491619999999,37.112172000000214],[-105.74543999999997,37.10627900000003],[-105.74430499999994,37.104937000000064],[-105.741646,37.10406400000028],[-105.74083699999977,37.103297000000055],[-105.74092799999994,37.10145300000005],[-105.74263299999996,37.09876399999996],[-105.74606399999999,37.0952850000001],[-105.74789699999997,37.09239100000008],[-105.74866999999995,37.08970799999997],[-105.74845399999975,37.0889660000002],[-105.75020599999976,37.0870800000003],[-105.75130399999995,37.08665200000007],[-105.753941,37.08352099999996],[-105.75440400000002,37.08226500000001],[-105.75701299999997,37.078155000000265],[-105.75966699999992,37.07700000000011],[-105.76060799999988,37.07581000000022],[-105.76117399999998,37.073976000000016],[-105.76272399999999,37.07340600000026],[-105.76565800000003,37.06847499999992],[-105.76668799999987,37.06623500000023],[-105.77026000000001,37.06459700000005],[-105.77080699999999,37.062527000000216],[-105.77017099999995,37.06116600000007],[-105.76772399999999,37.06042900000017],[-105.76411999999993,37.056199000000106],[-105.76788199999999,37.05120000000022],[-105.76691699999992,37.04903299999995],[-105.76749999999993,37.047832000000255],[-105.770306,37.047196999999926],[-105.771391,37.045725000000004],[-105.77052500000002,37.0446130000002],[-105.77067299999999,37.041777000000195],[-105.76999799999993,37.041258000000084],[-105.76710600000001,37.041475000000105],[-105.76639999999986,37.04026000000027],[-105.76879499999995,37.03760300000016],[-105.76872900000001,37.03645300000022],[-105.76576699999998,37.034923000000106],[-105.76518099999998,37.03317199999998],[-105.76210099999986,37.027462999999955],[-105.76096099999995,37.02614700000015],[-105.75861199999986,37.02596100000022],[-105.75670300000002,37.02629200000007],[-105.75412799999992,37.02761400000003],[-105.75194999999991,37.02794300000005],[-105.74971399999993,37.026668000000086],[-105.74904599999991,37.025274000000024],[-105.74897699999997,37.02350300000023],[-105.75073299999997,37.021360000000016],[-105.75021100000004,37.019476999999995],[-105.74922099999998,37.01802200000009],[-105.74718899999999,37.018260000000055],[-105.7452899999999,37.01757000000015],[-105.74651699999998,37.01476800000012],[-105.74505199999993,37.013911000000064],[-105.74147899999997,37.01367500000009],[-105.73911800000002,37.013030000000015],[-105.73785199999998,37.013923000000204],[-105.73352699999998,37.01478700000007],[-105.73152499999998,37.014057000000264],[-105.7284709999999,37.01498200000009],[-105.72725300000002,37.014786000000186],[-105.72494899999998,37.01338300000009],[-105.723793,37.0107790000003],[-105.72544799999997,37.00958900000006],[-105.72657199999998,37.00810200000012],[-105.72652299999999,37.006542000000195],[-105.72301899999997,37.00405299999994],[-105.72335399999997,37.00225500000005],[-105.72278299999999,37.00079200000016],[-105.71871599999992,36.99987200000004],[-105.71802099999996,36.99896899999993],[-105.7195999999999,36.99672799999996],[-105.71846299999987,36.99584099999993],[-105.75056999999987,36.99569400000007],[-105.77990399999993,36.995795999999984],[-105.82742899999994,36.99573399999997],[-105.87176,36.99559900000003],[-105.89964499999991,36.99555099999992],[-105.96907399999992,36.99556899999999],[-105.98656399999993,36.995406],[-105.99615899999998,36.99541799999997],[-106.01515599999999,36.99526300000008],[-106.05275799999998,36.99503800000031],[-106.06632899999994,36.99492600000008],[-106.07057299999991,36.99497200000002],[-106.09982199999996,36.99478600000003],[-106.11476799999991,36.994731000000115],[-106.12557900000002,36.99485900000025],[-106.15385900000001,36.994694000000266],[-106.15562799999981,36.99451199999993],[-106.17938500000002,36.99440700000014],[-106.17969199999993,36.994237999999996],[-106.20146899999992,36.994122000000004],[-106.23479199999991,36.99413900000002],[-106.23603200000002,36.994249000000025],[-106.24868500000002,36.99423999999999],[-106.25057200000003,36.99415700000009],[-106.28720199999998,36.99413800000008],[-106.29805499999992,36.993891000000076],[-106.30894799999999,36.99416600000012],[-106.32253500000002,36.99404900000019],[-106.327605,36.994211000000064],[-106.33977299999987,36.9941350000002],[-106.343432,36.99423700000028],[-106.37530099999987,36.99400800000001],[-106.42284299999994,36.99388900000008],[-106.47622899999988,36.99375800000007],[-106.47974399999998,36.99936100000008],[-106.479963,37.00001000000009],[-106.48906199999993,37.01508000000018],[-106.50058899999988,37.02944500000018],[-106.52130899999997,37.05536100000006],[-106.5509019999999,37.09237400000012],[-106.57698699999992,37.12500799999998],[-106.58917799999989,37.14018799999997],[-106.59738400000003,37.19497400000023],[-106.603406,37.19740100000013],[-106.61440600000003,37.20214300000026],[-106.62558999999993,37.20669200000009],[-106.6783539999999,37.22856600000017],[-106.67870499999998,37.2500060000001],[-106.67842200000001,37.30310700000001],[-106.67810199999991,37.36346100000003],[-106.67830300000003,37.37500700000004],[-106.678246,37.3965730000001],[-106.67837299999997,37.40359600000011],[-106.67837299999997,37.403663000000165],[-106.64781099999988,37.40315500000003],[-106.64698099999998,37.40325400000006],[-106.62659399999995,37.402415000000076],[-106.61435099999994,37.402221000000054],[-106.5884779999999,37.4027200000001],[-106.57610699999992,37.40233500000005],[-106.56366699999995,37.402247000000045],[-106.54458199999993,37.402262000000235],[-106.52188999999981,37.402206000000035],[-106.48797000000002,37.40158500000018],[-106.48796800000002,37.39934999999997],[-106.42359499999992,37.400092000000086],[-106.37558999999999,37.400667999999996],[-106.36918400000002,37.400847000000226],[-106.35087499999992,37.401044000000184],[-106.35024399999992,37.40127799999999],[-106.288002,37.40106900000029],[-106.25058699999994,37.40095400000018],[-106.24478499999992,37.40090100000009],[-106.17632000000003,37.400938999999994],[-106.17390599999993,37.40108399999991],[-106.162599,37.40104999999994],[-106.14886599999988,37.401099999999985],[-106.09406599999994,37.401112000000296],[-106.07809600000002,37.401033000000155],[-106.06113299999998,37.40106000000026],[-106.03933099999989,37.40085200000027]]]},"properties":{"OBJECTID":12,"COUNTYFP":"021","Shape_Leng":2.79157552906,"Shape_Area":0.339353730078,"low":2635,"lowmod":4105,"LMMI":6100,"lowmoduniv":8190,"lowmod_pct":0.501221001221,"NAMELSAD10":"Conejos County"}},
{"type":"Feature","geometry":{"type":"Polygon","coordinates":[[[-105.15417599999995,37.29312900000008],[-105.15458299999989,37.25003200000009],[-105.15468799999996,37.21413899999999],[-105.154898,37.164156000000105],[-105.15493399999997,37.16296200000011],[-105.15506299999998,37.12501900000001],[-105.154988,37.11112000000003],[-105.15495499999986,37.07718600000021],[-105.15490799999998,37.02206500000011],[-105.15506099999999,37.000023999999996],[-105.15504199999992,36.995262000000025],[-105.16121899999996,36.99529400000017],[-105.22239799999994,36.99516600000004],[-105.2506709999999,36.99564500000025],[-105.26079400000003,36.99560299999996],[-105.29054299999996,36.99560400000024],[-105.33468299999993,36.99566700000014],[-105.41938600000003,36.99585700000006],[-105.43799200000001,36.99596999999994],[-105.45504399999993,36.99588599999993],[-105.46518199999997,36.99599100000029],[-105.48447699999986,36.99598300000014],[-105.49846500000001,36.995795000000044],[-105.50056399999977,36.995915000000025],[-105.51088600000003,36.99595899999997],[-105.52856199999997,36.995795999999984],[-105.53401599999995,36.995875000000126],[-105.57853799999992,36.995945000000006],[-105.60953099999995,36.99586800000003],[-105.62926799999997,36.995679999999936],[-105.660639,36.995781000000136],[-105.66471999999999,36.995874000000015],[-105.69372099999998,36.99580699999996],[-105.69766099999998,36.99573800000013],[-105.71315999999985,36.99587200000002],[-105.71846299999987,36.99584099999993],[-105.7195999999999,36.99672799999996],[-105.71802099999996,36.99896899999993],[-105.71871599999992,36.99987200000004],[-105.72278299999999,37.00079200000016],[-105.72335399999997,37.00225500000005],[-105.72301899999997,37.00405299999994],[-105.72652299999999,37.006542000000195],[-105.72657199999998,37.00810200000012],[-105.72544799999997,37.00958900000006],[-105.723793,37.0107790000003],[-105.72494899999998,37.01338300000009],[-105.72725300000002,37.014786000000186],[-105.7284709999999,37.01498200000009],[-105.73152499999998,37.014057000000264],[-105.73352699999998,37.01478700000007],[-105.73785199999998,37.013923000000204],[-105.73911800000002,37.013030000000015],[-105.74147899999997,37.01367500000009],[-105.74505199999993,37.013911000000064],[-105.74651699999998,37.01476800000012],[-105.7452899999999,37.01757000000015],[-105.74718899999999,37.018260000000055],[-105.74922099999998,37.01802200000009],[-105.75021100000004,37.019476999999995],[-105.75073299999997,37.021360000000016],[-105.74897699999997,37.02350300000023],[-105.74904599999991,37.025274000000024],[-105.74971399999993,37.026668000000086],[-105.75194999999991,37.02794300000005],[-105.75412799999992,37.02761400000003],[-105.75670300000002,37.02629200000007],[-105.75861199999986,37.02596100000022],[-105.76096099999995,37.02614700000015],[-105.76210099999986,37.027462999999955],[-105.76518099999998,37.03317199999998],[-105.76576699999998,37.034923000000106],[-105.76872900000001,37.03645300000022],[-105.76879499999995,37.03760300000016],[-105.76639999999986,37.04026000000027],[-105.76710600000001,37.041475000000105],[-105.76999799999993,37.041258000000084],[-105.77067299999999,37.041777000000195],[-105.77052500000002,37.0446130000002],[-105.771391,37.045725000000004],[-105.770306,37.047196999999926],[-105.76749999999993,37.047832000000255],[-105.76691699999992,37.04903299999995],[-105.76788199999999,37.05120000000022],[-105.76411999999993,37.056199000000106],[-105.76772399999999,37.06042900000017],[-105.77017099999995,37.06116600000007],[-105.77080699999999,37.062527000000216],[-105.77026000000001,37.06459700000005],[-105.76668799999987,37.06623500000023],[-105.76565800000003,37.06847499999992],[-105.76272399999999,37.07340600000026],[-105.76117399999998,37.073976000000016],[-105.76060799999988,37.07581000000022],[-105.75966699999992,37.07700000000011],[-105.75701299999997,37.078155000000265],[-105.75440400000002,37.08226500000001],[-105.753941,37.08352099999996],[-105.75130399999995,37.08665200000007],[-105.75020599999976,37.0870800000003],[-105.74845399999975,37.0889660000002],[-105.74866999999995,37.08970799999997],[-105.74789699999997,37.09239100000008],[-105.74606399999999,37.0952850000001],[-105.74263299999996,37.09876399999996],[-105.74092799999994,37.10145300000005],[-105.74083699999977,37.103297000000055],[-105.741646,37.10406400000028],[-105.74430499999994,37.104937000000064],[-105.74543999999997,37.10627900000003],[-105.7491619999999,37.112172000000214],[-105.74978099999998,37.11464699999999],[-105.74987099999987,37.11967900000013],[-105.74872399999998,37.12241000000017],[-105.7477899999999,37.12859000000026],[-105.74818299999998,37.1332680000001],[-105.74883399999993,37.1350010000001],[-105.74786499999988,37.137471000000005],[-105.74575799999991,37.1400730000002],[-105.74409599999996,37.14349100000021],[-105.74062000000004,37.14504499999998],[-105.73725499999989,37.147736000000066],[-105.73413499999992,37.149888000000146],[-105.732121,37.15329700000029],[-105.731404,37.156249000000116],[-105.73153500000001,37.157985000000224],[-105.73225000000002,37.159368000000256],[-105.7341669999999,37.160997000000066],[-105.7358349999999,37.16333700000001],[-105.73685199999994,37.16370100000012],[-105.73803499999985,37.16548400000022],[-105.73678899999987,37.16853100000026],[-105.73690799999997,37.17071499999997],[-105.73514199999994,37.172941000000094],[-105.7309229999999,37.17602500000004],[-105.72974599999998,37.17945900000029],[-105.7297549999999,37.18050600000004],[-105.732235,37.18455300000005],[-105.73284599999994,37.18822200000022],[-105.73464099999995,37.189620000000104],[-105.7358769999999,37.189422000000036],[-105.73815400000001,37.19014400000009],[-105.73892599999994,37.191314999999975],[-105.73900100000003,37.194699000000014],[-105.73806099999996,37.19608000000005],[-105.73655599999995,37.19717300000008],[-105.73597699999993,37.19931500000007],[-105.73644200000001,37.20021900000023],[-105.738271,37.200807000000054],[-105.7392539999999,37.202779000000305],[-105.73801099999991,37.20389799999998],[-105.73864099999997,37.20633199999992],[-105.73834299999993,37.20733699999994],[-105.73989899999998,37.21054899999996],[-105.73955199999995,37.2118430000001],[-105.74105499999996,37.21332800000005],[-105.74424999999991,37.21499400000005],[-105.74576499999995,37.218131000000085],[-105.74558699999994,37.21889600000014],[-105.74359299999992,37.22148900000013],[-105.74376799999999,37.22354800000011],[-105.74604499999992,37.22657700000008],[-105.746693,37.230367],[-105.74605399999996,37.23133400000023],[-105.74171899999993,37.23391200000003],[-105.74077799999998,37.2362040000001],[-105.7408779999999,37.24092500000029],[-105.74383699999993,37.24393900000007],[-105.74422799999996,37.24655800000005],[-105.74609899999996,37.24777399999999],[-105.7462099999999,37.24897599999997],[-105.74547799999999,37.25104399999998],[-105.74352299999998,37.253714000000286],[-105.74278200000003,37.255384000000106],[-105.7408769999999,37.255353000000184],[-105.73757899999987,37.25766699999997],[-105.73667299999994,37.26014100000003],[-105.73877900000002,37.263563000000204],[-105.73765300000002,37.26554600000014],[-105.73618499999998,37.266447000000255],[-105.7328839999999,37.26637600000009],[-105.72967699999992,37.27134100000018],[-105.72391399999987,37.27461100000005],[-105.7234709999999,37.27667500000007],[-105.72538799999978,37.28007500000001],[-105.72528,37.281026000000054],[-105.72834599999993,37.28353099999998],[-105.72751399999993,37.28586600000028],[-105.72612500000002,37.286509000000194],[-105.72416199999998,37.28635399999996],[-105.72236399999997,37.28533600000014],[-105.72030799999993,37.28561400000001],[-105.720036,37.28695400000004],[-105.72053800000003,37.28849300000002],[-105.72009200000002,37.2898390000002],[-105.72049500000003,37.29112900000018],[-105.722285,37.29241400000012],[-105.72245099999998,37.29473900000028],[-105.72556599999996,37.29550200000017],[-105.72784000000001,37.2988850000001],[-105.7279999999999,37.30146300000007],[-105.72916499999985,37.30487100000016],[-105.73145699999986,37.30463499999996],[-105.73224399999987,37.303111],[-105.73456299999998,37.30269900000002],[-105.73582699999986,37.30384900000013],[-105.73667699999993,37.30565800000005],[-105.73810699999984,37.30624899999992],[-105.73792799999995,37.3076980000003],[-105.73656599999998,37.308084000000065],[-105.736535,37.309567000000015],[-105.73790200000002,37.31095499999992],[-105.73712599999988,37.31327400000015],[-105.73819299999985,37.31423800000016],[-105.74110199999996,37.31450700000005],[-105.74247399999996,37.315894000000014],[-105.74160999999998,37.31746300000003],[-105.73904400000004,37.31815600000016],[-105.73821799999979,37.31885700000004],[-105.73893499999997,37.320687000000305],[-105.74336199999999,37.32194300000015],[-105.74584899999996,37.32386100000002],[-105.74758699999978,37.327041000000236],[-105.74759399999994,37.32964900000002],[-105.74685199999999,37.33092500000026],[-105.7489589999999,37.33277000000015],[-105.75058999999993,37.33360100000016],[-105.75071499999996,37.33596699999998],[-105.74956500000002,37.337653000000046],[-105.74943399999995,37.33904900000027],[-105.75067299999995,37.34146000000027],[-105.75223,37.34195100000022],[-105.75319100000002,37.34365699999995],[-105.75252999999987,37.34555899999998],[-105.74800299999993,37.34802400000001],[-105.747526,37.34950300000014],[-105.74977799999999,37.349334],[-105.75004999999993,37.35115399999995],[-105.74333899999993,37.35679800000031],[-105.73396000000002,37.364874999999984],[-105.72195299999998,37.37500799999998],[-105.69005900000002,37.401638000000105],[-105.6365889999999,37.44630600000005],[-105.62163399999991,37.45944300000008],[-105.60232499999995,37.475031],[-105.59701899999988,37.4801750000002],[-105.57548999999989,37.497919000000195],[-105.57287899999994,37.49989700000026],[-105.56101799999999,37.50973400000004],[-105.55409599999996,37.515598000000125],[-105.54240600000003,37.52531800000003],[-105.53596199999976,37.530558000000156],[-105.50056699999993,37.560141000000044],[-105.49630999999988,37.56385199999994],[-105.49730499999998,37.56685999999996],[-105.49605999999989,37.57006000000007],[-105.49422700000002,37.572173000000134],[-105.49278299999992,37.572948999999994],[-105.49066799999997,37.57546200000013],[-105.48834099999999,37.57691400000016],[-105.48551600000002,37.577899000000116],[-105.48095399999994,37.57865500000014],[-105.47884299999998,37.57987700000018],[-105.47594100000003,37.580629000000044],[-105.47144700000001,37.58126300000009],[-105.469855,37.58187100000015],[-105.46727499999986,37.58442400000001],[-105.46447799999999,37.584609],[-105.4633209999999,37.585952000000304],[-105.46026799999999,37.58672999999999],[-105.45449200000002,37.590483000000006],[-105.45494400000001,37.59110400000026],[-105.45452799999998,37.59324900000007],[-105.45522399999999,37.59391499999998],[-105.45245399999988,37.5961970000003],[-105.45130199999994,37.59846700000003],[-105.44784999999996,37.60025200000007],[-105.44817199999994,37.60240900000002],[-105.4458869999998,37.604287],[-105.44504399999988,37.607875000000206],[-105.44313799999992,37.61107800000019],[-105.44227899999998,37.611851],[-105.43995000000001,37.612188],[-105.43834099999992,37.61328800000007],[-105.43602099999998,37.613578000000075],[-105.43158299999988,37.61338599999999],[-105.4285099999999,37.61390500000016],[-105.42350699999986,37.616780000000176],[-105.42090499999995,37.61671900000027],[-105.41699699999998,37.61741299999994],[-105.41491099999996,37.61588300000005],[-105.41085199999992,37.61698300000012],[-105.40839999999992,37.61800000000022],[-105.403256,37.61565700000017],[-105.40107499999993,37.61433899999997],[-105.39985799999994,37.61423800000017],[-105.39615000000003,37.616776000000016],[-105.39437699999996,37.61745799999994],[-105.39342599999998,37.61709500000012],[-105.39136499999995,37.61835500000029],[-105.38966899999997,37.62116599999996],[-105.389837,37.62179299999997],[-105.386551,37.62643800000012],[-105.38457099999977,37.62760500000002],[-105.38079699999997,37.62795300000022],[-105.38072899999986,37.62887999999998],[-105.37841000000003,37.63169199999999],[-105.37689,37.63445200000024],[-105.3737509999998,37.63579099999998],[-105.37080900000001,37.63824899999997],[-105.37030499999997,37.63939600000003],[-105.36824000000001,37.63857600000023],[-105.36584099999993,37.63824100000022],[-105.36404099999993,37.63845400000025],[-105.36111499999987,37.636666000000105],[-105.35967799999997,37.63641799999999],[-105.35677399999997,37.63672100000002],[-105.35503399999988,37.63615800000002],[-105.35450899999995,37.637241000000074],[-105.35202099999998,37.638196999999934],[-105.34855999999996,37.63775800000002],[-105.34695499999987,37.64116600000011],[-105.34544800000003,37.64202400000005],[-105.33715099999995,37.64264100000014],[-105.334945,37.64326500000004],[-105.329542,37.64298100000019],[-105.32749899999999,37.64434],[-105.32228900000001,37.64617800000002],[-105.31834400000002,37.648806000000036],[-105.31577399999992,37.64815500000003],[-105.31335599999994,37.64971600000001],[-105.309844,37.650526999999954],[-105.30911699999996,37.653434000000175],[-105.30319699999995,37.65308500000009],[-105.30180100000001,37.65360299999992],[-105.29870499999998,37.65664099999992],[-105.29762299999999,37.65814300000005],[-105.29620599999998,37.65787800000015],[-105.29540599999979,37.65679700000004],[-105.29558699999995,37.65441600000008],[-105.29410899999976,37.65282700000006],[-105.29081999999977,37.651546999999994],[-105.28969699999993,37.650442000000226],[-105.28700499999997,37.65095300000007],[-105.28554699999995,37.6468790000003],[-105.28491999999994,37.64588600000002],[-105.28334699999999,37.6452560000003],[-105.28152,37.645380999999986],[-105.27994999999993,37.64637499999998],[-105.27867599999996,37.646352000000036],[-105.27599399999991,37.645223999999985],[-105.27295899999996,37.64100500000029],[-105.26830399999994,37.64061200000009],[-105.26711,37.639871000000255],[-105.26736299999988,37.63874800000025],[-105.26593700000001,37.63824899999997],[-105.26350899999994,37.63911399999995],[-105.26146499999993,37.639003],[-105.259229,37.63956899999994],[-105.25814600000001,37.64054700000008],[-105.25530299999997,37.6402580000003],[-105.25364399999995,37.63673900000009],[-105.25436300000001,37.63522999999998],[-105.25421199999994,37.63232400000021],[-105.25219900000002,37.63025900000008],[-105.24876399999994,37.62958700000024],[-105.24581799999999,37.62995600000022],[-105.24336199999999,37.62922900000012],[-105.24259499999994,37.62606800000003],[-105.24156599999998,37.623724000000095],[-105.23659399999991,37.621665000000235],[-105.23597799999993,37.620773000000156],[-105.22974299999998,37.62039700000031],[-105.22614499999992,37.620368999999926],[-105.22121600000003,37.62117100000023],[-105.22002699999996,37.621920000000046],[-105.21950800000002,37.62423799999999],[-105.21579300000002,37.62638099999998],[-105.21453399999996,37.62642500000027],[-105.21211899999997,37.62353200000018],[-105.20549599999998,37.620626999999956],[-105.20150699999999,37.61919400000005],[-105.1944929999998,37.61966400000023],[-105.19216299999994,37.61926599999998],[-105.18754899999999,37.6196690000001],[-105.18444099999999,37.615984000000026],[-105.186597,37.615738000000135],[-105.18973900000003,37.61444700000004],[-105.19261699999993,37.611851],[-105.19285999999988,37.610478000000285],[-105.19531399999988,37.60944100000029],[-105.19483199999996,37.60736200000002],[-105.19540899999993,37.604673000000105],[-105.19634999999988,37.603176000000246],[-105.19524199999995,37.601424000000236],[-105.194095,37.601035000000024],[-105.19415499999991,37.599631000000215],[-105.19336299999992,37.59806300000014],[-105.1932369999999,37.592168000000186],[-105.19669499999992,37.59275400000001],[-105.19875199999996,37.594065000000285],[-105.20231299999989,37.59478800000005],[-105.2037699999999,37.59444900000011],[-105.20570699999996,37.59080899999998],[-105.2118999999999,37.58608900000007],[-105.21715699999999,37.584858],[-105.21843299999989,37.58349599999997],[-105.21833299999986,37.58243900000025],[-105.21604400000001,37.581186000000116],[-105.21425999999985,37.5797280000001],[-105.21385999999995,37.57788300000004],[-105.211681,37.57766100000015],[-105.20763099999999,37.57524000000001],[-105.20531499999993,37.574261000000206],[-105.20161599999994,37.57402300000024],[-105.19829699999997,37.57181499999996],[-105.19542099999984,37.569177000000195],[-105.19515999999993,37.56810600000023],[-105.19351,37.566921000000036],[-105.19372199999998,37.565516],[-105.19480899999985,37.56465500000013],[-105.19306199999988,37.56389000000007],[-105.19188600000001,37.56235600000002],[-105.18888599999985,37.55947800000007],[-105.18401699999987,37.55584100000004],[-105.18343299999987,37.55383499999999],[-105.18388499999998,37.55263800000006],[-105.1826089999999,37.55212799999998],[-105.17933899999997,37.548676],[-105.17835200000002,37.54631900000021],[-105.18267699999984,37.54678100000024],[-105.18295299999994,37.544651000000215],[-105.182098,37.5425800000001],[-105.18378799999994,37.54089600000003],[-105.18299799999994,37.53819800000008],[-105.17984099999995,37.53612600000025],[-105.17771399999998,37.53410400000013],[-105.17707999999993,37.53031400000003],[-105.17754799999994,37.52972700000015],[-105.17637999999994,37.52614400000016],[-105.17465600000003,37.52516900000012],[-105.17363799999998,37.52299800000009],[-105.17433399999993,37.519889999999975],[-105.17211099999986,37.519030999999984],[-105.1715329999999,37.51750200000009],[-105.17251499999998,37.51560700000016],[-105.17131999999987,37.51324600000021],[-105.17412399999995,37.50929900000011],[-105.17338499999994,37.50579500000009],[-105.17428299999989,37.50251400000025],[-105.1733319999999,37.501415000000065],[-105.172483,37.49922000000004],[-105.17766599999993,37.49139100000025],[-105.17571599999997,37.489526000000126],[-105.17449299999993,37.48697599999997],[-105.17481599999985,37.48510200000004],[-105.17642499999982,37.483408],[-105.18010300000003,37.48157900000001],[-105.18204299999996,37.47970400000014],[-105.17668299999985,37.47559400000006],[-105.17668500000002,37.47261300000014],[-105.17596999999984,37.470259000000055],[-105.17500099999995,37.46528799999999],[-105.17346799999996,37.461096000000055],[-105.17541399999993,37.45714199999992],[-105.17536999999993,37.45628799999997],[-105.17142699999982,37.455363000000034],[-105.16958099999994,37.45614100000006],[-105.16803400000003,37.45602799999995],[-105.16450699999996,37.45459900000003],[-105.16253099999989,37.454765000000066],[-105.15918499999998,37.457114000000104],[-105.156185,37.45709000000011],[-105.15515299999998,37.45567400000016],[-105.15443299999998,37.453554000000054],[-105.15218899999996,37.45155400000016],[-105.15098699999999,37.44706100000019],[-105.14956199999995,37.443182000000036],[-105.14682399999987,37.44109200000014],[-105.14590599999997,37.438586999999984],[-105.14646099999987,37.436001999999974],[-105.14453400000002,37.433515000000284],[-105.14395100000002,37.431737999999996],[-105.14488599999993,37.42896500000006],[-105.14221399999991,37.42917100000005],[-105.14091300000001,37.427918999999974],[-105.14152199999995,37.42365600000005],[-105.140695,37.42181099999999],[-105.14052599999997,37.418128999999965],[-105.13941599999998,37.41558500000002],[-105.13712699999985,37.413102000000094],[-105.13462700000002,37.411992000000055],[-105.13265199999995,37.41154500000022],[-105.130585,37.41165500000005],[-105.12972000000002,37.40925400000003],[-105.13125600000001,37.406812000000286],[-105.13266899999985,37.40360600000008],[-105.13856699999997,37.40069100000028],[-105.14251599999994,37.40094600000003],[-105.14499799999999,37.40023500000001],[-105.146323,37.40041500000018],[-105.15132399999987,37.400192000000004],[-105.15516899999989,37.399636000000214],[-105.15726899999999,37.39843600000006],[-105.16143299999999,37.39320900000007],[-105.16264899999999,37.39030200000008],[-105.16641799999991,37.388769999999965],[-105.16716699999978,37.38583600000004],[-105.16531700000002,37.38117399999999],[-105.16887899999995,37.37563800000021],[-105.16841199999988,37.37366900000001],[-105.16675799999996,37.372253000000114],[-105.16574899999989,37.36982300000011],[-105.16645699999992,37.36559799999992],[-105.16890599999999,37.36151400000006],[-105.16866799999991,37.36024100000026],[-105.16697599999998,37.3581470000002],[-105.16804699999994,37.354693000000225],[-105.16636099999988,37.35305900000003],[-105.16600099999994,37.351674],[-105.16642399999995,37.34988800000002],[-105.16586899999987,37.3473180000002],[-105.16411600000004,37.34521599999999],[-105.1640499999998,37.34302200000019],[-105.16568899999999,37.34150900000009],[-105.16794900000002,37.33703100000008],[-105.16727700000001,37.33484199999998],[-105.16560499999986,37.333529999999996],[-105.16393899999986,37.33096800000027],[-105.16309599999994,37.32890900000007],[-105.163523,37.32738900000027],[-105.16531499999996,37.325988000000166],[-105.16669200000001,37.323735000000056],[-105.16669300000001,37.32120899999995],[-105.16543899999999,37.316780000000165],[-105.16691899999995,37.313792000000205],[-105.164938,37.308520999999985],[-105.16476899999998,37.304430999999965],[-105.16339299999999,37.29994600000015],[-105.16533699999991,37.295228000000066],[-105.16515800000002,37.292792000000134],[-105.16547199999997,37.291241000000014],[-105.164559,37.28911700000015],[-105.16019,37.29110900000012],[-105.158953,37.29236900000012],[-105.15711299999998,37.293035000000145],[-105.15417599999995,37.29312900000008]]]},"properties":{"OBJECTID":13,"COUNTYFP":"023","Shape_Leng":2.63247283097,"Shape_Area":0.323758707204,"low":1505,"lowmod":2155,"LMMI":2680,"lowmoduniv":3575,"lowmod_pct":0.602797202797,"NAMELSAD10":"Costilla County"}},
{"type":"Feature","geometry":{"type":"Polygon","coordinates":[[[-103.50466599999993,38.51640900000024],[-103.50465300000002,38.50000100000011],[-103.50525199999993,38.491281000000185],[-103.50582099999986,38.4588],[-103.50629100000003,38.42925200000013],[-103.50666899999993,38.40791800000011],[-103.50677000000002,38.37500300000005],[-103.50698499999999,38.34225600000002],[-103.50174700000002,38.342295000000036],[-103.50138699999997,38.32697500000006],[-103.50161000000003,38.32251000000031],[-103.501643,38.276535000000024],[-103.50176399999998,38.265015999999946],[-103.51006000000001,38.265169000000185],[-103.50987899999996,38.26409100000001],[-103.50971099999992,38.232119000000125],[-103.50955299999998,38.21621299999998],[-103.50955199999999,38.204075000000046],[-103.5094219999998,38.19295699999992],[-103.509364,38.17251400000009],[-103.53579099999996,38.17265500000002],[-103.55947300000003,38.172672000000205],[-103.56033300000001,38.17276400000003],[-103.58393499999994,38.17271500000021],[-103.60107499999987,38.17171900000022],[-103.61928299999988,38.17145500000004],[-103.619011,38.143152000000214],[-103.61917199999993,38.11335500000001],[-103.62549999999993,38.11333500000012],[-103.67373099999998,38.11290600000018],[-103.70132699999988,38.11315300000018],[-103.747591,38.113074999999924],[-103.750674,38.113214000000255],[-103.77033,38.11292500000013],[-103.79126699999995,38.112437000000114],[-103.80303300000003,38.11253300000021],[-103.805028,38.11457799999994],[-103.80731600000001,38.116400000000056],[-103.8098179999999,38.117508999999984],[-103.8119099999999,38.1178450000001],[-103.81376499999999,38.117418999999984],[-103.81859099999997,38.114835000000255],[-103.82186599999994,38.113516000000004],[-103.82590799999991,38.11306200000013],[-103.82870999999983,38.11441000000008],[-103.83144799999991,38.11420500000003],[-103.83561599999996,38.113339999999994],[-103.83770299999998,38.114163000000076],[-103.84168699999992,38.117346],[-103.84459200000003,38.12079399999999],[-103.84621099999998,38.12340500000022],[-103.85033499999997,38.12504800000028],[-103.85508299999987,38.12550399999998],[-103.85650599999985,38.125185999999985],[-103.86028599999997,38.1261320000001],[-103.86348999999996,38.1280720000002],[-103.86626199999989,38.128760000000284],[-103.86867199999989,38.12880900000005],[-103.87094099999996,38.127808000000016],[-103.87324899999999,38.12629900000002],[-103.87800099999987,38.12464899999992],[-103.88368199999996,38.124577000000045],[-103.88684599999993,38.12527799999992],[-103.89252999999985,38.12524300000024],[-103.895059,38.1247400000002],[-103.9006619999999,38.125409000000104],[-103.90028899999993,38.127281000000096],[-103.89904599999994,38.128473000000156],[-103.89851199999998,38.13113200000009],[-103.90023899999989,38.13301000000001],[-103.902064,38.133975000000135],[-103.90619400000003,38.13485999999995],[-103.91250699999995,38.13434600000005],[-103.91352499999988,38.13162400000027],[-103.91173300000003,38.129885000000286],[-103.91087399999992,38.12794500000001],[-103.91098,38.12656300000003],[-103.91439600000001,38.124760000000094],[-103.91630199999997,38.12472900000017],[-103.91868699999986,38.12554200000011],[-103.91990299999992,38.127531000000204],[-103.92015200000003,38.12927400000001],[-103.92339900000002,38.13042400000029],[-103.92593,38.12964600000004],[-103.92904099999998,38.12732800000009],[-103.93011999999993,38.12500500000027],[-103.93398899999994,38.12441700000011],[-103.94458600000002,38.126687000000175],[-103.94823199999979,38.12815800000021],[-103.95126999999991,38.13020400000005],[-103.95494699999995,38.13081200000005],[-103.96015499999999,38.130401000000006],[-103.96678500000002,38.131674000000146],[-103.97003000000001,38.13161700000023],[-103.97437799999989,38.132119000000046],[-103.976407,38.13298999999995],[-103.97695699999986,38.134348000000045],[-103.9741279999999,38.13862700000027],[-103.97499099999999,38.140336000000275],[-103.9777529999999,38.141068000000075],[-103.98070399999989,38.140579000000116],[-103.9842129999999,38.14091400000012],[-103.98759899999999,38.142776000000026],[-103.99120299999987,38.14543300000008],[-103.99469599999986,38.14665400000024],[-103.99828699999989,38.146256000000164],[-103.99922800000002,38.14578300000011],[-104.00096799999994,38.14333199999999],[-104.00256100000001,38.14277000000021],[-104.00510600000001,38.143541000000084],[-104.00775099999993,38.145547000000306],[-104.01022699999999,38.145737000000054],[-104.015061,38.14280900000006],[-104.01943499999987,38.14185800000007],[-104.02215899999993,38.14095099999997],[-104.02719000000002,38.140489000000116],[-104.03312799999998,38.14057700000012],[-104.03621599999997,38.14139699999993],[-104.04398399999991,38.141594000000055],[-104.04704500000003,38.141389000000174],[-104.047913,38.14523800000012],[-104.05040599999995,38.14654900000005],[-104.05595399999999,38.14601900000008],[-104.058242,38.14649199999997],[-104.05837500000001,38.16561899999999],[-104.05822999999992,38.17407900000012],[-104.05834899999991,38.178716000000065],[-104.05845099999999,38.19996100000003],[-104.05829799999987,38.22582800000009],[-104.05848799999995,38.25220500000029],[-104.05840199999994,38.2608410000002],[-104.05756399999996,38.26164799999998],[-104.0547249999999,38.261889000000224],[-104.05500399999988,38.289355000000285],[-104.05542400000002,38.323657000000026],[-104.05548399999998,38.33614000000006],[-104.055699,38.35692699999993],[-104.05593599999992,38.392679999999984],[-104.05593699999991,38.40302200000002],[-104.05613700000004,38.41346500000009],[-104.0563489999999,38.43569000000008],[-104.05631399999993,38.4386820000002],[-104.05557699999997,38.46448600000008],[-104.05523499999998,38.47333200000014],[-104.0550169999999,38.484117000000026],[-104.054712,38.491898000000106],[-104.05465399999997,38.50254200000023],[-104.05419299999994,38.51961099999994],[-104.05392099999989,38.522393000000136],[-104.041339,38.52234200000015],[-104.03722800000003,38.52247899999992],[-104.03446999999989,38.52230800000018],[-104.0172849999999,38.522594000000026],[-103.98503800000003,38.52277500000008],[-103.96255899999994,38.52313700000002],[-103.96225099999987,38.523212],[-103.93145199999992,38.5231740000001],[-103.91913199999999,38.52311300000002],[-103.91051599999986,38.52332999999999],[-103.90625599999981,38.52289400000018],[-103.89643999999998,38.52289500000006],[-103.88614999999999,38.5226330000001],[-103.87550799999991,38.5227720000002],[-103.87260899999995,38.52211700000004],[-103.85170699999998,38.52202100000005],[-103.77915300000001,38.52177100000017],[-103.77002899999997,38.52215200000006],[-103.76080699999989,38.52210900000006],[-103.75088800000003,38.522231000000204],[-103.73354199999994,38.52193300000005],[-103.71902899999992,38.52179600000028],[-103.71547599999997,38.52156800000006],[-103.71345599999995,38.521817000000055],[-103.70652799999988,38.521939999999915],[-103.70572499999997,38.52224400000023],[-103.68208600000003,38.52243199999998],[-103.67791799999986,38.522210000000086],[-103.64861399999995,38.52261699999997],[-103.61298999999991,38.52254800000014],[-103.61298299999999,38.51787899999999],[-103.56753099999997,38.51726100000002],[-103.50466599999993,38.51640900000024]]]},"properties":{"OBJECTID":14,"COUNTYFP":"025","Shape_Leng":1.95538656369,"Shape_Area":0.213559171438,"low":1975,"lowmod":2955,"LMMI":3725,"lowmoduniv":4425,"lowmod_pct":0.667796610169,"NAMELSAD10":"Crowley County"}},
{"type":"Feature","geometry":{"type":"Polygon","coordinates":[[[-105.04921499999989,38.25797300000022],[-105.04922299999993,38.25000300000016],[-105.04873099999998,38.239697999999976],[-105.04867899999994,38.22201800000016],[-105.04848599999997,38.17593800000026],[-105.04846099999992,38.15870000000001],[-105.04828700000002,38.12777299999999],[-105.05016499999999,38.11736100000019],[-105.05067799999995,38.11382400000008],[-105.04950999999994,38.03990500000003],[-105.04998099999983,38.006885000000125],[-105.05018999999999,38.0000060000001],[-105.04997299999997,37.98532400000016],[-105.05012399999987,37.984590000000026],[-105.04991699999988,37.915479000000005],[-105.05430699999994,37.91750900000028],[-105.05720100000002,37.915946000000076],[-105.06149700000003,37.91606100000001],[-105.06319999999988,37.916976000000034],[-105.06584999999995,37.92076300000002],[-105.06513499999994,37.921456000000035],[-105.06503399999986,37.923241000000075],[-105.06630099999995,37.925048000000004],[-105.06606299999999,37.92624999999998],[-105.063737,37.92926900000003],[-105.06357200000002,37.935001],[-105.06434199999995,37.936635000000024],[-105.06603000000001,37.93744700000008],[-105.06642499999992,37.938525000000084],[-105.06547099999995,37.941802000000166],[-105.06605599999995,37.944209],[-105.070472,37.945802000000015],[-105.07221899999996,37.947789000000284],[-105.07424400000002,37.94851300000016],[-105.07779999999997,37.95296400000001],[-105.08182999999991,37.95425],[-105.08438299999983,37.95538400000004],[-105.08553199999989,37.95114100000001],[-105.088011,37.94858800000014],[-105.09029499999997,37.947636999999986],[-105.09382799999992,37.94732900000008],[-105.09909700000003,37.94494300000002],[-105.10312899999997,37.94494000000009],[-105.10850700000003,37.94339899999994],[-105.11298399999993,37.94544900000011],[-105.11560799999995,37.94749999999999],[-105.117163,37.949141000000054],[-105.12066599999986,37.94959699999998],[-105.12261199999995,37.94917700000002],[-105.12597499999993,37.951170000000104],[-105.12511099999995,37.95308700000004],[-105.12555099999986,37.95459200000005],[-105.12690599999996,37.95513400000016],[-105.12710499999991,37.956457],[-105.12618799999996,37.95781200000022],[-105.12641099999996,37.9607500000003],[-105.12757599999998,37.96117400000003],[-105.12790899999999,37.962591999999916],[-105.13340899999997,37.96303900000015],[-105.13564699999995,37.96527700000024],[-105.13558,37.96729400000015],[-105.13319499999983,37.97042300000004],[-105.13133500000004,37.9711880000001],[-105.13102499999997,37.97268100000002],[-105.13235999999995,37.97464299999996],[-105.1405499999999,37.979774000000134],[-105.14214499999997,37.97979900000007],[-105.14164099999994,37.98203400000011],[-105.139501,37.98460699999998],[-105.14124099999998,37.98686000000009],[-105.14390900000001,37.98783800000001],[-105.14645200000001,37.98658900000021],[-105.14796499999989,37.986841000000084],[-105.15071699999993,37.99078900000006],[-105.152491,37.99211300000002],[-105.15226099999995,37.99435599999998],[-105.15319899999997,38.00078400000018],[-105.15190999999987,38.003330000000005],[-105.15038599999991,38.004547000000116],[-105.148416,38.005078000000196],[-105.14796899999993,38.006239000000164],[-105.15141399999987,38.00844799999999],[-105.15286800000001,38.00974699999995],[-105.15729899999991,38.012955000000034],[-105.158366,38.01313299999998],[-105.16076499999997,38.01494700000001],[-105.16110099999997,38.01586600000019],[-105.16582599999987,38.01682099999999],[-105.166898,38.01812899999999],[-105.16865200000001,38.018946000000085],[-105.17173100000002,38.01804400000026],[-105.17326399999996,38.017142000000035],[-105.17670799999979,38.01635200000004],[-105.17890299999988,38.01647400000002],[-105.18100599999997,38.01559500000013],[-105.18181900000002,38.014693000000136],[-105.18525199999993,38.0133790000001],[-105.18774999999994,38.010598000000016],[-105.18928299999993,38.00975200000005],[-105.19280800000001,38.00861600000019],[-105.19458599999996,38.00756000000001],[-105.19756799999993,38.00436600000006],[-105.19891899999999,38.00378499999999],[-105.20004399999999,38.00083499999994],[-105.19707299999993,37.99241600000005],[-105.19881399999991,37.990181000000234],[-105.20192400000002,37.99044800000013],[-105.2060679999999,37.98966800000005],[-105.20583699999997,37.98861899999997],[-105.20589699999977,37.983268000000066],[-105.20770699999997,37.98165499999999],[-105.20924099999996,37.982228000000134],[-105.21398599999992,37.980808999999965],[-105.2142209999999,37.980260000000214],[-105.21310899999992,37.97733100000016],[-105.21463299999988,37.97688700000003],[-105.21650499999998,37.975378000000035],[-105.21635799999996,37.97253000000029],[-105.21536599999996,37.97052000000025],[-105.21662700000002,37.96944700000029],[-105.21809499999989,37.96990299999999],[-105.2231359999999,37.968804000000205],[-105.22468300000003,37.96667400000018],[-105.22464599999995,37.965036],[-105.22692099999989,37.964567999999986],[-105.22941800000001,37.965245000000095],[-105.23442599999993,37.964019000000235],[-105.23580899999996,37.96280000000007],[-105.23855999999995,37.96284000000003],[-105.23993299999995,37.96039300000007],[-105.23868899999997,37.95941900000031],[-105.23759799999993,37.957568000000094],[-105.24003299999998,37.955080000000294],[-105.2417989999999,37.95380899999992],[-105.24471599999993,37.95372800000001],[-105.24669399999999,37.95211000000023],[-105.24861699999985,37.95210899999995],[-105.25001799999995,37.950378000000114],[-105.25543599999997,37.94291599999997],[-105.25676399999998,37.94255499999997],[-105.25754599999993,37.938131],[-105.26154899999995,37.93716800000027],[-105.26316799999978,37.93245800000011],[-105.26394199999987,37.93205899999998],[-105.26379999999995,37.9288200000002],[-105.26184199999994,37.92563700000028],[-105.264815,37.921377000000234],[-105.26804999999996,37.91830399999998],[-105.26990799999987,37.91366100000022],[-105.271433,37.91281500000002],[-105.27341799999994,37.910716000000264],[-105.273818,37.90767699999998],[-105.27623299999988,37.905875000000094],[-105.2769879999999,37.90410300000002],[-105.27892399999996,37.90364499999998],[-105.28054899999995,37.901215000000036],[-105.28507599999995,37.899421000000075],[-105.28673600000002,37.90019400000011],[-105.28936699999991,37.90286900000007],[-105.29103999999995,37.90389700000003],[-105.29078399999992,37.90536800000007],[-105.29154299999999,37.90754599999997],[-105.29240699999991,37.908625000000086],[-105.29481099999987,37.90822000000014],[-105.29743100000002,37.909349000000134],[-105.29801099999997,37.911263000000076],[-105.300254,37.91248800000017],[-105.30111499999992,37.91462700000022],[-105.30065499999995,37.915887],[-105.30165899999997,37.91811899999993],[-105.300388,37.921586000000104],[-105.30054099999995,37.92308500000013],[-105.30360699999977,37.926188000000025],[-105.30436599999996,37.92839300000003],[-105.30631599999992,37.93096700000001],[-105.30657399999996,37.932292000000075],[-105.31028099999997,37.93354000000011],[-105.31109300000003,37.93594499999995],[-105.31377199999991,37.93688800000001],[-105.31450699999988,37.93844399999995],[-105.31708099999997,37.93894900000004],[-105.32452599999993,37.93843400000003],[-105.331702,37.93580100000014],[-105.33570999999989,37.933468000000005],[-105.3374609999999,37.93205500000016],[-105.34007500000001,37.92932399999995],[-105.34207600000002,37.92841200000004],[-105.344536,37.92637800000011],[-105.34720900000002,37.92475900000028],[-105.35081399999984,37.92488900000001],[-105.35605399999997,37.92285400000003],[-105.36113899999998,37.92259700000011],[-105.36540999999988,37.92142000000024],[-105.36782900000003,37.91978400000022],[-105.37500299999994,37.917115000000194],[-105.37708799999996,37.91610200000002],[-105.37839600000001,37.91379999999998],[-105.38056399999988,37.91332499999993],[-105.38398399999988,37.910469999999975],[-105.38503999999995,37.91017799999997],[-105.38962799999996,37.906521999999995],[-105.39030500000001,37.905196000000046],[-105.39407999999997,37.90291800000023],[-105.394792,37.90136600000005],[-105.399585,37.898816000000295],[-105.40342099999998,37.895782000000054],[-105.40508699999998,37.89515],[-105.40869699999996,37.894442000000026],[-105.40978399999995,37.893205000000194],[-105.41155300000003,37.8925320000003],[-105.413635,37.890527000000304],[-105.41548599999993,37.890814999999975],[-105.41868899999992,37.88970300000017],[-105.42159700000002,37.89065199999999],[-105.42415099999994,37.89017299999995],[-105.42685499999993,37.89163400000024],[-105.42779000000002,37.89132000000001],[-105.4298859999999,37.893837000000076],[-105.43197199999992,37.895167000000015],[-105.43727899999999,37.896011000000044],[-105.440272,37.89746100000008],[-105.45236,37.89568100000025],[-105.45923500000004,37.897169000000076],[-105.461319,37.89689099999998],[-105.46696999999995,37.89551499999999],[-105.47105699999997,37.895264000000054],[-105.4732039999999,37.89596999999998],[-105.47330699999998,37.896649000000025],[-105.47605599999997,37.89938900000027],[-105.47708299999994,37.899298000000044],[-105.47887499999996,37.90050500000024],[-105.47899999999993,37.902629000000104],[-105.47835099999998,37.90334899999999],[-105.48260399999987,37.90500399999996],[-105.48814500000003,37.90984500000013],[-105.4911449999999,37.915225000000305],[-105.49410899999992,37.917100000000005],[-105.49719699999991,37.920339000000126],[-105.50227299999978,37.922950000000014],[-105.50311599999998,37.92372500000016],[-105.50489299999992,37.92721300000028],[-105.50568699999991,37.927891000000045],[-105.50704200000001,37.93101999999999],[-105.50836399999997,37.936094000000026],[-105.51108599999998,37.939131000000145],[-105.514591,37.939930000000174],[-105.51572899999991,37.942015000000026],[-105.523572,37.948037],[-105.525534,37.94837000000007],[-105.52714699999996,37.94953099999998],[-105.53037799999987,37.95336299999997],[-105.53331499999996,37.95518200000021],[-105.53770299999985,37.95503600000001],[-105.54194899999987,37.95604300000002],[-105.54273699999999,37.95650999999998],[-105.54619099999985,37.95604800000012],[-105.55010099999998,37.95387100000028],[-105.551826,37.950584000000276],[-105.55635699999999,37.950980000000015],[-105.56031100000001,37.95188600000017],[-105.56235099999998,37.953744000000256],[-105.56719199999986,37.95714500000008],[-105.57069200000001,37.958265999999924],[-105.5732809999999,37.96158200000002],[-105.573982,37.96175800000003],[-105.57561999999979,37.96398399999998],[-105.577676,37.965535999999986],[-105.58128599999992,37.96618000000001],[-105.58261599999992,37.96593400000023],[-105.58446199999997,37.96785500000004],[-105.58415799999995,37.96908700000017],[-105.58505400000001,37.970451000000025],[-105.58396399999998,37.973008999999934],[-105.5815219999999,37.975224999999966],[-105.58185500000002,37.97797200000008],[-105.58319099999989,37.97997800000013],[-105.58590999999996,37.97968700000024],[-105.58715999999993,37.98450900000023],[-105.58815599999991,37.986689000000126],[-105.58989899999989,37.98743100000024],[-105.59207099999992,37.99214800000027],[-105.59366299999982,37.99277500000028],[-105.59573899999992,37.995069],[-105.59589199999988,37.996370000000184],[-105.598413,37.99702100000002],[-105.60024399999998,37.99999900000006],[-105.600596,38.002274],[-105.60390699999994,38.00373900000005],[-105.60463600000003,38.005014000000074],[-105.60486799999995,38.00738799999999],[-105.60435499999994,38.008745000000204],[-105.59887599999985,38.01124700000008],[-105.59727299999992,38.012664000000086],[-105.59711199999992,38.013638000000014],[-105.59832499999993,38.01633800000013],[-105.59795199999996,38.02001900000005],[-105.59869799999996,38.02264900000006],[-105.60080499999987,38.023746000000244],[-105.60311399999995,38.02714400000002],[-105.60352699999993,38.03126299999997],[-105.6065129999999,38.03160400000007],[-105.60937999999999,38.032338000000095],[-105.6085609999999,38.033708000000274],[-105.60885399999978,38.03495300000026],[-105.613246,38.03808500000008],[-105.61400300000003,38.03953700000011],[-105.614957,38.04366199999998],[-105.61720600000001,38.04592900000017],[-105.617996,38.04726100000005],[-105.61756399999996,38.04977300000007],[-105.61780699999991,38.05326000000014],[-105.62556299999994,38.05569299999996],[-105.62820599999998,38.05742099999998],[-105.62674900000002,38.059149000000104],[-105.6259619999999,38.06116100000014],[-105.62626299999994,38.06353300000029],[-105.62692799999991,38.064592000000175],[-105.627206,38.067309999999964],[-105.63139699999982,38.06811499999998],[-105.63415900000001,38.069439999999986],[-105.63727599999999,38.07279699999992],[-105.636484,38.07558400000016],[-105.63936899999999,38.07848100000007],[-105.64124099999987,38.078883000000076],[-105.64528399999989,38.08240400000011],[-105.64644900000002,38.08503700000017],[-105.650105,38.08695300000005],[-105.65196499999996,38.08939700000013],[-105.65364899999997,38.090880000000084],[-105.65529599999991,38.09134300000005],[-105.65585599999997,38.09229099999993],[-105.65509399999996,38.09358700000007],[-105.65557899999988,38.095490000000154],[-105.65882599999992,38.097276000000306],[-105.66030199999994,38.09886899999992],[-105.66147999999998,38.10091200000005],[-105.66146599999996,38.10313600000006],[-105.66222099999999,38.10434800000013],[-105.66148499999986,38.10638300000011],[-105.66224599999993,38.106845000000135],[-105.66122899999999,38.109781999999996],[-105.66133999999994,38.11149800000021],[-105.65983599999993,38.11242800000008],[-105.6599129999999,38.11324900000011],[-105.65849400000002,38.11518500000011],[-105.6576629999999,38.11745400000024],[-105.660145,38.11806800000005],[-105.66265399999986,38.1191580000002],[-105.6632899999999,38.12154499999997],[-105.66501399999999,38.12428300000022],[-105.66644100000002,38.12585200000018],[-105.67021499999998,38.12855400000012],[-105.66968099999997,38.13074100000023],[-105.67205200000001,38.13406700000007],[-105.67339500000003,38.137396000000194],[-105.67254699999995,38.140220000000056],[-105.6743219999999,38.14133800000002],[-105.67626999999987,38.14617300000003],[-105.67306599999989,38.14797299999992],[-105.6726129999999,38.14936200000011],[-105.67105900000001,38.15107700000027],[-105.67016899999993,38.15309700000006],[-105.67134299999998,38.15385200000003],[-105.67284099999995,38.156777999999974],[-105.67304199999995,38.15912600000013],[-105.67744700000003,38.15952599999997],[-105.68078199999991,38.16050200000012],[-105.6861599999998,38.165306000000214],[-105.68683799999997,38.166845000000194],[-105.69133199999999,38.166494000000284],[-105.69321500000001,38.16573800000026],[-105.69798000000003,38.16720200000003],[-105.70041399999997,38.1691800000001],[-105.70111699999995,38.17197900000025],[-105.70219499999996,38.174203000000034],[-105.705374,38.176438000000076],[-105.70684799999992,38.178152000000296],[-105.70808,38.180352000000255],[-105.70830999999993,38.18186000000014],[-105.71292399999987,38.183235000000025],[-105.716001,38.185210999999924],[-105.71733799999987,38.1881370000001],[-105.71883300000002,38.18983500000007],[-105.7212659999999,38.19155500000011],[-105.72204899999991,38.19344899999999],[-105.72675300000003,38.19947200000007],[-105.73107299999998,38.200911000000076],[-105.7325909999999,38.20067999999998],[-105.73333000000002,38.20174300000019],[-105.7342099999999,38.205057000000295],[-105.73580999999996,38.20720700000004],[-105.73631799999998,38.20923100000016],[-105.74069800000001,38.210283000000004],[-105.74442499999992,38.213701000000015],[-105.74480599999998,38.21460300000001],[-105.74850699999996,38.2141400000001],[-105.750563,38.21421300000026],[-105.75362899999993,38.21667500000001],[-105.75602600000002,38.218127000000266],[-105.756482,38.218975000000114],[-105.75588599999998,38.222030000000075],[-105.75653899999992,38.22496000000007],[-105.75949000000003,38.2284130000001],[-105.76560499999988,38.23080300000015],[-105.76720699999993,38.231815999999924],[-105.76823200000001,38.233763000000124],[-105.77106000000003,38.23483600000009],[-105.7754539999998,38.23584900000003],[-105.780933,38.23914000000019],[-105.783928,38.23963300000031],[-105.78788700000001,38.24205600000005],[-105.789761,38.244360000000086],[-105.78921099999997,38.24627799999996],[-105.78914299999991,38.251824],[-105.79218000000003,38.254815000000235],[-105.79274399999997,38.256294000000025],[-105.79566499999999,38.26013900000021],[-105.79689699999977,38.26504700000004],[-105.76555199999996,38.26500800000019],[-105.76294999999976,38.26489200000003],[-105.750562,38.26501300000007],[-105.73743099999996,38.264825000000144],[-105.72130300000003,38.265034000000014],[-105.68741799999998,38.26496700000018],[-105.68245499999995,38.26483200000001],[-105.665773,38.26482299999998],[-105.65608899999995,38.265079000000014],[-105.65203200000002,38.26497999999998],[-105.64053899999988,38.26519100000007],[-105.62490099999997,38.264850999999965],[-105.59194699999995,38.264905],[-105.56901699999986,38.264377000000025],[-105.55623400000002,38.26403200000027],[-105.54757699999988,38.26429400000006],[-105.53696300000001,38.263369000000296],[-105.50573199999991,38.26278400000001],[-105.50055500000002,38.26278400000001],[-105.49963300000002,38.262457000000154],[-105.49292700000001,38.26246099999997],[-105.47820899999994,38.26213699999994],[-105.46719399999989,38.26199100000014],[-105.464267,38.26184400000005],[-105.42730699999993,38.26102000000009],[-105.40978099999995,38.26034100000004],[-105.39185399999991,38.25975500000004],[-105.34118999999998,38.258973000000196],[-105.3289309999999,38.2589260000002],[-105.30404499999997,38.258225000000095],[-105.29182999999995,38.258351999999945],[-105.26291400000002,38.25904800000018],[-105.24542799999995,38.25896999999992],[-105.21484399999986,38.25860300000011],[-105.15413999999981,38.25788300000005],[-105.12554399999999,38.25754800000004],[-105.12327999999997,38.25768500000004],[-105.10713299999998,38.25769300000013],[-105.07547199999999,38.257829000000015],[-105.07445999999993,38.258059999999944],[-105.04921499999989,38.25797300000022]]]},"properties":{"OBJECTID":15,"COUNTYFP":"027","Shape_Leng":2.36449541032,"Shape_Area":0.196859175327,"low":1430,"lowmod":2420,"LMMI":3225,"lowmoduniv":4280,"lowmod_pct":0.565420560748,"NAMELSAD10":"Custer County"}},
{"type":"Feature","geometry":{"type":"Polygon","coordinates":[[[-107.500606,39.217916],[-107.50063699999993,39.169308],[-107.50058599999988,39.155143000000066],[-107.50067999999993,39.12169400000005],[-107.50061499999987,39.064982999999984],[-107.500607,39.0101590000001],[-107.50063199999994,38.95639100000005],[-107.50063899999998,38.91343100000017],[-107.50055299999997,38.89911600000022],[-107.50062400000002,38.86979500000001],[-107.50059099999993,38.841075000000274],[-107.50063799999981,38.81245000000024],[-107.50060199999996,38.78349700000007],[-107.50063399999993,38.74806000000024],[-107.50061099999999,38.69829599999997],[-107.500655,38.66855300000003],[-107.52087099999994,38.66856799999999],[-107.53608499999996,38.66851800000012],[-107.61010499999998,38.66843899999998],[-107.66082299999994,38.668378000000075],[-107.72244599999988,38.66828000000015],[-107.74917700000003,38.66826000000009],[-107.75119199999995,38.66805100000022],[-107.77811500000001,38.66798],[-107.83378499999992,38.66810399999997],[-107.85308299999997,38.668165999999985],[-107.89265,38.66814400000027],[-107.92980799999998,38.668304000000035],[-107.99888899999996,38.668476000000055],[-108.00402400000002,38.6684020000003],[-108.05960799999997,38.66839199999998],[-108.07733999999994,38.66848500000009],[-108.11350599999997,38.66852900000015],[-108.13296400000002,38.66839900000008],[-108.15012899999999,38.668510000000026],[-108.18716199999989,38.668442000000084],[-108.26107099999996,38.66842100000008],[-108.30350399999998,38.66831400000012],[-108.37627499999996,38.66812200000004],[-108.37869799999999,38.6680970000001],[-108.37875999999994,38.692616000000044],[-108.37880699999994,38.734037000000285],[-108.3787099999999,38.754647999999975],[-108.37876799999998,38.765274999999974],[-108.37899900000002,38.77661000000012],[-108.37890400000003,38.79805399999998],[-108.37884799999995,38.823026000000084],[-108.37895300000002,38.829108000000076],[-108.37621999999993,38.83017400000017],[-108.33432299999993,38.84374700000001],[-108.32081299999987,38.848239999999976],[-108.28628100000003,38.859587000000204],[-108.25135699999993,38.8711770000001],[-108.23610699999978,38.875906999999984],[-108.21042299999999,38.884184000000005],[-108.18309699999998,38.893238999999994],[-108.176988,38.89531499999998],[-108.176152,38.89727499999998],[-108.17420900000002,38.8981560000002],[-108.17052399999983,38.89831400000014],[-108.16861799999987,38.900750000000244],[-108.16630999999995,38.90224500000011],[-108.16322599999995,38.90213],[-108.15942000000001,38.901053000000275],[-108.15548499999994,38.90148400000004],[-108.15247799999992,38.9012680000003],[-108.15069499999993,38.90266800000012],[-108.14909399999988,38.90556600000008],[-108.146953,38.90747100000016],[-108.14477399999993,38.90747600000003],[-108.14176299999986,38.90637700000025],[-108.14072299999987,38.906846000000144],[-108.14032299999997,38.908767000000125],[-108.1418109999999,38.91192500000011],[-108.14420199999995,38.91488700000002],[-108.144834,38.91640199999995],[-108.14439799999991,38.91853000000003],[-108.14327699999996,38.91952500000008],[-108.14061599999997,38.92062299999998],[-108.14028299999995,38.92146100000002],[-108.14092199999999,38.92417300000011],[-108.14048099999991,38.92515800000024],[-108.13774099999995,38.92605200000014],[-108.1333939999999,38.92396200000002],[-108.13259700000003,38.92290200000008],[-108.13065699999993,38.92230500000005],[-108.12741399999993,38.923834000000284],[-108.12557899999996,38.92405400000018],[-108.12079299999994,38.92720900000029],[-108.11894599999977,38.92902400000003],[-108.11753599999992,38.9318550000001],[-108.113268,38.93578800000029],[-108.11078499999991,38.93755700000008],[-108.10797099999996,38.941010000000006],[-108.10307699999993,38.947806999999955],[-108.10227099999997,38.94918000000007],[-108.09763999999996,38.953564000000256],[-108.09546699999993,38.95453200000003],[-108.09556399999997,38.95676300000008],[-108.09344899999985,38.957458999999915],[-108.0914659999998,38.95936700000004],[-108.08977199999998,38.96020499999997],[-108.08876399999997,38.96223199999997],[-108.08730499999996,38.96318000000025],[-108.08752600000003,38.964814000000274],[-108.08686,38.96780400000023],[-108.088574,38.970074000000125],[-108.09037199999989,38.971612000000164],[-108.09110099999998,38.974312999999995],[-108.09204799999992,38.97618000000011],[-108.09478599999989,38.979684000000134],[-108.09343999999999,38.98058400000002],[-108.08981599999998,38.98113100000012],[-108.08691899999991,38.98298200000016],[-108.08565199999998,38.985160000000064],[-108.08288199999993,38.98728700000021],[-108.08173899999997,38.989408000000196],[-108.07971199999992,38.99112800000023],[-108.07616399999989,38.99261800000022],[-108.07299,38.99503500000003],[-108.06996300000003,38.99659200000008],[-108.06726999999995,38.99998699999992],[-108.06626899999992,39.00085100000001],[-108.06450599999988,39.004149999999925],[-108.06142699999992,39.00530300000031],[-108.05965199999997,39.00671800000015],[-108.05597299999988,39.010572000000025],[-108.05377899999996,39.01405200000016],[-108.05036099999995,39.015133000000105],[-108.04949799999997,39.014651000000015],[-108.04780099999988,39.01145199999996],[-108.04679599999997,39.01224400000012],[-108.04445699999997,39.01209800000015],[-108.04332099999993,39.013866000000235],[-108.038297,39.0157270000002],[-108.03611899999981,39.01718000000017],[-108.03580299999999,39.02060300000005],[-108.034179,39.023393],[-108.03294999999991,39.0246410000002],[-108.03200399999997,39.02736800000025],[-108.030127,39.02884000000017],[-108.02808499999998,39.029704000000265],[-108.02575399999984,39.029765],[-108.02302999999989,39.03088900000006],[-108.02070599999996,39.0332650000002],[-108.01936499999994,39.03351600000019],[-108.01592999999986,39.03611799999999],[-108.01311699999991,39.03734400000019],[-108.01227699999987,39.03823100000022],[-108.00948800000003,39.039743999999985],[-108.004278,39.04191500000002],[-108.00230099999982,39.04323600000026],[-108.00063799999998,39.04544100000027],[-107.99516699999992,39.048563000000115],[-107.991019,39.05046299999992],[-107.98720400000002,39.05287599999991],[-107.98382600000002,39.05824999999999],[-107.98217299999993,39.0598780000002],[-107.97261500000002,39.06121200000007],[-107.96429999999987,39.05965900000001],[-107.95922899999994,39.06025600000021],[-107.95684199999988,39.059821000000284],[-107.95122100000003,39.059702000000016],[-107.949501,39.06030400000009],[-107.94179299999996,39.06084200000021],[-107.93687499999999,39.06206500000013],[-107.93320199999994,39.06209400000023],[-107.92732899999993,39.062593000000106],[-107.92488099999997,39.06317700000011],[-107.91669200000001,39.063833000000045],[-107.91340400000001,39.06256400000001],[-107.91167199999995,39.06259799999998],[-107.90954699999998,39.06151599999998],[-107.90904399999994,39.06054699999993],[-107.906811,39.060729000000265],[-107.90352999999988,39.06299899999999],[-107.90066399999989,39.065521000000274],[-107.89729499999993,39.06621800000005],[-107.89313999999996,39.065798000000086],[-107.88817799999993,39.06681300000008],[-107.88621599999993,39.068285000000174],[-107.88308899999993,39.06956100000008],[-107.88017000000002,39.071101999999996],[-107.87562600000001,39.07076400000011],[-107.87481399999996,39.072387000000106],[-107.87325199999987,39.0734800000003],[-107.86998199999994,39.074929999999995],[-107.86905599999994,39.0743130000003],[-107.86707099999995,39.074677000000065],[-107.86647499999992,39.07623200000012],[-107.86523099999994,39.07661400000006],[-107.862618,39.07887600000004],[-107.86185499999993,39.07772400000027],[-107.85950000000003,39.07981800000016],[-107.85720600000002,39.07938800000028],[-107.85640399999994,39.07726300000013],[-107.85532699999999,39.07665000000003],[-107.85335999999995,39.078310999999985],[-107.85210799999993,39.07852200000002],[-107.84890199999995,39.077825000000075],[-107.84734300000002,39.078143000000296],[-107.84589900000003,39.077313000000004],[-107.841793,39.07666599999993],[-107.84001,39.077168000000086],[-107.83553499999994,39.075368000000196],[-107.83212599999996,39.07336000000015],[-107.82973500000003,39.07283300000006],[-107.82815600000004,39.071762999999976],[-107.82583699999998,39.071576000000164],[-107.82243900000003,39.07042400000023],[-107.81933299999992,39.07078799999999],[-107.81897500000002,39.0696250000002],[-107.81346099999996,39.06726299999997],[-107.80904999999996,39.06648700000011],[-107.80671199999995,39.06551600000006],[-107.80212999999992,39.06478800000019],[-107.80133299999989,39.06361800000002],[-107.79745200000002,39.06280200000003],[-107.79374699999988,39.06144900000015],[-107.79227700000001,39.06051099999996],[-107.78987699999976,39.05795599999999],[-107.78894400000001,39.05627600000008],[-107.78634099999994,39.056194000000005],[-107.78337899999991,39.053733000000136],[-107.78009699999996,39.05254999999994],[-107.77621599999992,39.04785900000013],[-107.77396799999991,39.047084000000154],[-107.77108499999986,39.043693000000076],[-107.76721499999991,39.04398300000025],[-107.76549,39.04329400000006],[-107.76505899999978,39.04468300000008],[-107.76304399999987,39.04722599999997],[-107.75869099999994,39.047879000000194],[-107.75782199999998,39.049314000000265],[-107.75607699999983,39.04989500000005],[-107.75640099999993,39.05229300000019],[-107.75560899999994,39.05389800000012],[-107.75165299999992,39.05439600000011],[-107.75059799999991,39.053989],[-107.74739099999988,39.055929000000106],[-107.74556299999989,39.057648000000086],[-107.74388999999985,39.05770600000005],[-107.74248599999993,39.05934200000007],[-107.74488099999996,39.05972700000012],[-107.74439899999993,39.062189000000274],[-107.74188099999998,39.064023000000134],[-107.73798799999992,39.066160000000025],[-107.73700399999996,39.06551600000006],[-107.7350229999999,39.0662750000003],[-107.73296900000003,39.063052000000084],[-107.732216,39.063007000000084],[-107.73075499999999,39.06588800000026],[-107.729149,39.06619600000016],[-107.72305499999999,39.065184999999985],[-107.7207899999998,39.0700040000001],[-107.718684,39.07173800000004],[-107.71756499999987,39.073899000000154],[-107.716273,39.074281000000155],[-107.71304699999996,39.07906800000012],[-107.71326699999997,39.08027900000013],[-107.71084199999996,39.081254000000115],[-107.71020999999979,39.08224200000018],[-107.70778000000001,39.08369499999998],[-107.70387599999992,39.084993000000054],[-107.70000699999991,39.08385700000002],[-107.695694,39.085038000000225],[-107.69411400000001,39.08513800000014],[-107.69411100000002,39.08688599999999],[-107.69284199999993,39.08757300000019],[-107.68955599999993,39.087171000000126],[-107.68688299999991,39.08862899999997],[-107.68529399999989,39.090918000000045],[-107.68294499999985,39.09270200000003],[-107.68081499999994,39.100256000000286],[-107.67634499999991,39.1003850000003],[-107.67383999999987,39.101091999999994],[-107.67277999999999,39.10546700000015],[-107.67143899999996,39.107633000000305],[-107.66985799999998,39.10862400000008],[-107.66460399999994,39.1094470000001],[-107.66030899999993,39.1117710000002],[-107.66236699999996,39.114823000000115],[-107.66267399999987,39.116239000000235],[-107.66173399999997,39.11767500000002],[-107.65844899999996,39.11982300000011],[-107.6566029999999,39.12269399999997],[-107.65376000000003,39.12400200000002],[-107.65362499999998,39.12656300000003],[-107.6513139999999,39.13093099999992],[-107.64820099999997,39.13121800000005],[-107.64453699999996,39.13559900000001],[-107.64140199999991,39.138094000000194],[-107.64004499999999,39.140458000000024],[-107.63789300000002,39.142020000000116],[-107.634726,39.14085699999998],[-107.63306699999987,39.140715],[-107.6318,39.141797],[-107.63081199999999,39.14384699999994],[-107.63010300000002,39.1465750000001],[-107.62784499999998,39.14900399999999],[-107.62624399999993,39.149184000000105],[-107.62455699999998,39.150204000000144],[-107.6212789999999,39.15103500000015],[-107.62012899999996,39.152169000000185],[-107.61994800000002,39.15384000000006],[-107.61910799999998,39.15488600000009],[-107.61694499999987,39.155529],[-107.61467499999998,39.15453500000001],[-107.61105599999996,39.15649900000017],[-107.60976699999998,39.15887900000007],[-107.60745999999995,39.16157100000004],[-107.605325,39.16206],[-107.60102699999999,39.16412000000014],[-107.59805599999993,39.16422200000005],[-107.59585599999997,39.16480300000012],[-107.59387700000002,39.164820000000304],[-107.59281599999997,39.16585299999997],[-107.58895999999987,39.16833500000001],[-107.5884559999999,39.169112000000155],[-107.58473499999997,39.17178899999993],[-107.58362599999987,39.17373300000003],[-107.58128099999999,39.174504999999954],[-107.57702499999994,39.1748970000001],[-107.572992,39.176347000000135],[-107.570491,39.17823100000004],[-107.56983100000002,39.180146000000036],[-107.56742300000002,39.181397000000004],[-107.56389399999989,39.18016],[-107.56082099999998,39.17842600000006],[-107.55692899999985,39.17834000000005],[-107.55215099999992,39.179688],[-107.54895099999993,39.18116400000008],[-107.54336399999994,39.18192499999998],[-107.54235099999988,39.18272200000001],[-107.54171199999996,39.184565000000134],[-107.54339700000003,39.18541099999993],[-107.54377699999992,39.18693500000029],[-107.54287599999998,39.190522000000044],[-107.54345499999994,39.1944010000002],[-107.54157599999996,39.19623500000006],[-107.53758399999998,39.19789900000006],[-107.53482099999991,39.19867099999999],[-107.53116599999993,39.19910200000015],[-107.52642900000001,39.198717999999985],[-107.524768,39.198916000000054],[-107.52607099999989,39.200708000000134],[-107.528348,39.20188400000029],[-107.52883799999995,39.202936000000136],[-107.52750499999996,39.203884000000016],[-107.52448900000002,39.20468799999992],[-107.52367299999997,39.205516999999986],[-107.52067799999998,39.20628400000004],[-107.51664199999999,39.210327000000234],[-107.51383199999992,39.214],[-107.51179499999995,39.21567500000003],[-107.50983499999978,39.21655500000003],[-107.50729599999994,39.21682600000025],[-107.50640299999998,39.21645799999999],[-107.5047899999999,39.21733900000004],[-107.50240600000001,39.21739999999994],[-107.500606,39.217916]]]},"properties":{"OBJECTID":16,"COUNTYFP":"029","Shape_Leng":2.74901422563,"Shape_Area":0.30874622014,"low":8365,"lowmod":13970,"LMMI":19860,"lowmoduniv":29100,"lowmod_pct":0.480068728522,"NAMELSAD10":"Delta County"}},
{"type":"Feature","geometry":{"type":"MultiPolygon","coordinates":[[[[-104.93389699999994,39.698481000000015],[-104.93413499999991,39.70018100000016],[-104.93171399999989,39.70020800000009],[-104.93165099999993,39.698398000000054],[-104.93389699999994,39.698481000000015]]],[[[-104.88464599999998,39.7401560000003],[-104.88473499999986,39.73083100000002],[-104.88661300000001,39.72921900000006],[-104.88502599999987,39.72813700000006],[-104.881934,39.724960000000124],[-104.87973499999998,39.72562300000004],[-104.8753299999999,39.725576000000274],[-104.87534199999993,39.71894400000025],[-104.87404100000003,39.71817799999991],[-104.8708309999999,39.718175999999914],[-104.870653,39.71016900000001],[-104.86597099999994,39.71105299999999],[-104.86596699999996,39.704317999999944],[-104.86893199999997,39.70298699999995],[-104.869393,39.70108900000031],[-104.87064800000002,39.700535000000116],[-104.87066199999975,39.69847500000003],[-104.8683009999998,39.69853100000023],[-104.8659639999999,39.699086000000136],[-104.86596199999991,39.69655899999998],[-104.87530899999985,39.696505000000286],[-104.875316,39.69799599999999],[-104.87769800000001,39.69798800000024],[-104.877611,39.696506],[-104.89678200000003,39.696587000000136],[-104.89880599999987,39.69509099999999],[-104.900304,39.696254000000295],[-104.8992849999998,39.69697600000018],[-104.90354399999995,39.69853500000005],[-104.90350699999999,39.69662199999999],[-104.90011499999991,39.69298500000002],[-104.90351099999998,39.69297],[-104.90351199999998,39.68935399999998],[-104.89881000000003,39.6893410000003],[-104.89876099999998,39.69299200000023],[-104.894092,39.692874000000074],[-104.89405799999997,39.68569400000001],[-104.88932799999992,39.68568400000004],[-104.88930800000003,39.6820560000001],[-104.89404599999995,39.682060999999976],[-104.89405799999997,39.678448],[-104.90343999999988,39.67859800000031],[-104.903888,39.68033200000025],[-104.90698199999991,39.679737000000046],[-104.90698399999991,39.67879599999998],[-104.90478199999995,39.67781800000006],[-104.90357899999992,39.67667499999999],[-104.90350899999999,39.66762400000022],[-104.89614,39.66761700000018],[-104.89394600000003,39.66746400000011],[-104.89018799999985,39.66439300000002],[-104.8880329999999,39.66578700000008],[-104.88963299999983,39.66658700000005],[-104.89013299999999,39.66758699999997],[-104.88413299999996,39.66748700000005],[-104.879433,39.66388700000027],[-104.87623300000001,39.66398700000002],[-104.87583199999978,39.66978699999993],[-104.87823300000002,39.6727870000002],[-104.87531000000001,39.67278399999992],[-104.87552700000003,39.67476399999998],[-104.87088199999994,39.67476399999998],[-104.8660509999998,39.670847000000094],[-104.86633399999977,39.660308999999984],[-104.85644099999996,39.66034000000025],[-104.85543699999994,39.66080400000027],[-104.85306699999984,39.659914000000185],[-104.84833200000003,39.65916100000004],[-104.84773299999995,39.656788000000006],[-104.85294399999998,39.65570600000001],[-104.85603800000001,39.65417800000023],[-104.85675300000003,39.653456000000006],[-104.87622399999998,39.6443010000001],[-104.876306,39.65229500000021],[-104.87749699999989,39.653052000000116],[-104.88013999999998,39.65305699999999],[-104.8803289999999,39.641491000000144],[-104.88511199999988,39.63926200000009],[-104.885265,39.62408300000004],[-104.89471900000001,39.62402800000012],[-104.90086999999994,39.62372299999993],[-104.906969,39.62407200000007],[-104.91340699999995,39.62408300000004],[-104.91325699999999,39.63869100000028],[-104.92246,39.64584900000011],[-104.931873,39.64582600000017],[-104.93170299999997,39.65304400000002],[-104.94540099999995,39.6529680000001],[-104.95590299999998,39.65299800000008],[-104.95495199999988,39.655797000000234],[-104.95378800000003,39.65668299999999],[-104.95948499999997,39.65669100000031],[-104.95944299999996,39.66152600000015],[-104.96314899999999,39.66156000000029],[-104.96322399999997,39.66037000000006],[-104.96878500000003,39.66041700000022],[-104.96880499999997,39.66583900000012],[-104.97348899999986,39.66586600000005],[-104.97348599999998,39.66768400000001],[-104.992299,39.667679000000135],[-104.9999059999999,39.66788400000007],[-105.00437399999998,39.66630600000008],[-105.00486199999995,39.664462999999955],[-105.00449299999997,39.6607860000002],[-105.00488200000001,39.660455000000184],[-105.01066499999996,39.66050700000005],[-105.00771700000001,39.664220000000114],[-105.00647100000003,39.66691300000002],[-105.00663700000001,39.667986000000155],[-105.00642099999999,39.67318],[-105.00657599999994,39.67677800000018],[-105.00863700000002,39.67678000000001],[-105.00866399999995,39.67862700000023],[-105.01569599999993,39.678684000000146],[-105.01580000000001,39.66050800000011],[-105.02298199999996,39.660526000000004],[-105.02300300000002,39.65683000000007],[-105.03083800000002,39.657187000000135],[-105.032038,39.65708700000022],[-105.03216199999997,39.653276000000005],[-105.03401699999995,39.65346500000021],[-105.03463099999988,39.65142600000024],[-105.03465899999998,39.64399100000003],[-105.03123799999997,39.643987000000095],[-105.031338,39.64308700000021],[-105.02997899999991,39.64224500000017],[-105.02999799999998,39.63857500000029],[-105.03470700000003,39.638593000000014],[-105.03475200000003,39.63359000000008],[-105.03005699999977,39.63349199999999],[-105.02956299999994,39.632355000000246],[-105.02535999999981,39.63144500000027],[-105.02536899999996,39.62859800000007],[-105.02912700000002,39.627838000000054],[-105.03680199999991,39.62812900000017],[-105.038005,39.62683600000014],[-105.040618,39.628272000000095],[-105.04404799999992,39.62801600000023],[-105.04433899999992,39.63158699999991],[-105.05341099999998,39.63158600000003],[-105.0533329999999,39.63889300000011],[-105.05899099999999,39.638886999999954],[-105.0594569999999,39.63466200000016],[-105.05700899999994,39.63167900000008],[-105.05345699999987,39.62952600000011],[-105.05333000000002,39.62798900000013],[-105.05583599999989,39.625994000000105],[-105.05583999999993,39.624193000000105],[-105.05343899999997,39.62438700000018],[-105.05343899999997,39.62142300000005],[-105.05715099999998,39.62158500000015],[-105.05643899999995,39.619932000000006],[-105.05819300000002,39.61736600000012],[-105.05812500000002,39.6144300000002],[-105.06419900000003,39.61457400000006],[-105.06992499999996,39.61442400000004],[-105.07292399999994,39.61453899999998],[-105.09690899999993,39.61442300000016],[-105.09695999999997,39.618362000000104],[-105.08725199999998,39.618317000000104],[-105.08592799999997,39.618668000000014],[-105.07939799999986,39.61847300000005],[-105.06514099999998,39.618582],[-105.06496599999991,39.620238000000256],[-105.07595200000003,39.62414300000023],[-105.07623999999987,39.625587000000166],[-105.08145000000002,39.631858000000136],[-105.08371199999993,39.63132700000028],[-105.08373499999988,39.630155000000286],[-105.08169299999986,39.63014900000013],[-105.08173,39.62731299999996],[-105.08381300000002,39.62731800000006],[-105.08378899999991,39.628527000000076],[-105.08557999999994,39.628527000000076],[-105.08562599999993,39.62610700000005],[-105.08738399999999,39.62709000000018],[-105.09116399999994,39.627081000000146],[-105.09125599999999,39.62419799999998],[-105.10860000000002,39.624208000000294],[-105.10858999999999,39.626986999999986],[-105.10992699999986,39.62698800000027],[-105.10987399999999,39.635185999999976],[-105.09852699999993,39.63507500000003],[-105.09158199999996,39.635178000000224],[-105.09157699999992,39.63872900000024],[-105.08167599999996,39.63875900000022],[-105.08156500000001,39.63616600000006],[-105.07981799999988,39.636688000000106],[-105.07695399999989,39.63638999999995],[-105.073308,39.63650300000006],[-105.07327099999998,39.6387400000001],[-105.07680299999998,39.638735],[-105.07575999999995,39.64205000000021],[-105.07445399999989,39.64410399999997],[-105.07232599999992,39.645350000000235],[-105.0720419999999,39.646090000000015],[-105.06353899999988,39.6459880000001],[-105.063399,39.65008000000006],[-105.05734499999994,39.6499840000003],[-105.05721499999993,39.65294600000004],[-105.05854299999993,39.65297900000013],[-105.05883900000003,39.65138700000023],[-105.06213699999995,39.65138399999995],[-105.06392399999993,39.64956599999999],[-105.07102900000001,39.64957900000019],[-105.06694099999999,39.653594000000226],[-105.06704499999995,39.65658700000023],[-105.07072699999998,39.6565940000001],[-105.07079299999992,39.659170000000074],[-105.07161699999989,39.659994999999924],[-105.07764199999997,39.65877900000004],[-105.07733799999994,39.660389000000066],[-105.07982200000004,39.66216400000019],[-105.0814539999999,39.66207400000019],[-105.08134999999999,39.67009899999999],[-105.07693899999987,39.6696860000003],[-105.07493999999997,39.67068700000016],[-105.07220399999989,39.669521000000145],[-105.07217899999995,39.66776299999998],[-105.05325600000003,39.66778500000021],[-105.05324999999988,39.674988999999925],[-105.06289800000002,39.67498699999999],[-105.062907,39.67922300000015],[-105.06073199999997,39.6782790000002],[-105.05817499999995,39.67866600000025],[-105.05804399999988,39.68227000000002],[-105.05322099999995,39.682290999999964],[-105.05319400000002,39.69136400000002],[-105.05327899999997,39.718233000000055],[-105.05322699999982,39.733110000000124],[-105.0555389999999,39.73309800000004],[-105.055567,39.73497700000024],[-105.05322799999999,39.73496499999993],[-105.05324300000001,39.78219000000007],[-105.05318299999993,39.78379699999999],[-105.06265899999988,39.78384700000004],[-105.06294400000002,39.788087000000075],[-105.0650829999999,39.788086000000135],[-105.06511499999999,39.78968000000003],[-105.06189899999998,39.79008299999998],[-105.06147999999996,39.791075000000035],[-105.05324599999989,39.791063000000065],[-105.03461799999985,39.791142000000036],[-105.0346419999999,39.791870000000245],[-105.02998199999996,39.79187999999999],[-105.03003799999999,39.79108600000001],[-105.02518999999995,39.790933000000166],[-105.02514899999994,39.794002999999975],[-105.02083700000003,39.79278599999998],[-105.02078699999998,39.791089000000284],[-104.99998600000004,39.79103500000025],[-104.99343599999986,39.79118599999998],[-104.99333999999999,39.79208199999999],[-104.99080900000001,39.792139000000134],[-104.990434,39.79108300000013],[-104.97595000000001,39.79103800000013],[-104.97605699999997,39.79230100000001],[-104.97323499999987,39.79247000000015],[-104.97324500000002,39.79413100000011],[-104.97483799999992,39.79422599999998],[-104.97484699999995,39.79561000000007],[-104.97324300000002,39.79630199999997],[-104.973143,39.79829200000029],[-104.9687159999998,39.798396000000196],[-104.96876499999996,39.79383999999999],[-104.966611,39.793836],[-104.96775799999995,39.791721000000166],[-104.968861,39.790974000000176],[-104.96026799999999,39.790966000000026],[-104.955356,39.79468600000018],[-104.94047599999993,39.79460100000006],[-104.94048699999996,39.79100199999999],[-104.92703399999999,39.79105400000003],[-104.92700200000002,39.78377800000021],[-104.90343799999988,39.78380800000002],[-104.90356099999985,39.79839099999998],[-104.90045199999992,39.798377000000016],[-104.900394,39.80113500000016],[-104.89693299999999,39.801158999999984],[-104.89690499999989,39.79836400000022],[-104.88474599999978,39.7983220000001],[-104.88458499999996,39.81297499999994],[-104.86607500000002,39.81298599999997],[-104.86604099999988,39.79838600000005],[-104.84934199999992,39.798347000000035],[-104.84230699999995,39.79815700000012],[-104.82913100000002,39.798059000000194],[-104.827922,39.798208999999986],[-104.80950999999999,39.798317000000225],[-104.79089499999998,39.79825100000028],[-104.79105399999997,39.81046200000003],[-104.79070799999994,39.8272140000002],[-104.7908589999999,39.842009000000076],[-104.762317,39.841874000000075],[-104.76213899999993,39.8454440000001],[-104.75051299999978,39.845434000000125],[-104.7321379999999,39.845085999999924],[-104.73185899999993,39.855828999999915],[-104.73190299999993,39.87533300000024],[-104.73154999999986,39.89861099999996],[-104.70556799999991,39.898490000000095],[-104.70537499999995,39.90641800000003],[-104.69260600000001,39.906614000000104],[-104.69293399999987,39.91418300000021],[-104.68026099999997,39.91424699999999],[-104.68021299999992,39.906599000000085],[-104.63409200000001,39.90619100000009],[-104.63385799999992,39.90606699999995],[-104.63396899999987,39.899289000000124],[-104.62584400000003,39.89929599999999],[-104.600435,39.89945899999998],[-104.600302,39.88228900000024],[-104.60585300000002,39.882244999999955],[-104.60579099999995,39.87382699999995],[-104.61984100000001,39.87376700000016],[-104.61981100000003,39.855781000000036],[-104.619709,39.84323100000029],[-104.61990599999996,39.82663300000007],[-104.62743799999993,39.826888000000224],[-104.630221,39.82678500000003],[-104.63964599999991,39.82689799999997],[-104.63967300000002,39.82255199999992],[-104.65551099999993,39.82247000000007],[-104.65545500000002,39.81404800000013],[-104.66619600000001,39.81400500000012],[-104.66627699999998,39.822850000000074],[-104.6799529999999,39.822869000000026],[-104.71709599999986,39.82321900000005],[-104.76246999999995,39.82297999999997],[-104.762901,39.813038000000006],[-104.76263199999994,39.80367800000005],[-104.76283399999988,39.79987800000009],[-104.762606,39.798433000000045],[-104.74458400000003,39.79850800000003],[-104.73439899999994,39.79844200000008],[-104.73442,39.78968900000007],[-104.73460099999994,39.769181],[-104.73665499999998,39.76939300000015],[-104.76077499999997,39.769488000000194],[-104.76457600000003,39.769390999999985],[-104.76455999999996,39.77159799999998],[-104.76762699999989,39.77168700000004],[-104.77216599999991,39.77146600000026],[-104.77215699999999,39.776408999999944],[-104.78181799999987,39.77648500000004],[-104.78172699999999,39.773387000000014],[-104.80667399999999,39.773190000000056],[-104.82822899999991,39.77317399999998],[-104.82832299999995,39.769156000000066],[-104.83300899999989,39.76917700000007],[-104.83296100000001,39.76619500000004],[-104.8375959999999,39.76726700000029],[-104.8563769999999,39.76867300000026],[-104.85653100000002,39.767818999999974],[-104.85663099999994,39.76028600000018],[-104.85643099999993,39.758387000000255],[-104.8512199999999,39.756981999999994],[-104.84822999999994,39.75648500000011],[-104.84693999999996,39.75468699999999],[-104.85872499999994,39.75475900000009],[-104.87990100000002,39.75473500000004],[-104.87997099999995,39.74743200000012],[-104.88466399999993,39.74746700000003],[-104.88464599999998,39.7401560000003]],[[-104.92898299999996,39.70847500000019],[-104.93152999999995,39.70908800000029],[-104.93164799999994,39.71109799999999],[-104.94068699999997,39.711221000000194],[-104.94073799999984,39.69795600000026],[-104.93871000000001,39.698181999999974],[-104.93863499999992,39.696605000000204],[-104.94061199999999,39.69656999999995],[-104.93953899999997,39.69468800000004],[-104.9362129999999,39.69478200000003],[-104.93591700000002,39.69659900000005],[-104.92786399999994,39.696654000000024],[-104.92856399999994,39.69842900000015],[-104.92857799999985,39.700221000000056],[-104.93283099999996,39.70028600000012],[-104.93399599999998,39.70138000000003],[-104.93402699999996,39.70389800000021],[-104.93283899999989,39.7037940000003],[-104.93286799999998,39.70218199999999],[-104.93163400000003,39.70218700000021],[-104.931423,39.703789000000256],[-104.92692399999999,39.70360000000005],[-104.926963,39.70669900000013],[-104.92911799999996,39.70678700000013],[-104.92898299999996,39.70847500000019]],[[-104.91103199999992,39.68207300000006],[-104.91103999999996,39.67988700000001],[-104.90820299999996,39.679883000000075],[-104.90820499999995,39.68207000000018],[-104.91103199999992,39.68207300000006]],[[-104.92446499999988,39.67664800000006],[-104.92572499999989,39.67528099999993],[-104.92757599999999,39.67482599999994],[-104.93024199999996,39.674832000000094],[-104.92642899999987,39.669394000000295],[-104.92291899999998,39.66788400000007],[-104.92222099999987,39.666947000000164],[-104.92223999999982,39.66376000000025],[-104.92101600000001,39.66033800000008],[-104.91887500000001,39.66033800000008],[-104.91824699999995,39.65934700000014],[-104.91675700000002,39.65900999999997],[-104.91589599999992,39.657149000000004],[-104.9165719999998,39.65611900000005],[-104.91298499999994,39.65620900000022],[-104.91296999999986,39.663133000000244],[-104.91593899999992,39.66347100000013],[-104.91594999999995,39.666152000000125],[-104.91637800000001,39.66758500000003],[-104.91754400000002,39.66757999999993],[-104.91755899999987,39.672983000000045],[-104.919218,39.673880000000054],[-104.92225000000002,39.674833000000035],[-104.92446499999988,39.67664800000006]],[[-104.9294349999999,39.67238700000013],[-104.9316409999999,39.67337400000008],[-104.93421799999987,39.67338799999999],[-104.93412899999998,39.67128100000008],[-104.93169499999982,39.67056400000013],[-104.931622,39.66747299999997],[-104.92954199999997,39.6693810000001],[-104.92733099999998,39.669484000000125],[-104.9294349999999,39.67238700000013]],[[-104.89878599999992,39.6857040000001],[-104.902671,39.68571200000008],[-104.89878299999992,39.68340200000006],[-104.89878599999992,39.6857040000001]]]]},"properties":{"OBJECTID":17,"COUNTYFP":"031","Shape_Leng":2.74422891914,"Shape_Area":0.0421031256555,"low":214070,"lowmod":330205,"LMMI":441065,"lowmoduniv":634420,"lowmod_pct":0.520483276063,"NAMELSAD10":"Denver County"}},
{"type":"Feature","geometry":{"type":"Polygon","coordinates":[[[-107.86084499999976,37.77551700000009],[-107.86193799999995,37.7741620000001],[-107.86315099999996,37.77296800000005],[-107.86315199999996,37.77266100000003],[-107.86370199999999,37.77241200000003],[-107.86521800000003,37.77170400000011],[-107.86540299999996,37.771058000000096],[-107.86489199999994,37.769093000000055],[-107.86660199999994,37.76794500000011],[-107.86739699999993,37.76767699999999],[-107.86819199999996,37.767322999999976],[-107.86863900000003,37.7669360000001],[-107.87043699999992,37.76347299999992],[-107.87048099999993,37.762762000000066],[-107.86709799999994,37.761890999999935],[-107.86570699999987,37.76171000000011],[-107.86307799999997,37.760016000000064],[-107.86309900000003,37.75682699999999],[-107.86202399999996,37.756169000000114],[-107.86461499999996,37.75261200000011],[-107.86456099999992,37.75055200000003],[-107.86603600000001,37.74801700000006],[-107.86590199999995,37.747268000000076],[-107.86736399999995,37.74654600000002],[-107.86992299999991,37.74526400000002],[-107.87168799999995,37.74354900000003],[-107.872951,37.74164100000013],[-107.87562200000002,37.73885100000018],[-107.87562200000002,37.73783600000007],[-107.87560199999996,37.73755700000004],[-107.87494900000002,37.73430400000001],[-107.87514499999997,37.732173999999986],[-107.87454400000001,37.72887900000006],[-107.87562200000002,37.726861999999926],[-107.87562200000002,37.725787000000025],[-107.87296600000002,37.72241599999995],[-107.87371000000002,37.720083000000045],[-107.87562200000002,37.718142],[-107.87719499999997,37.716309000000024],[-107.88039599999996,37.7175610000001],[-107.88158799999991,37.71841399999994],[-107.88551299999995,37.71785100000011],[-107.88939599999998,37.71844100000004],[-107.89271999999994,37.717808000000105],[-107.89498900000001,37.717875000000106],[-107.89673299999998,37.71917200000013],[-107.897895,37.719186000000036],[-107.899474,37.718771000000004],[-107.90134899999998,37.7190020000001],[-107.90341799999993,37.71839299999999],[-107.907015,37.71766699999995],[-107.9089189999998,37.71620200000007],[-107.91033599999992,37.71642500000007],[-107.91061000000002,37.715182000000084],[-107.91025299999995,37.71448100000015],[-107.91097500000001,37.71255900000028],[-107.91349200000002,37.71242500000005],[-107.91616199999993,37.711700000000064],[-107.916967,37.71103699999992],[-107.92051099999998,37.71037599999994],[-107.921805,37.71015600000004],[-107.92395799999997,37.71040100000005],[-107.92647399999998,37.71036400000003],[-107.92985599999992,37.709154000000126],[-107.93128199999995,37.70803799999993],[-107.93305299999997,37.70591200000007],[-107.93474299999997,37.705246000000045],[-107.93524199999996,37.70436600000005],[-107.93720599999995,37.702562],[-107.939886,37.70289500000001],[-107.94105400000001,37.70276999999999],[-107.94243499999988,37.704139999999995],[-107.94334200000003,37.70465400000012],[-107.94386299999996,37.706041000000084],[-107.94638099999997,37.706277000000114],[-107.94918399999989,37.70682000000005],[-107.94963200000001,37.706414999999936],[-107.95061999999996,37.706389000000115],[-107.952628,37.706788000000074],[-107.95430899999991,37.70647700000029],[-107.95758699999999,37.704678000000115],[-107.95810499999993,37.703184000000135],[-107.96005999999994,37.70217100000002],[-107.96364399999993,37.700540000000046],[-107.96435499999995,37.69926399999997],[-107.96770400000003,37.697125000000085],[-107.968955,37.695516000000055],[-107.96903299999997,37.69349899999992],[-107.97031299999998,37.69120399999997],[-107.97042199999987,37.689162000000124],[-107.97030799999993,37.68767000000008],[-107.97176399999995,37.68694000000028],[-107.97384599999992,37.68477400000012],[-107.97505999999993,37.68364800000006],[-107.97197999999997,37.681856000000266],[-107.96942000000001,37.67923200000007],[-107.96934699999991,37.67866700000002],[-107.96941600000002,37.677134000000194],[-107.96823199999994,37.67495800000012],[-107.96706899999998,37.673452],[-107.96548599999994,37.67059900000021],[-107.96586099999996,37.66950900000006],[-107.96441199999998,37.66763300000014],[-107.96469399999995,37.66534000000013],[-107.96355199999988,37.66365600000006],[-107.96155299999992,37.661748999999986],[-107.96132599999999,37.659628],[-107.96167999999989,37.65809400000006],[-107.96102699999994,37.657015],[-107.95831099999992,37.65108200000009],[-107.95707799999997,37.64889400000004],[-107.95861399999995,37.647113000000104],[-107.95943199999994,37.64373599999993],[-107.96006099999994,37.642638000000034],[-107.96157499999998,37.64185299999991],[-107.96338899999989,37.64131400000008],[-107.96650899999992,37.641197000000034],[-107.97008599999987,37.639587000000006],[-108.0006249999999,37.639378000000136],[-108.00742599999995,37.63926700000002],[-108.04897399999976,37.639049000000284],[-108.05821800000001,37.639094000000114],[-108.06643399999996,37.63895700000012],[-108.09115600000001,37.638863000000015],[-108.1079959999999,37.63875300000012],[-108.125632,37.638104999999996],[-108.1404389999999,37.636827000000096],[-108.16114199999993,37.635265000000004],[-108.17917199999994,37.63462700000014],[-108.19448399999999,37.63397000000015],[-108.20325899999995,37.633736],[-108.22679699999998,37.6332460000001],[-108.25063499999987,37.63284700000014],[-108.30394100000001,37.6323890000001],[-108.30925699999983,37.63229700000028],[-108.335642,37.632150000000195],[-108.36353400000002,37.631930000000295],[-108.37564099999997,37.63210700000019],[-108.39176199999997,37.63207],[-108.40829299999996,37.63188600000001],[-108.41719199999989,37.63248100000004],[-108.42400499999997,37.632666000000256],[-108.44454999999988,37.632834000000116],[-108.45766100000003,37.632498000000055],[-108.47451099999995,37.63252200000005],[-108.48342899999994,37.63246800000002],[-108.50064700000001,37.632526999999925],[-108.56441499999988,37.6326380000001],[-108.60990099999992,37.63271900000001],[-108.62564999999978,37.63261000000006],[-108.64649800000001,37.63269300000019],[-108.66605599999997,37.633195],[-108.66605999999996,37.63158299999998],[-108.69320299999993,37.63151100000027],[-108.72576400000003,37.63159700000028],[-108.74361799999997,37.63167299999998],[-108.75807700000001,37.6318],[-108.77382999999992,37.63178600000009],[-108.79727200000002,37.63193600000005],[-108.8373939999999,37.63184600000011],[-108.85712599999988,37.63193500000017],[-108.87565999999987,37.632202000000234],[-108.917081,37.63190199999991],[-108.92284999999976,37.624989000000085],[-108.92902999999995,37.617859000000124],[-108.97593899999998,37.56298900000013],[-109.00066399999997,37.53421700000001],[-109.00849499999993,37.52519699999999],[-109.02494499999995,37.504545000000064],[-109.02926899999994,37.49992400000002],[-109.03912000000003,37.4888260000003],[-109.04113599999977,37.487262000000214],[-109.04348799999997,37.48468400000024],[-109.04313699999989,37.499992000000134],[-109.04192,37.53052500000007],[-109.04180999999994,37.54669300000006],[-109.04191800000001,37.56905200000017],[-109.041789,37.60965500000003],[-109.04188799999991,37.62075599999997],[-109.04207299999996,37.62499100000008],[-109.04226799999992,37.6661280000003],[-109.04201099999989,37.687701000000004],[-109.04204400000003,37.70898300000016],[-109.0417599999999,37.71318200000013],[-109.04163599999993,37.74020999999999],[-109.04209400000002,37.749872000000266],[-109.04146100000003,37.80010500000009],[-109.04175399999986,37.83582600000011],[-109.04192299999988,37.842118000000255],[-109.04177599999997,37.8586260000003],[-109.0418469999999,37.87324700000016],[-109.04162500000001,37.87516200000016],[-109.0415119999999,37.881167000000175],[-109.01440699999995,37.88110100000006],[-109.00066399999997,37.881208000000186],[-108.99526900000001,37.88106300000027],[-108.96572199999997,37.88100400000019],[-108.93164499999995,37.88104199999998],[-108.92543399999988,37.88107900000017],[-108.90083499999997,37.88100500000007],[-108.88332500000001,37.881034],[-108.85372999999998,37.880942000000005],[-108.82134799999989,37.880920000000174],[-108.82052199999993,37.88085100000012],[-108.81988999999987,37.88413100000008],[-108.820515,37.88508200000007],[-108.82020699999998,37.88669700000031],[-108.81769899999995,37.888087999999925],[-108.81870399999997,37.889478],[-108.81808899999999,37.891460999999936],[-108.81812399999995,37.89375300000006],[-108.81671,37.89553799999999],[-108.80031200000002,37.89545100000004],[-108.750654,37.895312000000274],[-108.74628799999999,37.89538900000025],[-108.72616199999999,37.89533000000017],[-108.670234,37.89524200000017],[-108.65905899999996,37.895263000000114],[-108.62564999999978,37.89522400000027],[-108.62422400000003,37.895333999999934],[-108.61708099999998,37.89510600000028],[-108.5871679999999,37.89477900000003],[-108.55417999999992,37.89458100000019],[-108.50064500000002,37.894259000000034],[-108.45486699999998,37.894195000000025],[-108.43765599999995,37.89389900000009],[-108.39089899999999,37.893508000000224],[-108.37564099999997,37.893635000000245],[-108.37428699999987,37.89347700000013],[-108.36516299999988,37.89386999999999],[-108.35744399999999,37.893638000000124],[-108.34787599999987,37.89398399999999],[-108.336589,37.89383800000002],[-108.30669699999987,37.894120999999984],[-108.25649799999997,37.89460700000001],[-108.25653699999998,37.89355699999999],[-108.25773099999992,37.89104500000019],[-108.25739199999992,37.88968100000011],[-108.25424299999997,37.88647600000013],[-108.25268399999993,37.8853630000001],[-108.25036299999982,37.88457199999999],[-108.24793399999993,37.87937999999991],[-108.245252,37.87865499999998],[-108.23926099999989,37.87639600000011],[-108.23783700000001,37.87639199999995],[-108.23598399999992,37.87261799999999],[-108.23506099999992,37.86931300000009],[-108.23286299999995,37.86712],[-108.23194999999993,37.86506600000024],[-108.23159900000002,37.86174100000028],[-108.22958599999998,37.861041000000114],[-108.2300239999999,37.85717000000011],[-108.2294829999999,37.85421800000006],[-108.22652199999987,37.84552900000023],[-108.22700499999996,37.84347500000007],[-108.225796,37.84245599999997],[-108.22403999999995,37.842070000000035],[-108.22364400000004,37.840933000000064],[-108.22195499999981,37.839731000000086],[-108.22218399999986,37.83907600000009],[-108.21852799999988,37.837755000000016],[-108.21807799999993,37.83616599999999],[-108.21842499999997,37.834904999999935],[-108.21568200000002,37.83449099999996],[-108.21464300000002,37.83313199999998],[-108.212354,37.83203800000024],[-108.21099799999996,37.829049],[-108.21073200000001,37.82483600000012],[-108.20962199999991,37.82348500000006],[-108.21032300000002,37.82204400000023],[-108.209093,37.82093600000002],[-108.20545499999992,37.82177899999999],[-108.19906300000002,37.824200000000076],[-108.19572799999997,37.82408200000009],[-108.19239099999999,37.82441400000005],[-108.19021799999996,37.82524000000001],[-108.1831709999999,37.82647900000018],[-108.17834799999997,37.83117800000031],[-108.17490599999996,37.83259200000009],[-108.16914199999997,37.831110000000194],[-108.16710199999994,37.831928000000005],[-108.16620499999999,37.83331300000003],[-108.16385399999996,37.834923],[-108.16222299999993,37.83472899999998],[-108.16037399999988,37.83380900000003],[-108.15641600000004,37.83427200000017],[-108.15314199999989,37.835103000000174],[-108.15017399999999,37.83432600000003],[-108.1489719999999,37.83736000000022],[-108.14724099999995,37.838852999999915],[-108.14590099999992,37.83789100000013],[-108.14448799999997,37.83831700000019],[-108.14364499999994,37.83553500000005],[-108.14152699999994,37.83245400000004],[-108.14013399999999,37.832813999999985],[-108.14007699999996,37.83487400000024],[-108.13766799999996,37.83707300000009],[-108.137429,37.837890000000016],[-108.1322679999999,37.841547000000105],[-108.13110699999993,37.84295700000007],[-108.12843499999985,37.84397799999999],[-108.12663099999997,37.8453430000003],[-108.12465399999991,37.84859100000011],[-108.12471199999999,37.851075000000264],[-108.12221699999998,37.852539999999976],[-108.12095099999993,37.85384100000016],[-108.119731,37.85335099999992],[-108.11737399999993,37.85344399999997],[-108.11252300000001,37.85523500000022],[-108.11169899999993,37.854672000000164],[-108.10823199999987,37.853645000000085],[-108.10595199999989,37.85159200000004],[-108.10410200000001,37.85115200000024],[-108.10162000000003,37.84959100000003],[-108.100055,37.84940000000023],[-108.098275,37.84841100000011],[-108.09547299999997,37.84486600000025],[-108.09500599999996,37.84085000000016],[-108.09329999999994,37.840837000000136],[-108.09052600000001,37.83999400000005],[-108.08724799999987,37.8416450000002],[-108.08457699999985,37.84161700000004],[-108.081029,37.844250999999986],[-108.07897799999995,37.84450500000003],[-108.07561099999998,37.84674400000006],[-108.07184599999994,37.84793500000001],[-108.07007299999998,37.85045400000007],[-108.06893699999995,37.85132699999997],[-108.06647899999996,37.85104800000016],[-108.06420500000002,37.846727000000044],[-108.06173299999995,37.84499599999998],[-108.06034199999988,37.84487700000028],[-108.05981899999995,37.84262799999999],[-108.05847599999993,37.84100000000012],[-108.055298,37.841625000000136],[-108.05468999999994,37.842215000000294],[-108.05574899999999,37.84389100000004],[-108.05545599999999,37.844658000000095],[-108.05294899999996,37.84542699999997],[-108.04767700000002,37.85072500000007],[-108.04540299999996,37.851139000000046],[-108.04122799999976,37.85128900000001],[-108.03958399999999,37.85183900000004],[-108.03583700000001,37.8521820000002],[-108.03477199999992,37.85315100000008],[-108.03183899999999,37.854631000000154],[-108.03086499999995,37.855495000000076],[-108.02569399999993,37.855226000000016],[-108.0239479999999,37.85593700000021],[-108.0189089999999,37.85551499999991],[-108.01529399999993,37.85613900000004],[-108.01294899999976,37.85704500000003],[-108.01124300000004,37.85690899999997],[-108.01031699999999,37.855950000000064],[-108.0072889999999,37.85506600000008],[-108.00400899999994,37.85621400000002],[-108.00267899999994,37.85709400000019],[-107.998986,37.8558330000003],[-107.99545999999992,37.85659499999991],[-107.99213499999996,37.85641200000009],[-107.9892079999999,37.85663999999991],[-107.98796699999991,37.85460400000022],[-107.98747699999996,37.8509590000001],[-107.98559499999993,37.84712100000013],[-107.98417199999977,37.84503699999999],[-107.97982100000002,37.84301200000016],[-107.9776349999999,37.84033000000011],[-107.9772989999999,37.83922200000012],[-107.97530599999993,37.835869000000116],[-107.97334399999994,37.8335910000003],[-107.97139299999992,37.83242799999999],[-107.96759199999991,37.83178300000009],[-107.96589499999993,37.8318240000001],[-107.96334499999989,37.83285200000006],[-107.96189200000003,37.83279600000026],[-107.95900499999988,37.83451900000017],[-107.95492000000002,37.83444799999995],[-107.95076299999988,37.83575900000028],[-107.94818199999992,37.83382000000006],[-107.94671900000003,37.83067599999998],[-107.947248,37.828817000000186],[-107.94473799999992,37.82808200000011],[-107.94273599999991,37.82629600000013],[-107.94053199999996,37.82521400000013],[-107.938673,37.82300800000007],[-107.93764599999992,37.822667999999965],[-107.9338659999998,37.823494000000096],[-107.93008699999996,37.823341000000255],[-107.9276989999999,37.82362400000022],[-107.92591099999993,37.82334900000001],[-107.92134699999997,37.823713999999995],[-107.91921499999995,37.820715000000064],[-107.91774199999992,37.81957500000021],[-107.91808799999995,37.818686000000014],[-107.91561999999993,37.81654400000025],[-107.91360999999989,37.814111000000025],[-107.91080699999998,37.81240600000018],[-107.90964999999989,37.811145000000295],[-107.90624000000003,37.81102800000002],[-107.90552600000001,37.81014200000027],[-107.90484699999996,37.80745899999994],[-107.903435,37.80635900000004],[-107.90012399999989,37.80555900000013],[-107.89644900000002,37.80357900000007],[-107.8925079999999,37.80265700000018],[-107.89030699999995,37.79905100000025],[-107.88897699999995,37.79808100000008],[-107.88874499999986,37.7961490000003],[-107.89061800000002,37.79241500000006],[-107.89087299999989,37.7909200000002],[-107.88992999999994,37.788632000000064],[-107.88723099999999,37.78664299999997],[-107.88625699999994,37.785454000000016],[-107.88249999999988,37.78509800000023],[-107.8806709999999,37.78597800000006],[-107.87318699999997,37.78623900000031],[-107.87000899999992,37.78430200000008],[-107.870497,37.78145100000029],[-107.86739999999998,37.77970800000014],[-107.86424999999997,37.77912700000019],[-107.86139000000003,37.776859000000115],[-107.86084499999976,37.77551700000009]]]},"properties":{"OBJECTID":18,"COUNTYFP":"033","Shape_Leng":3.16643803237,"Shape_Area":0.282825157397,"low":755,"lowmod":1150,"LMMI":1475,"lowmoduniv":1865,"lowmod_pct":0.616621983914,"NAMELSAD10":"Dolores County"}},
{"type":"Feature","geometry":{"type":"Polygon","coordinates":[[[-104.66062599999987,39.56590899999992],[-104.66058699999991,39.55404500000009],[-104.66107299999999,39.538271000000236],[-104.66134499999993,39.526560000000075],[-104.66138999999993,39.51579800000013],[-104.66171099999991,39.503136000000154],[-104.66207099999997,39.484819000000186],[-104.66230599999994,39.463516000000254],[-104.66280799999993,39.44913300000002],[-104.66266599999989,39.4343980000001],[-104.66272899999996,39.42749200000014],[-104.66262699999976,39.41774499999997],[-104.66305899999998,39.37498900000014],[-104.66319799999997,39.36705499999999],[-104.66326099999992,39.34991700000006],[-104.66319499999997,39.34349100000003],[-104.66321899999991,39.31831700000009],[-104.66318399999994,39.255674000000226],[-104.66289099999995,39.24582200000009],[-104.66300999999993,39.19282199999992],[-104.66283599999991,39.15367300000008],[-104.66293699999994,39.13548400000025],[-104.66289599999999,39.12952700000011],[-104.671022,39.12947900000023],[-104.69096699999983,39.12956500000024],[-104.7164459999999,39.129574000000275],[-104.74769800000001,39.12967800000018],[-104.77604799999995,39.12964400000021],[-104.78457200000003,39.12971400000015],[-104.80231400000002,39.12967600000019],[-104.81677799999994,39.12976400000019],[-104.82525699999985,39.12971500000003],[-104.8471409999998,39.12988900000005],[-104.86376199999995,39.12978800000002],[-104.89930700000002,39.12985700000024],[-104.90309100000002,39.12981000000008],[-104.9446749999999,39.12982700000009],[-105.00053999999994,39.129917000000034],[-105.033544,39.12981900000011],[-105.10656999999992,39.129604000000086],[-105.12554399999999,39.12949400000008],[-105.16260999999997,39.12947900000023],[-105.17177700000002,39.12953900000002],[-105.19910400000003,39.12956500000024],[-105.20147500000002,39.12965100000025],[-105.221676,39.129585000000304],[-105.24622299999993,39.12961000000024],[-105.25054799999992,39.12954600000012],[-105.27549399999992,39.129525000000115],[-105.28566299999994,39.12967600000019],[-105.3292199999999,39.12968900000021],[-105.32739299999992,39.13087300000029],[-105.32477399999993,39.13386300000025],[-105.32275700000002,39.135595000000194],[-105.32196199999998,39.1369670000002],[-105.32177799999994,39.139017000000024],[-105.32278199999996,39.14111400000013],[-105.32214699999992,39.142958000000135],[-105.323285,39.144992],[-105.32155599999993,39.14619100000027],[-105.31991599999986,39.14575400000018],[-105.31678499999998,39.1455730000003],[-105.31534699999992,39.146712000000036],[-105.31619299999994,39.14885500000025],[-105.31557899999996,39.14991100000003],[-105.31594899999999,39.151302000000214],[-105.3153099999999,39.154895000000124],[-105.31413600000002,39.15551500000009],[-105.313087,39.157077000000015],[-105.31355699999989,39.15912100000003],[-105.31306299999994,39.16001900000026],[-105.3106919999999,39.161526000000094],[-105.30998199999993,39.16351900000012],[-105.30705899999987,39.16535600000009],[-105.30519699999996,39.16402399999998],[-105.30445399999996,39.1642810000003],[-105.30227100000002,39.16668900000002],[-105.30102199999988,39.166777000000025],[-105.30021399999998,39.168017000000134],[-105.30201399999999,39.169246000000044],[-105.30152899999979,39.17061200000012],[-105.29843800000003,39.172810000000084],[-105.294216,39.172977],[-105.293635,39.173974999999984],[-105.29474799999997,39.174837999999966],[-105.28949799999981,39.177494000000195],[-105.28737699999982,39.17944200000005],[-105.28733099999994,39.181254999999965],[-105.2892359999999,39.18341600000008],[-105.28918199999998,39.1841720000001],[-105.28638999999998,39.18631400000004],[-105.28592800000001,39.189584999999965],[-105.28470199999992,39.19270300000022],[-105.2851,39.19458800000007],[-105.28292899999997,39.19995300000005],[-105.28126099999997,39.20175700000027],[-105.27881400000001,39.201811000000134],[-105.27769599999988,39.202801000000306],[-105.27629099999996,39.20721600000002],[-105.27277800000002,39.20775500000002],[-105.27217599999994,39.20865100000009],[-105.269519,39.20941900000008],[-105.26754599999992,39.209225000000174],[-105.26311099999998,39.208023000000196],[-105.26211999999987,39.20991800000013],[-105.26005399999997,39.21044500000022],[-105.26008599999994,39.21255800000006],[-105.26151999999985,39.21475800000002],[-105.26057600000001,39.216688000000204],[-105.26286800000003,39.2194630000003],[-105.26352800000001,39.22116900000003],[-105.26130199999994,39.22326900000007],[-105.26242499999995,39.22483000000028],[-105.26166,39.22680500000007],[-105.26342399999993,39.22836000000012],[-105.26351099999994,39.229261000000065],[-105.26178400000003,39.23148199999997],[-105.25834299999997,39.23280100000005],[-105.257228,39.23354700000016],[-105.25432,39.237923000000194],[-105.25503000000003,39.240274000000056],[-105.25353599999988,39.2413720000003],[-105.25347199999993,39.243314000000055],[-105.25242799999995,39.24369500000029],[-105.24930299999994,39.24178800000004],[-105.24853599999989,39.24084600000009],[-105.24601699999994,39.24056600000006],[-105.243403,39.24265500000001],[-105.24129900000003,39.243437000000256],[-105.23609499999992,39.24405800000011],[-105.23436700000002,39.245562000000064],[-105.23393799999997,39.24686800000029],[-105.234781,39.24929700000018],[-105.23733600000003,39.24950000000007],[-105.2373879999999,39.25094200000001],[-105.23525599999988,39.25131299999998],[-105.23420199999998,39.25241300000005],[-105.234825,39.25457300000011],[-105.23439299999995,39.25530700000024],[-105.23180699999995,39.253962],[-105.22812099999999,39.2537660000001],[-105.22735499999993,39.254234999999994],[-105.22668099999993,39.25650200000018],[-105.22344299999997,39.25615400000015],[-105.22340199999996,39.25841700000001],[-105.22514199999989,39.25944300000015],[-105.22476099999989,39.26070700000008],[-105.21959800000002,39.25964099999999],[-105.217419,39.260395000000074],[-105.217784,39.26354799999996],[-105.21875,39.26534900000013],[-105.21869899999996,39.26704200000006],[-105.22077599999994,39.268062999999984],[-105.22055899999992,39.26917400000025],[-105.21917899999988,39.270503000000076],[-105.21557999999999,39.272232000000145],[-105.21566199999995,39.273348999999996],[-105.21750700000001,39.274595999999974],[-105.21554099999997,39.276635999999996],[-105.21638599999989,39.277618000000075],[-105.21543799999978,39.27857399999999],[-105.21389699999992,39.27869300000003],[-105.21325999999999,39.2797240000001],[-105.21132499999982,39.28096700000026],[-105.21133399999997,39.282914999999946],[-105.21247299999987,39.284484000000134],[-105.21017,39.28774000000021],[-105.20804199999998,39.289612000000204],[-105.20873599999993,39.29061599999994],[-105.20790799999997,39.291525000000036],[-105.20649799999995,39.2906780000003],[-105.20553099999995,39.29168100000015],[-105.20698099999993,39.293202000000235],[-105.2055269999999,39.293990000000065],[-105.20305400000001,39.29306900000006],[-105.20288700000003,39.294752000000244],[-105.20523100000003,39.29489899999999],[-105.20717300000001,39.296228000000156],[-105.20768099999987,39.29819400000008],[-105.206793,39.29919600000022],[-105.20497899999992,39.299798000000294],[-105.20136799999995,39.29926900000004],[-105.20086199999992,39.30164400000007],[-105.19920099999996,39.3020480000003],[-105.19599999999997,39.30208200000027],[-105.1965219999999,39.303949000000216],[-105.19782399999991,39.30498800000021],[-105.19668000000001,39.3085440000001],[-105.19853199999994,39.30943700000006],[-105.19815599999993,39.31043199999999],[-105.19466199999994,39.31243400000005],[-105.19348699999989,39.31258200000008],[-105.1901969999999,39.31619400000011],[-105.18858699999998,39.317384000000175],[-105.18912299999994,39.31910500000009],[-105.18881399999987,39.321153000000095],[-105.18780299999997,39.322853000000066],[-105.18629199999981,39.323304000000064],[-105.18548599999991,39.32440500000007],[-105.18588299999999,39.326155000000256],[-105.18805399999997,39.326902000000246],[-105.18818399999998,39.3279610000003],[-105.186057,39.32829000000015],[-105.18230499999999,39.329796999999985],[-105.18226800000002,39.33137400000004],[-105.18557199999992,39.33300300000002],[-105.187432,39.333256000000176],[-105.18783699999994,39.334186000000045],[-105.18507099999988,39.33483100000012],[-105.1839559999998,39.33549000000028],[-105.182144,39.33858800000007],[-105.18009999999998,39.33915500000029],[-105.17872899999998,39.34078600000004],[-105.17953499999993,39.342309000000114],[-105.17917399999999,39.34346199999999],[-105.1771579999999,39.34471000000019],[-105.1765319999999,39.3475150000001],[-105.17719299999999,39.349579000000176],[-105.17515700000001,39.350490000000036],[-105.171853,39.35345000000001],[-105.17235700000003,39.355817000000286],[-105.171517,39.35722000000004],[-105.16912699999995,39.35893400000026],[-105.16786000000002,39.359016999999994],[-105.16657399999986,39.36175400000002],[-105.16643499999998,39.362957999999935],[-105.1677919999999,39.36637100000007],[-105.16892199999995,39.367264999999975],[-105.16911399999987,39.36909800000012],[-105.16763599999996,39.370426000000066],[-105.16709899999984,39.37220000000008],[-105.16790000000003,39.3739800000003],[-105.1699549999999,39.374437000000285],[-105.17033000000004,39.37534700000026],[-105.17225999999994,39.37641100000019],[-105.1729969999999,39.37772200000029],[-105.17124999999993,39.379638],[-105.16961500000002,39.38068200000009],[-105.17085600000001,39.38226099999997],[-105.17312200000003,39.38222800000011],[-105.17351500000001,39.383203000000094],[-105.172978,39.3857450000001],[-105.17121899999995,39.386803999999984],[-105.16996799999998,39.38909600000011],[-105.16791299999988,39.389823000000206],[-105.16566,39.391178000000025],[-105.16433699999993,39.3911360000003],[-105.16549499999991,39.3940080000001],[-105.16725799999995,39.394308000000024],[-105.167642,39.39574799999997],[-105.16697199999999,39.39818700000018],[-105.167194,39.39975300000026],[-105.16940799999986,39.40159],[-105.169441,39.403326000000106],[-105.17131499999999,39.40655100000015],[-105.17096299999992,39.40782500000023],[-105.16904199999993,39.408935999999926],[-105.16667099999995,39.408510000000035],[-105.16261999999989,39.411928999999986],[-105.16126099999997,39.413758000000314],[-105.15993699999984,39.41432299999997],[-105.15597699999995,39.41380200000003],[-105.15176699999995,39.41568100000006],[-105.1494199999999,39.41760000000005],[-105.14531599999998,39.41881399999994],[-105.14140800000001,39.42113600000005],[-105.14056699999998,39.42240100000009],[-105.13820599999985,39.42307000000005],[-105.1369729999999,39.42542100000003],[-105.13542499999988,39.426779000000124],[-105.13307099999997,39.42743900000005],[-105.13156399999997,39.42923500000012],[-105.13005399999992,39.43006399999996],[-105.128424,39.43175700000006],[-105.12347699999992,39.43449400000003],[-105.12467400000003,39.43587900000006],[-105.12614399999978,39.44018699999998],[-105.12470400000001,39.44273600000008],[-105.12474199999997,39.44402000000025],[-105.12321400000002,39.447831000000065],[-105.12372799999991,39.449464000000205],[-105.12306199999995,39.45084700000024],[-105.1241839999999,39.454985000000306],[-105.1250169999999,39.45636700000006],[-105.12669599999998,39.45720000000006],[-105.12747399999989,39.4584670000001],[-105.13039899999995,39.45952700000004],[-105.13294099999996,39.46191200000004],[-105.13257699999997,39.46432100000004],[-105.13318599999991,39.46612500000009],[-105.13262500000002,39.46790800000019],[-105.13479699999988,39.4702010000002],[-105.13493099999994,39.47095200000001],[-105.13339299999996,39.47211000000004],[-105.13158199999998,39.472679000000085],[-105.12876399999993,39.475783000000206],[-105.12839799999995,39.47738800000013],[-105.12451799999991,39.479384000000266],[-105.12182299999995,39.48130400000014],[-105.12101699999977,39.4814540000001],[-105.11853499999995,39.48020699999995],[-105.11627599999991,39.48145100000022],[-105.11588299999988,39.48253500000021],[-105.11826599999989,39.482988000000205],[-105.11764599999992,39.48539200000016],[-105.11567100000002,39.48596000000026],[-105.11315399999995,39.483903999999995],[-105.11068499999988,39.483553000000086],[-105.10694199999989,39.48471900000027],[-105.10464100000002,39.485020000000304],[-105.1033129999999,39.48471500000028],[-105.10035900000003,39.485487000000205],[-105.09870699999982,39.485394000000156],[-105.09135300000003,39.48928000000001],[-105.08851399999986,39.49239200000005],[-105.08675099999994,39.49342300000029],[-105.08610899999991,39.494923000000085],[-105.08718499999998,39.496061000000054],[-105.08614799999987,39.49763300000029],[-105.0864949999999,39.49913300000003],[-105.08484099999998,39.50121000000013],[-105.08527500000002,39.50360799999993],[-105.08466499999992,39.506585000000086],[-105.08113299999997,39.50966699999998],[-105.07870000000003,39.5108360000001],[-105.07885899999985,39.51285000000007],[-105.077945,39.517495],[-105.07899999999995,39.519142000000215],[-105.08009599999997,39.52315599999997],[-105.0792879999999,39.523893000000214],[-105.07946099999992,39.525888000000066],[-105.076434,39.53162400000002],[-105.07610599999998,39.53269100000006],[-105.07657099999989,39.534967999999935],[-105.07774499999994,39.53670600000004],[-105.07641899999993,39.54005500000022],[-105.07459399999999,39.541652],[-105.07411100000002,39.5442680000001],[-105.07138499999996,39.54642900000022],[-105.07030799999995,39.547896000000264],[-105.066735,39.55049900000017],[-105.06567199999995,39.553345999999976],[-105.0637979999999,39.55523900000031],[-105.06112999999993,39.556906000000026],[-105.0607139999999,39.55994100000015],[-105.05825699999991,39.560378999999955],[-105.05725199999989,39.56230600000026],[-105.05427199999997,39.56299700000005],[-105.05136600000003,39.56325700000025],[-105.04874000000001,39.566088000000036],[-105.04028499999998,39.56619000000012],[-105.02899400000001,39.56618700000001],[-105.01031699999993,39.56608600000004],[-105.0020679999999,39.566233000000125],[-105.000268,39.566109000000154],[-104.97178099999985,39.566064000000154],[-104.96097199999997,39.56615500000004],[-104.94170500000001,39.56602499999997],[-104.890221,39.56600700000024],[-104.88282600000002,39.56595100000004],[-104.86066499999987,39.566050000000246],[-104.85533299999986,39.56598800000006],[-104.813131,39.56598800000006],[-104.81023099999976,39.566088000000036],[-104.80137099999996,39.56588700000026],[-104.79354399999994,39.566091000000256],[-104.74922899999996,39.566088000000036],[-104.73912799999988,39.56588799999997],[-104.72542799999985,39.56588799999997],[-104.72092800000001,39.56598800000006],[-104.68143800000001,39.56598800000006],[-104.66062599999987,39.56590899999992]]]},"properties":{"OBJECTID":19,"COUNTYFP":"035","Shape_Leng":2.24626364842,"Shape_Area":0.228049406124,"low":28280,"lowmod":56195,"LMMI":110920,"lowmoduniv":306310,"lowmod_pct":0.183457934772,"NAMELSAD10":"Douglas County"}},
{"type":"Feature","geometry":{"type":"Polygon","coordinates":[[[-106.43450799999994,39.924914000000115],[-106.43442399999992,39.92338700000022],[-106.43920200000002,39.919944999999984],[-106.44030099999998,39.91832300000027],[-106.43917899999997,39.91563200000019],[-106.43287499999991,39.91260700000004],[-106.43040000000002,39.90890600000006],[-106.42998999999986,39.90540300000009],[-106.43064900000002,39.90390099999996],[-106.43075999999996,39.901620000000264],[-106.429237,39.89838700000013],[-106.42868299999998,39.89510699999994],[-106.42947399999997,39.88711399999994],[-106.42810299999996,39.882164000000046],[-106.43109699999997,39.88010300000002],[-106.43396199999995,39.87737500000026],[-106.43379199999993,39.8749870000002],[-106.43441899999993,39.873001000000045],[-106.4339589999999,39.87026400000002],[-106.43273999999985,39.865968000000066],[-106.43268799999998,39.86455699999999],[-106.43038199999995,39.861526000000026],[-106.42708999999991,39.857675000000256],[-106.42578600000002,39.85536900000005],[-106.42362500000002,39.85010599999998],[-106.423497,39.84805200000005],[-106.42557499999998,39.84548800000016],[-106.42525499999994,39.844172000000015],[-106.42373900000001,39.84142500000007],[-106.420612,39.840003000000195],[-106.41872399999994,39.83627300000012],[-106.41510699999998,39.831440999999984],[-106.4120109999999,39.828670000000045],[-106.41222999999997,39.82678600000014],[-106.41111999999998,39.825491000000284],[-106.4106789999999,39.823698999999976],[-106.41119900000001,39.822089000000176],[-106.41071299999987,39.82036400000027],[-106.40946799999989,39.81920200000002],[-106.40716099999997,39.8128040000002],[-106.40618899999993,39.811240999999995],[-106.40188899999993,39.80754500000012],[-106.40109999999993,39.80564800000002],[-106.40090399999997,39.80281500000007],[-106.40127999999999,39.79985499999992],[-106.40073699999999,39.798736000000076],[-106.39642000000003,39.795781000000034],[-106.39579100000003,39.792707000000064],[-106.39664799999991,39.79170700000026],[-106.39686199999994,39.7900930000003],[-106.39430700000003,39.79056400000019],[-106.39031599999993,39.788725000000056],[-106.38856999999996,39.78654499999999],[-106.3888879999999,39.78422500000005],[-106.39092299999987,39.77791700000023],[-106.391033,39.77694200000019],[-106.38590299999993,39.76956200000029],[-106.38547800000003,39.76716400000009],[-106.38161799999995,39.76542700000027],[-106.37993299999988,39.764325999999926],[-106.37921900000003,39.763036],[-106.37419699999992,39.76221800000013],[-106.3733529999999,39.759549000000106],[-106.37053899999995,39.75661200000013],[-106.368472,39.75558700000022],[-106.36431299999987,39.75223000000028],[-106.36219800000003,39.75194699999997],[-106.35913799999986,39.753152],[-106.3539859999999,39.75255700000014],[-106.35210899999987,39.7555880000001],[-106.3507439999999,39.756879000000026],[-106.34591799999993,39.758111000000156],[-106.34355299999999,39.75904200000008],[-106.34333799999996,39.75529199999994],[-106.342105,39.753030000000024],[-106.33889599999998,39.750670000000014],[-106.33562799999993,39.749526],[-106.33387899999991,39.74655199999995],[-106.332986,39.74407100000002],[-106.33136300000001,39.74254799999994],[-106.33127200000001,39.74013100000002],[-106.32938899999994,39.73782700000015],[-106.32802499999985,39.73760000000004],[-106.32696299999992,39.73655200000002],[-106.32541500000002,39.733232999999984],[-106.32214599999998,39.73252100000025],[-106.32048099999997,39.731674],[-106.31849499999998,39.72865999999999],[-106.316195,39.72717599999993],[-106.31253199999998,39.72575400000005],[-106.31201199999992,39.72519000000011],[-106.30941899999988,39.72480100000007],[-106.30416300000002,39.72291900000016],[-106.3040069999999,39.72256400000009],[-106.29937699999988,39.720488999999986],[-106.29834599999992,39.72056900000007],[-106.29654900000003,39.71732900000001],[-106.29518699999994,39.716417000000035],[-106.2951789999999,39.71469400000012],[-106.29393299999992,39.7123170000001],[-106.2941879999999,39.70934500000004],[-106.29283800000002,39.7064840000001],[-106.29131999999993,39.70538699999997],[-106.29091599999998,39.70406600000007],[-106.28929299999999,39.703201999999976],[-106.28729499999997,39.7029500000001],[-106.28548499999994,39.70195699999999],[-106.28325799999988,39.70151100000004],[-106.28069899999997,39.70042100000029],[-106.27740099999994,39.69953900000013],[-106.27592199999998,39.70004399999999],[-106.27362599999998,39.702997000000266],[-106.27205200000003,39.70434799999998],[-106.26902599999988,39.702563000000055],[-106.26429099999996,39.7012840000001],[-106.26262199999996,39.69785100000007],[-106.26318900000001,39.69558100000006],[-106.26405499999998,39.6945750000001],[-106.26433099999997,39.691805000000045],[-106.265897,39.68953600000009],[-106.26563899999996,39.688529000000074],[-106.26399799999996,39.68705299999999],[-106.26161699999994,39.685874000000126],[-106.2613209999999,39.68271600000014],[-106.25824699999993,39.679804000000104],[-106.25558199999995,39.68003400000009],[-106.25199399999991,39.67903700000022],[-106.2505809999999,39.680210999999986],[-106.24284699999998,39.68091800000025],[-106.24035900000001,39.68024200000008],[-106.23874499999988,39.680274000000054],[-106.23909099999992,39.67774500000007],[-106.23882700000001,39.67599200000001],[-106.23930099999995,39.6709350000001],[-106.240476,39.66931800000003],[-106.23986300000001,39.66695700000008],[-106.23483999999979,39.66203000000013],[-106.232485,39.662229000000025],[-106.22951699999999,39.66114700000003],[-106.22646299999991,39.66094100000004],[-106.224196,39.66259099999996],[-106.22245199999998,39.66302700000017],[-106.21996499999989,39.66258600000009],[-106.21839299999994,39.66076300000026],[-106.2191749999999,39.657359000000156],[-106.21895499999988,39.65628000000004],[-106.21716500000002,39.65459200000004],[-106.20922699999994,39.65427800000015],[-106.20770099999999,39.65528800000004],[-106.20665399999996,39.6544430000003],[-106.20216099999999,39.654034000000024],[-106.19801499999988,39.65234200000003],[-106.19709899999992,39.65146400000003],[-106.19439399999993,39.65112300000004],[-106.19147299999992,39.651861],[-106.18920499999996,39.65334100000024],[-106.18384000000003,39.652153],[-106.1842319999999,39.65063600000025],[-106.18348999999995,39.6486120000003],[-106.18205,39.647666000000186],[-106.1813949999999,39.64591200000018],[-106.17934400000001,39.642503000000204],[-106.17956100000004,39.63960400000013],[-106.17820799999998,39.637140000000045],[-106.17646400000001,39.6357220000001],[-106.17615599999993,39.63332700000001],[-106.17746499999998,39.63167700000008],[-106.176218,39.62391900000017],[-106.17641299999997,39.619729000000234],[-106.17591999999996,39.616111000000046],[-106.17611499999992,39.614220999999986],[-106.17796699999997,39.61179500000031],[-106.17760399999992,39.60905600000018],[-106.17874999999998,39.60786400000012],[-106.1820679999999,39.60591400000027],[-106.18397400000003,39.60417300000029],[-106.18587399999984,39.60330399999998],[-106.19032299999998,39.60397900000021],[-106.19396499999988,39.605502000000286],[-106.20041800000001,39.60640400000011],[-106.20210499999996,39.60583600000001],[-106.20566300000002,39.603866000000096],[-106.20605599999993,39.60204500000009],[-106.20737400000002,39.601247000000114],[-106.20903699999997,39.59939600000007],[-106.2112239999999,39.593213000000276],[-106.21166,39.58981100000017],[-106.21086500000001,39.586803000000145],[-106.21395100000001,39.5820150000003],[-106.21364899999998,39.578776000000175],[-106.21502699999996,39.575186000000144],[-106.21453099999997,39.57432100000017],[-106.21160899999995,39.57336900000007],[-106.20982299999997,39.57033900000005],[-106.20992699999988,39.567047000000116],[-106.20914199999993,39.56464299999999],[-106.20946500000002,39.56358300000022],[-106.20864599999993,39.56163200000009],[-106.20571899999976,39.55821100000003],[-106.20376599999992,39.55779700000022],[-106.19874199999998,39.557885000000056],[-106.19645299999996,39.55734600000005],[-106.19764799999996,39.55506600000001],[-106.197248,39.55265000000014],[-106.19772199999989,39.5507570000002],[-106.19870700000001,39.55002000000013],[-106.2020139999999,39.54998300000028],[-106.20431399999995,39.54890900000004],[-106.20794099999995,39.54816699999992],[-106.20992599999988,39.54694300000028],[-106.2106819999999,39.54517600000008],[-106.21011099999998,39.5425120000001],[-106.21178500000002,39.53971000000007],[-106.21039599999995,39.534595000000195],[-106.21095500000001,39.533870000000036],[-106.21126799999996,39.53013200000004],[-106.21256499999998,39.53099700000001],[-106.21435499999995,39.53105700000003],[-106.21667999999994,39.53191900000013],[-106.21818999999999,39.53200700000008],[-106.22238899999996,39.53126600000007],[-106.22443699999997,39.53121900000008],[-106.22533299999986,39.53187000000031],[-106.228523,39.53704599999992],[-106.228861,39.538405000000296],[-106.23297099999996,39.542191],[-106.23427299999997,39.54278099999999],[-106.23539899999986,39.54454299999992],[-106.23541,39.548261999999966],[-106.23674800000003,39.548594000000094],[-106.24010599999997,39.546820000000025],[-106.24342399999995,39.54411900000002],[-106.24521599999997,39.54383300000018],[-106.24829499999993,39.54425099999992],[-106.24992799999995,39.54209400000002],[-106.2505809999999,39.53854400000006],[-106.2540259999999,39.53534700000023],[-106.25577399999986,39.532234000000244],[-106.25451800000002,39.52853399999998],[-106.2546099999999,39.52635699999996],[-106.25541299999992,39.524061000000074],[-106.25576699999999,39.51989699999996],[-106.25368800000001,39.51791900000012],[-106.2505809999999,39.517122000000086],[-106.24843900000002,39.51569900000027],[-106.24641599999978,39.51275200000015],[-106.24692299999992,39.51039899999995],[-106.24811399999987,39.508148999999946],[-106.24890199999987,39.50485999999995],[-106.25107999999994,39.502098000000046],[-106.25366399999979,39.50032500000009],[-106.25687699999997,39.49966100000006],[-106.25826599999999,39.49755300000004],[-106.25916399999994,39.49421500000028],[-106.25579399999992,39.493043000000114],[-106.25243499999999,39.49286200000006],[-106.24728500000003,39.49130000000014],[-106.24554699999993,39.48939200000001],[-106.24906099999998,39.48460300000005],[-106.25008699999995,39.48204100000004],[-106.24940900000001,39.48078600000008],[-106.250313,39.47743400000002],[-106.25148300000001,39.47671400000013],[-106.25167999999996,39.474300000000255],[-106.25040000000001,39.47093200000029],[-106.2505799999999,39.468218999999976],[-106.25152800000001,39.46752100000015],[-106.25163799999996,39.46503899999993],[-106.249933,39.46334500000012],[-106.24689899999998,39.461910000000046],[-106.24514899999997,39.461476000000005],[-106.24354099999994,39.45993199999998],[-106.24179200000003,39.45738000000023],[-106.23925899999995,39.45585200000028],[-106.23811,39.45415800000012],[-106.23892199999995,39.451361000000304],[-106.23599100000001,39.44596200000001],[-106.23406399999999,39.44421300000005],[-106.23185799999999,39.44097100000022],[-106.23015499999997,39.4409],[-106.22835399999997,39.43975699999993],[-106.224985,39.43660100000011],[-106.223251,39.43380700000023],[-106.22320999999988,39.43310500000024],[-106.22544299999998,39.43184300000007],[-106.2256769999999,39.430153000000246],[-106.22759899999994,39.42809900000009],[-106.232687,39.4262480000001],[-106.23005799999999,39.42226100000005],[-106.23001099999993,39.419892000000004],[-106.23091,39.418867999999975],[-106.23040900000001,39.41695400000026],[-106.22855499999997,39.41465000000022],[-106.22536100000002,39.41362300000014],[-106.2222129999999,39.41302399999995],[-106.22090500000002,39.41229600000014],[-106.21898299999998,39.412328000000116],[-106.21698499999997,39.40993400000008],[-106.21786099999997,39.408346000000165],[-106.216321,39.407196000000226],[-106.21350499999988,39.405970000000025],[-106.213074,39.404274000000214],[-106.211588,39.403609000000074],[-106.20237500000002,39.39713900000015],[-106.20104200000003,39.39481500000005],[-106.20304099999998,39.38973900000019],[-106.20678199999998,39.38702600000005],[-106.20821599999988,39.38630500000011],[-106.21039999999994,39.3841240000001],[-106.210534,39.3827950000001],[-106.20791300000002,39.38110500000005],[-106.20673199999993,39.379636000000005],[-106.22562700000003,39.37963000000025],[-106.22678699999989,39.37705400000027],[-106.226898,39.37416499999995],[-106.22958800000004,39.373167000000024],[-106.23392499999994,39.372936000000095],[-106.23648099999986,39.3703270000002],[-106.23731700000002,39.368517],[-106.23951699999998,39.36566700000009],[-106.24500599999993,39.36410699999999],[-106.24739599999998,39.36280399999998],[-106.2501539999999,39.36070100000006],[-106.250898,39.35876999999999],[-106.25248499999987,39.359111000000155],[-106.25755800000002,39.361515999999995],[-106.25945599999994,39.36186100000015],[-106.25996299999997,39.36084599999998],[-106.26382199999989,39.35876900000028],[-106.26661799999994,39.358835000000056],[-106.27095399999996,39.35807400000016],[-106.27257599999996,39.356358000000284],[-106.27473199999991,39.355683000000056],[-106.27966199999997,39.35234900000023],[-106.28321199999999,39.35046900000009],[-106.286587,39.352220000000216],[-106.29231199999992,39.35184900000024],[-106.29530899999992,39.35293600000011],[-106.29631399999994,39.35733499999992],[-106.29780999999991,39.35795200000018],[-106.30041499999999,39.360067000000015],[-106.30255699999992,39.36111900000003],[-106.30637899999999,39.3613390000001],[-106.31237099999998,39.362278],[-106.31624899999986,39.361211000000026],[-106.32060099999995,39.359375],[-106.32383199999987,39.35898200000014],[-106.32800299999991,39.35923600000001],[-106.3329159999999,39.358230000000276],[-106.33478999999988,39.35895100000005],[-106.33925199999993,39.35906399999999],[-106.34413199999995,39.35771200000022],[-106.34519999999992,39.35678600000017],[-106.34990399999987,39.354936000000066],[-106.35174199999989,39.3536830000001],[-106.35557299999977,39.35486100000003],[-106.358022,39.35514000000023],[-106.36264299999988,39.357217000000105],[-106.36313899999999,39.359234000000015],[-106.36678899999993,39.36064800000014],[-106.369595,39.360284000000036],[-106.37157299999996,39.361721999999986],[-106.37197500000002,39.3637080000002],[-106.37469399999992,39.365700000000174],[-106.37638300000003,39.368541999999934],[-106.37820199999982,39.36940100000015],[-106.37857699999995,39.371459999999956],[-106.38063299999999,39.37615000000005],[-106.38365399999992,39.37660600000015],[-106.38603999999998,39.37655600000028],[-106.39025900000001,39.37812500000024],[-106.39165799999995,39.379397000000154],[-106.39760299999995,39.37984500000027],[-106.40063399999991,39.380962000000125],[-106.4039249999999,39.37984799999998],[-106.40514699999994,39.378926999999976],[-106.40745099999998,39.37567800000028],[-106.40815399999997,39.37369600000005],[-106.41093499999994,39.37199700000019],[-106.41310699999997,39.37005799999997],[-106.414785,39.367489999999975],[-106.41727199999997,39.36729600000024],[-106.42162599999989,39.36379700000009],[-106.42648700000001,39.361871000000065],[-106.48936199999997,39.36181500000009],[-106.500587,39.362319000000014],[-106.54458499999993,39.3622170000001],[-106.54699399999993,39.362088000000085],[-106.62558799999994,39.362107000000094],[-106.64809399999984,39.36228100000005],[-106.65868599999999,39.36228300000005],[-106.67266899999998,39.362081000000046],[-106.68404900000002,39.36242599999997],[-106.70236899999998,39.36237800000009],[-106.70829000000003,39.362009000000114],[-106.71282999999988,39.36200699999995],[-106.72418899999985,39.36263800000012],[-106.72846399999992,39.36248999999998],[-106.73254499999996,39.362599999999986],[-106.740141,39.36219900000003],[-106.74111099999988,39.36242900000025],[-106.75059199999998,39.36267900000013],[-106.77502200000004,39.363018000000125],[-106.81827900000002,39.36373100000014],[-106.84653299999991,39.364299000000244],[-106.87559499999986,39.364535000000046],[-106.90858999999989,39.364901000000145],[-106.97340299999996,39.36564300000009],[-107.02297499999997,39.366167000000075],[-107.04818999999986,39.36615400000005],[-107.05634800000001,39.366300999999964],[-107.05951400000004,39.36603800000006],[-107.0619759999999,39.366261000000236],[-107.08330299999989,39.366100000000074],[-107.11344600000001,39.3660660000001],[-107.11358399999989,39.37498300000016],[-107.11354899999992,39.391594],[-107.11361899999986,39.3919690000003],[-107.11358799999994,39.423967000000175],[-107.11352399999998,39.4324400000001],[-107.113564,39.454196000000024],[-107.11366299999992,39.46382100000005],[-107.11363299999994,39.499831000000256],[-107.11394699999988,39.50861299999997],[-107.11374499999988,39.5564710000001],[-107.11359899999997,39.59133600000007],[-107.11386900000002,39.62509499999993],[-107.11357099999998,39.63419100000027],[-107.11350299999992,39.64050600000019],[-107.11342899999994,39.69720500000011],[-107.11357199999998,39.73128900000012],[-107.11353400000002,39.75172100000026],[-107.11354999999992,39.782420000000116],[-107.11352399999998,39.805357999999956],[-107.11354799999992,39.86676100000028],[-107.11363999999998,39.87502699999999],[-107.11365799999999,39.89959300000004],[-107.11359599999997,39.90038600000014],[-107.11367199999995,39.919111999999984],[-107.05250199999983,39.91892400000006],[-107.03396799999996,39.91891300000003],[-107.00045599999999,39.91904800000003],[-106.99356999999998,39.91879700000004],[-106.98359099999988,39.91886900000014],[-106.96506599999992,39.9187850000003],[-106.95635500000003,39.91881600000005],[-106.93617199999994,39.91874100000001],[-106.91506599999985,39.918911000000094],[-106.90945199999999,39.91885199999996],[-106.87607199999997,39.918814000000225],[-106.86030299999987,39.91917000000018],[-106.836974,39.91919700000028],[-106.8331649999999,39.91940599999998],[-106.79856399999994,39.919825],[-106.79236499999996,39.9200000000003],[-106.78554999999989,39.919811000000095],[-106.77423299999992,39.9199280000002],[-106.76631099999992,39.91983400000004],[-106.75058999999999,39.92002300000024],[-106.74851100000001,39.92012699999998],[-106.72531099999998,39.92060800000013],[-106.71298300000001,39.920618000000104],[-106.6536529999999,39.91992700000009],[-106.653257,39.91996999999992],[-106.62655499999994,39.91867100000013],[-106.62656900000002,39.9247850000001],[-106.62559099999993,39.92467199999999],[-106.61877900000002,39.92522500000024],[-106.61515700000001,39.92529200000013],[-106.6079269999999,39.925085000000024],[-106.57848599999994,39.9249530000003],[-106.562052,39.9248070000001],[-106.55299999999994,39.92491900000016],[-106.53424499999988,39.924946999999975],[-106.52785899999986,39.92516600000016],[-106.52211899999986,39.9250320000001],[-106.50061899999986,39.92498599999999],[-106.48625400000003,39.925250000000176],[-106.48539799999992,39.92504500000007],[-106.47290699999996,39.92510900000025],[-106.44377999999989,39.924948000000086],[-106.44257399999975,39.92480400000022],[-106.43450799999994,39.924914000000115]]]},"properties":{"OBJECTID":20,"COUNTYFP":"037","Shape_Leng":3.15804234846,"Shape_Area":0.45967127501,"low":11295,"lowmod":23615,"LMMI":33005,"lowmoduniv":52500,"lowmod_pct":0.44980952381,"NAMELSAD10":"Eagle County"}},
{"type":"Feature","geometry":{"type":"Polygon","coordinates":[[[-103.71435500000001,39.56633100000022],[-103.71411699999993,39.548748000000046],[-103.71439700000002,39.53393999999997],[-103.71429099999995,39.49999000000008],[-103.71449699999994,39.490616000000216],[-103.71449699999994,39.483903000000055],[-103.714765,39.46201200000013],[-103.71520199999992,39.418088000000125],[-103.71528799999993,39.39463800000016],[-103.71525199999996,39.374986999999976],[-103.71484900000002,39.37388300000026],[-103.71479599999998,39.344540000000165],[-103.7150769999999,39.30988500000001],[-103.71545799999996,39.3023400000003],[-103.71489200000002,39.28515399999992],[-103.71488899999997,39.27739200000002],[-103.71504599999997,39.24998700000009],[-103.71489400000002,39.244187000000124],[-103.7150739999999,39.22974200000027],[-103.71517099999994,39.20136499999995],[-103.71491999999995,39.179022000000145],[-103.71511599999991,39.17814900000002],[-103.7150769999999,39.171722999999986],[-103.71528499999994,39.15763600000008],[-103.71534099999997,39.135289000000284],[-103.71525399999996,39.12824599999999],[-103.7211519999999,39.12830700000006],[-103.720801,39.11989199999999],[-103.72074999999995,39.11093000000005],[-103.72082399999994,39.09874200000007],[-103.72200699999996,39.08408500000024],[-103.72178699999995,39.079755999999975],[-103.7209519999999,39.05517999999995],[-103.72089399999987,39.042463999999995],[-103.72093899999987,39.011095000000296],[-103.72142299999996,38.999990000000025],[-103.72134499999999,38.98657700000007],[-103.7206119999999,38.93243800000022],[-103.72019299999988,38.905647000000044],[-103.71941799999996,38.87499400000007],[-103.7196639999998,38.86682700000006],[-103.75038799999987,38.86643600000025],[-103.75746800000002,38.86655900000011],[-103.76509199999987,38.86646300000001],[-103.77709400000003,38.866729000000134],[-103.83860500000003,38.8669670000001],[-103.85202899999996,38.866919999999936],[-103.87881299999998,38.86751499999997],[-103.940473,38.86822599999999],[-103.98071199999993,38.8686910000003],[-103.98137099999985,38.8687520000002],[-104.00829499999998,38.86877000000027],[-104.05552799999998,38.86886800000002],[-104.055002,38.88608700000009],[-104.05437,38.90960600000005],[-104.0540519999999,38.91790000000009],[-104.05308600000001,38.95496600000007],[-104.05316299999998,38.974776000000304],[-104.05326699999989,38.982041000000095],[-104.05342100000001,39.013642000000175],[-104.05346099999986,39.04382800000002],[-104.05300699999987,39.062974000000054],[-104.05237899999986,39.093593],[-104.05216200000001,39.10046400000027],[-104.05201099999988,39.11418000000003],[-104.05158900000004,39.12837000000013],[-104.07774299999988,39.12845300000026],[-104.088569,39.12831800000009],[-104.10595799999993,39.12849299999999],[-104.12497899999988,39.12889000000018],[-104.12551500000001,39.1286530000001],[-104.14432299999999,39.12863500000003],[-104.16093199999995,39.12867899999992],[-104.17417199999988,39.12840200000011],[-104.18050599999992,39.12873600000023],[-104.199141,39.128600000000006],[-104.20932699999992,39.12866700000001],[-104.2363979999999,39.128644000000065],[-104.24115099999995,39.12882500000012],[-104.24648399999995,39.12870099999998],[-104.26423599999998,39.128659000000084],[-104.29755699999998,39.12875100000002],[-104.31072799999987,39.12900500000012],[-104.34792600000003,39.1288550000001],[-104.378401,39.12902200000008],[-104.43879800000002,39.1293740000001],[-104.45068499999991,39.12940900000012],[-104.46115099999997,39.12954300000001],[-104.46351899999996,39.12980300000004],[-104.46677399999999,39.129653000000246],[-104.48395199999987,39.129749000000004],[-104.50052299999987,39.129540000000304],[-104.54981499999997,39.12939300000022],[-104.58264500000001,39.129457],[-104.59131400000001,39.129391000000055],[-104.6134439999999,39.1294410000001],[-104.62231099999991,39.12979600000017],[-104.62427899999994,39.129485000000045],[-104.62820699999997,39.129440000000045],[-104.64430699999991,39.129533000000265],[-104.66289599999999,39.12952700000011],[-104.66293699999994,39.13548400000025],[-104.66283599999991,39.15367300000008],[-104.66300999999993,39.19282199999992],[-104.66289099999995,39.24582200000009],[-104.66318399999994,39.255674000000226],[-104.66321899999991,39.31831700000009],[-104.66319499999997,39.34349100000003],[-104.66326099999992,39.34991700000006],[-104.66319799999997,39.36705499999999],[-104.66305899999998,39.37498900000014],[-104.66262699999976,39.41774499999997],[-104.66272899999996,39.42749200000014],[-104.66266599999989,39.4343980000001],[-104.66280799999993,39.44913300000002],[-104.66230599999994,39.463516000000254],[-104.66207099999997,39.484819000000186],[-104.66171099999991,39.503136000000154],[-104.66138999999993,39.51579800000013],[-104.66134499999993,39.526560000000075],[-104.66107299999999,39.538271000000236],[-104.66058699999991,39.55404500000009],[-104.66062599999987,39.56590899999992],[-104.6456829999999,39.56588799999997],[-104.60444399999989,39.56590899999992],[-104.55995000000001,39.56514800000002],[-104.55505399999993,39.56500799999998],[-104.52376800000002,39.56457300000005],[-104.5161839999999,39.56438600000024],[-104.49831399999988,39.564143],[-104.47823800000003,39.56406500000031],[-104.4693769999999,39.563946000000044],[-104.45736699999998,39.56397600000008],[-104.44770399999999,39.563741000000164],[-104.42745199999996,39.56360100000029],[-104.4174349999999,39.563764000000276],[-104.38113099999993,39.56409700000012],[-104.37597999999997,39.56394899999998],[-104.36169799999999,39.564335000000085],[-104.343301,39.564429000000246],[-104.32344999999992,39.56438600000024],[-104.28760799999998,39.56471700000026],[-104.26482299999998,39.56471799999997],[-104.25420500000001,39.56494299999997],[-104.22040400000003,39.56514600000003],[-104.20723499999991,39.56517299999996],[-104.18120899999991,39.565313],[-104.15622799999994,39.56532800000019],[-104.11129799999998,39.56557700000019],[-104.10238099999998,39.565698999999995],[-104.06732099999999,39.56582000000003],[-104.05656999999997,39.56595399999992],[-104.02768600000002,39.56594999999999],[-103.98697599999997,39.56615100000005],[-103.97863899999993,39.566360000000145],[-103.95075600000001,39.56623000000002],[-103.90856100000002,39.56635800000015],[-103.87551199999979,39.56662300000022],[-103.84194599999995,39.56657100000018],[-103.82649199999992,39.566597],[-103.75192500000003,39.56656100000009],[-103.71435500000001,39.56633100000022]]]},"properties":{"OBJECTID":21,"COUNTYFP":"039","Shape_Leng":3.29597836752,"Shape_Area":0.500512675711,"low":3620,"lowmod":7575,"LMMI":12270,"lowmoduniv":23775,"lowmod_pct":0.318611987382,"NAMELSAD10":"Elbert County"}},
{"type":"Feature","geometry":{"type":"Polygon","coordinates":[[[-104.66289599999999,39.12952700000011],[-104.64430699999991,39.129533000000265],[-104.62820699999997,39.129440000000045],[-104.62427899999994,39.129485000000045],[-104.62231099999991,39.12979600000017],[-104.6134439999999,39.1294410000001],[-104.59131400000001,39.129391000000055],[-104.58264500000001,39.129457],[-104.54981499999997,39.12939300000022],[-104.50052299999987,39.129540000000304],[-104.48395199999987,39.129749000000004],[-104.46677399999999,39.129653000000246],[-104.46351899999996,39.12980300000004],[-104.46115099999997,39.12954300000001],[-104.45068499999991,39.12940900000012],[-104.43879800000002,39.1293740000001],[-104.378401,39.12902200000008],[-104.34792600000003,39.1288550000001],[-104.31072799999987,39.12900500000012],[-104.29755699999998,39.12875100000002],[-104.26423599999998,39.128659000000084],[-104.24648399999995,39.12870099999998],[-104.24115099999995,39.12882500000012],[-104.2363979999999,39.128644000000065],[-104.20932699999992,39.12866700000001],[-104.199141,39.128600000000006],[-104.18050599999992,39.12873600000023],[-104.17417199999988,39.12840200000011],[-104.16093199999995,39.12867899999992],[-104.14432299999999,39.12863500000003],[-104.12551500000001,39.1286530000001],[-104.12497899999988,39.12889000000018],[-104.10595799999993,39.12849299999999],[-104.088569,39.12831800000009],[-104.07774299999988,39.12845300000026],[-104.05158900000004,39.12837000000013],[-104.05201099999988,39.11418000000003],[-104.05216200000001,39.10046400000027],[-104.05237899999986,39.093593],[-104.05300699999987,39.062974000000054],[-104.05346099999986,39.04382800000002],[-104.05342100000001,39.013642000000175],[-104.05326699999989,38.982041000000095],[-104.05316299999998,38.974776000000304],[-104.05308600000001,38.95496600000007],[-104.0540519999999,38.91790000000009],[-104.05437,38.90960600000005],[-104.055002,38.88608700000009],[-104.05552799999998,38.86886800000002],[-104.05526999999995,38.85270700000012],[-104.054825,38.81115100000005],[-104.05450300000001,38.78266899999994],[-104.0541389999999,38.778841000000114],[-104.05428999999987,38.77647500000012],[-104.05397600000003,38.759877000000074],[-104.05328499999996,38.738332000000014],[-104.05317100000002,38.73110699999995],[-104.0524749999999,38.709235000000206],[-104.05194199999994,38.69434700000028],[-104.05298199999993,38.69434100000012],[-104.05333999999988,38.68163700000025],[-104.05372799999992,38.65366],[-104.05408299999999,38.638667999999996],[-104.05409299999991,38.62497600000006],[-104.05423399999995,38.615195000000085],[-104.05438300000003,38.59530400000028],[-104.05423299999995,38.59259000000026],[-104.05403099999995,38.580183999999974],[-104.05427499999996,38.57362200000006],[-104.05421100000001,38.55280300000021],[-104.05430799999993,38.54515400000025],[-104.05426299999988,38.533718000000135],[-104.054127,38.523086999999975],[-104.05392099999989,38.522393000000136],[-104.08346799999993,38.522433999999976],[-104.09017899999998,38.522004000000265],[-104.124347,38.521746000000235],[-104.12877200000003,38.521832000000245],[-104.15015199999993,38.52173199999993],[-104.15250100000003,38.52190200000001],[-104.16528499999987,38.52227099999999],[-104.17450899999994,38.52218699999992],[-104.19352200000003,38.52183400000007],[-104.25051899999994,38.52130699999998],[-104.29851500000001,38.520991000000095],[-104.32649699999996,38.52087700000027],[-104.35418399999992,38.52080400000011],[-104.37552099999999,38.52060699999993],[-104.38180699999998,38.520733000000064],[-104.40023100000002,38.52061400000002],[-104.410798,38.52061700000024],[-104.41311799999994,38.520461000000125],[-104.42855299999985,38.52032500000024],[-104.436511,38.520452999999975],[-104.45648599999998,38.52032900000006],[-104.47999199999987,38.52028999999999],[-104.50052299999987,38.51989800000007],[-104.53263700000002,38.51984300000015],[-104.5670869999999,38.52000100000026],[-104.57371899999993,38.51993900000008],[-104.5909969999999,38.52000399999997],[-104.60410799999988,38.52029800000014],[-104.62565699999993,38.51994300000007],[-104.66037599999987,38.51975400000026],[-104.6843219999999,38.5194400000002],[-104.70694899999995,38.51954699999999],[-104.72334000000001,38.519566],[-104.7413499999999,38.51965200000001],[-104.75052999999997,38.519870999999966],[-104.77057899999988,38.51983899999999],[-104.78721699999994,38.52011900000025],[-104.79610400000001,38.520165000000134],[-104.79812199999992,38.51998400000008],[-104.81597599999986,38.52002900000008],[-104.82316499999985,38.5197790000002],[-104.83616699999988,38.51997999999992],[-104.84474699999998,38.51990600000022],[-104.86615,38.51985700000006],[-104.86667399999993,38.520017000000166],[-104.88197099999996,38.52008799999999],[-104.88834299999996,38.5199090000001],[-104.89568300000002,38.520300000000304],[-104.93294899999984,38.520286000000056],[-104.94153299999999,38.51956900000022],[-104.94178299999999,38.55255200000005],[-104.94214399999993,38.587920000000224],[-104.94252799999998,38.624993000000245],[-104.9424259999999,38.64988200000005],[-104.94223499999998,38.696199000000206],[-104.93974400000002,38.69614200000029],[-104.93955499999993,38.70364500000005],[-104.93900599999989,38.71052600000007],[-104.93942600000003,38.74998800000026],[-104.93926399999992,38.75715700000029],[-104.93920200000002,38.781811000000005],[-104.93897699999991,38.79229100000009],[-104.93899399999998,38.79688399999998],[-104.98576199999997,38.79763500000013],[-105.00535899999988,38.79739600000005],[-105.01103299999994,38.79698700000017],[-105.01531799999992,38.796947000000046],[-105.030619,38.797132999999974],[-105.05318399999982,38.79912900000011],[-105.072184,38.799382000000264],[-105.07252900000003,38.808400000000006],[-105.07163699999995,38.81944000000021],[-105.06765999999988,38.85899100000029],[-105.06795399999987,38.85968800000006],[-105.067409,38.86215300000009],[-105.06732999999991,38.86530500000009],[-105.06670000000003,38.86950800000022],[-105.02890299999996,38.868907000000036],[-105.02875499999988,38.87603700000005],[-105.02819499999998,38.914515000000165],[-105.02787799999999,38.93277400000011],[-105.02710799999994,38.93617400000005],[-105.02747299999993,38.93894499999999],[-105.02776999999992,38.94573300000013],[-105.0272,38.985334999999964],[-105.02982700000001,38.9994040000002],[-105.02899500000001,39.00407299999995],[-105.02914299999992,39.00557500000008],[-105.02862199999998,39.00771199999997],[-105.0279339999999,39.013127],[-105.027626,39.02067500000015],[-105.02731499999993,39.023081000000275],[-105.02694499999996,39.029831000000286],[-105.02718799999991,39.03651600000006],[-105.02809200000002,39.04651499999994],[-105.02809300000001,39.05089200000009],[-105.02848199999994,39.055499000000054],[-105.02868899999993,39.06078200000002],[-105.02926699999989,39.06452100000013],[-105.02954799999986,39.07097299999998],[-105.03050899999988,39.08036000000004],[-105.03056099999992,39.086398000000145],[-105.03141499999987,39.091922000000125],[-105.03122999999994,39.09790099999992],[-105.031746,39.10170400000004],[-105.03179499999999,39.10423500000002],[-105.03255799999994,39.11008099999998],[-105.03255699999994,39.11376500000017],[-105.03290799999996,39.119080999999994],[-105.03278499999999,39.1230690000001],[-105.033544,39.12981900000011],[-105.00053999999994,39.129917000000034],[-104.9446749999999,39.12982700000009],[-104.90309100000002,39.12981000000008],[-104.89930700000002,39.12985700000024],[-104.86376199999995,39.12978800000002],[-104.8471409999998,39.12988900000005],[-104.82525699999985,39.12971500000003],[-104.81677799999994,39.12976400000019],[-104.80231400000002,39.12967600000019],[-104.78457200000003,39.12971400000015],[-104.77604799999995,39.12964400000021],[-104.74769800000001,39.12967800000018],[-104.7164459999999,39.129574000000275],[-104.69096699999983,39.12956500000024],[-104.671022,39.12947900000023],[-104.66289599999999,39.12952700000011]]]},"properties":{"OBJECTID":22,"COUNTYFP":"041","Shape_Leng":3.26109919342,"Shape_Area":0.572203417629,"low":151625,"lowmod":266290,"LMMI":393145,"lowmoduniv":638400,"lowmod_pct":0.417120927318,"NAMELSAD10":"El Paso County"}},
{"type":"Feature","geometry":{"type":"Polygon","coordinates":[[[-105.3291339999999,38.69720499999994],[-105.31351299999989,38.69699899999995],[-105.28544,38.69675200000012],[-105.279222,38.69674300000008],[-105.25053500000001,38.69648400000017],[-105.24243199999995,38.69698100000005],[-105.23779200000001,38.696901000000025],[-105.23992399999992,38.677058999999986],[-105.24042999999995,38.659563000000105],[-105.240362,38.647595000000194],[-105.19589699999989,38.64770400000015],[-105.125497,38.647860000000264],[-105.11587600000001,38.64762700000017],[-105.10685399999988,38.64753600000029],[-105.09295399999996,38.64776599999993],[-105.07152899999988,38.64855],[-105.06694800000002,38.64976100000001],[-105.05755399999993,38.65169300000014],[-105.05392599999993,38.65309600000006],[-105.03565900000001,38.65139300000021],[-105.00053199999991,38.65118300000023],[-104.99491999999998,38.650874000000044],[-104.9424259999999,38.64988200000005],[-104.94252799999998,38.624993000000245],[-104.94214399999993,38.587920000000224],[-104.94178299999999,38.55255200000005],[-104.94153299999999,38.51956900000022],[-104.941415,38.50315800000021],[-104.94131199999993,38.499998999999946],[-104.9412989999999,38.47593400000011],[-104.94108499999987,38.46133800000018],[-104.94096100000002,38.45820300000031],[-104.94088599999992,38.43295300000011],[-104.94093299999992,38.38993599999998],[-104.94100400000002,38.37790800000005],[-104.94091300000002,38.37041200000016],[-104.94075099999986,38.338365000000124],[-104.94048299999997,38.28478899999999],[-104.94036599999998,38.25827499999997],[-104.9939819999999,38.258046000000036],[-105.00054099999994,38.25798099999997],[-105.02759400000002,38.25791700000002],[-105.04921499999989,38.25797300000022],[-105.07445999999993,38.258059999999944],[-105.07547199999999,38.257829000000015],[-105.10713299999998,38.25769300000013],[-105.12327999999997,38.25768500000004],[-105.12554399999999,38.25754800000004],[-105.15413999999981,38.25788300000005],[-105.21484399999986,38.25860300000011],[-105.24542799999995,38.25896999999992],[-105.26291400000002,38.25904800000018],[-105.29182999999995,38.258351999999945],[-105.30404499999997,38.258225000000095],[-105.3289309999999,38.2589260000002],[-105.34118999999998,38.258973000000196],[-105.39185399999991,38.25975500000004],[-105.40978099999995,38.26034100000004],[-105.42730699999993,38.26102000000009],[-105.464267,38.26184400000005],[-105.46719399999989,38.26199100000014],[-105.47820899999994,38.26213699999994],[-105.49292700000001,38.26246099999997],[-105.49963300000002,38.262457000000154],[-105.50055500000002,38.26278400000001],[-105.50573199999991,38.26278400000001],[-105.53696300000001,38.263369000000296],[-105.54757699999988,38.26429400000006],[-105.55623400000002,38.26403200000027],[-105.56901699999986,38.264377000000025],[-105.59194699999995,38.264905],[-105.62490099999997,38.264850999999965],[-105.64053899999988,38.26519100000007],[-105.65203200000002,38.26497999999998],[-105.65608899999995,38.265079000000014],[-105.665773,38.26482299999998],[-105.68245499999995,38.26483200000001],[-105.68741799999998,38.26496700000018],[-105.72130300000003,38.265034000000014],[-105.73743099999996,38.264825000000144],[-105.750562,38.26501300000007],[-105.76294999999976,38.26489200000003],[-105.76555199999996,38.26500800000019],[-105.79689699999977,38.26504700000004],[-105.79767099999987,38.26675699999993],[-105.80001300000004,38.26992200000018],[-105.80401499999988,38.27089100000023],[-105.8066419999999,38.270917000000054],[-105.80843500000003,38.271837000000005],[-105.81087400000001,38.27249999999998],[-105.81460399999992,38.27266800000018],[-105.81786799999998,38.27391600000004],[-105.81897299999991,38.27359700000011],[-105.82147399999991,38.27611700000011],[-105.82712300000003,38.27856800000001],[-105.82824399999987,38.281130000000076],[-105.83076899999998,38.28506199999998],[-105.83282500000001,38.28674100000018],[-105.83720199999988,38.288410999999996],[-105.84082999999987,38.29245999999995],[-105.84278,38.29322600000029],[-105.84547900000001,38.29115400000012],[-105.84616900000003,38.29020400000019],[-105.8494189999999,38.29238700000013],[-105.85017599999998,38.29445600000008],[-105.85323299999988,38.29641199999992],[-105.858315,38.29835900000012],[-105.86073799999991,38.302498999999955],[-105.86162299999995,38.303422000000126],[-105.86498099999989,38.30465300000003],[-105.86681699999986,38.3057],[-105.86847399999999,38.30779600000005],[-105.86861199999987,38.30872500000015],[-105.87011199999989,38.309392000000116],[-105.87194399999987,38.31130200000001],[-105.87390999999991,38.31514200000015],[-105.87637599999994,38.3159510000001],[-105.87671799999998,38.31880200000029],[-105.87929199999996,38.320628],[-105.880044,38.323222000000044],[-105.88140399999992,38.324881000000005],[-105.88224699999995,38.327492000000234],[-105.88514199999986,38.32889000000006],[-105.88691999999998,38.33254499999998],[-105.88875999999988,38.33436000000023],[-105.88886599999995,38.33731499999993],[-105.88829099999998,38.33788600000014],[-105.88876600000003,38.3408110000002],[-105.89076499999987,38.34137400000009],[-105.89257399999997,38.34334200000018],[-105.8934779999999,38.346292000000005],[-105.895423,38.34741200000013],[-105.89764699999989,38.35005100000018],[-105.899831,38.35072100000019],[-105.90260199999989,38.3523850000002],[-105.90630699999991,38.35326800000007],[-105.90844699999997,38.35447800000003],[-105.91023599999994,38.35641300000003],[-105.91080399999993,38.35924100000028],[-105.912489,38.361539000000164],[-105.91696899999988,38.36294099999998],[-105.918406,38.36387400000012],[-105.91985099999994,38.36715600000002],[-105.92406999999997,38.3679360000001],[-105.92463799999979,38.368737000000294],[-105.92700999999994,38.36871999999994],[-105.93135999999998,38.369791000000134],[-105.93262400000003,38.37043200000005],[-105.93576999999999,38.37321900000023],[-105.938536,38.37407400000029],[-105.94043299999998,38.376972999999964],[-105.94463699999994,38.37863800000025],[-105.94543199999998,38.38274000000001],[-105.947067,38.38549500000005],[-105.94760600000001,38.39052600000031],[-105.94996899999995,38.39233599999994],[-105.95065399999999,38.39373300000011],[-105.95566799999995,38.394437000000096],[-105.95717199999996,38.395180000000266],[-105.95949499999978,38.39709500000026],[-105.95994499999989,38.39802299999997],[-105.95981999999987,38.40127800000016],[-105.96066899999994,38.40309500000001],[-105.963391,38.404750000000035],[-105.964721,38.40626000000009],[-105.96793300000002,38.408772000000056],[-105.97022600000003,38.41004000000004],[-105.973389,38.409459000000254],[-105.974647,38.41111499999994],[-105.97691899999995,38.41298000000006],[-105.97922199999988,38.41412700000012],[-105.98040899999995,38.41727700000001],[-105.98349999999999,38.419392000000016],[-105.98730799999987,38.41880200000003],[-105.992008,38.41956900000008],[-105.99393199999992,38.41937500000006],[-105.99676799999992,38.420303000000104],[-105.997751,38.42203100000023],[-105.99978799999997,38.42362200000025],[-106.00081399999993,38.425809000000186],[-106.00030300000003,38.4270200000002],[-106.001038,38.42992300000003],[-106.00193000000002,38.431466],[-106.0064769999999,38.43251800000019],[-106.00793399999992,38.43371500000012],[-106.00866200000002,38.43545899999998],[-106.01101599999993,38.437231000000224],[-106.01230699999991,38.4389450000001],[-106.012789,38.44077699999997],[-106.0127,38.44412399999999],[-106.01230399999992,38.44535700000023],[-106.01075100000003,38.446566000000246],[-106.00056899999993,38.45464300000009],[-105.95229499999994,38.49237999999997],[-105.93803999999989,38.50347000000028],[-105.90871700000002,38.505630999999994],[-105.90685399999995,38.50626600000015],[-105.90607399999999,38.50759499999998],[-105.905887,38.509577000000036],[-105.9050539999999,38.51166300000011],[-105.905666,38.51291299999997],[-105.907872,38.51489000000015],[-105.90875900000003,38.516882000000294],[-105.90822999999995,38.520998000000134],[-105.9083399999999,38.522489000000235],[-105.9073459999999,38.52308400000027],[-105.90437299999996,38.52615100000003],[-105.90441099999998,38.52763400000015],[-105.90341699999999,38.52852699999994],[-105.90385200000003,38.53142900000006],[-105.90573299999994,38.532358000000215],[-105.90459399999986,38.534154000000115],[-105.90505000000002,38.53672500000005],[-105.90392899999989,38.53935100000007],[-105.90334899999976,38.54304900000017],[-105.90376000000003,38.547982000000104],[-105.903411,38.548554000000195],[-105.90024499999998,38.54944400000011],[-105.89714100000003,38.55079400000005],[-105.89352699999995,38.55093500000021],[-105.89244999999994,38.55213400000008],[-105.89217899999989,38.554576],[-105.89335999999992,38.55801000000008],[-105.89322500000003,38.55870300000004],[-105.89060999999992,38.561053000000015],[-105.88542899999987,38.5613120000001],[-105.88463899999999,38.562270000000126],[-105.88420199999996,38.56550099999998],[-105.885876,38.56821899999994],[-105.88552400000003,38.569782000000146],[-105.88805000000002,38.571220000000096],[-105.88846899999999,38.571978000000115],[-105.89248999999995,38.573289000000045],[-105.891299,38.57488299999994],[-105.89052599999991,38.57735600000029],[-105.89095299999997,38.579088999999954],[-105.89051699999987,38.581965999999966],[-105.89113800000001,38.583957000000055],[-105.88981999999993,38.58709800000008],[-105.88539000000003,38.590106000000105],[-105.88383899999997,38.59224800000021],[-105.88216699999998,38.59314800000004],[-105.88059599999997,38.59486300000003],[-105.8799049999999,38.59691699999996],[-105.87886699999996,38.598301000000276],[-105.87803299999979,38.60058100000009],[-105.87854499999997,38.60523200000017],[-105.87917599999986,38.60731199999998],[-105.88186399999995,38.61004700000012],[-105.88417500000003,38.61286300000006],[-105.88574699999998,38.6155490000001],[-105.88568099999986,38.6175310000001],[-105.88490999999993,38.61873100000008],[-105.88597199999987,38.62062600000007],[-105.88714900000002,38.62132900000012],[-105.8943579999999,38.623723000000155],[-105.89634799999988,38.622653999999955],[-105.89967899999988,38.62197300000014],[-105.90055499999988,38.62473800000009],[-105.90255100000002,38.62527400000022],[-105.90429499999988,38.62483099999997],[-105.90498999999983,38.62614600000006],[-105.90666799999997,38.627140000000054],[-105.90964599999995,38.62741900000009],[-105.91197499999993,38.62907799999999],[-105.91394599999995,38.629281000000105],[-105.91566499999999,38.63004800000016],[-105.91773899999998,38.633255000000304],[-105.917327,38.63452100000006],[-105.915528,38.635637999999915],[-105.91356399999995,38.63805999999994],[-105.91114999999996,38.639163999999994],[-105.91055099999988,38.64169400000026],[-105.91128900000001,38.64321400000006],[-105.91317400000003,38.64558300000016],[-105.911202,38.64701500000001],[-105.91096199999987,38.64835899999997],[-105.90945899999997,38.64916199999999],[-105.90791200000001,38.65093900000011],[-105.90792399999987,38.65267000000017],[-105.90917200000001,38.65364100000022],[-105.9129299999999,38.65379700000017],[-105.91410399999995,38.656000000000176],[-105.91713500000003,38.65956500000027],[-105.92090299999995,38.65979400000003],[-105.91974299999998,38.66280200000023],[-105.92023499999999,38.664314000000275],[-105.92548399999998,38.666365000000155],[-105.92575099999988,38.669384000000036],[-105.92410499999994,38.670388000000116],[-105.92497199999991,38.67256800000001],[-105.92231999999984,38.67508099999992],[-105.92264099999994,38.67700499999995],[-105.92458499999987,38.67751300000026],[-105.92599000000001,38.67886900000019],[-105.9262129999999,38.67990700000013],[-105.93079699999993,38.680761000000246],[-105.932324,38.68187500000022],[-105.93671399999994,38.68264900000014],[-105.93949299999986,38.68372600000009],[-105.94223599999998,38.6839820000003],[-105.944275,38.68485199999998],[-105.94416699999994,38.68556100000001],[-105.94538599999981,38.688078000000075],[-105.94530899999995,38.69022799999999],[-105.94834299999997,38.691013999999996],[-105.9510019999999,38.69091600000007],[-105.95262000000002,38.6902980000001],[-105.9549179999999,38.69016099999993],[-105.95751899999999,38.690562000000114],[-105.95890199999985,38.691619],[-105.96247299999999,38.69119500000005],[-105.96475099999981,38.692427000000066],[-105.96974999999998,38.69355100000013],[-105.94448799999998,38.6930890000001],[-105.92093799999998,38.693483000000015],[-105.89838999999989,38.69376900000003],[-105.88891199999995,38.695153000000175],[-105.88157399999994,38.694118],[-105.87556699999988,38.6943060000001],[-105.85950599999995,38.69399000000004],[-105.85085900000001,38.693924000000266],[-105.81383900000003,38.69450699999999],[-105.77730099999997,38.69460400000003],[-105.75957599999992,38.694589000000065],[-105.75058000000001,38.69469100000015],[-105.74609599999997,38.69459100000006],[-105.7268479999999,38.6945620000003],[-105.71345199999996,38.694675000000075],[-105.66739699999988,38.69482000000016],[-105.6494209999999,38.69505300000026],[-105.63423499999999,38.69515899999993],[-105.61617899999987,38.69541899999996],[-105.613586,38.69534599999997],[-105.58039200000002,38.695896000000005],[-105.57085699999988,38.69584300000008],[-105.53154699999988,38.69593500000002],[-105.51445799999999,38.69659000000024],[-105.51041599999996,38.69649500000003],[-105.50054799999998,38.696641],[-105.49824599999982,38.69712400000003],[-105.44838799999997,38.69758099999996],[-105.43429000000003,38.69755299999997],[-105.37554499999993,38.69751600000029],[-105.3291339999999,38.69720499999994]]]},"properties":{"OBJECTID":23,"COUNTYFP":"043","Shape_Leng":3.08744012751,"Shape_Area":0.410130851716,"low":10815,"lowmod":18520,"LMMI":26160,"lowmoduniv":36180,"lowmod_pct":0.511885019348,"NAMELSAD10":"Fremont County"}},
{"type":"Feature","geometry":{"type":"Polygon","coordinates":[[[-107.03736300000003,40.09153800000013],[-107.03729499999997,40.05656099999999],[-107.03724799999992,40.053607],[-107.03735899999987,40.00308100000029],[-107.03204999999997,40.00293900000008],[-107.031971,39.983894000000134],[-107.03177199999988,39.963727000000006],[-107.032648,39.96372500000001],[-107.03217099999995,39.93966900000021],[-107.03396799999996,39.91891300000003],[-107.05250199999983,39.91892400000006],[-107.11367199999995,39.919111999999984],[-107.11359599999997,39.90038600000014],[-107.11365799999999,39.89959300000004],[-107.11363999999998,39.87502699999999],[-107.11354799999992,39.86676100000028],[-107.11352399999998,39.805357999999956],[-107.11354999999992,39.782420000000116],[-107.11353400000002,39.75172100000026],[-107.11357199999998,39.73128900000012],[-107.11342899999994,39.69720500000011],[-107.11350299999992,39.64050600000019],[-107.11357099999998,39.63419100000027],[-107.11386900000002,39.62509499999993],[-107.11359899999997,39.59133600000007],[-107.11374499999988,39.5564710000001],[-107.11394699999988,39.50861299999997],[-107.11363299999994,39.499831000000256],[-107.11366299999992,39.46382100000005],[-107.113564,39.454196000000024],[-107.11352399999998,39.4324400000001],[-107.11358799999994,39.423967000000175],[-107.11361899999986,39.3919690000003],[-107.11354899999992,39.391594],[-107.11358399999989,39.37498300000016],[-107.11344600000001,39.3660660000001],[-107.12560400000001,39.366103000000294],[-107.202609,39.36596399999996],[-107.21479499999992,39.36602800000031],[-107.24880100000001,39.366002000000094],[-107.25039400000003,39.366186000000084],[-107.291316,39.36609400000026],[-107.30730599999998,39.366126000000065],[-107.3189099999999,39.365859],[-107.35572400000001,39.36612800000023],[-107.37560799999994,39.36602700000003],[-107.40809799999982,39.36618299999998],[-107.43094899999977,39.366178000000104],[-107.49162699999988,39.36626300000006],[-107.500609,39.36611000000016],[-107.55070699999999,39.36604600000021],[-107.55956899999995,39.366130000000226],[-107.60725699999995,39.36614900000001],[-107.62561299999976,39.36637300000007],[-107.67391800000001,39.36623700000001],[-107.71333799999996,39.366130000000226],[-107.75061599999998,39.36624400000005],[-107.81093099999998,39.366253000000086],[-107.86997499999995,39.36626699999999],[-107.88413799999995,39.36619500000012],[-107.9097819999999,39.36619600000017],[-107.95408500000002,39.366254000000026],[-107.98518499999994,39.36618700000014],[-107.99041,39.36613100000011],[-108.00062699999995,39.366268000000275],[-108.00686999999994,39.366174000000115],[-108.03071499999999,39.366160000000036],[-108.0383139999999,39.36620000000016],[-108.10601500000001,39.366151],[-108.11387399999995,39.366234000000304],[-108.13571400000001,39.36619200000001],[-108.17016899999987,39.366265],[-108.20223699999985,39.3661620000002],[-108.21223199999986,39.36617199999995],[-108.24874599999998,39.36661400000014],[-108.25063699999987,39.366361000000154],[-108.25835999999998,39.3662070000002],[-108.26629899999989,39.36632900000018],[-108.29494999999991,39.36626899999999],[-108.36774000000003,39.366122000000075],[-108.37564299999997,39.36663500000026],[-108.38280199999997,39.36639600000001],[-108.39200499999993,39.36654300000009],[-108.402353,39.366358000000105],[-108.40824299999991,39.3664540000002],[-108.40960699999988,39.36598300000031],[-108.42853799999995,39.36605300000025],[-108.46601599999997,39.365950999999995],[-108.48542999999995,39.36598500000031],[-108.49178499999988,39.36610900000011],[-108.50064700000001,39.366016],[-108.5701709999999,39.366488000000174],[-108.63135099999988,39.366481000000306],[-108.69368800000001,39.366472999999985],[-108.73675700000001,39.36646700000006],[-108.73813399999989,39.36659200000025],[-108.74738899999994,39.366608000000156],[-108.750655,39.36648600000001],[-108.77864799999998,39.36642999999998],[-108.84618599999993,39.36631900000003],[-108.86585299999996,39.36624800000021],[-108.87503500000003,39.36627300000015],[-108.87565899999987,39.36653100000018],[-108.9052749999999,39.36652400000031],[-108.90859999999986,39.366690000000006],[-108.92527899999993,39.36651000000006],[-108.951527,39.366525000000024],[-108.97373500000003,39.36659900000012],[-108.98127099999999,39.36632199999997],[-109.00066299999992,39.36637700000023],[-109.05076499999996,39.36667699999998],[-109.05086399999999,39.37497600000029],[-109.05102799999997,39.41529900000012],[-109.0511719999999,39.45046700000023],[-109.05117799999994,39.485711000000265],[-109.05122599999993,39.497739000000024],[-109.0514,39.5181800000002],[-109.05145900000002,39.54931300000027],[-109.05139899999995,39.56088599999998],[-109.05142399999988,39.58751000000001],[-109.05127799999997,39.60711400000025],[-109.05118899999997,39.65115200000031],[-109.05122699999993,39.66046600000004],[-109.01023099999992,39.660392000000286],[-108.99281400000001,39.66029300000008],[-108.95567599999993,39.66036800000006],[-108.89570399999991,39.66048700000016],[-108.89572800000002,39.650337000000206],[-108.8886599999999,39.65032200000002],[-108.88604199999992,39.650473999999974],[-108.87571899999995,39.650198000000046],[-108.858247,39.65109600000011],[-108.8519649999999,39.65032700000006],[-108.81980399999986,39.65015400000016],[-108.78844600000002,39.65015900000003],[-108.78069399999987,39.65007400000013],[-108.75065599999999,39.65006900000003],[-108.73614299999991,39.64987899999994],[-108.73030499999987,39.64998000000014],[-108.71536299999991,39.649780000000305],[-108.68142499999993,39.64971200000002],[-108.64379400000001,39.64978100000002],[-108.61607899999996,39.64999400000005],[-108.60153699999995,39.64988000000022],[-108.55462699999987,39.650164000000075],[-108.55472199999991,39.693640000000016],[-108.52856799999995,39.693958000000066],[-108.51728899999995,39.69416200000023],[-108.49509199999994,39.693963999999994],[-108.42204099999998,39.694222000000025],[-108.3959129999999,39.69424400000025],[-108.38704799999994,39.694300999999996],[-108.37564399999997,39.69420800000012],[-108.35062399999987,39.69415300000003],[-108.32819499999994,39.694330000000264],[-108.31131099999988,39.69471400000009],[-108.29255699999999,39.69425799999999],[-108.27281299999999,39.69436499999995],[-108.25493699999993,39.69404099999997],[-108.21791199999996,39.69385500000004],[-108.217896,39.695137000000045],[-108.17873900000001,39.695148000000074],[-108.16415799999999,39.69507500000009],[-108.12630999999999,39.695274999999924],[-108.120971,39.695152000000064],[-108.10754099999991,39.69510200000019],[-108.069864,39.69521099999997],[-108.06027399999994,39.69533999999999],[-108.05116099999998,39.695231000000206],[-108.03268800000001,39.69454400000001],[-108.01337699999993,39.69522400000017],[-108.00292699999994,39.69486299999994],[-108.00062899999995,39.69442600000002],[-107.98412999999994,39.69429500000001],[-107.96904399999988,39.69437399999998],[-107.94478499999997,39.6942830000001],[-107.93854599999997,39.69446399999998],[-107.93857099999991,39.70161099999996],[-107.93877199999991,39.70531300000022],[-107.93851599999988,39.73212600000005],[-107.93868199999986,39.74997500000006],[-107.93857999999994,39.79172899999992],[-107.93850199999997,39.824996],[-107.87562600000001,39.824952999999994],[-107.87167299999987,39.824816000000055],[-107.8387699999999,39.82490300000012],[-107.78720599999986,39.825046999999984],[-107.75062199999996,39.827146000000255],[-107.7279069999999,39.82826400000022],[-107.69422899999995,39.83010700000011],[-107.691149,39.82999900000004],[-107.66395,39.8297850000003],[-107.62561700000003,39.8295720000001],[-107.60915499999999,39.829342000000054],[-107.54470199999997,39.82808300000016],[-107.50061199999999,39.828308000000106],[-107.47653199999996,39.828267000000096],[-107.44751799999989,39.82831100000004],[-107.43177000000003,39.82837700000016],[-107.4317479999998,39.85465100000016],[-107.431601,39.874980000000164],[-107.43166499999995,39.91772500000002],[-107.38904000000002,39.916698999999994],[-107.32896199999993,39.9152490000003],[-107.3187509999999,39.91496799999999],[-107.31886099999986,39.93868700000013],[-107.31930599999998,39.999980999999934],[-107.3192469999999,40.003765000000044],[-107.31709499999982,40.003762000000165],[-107.317025,40.059552],[-107.31699300000002,40.090764999999976],[-107.25060999999994,40.090858000000026],[-107.21877899999987,40.090846000000056],[-107.16700300000002,40.09098300000022],[-107.125607,40.09109000000001],[-107.10183599999988,40.09146300000015],[-107.07419299999992,40.09152299999994],[-107.03736300000003,40.09153800000013]]]},"properties":{"OBJECTID":24,"COUNTYFP":"045","Shape_Leng":5.50838909547,"Shape_Area":0.802786705798,"low":12705,"lowmod":22535,"LMMI":33360,"lowmoduniv":56030,"lowmod_pct":0.402195252543,"NAMELSAD10":"Garfield County"}},
{"type":"Feature","geometry":{"type":"Polygon","coordinates":[[[-105.3978489999999,39.912886000000185],[-105.39774899999998,39.90408600000001],[-105.39784800000001,39.895341000000144],[-105.39764899999989,39.883187000000135],[-105.39794899999993,39.881087000000264],[-105.39804499999997,39.873886000000084],[-105.397877,39.866849000000286],[-105.39756799999992,39.866061000000286],[-105.39794899999993,39.86038700000012],[-105.39744999999994,39.853697000000125],[-105.39794799999993,39.84094000000027],[-105.39804900000001,39.81608700000004],[-105.39794899999993,39.814886],[-105.39794899999993,39.74708700000019],[-105.40216499999997,39.7489020000001],[-105.4032299999999,39.750214000000085],[-105.40657599999997,39.75139500000029],[-105.40818799999994,39.751037],[-105.409672,39.74988900000005],[-105.41498199999995,39.749501000000066],[-105.4182669999999,39.74723499999999],[-105.42166900000001,39.74589099999997],[-105.42325800000003,39.74821599999996],[-105.42355199999992,39.749915000000044],[-105.42243499999995,39.7508600000001],[-105.42253399999998,39.753385000000094],[-105.42199399999998,39.75501800000001],[-105.42734999999993,39.755798000000084],[-105.43020000000001,39.75462700000003],[-105.43797799999993,39.755855],[-105.44138999999996,39.755517999999995],[-105.44801499999994,39.75320900000008],[-105.45032800000001,39.75288200000023],[-105.45104200000003,39.75379000000021],[-105.45462899999978,39.755789999999934],[-105.46129799999994,39.755612000000156],[-105.46265399999987,39.75701800000013],[-105.46569,39.756359000000145],[-105.46978100000001,39.75653600000004],[-105.47240799999992,39.757509000000255],[-105.47893499999998,39.75908300000009],[-105.4840299999999,39.759205000000065],[-105.48636499999992,39.75841200000019],[-105.489511,39.75816199999991],[-105.49070999999998,39.757189000000096],[-105.49296700000002,39.75690600000013],[-105.49536699999993,39.75767500000012],[-105.49668300000002,39.759650000000306],[-105.49593400000003,39.76138400000025],[-105.49386099999992,39.76271800000012],[-105.49425299999996,39.76379800000012],[-105.50138900000002,39.76300800000013],[-105.50408299999998,39.76318900000001],[-105.5083899999999,39.76444800000007],[-105.51018799999991,39.76581500000003],[-105.5122209999999,39.76598600000017],[-105.51650899999987,39.76498400000003],[-105.51850100000001,39.76511399999998],[-105.52128399999998,39.766094000000066],[-105.52558899999991,39.768686000000116],[-105.53020800000002,39.769078000000206],[-105.5311329999999,39.76969200000002],[-105.533909,39.77015499999999],[-105.53860499999996,39.77162000000021],[-105.54130799999996,39.773810000000026],[-105.545076,39.77363500000013],[-105.54425499999991,39.77692000000013],[-105.54467499999993,39.777990000000216],[-105.54787099999987,39.781057000000146],[-105.54863399999994,39.782812000000035],[-105.54876199999995,39.784635000000094],[-105.54597999999999,39.78698300000019],[-105.55112899999995,39.79023600000022],[-105.55433999999991,39.792028000000016],[-105.55526599999996,39.79424900000026],[-105.55832199999986,39.795322000000226],[-105.56041099999976,39.79486900000006],[-105.56157000000002,39.79586100000023],[-105.5641,39.80036300000023],[-105.5664119999999,39.8003360000003],[-105.57135299999987,39.80123200000003],[-105.57191999999998,39.802362000000244],[-105.57804999999996,39.80556200000001],[-105.57854099999997,39.81100000000015],[-105.57971599999996,39.812833000000296],[-105.57883599999997,39.81371500000006],[-105.5760709999999,39.81432900000027],[-105.57454199999995,39.81745000000001],[-105.57417799999996,39.81901300000021],[-105.57574399999993,39.82072000000005],[-105.57928099999992,39.822533000000135],[-105.58399599999996,39.82276699999994],[-105.5860689999999,39.82192600000019],[-105.588638,39.82302800000025],[-105.59202099999987,39.822694000000126],[-105.59466399999997,39.82013600000005],[-105.59744999999981,39.818813000000034],[-105.60197199999988,39.8179190000003],[-105.60344999999995,39.82076500000022],[-105.606043,39.82282600000002],[-105.607755,39.823030000000074],[-105.61156299999993,39.82473399999998],[-105.61362999999989,39.82268900000025],[-105.61632800000001,39.82554100000016],[-105.61759799999993,39.826168000000166],[-105.62126499999994,39.82896200000005],[-105.62356899999997,39.82989100000003],[-105.627569,39.832087],[-105.62812400000001,39.83347599999996],[-105.63024100000001,39.83656200000007],[-105.631485,39.83767800000021],[-105.63115099999999,39.839338000000055],[-105.63250299999987,39.841575000000034],[-105.63306499999999,39.84334299999995],[-105.63617599999992,39.84455399999996],[-105.64012600000001,39.84493700000007],[-105.64390200000003,39.84647200000006],[-105.64802299999997,39.846315000000004],[-105.64995999999991,39.84677199999999],[-105.65175599999992,39.84826100000009],[-105.65463799999998,39.84922799999998],[-105.66047099999997,39.85056800000001],[-105.66331100000002,39.850036000000046],[-105.66740500000003,39.84758000000005],[-105.67175199999997,39.84798000000029],[-105.67533999999984,39.84976300000005],[-105.67658599999999,39.84897600000028],[-105.677347,39.84743099999997],[-105.68144499999977,39.84749000000005],[-105.68299399999995,39.84836500000017],[-105.686462,39.849229000000264],[-105.688849,39.851404000000116],[-105.69034799999986,39.85199600000004],[-105.69269899999995,39.854993000000036],[-105.69365599999992,39.85725299999996],[-105.69357699999995,39.858228],[-105.696125,39.86151600000028],[-105.69693799999993,39.86550100000011],[-105.69776899999988,39.86798600000003],[-105.69685099999975,39.8717000000002],[-105.69915399999996,39.87414100000001],[-105.7007309999999,39.87915299999992],[-105.69838300000004,39.88095600000025],[-105.697789,39.88445500000023],[-105.69942500000002,39.88587300000012],[-105.69903799999997,39.88789400000019],[-105.69970699999999,39.88945200000012],[-105.69971900000002,39.89149900000007],[-105.69791299999997,39.89399900000012],[-105.693918,39.89630000000011],[-105.69117900000003,39.897390999999914],[-105.69112999999999,39.89921800000013],[-105.69031499999988,39.901899999999955],[-105.68723299999988,39.904192999999964],[-105.68853199999995,39.90714899999995],[-105.68785899999989,39.911700999999994],[-105.686507,39.912838000000306],[-105.68573500000002,39.91582699999998],[-105.68300099999999,39.91725400000013],[-105.68274399999996,39.92044299999998],[-105.68227200000001,39.92190800000009],[-105.68019400000003,39.92525900000021],[-105.6800799999998,39.92650300000014],[-105.67887999999999,39.92765300000008],[-105.67661399999992,39.92856499999999],[-105.67788099999996,39.93071700000007],[-105.67579799999999,39.93244500000009],[-105.65308699999991,39.93228500000015],[-105.64618299999995,39.93228500000015],[-105.62876999999997,39.9320860000002],[-105.62566800000002,39.93238599999995],[-105.5938559999999,39.93238700000023],[-105.57655499999998,39.932286999999974],[-105.55905499999994,39.932286999999974],[-105.54175399999991,39.93238700000023],[-105.50086599999997,39.932387999999946],[-105.46747099999999,39.93228400000004],[-105.45735099999996,39.932186000000115],[-105.45565099999999,39.93228600000003],[-105.45344699999998,39.93419100000028],[-105.44948299999999,39.93520600000005],[-105.447069,39.93691000000001],[-105.44459299999994,39.93729000000019],[-105.4434159999999,39.9379600000002],[-105.44003399999997,39.93838800000009],[-105.43847199999993,39.937531000000035],[-105.4372939999999,39.93609200000003],[-105.435989,39.93565800000016],[-105.43254999999988,39.93340599999999],[-105.43083100000001,39.931764000000044],[-105.429325,39.93120899999991],[-105.42588599999988,39.9310780000003],[-105.42206299999992,39.931626000000165],[-105.42081399999995,39.930901000000006],[-105.41902399999992,39.92873600000013],[-105.417326,39.930669000000194],[-105.41550599999988,39.931570000000136],[-105.41502699999995,39.93324500000017],[-105.41397099999989,39.93440900000002],[-105.41152499999993,39.932690000000264],[-105.40798599999994,39.93185399999999],[-105.40603899999996,39.93174300000004],[-105.40084300000001,39.935032000000035],[-105.39797099999998,39.93486700000011],[-105.398167,39.92954299999991],[-105.3978489999999,39.912886000000185]]]},"properties":{"OBJECTID":25,"COUNTYFP":"047","Shape_Leng":0.973252023757,"Shape_Area":0.0409605563235,"low":1135,"lowmod":1910,"LMMI":3535,"lowmoduniv":5505,"lowmod_pct":0.346957311535,"NAMELSAD10":"Gilpin County"}},
{"type":"Feature","geometry":{"type":"Polygon","coordinates":[[[-105.653321,40.26045700000003],[-105.651656,40.2585170000001],[-105.65119700000002,40.257257000000266],[-105.65119900000002,40.25424700000008],[-105.64978899999988,40.25287700000024],[-105.64742899999993,40.25136900000018],[-105.64401999999995,40.249939000000154],[-105.64806799999997,40.24907700000006],[-105.650396,40.24937900000003],[-105.658932,40.249413000000004],[-105.6601619999999,40.248351000000014],[-105.66020400000002,40.245177000000126],[-105.66212799999994,40.243224],[-105.66209499999997,40.24252200000001],[-105.66494999999992,40.23855500000025],[-105.665437,40.23726999999997],[-105.66447900000003,40.23510100000027],[-105.66437799999994,40.233213000000205],[-105.66608999999994,40.23146600000007],[-105.66735399999988,40.23095900000004],[-105.66726699999998,40.22981800000014],[-105.66844400000002,40.22814000000005],[-105.66776700000003,40.22671900000023],[-105.66904599999998,40.22417900000022],[-105.66859399999998,40.223285000000146],[-105.66368799999998,40.2203540000001],[-105.66453200000001,40.21981599999998],[-105.6676599999999,40.22037],[-105.67030299999999,40.21935700000006],[-105.67125499999997,40.21707900000001],[-105.6709459999999,40.21460200000024],[-105.67019199999987,40.21360800000025],[-105.67111899999992,40.21216300000003],[-105.67119400000001,40.209692000000246],[-105.67055900000003,40.207802000000186],[-105.6736959999999,40.204421000000025],[-105.67525199999994,40.20321300000012],[-105.67764499999998,40.2023310000003],[-105.67640499999993,40.19950300000011],[-105.67764399999999,40.19721100000015],[-105.67775099999994,40.192984000000024],[-105.67755399999999,40.1885070000003],[-105.67546899999985,40.18639200000007],[-105.67379299999993,40.18574200000012],[-105.67062599999997,40.18539300000003],[-105.67031500000002,40.183417000000134],[-105.67168199999986,40.18200400000029],[-105.67179099999993,40.180028999999934],[-105.670771,40.17848900000007],[-105.66828099999992,40.176914000000124],[-105.66908799999982,40.1762590000003],[-105.6697329999999,40.17425800000029],[-105.66857299999992,40.17357999999996],[-105.66811499999989,40.17085800000018],[-105.66833399999996,40.16961500000002],[-105.66950599999996,40.16862499999996],[-105.66959099999997,40.16748500000017],[-105.67133599999994,40.166497000000106],[-105.67147299999999,40.16542199999998],[-105.67057999999992,40.162691000000166],[-105.66753599999993,40.16118600000016],[-105.66800399999994,40.159886000000256],[-105.66772900000001,40.15827600000006],[-105.66468399999997,40.15674300000006],[-105.66134299999999,40.15652400000005],[-105.66024799999991,40.15473500000002],[-105.65819099999999,40.15399600000001],[-105.65686199999988,40.15293700000029],[-105.6559959999999,40.15128100000004],[-105.65053899999992,40.15057100000013],[-105.64730999999995,40.15080700000027],[-105.64734099999993,40.14921300000003],[-105.64480899999995,40.14807600000006],[-105.64108899999991,40.14754899999997],[-105.63655099999994,40.146375000000205],[-105.63535399999995,40.14556600000003],[-105.63509699999992,40.144190000000094],[-105.63546799999995,40.140417000000184],[-105.6341779999999,40.139125000000206],[-105.63254099999989,40.13843099999997],[-105.630156,40.13809900000024],[-105.62938799999995,40.136150999999984],[-105.63004799999987,40.13458800000001],[-105.62998299999998,40.13043399999998],[-105.62913300000002,40.1291580000003],[-105.62608999999992,40.1273460000001],[-105.62818499999992,40.124986999999976],[-105.628782,40.12351199999995],[-105.62757399999998,40.122062000000255],[-105.62820999999985,40.12035300000025],[-105.62768899999998,40.11843699999997],[-105.629121,40.1170360000001],[-105.62997299999995,40.114157999999975],[-105.63162999999992,40.11348900000007],[-105.63465199999996,40.113232000000096],[-105.63542299999989,40.11117200000018],[-105.63570500000003,40.10830800000019],[-105.63668099999995,40.106993999999986],[-105.63610399999999,40.104741000000274],[-105.63660999999985,40.10253600000027],[-105.63538599999993,40.1004430000001],[-105.63208099999986,40.09673799999996],[-105.63326599999994,40.09545400000013],[-105.63453900000002,40.095485000000224],[-105.6347189999999,40.09384799999998],[-105.63430199999993,40.09016400000013],[-105.63375999999994,40.08837900000009],[-105.63521100000003,40.08718300000004],[-105.63514699999996,40.08497600000004],[-105.6360039999999,40.08455300000003],[-105.63563299999993,40.082871000000125],[-105.63322899999997,40.08118500000006],[-105.6348359999999,40.07921499999998],[-105.6346769999999,40.077035999999964],[-105.63542899999993,40.07494700000001],[-105.63731299999995,40.07445400000023],[-105.63883799999991,40.07340499999992],[-105.641121,40.07299999999998],[-105.640739,40.06987100000009],[-105.63887299999988,40.06703099999993],[-105.64143299999995,40.06788399999999],[-105.64669199999986,40.06522000000001],[-105.64753300000001,40.06428600000021],[-105.64941899999991,40.05999200000019],[-105.65106099999997,40.05844500000006],[-105.64842499999992,40.0539370000003],[-105.6464519999999,40.05346499999996],[-105.64558199999988,40.052660000000174],[-105.64509899999996,40.050670000000025],[-105.64233799999994,40.048120999999924],[-105.63850699999989,40.04673900000017],[-105.63748599999997,40.0458450000001],[-105.63829199999986,40.04432600000018],[-105.63914899999997,40.04046900000009],[-105.6387969999999,40.038787000000184],[-105.64019300000001,40.03725500000007],[-105.64119700000003,40.03417200000007],[-105.64393799999999,40.03330099999994],[-105.64957400000003,40.0277420000001],[-105.65279299999992,40.023772000000235],[-105.65307100000001,40.020425000000046],[-105.65740800000003,40.019528000000264],[-105.66019,40.01834900000023],[-105.661606,40.01646600000004],[-105.66709800000001,40.01631700000013],[-105.67013599999996,40.01687900000013],[-105.67451,40.01618599999995],[-105.67797699999988,40.01454100000012],[-105.680431,40.01401999999996],[-105.68305199999998,40.014141999999936],[-105.68634700000001,40.01509499999992],[-105.6878259999998,40.01597900000024],[-105.68966599999999,40.01554100000004],[-105.69001500000002,40.00931700000024],[-105.688469,40.00694599999997],[-105.68976700000002,40.005955000000085],[-105.69105200000001,40.00376399999999],[-105.69049699999994,40.00086900000008],[-105.69117,39.99952100000007],[-105.69119399999994,39.99731300000019],[-105.69018499999987,39.99593399999992],[-105.687929,39.995062000000075],[-105.68501300000003,39.99451699999992],[-105.68320999999997,39.99465000000009],[-105.68316399999998,39.992881000000295],[-105.68390599999992,39.99163200000015],[-105.685968,39.989742999999976],[-105.68604799999997,39.98747700000007],[-105.68747999999988,39.985418000000266],[-105.68512699999997,39.98310000000015],[-105.68546199999997,39.981387000000154],[-105.68493799999987,39.97933000000012],[-105.68841399999997,39.975426000000084],[-105.68967800000001,39.97100100000023],[-105.69034499999998,39.96985500000005],[-105.68882699999995,39.96776200000028],[-105.68865399999993,39.96643300000028],[-105.689255,39.96473000000003],[-105.68833099999995,39.9628800000001],[-105.68818499999986,39.959663000000205],[-105.68510800000001,39.95874099999992],[-105.68838299999999,39.95653000000016],[-105.68850800000001,39.95483100000007],[-105.68939999999992,39.95350600000023],[-105.69119899999993,39.952991000000054],[-105.69366199999996,39.951153999999974],[-105.69436199999996,39.94968399999999],[-105.69281499999988,39.94816300000008],[-105.69179600000001,39.94624100000004],[-105.69345299999998,39.94401700000026],[-105.69319999999993,39.94123900000011],[-105.69029999999998,39.939525],[-105.68962699999997,39.93863400000021],[-105.68661599999996,39.93746900000008],[-105.68294300000002,39.93378900000005],[-105.67894100000001,39.93286000000029],[-105.67526499999997,39.93332300000003],[-105.67579799999999,39.93244500000009],[-105.67788099999996,39.93071700000007],[-105.67661399999992,39.92856499999999],[-105.67887999999999,39.92765300000008],[-105.6800799999998,39.92650300000014],[-105.68019400000003,39.92525900000021],[-105.68227200000001,39.92190800000009],[-105.68274399999996,39.92044299999998],[-105.68300099999999,39.91725400000013],[-105.68573500000002,39.91582699999998],[-105.686507,39.912838000000306],[-105.68785899999989,39.911700999999994],[-105.68853199999995,39.90714899999995],[-105.68723299999988,39.904192999999964],[-105.69031499999988,39.901899999999955],[-105.69112999999999,39.89921800000013],[-105.69117900000003,39.897390999999914],[-105.693918,39.89630000000011],[-105.69791299999997,39.89399900000012],[-105.69971900000002,39.89149900000007],[-105.69970699999999,39.88945200000012],[-105.69903799999997,39.88789400000019],[-105.69942500000002,39.88587300000012],[-105.697789,39.88445500000023],[-105.69838300000004,39.88095600000025],[-105.7007309999999,39.87915299999992],[-105.69915399999996,39.87414100000001],[-105.69685099999975,39.8717000000002],[-105.69776899999988,39.86798600000003],[-105.69693799999993,39.86550100000011],[-105.696125,39.86151600000028],[-105.69357699999995,39.858228],[-105.69365599999992,39.85725299999996],[-105.69269899999995,39.854993000000036],[-105.69034799999986,39.85199600000004],[-105.69131099999987,39.85133799999994],[-105.69354199999998,39.84785300000027],[-105.69379499999985,39.845475000000135],[-105.69461799999993,39.843711999999925],[-105.69574699999998,39.83948200000009],[-105.69634500000001,39.8389190000002],[-105.70126600000003,39.83712600000018],[-105.70327099999992,39.836712000000205],[-105.7081419999999,39.83672400000012],[-105.71233899999987,39.837710000000186],[-105.713369,39.838212],[-105.71698099999992,39.835472000000095],[-105.71933999999987,39.83105100000006],[-105.71896500000003,39.82896200000005],[-105.719243,39.82754399999999],[-105.72327899999999,39.825353000000234],[-105.724245,39.824073],[-105.72455799999994,39.82198599999998],[-105.72388399999988,39.819332000000145],[-105.72517600000003,39.81732800000009],[-105.73142599999994,39.814319000000125],[-105.73345699999987,39.81254200000001],[-105.73498099999995,39.8102310000001],[-105.73642599999994,39.804801000000055],[-105.73855999999995,39.805015000000196],[-105.74141999999989,39.804489000000046],[-105.74448999999987,39.80337500000002],[-105.747074,39.80132500000025],[-105.74968199999995,39.80094399999996],[-105.750561,39.80130300000002],[-105.75266699999992,39.79994000000022],[-105.75678099999988,39.79822200000018],[-105.76095999999995,39.797101],[-105.76388199999997,39.79498399999994],[-105.76595799999996,39.79485499999993],[-105.76945999999981,39.79570400000006],[-105.77337399999993,39.79723700000022],[-105.77701200000001,39.7981020000002],[-105.77971199999996,39.79905000000031],[-105.78231399999987,39.80117700000005],[-105.78466199999997,39.802434000000176],[-105.78670799999998,39.802474000000075],[-105.79093999999992,39.80310099999991],[-105.79373099999992,39.801944000000105],[-105.79580399999986,39.802919000000145],[-105.79901299999989,39.80343000000016],[-105.80126499999989,39.80145200000027],[-105.80193999999995,39.79933599999998],[-105.80486099999996,39.79903300000012],[-105.8075169999999,39.79805399999998],[-105.80971999999991,39.794962999999996],[-105.81051100000002,39.79147500000005],[-105.81417699999992,39.79029800000001],[-105.81841299999996,39.790013000000044],[-105.81987100000003,39.789491000000226],[-105.821416,39.78806400000008],[-105.82295399999998,39.78560500000003],[-105.82568699999996,39.78685900000005],[-105.82713899999993,39.78699799999998],[-105.82999699999988,39.78821900000031],[-105.83341899999988,39.78788900000001],[-105.83860900000002,39.78813200000002],[-105.84072299999997,39.78893700000003],[-105.84408799999994,39.78951800000016],[-105.84782299999989,39.78919400000001],[-105.84992599999981,39.788564000000065],[-105.85090100000002,39.789090999999985],[-105.85207500000001,39.791127000000074],[-105.85335700000002,39.79227400000008],[-105.85481099999998,39.795590000000004],[-105.85735,39.79578800000024],[-105.8601149999999,39.79694399999994],[-105.86873500000002,39.796628999999996],[-105.87230599999998,39.796692000000064],[-105.87474599999996,39.79903500000012],[-105.88335799999993,39.79800900000015],[-105.88739499999991,39.79699599999998],[-105.88786499999992,39.79582400000004],[-105.88790199999994,39.79013400000014],[-105.88769499999995,39.78902199999999],[-105.89073899999994,39.786384],[-105.89234599999986,39.784108000000174],[-105.89265699999993,39.78263800000002],[-105.89209,39.7808],[-105.89002899999991,39.778639999999996],[-105.88956699999994,39.77567299999998],[-105.88963799999999,39.77365800000007],[-105.88754399999999,39.77047700000031],[-105.88671599999998,39.766002999999955],[-105.8851059999999,39.76415500000019],[-105.88255400000003,39.762989000000175],[-105.88152400000001,39.76051700000028],[-105.88048100000003,39.75930299999999],[-105.88212099999981,39.75576899999999],[-105.88458800000001,39.752859000000285],[-105.88500099999999,39.749986000000035],[-105.88316599999996,39.74763500000023],[-105.88234299999999,39.745579000000134],[-105.88326599999988,39.74380700000006],[-105.88237499999997,39.74032799999998],[-105.883714,39.73701200000022],[-105.88694999999996,39.73417600000022],[-105.88943999999992,39.7315480000002],[-105.89175499999999,39.730387000000064],[-105.89995899999997,39.72878100000008],[-105.90472899999992,39.72877900000026],[-105.90827999999999,39.72624600000029],[-105.90746000000001,39.72404500000005],[-105.90884499999976,39.72294299999999],[-105.91049900000002,39.720713000000046],[-105.91201399999989,39.71615500000007],[-105.91177499999998,39.71529499999997],[-105.91676499999994,39.71357599999999],[-105.91806499999996,39.712084000000004],[-105.92140699999999,39.709936000000084],[-105.92185999999998,39.708124999999995],[-105.9245459999999,39.70342300000016],[-105.92406099999994,39.70157400000011],[-105.92461800000001,39.69897200000014],[-105.92665999999997,39.69868700000018],[-105.9300899999999,39.69751800000006],[-105.93280099999993,39.69468200000006],[-105.93649499999987,39.69256999999993],[-105.94057099999998,39.69060200000018],[-105.94560099999984,39.688866000000075],[-105.95103599999976,39.695149000000015],[-105.96194100000002,39.690781999999956],[-105.97042699999997,39.6905720000002],[-105.97235,39.691689999999994],[-105.97818699999988,39.689762000000144],[-105.98095099999995,39.68976900000018],[-105.98540099999991,39.687376000000256],[-105.98857099999992,39.68700100000012],[-105.99097799999993,39.68752000000006],[-105.99633900000003,39.684122000000116],[-106.00289999999995,39.68583799999999],[-106.00460399999992,39.68734400000028],[-106.00648399999994,39.68795300000022],[-106.00881399999992,39.68777399999999],[-106.01298699999995,39.686341000000084],[-106.01560099999995,39.68711600000023],[-106.01943599999993,39.6875500000001],[-106.02118799999994,39.68840299999994],[-106.02378299999998,39.68842000000012],[-106.02668599999998,39.68952100000007],[-106.02633200000002,39.690751999999975],[-106.03188899999981,39.69417500000003],[-106.03364199999999,39.69676000000004],[-106.03430999999995,39.70295800000008],[-106.03675099999992,39.70552400000008],[-106.03788799999995,39.70757900000012],[-106.03826199999997,39.70959699999992],[-106.03810599999997,39.712058000000184],[-106.04158299999989,39.71602000000007],[-106.04406499999999,39.71767500000004],[-106.04466999999994,39.718924000000186],[-106.04792999999995,39.72001900000021],[-106.04863499999982,39.72107200000028],[-106.05126199999995,39.72148299999998],[-106.05595099999977,39.721404000000064],[-106.056649,39.72301299999992],[-106.05854799999997,39.72470099999998],[-106.05866100000003,39.72627200000011],[-106.05803899999995,39.72881200000012],[-106.06001700000002,39.73317600000007],[-106.05982099999989,39.73610500000012],[-106.06026700000001,39.73801700000024],[-106.05946899999998,39.73943200000025],[-106.05971899999997,39.74062100000003],[-106.06086399999992,39.741369000000304],[-106.06402600000001,39.74592400000023],[-106.0648149999999,39.748201000000165],[-106.06394699999993,39.74891600000001],[-106.06340399999988,39.75074200000006],[-106.064055,39.75218100000012],[-106.06589799999989,39.75379900000024],[-106.06573600000002,39.757558000000074],[-106.06501500000002,39.75855400000006],[-106.06482699999992,39.760733000000016],[-106.06555500000002,39.76262400000002],[-106.06752099999989,39.763934000000006],[-106.06956999999989,39.7662440000002],[-106.07104499999997,39.76974400000006],[-106.07103199999995,39.77267900000027],[-106.07300900000001,39.77713499999999],[-106.07516799999996,39.78096000000011],[-106.07765199999977,39.7828880000003],[-106.07898199999988,39.785356000000036],[-106.08120699999995,39.78739899999999],[-106.07919699999991,39.792266000000154],[-106.07806900000003,39.79341000000011],[-106.07948699999992,39.79415900000009],[-106.08216299999998,39.79633600000028],[-106.08235300000001,39.80070599999999],[-106.08394599999991,39.80332499999997],[-106.083711,39.80596400000002],[-106.08411499999994,39.80665499999998],[-106.08653199999992,39.80799400000012],[-106.08997399999987,39.809336000000144],[-106.09201599999989,39.81287100000009],[-106.09434399999986,39.8146230000001],[-106.09580299999988,39.816201999999976],[-106.09900299999987,39.81842999999992],[-106.09991600000001,39.81980399999992],[-106.10288500000001,39.82084000000003],[-106.10528399999998,39.82238100000018],[-106.10986299999996,39.82745700000004],[-106.11242700000003,39.82970600000016],[-106.11458700000003,39.83038600000009],[-106.11703999999986,39.8319920000003],[-106.12030999999996,39.83567199999993],[-106.12069399999996,39.83739500000024],[-106.1217669999998,39.83897700000006],[-106.12345399999987,39.84034500000024],[-106.12445300000002,39.84179800000021],[-106.12972699999995,39.84400399999993],[-106.131483,39.84420000000017],[-106.13452699999999,39.84367700000024],[-106.13791200000003,39.84353799999991],[-106.13941299999988,39.84301100000005],[-106.14243699999997,39.83996500000006],[-106.14340400000003,39.83934799999997],[-106.14950699999997,39.83956100000006],[-106.15165699999989,39.840687000000116],[-106.15631699999989,39.84220100000016],[-106.16161099999988,39.84366499999999],[-106.1651169999999,39.84382200000016],[-106.16760299999999,39.84554100000008],[-106.16890599999994,39.84502400000014],[-106.17114300000003,39.8452930000002],[-106.1725219999999,39.84639700000008],[-106.17251099999999,39.84791300000006],[-106.17474299999992,39.84794900000003],[-106.17769999999996,39.84925799999996],[-106.18044599999979,39.852456000000075],[-106.18330500000002,39.85352399999999],[-106.18567999999988,39.85542100000009],[-106.19428899999997,39.85968100000014],[-106.195424,39.861084999999946],[-106.20056599999992,39.86404200000004],[-106.20154500000001,39.86546499999997],[-106.20139099999977,39.86852399999998],[-106.2027619999999,39.86975600000011],[-106.20320099999998,39.87125800000007],[-106.20519199999995,39.873167999999964],[-106.206368,39.87498500000004],[-106.20649900000001,39.87770100000023],[-106.207989,39.87839900000006],[-106.21147999999994,39.87844000000007],[-106.2141739999999,39.878801000000294],[-106.21686,39.88063000000028],[-106.21932099999998,39.8836110000002],[-106.21937400000002,39.8853040000003],[-106.22078199999999,39.88768600000003],[-106.2230889999999,39.888304000000005],[-106.2241259999999,39.88996800000001],[-106.22751999999997,39.89250800000002],[-106.228361,39.89351700000003],[-106.22972399999998,39.89656000000008],[-106.23000899999994,39.89799600000026],[-106.23188099999993,39.899895000000015],[-106.23168799999996,39.900806000000046],[-106.234824,39.90270800000002],[-106.23929799999996,39.90681399999994],[-106.24111499999987,39.90726899999993],[-106.2436239999999,39.90929],[-106.2469099999999,39.912700000000086],[-106.2505809999999,39.91304800000006],[-106.252027,39.91471200000012],[-106.2716519999999,39.916685000000086],[-106.289785,39.918851000000075],[-106.29477699999995,39.91892000000013],[-106.294624,39.91396000000003],[-106.32628299999999,39.91374800000028],[-106.33845799999978,39.91370100000029],[-106.34351800000002,39.913798000000156],[-106.36235499999987,39.91373599999997],[-106.36748999999992,39.91382599999997],[-106.375585,39.913636000000054],[-106.38111499999997,39.914016999999944],[-106.38741099999999,39.91391199999998],[-106.39543300000003,39.91401600000006],[-106.39525700000002,39.924589000000196],[-106.4077099999999,39.92476600000009],[-106.43450799999994,39.924914000000115],[-106.44257399999975,39.92480400000022],[-106.44377999999989,39.924948000000086],[-106.47290699999996,39.92510900000025],[-106.48539799999992,39.92504500000007],[-106.48625400000003,39.925250000000176],[-106.50061899999986,39.92498599999999],[-106.52211899999986,39.9250320000001],[-106.52785899999986,39.92516600000016],[-106.53424499999988,39.924946999999975],[-106.55299999999994,39.92491900000016],[-106.562052,39.9248070000001],[-106.57848599999994,39.9249530000003],[-106.6079269999999,39.925085000000024],[-106.61515700000001,39.92529200000013],[-106.61877900000002,39.92522500000024],[-106.62559099999993,39.92467199999999],[-106.62656900000002,39.9247850000001],[-106.62657300000001,39.95199900000006],[-106.62611200000003,39.999986000000035],[-106.62653,40.001870999999994],[-106.63703099999987,40.00210600000008],[-106.63704699999994,40.03635700000001],[-106.63700299999994,40.058168999999964],[-106.63697400000001,40.12357800000029],[-106.63710900000001,40.12498499999998],[-106.6369949999999,40.17898500000007],[-106.6331209999999,40.1789960000001],[-106.63298700000001,40.203864000000124],[-106.63272099999995,40.22994299999999],[-106.63248499999997,40.26697700000017],[-106.632477,40.28355600000003],[-106.63232399999987,40.30975700000005],[-106.63247999999999,40.3111370000002],[-106.63224999999994,40.32318000000009],[-106.6323779999999,40.32877500000029],[-106.63225699999992,40.341559000000075],[-106.64987199999996,40.34163799999999],[-106.65283199999993,40.341562000000295],[-106.65295699999996,40.36440499999992],[-106.652601,40.403019000000086],[-106.65218700000003,40.44520700000021],[-106.65211199999987,40.445231000000035],[-106.6518559999999,40.44214499999998],[-106.65034199999997,40.43978100000015],[-106.6486119999999,40.43854400000009],[-106.643351,40.43604099999993],[-106.63912599999992,40.432920000000195],[-106.638601,40.432244000000026],[-106.63613599999996,40.43200900000011],[-106.63250799999997,40.432565000000295],[-106.62814600000002,40.431107999999995],[-106.62463599999995,40.431164000000194],[-106.62349099999989,40.431729000000075],[-106.62216699999988,40.429763000000094],[-106.6236649999999,40.42958299999992],[-106.62651799999998,40.426955000000135],[-106.6288399999998,40.42217800000009],[-106.62915299999997,40.41892500000006],[-106.62839199999996,40.41762600000004],[-106.62708899999996,40.41731600000014],[-106.62519600000002,40.41500700000006],[-106.62391999999994,40.41090300000013],[-106.62099,40.40801800000003],[-106.61952500000001,40.40524500000009],[-106.61803299999985,40.39892900000007],[-106.61828200000002,40.39528200000012],[-106.61764499999998,40.393197999999984],[-106.61519499999997,40.391009000000054],[-106.61312599999991,40.388716000000045],[-106.61198099999996,40.38683100000026],[-106.611693,40.383248000000265],[-106.61232100000001,40.37938300000002],[-106.60942999999992,40.378288000000055],[-106.60818999999998,40.37973199999993],[-106.60373399999997,40.37833400000028],[-106.60281199999997,40.37768400000016],[-106.6025049999999,40.37608200000011],[-106.601046,40.37580400000002],[-106.59652499999993,40.37885300000005],[-106.59635899999995,40.38117600000004],[-106.59453500000001,40.38142700000003],[-106.59105299999987,40.38058100000006],[-106.58943199999993,40.38180100000011],[-106.58769699999993,40.38143000000014],[-106.58542499999999,40.38226000000003],[-106.58361199999996,40.38102600000008],[-106.58153999999985,40.38067000000029],[-106.58044099999995,40.37981000000019],[-106.57655299999999,40.37855800000017],[-106.56941999999998,40.3785670000002],[-106.56516999999997,40.37786300000022],[-106.56250099999983,40.37839200000013],[-106.559301,40.37807900000013],[-106.55668500000002,40.37727599999994],[-106.55535299999985,40.37423300000006],[-106.55529199999995,40.37203999999997],[-106.55345799999981,40.36986900000011],[-106.55066499999992,40.37043500000004],[-106.54767199999998,40.370170000000144],[-106.54560500000002,40.37076600000012],[-106.54371599999996,40.372064000000194],[-106.53989899999999,40.37134200000014],[-106.53832999999992,40.3698730000001],[-106.53833499999979,40.36864100000031],[-106.53492099999994,40.365080000000034],[-106.53416500000003,40.36583900000028],[-106.53132800000003,40.366622000000234],[-106.52971999999994,40.369113000000084],[-106.52953899999989,40.371784000000105],[-106.527691,40.374982000000045],[-106.52681899999999,40.377211999999986],[-106.5275059999999,40.37819900000011],[-106.52646499999992,40.38055300000002],[-106.52714299999997,40.381182000000024],[-106.52674100000002,40.384085000000084],[-106.52527900000001,40.38580700000006],[-106.52350699999994,40.3870740000001],[-106.52040799999997,40.388455000000135],[-106.52022799999997,40.38971300000031],[-106.51889899999998,40.39102200000008],[-106.515783,40.388383999999974],[-106.51468799999998,40.38833000000028],[-106.51229999999993,40.386753],[-106.51029599999976,40.38927700000011],[-106.51047999999997,40.393386000000135],[-106.509029,40.394120999999984],[-106.50635899999997,40.39694900000006],[-106.50247999999993,40.400400000000104],[-106.50098399999996,40.402596000000074],[-106.498827,40.403594000000055],[-106.49578099999997,40.40358500000002],[-106.49490199999997,40.40687900000006],[-106.49274700000001,40.407976000000076],[-106.49185199999994,40.410045000000025],[-106.4898419999999,40.41157800000002],[-106.48539999999991,40.410122],[-106.48252000000002,40.40842900000007],[-106.47824799999984,40.40480500000007],[-106.4771419999999,40.40253000000013],[-106.47789699999993,40.40104100000002],[-106.477082,40.39971700000007],[-106.47808299999997,40.39755200000019],[-106.47998199999978,40.39480200000003],[-106.47968000000003,40.39194900000007],[-106.48128299999996,40.38943100000006],[-106.4778169999999,40.389251000000286],[-106.47449399999994,40.387984000000074],[-106.473477,40.3868360000003],[-106.47264399999989,40.38481100000007],[-106.47121800000002,40.38343600000019],[-106.4711079999999,40.38227599999993],[-106.47305799999987,40.379961000000094],[-106.47058599999997,40.379034000000104],[-106.46711799999991,40.37919299999993],[-106.46439499999985,40.375912000000255],[-106.46413100000001,40.371870000000115],[-106.46300099999996,40.36869699999994],[-106.46317799999997,40.36699300000021],[-106.46131600000001,40.366570000000195],[-106.45833799999997,40.36652800000007],[-106.45614599999999,40.36333700000006],[-106.45630399999999,40.3606410000001],[-106.45554899999996,40.35798399999999],[-106.45782599999978,40.356117999999924],[-106.457018,40.35451000000012],[-106.45428300000003,40.35429400000004],[-106.45219099999986,40.352969000000144],[-106.450062,40.35095499999994],[-106.44820999999996,40.35031100000015],[-106.44586400000003,40.34811200000013],[-106.44433399999991,40.34768200000008],[-106.44341999999995,40.34653300000019],[-106.439525,40.34879200000023],[-106.43908499999992,40.35015100000004],[-106.4361189999999,40.35035600000015],[-106.43303700000001,40.34956],[-106.43192499999992,40.35074800000007],[-106.42919799999987,40.35137599999996],[-106.42745400000001,40.350814000000014],[-106.42362200000002,40.352525000000014],[-106.42179299999992,40.35279299999996],[-106.4200449999999,40.35221400000023],[-106.41965199999999,40.35086200000006],[-106.41709699999996,40.35008000000022],[-106.41403200000002,40.350945000000024],[-106.41032799999999,40.35015599999991],[-106.40768899999995,40.34642000000008],[-106.40560199999993,40.345965999999976],[-106.40401599999979,40.34495199999992],[-106.40010499999994,40.34565399999997],[-106.395828,40.34701200000006],[-106.39429999999987,40.34602500000011],[-106.390872,40.34565000000015],[-106.38905899999992,40.34611100000012],[-106.38815699999998,40.34832699999993],[-106.38646299999999,40.34958800000004],[-106.385264,40.35237300000023],[-106.38309099999987,40.35289399999999],[-106.38122699999991,40.35279700000012],[-106.37803899999994,40.354926999999975],[-106.37493,40.35539500000016],[-106.37246199999998,40.357031000000006],[-106.37000499999999,40.357474000000025],[-106.36825499999998,40.35639700000013],[-106.366308,40.35592200000025],[-106.36423899999994,40.35645999999997],[-106.36365,40.35754600000013],[-106.360098,40.35835000000003],[-106.35480099999995,40.35753700000009],[-106.35145599999993,40.35583700000012],[-106.35037399999993,40.35311100000018],[-106.34520299999991,40.35005100000012],[-106.34388999999999,40.34653100000003],[-106.34146499999997,40.345499000000075],[-106.3374159999999,40.34265900000014],[-106.3371239999999,40.33973000000009],[-106.334562,40.33894300000014],[-106.32947999999999,40.33914800000002],[-106.32278699999989,40.33783499999993],[-106.32122099999992,40.33732500000002],[-106.31973399999998,40.33518800000013],[-106.31622099999993,40.333565000000135],[-106.31224099999997,40.330183999999974],[-106.30805399999991,40.32931500000001],[-106.30578199999997,40.328463000000056],[-106.30533500000001,40.33082900000005],[-106.30283400000002,40.33026600000005],[-106.29924499999998,40.33055999999999],[-106.29833899999988,40.33240600000016],[-106.29597000000001,40.333091000000024],[-106.29647599999993,40.33528400000006],[-106.29386999999997,40.337625],[-106.29645299999999,40.33899200000013],[-106.29805499999992,40.34038900000007],[-106.29886899999997,40.34285299999999],[-106.29869599999995,40.34483300000022],[-106.29718200000002,40.34692700000011],[-106.29862099999986,40.34995800000007],[-106.297077,40.3505370000002],[-106.29551400000003,40.35213799999997],[-106.29551600000002,40.354242],[-106.29045500000001,40.358371000000034],[-106.28840999999983,40.35803800000002],[-106.28662700000001,40.358853000000295],[-106.28422899999993,40.355789999999956],[-106.28057699999994,40.35495300000002],[-106.27794699999993,40.353723],[-106.27676400000001,40.35493600000001],[-106.27542800000003,40.355203000000074],[-106.27121599999998,40.357282],[-106.26959599999998,40.35756200000003],[-106.26920799999994,40.35929500000026],[-106.26566199999996,40.36095000000006],[-106.261435,40.35810400000031],[-106.26034700000002,40.35819000000015],[-106.25768600000004,40.35517700000025],[-106.25678899999991,40.35239700000011],[-106.25555700000001,40.35092400000025],[-106.2535729999999,40.350325000000055],[-106.25356699999986,40.34662600000007],[-106.25165300000003,40.34366599999993],[-106.25090799999992,40.341783000000135],[-106.25122299999998,40.34076500000026],[-106.247318,40.33918799999998],[-106.24666100000002,40.338230000000294],[-106.24477100000001,40.33732199999997],[-106.24343499999998,40.3356940000001],[-106.24265100000002,40.333685000000116],[-106.24121499999995,40.33168400000011],[-106.2389,40.32981600000028],[-106.23898400000002,40.32795600000003],[-106.23621999999995,40.32574599999998],[-106.2307459999999,40.32448599999998],[-106.230074,40.323381999999924],[-106.22566799999976,40.32516900000019],[-106.22171899999995,40.32579099999998],[-106.22099600000001,40.32757900000013],[-106.21869599999985,40.32879700000001],[-106.21706399999994,40.329079999999976],[-106.21435099999997,40.32799100000011],[-106.21149200000002,40.32766300000014],[-106.20621099999988,40.32789000000025],[-106.20173399999993,40.32645100000002],[-106.19886599999995,40.327107000000126],[-106.1969269999999,40.32561700000031],[-106.1947669999999,40.32646700000009],[-106.19272999999993,40.32585300000011],[-106.18917799999986,40.32653200000016],[-106.18900500000001,40.3276800000001],[-106.1810299999998,40.333424000000036],[-106.17982899999998,40.333991000000026],[-106.180137,40.33601400000026],[-106.17919399999994,40.338264000000265],[-106.17740699999996,40.340565000000026],[-106.17574799999994,40.34150800000009],[-106.17473399999989,40.343183000000124],[-106.17048999999992,40.344006000000036],[-106.168856,40.34354599999995],[-106.16429699999998,40.343328999999926],[-106.162011,40.342612000000145],[-106.160686,40.34050000000002],[-106.15965599999993,40.340483000000006],[-106.15761199999992,40.3389810000001],[-106.15639599999997,40.33704900000009],[-106.15224499999994,40.335359000000096],[-106.14938399999994,40.33196700000008],[-106.14752699999985,40.33208999999994],[-106.14675599999993,40.33139400000027],[-106.1414539999999,40.330054000000246],[-106.13637199999988,40.33029000000005],[-106.13469900000001,40.33543900000029],[-106.13317099999978,40.33651299999997],[-106.13138799999996,40.33663800000005],[-106.12789899999996,40.3380420000002],[-106.12121100000002,40.34252400000031],[-106.12047099999995,40.34379000000007],[-106.116738,40.34585199999998],[-106.114826,40.34630500000014],[-106.10505699999993,40.3501710000001],[-106.10162099999985,40.353840000000105],[-106.09938999999991,40.3544730000001],[-106.09777099999991,40.35284899999999],[-106.09497799999991,40.35114199999998],[-106.09152799999993,40.35009100000025],[-106.08744399999995,40.35037900000009],[-106.08568100000002,40.34718299999997],[-106.07653199999993,40.34995199999992],[-106.07171299999999,40.35232899999994],[-106.06995199999994,40.352693000000045],[-106.06576899999988,40.35435700000005],[-106.06439599999999,40.356601000000296],[-106.06324299999994,40.357242999999926],[-106.06111499999992,40.35741800000005],[-106.05790899999988,40.35702900000018],[-106.05456800000002,40.35763600000013],[-106.05182100000002,40.35929599999997],[-106.0505389999999,40.359454000000085],[-106.046986,40.361864000000196],[-106.04338399999995,40.36289599999998],[-106.04085499999997,40.36477500000018],[-106.0397109999999,40.36665000000005],[-106.03602799999999,40.369099000000006],[-106.03117999999995,40.36826700000012],[-106.02983399999988,40.36843200000004],[-106.02738999999997,40.37086700000009],[-106.02603699999986,40.37085500000018],[-106.02456099999995,40.371777000000066],[-106.01862199999988,40.37329500000027],[-106.01458100000002,40.372863999999936],[-106.0133019999999,40.37108800000027],[-106.01160499999992,40.37027600000005],[-106.01307500000001,40.36797700000005],[-106.01254,40.36548299999998],[-106.01112999999987,40.36395600000009],[-106.01085099999995,40.36189999999999],[-106.00777399999993,40.362476000000015],[-105.99834099999998,40.3624930000002],[-105.99667099999999,40.36380300000002],[-105.99439499999977,40.36456800000008],[-105.99220399999996,40.36418900000018],[-105.98665399999993,40.36458200000004],[-105.9838529999999,40.36313400000029],[-105.98126300000001,40.35963099999998],[-105.97834499999999,40.35802300000017],[-105.9763699999998,40.35828700000002],[-105.9753639999999,40.357617000000005],[-105.97314900000003,40.35469000000012],[-105.97231,40.351561000000004],[-105.96936799999992,40.34955000000025],[-105.96614999999986,40.34952300000015],[-105.96240299999988,40.348050000000114],[-105.95965899999993,40.34814900000015],[-105.959723,40.34938100000011],[-105.9576199999999,40.351692000000014],[-105.95740999999992,40.353007000000105],[-105.95543900000001,40.3548100000001],[-105.95434299999982,40.36029700000006],[-105.95117899999997,40.36175300000025],[-105.94981200000001,40.36294700000008],[-105.94255699999997,40.364340000000084],[-105.94021399999991,40.36581400000017],[-105.93870099999987,40.366058000000294],[-105.93872399999998,40.368684000000144],[-105.93731199999996,40.37194099999999],[-105.94162299999988,40.37412899999998],[-105.94671199999993,40.37724800000012],[-105.94722299999995,40.37806900000021],[-105.94591899999995,40.38144900000009],[-105.94660699999997,40.3850470000001],[-105.943896,40.38454200000001],[-105.9425399999999,40.38492700000012],[-105.94046199999991,40.38452399999994],[-105.93632200000002,40.3861960000001],[-105.93386099999998,40.38850600000012],[-105.93178999999986,40.3890880000003],[-105.93081699999999,40.39023800000024],[-105.92753600000003,40.39102600000024],[-105.92440999999997,40.393162000000075],[-105.92347299999994,40.3964840000001],[-105.92207099999996,40.39812000000029],[-105.92073399999998,40.40156500000023],[-105.91919399999995,40.40132900000009],[-105.91661599999998,40.40186299999999],[-105.91359999999992,40.40161700000027],[-105.91084299999989,40.400513000000046],[-105.90916900000002,40.40076200000004],[-105.90598999999997,40.39965200000006],[-105.90506799999997,40.398815999999954],[-105.90331999999995,40.400751000000014],[-105.90319299999993,40.40265699999998],[-105.90165799999983,40.406490000000076],[-105.90231299999994,40.410258000000056],[-105.90209800000002,40.41290700000002],[-105.89901299999991,40.415211000000056],[-105.89766900000001,40.41772700000024],[-105.898098,40.41898500000008],[-105.89771000000002,40.420948000000294],[-105.897831,40.42358000000013],[-105.89850000000001,40.4242960000002],[-105.89857799999999,40.426806000000056],[-105.90043100000003,40.428075000000035],[-105.90267900000003,40.43034200000022],[-105.90326500000003,40.43276200000025],[-105.90183300000001,40.43477800000011],[-105.90217599999988,40.43565600000005],[-105.90142000000003,40.43787899999995],[-105.89981399999994,40.44076899999999],[-105.8987719999999,40.446206000000075],[-105.89706899999993,40.44878800000021],[-105.89889099999988,40.45025800000013],[-105.89908500000001,40.452517],[-105.89982399999997,40.45450000000011],[-105.89833399999998,40.457397000000014],[-105.89824799999997,40.46036099999992],[-105.89920899999998,40.46575800000028],[-105.89722499999988,40.469205000000045],[-105.89430500000003,40.46954700000026],[-105.89221299999997,40.47414800000007],[-105.89175399999999,40.47666099999998],[-105.88869299999993,40.476413000000264],[-105.88601900000003,40.4773430000003],[-105.88154800000001,40.47586600000011],[-105.87914999999992,40.47709400000031],[-105.874101,40.4784410000002],[-105.87226499999997,40.478296000000284],[-105.87113599999998,40.48006500000008],[-105.86915199999987,40.48098600000009],[-105.86558499999995,40.48137300000013],[-105.86247400000002,40.48022600000007],[-105.86059499999999,40.48203500000005],[-105.85717499999998,40.483271],[-105.85492599999998,40.48625200000009],[-105.847308,40.48393100000004],[-105.84696699999995,40.48279100000008],[-105.84306399999997,40.48299400000013],[-105.84108200000003,40.48361800000026],[-105.83782300000001,40.48303299999998],[-105.83460499999995,40.479869000000235],[-105.83060799999998,40.4795680000002],[-105.82795599999986,40.478289000000245],[-105.82472899999993,40.47764100000006],[-105.82011399999999,40.47597200000001],[-105.81542000000002,40.47533900000002],[-105.812793,40.47585500000008],[-105.81105300000002,40.47577300000006],[-105.8073409999999,40.474527000000194],[-105.80671000000001,40.47308100000026],[-105.80886499999997,40.47107800000009],[-105.81083699999999,40.47053099999994],[-105.80817999999988,40.46886800000004],[-105.80569099999991,40.46650000000005],[-105.8044789999999,40.464224],[-105.80444899999998,40.46296500000028],[-105.8057619999999,40.45688999999999],[-105.80761399999994,40.452262000000076],[-105.8072259999999,40.45066200000019],[-105.80568999999991,40.44788200000005],[-105.80655699999988,40.446201999999914],[-105.81039899999996,40.44323100000014],[-105.8117279999999,40.4428530000003],[-105.81301899999994,40.43916100000018],[-105.81578400000001,40.43743400000028],[-105.81636599999985,40.43551900000028],[-105.81433700000002,40.43133400000005],[-105.814456,40.42852200000027],[-105.81723099999994,40.426488000000006],[-105.8172659999999,40.4243230000003],[-105.81612799999988,40.42353800000018],[-105.81533999999988,40.421486000000016],[-105.811126,40.419399],[-105.80826499999989,40.41495400000031],[-105.80129399999998,40.410174999999924],[-105.80117000000001,40.40842200000003],[-105.80042300000002,40.40721800000006],[-105.79826500000001,40.405835000000025],[-105.79418199999986,40.40188200000017],[-105.79381499999994,40.400800000000174],[-105.79170899999997,40.39972300000005],[-105.79095099999995,40.39864],[-105.79166399999997,40.3943350000003],[-105.79103499999997,40.39278300000012],[-105.79054500000001,40.389639999999986],[-105.79123199999992,40.38786300000004],[-105.79072899999989,40.386262999999985],[-105.7885609999999,40.38428200000004],[-105.78724999999997,40.380872000000124],[-105.78499699999998,40.37971500000015],[-105.78181999999993,40.37699000000009],[-105.78146399999997,40.37555100000003],[-105.78194999999994,40.37375900000029],[-105.78014200000001,40.37210399999998],[-105.77643699999999,40.36954500000013],[-105.77421900000002,40.366759],[-105.77410500000002,40.365233000000046],[-105.77259600000002,40.364411000000075],[-105.76946899999996,40.364597],[-105.76832200000001,40.3620820000001],[-105.76769899999988,40.359763999999984],[-105.76444100000003,40.35926300000028],[-105.76302800000002,40.35711800000007],[-105.75976499999985,40.355778999999984],[-105.75927100000001,40.353307000000086],[-105.75711699999988,40.35165000000012],[-105.75287300000002,40.35084599999999],[-105.75238299999995,40.349093000000266],[-105.74866099999991,40.348086999999964],[-105.74775699999992,40.34750800000006],[-105.74500699999993,40.34815800000018],[-105.74244099999999,40.34926300000012],[-105.74156099999999,40.34816100000006],[-105.73961300000002,40.34710800000016],[-105.73634499999991,40.34661500000004],[-105.736444,40.34490600000004],[-105.73544099999998,40.342984],[-105.73310799999996,40.340774000000295],[-105.72994399999999,40.339624000000185],[-105.72878600000001,40.33793500000007],[-105.72908799999988,40.3352780000003],[-105.727372,40.333979000000284],[-105.72453300000001,40.33270000000016],[-105.72143799999992,40.33201800000006],[-105.71312699999999,40.325567999999976],[-105.70677299999994,40.32505100000003],[-105.70168000000001,40.324309000000255],[-105.70235000000002,40.32264900000007],[-105.70201999999989,40.32171100000005],[-105.69871899999998,40.3207800000003],[-105.6976479999999,40.31827299999998],[-105.70016699999996,40.316364000000135],[-105.70223499999992,40.31261000000006],[-105.70090899999997,40.31071500000007],[-105.697764,40.31093899999996],[-105.69617,40.31051700000023],[-105.69483399999984,40.309323000000006],[-105.69186299999996,40.30937300000011],[-105.69013999999987,40.30864800000012],[-105.6912779999999,40.30616400000014],[-105.69049499999994,40.30442100000016],[-105.68929499999985,40.30347900000004],[-105.68617699999999,40.30307499999998],[-105.68577199999987,40.30255400000004],[-105.68827199999987,40.29733400000015],[-105.68736799999994,40.29373300000009],[-105.68475799999999,40.29238299999997],[-105.681848,40.29215500000009],[-105.6853979999999,40.290079000000276],[-105.68640299999993,40.28891800000014],[-105.68634600000001,40.287485999999944],[-105.685046,40.28640900000005],[-105.68057899999997,40.28446000000008],[-105.68073200000003,40.28292700000003],[-105.679262,40.27891200000005],[-105.68124199999994,40.277495000000044],[-105.68034699999998,40.27466800000025],[-105.68242800000002,40.27280000000002],[-105.67890299999988,40.26986900000003],[-105.67818699999987,40.26812000000024],[-105.67559,40.267267000000004],[-105.672055,40.268101000000115],[-105.66918699999985,40.26756800000004],[-105.6679089999999,40.267718],[-105.66523599999988,40.266894000000036],[-105.66441399999991,40.265630999999985],[-105.66237799999993,40.26412400000004],[-105.66071999999997,40.26422800000012],[-105.65723500000001,40.262886000000094],[-105.653321,40.26045700000003]]]},"properties":{"OBJECTID":26,"COUNTYFP":"049","Shape_Leng":4.13095784259,"Shape_Area":0.51144605436,"low":2670,"lowmod":5775,"LMMI":8985,"lowmoduniv":14120,"lowmod_pct":0.408994334278,"NAMELSAD10":"Grand County"}},
{"type":"Feature","geometry":{"type":"Polygon","coordinates":[[[-107.39448599999997,39.25629900000007],[-107.39219399999985,39.255009000000086],[-107.39177099999995,39.252311000000134],[-107.39042199999989,39.25059099999993],[-107.38639399999994,39.25030300000009],[-107.38438699999978,39.24792000000008],[-107.37969499999997,39.24728900000014],[-107.37801000000002,39.24680500000022],[-107.37597099999977,39.24551800000012],[-107.37583799999987,39.24289100000021],[-107.373829,39.24058100000002],[-107.37189799999993,39.23981600000013],[-107.37149799999997,39.23881000000023],[-107.37572799999992,39.237004000000184],[-107.37781599999994,39.234016000000224],[-107.37931499999996,39.23109099999999],[-107.37797399999994,39.22875699999997],[-107.3785979999999,39.22754900000018],[-107.37783100000001,39.2241130000001],[-107.37683700000002,39.22338000000002],[-107.37684300000001,39.220364000000245],[-107.37800199999987,39.2148600000001],[-107.37987499999986,39.212068000000215],[-107.38062300000001,39.21022900000008],[-107.37947999999994,39.208578999999986],[-107.37839699999995,39.208146000000056],[-107.37845099999998,39.20626600000014],[-107.37709399999989,39.204134000000295],[-107.37765499999983,39.20082200000002],[-107.37677899999994,39.199024000000065],[-107.37560799999994,39.19793100000004],[-107.37303299999996,39.19671299999999],[-107.370116,39.19376000000028],[-107.36689799999994,39.19204099999996],[-107.36511299999995,39.188244000000225],[-107.3635279999998,39.18632400000001],[-107.36120299999999,39.18510900000018],[-107.35964999999993,39.18483399999997],[-107.35870699999998,39.181565000000035],[-107.355503,39.17719300000016],[-107.35411999999997,39.17608700000011],[-107.35365699999994,39.17332399999998],[-107.34823899999992,39.17030799999998],[-107.34282999999994,39.168453],[-107.33707499999991,39.169067000000155],[-107.33613500000001,39.16766800000022],[-107.33414199999987,39.1664430000003],[-107.32826299999999,39.165339000000074],[-107.32484399999998,39.16579600000023],[-107.32177300000001,39.16576800000007],[-107.31874099999987,39.16531300000008],[-107.315785,39.163842000000045],[-107.31416100000001,39.161977999999976],[-107.311939,39.160431000000244],[-107.31132300000002,39.159490000000005],[-107.30763799999994,39.1571340000001],[-107.30517399999991,39.15665200000001],[-107.3042989999999,39.15555100000023],[-107.30039799999997,39.15658200000007],[-107.29628100000002,39.15433300000018],[-107.29568699999999,39.15364999999997],[-107.29536300000001,39.149435000000096],[-107.29440899999997,39.147261000000015],[-107.29383599999989,39.144709000000034],[-107.29212599999988,39.14390900000029],[-107.290708,39.141673000000196],[-107.28819499999997,39.139764000000014],[-107.28768899999994,39.13789500000024],[-107.2865559999999,39.13692300000014],[-107.28591399999988,39.13481299999995],[-107.28629999999993,39.13370000000009],[-107.28550899999976,39.13094600000011],[-107.28358100000003,39.128293000000156],[-107.28372999999993,39.12696200000016],[-107.28482199999996,39.12639300000001],[-107.285372,39.12498600000009],[-107.28273399999995,39.12405400000006],[-107.28303900000003,39.1228760000003],[-107.28186899999986,39.120879000000286],[-107.28153500000002,39.11932000000024],[-107.28249,39.11871800000017],[-107.26751300000001,39.11870699999997],[-107.21625699999993,39.11879099999999],[-107.18032799999997,39.11878899999999],[-107.12560400000001,39.118787],[-107.07925899999987,39.118876000000284],[-107.06631999999996,39.118805000000066],[-107.06317000000001,39.11642500000016],[-107.06292400000001,39.11540100000002],[-107.06019000000003,39.11195800000007],[-107.05539399999986,39.11080900000019],[-107.05323299999992,39.110573000000045],[-107.05174699999998,39.10908899999998],[-107.05076899999989,39.10678299999995],[-107.04859299999998,39.10428300000024],[-107.04475399999995,39.10084499999999],[-107.04072500000001,39.09908400000012],[-107.03632999999991,39.09492799999998],[-107.03173300000003,39.093472000000304],[-107.03007899999989,39.0934290000003],[-107.029266,39.09191300000009],[-107.026229,39.090910000000065],[-107.019565,39.08937000000003],[-107.01796699999994,39.087619000000075],[-107.01525300000003,39.085681000000136],[-107.01248599999997,39.084380000000124],[-107.01093900000001,39.08304800000025],[-107.00928199999998,39.0804830000003],[-107.00570500000003,39.07675400000011],[-107.00425399999989,39.074497000000065],[-107.00146799999993,39.073130000000106],[-107.00060099999996,39.07190400000013],[-106.99695799999989,39.06934799999999],[-106.99424599999998,39.069645000000264],[-106.98897199999993,39.07071200000007],[-106.98682099999996,39.06716200000017],[-106.98627799999997,39.06427500000001],[-106.98639999999995,39.062470000000246],[-106.99084999999991,39.06004100000018],[-106.99105799999995,39.058524999999975],[-106.989326,39.05547899999999],[-106.98975599999989,39.05198000000007],[-106.993248,39.05132200000014],[-106.99683600000003,39.04934300000002],[-106.99781099999996,39.048055000000204],[-106.99772599999994,39.04623500000025],[-106.99967400000003,39.04277200000024],[-107.001622,39.042272999999966],[-106.9993859999999,39.0388760000003],[-106.99689799999999,39.03566700000016],[-106.99614499999996,39.034175000000175],[-106.99144099999995,39.03270900000001],[-106.98801000000003,39.03079700000012],[-106.98198799999994,39.030347000000006],[-106.97952599999996,39.03076200000004],[-106.97762699999998,39.02993199999992],[-106.97378499999996,39.02888400000006],[-106.97114399999992,39.028985999999975],[-106.9680679999999,39.029921000000115],[-106.96690299999995,39.030801000000054],[-106.96430700000002,39.02958100000001],[-106.96394299999992,39.02851299999992],[-106.9652309999999,39.023324000000116],[-106.96413000000001,39.02106099999992],[-106.96502199999992,39.01892800000002],[-106.96469499999995,39.017544999999984],[-106.95810699999993,39.01551200000017],[-106.95519000000002,39.01663100000002],[-106.9509799999999,39.01659999999998],[-106.94833599999998,39.01472000000001],[-106.944638,39.014427000000296],[-106.94070699999992,39.01111600000024],[-106.937815,39.00825500000013],[-106.93576199999995,39.00741800000014],[-106.93506500000001,39.006078000000116],[-106.93284699999998,39.005298000000266],[-106.92839199999997,39.00455900000003],[-106.92332599999986,39.001925000000085],[-106.9215329999999,39.001230000000135],[-106.91787499999992,38.99918400000013],[-106.91411900000003,38.99986499999994],[-106.91007300000001,38.99999100000008],[-106.90763099999992,38.995085000000245],[-106.90819599999998,38.992579000000205],[-106.90272599999997,38.992053],[-106.900328,38.9924910000002],[-106.89589899999999,38.990106000000026],[-106.89313300000003,38.98952500000013],[-106.89119799999997,38.99102200000016],[-106.88689299999993,38.99103100000019],[-106.88435299999992,38.99040600000001],[-106.88369899999998,38.99136499999992],[-106.88158599999997,38.99276000000009],[-106.878041,38.99427500000019],[-106.87761499999982,38.99645799999996],[-106.87681499999991,38.99731400000013],[-106.87663399999985,38.99999100000008],[-106.87867199999988,39.002861999999936],[-106.87808999999987,39.00590900000003],[-106.87648999999982,39.00694000000004],[-106.87052199999994,39.00655000000029],[-106.868968,39.00726800000018],[-106.867752,39.00685900000008],[-106.86515599999996,39.008371000000125],[-106.8616869999999,39.00943900000004],[-106.85982299999995,39.00829399999998],[-106.859128,39.006482000000005],[-106.85954699999985,39.00572900000003],[-106.85833299999996,39.00294200000019],[-106.85682099999991,39.00113100000027],[-106.85694499999994,38.99999100000008],[-106.85483499999998,38.996912000000236],[-106.85288199999997,38.995601000000306],[-106.85103899999996,38.9937240000001],[-106.84969299999989,38.991468999999995],[-106.84897899999999,38.989333000000215],[-106.84689800000001,38.98700700000012],[-106.84386299999989,38.98570500000022],[-106.840848,38.985612000000174],[-106.83967799999999,38.98461299999991],[-106.83591699999994,38.98357900000019],[-106.8354129999999,38.982045000000085],[-106.834269,38.98084399999999],[-106.83206999999987,38.980772000000115],[-106.82960199999985,38.978605000000016],[-106.82724799999977,38.97771999999998],[-106.82579299999992,38.978095999999994],[-106.82362999999992,38.97947599999998],[-106.81936300000001,38.98006400000014],[-106.81692999999996,38.98138300000005],[-106.81527499999999,38.98034999999999],[-106.81244199999998,38.97970600000019],[-106.81068399999998,38.97868199999999],[-106.80827699999998,38.97899200000006],[-106.801874,38.97830300000027],[-106.79968899999994,38.97965000000016],[-106.80024600000002,38.982176000000266],[-106.79742699999991,38.98333100000008],[-106.79659100000003,38.98416600000007],[-106.79460599999993,38.98482500000006],[-106.79068899999993,38.98413100000005],[-106.78764999999993,38.98432900000006],[-106.78398899999996,38.986916000000235],[-106.78221400000001,38.99074300000001],[-106.78257499999995,38.99242400000014],[-106.783592,38.99395100000021],[-106.78372000000002,38.995814000000166],[-106.782082,38.998879999999986],[-106.77811899999995,39.001278000000184],[-106.77501099999989,39.00612500000011],[-106.773551,39.00681300000019],[-106.76972699999988,39.00742800000012],[-106.76859400000001,39.00816900000012],[-106.76817499999987,39.0102310000002],[-106.765266,39.01206999999994],[-106.76241899999997,39.014897000000076],[-106.76160299999987,39.01618400000018],[-106.7619029999999,39.019301000000155],[-106.75736699999993,39.01971000000009],[-106.75508299999996,39.021142999999995],[-106.754547,39.02259300000003],[-106.75134299999985,39.02579400000002],[-106.74701800000003,39.02696700000013],[-106.7452869999999,39.02681800000005],[-106.74496599999998,39.02964600000007],[-106.74584599999997,39.032015],[-106.74661100000003,39.035535000000095],[-106.747183,39.040020000000084],[-106.74935700000003,39.04252800000012],[-106.74662199999995,39.04266300000029],[-106.744642,39.043521000000055],[-106.74214,39.04399700000016],[-106.7406949999999,39.045906],[-106.73897999999997,39.046105000000125],[-106.73713599999996,39.04503600000015],[-106.73460999999986,39.04491500000012],[-106.73208599999992,39.04636499999998],[-106.72949599999993,39.04647700000004],[-106.7245779999999,39.04532300000028],[-106.72266100000002,39.04515700000002],[-106.72129699999994,39.04622800000004],[-106.71721799999995,39.04829699999999],[-106.713325,39.04860100000025],[-106.71031499999998,39.047573000000284],[-106.70939399999997,39.050396999999975],[-106.70785099999995,39.05345799999998],[-106.70567899999992,39.054790000000025],[-106.70304499999986,39.05591700000019],[-106.70209099999988,39.05546700000008],[-106.69865099999998,39.055806000000246],[-106.69496900000001,39.055666999999914],[-106.692996,39.05506500000007],[-106.69154400000002,39.05310300000025],[-106.69084299999992,39.05134900000024],[-106.68796299999985,39.049796000000185],[-106.68614999999994,39.04789700000026],[-106.68563799999976,39.045893000000035],[-106.68445099999997,39.04513500000019],[-106.681465,39.04553300000009],[-106.6790959999999,39.045186000000115],[-106.67649999999998,39.04576400000002],[-106.67405400000001,39.04581099999996],[-106.673182,39.044660000000306],[-106.67085999999995,39.043838000000164],[-106.66807699999993,39.043689000000256],[-106.66543699999988,39.04196000000002],[-106.66285900000003,39.04190100000011],[-106.6589479999999,39.04359000000005],[-106.65549199999992,39.04442000000017],[-106.65266699999995,39.041899000000114],[-106.651927,39.040838000000065],[-106.65222,39.03891300000015],[-106.65027399999985,39.037036000000285],[-106.648234,39.03604500000017],[-106.64684599999998,39.03328500000009],[-106.64652699999988,39.030846999999994],[-106.64709699999997,39.02981100000022],[-106.64988199999999,39.02735599999994],[-106.65004999999996,39.02633300000002],[-106.64895999999993,39.02445500000027],[-106.648392,39.02215199999995],[-106.6444939999999,39.0212930000003],[-106.6441099999999,39.020741999999984],[-106.64115700000002,39.02018800000019],[-106.63833399999993,39.01922300000007],[-106.63788699999998,39.01470400000011],[-106.63324,39.01210500000019],[-106.63004099999989,39.01220999999998],[-106.62971299999992,39.01050600000002],[-106.62763299999995,39.00790200000006],[-106.62474999999989,39.00605300000001],[-106.62417399999993,39.003635000000145],[-106.62301499999995,39.003157000000215],[-106.61931099999998,39.002593000000275],[-106.61755199999993,39.001741000000095],[-106.6157179999999,38.99999200000002],[-106.61395500000003,38.998892000000296],[-106.61350199999976,38.99631799999992],[-106.61159999999995,38.99543400000016],[-106.60769699999997,38.99594100000019],[-106.60309799999993,38.996818000000076],[-106.601114,38.99799900000011],[-106.5992139999999,38.99799400000023],[-106.59550899999988,38.99127699999991],[-106.59415299999995,38.99093600000015],[-106.5904329999999,38.991271000000154],[-106.58680200000003,38.99269400000014],[-106.58559500000001,38.99269700000019],[-106.58276899999993,38.991476000000034],[-106.58093999999994,38.991970000000265],[-106.57952299999994,38.99170200000009],[-106.57716099999999,38.99044400000014],[-106.57472100000001,38.99282099999999],[-106.56852200000003,38.9944780000003],[-106.56612899999999,38.99379200000004],[-106.560427,38.99823200000003],[-106.55910099999988,38.99862400000012],[-106.55685499999987,38.99814600000002],[-106.55421099999995,38.99871100000007],[-106.55218300000001,38.99347600000016],[-106.55016299999994,38.99154500000026],[-106.546762,38.991078000000016],[-106.54426699999999,38.98914400000001],[-106.54214999999999,38.98827700000004],[-106.53605299999998,38.983491000000186],[-106.53201999999999,38.98138600000027],[-106.52838899999995,38.980727000000286],[-106.52628700000002,38.976588000000106],[-106.5246259999999,38.97231600000015],[-106.52442299999979,38.971093000000224],[-106.521861,38.96866599999993],[-106.523147,38.965535000000045],[-106.52405599999997,38.96420499999999],[-106.52584499999995,38.96312400000005],[-106.52851099999981,38.959882999999934],[-106.52628600000003,38.95505200000008],[-106.52312199999989,38.954445000000135],[-106.52200999999991,38.95387500000004],[-106.5198769999999,38.95396900000003],[-106.518284,38.95151699999997],[-106.51670000000001,38.95034700000002],[-106.515738,38.94866400000001],[-106.513195,38.94748700000014],[-106.50807700000001,38.94589000000002],[-106.507162,38.94609200000019],[-106.50531299999994,38.94439999999997],[-106.50583899999975,38.94292000000013],[-106.505584,38.93945700000029],[-106.50437099999999,38.93684899999994],[-106.50269499999996,38.936445000000106],[-106.500583,38.93474800000001],[-106.49875999999989,38.934690000000046],[-106.49625800000001,38.93347000000017],[-106.49385599999994,38.931089000000156],[-106.49071400000003,38.929685000000006],[-106.48993799999994,38.9263160000001],[-106.4873589999998,38.92350700000003],[-106.48565199999996,38.922108000000094],[-106.48211999999995,38.92093000000017],[-106.48090499999995,38.91864299999992],[-106.47491199999996,38.91612299999997],[-106.47220899999996,38.915927000000124],[-106.47004399999997,38.91305300000016],[-106.46748300000002,38.91226000000029],[-106.46503199999995,38.910353999999984],[-106.46194100000002,38.911303000000146],[-106.45781099999999,38.910078000000226],[-106.45380799999998,38.912755000000004],[-106.44995099999977,38.91301200000015],[-106.44800499999997,38.9143620000001],[-106.44606499999998,38.9138660000001],[-106.44214999999986,38.91557400000022],[-106.43933300000003,38.915458],[-106.43717499999997,38.91490600000003],[-106.43651999999997,38.91644699999995],[-106.43486399999989,38.918184],[-106.43433299999992,38.919999000000075],[-106.43206199999997,38.92304300000001],[-106.43026199999997,38.92490900000007],[-106.42744800000003,38.925922000000014],[-106.42495200000002,38.926094999999975],[-106.421831,38.92806800000011],[-106.41935399999983,38.92767700000013],[-106.41822699999994,38.92663000000016],[-106.41824400000002,38.925396000000205],[-106.41717999999992,38.92089800000019],[-106.41562699999992,38.91853000000003],[-106.41427399999998,38.91812900000002],[-106.41316599999988,38.91626800000006],[-106.41209100000003,38.915665000000274],[-106.40760199999994,38.91476200000017],[-106.406071,38.912982],[-106.40497699999992,38.912668999999994],[-106.403208,38.910864999999944],[-106.40212400000001,38.91061700000023],[-106.399497,38.91194300000001],[-106.393959,38.91224900000026],[-106.392991,38.91771800000015],[-106.39234399999987,38.91877500000021],[-106.38899399999997,38.91998100000001],[-106.38691299999999,38.92112099999997],[-106.386257,38.922968999999966],[-106.38345099999998,38.922964000000036],[-106.38299899999998,38.923932000000036],[-106.38083599999999,38.92568499999993],[-106.38035999999994,38.92487100000028],[-106.37698199999988,38.92267400000003],[-106.37716799999998,38.917981000000054],[-106.37574499999994,38.91303900000025],[-106.37622199999998,38.90953900000005],[-106.3734399999999,38.906608000000006],[-106.372211,38.90639699999991],[-106.36748199999988,38.906917000000135],[-106.36577299999988,38.90987100000012],[-106.36154399999998,38.91233900000009],[-106.3570729999999,38.91294500000009],[-106.35303499999992,38.91093599999999],[-106.35150099999993,38.90988800000031],[-106.34733399999993,38.908467000000144],[-106.34524099999987,38.90706699999993],[-106.34404899999993,38.90760599999999],[-106.33812799999993,38.9087200000003],[-106.33498400000002,38.91166599999997],[-106.33105599999982,38.911695000000066],[-106.32609899999994,38.911004000000275],[-106.324115,38.904795999999976],[-106.32471999999996,38.90292900000003],[-106.32288899999992,38.89801900000003],[-106.32086299999986,38.895859999999914],[-106.32055499999996,38.89470800000004],[-106.32131299999998,38.8925990000003],[-106.32470999999993,38.89051499999999],[-106.32603899999987,38.88892700000008],[-106.32645600000001,38.88686800000005],[-106.32644600000003,38.8841010000001],[-106.32750499999992,38.88229700000022],[-106.33052199999997,38.881772000000296],[-106.33248300000002,38.88069600000006],[-106.33406500000001,38.88130899999999],[-106.33524499999987,38.87927000000002],[-106.33677899999998,38.87499400000007],[-106.33584599999995,38.87229000000019],[-106.338819,38.871722999999974],[-106.34085399999998,38.87078300000002],[-106.34267399999993,38.869350999999995],[-106.34478000000001,38.86860500000029],[-106.34698200000003,38.86721300000022],[-106.34924999999993,38.86438600000008],[-106.34897999999998,38.86191000000002],[-106.34890599999989,38.856337999999994],[-106.34955099999996,38.85490300000026],[-106.35269299999987,38.85563400000001],[-106.35689100000002,38.855318000000125],[-106.36056599999989,38.85379999999992],[-106.36430000000001,38.85341900000003],[-106.36537799999996,38.85171900000006],[-106.36537099999993,38.8476060000001],[-106.36445799999996,38.846185000000105],[-106.36644199999989,38.84468100000021],[-106.36835699999989,38.84208700000016],[-106.37012599999997,38.840485000000115],[-106.37020399999994,38.8387190000002],[-106.37457899999998,38.83738600000004],[-106.37601000000001,38.8364160000001],[-106.37881899999991,38.83612500000015],[-106.37901999999991,38.835132000000044],[-106.38282199999992,38.83157900000026],[-106.38358900000003,38.82850400000001],[-106.38363200000003,38.82470500000028],[-106.3878479999999,38.823804000000166],[-106.39126899999991,38.82276000000013],[-106.39502399999992,38.82239299999992],[-106.39752899999996,38.82313400000015],[-106.39902099999995,38.825298000000146],[-106.403505,38.82659699999999],[-106.405284,38.828397999999936],[-106.40698699999996,38.8293340000003],[-106.40977799999996,38.827541999999994],[-106.41188799999992,38.82573599999995],[-106.41626299999996,38.82385000000005],[-106.41533400000003,38.81944200000021],[-106.4150449999999,38.81528200000008],[-106.41602499999999,38.81378700000005],[-106.42055099999993,38.81160899999992],[-106.42101799999995,38.81041399999998],[-106.42243199999996,38.809296000000074],[-106.42271099999977,38.807633000000294],[-106.42066599999987,38.804149000000166],[-106.41858100000002,38.80295400000023],[-106.417911,38.80069100000003],[-106.41505499999994,38.799636000000135],[-106.41436299999998,38.795203000000015],[-106.41288099999991,38.791161000000216],[-106.4130909999999,38.78999900000002],[-106.41451799999993,38.78944500000017],[-106.41658599999988,38.787623000000224],[-106.41663499999993,38.785196000000155],[-106.414737,38.780186000000015],[-106.413816,38.779093000000046],[-106.41549800000001,38.7744170000002],[-106.41549800000001,38.7724410000003],[-106.41745200000003,38.77047400000026],[-106.41805799999997,38.76827000000014],[-106.4192589999999,38.766000000000076],[-106.41714299999995,38.764435000000105],[-106.41626499999995,38.76172800000012],[-106.41614899999996,38.7592370000001],[-106.41509899999994,38.757094999999936],[-106.41728,38.75542600000006],[-106.41995299999985,38.751529000000005],[-106.4196369999998,38.7499950000003],[-106.41840099999985,38.749149999999986],[-106.41801099999992,38.74202800000012],[-106.418658,38.73963200000014],[-106.42004999999995,38.73880500000007],[-106.41849300000001,38.735303999999985],[-106.41675199999992,38.73375599999997],[-106.41666499999985,38.727814000000194],[-106.41743099999997,38.72567500000031],[-106.41918499999997,38.723989000000245],[-106.42346699999979,38.72084700000016],[-106.42278599999986,38.718857000000185],[-106.42322199999995,38.71616600000027],[-106.4247089999999,38.712693],[-106.425275,38.710307000000114],[-106.42946799999993,38.70981100000006],[-106.43165599999986,38.708708000000115],[-106.43458799999979,38.70927000000006],[-106.43814199999997,38.70863099999997],[-106.44349299999988,38.709231000000045],[-106.44468,38.70803700000022],[-106.44455299999998,38.706926000000294],[-106.4455299999999,38.705532999999946],[-106.44831299999998,38.704927999999995],[-106.45013499999999,38.702660000000094],[-106.45212599999996,38.69877700000018],[-106.451054,38.695364999999924],[-106.44835799999998,38.692945000000066],[-106.44618300000002,38.68983200000008],[-106.44595899999996,38.68898800000005],[-106.44017599999995,38.68687],[-106.437861,38.68250000000006],[-106.43703199999987,38.679374999999936],[-106.43705499999999,38.67806900000011],[-106.43807200000003,38.67663500000009],[-106.43617899999998,38.673735000000306],[-106.43590699999987,38.671150000000296],[-106.4340899999998,38.66887800000006],[-106.43220899999977,38.66792099999992],[-106.43113999999997,38.66506600000014],[-106.43131699999986,38.66328900000002],[-106.42908799999992,38.65762000000029],[-106.430159,38.65346799999992],[-106.4295449999999,38.65123500000027],[-106.42713899999995,38.65067299999998],[-106.424037,38.648845999999935],[-106.42180399999995,38.6467890000003],[-106.42017899999996,38.64670000000001],[-106.41722299999992,38.6449190000003],[-106.41467599999993,38.64540699999998],[-106.41175199999986,38.64702900000026],[-106.40794399999999,38.64767900000021],[-106.40418399999993,38.64673200000004],[-106.4017189999999,38.645782999999994],[-106.40150699999987,38.64307600000001],[-106.40223799999995,38.64216299999998],[-106.40118899999993,38.639702000000284],[-106.39846799999987,38.639748],[-106.39726699999994,38.640297000000146],[-106.39282499999996,38.640085],[-106.39016399999997,38.64103399999999],[-106.38898999999998,38.64089700000022],[-106.38663200000002,38.63965600000006],[-106.38578799999999,38.638627000000156],[-106.38316399999997,38.638922000000036],[-106.37989899999991,38.63848299999995],[-106.38036199999993,38.637057000000084],[-106.38191899999998,38.63541500000014],[-106.38417700000002,38.633827],[-106.385174,38.632362000000114],[-106.38626899999991,38.627782999999965],[-106.38536799999997,38.62499500000024],[-106.384208,38.62386900000013],[-106.38264100000004,38.623416000000134],[-106.38212499999997,38.62171900000004],[-106.37914899999993,38.620118000000105],[-106.37557999999996,38.621223999999984],[-106.369126,38.61867800000016],[-106.36525099999994,38.61589900000007],[-106.36384299999997,38.61362500000007],[-106.36364600000002,38.61022400000002],[-106.36469699999992,38.60588899999999],[-106.36260800000002,38.603493000000014],[-106.362052,38.602346000000296],[-106.3598219999999,38.60060499999997],[-106.35931700000003,38.59939300000002],[-106.36123499999991,38.595879999999966],[-106.36102399999987,38.594487000000186],[-106.35818799999987,38.59271300000012],[-106.35724199999993,38.5910370000002],[-106.35595099999989,38.59010000000029],[-106.35544900000002,38.5879360000003],[-106.353139,38.58517999999998],[-106.35129999999998,38.58348999999998],[-106.35341999999997,38.58146700000009],[-106.35197499999998,38.57883700000008],[-106.35167299999995,38.576905000000124],[-106.34987699999994,38.57476400000024],[-106.34738299999998,38.572598000000255],[-106.34461899999991,38.568564000000265],[-106.34686699999992,38.56510400000013],[-106.34668099999999,38.56278300000014],[-106.34745099999992,38.561041000000046],[-106.34664099999998,38.558864000000085],[-106.350144,38.55558300000001],[-106.35082999999992,38.55377699999997],[-106.35054299999996,38.552667000000156],[-106.35203399999989,38.55169100000023],[-106.35353299999991,38.54997300000002],[-106.35358399999996,38.54713300000003],[-106.35496399999982,38.54286800000028],[-106.35370599999993,38.53922000000006],[-106.35454299999992,38.53614600000009],[-106.35418500000003,38.53338400000001],[-106.35042299999998,38.53145000000006],[-106.34713099999993,38.528761000000145],[-106.34640799999988,38.52625300000028],[-106.344063,38.52258499999999],[-106.34259699999996,38.52153699999997],[-106.34370299999989,38.51746000000014],[-106.34504399999992,38.51667100000009],[-106.34581300000002,38.51496000000026],[-106.34810800000002,38.51249900000005],[-106.34659999999997,38.50966100000022],[-106.3454549999999,38.50823000000014],[-106.34542499999998,38.50713400000001],[-106.34422999999987,38.504590000000064],[-106.3428239999999,38.50288000000012],[-106.34135900000001,38.502411999999936],[-106.33875999999998,38.50056999999998],[-106.3388789999999,38.49926000000028],[-106.33630799999997,38.49665000000016],[-106.33246499999996,38.494530000000054],[-106.3301689999999,38.493690000000015],[-106.32784399999991,38.49732900000021],[-106.32302399999998,38.496943999999985],[-106.32052899999996,38.497645000000034],[-106.31831999999997,38.49515500000007],[-106.31554599999976,38.4944770000003],[-106.31138099999987,38.49465200000003],[-106.31053500000002,38.49122800000009],[-106.3096329999999,38.4905610000003],[-106.30929700000002,38.48796300000009],[-106.30824999999987,38.48586799999998],[-106.30680899999993,38.48414100000025],[-106.30648399999995,38.48234099999996],[-106.30412100000001,38.480052000000114],[-106.3045469999999,38.478707999999926],[-106.3030149999999,38.4769570000002],[-106.29981999999995,38.47571800000003],[-106.29559799999993,38.47558600000008],[-106.29299099999997,38.47480200000024],[-106.29054300000001,38.47352799999999],[-106.29049399999997,38.471802000000025],[-106.28966700000001,38.47008799999992],[-106.29002599999995,38.46685700000012],[-106.28913799999987,38.46432800000031],[-106.28928099999996,38.46275600000007],[-106.28853199999998,38.46137300000004],[-106.28380700000002,38.45679400000006],[-106.28060899999991,38.45433700000018],[-106.27724599999982,38.45428200000026],[-106.27265499999993,38.455196],[-106.27178999999995,38.45244200000025],[-106.27224699999994,38.44877600000012],[-106.27296799999999,38.447630000000174],[-106.27215199999989,38.44638900000018],[-106.2704839999999,38.445520000000215],[-106.26855499999999,38.443543000000034],[-106.2660709999999,38.44173500000022],[-106.26133299999992,38.44235500000002],[-106.25308699999994,38.44100300000002],[-106.25010299999997,38.43955600000004],[-106.245903,38.43930300000005],[-106.24814099999998,38.43602900000025],[-106.24948399999988,38.43324400000006],[-106.24837000000002,38.42863999999997],[-106.24861799999991,38.42604299999999],[-106.24790999999976,38.42395900000008],[-106.24694,38.42276800000013],[-106.25059499999998,38.42291200000011],[-106.319279,38.4233430000001],[-106.37561699999992,38.423877000000005],[-106.38664599999993,38.42367800000011],[-106.39327499999996,38.42370699999998],[-106.43907200000001,38.4240880000001],[-106.48190599999981,38.42421500000029],[-106.500585,38.42440199999993],[-106.56912999999997,38.42444800000021],[-106.61936300000002,38.42447200000004],[-106.62557700000002,38.42435499999999],[-106.65282000000002,38.42446100000001],[-106.72389599999997,38.42487299999999],[-106.76247999999987,38.42507199999994],[-106.80380699999995,38.42513500000001],[-106.83081500000003,38.425144000000046],[-106.85174299999994,38.425297000000285],[-106.8715259999999,38.42526299999997],[-106.8756019999999,38.425166000000274],[-106.94782199999992,38.42529500000029],[-106.98254999999989,38.425357000000076],[-107.00060099999996,38.4256190000001],[-107.00060199999996,38.384372999999925],[-107.00060199999996,38.33201400000013],[-107.00069400000001,38.29452700000013],[-107.00078999999988,38.24999400000013],[-107.00096999999994,38.23395900000014],[-107.00112899999988,38.20577700000018],[-107.001149,38.15394500000025],[-107.00111900000002,38.14707199999998],[-107.01365099999998,38.14702800000009],[-107.03101399999997,38.14705300000003],[-107.04426000000001,38.14707000000004],[-107.04668100000004,38.147073999999975],[-107.05104699999998,38.146899000000076],[-107.05569799999995,38.14707599999997],[-107.05830000000003,38.147123000000136],[-107.1056329999999,38.14688899999999],[-107.10884899999996,38.14689999999996],[-107.11072399999995,38.146906000000115],[-107.112233,38.146912999999984],[-107.125607,38.1471430000002],[-107.13427100000001,38.14714800000007],[-107.14530200000002,38.14715500000011],[-107.17379299999999,38.14717400000012],[-107.18953499999998,38.14718499999992],[-107.19812000000002,38.14719100000008],[-107.19901599999997,38.14719199999996],[-107.24666599999995,38.14722500000005],[-107.272536,38.14715600000005],[-107.29148599999996,38.147130000000004],[-107.29167799999993,38.147130000000004],[-107.29177199999998,38.147130000000004],[-107.296942,38.14712200000008],[-107.29728699999998,38.14712099999997],[-107.30233099999987,38.1471140000001],[-107.30372699999998,38.14711200000011],[-107.31063499999993,38.14710400000001],[-107.31838599999998,38.14709400000004],[-107.32304699999997,38.147087],[-107.32962999999995,38.147079000000076],[-107.33741699999996,38.14713300000005],[-107.35183899999998,38.14723300000003],[-107.35190499999993,38.14723300000003],[-107.37561299999999,38.14709000000005],[-107.38060199999995,38.147159999999985],[-107.383783,38.14720400000027],[-107.41937799999994,38.14716199999998],[-107.42458399999992,38.14715600000005],[-107.44541099999992,38.147131000000115],[-107.45374099999992,38.14712099999997],[-107.46709799999991,38.14710599999995],[-107.48339999999996,38.147087],[-107.48708599999986,38.14708300000001],[-107.50055899999995,38.14706000000007],[-107.53857699999992,38.147114999999985],[-107.55218000000002,38.14713500000005],[-107.553541,38.147131000000115],[-107.55762700000002,38.14712099999997],[-107.55898999999994,38.147118999999975],[-107.56021399999997,38.14711699999998],[-107.56862000000001,38.147267],[-107.56861399999997,38.14877000000001],[-107.56473299999993,38.14982199999997],[-107.56320799999986,38.15289700000022],[-107.56291799999997,38.15562700000015],[-107.56218899999993,38.158792000000005],[-107.56329800000003,38.162120000000016],[-107.56429200000002,38.16666399999997],[-107.56631099999993,38.168728000000044],[-107.56652299999996,38.170741000000305],[-107.567993,38.17146800000006],[-107.56656599999997,38.17664500000012],[-107.56804399999999,38.179538000000036],[-107.5680109999999,38.18081100000006],[-107.57025799999991,38.181353000000115],[-107.57079599999986,38.18396300000006],[-107.57189599999992,38.18611400000003],[-107.57246499999991,38.188852999999995],[-107.57207499999998,38.19014099999998],[-107.57261199999999,38.194064000000026],[-107.57350000000002,38.194773999999995],[-107.573283,38.196175000000096],[-107.57432499999999,38.196772000000294],[-107.574364,38.19835900000015],[-107.57535299999995,38.19947100000013],[-107.57583999999991,38.202725999999984],[-107.57508000000001,38.205625000000225],[-107.57557600000001,38.209797000000265],[-107.57739399999991,38.2104350000003],[-107.57871,38.21200699999997],[-107.58097799999996,38.212685000000306],[-107.57996500000002,38.214125999999965],[-107.57994299999996,38.215930000000014],[-107.58087,38.218090000000075],[-107.57953900000001,38.22028800000004],[-107.57901499999991,38.22266300000024],[-107.58037200000001,38.22496799999999],[-107.57986799999986,38.22785099999999],[-107.580285,38.23037199999999],[-107.58211299999999,38.23113900000004],[-107.58268199999992,38.23460299999999],[-107.58490799999998,38.23625500000003],[-107.58747999999997,38.240228],[-107.58664999999996,38.243029999999976],[-107.588619,38.24676099999999],[-107.588821,38.24958800000013],[-107.59070099999985,38.25204600000012],[-107.593951,38.25352300000009],[-107.5974139999999,38.253394999999955],[-107.60015900000002,38.25389499999994],[-107.60206,38.25549600000011],[-107.60699,38.25786400000027],[-107.61157000000003,38.26082700000029],[-107.61271699999998,38.26216000000011],[-107.61270899999994,38.26358700000003],[-107.61422099999999,38.26714599999997],[-107.61535500000002,38.26897100000002],[-107.61472499999985,38.270106999999996],[-107.61705099999995,38.27250700000002],[-107.61713699999996,38.27545000000015],[-107.619937,38.27527199999997],[-107.62316499999991,38.279028000000096],[-107.62447700000001,38.27971100000008],[-107.625269,38.28303200000005],[-107.62655699999999,38.28596999999996],[-107.62923599999988,38.287861000000305],[-107.62938500000001,38.29032700000005],[-107.63008299999996,38.29229200000009],[-107.62962499999998,38.29415700000004],[-107.631464,38.29563200000024],[-107.63169899999991,38.297259000000054],[-107.63398599999977,38.30003100000005],[-107.63503999999995,38.301896],[-107.60407499999985,38.30190500000003],[-107.57516099999998,38.30195900000024],[-107.5584409999999,38.30192900000009],[-107.50001999999995,38.30192599999998],[-107.5001279999999,38.36562100000003],[-107.50030099999992,38.380843000000084],[-107.5003779999999,38.40419500000007],[-107.50040200000001,38.43265299999996],[-107.50061299999987,38.43318699999992],[-107.50061299999987,38.476414000000034],[-107.50061399999987,38.5363330000003],[-107.50061199999999,38.608052999999984],[-107.50061099999999,38.66515199999998],[-107.500655,38.66855300000003],[-107.50061099999999,38.69829599999997],[-107.50063399999993,38.74806000000024],[-107.50060199999996,38.78349700000007],[-107.50063799999981,38.81245000000024],[-107.50059099999993,38.841075000000274],[-107.50062400000002,38.86979500000001],[-107.50055299999997,38.89911600000022],[-107.50063899999998,38.91343100000017],[-107.50063199999994,38.95639100000005],[-107.500607,39.0101590000001],[-107.50061499999987,39.064982999999984],[-107.50067999999993,39.12169400000005],[-107.50058599999988,39.155143000000066],[-107.50063699999993,39.169308],[-107.500606,39.217916],[-107.49755599999997,39.21746300000001],[-107.49454899999989,39.21842300000003],[-107.492819,39.221466000000305],[-107.48775299999988,39.22560900000025],[-107.48581299999995,39.22911300000027],[-107.48576699999995,39.23130700000007],[-107.48503499999987,39.23213700000019],[-107.48439299999995,39.23521699999992],[-107.48287700000003,39.2356200000001],[-107.48050799999987,39.23389499999996],[-107.47874100000001,39.23355099999992],[-107.47743699999995,39.230460000000164],[-107.47470099999998,39.22904600000004],[-107.47131699999994,39.2292030000001],[-107.46903499999996,39.22835599999996],[-107.46571599999987,39.228649000000246],[-107.46348599999993,39.22841399999999],[-107.4607729999999,39.230313999999964],[-107.45778599999994,39.22940200000005],[-107.45442300000002,39.23024299999997],[-107.45107499999995,39.230673000000024],[-107.44978499999996,39.23225100000002],[-107.44639499999988,39.233262000000195],[-107.44374699999997,39.23478100000028],[-107.44284199999998,39.23646800000006],[-107.43951500000003,39.23694599999999],[-107.43672900000001,39.238297000000046],[-107.43581099999989,39.23914600000012],[-107.43187699999982,39.23992000000004],[-107.43026099999997,39.24054100000012],[-107.42920000000004,39.24223000000006],[-107.43054399999994,39.24664300000012],[-107.43041599999992,39.248999000000026],[-107.42940299999987,39.250588999999934],[-107.43010400000003,39.25254100000018],[-107.42988700000001,39.255326000000025],[-107.430521,39.25650500000006],[-107.4295839999998,39.25728600000002],[-107.42691399999995,39.25772999999998],[-107.42438099999998,39.257596999999976],[-107.42440199999987,39.25649200000004],[-107.42194699999988,39.25557500000002],[-107.41968199999997,39.255334000000175],[-107.41814599999998,39.25584200000009],[-107.4148229999999,39.25559599999997],[-107.41150099999993,39.254277000000286],[-107.40931599999999,39.25444600000003],[-107.40716999999978,39.25341200000014],[-107.403279,39.253850000000114],[-107.40029699999991,39.25294600000029],[-107.39791600000001,39.25373000000013],[-107.39448599999997,39.25629900000007]]]},"properties":{"OBJECTID":27,"COUNTYFP":"051","Shape_Leng":5.35340385086,"Shape_Area":0.873849911695,"low":4569,"lowmod":6820,"LMMI":9545,"lowmoduniv":14700,"lowmod_pct":0.463945578231,"NAMELSAD10":"Gunnison County"}},
{"type":"Feature","geometry":{"type":"Polygon","coordinates":[[[-107.59136699999999,37.99225200000001],[-107.58746799999994,37.99344000000002],[-107.58666,37.99412799999993],[-107.58658999999989,37.997176999999965],[-107.58667899999995,37.99824999999993],[-107.58692999999994,37.99896800000005],[-107.58419299999997,38.00011100000012],[-107.58380599999992,38.00013300000012],[-107.58268199999992,38.00020500000005],[-107.582111,38.000221000000124],[-107.58148799999998,38.000255000000095],[-107.58095500000002,38.000375000000076],[-107.580421,38.00054800000004],[-107.57993299999993,38.00092999999998],[-107.57805799999994,38.00226400000008],[-107.57747799999993,38.00265200000001],[-107.57681599999995,38.00309600000014],[-107.57576799999993,38.003898000000106],[-107.57465399999995,38.00448200000011],[-107.57430599999992,38.00499400000001],[-107.57432599999999,38.00592400000005],[-107.57426899999996,38.006190000000004],[-107.572904,38.00614000000013],[-107.56793799999991,38.005203999999935],[-107.56600100000003,38.00519500000013],[-107.56546700000001,38.004898000000026],[-107.56389799999994,38.00391599999995],[-107.56286899999998,38.003417999999954],[-107.561466,38.00277700000004],[-107.559842,38.00111600000014],[-107.55965799999996,38.00098400000002],[-107.55724599999996,37.99947199999997],[-107.55650600000001,37.99916200000007],[-107.555792,37.9991040000001],[-107.55361999999997,37.999348],[-107.55058600000001,37.998356],[-107.548588,37.998852],[-107.54717299999999,37.99999500000007],[-107.54631699999993,38.000826000000075],[-107.54607699999997,38.00149399999992],[-107.54538300000002,38.00289100000009],[-107.5450259999999,38.00334700000013],[-107.54376200000002,38.00510800000001],[-107.54274499999997,38.00635600000004],[-107.54070599999994,38.00723400000004],[-107.53938499999998,38.007919999999956],[-107.53873299999992,38.00833899999998],[-107.53787799999998,38.009596999999985],[-107.53806099999986,38.01098100000013],[-107.53791799999999,38.01230500000008],[-107.53827899999993,38.01392600000008],[-107.539353,38.014707000000044],[-107.539556,38.01607599999994],[-107.53942699999999,38.01771500000007],[-107.53871700000002,38.01888500000001],[-107.53756599999991,38.01970199999994],[-107.53699699999993,38.021399999999915],[-107.53658899999999,38.0227440000001],[-107.53737799999999,38.02436299999994],[-107.53824599999996,38.02479299999999],[-107.53940299999994,38.02594000000005],[-107.53969599999994,38.0269540000001],[-107.538882,38.028235999999936],[-107.53685200000001,38.02868600000005],[-107.53694899999994,38.031112000000064],[-107.53747900000002,38.03204800000009],[-107.53750199999996,38.03215199999994],[-107.5372799999999,38.03270800000013],[-107.53674999999998,38.03316200000006],[-107.535055,38.034101000000135],[-107.53449799999993,38.03439300000014],[-107.53349000000003,38.03564000000006],[-107.53179599999999,38.036992000000055],[-107.53112899999996,38.038963000000024],[-107.53055699999999,38.03898799999996],[-107.52702699999986,38.0398780000001],[-107.52576299999998,38.039353000000006],[-107.52449399999989,38.03953799999999],[-107.52298000000002,38.041045],[-107.52223200000003,38.041930000000036],[-107.52195499999993,38.04256700000002],[-107.522152,38.04376600000006],[-107.52233699999994,38.04416400000014],[-107.52158199999991,38.045067000000074],[-107.52077400000002,38.0455740000001],[-107.520985,38.046627],[-107.52262699999994,38.047957999999994],[-107.52256599999998,38.04840199999995],[-107.52220799999992,38.04933399999999],[-107.52180499999997,38.05044399999997],[-107.521071,38.05218000000008],[-107.521728,38.05334399999998],[-107.52245199999993,38.05471100000011],[-107.522899,38.05613200000005],[-107.52182399999992,38.05719200000004],[-107.52082599999989,38.05750200000011],[-107.519768,38.05784399999999],[-107.51895899999994,38.05817200000013],[-107.51781199999999,38.05810100000008],[-107.51639999999998,38.05785499999996],[-107.51529799999997,38.05826899999994],[-107.51481999999999,38.058667000000014],[-107.51334699999995,38.05924500000009],[-107.511819,38.05998499999998],[-107.51129899999995,38.06025200000005],[-107.51087999999993,38.06087700000012],[-107.51255399999997,38.06264500000003],[-107.51388099999997,38.063535000000115],[-107.51500399999992,38.06525499999998],[-107.51511600000003,38.06609700000001],[-107.51527199999998,38.0669400000001],[-107.51567899999998,38.068287],[-107.51649099999992,38.06939699999998],[-107.519139,38.07006200000012],[-107.52053999999993,38.07042000000007],[-107.521522,38.07120100000003],[-107.52208199999995,38.07170100000002],[-107.52332999999993,38.07204900000005],[-107.52494999999993,38.073945000000094],[-107.52574099999993,38.07506300000006],[-107.52646599999997,38.075920999999994],[-107.52727900000002,38.07566500000013],[-107.528188,38.076081000000045],[-107.52913999999998,38.07683700000007],[-107.52974399999994,38.07676400000008],[-107.530259,38.07700499999993],[-107.531227,38.07798000000014],[-107.53243199999997,38.07885199999998],[-107.532648,38.07924300000002],[-107.5339489999999,38.07992300000001],[-107.536022,38.07944100000009],[-107.53658199999995,38.07951299999996],[-107.53739000000002,38.0794350000001],[-107.54154799999998,38.07974800000005],[-107.54245799999995,38.080115000000035],[-107.54290699999996,38.08013700000009],[-107.54437299999995,38.08073000000013],[-107.54481699999991,38.080526000000134],[-107.54725099999996,38.079425000000015],[-107.54825599999998,38.07874300000003],[-107.55028899999996,38.078236000000004],[-107.55128699999995,38.078815000000134],[-107.55285100000003,38.07913400000007],[-107.553585,38.079200000000014],[-107.55436499999996,38.07985200000013],[-107.55454500000002,38.08006899999998],[-107.55519599999997,38.08154799999994],[-107.555838,38.08250999999996],[-107.55653699999999,38.083625999999924],[-107.55784900000003,38.086148000000094],[-107.55955,38.08720499999998],[-107.56017300000002,38.08899900000006],[-107.55994699999997,38.091346999999985],[-107.56046200000003,38.09207300000003],[-107.56101000000001,38.09313100000003],[-107.56151599999993,38.09378400000003],[-107.56288,38.095337000000086],[-107.56369199999995,38.095978],[-107.56464999999992,38.09699200000006],[-107.564752,38.097453999999914],[-107.56531299999995,38.09838199999996],[-107.56539399999991,38.09886800000004],[-107.56474600000001,38.099555000000066],[-107.564164,38.10001599999998],[-107.56307199999998,38.10043000000013],[-107.56275700000003,38.10082200000005],[-107.562252,38.10193099999998],[-107.56248099999993,38.10263700000013],[-107.562501,38.103566000000114],[-107.56251399999991,38.1048110000001],[-107.56222999999994,38.105187000000114],[-107.56131399999992,38.107357000000036],[-107.56134299999985,38.10831900000005],[-107.56102799999996,38.111005999999975],[-107.56138199999998,38.112497000000076],[-107.56174099999998,38.11390399999999],[-107.56375500000001,38.11645500000003],[-107.5639799999999,38.117808000000025],[-107.56423799999993,38.11906400000004],[-107.56396999999987,38.11963400000013],[-107.56363499999986,38.12182800000011],[-107.56320499999998,38.12321200000008],[-107.563581,38.124219000000096],[-107.56433700000002,38.125851000000125],[-107.56456000000003,38.1273020000001],[-107.56518099999988,38.12834100000009],[-107.5669759999999,38.129679000000124],[-107.56756699999994,38.130187000000035],[-107.568667,38.131598000000054],[-107.56883700000003,38.13199199999997],[-107.56951300000003,38.1328630000001],[-107.56966499999999,38.13328199999995],[-107.56920500000001,38.13404700000001],[-107.56944999999996,38.13430500000004],[-107.57228600000002,38.135073000000034],[-107.57352199999997,38.13495300000005],[-107.57403999999997,38.136330999999984],[-107.57386600000001,38.137329000000136],[-107.57341600000001,38.13793300000003],[-107.57298699999996,38.13805300000001],[-107.57272099999989,38.13885900000008],[-107.572587,38.13988999999998],[-107.57249300000001,38.140872],[-107.571911,38.141427000000135],[-107.57178799999997,38.14181400000001],[-107.57110099999994,38.14389100000005],[-107.57000700000003,38.145421],[-107.56923999999998,38.146596000000045],[-107.56873599999994,38.147126000000014],[-107.56869799999998,38.147167000000024],[-107.56862000000001,38.147267],[-107.56021399999997,38.14711699999998],[-107.55898999999994,38.147118999999975],[-107.55762700000002,38.14712099999997],[-107.553541,38.147131000000115],[-107.55218000000002,38.14713500000005],[-107.53857699999992,38.147114999999985],[-107.50055899999995,38.14706000000007],[-107.48708599999986,38.14708300000001],[-107.48339999999996,38.147087],[-107.46709799999991,38.14710599999995],[-107.45374099999992,38.14712099999997],[-107.44541099999992,38.147131000000115],[-107.42458399999992,38.14715600000005],[-107.41937799999994,38.14716199999998],[-107.383783,38.1472040000001],[-107.38060199999995,38.147159999999985],[-107.37561299999999,38.14709000000005],[-107.35190499999993,38.14723300000003],[-107.35183899999998,38.14723300000003],[-107.33741699999996,38.14713300000005],[-107.32962999999995,38.147079000000076],[-107.32304699999997,38.147087],[-107.31838599999998,38.14709400000004],[-107.31063499999993,38.14710400000001],[-107.30372699999998,38.14711200000011],[-107.30233099999987,38.1471140000001],[-107.29728699999998,38.14712099999997],[-107.296942,38.14712200000008],[-107.29177199999998,38.147130000000004],[-107.29167799999993,38.147130000000004],[-107.29148599999996,38.147130000000004],[-107.272536,38.14715600000005],[-107.24666599999995,38.14722500000005],[-107.19901599999997,38.14719199999996],[-107.19812000000002,38.14719100000008],[-107.18953499999998,38.14718499999992],[-107.17379299999999,38.14717400000012],[-107.14530200000002,38.14715500000011],[-107.13427100000001,38.14714800000007],[-107.125607,38.14714300000003],[-107.112233,38.146912999999984],[-107.11072399999995,38.146906000000115],[-107.10884899999996,38.14689999999996],[-107.1056329999999,38.14688899999999],[-107.05830000000003,38.147123000000136],[-107.05569799999995,38.14707599999997],[-107.05104699999998,38.146899000000076],[-107.04668100000004,38.147073999999975],[-107.04426000000001,38.14707000000004],[-107.03101399999997,38.14705300000003],[-107.01365099999998,38.14702800000009],[-107.00111900000002,38.14707199999998],[-107.00114599999995,38.143955000000005],[-107.00115299999999,38.14317600000004],[-107.001192,38.138644000000056],[-107.00120300000003,38.137512000000015],[-107.00131399999998,38.124994999999956],[-107.00177399999995,38.09932300000003],[-107.00188899999995,38.09290600000003],[-107.00188000000003,38.09006299999999],[-107.00184499999995,38.08153400000003],[-107.001822,38.075776000000076],[-107.00173699999999,38.05504500000001],[-107.00172499999996,38.05201200000005],[-107.00168799999994,38.042902000000026],[-107.00160899999997,38.02341700000005],[-107.00157100000001,38.01410600000003],[-107.00154099999986,38.00654800000012],[-107.00151399999993,37.99999600000001],[-107.00101799999993,37.97662400000013],[-107.00079599999992,37.96618199999995],[-107.00060199999996,37.95604100000003],[-107.00231499999995,37.955977000000075],[-107.00468899999998,37.95692100000002],[-107.00626299999993,37.95621799999998],[-107.00669499999998,37.95410400000003],[-107.00675599999994,37.94920300000007],[-107.007654,37.947292000000004],[-107.00897299999997,37.94652200000007],[-107.01118099999997,37.94696099999999],[-107.01754199999993,37.949018000000024],[-107.02084299999996,37.95231299999995],[-107.02291799999995,37.953792000000135],[-107.02550599999995,37.95450099999999],[-107.02835199999998,37.954136000000005],[-107.03056400000003,37.95306500000004],[-107.03154699999993,37.95105200000006],[-107.03320599999995,37.95008100000001],[-107.03609599999999,37.94938000000013],[-107.04136299999988,37.94868100000002],[-107.044716,37.94932300000005],[-107.04942,37.95214900000008],[-107.05115899999993,37.952956000000086],[-107.05285700000002,37.95312600000011],[-107.05523700000003,37.95215500000006],[-107.05638399999998,37.952256999999975],[-107.05938800000001,37.9556510000001],[-107.05976399999997,37.957765999999936],[-107.06141499999995,37.95974799999999],[-107.06544899999994,37.95975199999998],[-107.06846100000001,37.960762000000045],[-107.06923199999994,37.95844699999998],[-107.072001,37.95536199999998],[-107.07565599999998,37.953922000000034],[-107.08015699999999,37.95429600000011],[-107.085552,37.95336099999997],[-107.08670000000001,37.952690000000075],[-107.08836199999996,37.95040899999998],[-107.09214399999996,37.94916999999998],[-107.09460599999994,37.94930699999992],[-107.09731999999997,37.95078600000011],[-107.100077,37.95189600000009],[-107.10147599999993,37.952904000000046],[-107.102105,37.95481399999994],[-107.10321199999993,37.95505300000002],[-107.10560199999998,37.94951600000002],[-107.10611699999998,37.94713400000012],[-107.10841600000003,37.944618000000105],[-107.11066900000003,37.943344000000025],[-107.11576699999995,37.942004],[-107.12196799999998,37.94086700000008],[-107.125606,37.93915600000008],[-107.12962699999991,37.93783800000011],[-107.13132000000002,37.937603000000024],[-107.13497899999999,37.93783500000001],[-107.13567699999999,37.93852700000008],[-107.138191,37.93884800000001],[-107.13816999999995,37.87499800000012],[-107.13860699999998,37.865860000000055],[-107.13735600000001,37.82878800000009],[-107.13994000000002,37.757197000000076],[-107.14070699999996,37.75],[-107.14274899999992,37.71680800000013],[-107.14513799999997,37.67549700000012],[-107.12560500000001,37.675196000000085],[-107.12386800000002,37.67502900000011],[-107.12373400000001,37.62500200000011],[-107.12448,37.612088000000085],[-107.12560400000001,37.60049200000003],[-107.12693999999999,37.539743000000044],[-107.12781799999999,37.50000400000005],[-107.12869899999987,37.47333600000013],[-107.12862499999994,37.45354900000001],[-107.12867999999997,37.42294199999998],[-107.13341699999995,37.42294300000003],[-107.1656079999999,37.422948000000076],[-107.16643999999991,37.422948000000076],[-107.17532799999998,37.42295000000007],[-107.187003,37.422952000000066],[-107.193578,37.42295300000001],[-107.19393699999995,37.42295300000001],[-107.19655999999998,37.42295300000001],[-107.19833800000004,37.42295300000001],[-107.20982400000003,37.422959000000105],[-107.21622399999995,37.422979],[-107.21748400000001,37.42298200000005],[-107.21776999999997,37.42298200000005],[-107.23717699999997,37.42297200000013],[-107.242006,37.422970000000134],[-107.25159100000002,37.42296799999997],[-107.28034699999989,37.42299799999995],[-107.300318,37.42302000000001],[-107.34469899999993,37.42306800000006],[-107.35321699999997,37.42307699999992],[-107.35602900000004,37.42308000000003],[-107.37307499999997,37.42355199999997],[-107.37560699999995,37.42362300000002],[-107.38116500000001,37.42357299999992],[-107.409559,37.42331900000005],[-107.46177299999994,37.42285500000003],[-107.48213099999987,37.42267300000009],[-107.48206199999993,37.45001600000006],[-107.48204399999986,37.457148000000075],[-107.48207300000001,37.46612099999999],[-107.48211099999997,37.47562900000014],[-107.48219399999999,37.49634100000003],[-107.48219799999998,37.497261000000094],[-107.48188600000003,37.50160299999999],[-107.48194099999995,37.522064000000114],[-107.48203699999999,37.55715100000009],[-107.48206299999993,37.56652299999996],[-107.48210699999998,37.58222500000011],[-107.482122,37.58745899999991],[-107.482169,37.62069900000006],[-107.48217899999992,37.63110799999998],[-107.48217899999992,37.639500999999996],[-107.48218399999996,37.68837200000013],[-107.48220700000002,37.74388400000004],[-107.48206099999993,37.749998000000005],[-107.48224500000003,37.768353000000104],[-107.48089599999997,37.768080999999995],[-107.47842000000003,37.769267999999954],[-107.47476699999993,37.76767100000001],[-107.47327899999999,37.767628],[-107.47081599999996,37.76655199999999],[-107.46718499999992,37.76603699999998],[-107.46673499999997,37.767163000000096],[-107.46754599999997,37.76941399999998],[-107.46488399999998,37.773610000000076],[-107.46518200000003,37.77439499999997],[-107.46387499999997,37.77708100000001],[-107.46074199999998,37.77983800000004],[-107.46061099999991,37.7812770000001],[-107.45834200000002,37.78179899999992],[-107.45963699999999,37.78619800000013],[-107.46071,37.78699899999998],[-107.45937499999997,37.78972900000008],[-107.45962500000002,37.792039999999986],[-107.459224,37.79408300000006],[-107.46022099999999,37.79491200000007],[-107.46096899999998,37.796884000000034],[-107.462173,37.79737200000005],[-107.46202899999997,37.798767],[-107.46336499999995,37.7992660000001],[-107.46557299999995,37.80089100000009],[-107.46678499999996,37.80461400000013],[-107.46882899999997,37.80780400000009],[-107.46884299999999,37.809726000000126],[-107.470955,37.81176199999999],[-107.472305,37.81233900000012],[-107.47497199999992,37.815687000000025],[-107.47887600000001,37.81668500000001],[-107.4813069999999,37.818590000000086],[-107.48227699999995,37.81988500000011],[-107.48547699999995,37.82242100000013],[-107.49030999999997,37.82427500000006],[-107.49489299999993,37.824080000000095],[-107.49640799999997,37.8230860000001],[-107.50356099999999,37.82370900000012],[-107.51292199999995,37.82533400000011],[-107.51532299999991,37.82656600000007],[-107.51654699999995,37.82645100000002],[-107.51728700000001,37.829391000000044],[-107.51656700000001,37.829769999999996],[-107.51612999999992,37.83193799999998],[-107.51683499999996,37.83347200000003],[-107.51620400000002,37.834546999999986],[-107.51837399999994,37.83898199999993],[-107.52058599999992,37.83949300000012],[-107.51966699999991,37.84119400000003],[-107.52177,37.8425840000001],[-107.52178300000003,37.84463499999998],[-107.52096399999994,37.84489200000007],[-107.52106100000003,37.84735600000005],[-107.51859299999995,37.849185000000034],[-107.51817399999993,37.85594000000009],[-107.51785899999999,37.856978000000026],[-107.518644,37.858086000000014],[-107.52078699999993,37.85954800000002],[-107.52107999999993,37.86186100000009],[-107.52234499999997,37.863435000000095],[-107.52440200000001,37.86516300000011],[-107.52414199999998,37.86660500000005],[-107.52601399999998,37.86895200000009],[-107.52774099999993,37.870361000000116],[-107.53256599999986,37.87284800000003],[-107.53553899999997,37.874062000000094],[-107.534967,37.874996000000124],[-107.53532799999994,37.87711300000012],[-107.53624399999995,37.87859400000008],[-107.53646800000001,37.880110000000116],[-107.53450299999997,37.88178800000003],[-107.53248100000002,37.884047000000066],[-107.53344400000003,37.88807700000012],[-107.53310599999986,37.88936100000012],[-107.53128200000003,37.88915900000012],[-107.529068,37.890022999999985],[-107.52716199999992,37.89137699999992],[-107.52605599999998,37.89364899999998],[-107.52642900000001,37.896213000000046],[-107.52303599999993,37.89753300000001],[-107.52182099999999,37.89877200000001],[-107.521638,37.900419000000056],[-107.52016699999996,37.90131200000002],[-107.52463499999999,37.90405400000009],[-107.52480600000001,37.905345000000125],[-107.527511,37.90619800000002],[-107.52850499999994,37.907097000000135],[-107.52851499999997,37.90878399999991],[-107.52971599999995,37.911748000000046],[-107.52892399999996,37.91368700000004],[-107.53074699999996,37.91681900000003],[-107.53590199999991,37.920123000000046],[-107.536652,37.92108800000011],[-107.536518,37.922412000000065],[-107.53454599999998,37.9246710000001],[-107.53735799999987,37.926144000000136],[-107.53835100000003,37.929819000000066],[-107.53770099999991,37.93464399999999],[-107.53414900000001,37.93713800000012],[-107.53430900000001,37.938170000000014],[-107.532644,37.94084700000002],[-107.53259299999996,37.943905000000086],[-107.53427099999999,37.944931000000054],[-107.53734600000001,37.9476130000001],[-107.54006900000002,37.94788399999999],[-107.54423300000002,37.94871499999999],[-107.55104599999999,37.949686999999926],[-107.5528589999999,37.951076000000114],[-107.55656099999993,37.95291600000013],[-107.559166,37.95369599999998],[-107.56336899999997,37.95593699999995],[-107.56491399999999,37.96121599999998],[-107.56721900000002,37.9613270000001],[-107.569005,37.963085999999976],[-107.56887499999999,37.96501500000005],[-107.56903499999999,37.96603900000002],[-107.57073399999996,37.96602700000011],[-107.57167400000003,37.96644700000007],[-107.57268699999992,37.96716600000002],[-107.5736399999999,37.96714599999996],[-107.57494399999996,37.96714599999996],[-107.576209,37.96742400000005],[-107.57655699999992,37.967520000000036],[-107.58029599999986,37.968474999999955],[-107.58298399999995,37.967922999999985],[-107.58463499999993,37.967997000000025],[-107.58700399999992,37.96847800000006],[-107.58822099999992,37.97031200000009],[-107.58775500000002,37.97122200000007],[-107.58643099999995,37.97319800000014],[-107.58571399999988,37.974092999999925],[-107.58416499999998,37.97601900000012],[-107.58512300000001,37.97878200000014],[-107.58583099999998,37.979457000000025],[-107.58690999999999,37.98059000000001],[-107.58756199999993,37.982038999999986],[-107.58890600000001,37.98465500000003],[-107.58973900000001,37.988444000000015],[-107.58980799999995,37.989629000000036],[-107.59136699999999,37.99225200000001]]]},"properties":{"OBJECTID":28,"COUNTYFP":"053","Shape_Leng":2.84694965073,"Shape_Area":0.297670982924,"low":175,"lowmod":325,"LMMI":500,"lowmoduniv":865,"lowmod_pct":0.375722543353,"NAMELSAD10":"Hinsdale County"}},
{"type":"Feature","geometry":{"type":"Polygon","coordinates":[[[-105.04991699999988,37.915479000000005],[-105.04456099999993,37.91165600000005],[-105.043905,37.909539999999936],[-105.039826,37.908124000000214],[-105.03866499999987,37.90444400000001],[-105.03598999999997,37.90411500000016],[-105.03295400000002,37.90532200000018],[-105.03157299999998,37.90414700000031],[-105.03457499999985,37.89611200000002],[-105.0348029999999,37.89458300000001],[-105.03362600000003,37.893560000000264],[-105.0304349999999,37.89206300000001],[-105.021748,37.887491000000296],[-105.01961299999994,37.886967000000084],[-105.01575099999985,37.885509999999954],[-105.01376299999987,37.88348900000028],[-105.0137289999999,37.881271000000254],[-104.98774799999995,37.88284099999993],[-104.9482139999999,37.8850060000002],[-104.94577699999996,37.88506200000006],[-104.92122099999989,37.88635799999997],[-104.87554199999994,37.88868099999996],[-104.86834399999981,37.88918500000011],[-104.84623199999999,37.890199000000166],[-104.78906499999988,37.893237],[-104.75041899999991,37.89529500000009],[-104.70021199999996,37.89777800000002],[-104.64638300000001,37.900527000000125],[-104.62553399999996,37.894561000000124],[-104.59752500000002,37.88670000000002],[-104.58280099999996,37.88266299999998],[-104.52681899999993,37.86709799999994],[-104.500269,37.85958600000015],[-104.43998599999998,37.84257000000002],[-104.37520299999989,37.82427000000001],[-104.35110899999995,37.817488000000196],[-104.372863,37.80040500000001],[-104.37552699999986,37.79858799999994],[-104.41587599999997,37.773325000000284],[-104.45311900000002,37.75001100000003],[-104.45672299999995,37.74788000000029],[-104.45492999999993,37.740838999999994],[-104.48313999999999,37.701188],[-104.50053200000002,37.67674899999997],[-104.51169699999991,37.661280999999974],[-104.51428199999992,37.65788800000007],[-104.5373009999999,37.62501200000008],[-104.54841699999986,37.609424000000104],[-104.54865199999995,37.57470000000029],[-104.58584499999995,37.55152899999996],[-104.620338,37.530053000000294],[-104.62553799999984,37.51611900000006],[-104.62765299999995,37.510902000000044],[-104.63291900000002,37.507204],[-104.64922899999993,37.495320000000106],[-104.66647699999999,37.494792000000075],[-104.67015300000003,37.494987000000094],[-104.681017,37.496026000000086],[-104.68574000000001,37.495992000000115],[-104.6897019999999,37.49441899999999],[-104.69210900000002,37.49122699999998],[-104.69093499999997,37.489239],[-104.69199399999991,37.48775400000005],[-104.69184099999995,37.48662400000018],[-104.69611099999992,37.48542500000008],[-104.69614100000001,37.48434999999995],[-104.69482199999976,37.482123000000115],[-104.69344799999988,37.481846000000075],[-104.69176399999998,37.47903100000008],[-104.69401800000003,37.47578199999998],[-104.69575599999996,37.47545200000002],[-104.69621899999999,37.47350999999992],[-104.69723099999999,37.47180700000007],[-104.69993099999999,37.46983800000004],[-104.69888800000001,37.467933000000016],[-104.69965300000001,37.46594900000014],[-104.69894899999991,37.464766],[-104.69998599999991,37.46373999999997],[-104.69937699999997,37.46280100000013],[-104.69932499999993,37.460794000000305],[-104.69838599999997,37.45897000000002],[-104.69948299999987,37.45725900000002],[-104.69952999999992,37.45447500000006],[-104.70045599999992,37.45354900000001],[-104.70027800000003,37.45078500000011],[-104.69829199999992,37.44808499999999],[-104.69835,37.44544599999995],[-104.69726500000002,37.44481300000018],[-104.69561099999987,37.442188999999985],[-104.69507499999992,37.4399150000001],[-104.69689800000003,37.43894000000006],[-104.69830200000001,37.43988300000029],[-104.70190799999995,37.43882900000011],[-104.70237199999997,37.43815499999994],[-104.705716,37.43772000000001],[-104.70896900000002,37.436638000000016],[-104.71047399999986,37.437501000000054],[-104.71254399999998,37.436649999999986],[-104.7157749999999,37.434713999999985],[-104.71809899999988,37.431825999999944],[-104.72039799999993,37.431166000000246],[-104.72115499999995,37.42941600000023],[-104.7237649999999,37.42945700000024],[-104.72513600000002,37.429989000000205],[-104.72542399999998,37.42745800000023],[-104.72679,37.42653599999994],[-104.73058199999986,37.42724199999992],[-104.73221000000001,37.42665500000004],[-104.73340299999995,37.425367999999935],[-104.73679900000002,37.42494899999991],[-104.73711199999997,37.42300100000023],[-104.73997199999991,37.42166800000024],[-104.74242600000002,37.41979000000009],[-104.74315399999983,37.417887000000064],[-104.74723899999998,37.41571000000022],[-104.74955899999992,37.413277000000164],[-104.749075,37.410437],[-104.74942199999987,37.40972399999998],[-104.748447,37.40743700000013],[-104.74893799999995,37.40702900000031],[-104.75213899999994,37.40684800000025],[-104.75436400000001,37.40634699999998],[-104.75628,37.40719400000012],[-104.75842699999993,37.40699600000005],[-104.7607559999999,37.40545099999997],[-104.76758699999993,37.40539000000024],[-104.76993899999985,37.404920999999945],[-104.77186899999987,37.40647300000012],[-104.77346299999994,37.40604799999994],[-104.77459799999997,37.40388199999995],[-104.77683699999994,37.40408600000001],[-104.77931999999998,37.403782000000035],[-104.78073699999987,37.40302300000013],[-104.78200799999996,37.40129200000007],[-104.78742199999994,37.400107000000276],[-104.79076099999997,37.40049700000003],[-104.79187699999994,37.401388000000225],[-104.79756500000002,37.40117400000008],[-104.79910499999988,37.400245000000155],[-104.80062299999997,37.40025700000007],[-104.803718,37.40293500000013],[-104.80715999999995,37.402214000000015],[-104.80853899999994,37.39967399999995],[-104.81104599999998,37.39863000000008],[-104.81251500000002,37.40025600000001],[-104.81546900000001,37.40038100000021],[-104.81688400000002,37.399487000000306],[-104.81726200000003,37.397500000000036],[-104.81970099999995,37.39719600000029],[-104.82208299999985,37.398168999999996],[-104.82426299999986,37.397567000000095],[-104.82528499999995,37.39604000000003],[-104.82666799999998,37.39565099999999],[-104.83146899999991,37.39541300000002],[-104.83282700000001,37.395797000000016],[-104.83475499999986,37.39526500000022],[-104.83527499999997,37.39784499999996],[-104.83828699999998,37.39867500000008],[-104.84189699999996,37.397951000000035],[-104.84351899999996,37.39691900000008],[-104.84720199999987,37.39569],[-104.849986,37.39568299999996],[-104.85375699999992,37.394586000000004],[-104.85883699999988,37.39363800000024],[-104.860658,37.39209900000009],[-104.86225899999994,37.392042000000174],[-104.86497199999997,37.39079300000003],[-104.86633399999977,37.391377000000034],[-104.868785,37.3916440000001],[-104.871644,37.391232000000116],[-104.872975,37.390405999999984],[-104.87394799999987,37.38867800000003],[-104.87723199999999,37.387762000000066],[-104.87837000000002,37.38788000000005],[-104.88425999999998,37.38581700000026],[-104.88691099999994,37.387861999999984],[-104.89291799999995,37.389424000000076],[-104.89867900000002,37.38920900000005],[-104.90099799999996,37.38944500000008],[-104.90294899999992,37.390190000000075],[-104.90606100000002,37.39011300000027],[-104.90787499999999,37.392337999999995],[-104.91114900000002,37.39393700000011],[-104.91332799999998,37.39353700000004],[-104.91597899999988,37.393533000000104],[-104.91763000000003,37.39397900000006],[-104.91918099999987,37.393688999999995],[-104.92218200000002,37.394318],[-104.92504300000002,37.396259000000214],[-104.92560699999996,37.39705699999996],[-104.93055300000003,37.39618000000024],[-104.93255699999992,37.395412000000306],[-104.93527099999994,37.39248500000008],[-104.93715199999997,37.38997199999994],[-104.93886099999997,37.38920900000005],[-104.94062899999989,37.38925000000006],[-104.944906,37.390499000000034],[-104.94673599999999,37.39144600000009],[-104.94831899999997,37.39068400000025],[-104.95263099999988,37.389715000000194],[-104.9567459999999,37.38737100000003],[-104.9588819999999,37.38498900000013],[-104.96243399999997,37.38191400000011],[-104.96703300000001,37.37872300000021],[-104.96928500000001,37.37778300000008],[-104.971228,37.37624299999999],[-104.97578199999987,37.37674700000031],[-104.97686899999991,37.37642900000009],[-104.98003299999988,37.37424100000027],[-104.98352699999987,37.373388000000205],[-104.98681299999998,37.37432000000024],[-104.98833499999995,37.37555400000019],[-104.98990100000003,37.37596800000017],[-104.99473599999988,37.37582900000001],[-104.99550499999998,37.375550999999916],[-105.00122999999991,37.368551000000025],[-105.00227799999993,37.366859000000034],[-105.00368700000001,37.36580000000015],[-105.00478799999996,37.36317300000002],[-105.00663299999997,37.36269200000004],[-105.00872400000003,37.361333],[-105.01061099999993,37.35948000000019],[-105.01285899999993,37.35839000000004],[-105.01614099999989,37.35835800000007],[-105.01767599999988,37.35701800000004],[-105.021454,37.35624100000024],[-105.02368499999989,37.35626400000018],[-105.02358900000002,37.35460100000006],[-105.02476000000001,37.34998100000007],[-105.02452999999997,37.34719499999994],[-105.02348599999993,37.34350499999999],[-105.02347399999991,37.341649000000075],[-105.02559899999989,37.33996800000028],[-105.02771699999988,37.339416000000256],[-105.03025999999988,37.33947400000011],[-105.03382899999997,37.34038099999998],[-105.03641500000003,37.341657000000225],[-105.03787999999986,37.34027600000019],[-105.03968899999995,37.339454000000046],[-105.04062799999986,37.33807600000023],[-105.04363299999989,37.3362420000002],[-105.04522399999996,37.335789000000034],[-105.046945,37.33344199999999],[-105.05059399999993,37.331049000000064],[-105.05445099999986,37.33028799999994],[-105.05626099999995,37.329386000000284],[-105.06101799999999,37.328283],[-105.06804599999992,37.32604599999996],[-105.07146699999981,37.32197200000007],[-105.07371899999998,37.31976600000013],[-105.07409799999999,37.31732499999998],[-105.07512599999995,37.31666500000006],[-105.07766100000003,37.316545000000076],[-105.08155599999998,37.31579899999997],[-105.08419300000003,37.31495300000017],[-105.08553099999989,37.31525600000003],[-105.088348,37.315106000000014],[-105.09215999999992,37.31535200000013],[-105.09424799999994,37.314703000000065],[-105.09617700000001,37.311816000000306],[-105.09481800000003,37.309496000000195],[-105.09485599999988,37.30822100000023],[-105.09700900000001,37.30586099999999],[-105.09797100000003,37.304151000000275],[-105.10395499999998,37.30131100000011],[-105.11052799999993,37.299507999999946],[-105.11259100000001,37.29841400000004],[-105.11542500000002,37.29629399999999],[-105.11989799999992,37.29599700000006],[-105.12808000000001,37.296822000000304],[-105.13019199999997,37.29789599999998],[-105.1315909999999,37.301114000000155],[-105.13440700000001,37.29919100000001],[-105.13783899999987,37.298223000000235],[-105.13992499999989,37.29706299999998],[-105.14304199999987,37.2959350000001],[-105.14595999999989,37.29582600000015],[-105.14848699999999,37.29609100000022],[-105.15049899999991,37.293785000000014],[-105.15200500000003,37.29300000000006],[-105.15417599999995,37.29312900000008],[-105.15711299999998,37.293035000000145],[-105.158953,37.29236900000012],[-105.16019,37.29110900000012],[-105.164559,37.28911700000015],[-105.16547199999997,37.291241000000014],[-105.16515800000002,37.292792000000134],[-105.16533699999991,37.295228000000066],[-105.16339299999999,37.29994600000015],[-105.16476899999998,37.304430999999965],[-105.164938,37.308520999999985],[-105.16691899999995,37.313792000000205],[-105.16543899999999,37.316780000000165],[-105.16669300000001,37.32120899999995],[-105.16669200000001,37.323735000000056],[-105.16531499999996,37.325988000000166],[-105.163523,37.32738900000027],[-105.16309599999994,37.32890900000007],[-105.16393899999986,37.33096800000027],[-105.16560499999986,37.333529999999996],[-105.16727700000001,37.33484199999998],[-105.16794900000002,37.33703100000008],[-105.16568899999999,37.34150900000009],[-105.1640499999998,37.34302200000019],[-105.16411600000004,37.34521599999999],[-105.16586899999987,37.3473180000002],[-105.16642399999995,37.34988800000002],[-105.16600099999994,37.351674],[-105.16636099999988,37.35305900000003],[-105.16804699999994,37.354693000000225],[-105.16697599999998,37.3581470000002],[-105.16866799999991,37.36024100000026],[-105.16890599999999,37.36151400000006],[-105.16645699999992,37.36559799999992],[-105.16574899999989,37.36982300000011],[-105.16675799999996,37.372253000000114],[-105.16841199999988,37.37366900000001],[-105.16887899999995,37.37563800000021],[-105.16531700000002,37.38117399999999],[-105.16716699999978,37.38583600000004],[-105.16641799999991,37.388769999999965],[-105.16264899999999,37.39030200000008],[-105.16143299999999,37.39320900000007],[-105.15726899999999,37.39843600000006],[-105.15516899999989,37.399636000000214],[-105.15132399999987,37.400192000000004],[-105.146323,37.40041500000018],[-105.14499799999999,37.40023500000001],[-105.14251599999994,37.40094600000003],[-105.13856699999997,37.40069100000028],[-105.13266899999985,37.40360600000008],[-105.13125600000001,37.406812000000286],[-105.12972000000002,37.40925400000003],[-105.130585,37.41165500000005],[-105.13265199999995,37.41154500000022],[-105.13462700000002,37.411992000000055],[-105.13712699999985,37.413102000000094],[-105.13941599999998,37.41558500000002],[-105.14052599999997,37.418128999999965],[-105.140695,37.42181099999999],[-105.14152199999995,37.42365600000005],[-105.14091300000001,37.427918999999974],[-105.14221399999991,37.42917100000005],[-105.14488599999993,37.42896500000006],[-105.14395100000002,37.431737999999996],[-105.14453400000002,37.433515000000284],[-105.14646099999987,37.436001999999974],[-105.14590599999997,37.438586999999984],[-105.14682399999987,37.44109200000014],[-105.14956199999995,37.443182000000036],[-105.15098699999999,37.44706100000019],[-105.15218899999996,37.45155400000016],[-105.15443299999998,37.453554000000054],[-105.15515299999998,37.45567400000016],[-105.156185,37.45709000000011],[-105.15918499999998,37.457114000000104],[-105.16253099999989,37.454765000000066],[-105.16450699999996,37.45459900000003],[-105.16803400000003,37.45602799999995],[-105.16958099999994,37.45614100000006],[-105.17142699999982,37.455363000000034],[-105.17536999999993,37.45628799999997],[-105.17541399999993,37.45714199999992],[-105.17346799999996,37.461096000000055],[-105.17500099999995,37.46528799999999],[-105.17596999999984,37.470259000000055],[-105.17668500000002,37.47261300000014],[-105.17668299999985,37.47559400000006],[-105.18204299999996,37.47970400000014],[-105.18010300000003,37.48157900000001],[-105.17642499999982,37.483408],[-105.17481599999985,37.48510200000004],[-105.17449299999993,37.48697599999997],[-105.17571599999997,37.489526000000126],[-105.17766599999993,37.49139100000025],[-105.172483,37.49922000000004],[-105.1733319999999,37.501415000000065],[-105.17428299999989,37.50251400000025],[-105.17338499999994,37.50579500000009],[-105.17412399999995,37.50929900000011],[-105.17131999999987,37.51324600000021],[-105.17251499999998,37.51560700000016],[-105.1715329999999,37.51750200000009],[-105.17211099999986,37.519030999999984],[-105.17433399999993,37.519889999999975],[-105.17363799999998,37.52299800000009],[-105.17465600000003,37.52516900000012],[-105.17637999999994,37.52614400000016],[-105.17754799999994,37.52972700000015],[-105.17707999999993,37.53031400000003],[-105.17771399999998,37.53410400000013],[-105.17984099999995,37.53612600000025],[-105.18299799999994,37.53819800000008],[-105.18378799999994,37.54089600000003],[-105.182098,37.5425800000001],[-105.18295299999994,37.544651000000215],[-105.18267699999984,37.54678100000024],[-105.17835200000002,37.54631900000021],[-105.17933899999997,37.548676],[-105.1826089999999,37.55212799999998],[-105.18388499999998,37.55263800000006],[-105.18343299999987,37.55383499999999],[-105.18401699999987,37.55584100000004],[-105.18888599999985,37.55947800000007],[-105.19188600000001,37.56235600000002],[-105.19306199999988,37.56389000000007],[-105.19480899999985,37.56465500000013],[-105.19372199999998,37.565516],[-105.19351,37.566921000000036],[-105.19515999999993,37.56810600000023],[-105.19542099999984,37.569177000000195],[-105.19829699999997,37.57181499999996],[-105.20161599999994,37.57402300000024],[-105.20531499999993,37.574261000000206],[-105.20763099999999,37.57524000000001],[-105.211681,37.57766100000015],[-105.21385999999995,37.57788300000004],[-105.21425999999985,37.5797280000001],[-105.21604400000001,37.581186000000116],[-105.21833299999986,37.58243900000025],[-105.21843299999989,37.58349599999997],[-105.21715699999999,37.584858],[-105.2118999999999,37.58608900000007],[-105.20570699999996,37.59080899999998],[-105.2037699999999,37.59444900000011],[-105.20231299999989,37.59478800000005],[-105.19875199999996,37.594065000000285],[-105.19669499999992,37.59275400000001],[-105.1932369999999,37.592168000000186],[-105.19336299999992,37.59806300000014],[-105.19415499999991,37.599631000000215],[-105.194095,37.601035000000024],[-105.19524199999995,37.601424000000236],[-105.19634999999988,37.603176000000246],[-105.19540899999993,37.604673000000105],[-105.19483199999996,37.60736200000002],[-105.19531399999988,37.60944100000029],[-105.19285999999988,37.610478000000285],[-105.19261699999993,37.611851],[-105.18973900000003,37.61444700000004],[-105.186597,37.615738000000135],[-105.18444099999999,37.615984000000026],[-105.18754899999999,37.6196690000001],[-105.19216299999994,37.61926599999998],[-105.1944929999998,37.61966400000023],[-105.20150699999999,37.61919400000005],[-105.20549599999998,37.620626999999956],[-105.21211899999997,37.62353200000018],[-105.21453399999996,37.62642500000027],[-105.21579300000002,37.62638099999998],[-105.21950800000002,37.62423799999999],[-105.22002699999996,37.621920000000046],[-105.22121600000003,37.62117100000023],[-105.22614499999992,37.620368999999926],[-105.22974299999998,37.62039700000031],[-105.23597799999993,37.620773000000156],[-105.23659399999991,37.621665000000235],[-105.24156599999998,37.623724000000095],[-105.24259499999994,37.62606800000003],[-105.24336199999999,37.62922900000012],[-105.24581799999999,37.62995600000022],[-105.24876399999994,37.62958700000024],[-105.25219900000002,37.63025900000008],[-105.25421199999994,37.63232400000021],[-105.25436300000001,37.63522999999998],[-105.25364399999995,37.63673900000009],[-105.25530299999997,37.6402580000003],[-105.25814600000001,37.64054700000008],[-105.259229,37.63956899999994],[-105.26146499999993,37.639003],[-105.26350899999994,37.63911399999995],[-105.26593700000001,37.63824899999997],[-105.26736299999988,37.63874800000025],[-105.26711,37.639871000000255],[-105.26830399999994,37.64061200000009],[-105.27295899999996,37.64100500000029],[-105.27599399999991,37.645223999999985],[-105.27867599999996,37.646352000000036],[-105.27994999999993,37.64637499999998],[-105.28152,37.645380999999986],[-105.28334699999999,37.6452560000003],[-105.28491999999994,37.64588600000002],[-105.28554699999995,37.6468790000003],[-105.28700499999997,37.65095300000007],[-105.28969699999993,37.650442000000226],[-105.29081999999977,37.651546999999994],[-105.29410899999976,37.65282700000006],[-105.29558699999995,37.65441600000008],[-105.29540599999979,37.65679700000004],[-105.29620599999998,37.65787800000015],[-105.29762299999999,37.65814300000005],[-105.29870499999998,37.65664099999992],[-105.30180100000001,37.65360299999992],[-105.30319699999995,37.65308500000009],[-105.30911699999996,37.653434000000175],[-105.309844,37.650526999999954],[-105.31335599999994,37.64971600000001],[-105.31577399999992,37.64815500000003],[-105.31834400000002,37.648806000000036],[-105.32228900000001,37.64617800000002],[-105.32749899999999,37.64434],[-105.329542,37.64298100000019],[-105.334945,37.64326500000004],[-105.33715099999995,37.64264100000014],[-105.34544800000003,37.64202400000005],[-105.34695499999987,37.64116600000011],[-105.34855999999996,37.63775800000002],[-105.35202099999998,37.638196999999934],[-105.35450899999995,37.637241000000074],[-105.35503399999988,37.63615800000002],[-105.35677399999997,37.63672100000002],[-105.35967799999997,37.63641799999999],[-105.36111499999987,37.636666000000105],[-105.36404099999993,37.63845400000025],[-105.36584099999993,37.63824100000022],[-105.36824000000001,37.63857600000023],[-105.37030499999997,37.63939600000003],[-105.37080900000001,37.63824899999997],[-105.3737509999998,37.63579099999998],[-105.37689,37.63445200000024],[-105.37841000000003,37.63169199999999],[-105.38072899999986,37.62887999999998],[-105.38079699999997,37.62795300000022],[-105.38457099999977,37.62760500000002],[-105.386551,37.62643800000012],[-105.389837,37.62179299999997],[-105.38966899999997,37.62116599999996],[-105.39136499999995,37.61835500000029],[-105.39342599999998,37.61709500000012],[-105.39437699999996,37.61745799999994],[-105.39615000000003,37.616776000000016],[-105.39985799999994,37.61423800000017],[-105.40107499999993,37.61433899999997],[-105.403256,37.61565700000017],[-105.40839999999992,37.61800000000022],[-105.41085199999992,37.61698300000012],[-105.41491099999996,37.61588300000005],[-105.41699699999998,37.61741299999994],[-105.42090499999995,37.61671900000027],[-105.42350699999986,37.616780000000176],[-105.4285099999999,37.61390500000016],[-105.43158299999988,37.61338599999999],[-105.43602099999998,37.613578000000075],[-105.43834099999992,37.61328800000007],[-105.43995000000001,37.612188],[-105.44227899999998,37.611851],[-105.44313799999992,37.61107800000019],[-105.44504399999988,37.607875000000206],[-105.4458869999998,37.604287],[-105.44817199999994,37.60240900000002],[-105.44784999999996,37.60025200000007],[-105.45130199999994,37.59846700000003],[-105.45245399999988,37.5961970000003],[-105.45522399999999,37.59391499999998],[-105.45452799999998,37.59324900000007],[-105.45494400000001,37.59110400000026],[-105.45449200000002,37.590483000000006],[-105.46026799999999,37.58672999999999],[-105.4633209999999,37.585952000000304],[-105.46447799999999,37.584609],[-105.46727499999986,37.58442400000001],[-105.469855,37.58187100000015],[-105.47144700000001,37.58126300000009],[-105.47594100000003,37.580629000000044],[-105.47884299999998,37.57987700000018],[-105.48095399999994,37.57865500000014],[-105.48551600000002,37.577899000000116],[-105.48594899999995,37.57927900000027],[-105.49057399999992,37.582620000000304],[-105.49215499999991,37.58268100000021],[-105.49468300000001,37.585541999999975],[-105.49413500000003,37.58907700000003],[-105.49459100000001,37.59235200000012],[-105.495635,37.59244699999999],[-105.50091299999997,37.59609700000004],[-105.501755,37.5975710000003],[-105.50128599999994,37.59860300000025],[-105.49327299999976,37.600919000000204],[-105.49349299999994,37.604449000000045],[-105.49304099999995,37.60528699999992],[-105.49483099999998,37.60849899999994],[-105.49609299999997,37.612348999999995],[-105.49792400000001,37.61306800000017],[-105.498602,37.61413700000014],[-105.49829199999999,37.61571600000025],[-105.497299,37.616749000000084],[-105.49638600000003,37.619185000000016],[-105.49260700000002,37.62306700000005],[-105.491513,37.62500300000005],[-105.49219799999992,37.62637400000011],[-105.49183699999998,37.63381700000008],[-105.492388,37.634783000000084],[-105.49035100000003,37.638667999999996],[-105.49020699999994,37.640386000000035],[-105.4878159999999,37.644521999999995],[-105.48685799999987,37.64966199999998],[-105.48697600000003,37.65348299999994],[-105.48540200000002,37.654730000000086],[-105.48145099999994,37.658525000000225],[-105.48111999999992,37.66170300000027],[-105.477351,37.66574700000001],[-105.4772769999999,37.667158000000086],[-105.47856799999988,37.668476000000226],[-105.47878699999995,37.67210200000022],[-105.478519,37.67334500000021],[-105.476899,37.675850000000025],[-105.47665099999989,37.67916400000013],[-105.47738900000002,37.681202000000155],[-105.47770699999995,37.68461100000013],[-105.47875699999997,37.68613900000008],[-105.48089900000002,37.68718000000024],[-105.48086899999993,37.69185600000026],[-105.48137399999996,37.69375500000001],[-105.48016599999994,37.69663800000012],[-105.48047600000001,37.69807600000013],[-105.479062,37.700155000000166],[-105.47792099999998,37.704162999999994],[-105.47807099999989,37.70577800000001],[-105.47730200000001,37.70768600000014],[-105.47449299999994,37.70760100000001],[-105.47212799999994,37.70946800000013],[-105.47111599999988,37.71124800000001],[-105.46797499999985,37.711260000000266],[-105.46659299999999,37.71184499999998],[-105.46392599999996,37.71380099999999],[-105.46325999999982,37.71599800000007],[-105.46165000000002,37.717419000000234],[-105.45963099999994,37.71792600000026],[-105.45715599999988,37.7198640000002],[-105.45510999999999,37.72315200000014],[-105.45424500000001,37.72395099999994],[-105.45436599999988,37.72606500000006],[-105.45361600000001,37.729813000000036],[-105.45358499999992,37.73198400000007],[-105.455104,37.733601000000135],[-105.45504799999992,37.73658700000027],[-105.45276399999995,37.74040499999995],[-105.45426099999997,37.744462999999996],[-105.45668899999998,37.74599500000011],[-105.45734199999993,37.74702400000001],[-105.45696399999991,37.748646000000065],[-105.45725499999992,37.751464],[-105.45978899999989,37.75276500000018],[-105.45957699999991,37.7536290000001],[-105.455827,37.756498000000136],[-105.45627300000001,37.75806800000004],[-105.4578689999999,37.75867699999998],[-105.45770400000004,37.76013000000012],[-105.45591100000001,37.76242700000006],[-105.45370300000002,37.76728400000013],[-105.45241599999991,37.76789900000006],[-105.45023299999986,37.77060999999998],[-105.45097099999981,37.77303300000028],[-105.45091599999989,37.77519500000028],[-105.44945399999989,37.77812700000004],[-105.44976899999995,37.780754],[-105.45170499999989,37.78263500000003],[-105.45142799999996,37.784759000000065],[-105.45166599999976,37.78689500000007],[-105.45245299999988,37.78829300000007],[-105.45518699999997,37.790814000000125],[-105.45480199999997,37.79328499999997],[-105.45491199999975,37.795696000000305],[-105.45368199999996,37.7968590000001],[-105.450695,37.7976560000003],[-105.44739199999998,37.80006100000014],[-105.44223699999998,37.802367000000174],[-105.44074,37.80250700000022],[-105.43689699999999,37.803641000000084],[-105.43517899999995,37.80476699999997],[-105.43505599999992,37.806301000000246],[-105.43582299999986,37.80782000000016],[-105.43552499999998,37.81082300000014],[-105.43591100000003,37.81179599999996],[-105.43466799999987,37.81373300000001],[-105.43552999999997,37.814637000000005],[-105.43673899999999,37.81921799999998],[-105.434573,37.821929000000296],[-105.43409699999995,37.824505999999985],[-105.43509299999994,37.82648300000017],[-105.4355829999999,37.82850500000012],[-105.43240700000001,37.83316700000006],[-105.43262300000004,37.8350690000002],[-105.43073099999998,37.837826000000234],[-105.43096500000001,37.838753],[-105.42867399999989,37.84366600000027],[-105.42864699999996,37.84477899999996],[-105.43091700000002,37.84736000000021],[-105.4305579999999,37.848653999999954],[-105.42889299999996,37.851733000000024],[-105.42915499999998,37.85236000000003],[-105.43188899999979,37.85482400000012],[-105.4334619999999,37.85751599999992],[-105.43677399999996,37.858202000000006],[-105.43721399999993,37.860231000000226],[-105.43894899999998,37.86127500000009],[-105.43946899999992,37.86324800000011],[-105.43924099999987,37.86620200000027],[-105.44059499999992,37.868353999999954],[-105.440742,37.86961200000013],[-105.44330899999994,37.870721999999944],[-105.44848200000001,37.876378999999986],[-105.45053699999988,37.87811199999999],[-105.4540849999999,37.87850100000003],[-105.45658799999995,37.87989399999998],[-105.460106,37.88035500000012],[-105.46454599999993,37.878974000000085],[-105.46530899999999,37.87953000000027],[-105.46915200000001,37.87875500000007],[-105.47191799999996,37.879687000000104],[-105.47758099999993,37.88044200000007],[-105.47621399999997,37.8832950000002],[-105.47705500000001,37.887961000000075],[-105.475435,37.89237300000008],[-105.47365899999988,37.894516000000124],[-105.4732039999999,37.89596999999998],[-105.47105699999997,37.895264000000054],[-105.46696999999995,37.89551499999999],[-105.461319,37.89689099999998],[-105.45923500000004,37.897169000000076],[-105.45236,37.89568100000025],[-105.440272,37.89746100000008],[-105.43727899999999,37.896011000000044],[-105.43197199999992,37.895167000000015],[-105.4298859999999,37.893837000000076],[-105.42779000000002,37.89132000000001],[-105.42685499999993,37.89163400000024],[-105.42415099999994,37.89017299999995],[-105.42159700000002,37.89065199999999],[-105.41868899999992,37.88970300000017],[-105.41548599999993,37.890814999999975],[-105.413635,37.890527000000304],[-105.41155300000003,37.8925320000003],[-105.40978399999995,37.893205000000194],[-105.40869699999996,37.894442000000026],[-105.40508699999998,37.89515],[-105.40342099999998,37.895782000000054],[-105.399585,37.898816000000295],[-105.394792,37.90136600000005],[-105.39407999999997,37.90291800000023],[-105.39030500000001,37.905196000000046],[-105.38962799999996,37.906521999999995],[-105.38503999999995,37.91017799999997],[-105.38398399999988,37.910469999999975],[-105.38056399999988,37.91332499999993],[-105.37839600000001,37.91379999999998],[-105.37708799999996,37.91610200000002],[-105.37500299999994,37.917115000000194],[-105.36782900000003,37.91978400000022],[-105.36540999999988,37.92142000000024],[-105.36113899999998,37.92259700000011],[-105.35605399999997,37.92285400000003],[-105.35081399999984,37.92488900000001],[-105.34720900000002,37.92475900000028],[-105.344536,37.92637800000011],[-105.34207600000002,37.92841200000004],[-105.34007500000001,37.92932399999995],[-105.3374609999999,37.93205500000016],[-105.33570999999989,37.933468000000005],[-105.331702,37.93580100000014],[-105.32452599999993,37.93843400000003],[-105.31708099999997,37.93894900000004],[-105.31450699999988,37.93844399999995],[-105.31377199999991,37.93688800000001],[-105.31109300000003,37.93594499999995],[-105.31028099999997,37.93354000000011],[-105.30657399999996,37.932292000000075],[-105.30631599999992,37.93096700000001],[-105.30436599999996,37.92839300000003],[-105.30360699999977,37.926188000000025],[-105.30054099999995,37.92308500000013],[-105.300388,37.921586000000104],[-105.30165899999997,37.91811899999993],[-105.30065499999995,37.915887],[-105.30111499999992,37.91462700000022],[-105.300254,37.91248800000017],[-105.29801099999997,37.911263000000076],[-105.29743100000002,37.909349000000134],[-105.29481099999987,37.90822000000014],[-105.29240699999991,37.908625000000086],[-105.29154299999999,37.90754599999997],[-105.29078399999992,37.90536800000007],[-105.29103999999995,37.90389700000003],[-105.28936699999991,37.90286900000007],[-105.28673600000002,37.90019400000011],[-105.28507599999995,37.899421000000075],[-105.28054899999995,37.901215000000036],[-105.27892399999996,37.90364499999998],[-105.2769879999999,37.90410300000002],[-105.27623299999988,37.905875000000094],[-105.273818,37.90767699999998],[-105.27341799999994,37.910716000000264],[-105.271433,37.91281500000002],[-105.26990799999987,37.91366100000022],[-105.26804999999996,37.91830399999998],[-105.264815,37.921377000000234],[-105.26184199999994,37.92563700000028],[-105.26379999999995,37.9288200000002],[-105.26394199999987,37.93205899999998],[-105.26316799999978,37.93245800000011],[-105.26154899999995,37.93716800000027],[-105.25754599999993,37.938131],[-105.25676399999998,37.94255499999997],[-105.25543599999997,37.94291599999997],[-105.25001799999995,37.950378000000114],[-105.24861699999985,37.95210899999995],[-105.24669399999999,37.95211000000023],[-105.24471599999993,37.95372800000001],[-105.2417989999999,37.95380899999992],[-105.24003299999998,37.955080000000294],[-105.23759799999993,37.957568000000094],[-105.23868899999997,37.95941900000031],[-105.23993299999995,37.96039300000007],[-105.23855999999995,37.96284000000003],[-105.23580899999996,37.96280000000007],[-105.23442599999993,37.964019000000235],[-105.22941800000001,37.965245000000095],[-105.22692099999989,37.964567999999986],[-105.22464599999995,37.965036],[-105.22468300000003,37.96667400000018],[-105.2231359999999,37.968804000000205],[-105.21809499999989,37.96990299999999],[-105.21662700000002,37.96944700000029],[-105.21536599999996,37.97052000000025],[-105.21635799999996,37.97253000000029],[-105.21650499999998,37.975378000000035],[-105.21463299999988,37.97688700000003],[-105.21310899999992,37.97733100000016],[-105.2142209999999,37.980260000000214],[-105.21398599999992,37.980808999999965],[-105.20924099999996,37.982228000000134],[-105.20770699999997,37.98165499999999],[-105.20589699999977,37.983268000000066],[-105.20583699999997,37.98861899999997],[-105.2060679999999,37.98966800000005],[-105.20192400000002,37.99044800000013],[-105.19881399999991,37.990181000000234],[-105.19707299999993,37.99241600000005],[-105.20004399999999,38.00083499999994],[-105.19891899999999,38.00378499999999],[-105.19756799999993,38.00436600000006],[-105.19458599999996,38.00756000000001],[-105.19280800000001,38.00861600000019],[-105.18928299999993,38.00975200000005],[-105.18774999999994,38.010598000000016],[-105.18525199999993,38.0133790000001],[-105.18181900000002,38.014693000000136],[-105.18100599999997,38.01559500000013],[-105.17890299999988,38.01647400000002],[-105.17670799999979,38.01635200000004],[-105.17326399999996,38.017142000000035],[-105.17173100000002,38.01804400000026],[-105.16865200000001,38.018946000000085],[-105.166898,38.01812899999999],[-105.16582599999987,38.01682099999999],[-105.16110099999997,38.01586600000019],[-105.16076499999997,38.01494700000001],[-105.158366,38.01313299999998],[-105.15729899999991,38.012955000000034],[-105.15286800000001,38.00974699999995],[-105.15141399999987,38.00844799999999],[-105.14796899999993,38.006239000000164],[-105.148416,38.005078000000196],[-105.15038599999991,38.004547000000116],[-105.15190999999987,38.003330000000005],[-105.15319899999997,38.00078400000018],[-105.15226099999995,37.99435599999998],[-105.152491,37.99211300000002],[-105.15071699999993,37.99078900000006],[-105.14796499999989,37.986841000000084],[-105.14645200000001,37.98658900000021],[-105.14390900000001,37.98783800000001],[-105.14124099999998,37.98686000000009],[-105.139501,37.98460699999998],[-105.14164099999994,37.98203400000011],[-105.14214499999997,37.97979900000007],[-105.1405499999999,37.979774000000134],[-105.13235999999995,37.97464299999996],[-105.13102499999997,37.97268100000002],[-105.13133500000004,37.9711880000001],[-105.13319499999983,37.97042300000004],[-105.13558,37.96729400000015],[-105.13564699999995,37.96527700000024],[-105.13340899999997,37.96303900000015],[-105.12790899999999,37.962591999999916],[-105.12757599999998,37.96117400000003],[-105.12641099999996,37.9607500000003],[-105.12618799999996,37.95781200000022],[-105.12710499999991,37.956457],[-105.12690599999996,37.95513400000016],[-105.12555099999986,37.95459200000005],[-105.12511099999995,37.95308700000004],[-105.12597499999993,37.951170000000104],[-105.12261199999995,37.94917700000002],[-105.12066599999986,37.94959699999998],[-105.117163,37.949141000000054],[-105.11560799999995,37.94749999999999],[-105.11298399999993,37.94544900000011],[-105.10850700000003,37.94339899999994],[-105.10312899999997,37.94494000000009],[-105.09909700000003,37.94494300000002],[-105.09382799999992,37.94732900000008],[-105.09029499999997,37.947636999999986],[-105.088011,37.94858800000014],[-105.08553199999989,37.95114100000001],[-105.08438299999983,37.95538400000004],[-105.08182999999991,37.95425],[-105.07779999999997,37.95296400000001],[-105.07424400000002,37.94851300000016],[-105.07221899999996,37.947789000000284],[-105.070472,37.945802000000015],[-105.06605599999995,37.944209],[-105.06547099999995,37.941802000000166],[-105.06642499999992,37.938525000000084],[-105.06603000000001,37.93744700000008],[-105.06434199999995,37.936635000000024],[-105.06357200000002,37.935001],[-105.063737,37.92926900000003],[-105.06606299999999,37.92624999999998],[-105.06630099999995,37.925048000000004],[-105.06503399999986,37.923241000000075],[-105.06513499999994,37.921456000000035],[-105.06584999999995,37.92076300000002],[-105.06319999999988,37.916976000000034],[-105.06149700000003,37.91606100000001],[-105.05720100000002,37.915946000000076],[-105.05430699999994,37.91750900000028],[-105.04991699999988,37.915479000000005]]]},"properties":{"OBJECTID":29,"COUNTYFP":"055","Shape_Leng":3.86938876795,"Shape_Area":0.421500955876,"low":2220,"lowmod":3925,"LMMI":4815,"lowmoduniv":6285,"lowmod_pct":0.624502784407,"NAMELSAD10":"Huerfano County"}},
{"type":"Feature","geometry":{"type":"Polygon","coordinates":[[[-106.19055400000002,40.99757800000003],[-106.19155599999988,40.996285],[-106.1923569999999,40.99368800000025],[-106.19423099999995,40.99197000000021],[-106.19537199999985,40.98993800000011],[-106.19493999999992,40.98693600000007],[-106.191124,40.986030000000255],[-106.19055300000002,40.9842550000003],[-106.19213899999988,40.982836000000304],[-106.19004000000001,40.97864000000004],[-106.19040499999994,40.976098000000036],[-106.18972999999994,40.9730890000003],[-106.187097,40.971167000000264],[-106.18675100000002,40.968052000000114],[-106.18770099999995,40.96490600000004],[-106.184844,40.963831000000255],[-106.18552699999992,40.96193599999998],[-106.18525299999999,40.9610320000001],[-106.18683799999997,40.95988800000015],[-106.18739099999988,40.95872000000014],[-106.18682899999999,40.957425000000285],[-106.18422599999991,40.95520700000026],[-106.185629,40.952248000000054],[-106.18765099999979,40.95016600000025],[-106.18929300000002,40.94589300000001],[-106.18716599999999,40.940825000000075],[-106.18759899999986,40.93774500000001],[-106.18617,40.9367270000003],[-106.18530699999991,40.93397299999998],[-106.181624,40.932018000000085],[-106.17994499999992,40.931521000000146],[-106.17872799999986,40.93025700000004],[-106.17589299999986,40.92825400000004],[-106.17343499999987,40.92478300000005],[-106.17002799999989,40.923804000000246],[-106.16908399999994,40.92286800000005],[-106.1694849999999,40.920315000000016],[-106.16816999999998,40.91839800000008],[-106.16304399999996,40.91672699999998],[-106.16116599999992,40.91510700000026],[-106.15920199999994,40.91472599999997],[-106.1595769999999,40.912765999999976],[-106.15855099999999,40.91061900000017],[-106.15459299999998,40.90942100000018],[-106.15219799999988,40.906789000000174],[-106.15069,40.90671500000008],[-106.14788099999993,40.904638000000205],[-106.144069,40.9049],[-106.14177699999988,40.903962000000035],[-106.13928599999991,40.90240200000028],[-106.13662199999987,40.901859000000115],[-106.13701799999996,40.90051999999997],[-106.13622499999997,40.89922800000022],[-106.13410199999998,40.89870900000005],[-106.13125100000002,40.89660800000013],[-106.13073400000002,40.895493999999985],[-106.13142999999997,40.892865000000256],[-106.128987,40.88928099999998],[-106.12904300000002,40.88794200000024],[-106.12816399999986,40.88702100000023],[-106.12761799999998,40.88488999999993],[-106.12825099999992,40.882002000000284],[-106.12997999999999,40.8782130000003],[-106.13242499999996,40.87655300000006],[-106.1306709999999,40.87386800000013],[-106.12769099999997,40.8717850000001],[-106.12689299999994,40.869556000000046],[-106.12464699999998,40.86880800000017],[-106.12254899999994,40.86736400000024],[-106.12080299999991,40.867632000000015],[-106.11897099999993,40.865775999999926],[-106.116536,40.866441000000066],[-106.11319199999997,40.86427900000007],[-106.10840399999995,40.863258000000144],[-106.10776299999992,40.861368000000084],[-106.10469799999998,40.86032900000009],[-106.10428099999996,40.859263],[-106.10628399999996,40.85407500000008],[-106.10773999999998,40.853024000000175],[-106.10749399999997,40.85207899999995],[-106.10422899999992,40.851985000000184],[-106.10244199999994,40.8523340000001],[-106.09851500000002,40.849736000000064],[-106.09899099999996,40.84804700000012],[-106.10013800000002,40.846980000000144],[-106.10126100000002,40.84356300000002],[-106.09877999999998,40.841699000000176],[-106.09553699999998,40.84164500000014],[-106.0926859999999,40.840839000000244],[-106.09249399999999,40.83978100000007],[-106.09050200000001,40.83718100000027],[-106.08859599999988,40.836133000000245],[-106.08791399999996,40.833402999999976],[-106.08322099999998,40.83117699999997],[-106.08086099999991,40.830903000000035],[-106.07782799999995,40.828498000000195],[-106.07639199999988,40.82792500000005],[-106.07530600000001,40.8256090000001],[-106.07312499999995,40.82463299999995],[-106.07131600000002,40.82233100000008],[-106.06843500000002,40.822196000000304],[-106.06483099999991,40.8215670000003],[-106.06253300000003,40.81950900000004],[-106.05771499999997,40.817120000000045],[-106.05779799999993,40.814437999999996],[-106.05636200000004,40.81350999999995],[-106.05509699999999,40.813631000000214],[-106.05377900000002,40.81266200000016],[-106.0532879999999,40.81041700000003],[-106.05150199999991,40.809140999999954],[-106.05165899999992,40.80543400000022],[-106.04958699999992,40.805507000000205],[-106.04797099999996,40.80464300000011],[-106.04647099999983,40.80244700000014],[-106.046694,40.80079100000012],[-106.044896,40.797682000000066],[-106.04597899999999,40.7963730000003],[-106.04600999999997,40.79428899999999],[-106.04537099999976,40.79185700000005],[-106.044308,40.790581000000145],[-106.04280999999997,40.79012999999998],[-106.0412579999998,40.78843399999994],[-106.03965199999999,40.78576000000004],[-106.03521099999989,40.78555100000017],[-106.03196999999994,40.78376700000001],[-106.03123599999998,40.782281000000125],[-106.02968499999997,40.78156200000001],[-106.03037499999988,40.77910600000001],[-106.02885499999996,40.77657800000026],[-106.02956599999999,40.77506800000003],[-106.02924699999988,40.773436000000174],[-106.02448800000002,40.77126400000003],[-106.02664299999998,40.76831500000009],[-106.02433799999989,40.76629600000001],[-106.02334999999994,40.76353400000011],[-106.0238799999999,40.76132800000005],[-106.02179799999988,40.75907499999994],[-106.02067299999987,40.75842900000015],[-106.02065099999999,40.75720099999995],[-106.02191500000004,40.75606200000004],[-106.02225499999992,40.754688999999985],[-106.01888799999995,40.75400300000024],[-106.01526599999994,40.75281600000005],[-106.01483100000002,40.7513780000001],[-106.015447,40.7506030000003],[-106.01435099999992,40.74860600000005],[-106.01397799999995,40.74664400000029],[-106.01268999999996,40.74549000000019],[-106.01221499999991,40.743946000000165],[-106.01035999999982,40.74240100000003],[-106.01069999999999,40.74104099999994],[-106.0083899999999,40.7400640000003],[-106.0060079999999,40.737618999999995],[-106.00402599999995,40.73679400000009],[-106.00056899999993,40.73433700000021],[-105.99867699999987,40.734222999999986],[-105.99626999999992,40.7331260000002],[-105.991965,40.728919000000076],[-105.99057999999997,40.72718000000026],[-105.99093900000003,40.72480100000024],[-105.98928399999994,40.72352000000012],[-105.98807899999997,40.72136800000004],[-105.99007699999999,40.71831300000008],[-105.98958099999999,40.71614500000027],[-105.98976299999987,40.71398199999999],[-105.98873899999995,40.70960500000024],[-105.98602099999988,40.70762700000017],[-105.98244799999992,40.70669200000026],[-105.98136099999994,40.70483200000001],[-105.97941699999996,40.70328000000001],[-105.97708499999999,40.702648000000124],[-105.97347300000001,40.69657400000028],[-105.97219599999977,40.69481100000007],[-105.97375999999997,40.691129000000046],[-105.97358399999996,40.68927400000024],[-105.9700259999999,40.68626499999999],[-105.97012699999993,40.68471600000004],[-105.9711089999999,40.684036000000276],[-105.97187299999996,40.68185600000021],[-105.96939199999997,40.67773999999997],[-105.96774899999997,40.67705000000012],[-105.96666399999992,40.67568100000017],[-105.96645499999994,40.67345499999999],[-105.96511399999991,40.671418000000074],[-105.96503899999993,40.66820700000011],[-105.96229199999993,40.666584000000114],[-105.95954099999994,40.66354899999999],[-105.958639,40.663009999999986],[-105.95783799999992,40.66087399999998],[-105.95588299999997,40.65846699999997],[-105.95668599999999,40.654964000000064],[-105.95598999999987,40.65259500000013],[-105.95476500000001,40.65020999999996],[-105.95461999999992,40.648072000000184],[-105.95384399999995,40.64697699999999],[-105.95107899999977,40.64587800000004],[-105.95081600000003,40.64328900000021],[-105.94915300000002,40.642776000000026],[-105.9483249999999,40.641745000000014],[-105.948062,40.635986000000116],[-105.94857300000001,40.63367800000026],[-105.94866699999994,40.63018500000027],[-105.94926099999998,40.62732],[-105.9471989999999,40.62374600000004],[-105.94549499999988,40.62274100000002],[-105.94511,40.62119100000001],[-105.94700199999994,40.617367],[-105.94658599999991,40.616028000000085],[-105.94474200000002,40.614231000000245],[-105.94453499999986,40.61209200000019],[-105.94206400000002,40.6090770000003],[-105.93868099999997,40.60881100000012],[-105.93471099999988,40.60642999999993],[-105.930207,40.60618999999997],[-105.92895299999998,40.603794000000164],[-105.92899999999986,40.601202000000285],[-105.9281299999999,40.60005799999993],[-105.92822099999995,40.59830500000021],[-105.92770499999989,40.594923000000165],[-105.92811599999999,40.59416299999998],[-105.92769699999997,40.59170900000015],[-105.92654299999987,40.59058900000002],[-105.92662399999989,40.589175000000125],[-105.92577499999999,40.58770700000014],[-105.92438799999991,40.58214600000008],[-105.92497699999996,40.580934000000184],[-105.9234249999999,40.57703700000013],[-105.92224799999985,40.57530300000002],[-105.92124799999999,40.57255900000018],[-105.91844399999997,40.570700000000215],[-105.91469399999994,40.570636000000036],[-105.91320999999988,40.568313000000046],[-105.91089199999993,40.56642000000028],[-105.909784,40.562134000000015],[-105.90991400000001,40.558806000000004],[-105.91081799999995,40.55671200000012],[-105.911833,40.552688000000046],[-105.9095099999999,40.54793000000001],[-105.90957700000001,40.544982000000005],[-105.90704900000003,40.54288800000012],[-105.90714999999994,40.540577999999925],[-105.90553699999998,40.5370200000001],[-105.90508999999986,40.535285000000044],[-105.905914,40.530519000000254],[-105.90786800000001,40.52621000000005],[-105.90976399999988,40.523452000000134],[-105.9111749999999,40.5221810000001],[-105.91077999999999,40.52017900000021],[-105.90785699999992,40.515493000000106],[-105.90482399999996,40.51421599999992],[-105.90417000000002,40.514443000000256],[-105.89911099999995,40.51822499999997],[-105.89457599999997,40.520196],[-105.89110399999993,40.52100300000018],[-105.88385199999988,40.52175100000005],[-105.88195399999995,40.52285300000011],[-105.87904099999997,40.52317400000021],[-105.87804799999998,40.52246500000001],[-105.87496199999998,40.52192900000023],[-105.871805,40.52174800000017],[-105.86909099999997,40.52073800000028],[-105.86751299999997,40.51971400000008],[-105.8663689999999,40.5172570000002],[-105.86310900000001,40.51552100000026],[-105.86189999999999,40.51444900000001],[-105.86062099999992,40.51154900000006],[-105.85820299999989,40.50962900000019],[-105.857058,40.50785600000006],[-105.85473200000001,40.507789],[-105.85230299999995,40.50655300000005],[-105.85229699999996,40.503534],[-105.85175499999991,40.50047500000022],[-105.85202600000002,40.49756900000011],[-105.85477300000002,40.49510600000008],[-105.8557869999999,40.49268200000006],[-105.85610500000001,40.49041199999999],[-105.85582199999988,40.48773800000009],[-105.85492599999998,40.48625200000009],[-105.85717499999998,40.483271],[-105.86059499999999,40.48203500000005],[-105.86247400000002,40.48022600000007],[-105.86558499999995,40.48137300000013],[-105.86915199999987,40.48098600000009],[-105.87113599999998,40.48006500000008],[-105.87226499999997,40.478296000000284],[-105.874101,40.4784410000002],[-105.87914999999992,40.47709400000031],[-105.88154800000001,40.47586600000011],[-105.88601900000003,40.4773430000003],[-105.88869299999993,40.476413000000264],[-105.89175399999999,40.47666099999998],[-105.89221299999997,40.47414800000007],[-105.89430500000003,40.46954700000026],[-105.89722499999988,40.469205000000045],[-105.89920899999998,40.46575800000028],[-105.89824799999997,40.46036099999992],[-105.89833399999998,40.457397000000014],[-105.89982399999997,40.45450000000011],[-105.89908500000001,40.452517],[-105.89889099999988,40.45025800000013],[-105.89706899999993,40.44878800000021],[-105.8987719999999,40.446206000000075],[-105.89981399999994,40.44076899999999],[-105.90142000000003,40.43787899999995],[-105.90217599999988,40.43565600000005],[-105.90183300000001,40.43477800000011],[-105.90326500000003,40.43276200000025],[-105.90267900000003,40.43034200000022],[-105.90043100000003,40.428075000000035],[-105.89857799999999,40.426806000000056],[-105.89850000000001,40.4242960000002],[-105.897831,40.42358000000013],[-105.89771000000002,40.420948000000294],[-105.898098,40.41898500000008],[-105.89766900000001,40.41772700000024],[-105.89901299999991,40.415211000000056],[-105.90209800000002,40.41290700000002],[-105.90231299999994,40.410258000000056],[-105.90165799999983,40.406490000000076],[-105.90319299999993,40.40265699999998],[-105.90331999999995,40.400751000000014],[-105.90506799999997,40.398815999999954],[-105.90598999999997,40.39965200000006],[-105.90916900000002,40.40076200000004],[-105.91084299999989,40.400513000000046],[-105.91359999999992,40.40161700000027],[-105.91661599999998,40.40186299999999],[-105.91919399999995,40.40132900000009],[-105.92073399999998,40.40156500000023],[-105.92207099999996,40.39812000000029],[-105.92347299999994,40.3964840000001],[-105.92440999999997,40.393162000000075],[-105.92753600000003,40.39102600000024],[-105.93081699999999,40.39023800000024],[-105.93178999999986,40.3890880000003],[-105.93386099999998,40.38850600000012],[-105.93632200000002,40.3861960000001],[-105.94046199999991,40.38452399999994],[-105.9425399999999,40.38492700000012],[-105.943896,40.38454200000001],[-105.94660699999997,40.3850470000001],[-105.94591899999995,40.38144900000009],[-105.94722299999995,40.37806900000021],[-105.94671199999993,40.37724800000012],[-105.94162299999988,40.37412899999998],[-105.93731199999996,40.37194099999999],[-105.93872399999998,40.368684000000144],[-105.93870099999987,40.366058000000294],[-105.94021399999991,40.36581400000017],[-105.94255699999997,40.364340000000084],[-105.94981200000001,40.36294700000008],[-105.95117899999997,40.36175300000025],[-105.95434299999982,40.36029700000006],[-105.95543900000001,40.3548100000001],[-105.95740999999992,40.353007000000105],[-105.9576199999999,40.351692000000014],[-105.959723,40.34938100000011],[-105.95965899999993,40.34814900000015],[-105.96240299999988,40.348050000000114],[-105.96614999999986,40.34952300000015],[-105.96936799999992,40.34955000000025],[-105.97231,40.351561000000004],[-105.97314900000003,40.35469000000012],[-105.9753639999999,40.357617000000005],[-105.9763699999998,40.35828700000002],[-105.97834499999999,40.35802300000017],[-105.98126300000001,40.35963099999998],[-105.9838529999999,40.36313400000029],[-105.98665399999993,40.36458200000004],[-105.99220399999996,40.36418900000018],[-105.99439499999977,40.36456800000008],[-105.99667099999999,40.36380300000002],[-105.99834099999998,40.3624930000002],[-106.00777399999993,40.362476000000015],[-106.01085099999995,40.36189999999999],[-106.01112999999987,40.36395600000009],[-106.01254,40.36548299999998],[-106.01307500000001,40.36797700000005],[-106.01160499999992,40.37027600000005],[-106.0133019999999,40.37108800000027],[-106.01458100000002,40.372863999999936],[-106.01862199999988,40.37329500000027],[-106.02456099999995,40.371777000000066],[-106.02603699999986,40.37085500000018],[-106.02738999999997,40.37086700000009],[-106.02983399999988,40.36843200000004],[-106.03117999999995,40.36826700000012],[-106.03602799999999,40.369099000000006],[-106.0397109999999,40.36665000000005],[-106.04085499999997,40.36477500000018],[-106.04338399999995,40.36289599999998],[-106.046986,40.361864000000196],[-106.0505389999999,40.359454000000085],[-106.05182100000002,40.35929599999997],[-106.05456800000002,40.35763600000013],[-106.05790899999988,40.35702900000018],[-106.06111499999992,40.35741800000005],[-106.06324299999994,40.357242999999926],[-106.06439599999999,40.356601000000296],[-106.06576899999988,40.35435700000005],[-106.06995199999994,40.352693000000045],[-106.07171299999999,40.35232899999994],[-106.07653199999993,40.34995199999992],[-106.08568100000002,40.34718299999997],[-106.08744399999995,40.35037900000009],[-106.09152799999993,40.35009100000025],[-106.09497799999991,40.35114199999998],[-106.09777099999991,40.35284899999999],[-106.09938999999991,40.3544730000001],[-106.10162099999985,40.353840000000105],[-106.10505699999993,40.3501710000001],[-106.114826,40.34630500000014],[-106.116738,40.34585199999998],[-106.12047099999995,40.34379000000007],[-106.12121100000002,40.34252400000031],[-106.12789899999996,40.3380420000002],[-106.13138799999996,40.33663800000005],[-106.13317099999978,40.33651299999997],[-106.13469900000001,40.33543900000029],[-106.13637199999988,40.33029000000005],[-106.1414539999999,40.330054000000246],[-106.14675599999993,40.33139400000027],[-106.14752699999985,40.33208999999994],[-106.14938399999994,40.33196700000008],[-106.15224499999994,40.335359000000096],[-106.15639599999997,40.33704900000009],[-106.15761199999992,40.3389810000001],[-106.15965599999993,40.340483000000006],[-106.160686,40.34050000000002],[-106.162011,40.342612000000145],[-106.16429699999998,40.343328999999926],[-106.168856,40.34354599999995],[-106.17048999999992,40.344006000000036],[-106.17473399999989,40.343183000000124],[-106.17574799999994,40.34150800000009],[-106.17740699999996,40.340565000000026],[-106.17919399999994,40.338264000000265],[-106.180137,40.33601400000026],[-106.17982899999998,40.333991000000026],[-106.1810299999998,40.333424000000036],[-106.18900500000001,40.3276800000001],[-106.18917799999986,40.32653200000016],[-106.19272999999993,40.32585300000011],[-106.1947669999999,40.32646700000009],[-106.1969269999999,40.32561700000031],[-106.19886599999995,40.327107000000126],[-106.20173399999993,40.32645100000002],[-106.20621099999988,40.32789000000025],[-106.21149200000002,40.32766300000014],[-106.21435099999997,40.32799100000011],[-106.21706399999994,40.329079999999976],[-106.21869599999985,40.32879700000001],[-106.22099600000001,40.32757900000013],[-106.22171899999995,40.32579099999998],[-106.22566799999976,40.32516900000019],[-106.230074,40.323381999999924],[-106.2307459999999,40.32448599999998],[-106.23621999999995,40.32574599999998],[-106.23898400000002,40.32795600000003],[-106.2389,40.32981600000028],[-106.24121499999995,40.33168400000011],[-106.24265100000002,40.333685000000116],[-106.24343499999998,40.3356940000001],[-106.24477100000001,40.33732199999997],[-106.24666100000002,40.338230000000294],[-106.247318,40.33918799999998],[-106.25122299999998,40.34076500000026],[-106.25090799999992,40.341783000000135],[-106.25165300000003,40.34366599999993],[-106.25356699999986,40.34662600000007],[-106.2535729999999,40.350325000000055],[-106.25555700000001,40.35092400000025],[-106.25678899999991,40.35239700000011],[-106.25768600000004,40.35517700000025],[-106.26034700000002,40.35819000000015],[-106.261435,40.35810400000031],[-106.26566199999996,40.36095000000006],[-106.26920799999994,40.35929500000026],[-106.26959599999998,40.35756200000003],[-106.27121599999998,40.357282],[-106.27542800000003,40.355203000000074],[-106.27676400000001,40.35493600000001],[-106.27794699999993,40.353723],[-106.28057699999994,40.35495300000002],[-106.28422899999993,40.355789999999956],[-106.28662700000001,40.358853000000295],[-106.28840999999983,40.35803800000002],[-106.29045500000001,40.358371000000034],[-106.29551600000002,40.354242],[-106.29551400000003,40.35213799999997],[-106.297077,40.3505370000002],[-106.29862099999986,40.34995800000007],[-106.29718200000002,40.34692700000011],[-106.29869599999995,40.34483300000022],[-106.29886899999997,40.34285299999999],[-106.29805499999992,40.34038900000007],[-106.29645299999999,40.33899200000013],[-106.29386999999997,40.337625],[-106.29647599999993,40.33528400000006],[-106.29597000000001,40.333091000000024],[-106.29833899999988,40.33240600000016],[-106.29924499999998,40.33055999999999],[-106.30283400000002,40.33026600000005],[-106.30533500000001,40.33082900000005],[-106.30578199999997,40.328463000000056],[-106.30805399999991,40.32931500000001],[-106.31224099999997,40.330183999999974],[-106.31622099999993,40.333565000000135],[-106.31973399999998,40.33518800000013],[-106.32122099999992,40.33732500000002],[-106.32278699999989,40.33783499999993],[-106.32947999999999,40.33914800000002],[-106.334562,40.33894300000014],[-106.3371239999999,40.33973000000009],[-106.3374159999999,40.34265900000014],[-106.34146499999997,40.345499000000075],[-106.34388999999999,40.34653100000003],[-106.34520299999991,40.35005100000012],[-106.35037399999993,40.35311100000018],[-106.35145599999993,40.35583700000012],[-106.35480099999995,40.35753700000009],[-106.360098,40.35835000000003],[-106.36365,40.35754600000013],[-106.36423899999994,40.35645999999997],[-106.366308,40.35592200000025],[-106.36825499999998,40.35639700000013],[-106.37000499999999,40.357474000000025],[-106.37246199999998,40.357031000000006],[-106.37493,40.35539500000016],[-106.37803899999994,40.354926999999975],[-106.38122699999991,40.35279700000012],[-106.38309099999987,40.35289399999999],[-106.385264,40.35237300000023],[-106.38646299999999,40.34958800000004],[-106.38815699999998,40.34832699999993],[-106.38905899999992,40.34611100000012],[-106.390872,40.34565000000015],[-106.39429999999987,40.34602500000011],[-106.395828,40.34701200000006],[-106.40010499999994,40.34565399999997],[-106.40401599999979,40.34495199999992],[-106.40560199999993,40.345965999999976],[-106.40768899999995,40.34642000000008],[-106.41032799999999,40.35015599999991],[-106.41403200000002,40.350945000000024],[-106.41709699999996,40.35008000000022],[-106.41965199999999,40.35086200000006],[-106.4200449999999,40.35221400000023],[-106.42179299999992,40.35279299999996],[-106.42362200000002,40.352525000000014],[-106.42745400000001,40.350814000000014],[-106.42919799999987,40.35137599999996],[-106.43192499999992,40.35074800000007],[-106.43303700000001,40.34956],[-106.4361189999999,40.35035600000015],[-106.43908499999992,40.35015100000004],[-106.439525,40.34879200000023],[-106.44341999999995,40.34653300000019],[-106.44433399999991,40.34768200000008],[-106.44586400000003,40.34811200000013],[-106.44820999999996,40.35031100000015],[-106.450062,40.35095499999994],[-106.45219099999986,40.352969000000144],[-106.45428300000003,40.35429400000004],[-106.457018,40.35451000000012],[-106.45782599999978,40.356117999999924],[-106.45554899999996,40.35798399999999],[-106.45630399999999,40.3606410000001],[-106.45614599999999,40.36333700000006],[-106.45833799999997,40.36652800000007],[-106.46131600000001,40.366570000000195],[-106.46317799999997,40.36699300000021],[-106.46300099999996,40.36869699999994],[-106.46413100000001,40.371870000000115],[-106.46439499999985,40.375912000000255],[-106.46711799999991,40.37919299999993],[-106.47058599999997,40.379034000000104],[-106.47305799999987,40.379961000000094],[-106.4711079999999,40.38227599999993],[-106.47121800000002,40.38343600000019],[-106.47264399999989,40.38481100000007],[-106.473477,40.3868360000003],[-106.47449399999994,40.387984000000074],[-106.4778169999999,40.389251000000286],[-106.48128299999996,40.38943100000006],[-106.47968000000003,40.39194900000007],[-106.47998199999978,40.39480200000003],[-106.47808299999997,40.39755200000019],[-106.477082,40.39971700000007],[-106.47789699999993,40.40104100000002],[-106.4771419999999,40.40253000000013],[-106.47824799999984,40.40480500000007],[-106.48252000000002,40.40842900000007],[-106.48539999999991,40.410122],[-106.4898419999999,40.41157800000002],[-106.49185199999994,40.410045000000025],[-106.49274700000001,40.407976000000076],[-106.49490199999997,40.40687900000006],[-106.49578099999997,40.40358500000002],[-106.498827,40.403594000000055],[-106.50098399999996,40.402596000000074],[-106.50247999999993,40.400400000000104],[-106.50635899999997,40.39694900000006],[-106.509029,40.394120999999984],[-106.51047999999997,40.393386000000135],[-106.51029599999976,40.38927700000011],[-106.51229999999993,40.386753],[-106.51468799999998,40.38833000000028],[-106.515783,40.388383999999974],[-106.51889899999998,40.39102200000008],[-106.52022799999997,40.38971300000031],[-106.52040799999997,40.388455000000135],[-106.52350699999994,40.3870740000001],[-106.52527900000001,40.38580700000006],[-106.52674100000002,40.384085000000084],[-106.52714299999997,40.381182000000024],[-106.52646499999992,40.38055300000002],[-106.5275059999999,40.37819900000011],[-106.52681899999999,40.377211999999986],[-106.527691,40.374982000000045],[-106.52953899999989,40.371784000000105],[-106.52971999999994,40.369113000000084],[-106.53132800000003,40.366622000000234],[-106.53416500000003,40.36583900000028],[-106.53492099999994,40.365080000000034],[-106.53833499999979,40.36864100000031],[-106.53832999999992,40.3698730000001],[-106.53989899999999,40.37134200000014],[-106.54371599999996,40.372064000000194],[-106.54560500000002,40.37076600000012],[-106.54767199999998,40.370170000000144],[-106.55066499999992,40.37043500000004],[-106.55345799999981,40.36986900000011],[-106.55529199999995,40.37203999999997],[-106.55535299999985,40.37423300000006],[-106.55668500000002,40.37727599999994],[-106.559301,40.37807900000013],[-106.56250099999983,40.37839200000013],[-106.56516999999997,40.37786300000022],[-106.56941999999998,40.3785670000002],[-106.57655299999999,40.37855800000017],[-106.58044099999995,40.37981000000019],[-106.58153999999985,40.38067000000029],[-106.58361199999996,40.38102600000008],[-106.58542499999999,40.38226000000003],[-106.58769699999993,40.38143000000014],[-106.58943199999993,40.38180100000011],[-106.59105299999987,40.38058100000006],[-106.59453500000001,40.38142700000003],[-106.59635899999995,40.38117600000004],[-106.59652499999993,40.37885300000005],[-106.601046,40.37580400000002],[-106.6025049999999,40.37608200000011],[-106.60281199999997,40.37768400000016],[-106.60373399999997,40.37833400000028],[-106.60818999999998,40.37973199999993],[-106.60942999999992,40.378288000000055],[-106.61232100000001,40.37938300000002],[-106.611693,40.383248000000265],[-106.61198099999996,40.38683100000026],[-106.61312599999991,40.388716000000045],[-106.61519499999997,40.391009000000054],[-106.61764499999998,40.393197999999984],[-106.61828200000002,40.39528200000012],[-106.61803299999985,40.39892900000007],[-106.61952500000001,40.40524500000009],[-106.62099,40.40801800000003],[-106.62391999999994,40.41090300000013],[-106.62519600000002,40.41500700000006],[-106.62708899999996,40.41731600000014],[-106.62839199999996,40.41762600000004],[-106.62915299999997,40.41892500000006],[-106.6288399999998,40.42217800000009],[-106.62651799999998,40.426955000000135],[-106.6236649999999,40.42958299999992],[-106.62216699999988,40.429763000000094],[-106.62349099999989,40.431729000000075],[-106.62463599999995,40.431164000000194],[-106.62814600000002,40.431107999999995],[-106.63250799999997,40.432565000000295],[-106.63613599999996,40.43200900000011],[-106.638601,40.432244000000026],[-106.63912599999992,40.432920000000195],[-106.643351,40.43604099999993],[-106.6486119999999,40.43854400000009],[-106.65034199999997,40.43978100000015],[-106.6518559999999,40.44214499999998],[-106.65211199999987,40.445231000000035],[-106.65152499999988,40.449377000000084],[-106.65385500000002,40.44951100000014],[-106.65580599999998,40.45019100000013],[-106.65641199999993,40.45149500000002],[-106.65649499999995,40.45604300000025],[-106.65895699999982,40.45937599999996],[-106.65745999999996,40.4624300000001],[-106.65844699999991,40.466593999999986],[-106.65788099999997,40.47155900000013],[-106.65960699999988,40.47174899999999],[-106.66348899999991,40.470582999999976],[-106.665301,40.47052400000007],[-106.66728099999995,40.47264700000005],[-106.66706299999987,40.47606100000007],[-106.66477199999991,40.47711799999996],[-106.66387099999997,40.47816300000011],[-106.66328099999993,40.480272000000014],[-106.65742999999998,40.483154999999954],[-106.65623299999987,40.48440800000026],[-106.65618499999988,40.48560800000007],[-106.65729399999998,40.48702799999995],[-106.65788299999997,40.48913900000019],[-106.65931,40.49019800000025],[-106.65761899999995,40.491209000000026],[-106.65392499999996,40.49243900000022],[-106.6531129999999,40.494732000000226],[-106.65460999999988,40.49644300000023],[-106.65481799999992,40.49902900000012],[-106.65604199999996,40.49997799999994],[-106.65631499999989,40.501768000000084],[-106.65558299999998,40.50341300000031],[-106.65545199999991,40.507103000000086],[-106.65580799999998,40.50914099999994],[-106.65928499999995,40.510577000000296],[-106.66110500000002,40.511944000000085],[-106.66079299999996,40.51295900000002],[-106.658974,40.51427600000028],[-106.66177700000003,40.515963000000056],[-106.667101,40.520143000000246],[-106.66947500000003,40.520139000000086],[-106.67206399999992,40.521214000000214],[-106.67369999999994,40.52368500000006],[-106.67328199999997,40.52693100000005],[-106.67135599999989,40.530157000000145],[-106.66948799999994,40.53248100000002],[-106.66984500000001,40.534446000000116],[-106.67168800000002,40.53584400000011],[-106.67468899999977,40.53693499999997],[-106.6766889999999,40.538035000000036],[-106.67732999999998,40.539734000000294],[-106.67736299999996,40.54246500000011],[-106.67911099999992,40.544489000000056],[-106.683378,40.54453200000006],[-106.68473999999998,40.54892100000012],[-106.68621799999988,40.55132700000007],[-106.68770999999987,40.552105000000154],[-106.6928509999999,40.5566390000003],[-106.69285999999994,40.55858100000023],[-106.69180699999993,40.56116900000012],[-106.69218799999993,40.56353700000011],[-106.69417799999991,40.56691500000005],[-106.696867,40.57054400000027],[-106.69713100000001,40.57266199999998],[-106.69889599999993,40.57514800000001],[-106.70234099999976,40.578025000000025],[-106.70014599999996,40.580269000000044],[-106.69992200000002,40.58196100000009],[-106.69864199999995,40.58349400000026],[-106.6997,40.586022000000014],[-106.69800299999986,40.5887640000002],[-106.6947659999999,40.59200900000013],[-106.69403499999999,40.59355699999992],[-106.69414799999993,40.59524099999999],[-106.6952609999999,40.598194999999976],[-106.69826699999993,40.599390000000085],[-106.69872299999992,40.602039000000104],[-106.701862,40.60440199999999],[-106.70234499999992,40.60599600000012],[-106.70434799999992,40.60738600000002],[-106.70365800000002,40.608620000000144],[-106.70374099999998,40.61055300000021],[-106.70600000000002,40.614399000000276],[-106.70614899999998,40.61659700000024],[-106.70379000000003,40.62013000000002],[-106.70395699999989,40.62183800000008],[-106.70101099999994,40.6259570000002],[-106.70284899999996,40.62770900000021],[-106.69826399999994,40.627482999999984],[-106.696958,40.62920600000007],[-106.69716199999999,40.62993200000011],[-106.694209,40.632902],[-106.69116600000001,40.634446000000025],[-106.687117,40.635775000000024],[-106.688875,40.63877800000017],[-106.68865599999998,40.639649000000304],[-106.68601599999988,40.64144100000004],[-106.68348599999996,40.64481400000011],[-106.68284800000004,40.64686499999999],[-106.68014199999976,40.64803600000022],[-106.68065399999995,40.64928400000008],[-106.68206399999991,40.649675000000116],[-106.68518799999993,40.64944600000018],[-106.68773099999993,40.64987299999996],[-106.68865599999998,40.650773000000015],[-106.69126799999992,40.655441999999994],[-106.69208200000003,40.65774600000003],[-106.69182599999999,40.659958000000245],[-106.69050199999987,40.659906000000205],[-106.68850499999985,40.661187000000155],[-106.68576499999989,40.662326000000235],[-106.6860989999999,40.666673],[-106.68723099999994,40.671944000000224],[-106.686083,40.67370700000026],[-106.68105700000001,40.675167999999985],[-106.68239099999988,40.677898000000084],[-106.68352599999992,40.67823300000009],[-106.68467599999985,40.679616000000124],[-106.68707299999994,40.68060799999995],[-106.68734599999988,40.682679000000064],[-106.68449599999997,40.68267400000002],[-106.68145800000002,40.68324899999999],[-106.67967099999998,40.68422299999992],[-106.67947600000002,40.685918000000015],[-106.67841900000002,40.686535000000276],[-106.67517900000001,40.68721500000004],[-106.67286199999995,40.688336000000106],[-106.67256099999992,40.690143000000205],[-106.67022899999989,40.69284499999992],[-106.66795299999995,40.69630500000005],[-106.66856499999989,40.698618000000295],[-106.6707669999999,40.70104700000019],[-106.67099699999994,40.70306300000004],[-106.67190299999993,40.70451100000008],[-106.67253699999998,40.70691300000004],[-106.6724789999999,40.70847800000024],[-106.668924,40.71025700000018],[-106.66887599999995,40.711725000000115],[-106.67046399999998,40.714131000000066],[-106.6711929999999,40.716379000000074],[-106.66978399999994,40.71978800000005],[-106.67037299999998,40.72139900000013],[-106.6719359999999,40.72281399999997],[-106.67331499999989,40.72297000000009],[-106.67380700000001,40.72402499999998],[-106.67244699999992,40.72618400000027],[-106.67017299999998,40.72768400000007],[-106.66952000000003,40.728791],[-106.67052799999993,40.73216700000006],[-106.67382599999996,40.73510900000031],[-106.676425,40.7355510000001],[-106.67669399999994,40.7366310000001],[-106.67474299999998,40.73667800000027],[-106.67114500000002,40.73865900000004],[-106.66832599999981,40.739120000000185],[-106.66499699999997,40.74214000000029],[-106.66295199999996,40.74301800000029],[-106.662556,40.74476900000002],[-106.66428499999995,40.748118000000204],[-106.66518899999988,40.748857000000044],[-106.66417799999999,40.750965000000065],[-106.66199099999994,40.75239799999997],[-106.66187099999996,40.75408500000026],[-106.66343699999987,40.754076000000225],[-106.6642349999999,40.756451000000084],[-106.66607399999992,40.75828100000001],[-106.66949799999998,40.75965699999995],[-106.6699809999999,40.76030800000001],[-106.66892899999988,40.76291199999997],[-106.66479399999997,40.76553400000023],[-106.66332699999992,40.76697900000005],[-106.66348899999991,40.76834200000002],[-106.66206199999988,40.77057700000023],[-106.6592369999999,40.772861000000034],[-106.65460599999989,40.77455000000015],[-106.65238299999999,40.77562000000006],[-106.64916099999994,40.77538800000002],[-106.6497839999999,40.777312999999936],[-106.64928299999991,40.780099000000064],[-106.64749699999993,40.782651000000214],[-106.64270099999993,40.78486500000025],[-106.64254799999998,40.78550300000029],[-106.63992699999994,40.78705100000013],[-106.63927100000001,40.78818499999994],[-106.63692199999997,40.78948900000006],[-106.63596099999995,40.79193099999992],[-106.63689399999998,40.79415300000011],[-106.63651699999997,40.7966650000003],[-106.63688499999995,40.79959200000019],[-106.6358239999999,40.802616000000285],[-106.63649999999978,40.80345900000003],[-106.63553799999988,40.804756000000054],[-106.63675399999994,40.80562800000007],[-106.63713899999999,40.80920900000007],[-106.63547599999998,40.81136500000031],[-106.63675199999994,40.81288299999994],[-106.638102,40.813303000000076],[-106.639227,40.81452200000007],[-106.63932399999987,40.81582900000018],[-106.63839300000001,40.817004000000054],[-106.64300399999996,40.82036700000015],[-106.64472799999999,40.82110799999998],[-106.64713499999999,40.821296000000075],[-106.65130699999997,40.82306399999999],[-106.65294199999988,40.8234020000001],[-106.65503699999994,40.82309100000009],[-106.65845899999982,40.82392699999997],[-106.66003399999994,40.826653000000306],[-106.66399199999995,40.82819600000005],[-106.66289,40.83231700000016],[-106.66404099999983,40.83496600000012],[-106.66350299999999,40.837211999999965],[-106.66270499999996,40.8379510000002],[-106.66130399999997,40.84090400000014],[-106.662644,40.84334100000012],[-106.65889700000002,40.84397600000011],[-106.654472,40.84554300000025],[-106.65454099999994,40.849117000000206],[-106.65755100000001,40.8532390000002],[-106.65998000000002,40.854395000000125],[-106.66159500000003,40.85571800000014],[-106.66405500000002,40.85682500000007],[-106.66448100000002,40.859082000000114],[-106.66596800000002,40.858233999999925],[-106.66736299999991,40.85979099999997],[-106.66709900000001,40.86096299999997],[-106.6679769999999,40.86279600000029],[-106.66816099999994,40.86514400000004],[-106.67350399999998,40.867057000000045],[-106.675475,40.86697300000003],[-106.67798699999992,40.86786999999998],[-106.6794789999999,40.86955500000016],[-106.68053599999996,40.87227600000023],[-106.68030299999987,40.874303000000054],[-106.68220500000001,40.87578900000028],[-106.68269699999996,40.877636000000166],[-106.6847699999999,40.87866200000013],[-106.68616199999997,40.88061500000026],[-106.68600700000002,40.88291200000026],[-106.685293,40.88460400000008],[-106.68588299999999,40.88582200000013],[-106.68758600000001,40.88630800000021],[-106.69166899999988,40.88819200000012],[-106.69415900000001,40.88801799999993],[-106.69926099999992,40.88537200000002],[-106.700376,40.88418799999994],[-106.70018899999997,40.882648000000074],[-106.70346799999993,40.88043499999992],[-106.70355699999993,40.878686000000016],[-106.70454699999988,40.87665600000025],[-106.7057559999999,40.87600400000014],[-106.71359799999993,40.87399600000026],[-106.72185400000001,40.87295599999999],[-106.72686899999991,40.871375000000285],[-106.7283329999999,40.871241000000055],[-106.73659199999997,40.87310000000019],[-106.73898799999989,40.87269599999996],[-106.73934099999997,40.87084300000015],[-106.74094100000002,40.8723260000001],[-106.74263199999996,40.87497100000013],[-106.7422019999999,40.877105000000086],[-106.74420999999995,40.87911400000024],[-106.74587099999991,40.87980000000016],[-106.7468199999999,40.88233200000002],[-106.744644,40.88457100000005],[-106.74085300000002,40.88979999999998],[-106.74356,40.891994000000295],[-106.74678899999992,40.892979999999966],[-106.75148200000001,40.894018000000074],[-106.75647100000003,40.89384200000006],[-106.76163999999989,40.89417400000019],[-106.76327100000003,40.895714000000055],[-106.76493399999987,40.89636700000028],[-106.76674499999996,40.89619800000014],[-106.77007099999992,40.896997],[-106.76754,40.89864099999994],[-106.76774199999994,40.90216300000003],[-106.769004,40.904138000000216],[-106.77117499999997,40.90428400000002],[-106.77315599999997,40.905283999999995],[-106.77636599999994,40.905743000000314],[-106.77780699999988,40.905437000000234],[-106.77867600000002,40.90642900000023],[-106.780911,40.90704200000016],[-106.78314599999993,40.90613100000007],[-106.78697899999992,40.90290700000014],[-106.788589,40.90284200000008],[-106.7878889999999,40.90056100000004],[-106.78949899999998,40.900779000000284],[-106.79200900000001,40.89995600000003],[-106.79425500000002,40.90056100000004],[-106.795365,40.89791700000012],[-106.797821,40.89646599999992],[-106.80063799999994,40.895901000000265],[-106.80206799999996,40.896908999999994],[-106.80489599999999,40.89731100000023],[-106.807324,40.89910800000007],[-106.807526,40.900551000000064],[-106.80534499999993,40.90197800000004],[-106.80592899999994,40.90303399999999],[-106.80633499999988,40.90664499999997],[-106.80845499999987,40.908547],[-106.80983500000002,40.91140100000001],[-106.81058999999993,40.91417300000006],[-106.81189499999999,40.91645300000005],[-106.81529799999993,40.9173390000002],[-106.81900699999994,40.91863600000022],[-106.8182569999999,40.92025599999994],[-106.81859799999995,40.922134000000085],[-106.81997899999999,40.92530999999997],[-106.82272599999999,40.92809800000026],[-106.823534,40.93028199999998],[-106.82608500000003,40.92801600000007],[-106.82640200000003,40.92707300000018],[-106.82857100000001,40.92466200000018],[-106.83120799999995,40.92392000000007],[-106.83384699999999,40.92420900000002],[-106.83718399999998,40.92365900000016],[-106.840555,40.924730000000295],[-106.84560999999991,40.92505800000026],[-106.85000599999984,40.92420100000004],[-106.84999799999997,40.925451000000066],[-106.84876999999989,40.926306000000125],[-106.850797,40.928537000000006],[-106.85238800000002,40.92972900000001],[-106.85224699999992,40.933792000000096],[-106.85611599999993,40.93497500000029],[-106.855887,40.936982000000114],[-106.85644899999994,40.93758599999995],[-106.859376,40.938423000000114],[-106.86210399999987,40.94092800000027],[-106.86246599999998,40.941927000000135],[-106.86211999999995,40.944329000000096],[-106.86219999999992,40.94727800000004],[-106.86301200000003,40.95086500000002],[-106.86399199999994,40.953701000000024],[-106.86462899999998,40.95419199999992],[-106.86831899999993,40.955125000000066],[-106.86989099999988,40.95672000000002],[-106.8695239999999,40.959001000000114],[-106.87012499999997,40.96260400000017],[-106.87245399999995,40.96656000000007],[-106.87274599999984,40.96938100000011],[-106.87246299999987,40.97108200000014],[-106.87321999999989,40.973032000000046],[-106.87282199999993,40.975877000000025],[-106.871374,40.9780870000003],[-106.87058400000001,40.98122300000006],[-106.86703499999982,40.98264400000022],[-106.86230699999999,40.983703000000276],[-106.85936799999996,40.98319700000013],[-106.85740199999992,40.98127999999997],[-106.85581599999989,40.983948999999996],[-106.85542900000002,40.98708500000015],[-106.854222,40.9883750000003],[-106.85073599999993,40.990247000000295],[-106.85086499999994,40.99127900000008],[-106.85709500000002,40.99385500000005],[-106.85985499999975,40.9953040000002],[-106.86160799999993,40.996802],[-106.86145399999998,40.99996800000008],[-106.8577729999999,41.002663000000155],[-106.79443299999991,41.00267999999994],[-106.75190199999992,41.002702],[-106.6875179999999,41.002390000000105],[-106.62559199999993,41.00211000000007],[-106.59389499999992,41.002129000000195],[-106.53188199999994,41.00197200000002],[-106.500588,41.00189500000005],[-106.49397899999991,41.002140000000225],[-106.455715,41.002117999999996],[-106.43956300000002,41.00197800000012],[-106.43741899999992,41.00179500000007],[-106.43094999999988,41.00175200000007],[-106.3986369999999,41.00109700000007],[-106.38635599999992,41.00114400000024],[-106.37556699999993,41.000719999999944],[-106.35396599999996,40.999971000000016],[-106.34641399999992,40.999637000000064],[-106.32645600000001,40.999275000000125],[-106.31421799999993,40.99892299999999],[-106.28359799999987,40.99863900000008],[-106.25037699999996,40.9982830000003],[-106.21801299999993,40.99773700000026],[-106.19055400000002,40.99757800000003]]]},"properties":{"OBJECTID":30,"COUNTYFP":"057","Shape_Leng":3.77162970318,"Shape_Area":0.447106252764,"low":360,"lowmod":655,"LMMI":895,"lowmoduniv":1320,"lowmod_pct":0.496212121212,"NAMELSAD10":"Jackson County"}},
{"type":"Feature","geometry":{"type":"MultiPolygon","coordinates":[[[[-105.05333000000002,39.62798900000013],[-105.05343899999997,39.62438700000018],[-105.05583999999993,39.624193000000105],[-105.05583599999989,39.625994000000105],[-105.05333000000002,39.62798900000013]]],[[[-105.05341099999998,39.63158600000003],[-105.05345699999987,39.62952600000011],[-105.05700899999994,39.63167900000008],[-105.0594569999999,39.63466200000016],[-105.05899099999999,39.638886999999954],[-105.0533329999999,39.63889300000011],[-105.05341099999998,39.63158600000003]]],[[[-105.14714299999997,39.91388900000021],[-105.15044599999999,39.91323999999997],[-105.15331299999997,39.913039000000026],[-105.15845799999988,39.9115960000002],[-105.16205699999995,39.91143399999993],[-105.16568999999998,39.91051499999992],[-105.165571,39.89249500000005],[-105.16516699999994,39.891486000000214],[-105.14743999999996,39.889449999999954],[-105.14698900000002,39.889211999999986],[-105.13485199999991,39.88929200000007],[-105.13645700000001,39.8909460000001],[-105.13630799999993,39.89202],[-105.13310499999994,39.89201200000002],[-105.13088500000003,39.892697000000226],[-105.13031899999999,39.89070600000014],[-105.12889499999994,39.89035300000006],[-105.12867799999992,39.89998499999996],[-105.13144999999997,39.90009900000018],[-105.14739599999996,39.90009399999997],[-105.14711699999998,39.91400200000015],[-105.12791099999998,39.914015000000006],[-105.10860899999994,39.914136999999926],[-105.10817800000001,39.91281000000009],[-105.10695099999981,39.91172900000004],[-105.100641,39.90863800000005],[-105.10010199999999,39.90220600000009],[-105.09668099999999,39.900787000000264],[-105.0966919999999,39.899788],[-105.09062599999993,39.899677000000054],[-105.09062399999993,39.89890299999996],[-105.08425099999994,39.898956999999996],[-105.08422999999988,39.89983699999999],[-105.071688,39.899833],[-105.071775,39.90701799999994],[-105.06239199999999,39.90702500000003],[-105.06238399999995,39.913987000000134],[-105.05767599999996,39.913702],[-105.05767599999996,39.9124290000002],[-105.05529399999989,39.91242599999998],[-105.055319,39.91372499999994],[-105.05289199999993,39.91422400000022],[-105.053042,39.896838],[-105.05313999999993,39.87521600000014],[-105.05333799999977,39.85648500000002],[-105.0528349999999,39.85254500000002],[-105.05318799999998,39.849649],[-105.05328199999997,39.821125000000166],[-105.05325600000003,39.80649700000026],[-105.05282899999986,39.80387400000029],[-105.05316199999987,39.799239000000114],[-105.05324599999989,39.791063000000065],[-105.06147999999996,39.791075000000035],[-105.06189899999998,39.79008299999998],[-105.06511499999999,39.78968000000003],[-105.0650829999999,39.788086000000135],[-105.06294400000002,39.788087000000075],[-105.06265899999988,39.78384700000004],[-105.05318299999993,39.78379699999999],[-105.05324300000001,39.78219000000007],[-105.05322799999999,39.73496499999993],[-105.055567,39.73497700000024],[-105.0555389999999,39.73309800000004],[-105.05322699999982,39.733110000000124],[-105.05327899999997,39.718233000000055],[-105.05319400000002,39.69136400000002],[-105.05322099999995,39.682290999999964],[-105.05804399999988,39.68227000000002],[-105.05817499999995,39.67866600000025],[-105.06073199999997,39.6782790000002],[-105.062907,39.67922300000015],[-105.06289800000002,39.67498699999999],[-105.05324999999988,39.674988999999925],[-105.05325600000003,39.66778500000021],[-105.07217899999995,39.66776299999998],[-105.07220399999989,39.669521000000145],[-105.07493999999997,39.67068700000016],[-105.07693899999987,39.6696860000003],[-105.08134999999999,39.67009899999999],[-105.0814539999999,39.66207400000019],[-105.07982200000004,39.66216400000019],[-105.07733799999994,39.660389000000066],[-105.07764199999997,39.65877900000004],[-105.07161699999989,39.659994999999924],[-105.07079299999992,39.659170000000074],[-105.07072699999998,39.6565940000001],[-105.06704499999995,39.65658700000023],[-105.06694099999999,39.653594000000226],[-105.07102900000001,39.64957900000019],[-105.06392399999993,39.64956599999999],[-105.06213699999995,39.65138399999995],[-105.05883900000003,39.65138700000023],[-105.05854299999993,39.65297900000013],[-105.05721499999993,39.65294600000004],[-105.05734499999994,39.6499840000003],[-105.063399,39.65008000000006],[-105.06353899999988,39.6459880000001],[-105.0720419999999,39.646090000000015],[-105.07232599999992,39.645350000000235],[-105.07445399999989,39.64410399999997],[-105.07575999999995,39.64205000000021],[-105.07680299999998,39.638735],[-105.07327099999998,39.6387400000001],[-105.073308,39.63650300000006],[-105.07695399999989,39.63638999999995],[-105.07981799999988,39.636688000000106],[-105.08156500000001,39.63616600000006],[-105.08167599999996,39.63875900000022],[-105.09157699999992,39.63872900000024],[-105.09158199999996,39.635178000000224],[-105.09852699999993,39.63507500000003],[-105.10987399999999,39.635185999999976],[-105.10992699999986,39.62698800000027],[-105.10858999999999,39.626986999999986],[-105.10860000000002,39.624208000000294],[-105.09125599999999,39.62419799999998],[-105.09116399999994,39.627081000000146],[-105.08738399999999,39.62709000000018],[-105.08562599999993,39.62610700000005],[-105.08557999999994,39.628527000000076],[-105.08378899999991,39.628527000000076],[-105.08381300000002,39.62731800000006],[-105.08173,39.62731299999996],[-105.08169299999986,39.63014900000013],[-105.08373499999988,39.630155000000286],[-105.08371199999993,39.63132700000028],[-105.08145000000002,39.631858000000136],[-105.07623999999987,39.625587000000166],[-105.07595200000003,39.62414300000023],[-105.06496599999991,39.620238000000256],[-105.06514099999998,39.618582],[-105.07939799999986,39.61847300000005],[-105.08592799999997,39.618668000000014],[-105.08725199999998,39.618317000000104],[-105.09695999999997,39.618362000000104],[-105.09690899999993,39.61442300000016],[-105.07292399999994,39.61453899999998],[-105.06992499999996,39.61442400000004],[-105.06419900000003,39.61457400000006],[-105.05812500000002,39.6144300000002],[-105.05819300000002,39.61736600000012],[-105.05643899999995,39.619932000000006],[-105.05715099999998,39.62158500000015],[-105.05343899999997,39.62142300000005],[-105.05364499999996,39.6170800000001],[-105.05362200000002,39.60972800000002],[-105.05383599999976,39.600885000000005],[-105.05367199999989,39.59780999999998],[-105.05383099999989,39.59187800000012],[-105.05394100000001,39.58059000000014],[-105.05424,39.569788000000074],[-105.05413999999996,39.566088000000036],[-105.04874000000001,39.566088000000036],[-105.05136600000003,39.56325700000025],[-105.05427199999997,39.56299700000005],[-105.05725199999989,39.56230600000026],[-105.05825699999991,39.560378999999955],[-105.0607139999999,39.55994100000015],[-105.06112999999993,39.556906000000026],[-105.0637979999999,39.55523900000031],[-105.06567199999995,39.553345999999976],[-105.066735,39.55049900000017],[-105.07030799999995,39.547896000000264],[-105.07138499999996,39.54642900000022],[-105.07411100000002,39.5442680000001],[-105.07459399999999,39.541652],[-105.07641899999993,39.54005500000022],[-105.07774499999994,39.53670600000004],[-105.07657099999989,39.534967999999935],[-105.07610599999998,39.53269100000006],[-105.076434,39.53162400000002],[-105.07946099999992,39.525888000000066],[-105.0792879999999,39.523893000000214],[-105.08009599999997,39.52315599999997],[-105.07899999999995,39.519142000000215],[-105.077945,39.517495],[-105.07885899999985,39.51285000000007],[-105.07870000000003,39.5108360000001],[-105.08113299999997,39.50966699999998],[-105.08466499999992,39.506585000000086],[-105.08527500000002,39.50360799999993],[-105.08484099999998,39.50121000000013],[-105.0864949999999,39.49913300000003],[-105.08614799999987,39.49763300000029],[-105.08718499999998,39.496061000000054],[-105.08610899999991,39.494923000000085],[-105.08675099999994,39.49342300000029],[-105.08851399999986,39.49239200000005],[-105.09135300000003,39.48928000000001],[-105.09870699999982,39.485394000000156],[-105.10035900000003,39.485487000000205],[-105.1033129999999,39.48471500000028],[-105.10464100000002,39.485020000000304],[-105.10694199999989,39.48471900000027],[-105.11068499999988,39.483553000000086],[-105.11315399999995,39.483903999999995],[-105.11567100000002,39.48596000000026],[-105.11764599999992,39.48539200000016],[-105.11826599999989,39.482988000000205],[-105.11588299999988,39.48253500000021],[-105.11627599999991,39.48145100000022],[-105.11853499999995,39.48020699999995],[-105.12101699999977,39.4814540000001],[-105.12182299999995,39.48130400000014],[-105.12451799999991,39.479384000000266],[-105.12839799999995,39.47738800000013],[-105.12876399999993,39.475783000000206],[-105.13158199999998,39.472679000000085],[-105.13339299999996,39.47211000000004],[-105.13493099999994,39.47095200000001],[-105.13479699999988,39.4702010000002],[-105.13262500000002,39.46790800000019],[-105.13318599999991,39.46612500000009],[-105.13257699999997,39.46432100000004],[-105.13294099999996,39.46191200000004],[-105.13039899999995,39.45952700000004],[-105.12747399999989,39.4584670000001],[-105.12669599999998,39.45720000000006],[-105.1250169999999,39.45636700000006],[-105.1241839999999,39.454985000000306],[-105.12306199999995,39.45084700000024],[-105.12372799999991,39.449464000000205],[-105.12321400000002,39.447831000000065],[-105.12474199999997,39.44402000000025],[-105.12470400000001,39.44273600000008],[-105.12614399999978,39.44018699999998],[-105.12467400000003,39.43587900000006],[-105.12347699999992,39.43449400000003],[-105.128424,39.43175700000006],[-105.13005399999992,39.43006399999996],[-105.13156399999997,39.42923500000012],[-105.13307099999997,39.42743900000005],[-105.13542499999988,39.426779000000124],[-105.1369729999999,39.42542100000003],[-105.13820599999985,39.42307000000005],[-105.14056699999998,39.42240100000009],[-105.14140800000001,39.42113600000005],[-105.14531599999998,39.41881399999994],[-105.1494199999999,39.41760000000005],[-105.15176699999995,39.41568100000006],[-105.15597699999995,39.41380200000003],[-105.15993699999984,39.41432299999997],[-105.16126099999997,39.413758000000314],[-105.16261999999989,39.411928999999986],[-105.16667099999995,39.408510000000035],[-105.16904199999993,39.408935999999926],[-105.17096299999992,39.40782500000023],[-105.17131499999999,39.40655100000015],[-105.169441,39.403326000000106],[-105.16940799999986,39.40159],[-105.167194,39.39975300000026],[-105.16697199999999,39.39818700000018],[-105.167642,39.39574799999997],[-105.16725799999995,39.394308000000024],[-105.16549499999991,39.3940080000001],[-105.16433699999993,39.3911360000003],[-105.16566,39.391178000000025],[-105.16791299999988,39.389823000000206],[-105.16996799999998,39.38909600000011],[-105.17121899999995,39.386803999999984],[-105.172978,39.3857450000001],[-105.17351500000001,39.383203000000094],[-105.17312200000003,39.38222800000011],[-105.17085600000001,39.38226099999997],[-105.16961500000002,39.38068200000009],[-105.17124999999993,39.379638],[-105.1729969999999,39.37772200000029],[-105.17225999999994,39.37641100000019],[-105.17033000000004,39.37534700000026],[-105.1699549999999,39.374437000000285],[-105.16790000000003,39.3739800000003],[-105.16709899999984,39.37220000000008],[-105.16763599999996,39.370426000000066],[-105.16911399999987,39.36909800000012],[-105.16892199999995,39.367264999999975],[-105.1677919999999,39.36637100000007],[-105.16643499999998,39.362957999999935],[-105.16657399999986,39.36175400000002],[-105.16786000000002,39.359016999999994],[-105.16912699999995,39.35893400000026],[-105.171517,39.35722000000004],[-105.17235700000003,39.355817000000286],[-105.171853,39.35345000000001],[-105.17515700000001,39.350490000000036],[-105.17719299999999,39.349579000000176],[-105.1765319999999,39.3475150000001],[-105.1771579999999,39.34471000000019],[-105.17917399999999,39.34346199999999],[-105.17953499999993,39.342309000000114],[-105.17872899999998,39.34078600000004],[-105.18009999999998,39.33915500000029],[-105.182144,39.33858800000007],[-105.1839559999998,39.33549000000028],[-105.18507099999988,39.33483100000012],[-105.18783699999994,39.334186000000045],[-105.187432,39.333256000000176],[-105.18557199999992,39.33300300000002],[-105.18226800000002,39.33137400000004],[-105.18230499999999,39.329796999999985],[-105.186057,39.32829000000015],[-105.18818399999998,39.3279610000003],[-105.18805399999997,39.326902000000246],[-105.18588299999999,39.326155000000256],[-105.18548599999991,39.32440500000007],[-105.18629199999981,39.323304000000064],[-105.18780299999997,39.322853000000066],[-105.18881399999987,39.321153000000095],[-105.18912299999994,39.31910500000009],[-105.18858699999998,39.317384000000175],[-105.1901969999999,39.31619400000011],[-105.19348699999989,39.31258200000008],[-105.19466199999994,39.31243400000005],[-105.19815599999993,39.31043199999999],[-105.19853199999994,39.30943700000006],[-105.19668000000001,39.3085440000001],[-105.19782399999991,39.30498800000021],[-105.1965219999999,39.303949000000216],[-105.19599999999997,39.30208200000027],[-105.19920099999996,39.3020480000003],[-105.20086199999992,39.30164400000007],[-105.20136799999995,39.29926900000004],[-105.20497899999992,39.299798000000294],[-105.206793,39.29919600000022],[-105.20768099999987,39.29819400000008],[-105.20717300000001,39.296228000000156],[-105.20523100000003,39.29489899999999],[-105.20288700000003,39.294752000000244],[-105.20305400000001,39.29306900000006],[-105.2055269999999,39.293990000000065],[-105.20698099999993,39.293202000000235],[-105.20553099999995,39.29168100000015],[-105.20649799999995,39.2906780000003],[-105.20790799999997,39.291525000000036],[-105.20873599999993,39.29061599999994],[-105.20804199999998,39.289612000000204],[-105.21017,39.28774000000021],[-105.21247299999987,39.284484000000134],[-105.21133399999997,39.282914999999946],[-105.21132499999982,39.28096700000026],[-105.21325999999999,39.2797240000001],[-105.21389699999992,39.27869300000003],[-105.21543799999978,39.27857399999999],[-105.21638599999989,39.277618000000075],[-105.21554099999997,39.276635999999996],[-105.21750700000001,39.274595999999974],[-105.21566199999995,39.273348999999996],[-105.21557999999999,39.272232000000145],[-105.21917899999988,39.270503000000076],[-105.22055899999992,39.26917400000025],[-105.22077599999994,39.268062999999984],[-105.21869899999996,39.26704200000006],[-105.21875,39.26534900000013],[-105.217784,39.26354799999996],[-105.217419,39.260395000000074],[-105.21959800000002,39.25964099999999],[-105.22476099999989,39.26070700000008],[-105.22514199999989,39.25944300000015],[-105.22340199999996,39.25841700000001],[-105.22344299999997,39.25615400000015],[-105.22668099999993,39.25650200000018],[-105.22735499999993,39.254234999999994],[-105.22812099999999,39.2537660000001],[-105.23180699999995,39.253962],[-105.23439299999995,39.25530700000024],[-105.234825,39.25457300000011],[-105.23420199999998,39.25241300000005],[-105.23525599999988,39.25131299999998],[-105.2373879999999,39.25094200000001],[-105.23733600000003,39.24950000000007],[-105.234781,39.24929700000018],[-105.23393799999997,39.24686800000029],[-105.23436700000002,39.245562000000064],[-105.23609499999992,39.24405800000011],[-105.24129900000003,39.243437000000256],[-105.243403,39.24265500000001],[-105.24601699999994,39.24056600000006],[-105.24853599999989,39.24084600000009],[-105.24930299999994,39.24178800000004],[-105.25242799999995,39.24369500000029],[-105.25347199999993,39.243314000000055],[-105.25353599999988,39.2413720000003],[-105.25503000000003,39.240274000000056],[-105.25432,39.237923000000194],[-105.257228,39.23354700000016],[-105.25834299999997,39.23280100000005],[-105.26178400000003,39.23148199999997],[-105.26351099999994,39.229261000000065],[-105.26342399999993,39.22836000000012],[-105.26166,39.22680500000007],[-105.26242499999995,39.22483000000028],[-105.26130199999994,39.22326900000007],[-105.26352800000001,39.22116900000003],[-105.26286800000003,39.2194630000003],[-105.26057600000001,39.216688000000204],[-105.26151999999985,39.21475800000002],[-105.26008599999994,39.21255800000006],[-105.26005399999997,39.21044500000022],[-105.26211999999987,39.20991800000013],[-105.26311099999998,39.208023000000196],[-105.26754599999992,39.209225000000174],[-105.269519,39.20941900000008],[-105.27217599999994,39.20865100000009],[-105.27277800000002,39.20775500000002],[-105.27629099999996,39.20721600000002],[-105.27769599999988,39.202801000000306],[-105.27881400000001,39.201811000000134],[-105.28126099999997,39.20175700000027],[-105.28292899999997,39.19995300000005],[-105.2851,39.19458800000007],[-105.28470199999992,39.19270300000022],[-105.28592800000001,39.189584999999965],[-105.28638999999998,39.18631400000004],[-105.28918199999998,39.1841720000001],[-105.2892359999999,39.18341600000008],[-105.28733099999994,39.181254999999965],[-105.28737699999982,39.17944200000005],[-105.28949799999981,39.177494000000195],[-105.29474799999997,39.174837999999966],[-105.293635,39.173974999999984],[-105.294216,39.172977],[-105.29843800000003,39.172810000000084],[-105.30152899999979,39.17061200000012],[-105.30201399999999,39.169246000000044],[-105.30021399999998,39.168017000000134],[-105.30102199999988,39.166777000000025],[-105.30227100000002,39.16668900000002],[-105.30445399999996,39.1642810000003],[-105.30519699999996,39.16402399999998],[-105.30705899999987,39.16535600000009],[-105.30998199999993,39.16351900000012],[-105.3106919999999,39.161526000000094],[-105.31306299999994,39.16001900000026],[-105.31355699999989,39.15912100000003],[-105.313087,39.157077000000015],[-105.31413600000002,39.15551500000009],[-105.3153099999999,39.154895000000124],[-105.31594899999999,39.151302000000214],[-105.31557899999996,39.14991100000003],[-105.31619299999994,39.14885500000025],[-105.31534699999992,39.146712000000036],[-105.31678499999998,39.1455730000003],[-105.31991599999986,39.14575400000018],[-105.32155599999993,39.14619100000027],[-105.323285,39.144992],[-105.32214699999992,39.142958000000135],[-105.32278199999996,39.14111400000013],[-105.32177799999994,39.139017000000024],[-105.32196199999998,39.1369670000002],[-105.32275700000002,39.135595000000194],[-105.32477399999993,39.13386300000025],[-105.32739299999992,39.13087300000029],[-105.3292199999999,39.12968900000021],[-105.33044399999994,39.12968500000022],[-105.37555199999997,39.12954300000001],[-105.397875,39.12956399999996],[-105.39772299999987,39.16151600000012],[-105.39788799999985,39.200616000000025],[-105.39783299999999,39.21113200000002],[-105.39789999999994,39.24999000000031],[-105.39671199999998,39.26488300000011],[-105.39804800000002,39.298427000000004],[-105.39813900000001,39.305295],[-105.398052,39.3146040000002],[-105.39865199999991,39.32618900000023],[-105.39825300000001,39.32718900000003],[-105.39865199999991,39.33707500000014],[-105.39845299999996,39.340289000000155],[-105.39814699999988,39.35069700000014],[-105.39845299999996,39.363088999999945],[-105.398552,39.378089999999986],[-105.39794999999992,39.38879100000025],[-105.3981569999998,39.39899500000024],[-105.398052,39.41193299999992],[-105.39806399999992,39.45225100000005],[-105.39851899999991,39.46011599999997],[-105.398552,39.46348800000004],[-105.39789799999988,39.46695700000021],[-105.39813900000001,39.46872500000029],[-105.39805100000001,39.48408800000027],[-105.39835099999988,39.50478900000013],[-105.39825099999996,39.51078799999999],[-105.398551,39.52688800000021],[-105.39885099999987,39.535588000000075],[-105.398751,39.54118799999998],[-105.39894899999996,39.566056],[-105.39898599999992,39.57307400000019],[-105.3991499999999,39.574388],[-105.3991499999999,39.581588000000124],[-105.39894999999996,39.585087999999985],[-105.39895099999995,39.60478599999999],[-105.39844999999997,39.628693999999996],[-105.39844999999997,39.634607000000074],[-105.39824999999996,39.63658800000019],[-105.39823999999993,39.65838000000008],[-105.39834899999988,39.6652880000002],[-105.39834000000002,39.676184000000035],[-105.39855,39.68598900000006],[-105.39854500000001,39.7010190000002],[-105.39824899999996,39.71788700000019],[-105.39814899999993,39.733282000000145],[-105.39794899999993,39.74708700000019],[-105.39794899999993,39.814886],[-105.39804900000001,39.81608700000004],[-105.39794799999993,39.84094000000027],[-105.39744999999994,39.853697000000125],[-105.39794899999993,39.86038700000012],[-105.39756799999992,39.866061000000286],[-105.397877,39.866849000000286],[-105.39804499999997,39.873886000000084],[-105.39794899999993,39.881087000000264],[-105.39764899999989,39.883187000000135],[-105.39784800000001,39.895341000000144],[-105.39774899999998,39.90408600000001],[-105.3978489999999,39.912886000000185],[-105.37394299999988,39.913181000000066],[-105.33954699999992,39.913786000000016],[-105.31414699999993,39.91388599999999],[-105.30684500000001,39.91398600000025],[-105.28265599999992,39.91368700000021],[-105.26944500000002,39.91398600000025],[-105.26494500000001,39.914186000000086],[-105.26394500000004,39.91398600000025],[-105.24526999999995,39.914084],[-105.23854399999988,39.91438599999992],[-105.23134299999992,39.914085000000114],[-105.21247499999976,39.91423600000013],[-105.20024099999995,39.91398600000025],[-105.19664699999987,39.914186000000086],[-105.19092899999998,39.91408200000001],[-105.16656499999999,39.91389300000003],[-105.14714299999997,39.91388900000021]]]]},"properties":{"OBJECTID":31,"COUNTYFP":"059","Shape_Leng":2.86268349353,"Shape_Area":0.210184517951,"low":104434,"lowmod":192680,"LMMI":300285,"lowmoduniv":543700,"lowmod_pct":0.354386610263,"NAMELSAD10":"Jefferson County"}},
{"type":"Feature","geometry":{"type":"Polygon","coordinates":[[[-102.04508199999992,38.61517200000014],[-102.045074,38.60245300000008],[-102.0452059999999,38.600680000000295],[-102.0452479999999,38.58867900000013],[-102.04514599999999,38.58827800000017],[-102.04517799999996,38.57714199999998],[-102.04488799999996,38.566394000000116],[-102.04491599999994,38.529295000000275],[-102.04504499999996,38.510226000000046],[-102.044985,38.48479400000008],[-102.04505199999994,38.46082200000029],[-102.04472800000002,38.44134600000018],[-102.04497700000002,38.42795200000023],[-102.044804,38.419676000000095],[-102.04482499999989,38.40427200000005],[-102.04492699999997,38.39331400000003],[-102.044804,38.38463600000023],[-102.04494599999987,38.384358000000304],[-102.04467599999998,38.3561840000001],[-102.04452400000002,38.34917400000006],[-102.04461200000003,38.31232400000022],[-102.04443900000001,38.304351999999994],[-102.04461100000003,38.28591000000017],[-102.04452300000003,38.28311600000012],[-102.04447499999998,38.26874900000013],[-102.06417799999997,38.268774000000064],[-102.066348,38.268643000000225],[-102.08395999999999,38.268673000000206],[-102.08802599999996,38.268543000000136],[-102.11015900000001,38.268550000000005],[-102.11882300000002,38.26842099999999],[-102.12034899999998,38.26860000000022],[-102.12546199999991,38.268452000000025],[-102.13687999999996,38.268456000000015],[-102.14909699999998,38.26831700000025],[-102.15740099999988,38.26842299999993],[-102.17696999999998,38.268354999999985],[-102.18113599999987,38.2682660000001],[-102.18578299999996,38.26852800000029],[-102.19835099999995,38.26840500000009],[-102.20418599999994,38.26825600000018],[-102.21660399999996,38.268147000000056],[-102.2291699999999,38.268277000000126],[-102.25046699999996,38.26817100000005],[-102.30531099999996,38.267967000000056],[-102.31956600000001,38.26795800000002],[-102.32253700000001,38.26807000000008],[-102.36054399999995,38.26809899999995],[-102.40006599999998,38.267943],[-102.432481,38.26786800000002],[-102.43425599999995,38.26791400000013],[-102.46767499999999,38.26763700000009],[-102.48534399999988,38.26796900000022],[-102.50045899999986,38.26774500000016],[-102.52652699999999,38.267634000000214],[-102.57296999999988,38.267227000000105],[-102.59685599999995,38.267313000000115],[-102.6183779999999,38.26749500000011],[-102.63704399999995,38.26749800000016],[-102.65492999999987,38.26731900000027],[-102.68547799999993,38.26727299999999],[-102.72213899999997,38.26710099999997],[-102.72382399999987,38.26725000000005],[-102.74232999999992,38.26697100000007],[-102.76133699999997,38.26667900000007],[-102.79301499999991,38.26766200000003],[-102.798991,38.26779599999992],[-102.82167499999997,38.26695000000029],[-102.82204399999995,38.266529000000276],[-102.83448099999998,38.26646100000016],[-102.83908499999995,38.26664100000011],[-102.84232099999991,38.26706600000011],[-102.84910400000001,38.26705400000003],[-102.904155,38.266722999999956],[-102.92304499999995,38.26668600000011],[-102.94106499999987,38.26675800000021],[-102.95007299999992,38.26692600000007],[-102.95866099999995,38.26669000000027],[-103.00047899999998,38.2667170000002],[-103.00047899999998,38.26637300000016],[-103.01146199999994,38.26646699999992],[-103.02298599999989,38.26640299999997],[-103.04906099999988,38.26649400000002],[-103.05772799999988,38.26659300000006],[-103.07586900000001,38.266364000000294],[-103.09691299999986,38.26632500000028],[-103.12547799999999,38.266464999999926],[-103.13109600000001,38.26639200000011],[-103.14004799999998,38.26597700000025],[-103.16310899999996,38.26597000000021],[-103.16357999999997,38.26602600000007],[-103.19971499999991,38.26580100000007],[-103.26694299999997,38.2656800000002],[-103.31794500000001,38.265596000000016],[-103.37548899999996,38.26550000000003],[-103.38504399999988,38.265293000000156],[-103.39993799999996,38.26543499999997],[-103.40981999999985,38.265442000000064],[-103.41835600000002,38.265162000000146],[-103.43012799999997,38.26516400000031],[-103.43730600000004,38.265031000000135],[-103.50176399999998,38.265015999999946],[-103.501643,38.276535000000024],[-103.50161000000003,38.32251000000031],[-103.50138699999997,38.32697500000006],[-103.50174700000002,38.342295000000036],[-103.50698499999999,38.34225600000002],[-103.50677000000002,38.37500300000005],[-103.50666899999993,38.40791800000011],[-103.50629100000003,38.42925200000013],[-103.50582099999986,38.4588],[-103.50525199999993,38.491281000000185],[-103.50465300000002,38.50000100000011],[-103.50466599999993,38.51640900000024],[-103.50467399999997,38.52366400000011],[-103.49244499999998,38.523893000000044],[-103.462785,38.52367800000002],[-103.45541499999985,38.52381700000001],[-103.42662299999995,38.52348500000022],[-103.38193099999995,38.52332700000011],[-103.37130899999994,38.52341700000028],[-103.35242,38.52370000000025],[-103.30348500000002,38.52359000000001],[-103.28255499999995,38.52364299999999],[-103.25056099999989,38.523657999999955],[-103.246127,38.52392200000014],[-103.23796900000002,38.524043000000006],[-103.22779299999996,38.524372000000255],[-103.19341500000002,38.524999000000264],[-103.19109100000003,38.52509399999997],[-103.17287799999986,38.52531499999992],[-103.17304200000001,38.56489799999997],[-103.17304899999999,38.58323700000017],[-103.17294299999998,38.612450000000024],[-103.13641699999994,38.61323900000008],[-103.09905299999991,38.614314000000036],[-103.06222799999989,38.615020000000186],[-103.05424099999993,38.615068000000065],[-103.02563699999996,38.61499399999997],[-103.01199999999989,38.61481600000019],[-102.98800099999988,38.615026],[-102.93345699999992,38.61510099999998],[-102.87547799999993,38.614597],[-102.82325999999995,38.61445400000025],[-102.80659999999995,38.61450200000013],[-102.78594699999996,38.614400000000046],[-102.76628199999993,38.61413299999998],[-102.70902899999999,38.61381100000017],[-102.68588999999992,38.61365399999994],[-102.66303799999997,38.61335600000018],[-102.66143399999987,38.61347000000001],[-102.63297399999999,38.61325400000027],[-102.62547099999995,38.61325099999999],[-102.62049699999989,38.612914000000046],[-102.61160899999999,38.61324399999995],[-102.58690099999995,38.61332700000008],[-102.57900999999987,38.613268000000176],[-102.56559500000003,38.613372000000254],[-102.54652199999998,38.613438999999914],[-102.51096699999994,38.61369899999994],[-102.50221599999992,38.61394900000022],[-102.47908100000001,38.61433499999998],[-102.45624399999991,38.61482600000011],[-102.43498399999999,38.61482199999995],[-102.42143599999997,38.615027000000055],[-102.39930699999991,38.61509700000016],[-102.39078099999995,38.6151780000003],[-102.36448799999994,38.61525599999999],[-102.34288900000001,38.61524499999996],[-102.32783799999993,38.615151000000196],[-102.32567299999994,38.61532500000021],[-102.29068499999994,38.614979000000176],[-102.25438600000001,38.615196000000196],[-102.2379489999999,38.61514500000004],[-102.21753999999987,38.61500600000028],[-102.19963399999989,38.61521700000014],[-102.17069100000003,38.615296000000285],[-102.14391399999994,38.61511200000001],[-102.12532899999991,38.6148860000003],[-102.10623399999997,38.615065000000186],[-102.096993,38.61501700000031],[-102.07432399999993,38.6150770000001],[-102.06883399999998,38.615176000000304],[-102.06553499999995,38.615688000000034],[-102.06382999999994,38.61515600000007],[-102.04508199999992,38.61517200000014]]]},"properties":{"OBJECTID":32,"COUNTYFP":"061","Shape_Leng":3.61711330293,"Shape_Area":0.477251855244,"low":405,"lowmod":685,"LMMI":1010,"lowmoduniv":1430,"lowmod_pct":0.479020979021,"NAMELSAD10":"Kiowa County"}},
{"type":"Feature","geometry":{"type":"Polygon","coordinates":[[[-102.04980599999999,39.57405800000009],[-102.04975499999995,39.56234999999998],[-102.04955199999995,39.53889499999997],[-102.04967299999998,39.53657900000002],[-102.04956500000003,39.506190999999944],[-102.04976899999997,39.50029799999999],[-102.04940599999992,39.478819999999985],[-102.04934099999997,39.47146200000009],[-102.04959199999996,39.47125300000005],[-102.04950599999995,39.465343000000246],[-102.0493709999999,39.418297000000166],[-102.04907899999989,39.4180570000002],[-102.04903100000001,39.40370899999999],[-102.04915800000003,39.403475000000185],[-102.04920400000003,39.37498800000003],[-102.04905199999996,39.37399099999999],[-102.04888399999999,39.35101000000009],[-102.04856000000001,39.31421500000016],[-102.04853000000003,39.30291599999998],[-102.04821999999996,39.27413899999999],[-102.04790100000002,39.25312200000013],[-102.04807399999987,39.24999400000013],[-102.04784099999995,39.23458200000016],[-102.04761799999989,39.19866600000012],[-102.04730599999999,39.148280000000284],[-102.04710499999999,39.125001999999995],[-102.047012,39.12124699999998],[-102.04674099999988,39.07562699999994],[-102.04657099999986,39.04703800000004],[-102.06064100000003,39.04708200000016],[-102.08731099999989,39.04676100000006],[-102.11485199999993,39.04658100000012],[-102.12528399999985,39.04662700000017],[-102.13047899999998,39.046535000000006],[-102.17730299999988,39.04644000000013],[-102.20429999999988,39.04627900000014],[-102.21752900000001,39.04626700000023],[-102.22723100000002,39.04636800000003],[-102.24421899999999,39.04634200000021],[-102.28180199999986,39.04634800000002],[-102.32576699999998,39.045887999999934],[-102.33004599999992,39.04591600000015],[-102.352438,39.04564900000025],[-102.36773199999993,39.04562500000003],[-102.37546800000001,39.04484899999994],[-102.4092599999999,39.044167000000016],[-102.42037199999987,39.04416900000018],[-102.43397099999999,39.04364200000009],[-102.44729999999998,39.04293500000023],[-102.45445699999993,39.04277500000012],[-102.46655399999997,39.04271600000004],[-102.47520199999997,39.04240100000027],[-102.499326,39.04166900000013],[-102.51178500000003,39.04139300000003],[-102.52075100000002,39.04135600000012],[-102.55807699999997,39.040782999999976],[-102.57685100000003,39.04067600000019],[-102.58558900000003,39.040547000000174],[-102.59910499999995,39.04020700000012],[-102.63174300000003,39.039747000000034],[-102.65052899999989,39.03964499999995],[-102.66920399999998,39.039427000000046],[-102.68731099999991,39.039038000000005],[-102.71576599999997,39.03867500000018],[-102.72411,39.04019900000014],[-102.75047999999987,39.039968000000044],[-102.798787,39.039710000000014],[-102.81717799999996,39.03979500000014],[-102.84918500000003,39.03949299999999],[-102.85794399999992,39.039337000000046],[-102.87766599999998,39.03937900000017],[-102.917934,39.0392470000001],[-102.99332900000002,39.03932600000002],[-103.04915999999997,39.03943899999996],[-103.05137199999996,39.039561000000106],[-103.07119999999998,39.03925000000015],[-103.09438299999994,39.03893899999997],[-103.11562899999996,39.03845300000029],[-103.12535300000002,39.038601000000085],[-103.16302499999995,39.03760999999997],[-103.16312099999999,39.05591900000019],[-103.16318499999988,39.08060600000016],[-103.16303699999997,39.085113000000035],[-103.163209,39.09557000000018],[-103.16306199999991,39.10630900000018],[-103.16317300000003,39.11026000000004],[-103.1630199999999,39.12499200000025],[-103.16307199999994,39.127991000000065],[-103.15525300000002,39.12798399999997],[-103.15508899999998,39.13347999999996],[-103.15512999999999,39.16701699999999],[-103.15492699999987,39.22420300000027],[-103.15515199999993,39.2439500000001],[-103.15511999999995,39.25820500000003],[-103.15519299999994,39.27312800000004],[-103.1551839999999,39.288686000000155],[-103.155009,39.33138300000007],[-103.15507099999996,39.33433600000018],[-103.15498499999995,39.367982000000154],[-103.15514599999989,39.3777970000001],[-103.15511799999996,39.388499999999965],[-103.15523599999995,39.38897000000014],[-103.15490199999994,39.40398600000003],[-103.15482999999995,39.43297100000001],[-103.15465899999998,39.4668710000002],[-103.15475600000002,39.49106500000005],[-103.154586,39.51047799999992],[-103.15437600000001,39.565654000000166],[-103.15399199999996,39.56595900000002],[-103.14348399999989,39.565922000000114],[-103.12412199999994,39.566263000000276],[-103.10793699999999,39.56660700000015],[-103.09036199999997,39.56690300000014],[-103.08613899999995,39.56685500000003],[-103.06813199999999,39.567275000000166],[-103.04343399999988,39.56757299999998],[-103.03390499999989,39.56773900000019],[-103.02819999999991,39.56759000000011],[-103.0192019999999,39.567611000000056],[-103.00949800000001,39.56786299999999],[-103.00049200000001,39.567892000000086],[-102.95418799999987,39.568265],[-102.95285499999989,39.568381000000045],[-102.92924700000003,39.56857700000006],[-102.889861,39.56840599999998],[-102.87547799999993,39.5682680000001],[-102.86478699999998,39.568504000000246],[-102.83131500000002,39.568033000000185],[-102.81615699999986,39.56797600000027],[-102.80293199999994,39.5678410000001],[-102.79398199999997,39.56774800000022],[-102.740925,39.56798300000031],[-102.718032,39.568022000000155],[-102.64960999999988,39.56861400000014],[-102.61494399999998,39.56890900000019],[-102.612368,39.56885700000015],[-102.59299799999997,39.5691030000001],[-102.57086199999992,39.56929400000024],[-102.5520279999999,39.57063900000014],[-102.54398200000003,39.57068700000002],[-102.526611,39.57065700000004],[-102.5,39.57069800000005],[-102.47448199999991,39.57082600000001],[-102.45893000000001,39.57085600000016],[-102.45860500000003,39.570764],[-102.44669399999987,39.570788999999934],[-102.42567799999995,39.57066400000008],[-102.41055799999992,39.570745000000215],[-102.38541499999997,39.57076599999999],[-102.37547299999989,39.57086199999998],[-102.36983199999997,39.57075400000025],[-102.32656800000001,39.57107899999994],[-102.3135519999999,39.571137000000135],[-102.30508399999985,39.5713750000001],[-102.29708,39.57133600000009],[-102.29131399999994,39.571544000000245],[-102.27351799999991,39.571535999999924],[-102.26033099999995,39.571750000000236],[-102.25252499999999,39.571970000000135],[-102.23545699999994,39.57215500000018],[-102.20345800000001,39.57257299999992],[-102.19106899999997,39.572792000000106],[-102.17287499999998,39.57296800000012],[-102.15821999999997,39.57300700000013],[-102.12544700000001,39.573262000000284],[-102.11035099999992,39.57328200000018],[-102.08799799999997,39.573647000000165],[-102.08209099999999,39.57366800000011],[-102.04980599999999,39.57405800000009]]]},"properties":{"OBJECTID":33,"COUNTYFP":"063","Shape_Leng":3.28544290363,"Shape_Area":0.584674127067,"low":2020,"lowmod":3420,"LMMI":4815,"lowmoduniv":7045,"lowmod_pct":0.485450674237,"NAMELSAD10":"Kit Carson County"}},
{"type":"Feature","geometry":{"type":"Polygon","coordinates":[[[-106.20673199999993,39.379636000000005],[-106.19141199999996,39.379696000000195],[-106.1364119999999,39.37969400000003],[-106.13552900000002,39.37954600000023],[-106.13655199999994,39.37884000000025],[-106.138441,39.37498800000003],[-106.13850499999995,39.37224099999992],[-106.1412939999999,39.370914000000084],[-106.14165000000003,39.36996999999997],[-106.144361,39.368120000000204],[-106.14672199999995,39.367972000000066],[-106.14879499999995,39.36835000000008],[-106.15294299999988,39.36276099999998],[-106.15067199999993,39.35973200000001],[-106.14844800000003,39.35861300000016],[-106.14826699999998,39.35684800000013],[-106.14718099999999,39.353628000000185],[-106.14721399999996,39.349904000000095],[-106.14636899999988,39.34828700000003],[-106.14458000000002,39.346852000000126],[-106.144407,39.34477300000026],[-106.14341699999994,39.343124999999986],[-106.13992599999995,39.33955300000002],[-106.14219300000002,39.336890000000096],[-106.14380199999994,39.333867999999995],[-106.14715999999987,39.32852399999996],[-106.147153,39.32760499999995],[-106.14895299999989,39.32646899999992],[-106.150485,39.324349999999924],[-106.1512459999999,39.32069400000012],[-106.15357299999988,39.32041800000002],[-106.1623259999999,39.321628000000146],[-106.16780499999999,39.32164299999994],[-106.16947399999987,39.32223200000021],[-106.17418099999998,39.31827600000008],[-106.17476399999998,39.317112000000236],[-106.17505399999988,39.31441800000027],[-106.17642799999999,39.31207500000005],[-106.18003799999997,39.311195000000225],[-106.18183599999998,39.3112270000002],[-106.18370899999996,39.310283000000254],[-106.18397199999987,39.30935399999993],[-106.18303799999995,39.30327199999999],[-106.18305099999998,39.30111199999993],[-106.18091099999998,39.29880000000014],[-106.18257799999998,39.293060000000025],[-106.18188700000002,39.28996799999999],[-106.18499699999995,39.28661500000021],[-106.18545799999998,39.28575100000012],[-106.18493899999993,39.28339100000028],[-106.18675899999988,39.27985100000029],[-106.18574799999999,39.27580899999998],[-106.18566499999997,39.27328599999993],[-106.1841419999999,39.26886000000002],[-106.18145299999998,39.26536599999997],[-106.18164300000001,39.2632930000002],[-106.18048199999993,39.26109700000006],[-106.1806489999999,39.25868600000007],[-106.18159100000003,39.25700500000022],[-106.18095099999994,39.25244500000002],[-106.18174599999998,39.25064299999997],[-106.18106599999993,39.24816100000015],[-106.18348299999991,39.24439400000006],[-106.18511999999998,39.242694000000256],[-106.18398499999995,39.24047800000005],[-106.18278699999996,39.239607999999976],[-106.17993000000001,39.239446999999984],[-106.17683999999997,39.23855600000002],[-106.17293499999988,39.23680500000006],[-106.16949799999998,39.23375800000019],[-106.16830399999986,39.23072000000019],[-106.16891199999992,39.22876000000019],[-106.16991999999999,39.22810100000021],[-106.16965799999991,39.22562200000027],[-106.1701579999999,39.222994000000085],[-106.17427299999991,39.22072700000007],[-106.17821600000002,39.219132000000116],[-106.17718100000002,39.21814699999993],[-106.17783299999996,39.21634],[-106.17840899999993,39.21297800000019],[-106.182051,39.20994300000007],[-106.18474399999997,39.209308000000135],[-106.18348699999996,39.20671400000009],[-106.18132399999996,39.20475299999998],[-106.179506,39.19999800000005],[-106.1797699999999,39.19490600000006],[-106.1793199999999,39.19349300000022],[-106.18221699999987,39.18967299999997],[-106.1840969999999,39.18826800000011],[-106.18552599999992,39.18658700000026],[-106.18332599999997,39.182443000000035],[-106.18281599999989,39.18040300000018],[-106.17917499999999,39.17598800000013],[-106.17840999999993,39.174579999999935],[-106.17812799999984,39.168884000000105],[-106.1785589999999,39.16596800000025],[-106.17944699999993,39.16503600000004],[-106.17886199999992,39.16333500000019],[-106.17685299999988,39.16031600000014],[-106.17651000000001,39.15788599999996],[-106.17534899999998,39.15629800000022],[-106.17554199999995,39.152918],[-106.17429599999997,39.149364000000105],[-106.17481299999992,39.14801800000009],[-106.17346099999997,39.144795999999985],[-106.17348600000003,39.14359500000006],[-106.17482099999978,39.14142299999992],[-106.17294599999991,39.13876500000009],[-106.17384700000002,39.13707599999998],[-106.17841999999996,39.134769999999946],[-106.181919,39.13148099999995],[-106.18229600000001,39.13029],[-106.18439899999993,39.12929900000006],[-106.18579999999997,39.129249000000016],[-106.18584099999998,39.12683900000013],[-106.18763100000001,39.12459700000005],[-106.18826200000001,39.12142899999998],[-106.189302,39.120574999999974],[-106.19146999999987,39.12086899999997],[-106.19949099999997,39.11550800000015],[-106.20348299999995,39.11408900000015],[-106.20339499999994,39.111962000000005],[-106.20389499999993,39.11128800000006],[-106.20629999999989,39.11043600000005],[-106.20736499999998,39.1095370000001],[-106.20770900000002,39.10807099999994],[-106.21017299999988,39.10391800000025],[-106.20953500000002,39.1030750000001],[-106.20785199999995,39.103423000000134],[-106.20519199999995,39.103255000000274],[-106.20339899999993,39.102732000000174],[-106.19988799999987,39.10436600000003],[-106.19576799999993,39.104472000000044],[-106.19292999999988,39.10305100000028],[-106.19042200000001,39.102752000000066],[-106.18785999999994,39.10373600000008],[-106.18327199999993,39.10340300000007],[-106.17921799999999,39.10346300000026],[-106.17811099999994,39.10295800000006],[-106.17740000000003,39.101312000000064],[-106.175389,39.09880500000014],[-106.17296099999999,39.097278000000074],[-106.17152399999992,39.09518200000002],[-106.16986699999995,39.090731000000176],[-106.1698639999999,39.0887230000003],[-106.16928299999995,39.08778600000022],[-106.16966299999996,39.0864170000001],[-106.17110200000002,39.08610900000002],[-106.17279499999995,39.08396700000026],[-106.17569000000003,39.082939000000124],[-106.18032799999997,39.07724500000006],[-106.18358899999998,39.07647500000013],[-106.18610899999993,39.074987000000306],[-106.18678899999998,39.07312900000022],[-106.1865239999999,39.07163800000012],[-106.18849199999994,39.07004000000006],[-106.19124599999998,39.06844800000016],[-106.19160799999992,39.067499],[-106.19048099999992,39.06398300000001],[-106.19069599999995,39.061944000000096],[-106.19173299999994,39.05921700000022],[-106.19093499999991,39.05748599999998],[-106.19100600000002,39.05607300000014],[-106.23164700000001,39.056459000000075],[-106.24643500000002,39.05638300000021],[-106.29677499999997,39.056437000000074],[-106.30868599999997,39.05615000000029],[-106.34715499999987,39.05646200000001],[-106.36863,39.0564720000001],[-106.37557999999996,39.05630499999995],[-106.38906799999995,39.056295000000205],[-106.45739400000002,39.056911000000014],[-106.49506599999995,39.057252000000176],[-106.500583,39.057475000000125],[-106.577966,39.05790800000011],[-106.57763,39.060598000000255],[-106.57686199999995,39.06336900000002],[-106.57722999999993,39.06615400000004],[-106.577587,39.07275800000025],[-106.57952399999994,39.07557400000019],[-106.579456,39.07753600000001],[-106.57707599999998,39.0794370000001],[-106.58010699999994,39.081751000000224],[-106.58323799999994,39.08295899999996],[-106.58491600000002,39.08483000000007],[-106.58633800000001,39.087743000000216],[-106.58615699999996,39.09404999999998],[-106.58378099999993,39.0979040000002],[-106.58031099999977,39.10216400000024],[-106.57658600000002,39.10400099999998],[-106.57334400000002,39.104423999999995],[-106.56893300000002,39.1040090000003],[-106.56843900000001,39.105205000000126],[-106.56698299999977,39.10506099999998],[-106.56398899999994,39.10604100000006],[-106.56349599999993,39.10824299999996],[-106.56469599999991,39.10908000000012],[-106.56453799999997,39.11197900000019],[-106.56498599999992,39.1140070000003],[-106.563895,39.11574800000028],[-106.56066699999997,39.117898000000196],[-106.55817099999996,39.12022300000018],[-106.55832499999991,39.12423700000011],[-106.560179,39.1271220000001],[-106.56041399999992,39.12844799999999],[-106.56372599999986,39.131199000000265],[-106.56457399999994,39.13242200000002],[-106.56424399999992,39.13423700000027],[-106.56053599999996,39.13782400000008],[-106.55949999999996,39.1396840000001],[-106.55964299999988,39.140881000000036],[-106.55867899999998,39.14384699999994],[-106.56025099999982,39.1453420000002],[-106.56104499999998,39.14776999999998],[-106.56108499999999,39.14996700000006],[-106.56263100000001,39.15194700000029],[-106.56126499999988,39.155677000000026],[-106.557186,39.1567100000002],[-106.55636400000003,39.15794900000003],[-106.55167699999998,39.16146500000002],[-106.54862599999996,39.160549999999944],[-106.54502200000002,39.15887200000003],[-106.54288399999996,39.15888199999995],[-106.53852599999988,39.15789400000011],[-106.536112,39.15943700000025],[-106.53577699999988,39.16258499999992],[-106.53513799999996,39.16314599999998],[-106.53263399999997,39.16246800000022],[-106.53128800000002,39.16122400000012],[-106.527357,39.161021000000005],[-106.52401999999995,39.158754000000215],[-106.52094199999999,39.157353000000114],[-106.51929899999993,39.15860200000026],[-106.518913,39.15969800000016],[-106.51637399999987,39.1603360000002],[-106.51386200000002,39.162258000000236],[-106.511077,39.163532000000146],[-106.50861399999997,39.16668100000027],[-106.50874399999998,39.169993000000034],[-106.50951099999992,39.171928000000264],[-106.50971700000002,39.17487700000004],[-106.50821599999989,39.17732300000006],[-106.50672599999996,39.17881500000004],[-106.50480299999992,39.17902000000015],[-106.50264899999996,39.180419000000086],[-106.49960699999997,39.183848000000296],[-106.49899099999999,39.186217],[-106.49831699999993,39.19201400000003],[-106.49773499999998,39.19322199999999],[-106.49583599999988,39.19441000000023],[-106.49364899999995,39.198326000000236],[-106.49459999999993,39.20014800000018],[-106.49377899999996,39.206367000000114],[-106.494954,39.20854100000008],[-106.49485399999992,39.210091000000034],[-106.49294299999997,39.21585800000008],[-106.49387000000002,39.218728000000056],[-106.49339399999991,39.22010299999994],[-106.49195099999997,39.22069799999997],[-106.49125800000002,39.22279300000031],[-106.49225999999987,39.2251950000001],[-106.49181799999997,39.226102000000196],[-106.48974900000002,39.22771799999998],[-106.4927209999999,39.22940799999998],[-106.49509399999988,39.23191800000001],[-106.49725699999988,39.232977999999946],[-106.49874,39.234931000000074],[-106.49764899999997,39.24202200000025],[-106.49677799999995,39.243321000000265],[-106.49397199999993,39.24495200000001],[-106.48825099999999,39.249264000000096],[-106.48615399999994,39.25390700000003],[-106.48312699999997,39.25510300000008],[-106.48297699999989,39.25801799999999],[-106.48122599999988,39.25916700000005],[-106.48300599999988,39.26213000000007],[-106.48327199999994,39.264272000000005],[-106.4840699999998,39.265396000000294],[-106.48341699999997,39.268852000000265],[-106.48372199999989,39.271702000000005],[-106.483292,39.272669000000235],[-106.48982899999999,39.273140000000126],[-106.49277699999993,39.27627600000005],[-106.49322899999993,39.27763199999998],[-106.49532199999999,39.27870200000007],[-106.49854399999987,39.279324999999915],[-106.4999259999999,39.28167300000007],[-106.50080999999994,39.28203100000019],[-106.504031,39.28476100000029],[-106.50458100000003,39.2873620000002],[-106.50286699999998,39.289993000000095],[-106.5035969999999,39.293609000000174],[-106.50356099999993,39.295051000000285],[-106.50268999999997,39.29806700000006],[-106.50164699999993,39.29981800000019],[-106.49989799999997,39.301483000000076],[-106.49617799999987,39.30284500000016],[-106.49423399999995,39.30307899999997],[-106.49251500000003,39.303995999999984],[-106.49260399999991,39.30573700000002],[-106.49184999999989,39.306620000000066],[-106.48814799999991,39.30901400000005],[-106.48743100000002,39.30897000000016],[-106.48639899999995,39.31147099999998],[-106.48512099999999,39.31256599999995],[-106.476224,39.31474800000001],[-106.476205,39.312498000000005],[-106.47287799999998,39.308423000000005],[-106.47072600000001,39.30788700000005],[-106.46872300000001,39.307896000000085],[-106.46696399999996,39.30924800000025],[-106.46680900000001,39.31041200000027],[-106.46551199999999,39.311578999999995],[-106.46338199999997,39.311494000000266],[-106.46148199999988,39.31261400000005],[-106.45907599999992,39.313041],[-106.4558649999999,39.31213500000001],[-106.45421899999997,39.312259000000154],[-106.45274599999993,39.31429200000014],[-106.45238899999998,39.315669000000014],[-106.45065199999988,39.31669800000026],[-106.44847199999998,39.3166940000001],[-106.44582500000001,39.315495],[-106.4425399999999,39.315291],[-106.43977100000001,39.317321000000106],[-106.438223,39.316854000000205],[-106.43451399999998,39.31710199999992],[-106.43245999999994,39.316352999999935],[-106.43042199999991,39.31937800000031],[-106.42801699999995,39.32145000000014],[-106.42694699999987,39.32335500000022],[-106.42870299999987,39.32818200000008],[-106.42866300000003,39.33255500000007],[-106.429438,39.33438899999993],[-106.42867599999994,39.33540200000027],[-106.43228199999999,39.335956000000124],[-106.43354799999992,39.337492],[-106.43493099999995,39.34109900000021],[-106.43536,39.343972000000235],[-106.43510399999985,39.34727499999997],[-106.4336689999999,39.351113999999995],[-106.43347999999997,39.35417400000023],[-106.43078500000001,39.35711400000031],[-106.42942799999997,39.35803200000004],[-106.42897099999993,39.35997500000008],[-106.42648700000001,39.361871000000065],[-106.42162599999989,39.36379700000009],[-106.41727199999997,39.36729600000024],[-106.414785,39.367489999999975],[-106.41310699999997,39.37005799999997],[-106.41093499999994,39.37199700000019],[-106.40815399999997,39.37369600000005],[-106.40745099999998,39.37567800000028],[-106.40514699999994,39.378926999999976],[-106.4039249999999,39.37984799999998],[-106.40063399999991,39.380962000000125],[-106.39760299999995,39.37984500000027],[-106.39165799999995,39.379397000000154],[-106.39025900000001,39.37812500000024],[-106.38603999999998,39.37655600000028],[-106.38365399999992,39.37660600000015],[-106.38063299999999,39.37615000000005],[-106.37857699999995,39.371459999999956],[-106.37820199999982,39.36940100000015],[-106.37638300000003,39.368541999999934],[-106.37469399999992,39.365700000000174],[-106.37197500000002,39.3637080000002],[-106.37157299999996,39.361721999999986],[-106.369595,39.360284000000036],[-106.36678899999993,39.36064800000014],[-106.36313899999999,39.359234000000015],[-106.36264299999988,39.357217000000105],[-106.358022,39.35514000000023],[-106.35557299999977,39.35486100000003],[-106.35174199999989,39.3536830000001],[-106.34990399999987,39.354936000000066],[-106.34519999999992,39.35678600000017],[-106.34413199999995,39.35771200000022],[-106.33925199999993,39.35906399999999],[-106.33478999999988,39.35895100000005],[-106.3329159999999,39.358230000000276],[-106.32800299999991,39.35923600000001],[-106.32383199999987,39.35898200000014],[-106.32060099999995,39.359375],[-106.31624899999986,39.361211000000026],[-106.31237099999998,39.362278],[-106.30637899999999,39.3613390000001],[-106.30255699999992,39.36111900000003],[-106.30041499999999,39.360067000000015],[-106.29780999999991,39.35795200000018],[-106.29631399999994,39.35733499999992],[-106.29530899999992,39.35293600000011],[-106.29231199999992,39.35184900000024],[-106.286587,39.352220000000216],[-106.28321199999999,39.35046900000009],[-106.27966199999997,39.35234900000023],[-106.27473199999991,39.355683000000056],[-106.27257599999996,39.356358000000284],[-106.27095399999996,39.35807400000016],[-106.26661799999994,39.358835000000056],[-106.26382199999989,39.35876900000028],[-106.25996299999997,39.36084599999998],[-106.25945599999994,39.36186100000015],[-106.25755800000002,39.361515999999995],[-106.25248499999987,39.359111000000155],[-106.250898,39.35876999999999],[-106.2501539999999,39.36070100000006],[-106.24739599999998,39.36280399999998],[-106.24500599999993,39.36410699999999],[-106.23951699999998,39.36566700000009],[-106.23731700000002,39.368517],[-106.23648099999986,39.3703270000002],[-106.23392499999994,39.372936000000095],[-106.22958800000004,39.373167000000024],[-106.226898,39.37416499999995],[-106.22678699999989,39.37705400000027],[-106.22562700000003,39.37963000000025],[-106.20673199999993,39.379636000000005]]]},"properties":{"OBJECTID":34,"COUNTYFP":"065","Shape_Leng":1.72745260896,"Shape_Area":0.103687162183,"low":1770,"lowmod":3115,"LMMI":5090,"lowmoduniv":7205,"lowmod_pct":0.432338653713,"NAMELSAD10":"Lake County"}},
{"type":"Feature","geometry":{"type":"Polygon","coordinates":[[[-107.75060999999994,37.640315000000044],[-107.74631899999991,37.63986999999997],[-107.74318700000003,37.63954700000028],[-107.69185199999998,37.63957700000009],[-107.691528,37.63957700000009],[-107.691371,37.63957799999997],[-107.69069999999999,37.63957799999997],[-107.686914,37.63957999999997],[-107.66195900000002,37.63959600000004],[-107.65369899999996,37.639602000000025],[-107.64196399999997,37.639610000000005],[-107.61692099999993,37.63962500000014],[-107.60139299999997,37.63962700000013],[-107.57629400000002,37.639632000000006],[-107.55132899999995,37.63963000000001],[-107.52941799999996,37.63962800000007],[-107.50320599999992,37.63962800000007],[-107.50061199999999,37.63962800000007],[-107.49909899999994,37.639478000000054],[-107.48217899999992,37.639500999999996],[-107.48217899999992,37.63110799999998],[-107.482169,37.62069900000006],[-107.482122,37.58745899999991],[-107.48210699999998,37.58222500000011],[-107.48206299999993,37.56652299999996],[-107.48203699999999,37.55715100000009],[-107.48194099999995,37.522064000000114],[-107.48188600000003,37.50160299999999],[-107.48219799999998,37.497261000000094],[-107.48219399999999,37.49634100000003],[-107.48211099999997,37.47562900000014],[-107.48207300000001,37.46612099999999],[-107.48204399999986,37.457148000000075],[-107.48206199999993,37.45001600000006],[-107.48213099999987,37.42267300000026],[-107.482238,37.37727200000023],[-107.48202499999996,37.371271000000206],[-107.48208399999999,37.33716300000003],[-107.48240900000002,37.32443500000011],[-107.48213299999992,37.32175300000023],[-107.482079,37.30602800000014],[-107.482211,37.270880000000034],[-107.48225899999989,37.25000299999999],[-107.4820529999999,37.213230000000294],[-107.4820959999999,37.158187000000055],[-107.48210699999998,37.12349900000021],[-107.48142399999989,37.093861000000004],[-107.48155099999991,37.055678],[-107.48173699999995,37.000005000000044],[-107.53134499999999,37.00000399999999],[-107.57427099999995,37.00000300000005],[-107.64934900000003,37.00000300000005],[-107.7291939999999,37.000001999999995],[-107.78002900000001,37.00000000000017],[-107.84289699999988,37.000001999999995],[-107.91631000000001,37.00000300000005],[-108.0006229999999,37.000001000000054],[-108.03345999999999,36.999748000000125],[-108.09125199999994,36.999645999999984],[-108.12508499999996,36.99959599999994],[-108.12632499999995,36.99950900000022],[-108.18858999999992,36.99926700000009],[-108.24935799999992,36.99901500000021],[-108.25062500000001,36.9995550000001],[-108.26723699999997,36.99958300000014],[-108.28840000000002,36.999520000000246],[-108.31510600000001,36.99949900000013],[-108.32075299999997,36.99954700000001],[-108.35717899999992,36.999423000000206],[-108.37920300000002,36.999513000000206],[-108.38106900000002,37.00146799999993],[-108.37961200000001,37.004177000000084],[-108.37814499999996,37.005641999999966],[-108.37643099999997,37.006620000000055],[-108.37728199999987,37.00923399999999],[-108.3766779999998,37.01183800000001],[-108.37478899999991,37.01365200000015],[-108.37509699999993,37.01556000000028],[-108.37405999999999,37.016590000000235],[-108.374323,37.01792300000022],[-108.37321299999991,37.01873600000016],[-108.37178,37.02123900000015],[-108.37334399999992,37.0246930000003],[-108.37250599999993,37.025756000000115],[-108.372319,37.028481],[-108.3717999999999,37.0291050000003],[-108.36868099999992,37.028875000000085],[-108.36582699999997,37.02827900000017],[-108.36183900000003,37.026828000000194],[-108.36019099999999,37.02676300000013],[-108.35902299999998,37.02593800000011],[-108.35555899999991,37.025555],[-108.35643999999996,37.028431000000126],[-108.355031,37.031158000000005],[-108.35286299999996,37.03329000000002],[-108.355118,37.03557000000018],[-108.35454099999987,37.038356000000135],[-108.35071199999987,37.04070200000007],[-108.34985399999994,37.042603000000156],[-108.34718799999979,37.045912000000044],[-108.34700299999997,37.047772000000066],[-108.34572400000002,37.04846900000001],[-108.34304399999996,37.048794999999984],[-108.34115299999996,37.05101500000001],[-108.33922199999989,37.05195800000024],[-108.33736699999997,37.05232100000006],[-108.334792,37.05422000000016],[-108.33421900000002,37.05585700000006],[-108.334719,37.059815000000185],[-108.33338700000002,37.06079800000015],[-108.33445999999998,37.06275900000003],[-108.3343129999999,37.06456100000008],[-108.33346499999999,37.06561500000009],[-108.33317699999998,37.067443000000026],[-108.33045299999998,37.0678760000003],[-108.32826599999987,37.06912100000028],[-108.32773099999991,37.07124699999997],[-108.33015099999994,37.07445400000029],[-108.32833499999998,37.075660000000084],[-108.32681400000001,37.0754340000002],[-108.32570099999992,37.07615699999997],[-108.32567499999999,37.0795270000001],[-108.32410600000003,37.081345999999996],[-108.325087,37.08337800000004],[-108.32360099999988,37.08520900000002],[-108.32331199999987,37.087814000000094],[-108.32241899999997,37.08876099999992],[-108.32307000000003,37.09071499999999],[-108.32207399999993,37.09343500000011],[-108.32061899999997,37.095444999999984],[-108.32106499999992,37.097099999999955],[-108.317341,37.09753500000011],[-108.316261,37.099834999999985],[-108.31662299999994,37.102363000000025],[-108.31460099999987,37.10403600000012],[-108.31059099999999,37.10462200000006],[-108.31028999999995,37.107331999999985],[-108.311173,37.10984199999996],[-108.30972199999997,37.111154],[-108.309845,37.11281300000013],[-108.31141799999995,37.11416500000007],[-108.31179899999995,37.11682400000018],[-108.30961999999994,37.11983900000024],[-108.31112299999995,37.12312600000007],[-108.31099299999994,37.12415200000004],[-108.3096579999999,37.12496699999997],[-108.30977100000001,37.12730200000027],[-108.31047999999998,37.128461999999956],[-108.309842,37.129968000000076],[-108.30724399999991,37.13182599999993],[-108.30410799999987,37.135432000000094],[-108.301513,37.136440000000164],[-108.30089299999997,37.13878700000021],[-108.29422699999998,37.14444700000007],[-108.29045899999994,37.14597500000002],[-108.29014999999987,37.146959000000265],[-108.29124100000001,37.14932799999997],[-108.29021399999993,37.152206999999976],[-108.29016899999993,37.154201000000114],[-108.29188999999997,37.156032000000096],[-108.29440999999991,37.15717200000029],[-108.29607299999986,37.158422000000144],[-108.29576199999997,37.159696000000224],[-108.29387999999994,37.16239400000018],[-108.29404499999993,37.16476000000017],[-108.29588199999995,37.167084999999986],[-108.29741499999994,37.171159999999986],[-108.29730799999999,37.17210399999993],[-108.29540099999986,37.175310000000195],[-108.29462199999989,37.18019199999998],[-108.2946,37.18186300000025],[-108.29592499999995,37.186874000000046],[-108.29572099999996,37.19007100000027],[-108.29378999999977,37.192518000000234],[-108.29290000000003,37.194323],[-108.29262999999992,37.19831900000008],[-108.293273,37.199993000000006],[-108.29288299999996,37.202462000000196],[-108.29380399999997,37.20333699999992],[-108.29485699999998,37.20560200000011],[-108.2971609999999,37.207557000000065],[-108.29746699999998,37.208429999999964],[-108.29728299999988,37.213233],[-108.29459599999996,37.215363000000025],[-108.29379599999993,37.21737000000019],[-108.2915349999999,37.22028699999993],[-108.29167599999994,37.22200300000003],[-108.29042299999992,37.22358400000007],[-108.289354,37.22353000000004],[-108.28627699999987,37.22478800000022],[-108.28465,37.226393000000144],[-108.28168299999999,37.2285590000003],[-108.27974199999989,37.228571999999986],[-108.27884499999993,37.22724400000021],[-108.27691300000004,37.22870400000022],[-108.27078699999987,37.22920700000009],[-108.26890600000002,37.230159000000185],[-108.26770099999999,37.231966000000114],[-108.267536,37.233087000000125],[-108.26144799999997,37.23503100000005],[-108.26126999999997,37.23675700000007],[-108.26007900000002,37.239305],[-108.25767100000002,37.240352000000314],[-108.25613699999991,37.24264900000014],[-108.25596999999976,37.243695],[-108.25249600000001,37.24561],[-108.25063999999992,37.246313999999984],[-108.24962099999999,37.24955100000011],[-108.24731199999991,37.24989000000005],[-108.246757,37.252312000000074],[-108.24660099999977,37.25492900000012],[-108.245046,37.25747300000006],[-108.24429099999998,37.25801400000006],[-108.24246199999999,37.26142800000014],[-108.23993999999993,37.26184600000005],[-108.23858899999999,37.262667000000135],[-108.23740999999995,37.266976],[-108.23715399999992,37.270433000000196],[-108.23528099999993,37.27261500000026],[-108.23483799999997,37.27476800000022],[-108.23352999999992,37.27573500000011],[-108.22867600000001,37.27696400000002],[-108.22731199999987,37.278166],[-108.22687599999995,37.28050200000001],[-108.22802999999993,37.281949],[-108.22569099999993,37.28627300000005],[-108.2242819999999,37.286950000000274],[-108.21968800000002,37.28741500000001],[-108.2197049999998,37.29075800000021],[-108.21896800000002,37.29215099999999],[-108.21937599999995,37.29380400000002],[-108.21798899999993,37.29725000000019],[-108.21377100000001,37.298118000000045],[-108.21250399999997,37.300151000000085],[-108.21308499999998,37.30329699999993],[-108.216094,37.305488000000025],[-108.21649399999995,37.30622900000026],[-108.21647699999988,37.31161500000002],[-108.21361799999994,37.31316199999992],[-108.21382099999988,37.3145100000001],[-108.21202499999987,37.31747300000012],[-108.21142299999997,37.320512000000065],[-108.21048200000001,37.321744000000194],[-108.20794599999988,37.32325600000024],[-108.20521499999995,37.32445699999994],[-108.20303599999988,37.32631600000013],[-108.204679,37.32812500000023],[-108.204811,37.32954600000005],[-108.2067019999999,37.33224100000007],[-108.20792099999994,37.33307200000007],[-108.20772799999997,37.33507700000007],[-108.20659599999999,37.337630000000274],[-108.20576,37.33856600000007],[-108.20498499999985,37.34158900000028],[-108.20483599999994,37.34407999999996],[-108.20325299999996,37.3472440000001],[-108.20198099999988,37.34869099999992],[-108.19824199999988,37.35149999999999],[-108.19777599999998,37.354939],[-108.19566899999995,37.35607300000004],[-108.19268799999998,37.35907000000003],[-108.19042899999994,37.36015100000009],[-108.18894599999999,37.36016600000028],[-108.18446299999994,37.36239600000022],[-108.18359899999996,37.363109000000236],[-108.17913899999996,37.36403000000024],[-108.17646200000001,37.364892],[-108.17303199999986,37.36638800000014],[-108.16658599999988,37.365174000000025],[-108.16463199999987,37.36345700000027],[-108.16429900000003,37.36234300000012],[-108.16170399999999,37.36222499999991],[-108.1581059999998,37.36389400000019],[-108.15247299999993,37.362589000000014],[-108.15001099999989,37.363550000000146],[-108.147041,37.3653790000003],[-108.14564899999993,37.367363000000125],[-108.14219599999996,37.369741999999974],[-108.14119999999997,37.37196700000004],[-108.13982099999993,37.37303800000001],[-108.13718299999988,37.37309900000008],[-108.13373000000001,37.373725000000206],[-108.13234899999986,37.37467800000019],[-108.12048099999987,37.37471500000004],[-108.11723199999989,37.37674700000031],[-108.11649099999994,37.37828300000001],[-108.11361599999992,37.3803460000002],[-108.10917399999994,37.381542000000024],[-108.10585700000001,37.38206800000006],[-108.10625999999996,37.38457299999999],[-108.1056539999999,37.38912000000016],[-108.1074549999999,37.39208600000006],[-108.10755599999999,37.3943230000001],[-108.10665399999988,37.398134000000084],[-108.10630800000001,37.4007380000001],[-108.10569199999986,37.401863000000105],[-108.10533599999997,37.40557300000012],[-108.10341799999992,37.408861],[-108.10126399999996,37.41034700000006],[-108.09954599999992,37.412627000000214],[-108.0991679999999,37.41544100000016],[-108.0977949999999,37.417473000000086],[-108.09514199999984,37.41931900000003],[-108.08950899999996,37.42122400000028],[-108.08754299999993,37.42514000000028],[-108.08728699999989,37.42776900000001],[-108.08603199999988,37.42805900000002],[-108.0828709999999,37.4278920000001],[-108.08004099999988,37.4291870000003],[-108.08105199999994,37.43241400000011],[-108.08062199999989,37.43409000000008],[-108.07954999999993,37.43530900000002],[-108.07904599999989,37.437306000000035],[-108.07804199999987,37.43888099999998],[-108.07341300000002,37.43896100000006],[-108.06880699999994,37.43787000000003],[-108.065406,37.43849100000023],[-108.06374499999998,37.43774499999995],[-108.06141400000001,37.43744600000008],[-108.05739699999992,37.43811800000009],[-108.05516899999998,37.437545000000114],[-108.05302799999993,37.43752100000006],[-108.04844299999996,37.44256400000006],[-108.04450099999997,37.442704000000106],[-108.04328299999997,37.44434900000016],[-108.04240499999992,37.448353],[-108.03987799999999,37.45044900000005],[-108.03826200000003,37.45245399999999],[-108.03914999999978,37.454616000000044],[-108.03915799999993,37.455875000000276],[-108.03746899999999,37.45906599999995],[-108.03475600000002,37.460602999999935],[-108.03400999999997,37.46347000000003],[-108.03314599999987,37.464939999999956],[-108.03638999999987,37.471178000000066],[-108.0359899999998,37.47270900000012],[-108.03470399999998,37.47503900000015],[-108.03451399999989,37.47682900000029],[-108.032644,37.478913000000034],[-108.0336529999999,37.48009899999994],[-108.03376100000003,37.484498000000144],[-108.035932,37.48573299999998],[-108.03654899999987,37.48775499999999],[-108.03655600000002,37.49112700000006],[-108.035345,37.49397499999998],[-108.03517999999985,37.496354],[-108.03460499999989,37.49717300000009],[-108.03632099999993,37.49948599999999],[-108.0379319999999,37.503189000000134],[-108.03727400000002,37.505269999999996],[-108.03407899999996,37.508407000000034],[-108.03147000000001,37.50905899999998],[-108.03275400000001,37.50991499999992],[-108.03405700000002,37.511650000000145],[-108.03452099999976,37.51313500000026],[-108.03369099999992,37.51632100000006],[-108.03214300000002,37.51804599999997],[-108.03358800000001,37.5199990000001],[-108.03463799999986,37.52322600000031],[-108.03380800000002,37.525743000000205],[-108.03374599999995,37.527461000000244],[-108.03452400000003,37.529527000000144],[-108.03489699999989,37.532043999999985],[-108.02890799999994,37.53625199999999],[-108.02543700000001,37.53971899999999],[-108.0254959999998,37.541106000000184],[-108.02411999999993,37.54292100000009],[-108.02423699999991,37.54494300000022],[-108.02353499999992,37.54659400000003],[-108.02153999999996,37.54866700000014],[-108.02190699999994,37.55127900000008],[-108.02025299999997,37.553780000000245],[-108.0205299999999,37.55642300000011],[-108.0196959999999,37.558524000000034],[-108.02119800000003,37.55941400000029],[-108.02174599999995,37.56282399999998],[-108.0182789999999,37.567140999999936],[-108.01922400000001,37.56831999999997],[-108.02085,37.57144500000027],[-108.01950599999998,37.57402700000006],[-108.01947999999987,37.575624000000005],[-108.02016099999992,37.578284],[-108.01974299999995,37.58090400000026],[-108.02053399999994,37.58249000000001],[-108.01971099999997,37.584947000000284],[-108.02120599999995,37.58820700000018],[-108.02300599999984,37.59099900000007],[-108.02243900000002,37.59142600000007],[-108.01738899999998,37.59179399999999],[-108.01644599999992,37.59389900000008],[-108.01688999999999,37.595398999999986],[-108.01394399999987,37.596469999999954],[-108.01410399999997,37.59902800000003],[-108.01553000000001,37.600851999999975],[-108.01429799999994,37.603594000000044],[-108.01318600000002,37.60479000000021],[-108.01184799999999,37.60485800000015],[-108.00983199999979,37.60577700000016],[-108.00766199999993,37.605671000000086],[-108.00467899999995,37.606246000000056],[-108.00461000000001,37.60731700000002],[-108.00264199999998,37.60967400000004],[-108.00074799999993,37.611180000000275],[-108.0001999999999,37.614325000000065],[-107.99488699999995,37.61485700000003],[-107.993153,37.61585900000017],[-107.99171299999995,37.619651999999974],[-107.98972199999997,37.62070700000004],[-107.98921899999993,37.62224099999992],[-107.9872209999998,37.625004000000104],[-107.98591599999992,37.625181],[-107.98195299999986,37.62717200000026],[-107.97987799999987,37.62887500000011],[-107.97208699999987,37.629816000000176],[-107.97244899999998,37.631641],[-107.97212000000002,37.63432100000006],[-107.96992299999988,37.63717000000008],[-107.970531,37.63847599999997],[-107.97008599999987,37.639587000000006],[-107.96500400000002,37.6396610000001],[-107.96473199999997,37.63967200000013],[-107.96380499999992,37.63965900000011],[-107.962083,37.639635999999996],[-107.96025400000002,37.63963500000011],[-107.94573699999995,37.639677000000006],[-107.93324599999994,37.63974500000012],[-107.926064,37.63978399999996],[-107.92579699999999,37.63978599999996],[-107.90325199999995,37.63990300000006],[-107.88625699999994,37.639972000000114],[-107.87181699999996,37.63993599999998],[-107.86156199999994,37.64000700000014],[-107.84563400000002,37.63999600000011],[-107.842265,37.64000800000002],[-107.83216099999993,37.640045999999984],[-107.82874699999996,37.64006000000006],[-107.81081,37.64013],[-107.81023900000002,37.640131999999994],[-107.80568199999999,37.64014800000007],[-107.80343099999999,37.640154999999936],[-107.80142499999994,37.64016399999997],[-107.79655399999996,37.64018200000004],[-107.79584699999998,37.640184999999974],[-107.78959799999996,37.640212000000076],[-107.780955,37.64024600000005],[-107.75890399999997,37.64033000000006],[-107.75060999999994,37.640315000000044]]]},"properties":{"OBJECTID":35,"COUNTYFP":"067","Shape_Leng":3.02310888718,"Shape_Area":0.447314636442,"low":12505,"lowmod":21920,"LMMI":32010,"lowmoduniv":51305,"lowmod_pct":0.427248806159,"NAMELSAD10":"La Plata County"}},
{"type":"Feature","geometry":{"type":"Polygon","coordinates":[[[-104.94337100000001,40.99821300000019],[-104.94315099999994,40.93839800000001],[-104.94305199999991,40.91091400000005],[-104.94312999999988,40.90294799999998],[-104.94309099999992,40.88489500000003],[-104.94321100000002,40.88226400000008],[-104.943242,40.86308600000012],[-104.94349699999998,40.84133900000006],[-104.94354099999998,40.826876000000084],[-104.94345399999997,40.81965100000002],[-104.94335199999995,40.79798500000027],[-104.94334200000003,40.769253000000276],[-104.94346399999989,40.7649390000002],[-104.9434629999999,40.754965000000084],[-104.9433479999999,40.74670699999996],[-104.94354299999986,40.69650900000005],[-104.94310999999999,40.69649500000014],[-104.94349,40.658139000000006],[-104.94348299999996,40.65257099999991],[-104.94394899999998,40.63496000000009],[-104.94401199999993,40.61080000000021],[-104.94410299999993,40.598078999999984],[-104.94412,40.55516700000004],[-104.94424800000002,40.53173500000031],[-104.94425100000001,40.52283700000021],[-104.94407100000001,40.490342000000055],[-104.94415299999997,40.46005000000014],[-104.94427499999995,40.44568600000002],[-104.94422199999991,40.435894000000076],[-104.94436400000001,40.417587000000196],[-104.94472499999995,40.4001640000003],[-104.94518399999993,40.37352000000004],[-104.94555100000002,40.34909900000008],[-104.98408199999994,40.34918800000031],[-105.05671999999993,40.34928100000019],[-105.05513499999995,40.33518300000026],[-105.05522099999996,40.3210450000002],[-105.05537600000002,40.319080000000156],[-105.05510799999985,40.31015900000028],[-105.0551749999999,40.29088300000001],[-105.0550889999999,40.26179300000024],[-105.08303699999993,40.26153500000021],[-105.08734800000002,40.261585999999966],[-105.10753799999998,40.261384000000135],[-105.110138,40.26128400000022],[-105.13017899999988,40.26158399999997],[-105.13105899999994,40.26107899999994],[-105.15903500000002,40.26121899999998],[-105.17964599999988,40.2610830000001],[-105.19647600000002,40.261085000000094],[-105.21311899999995,40.26118500000001],[-105.21394199999992,40.26128499999993],[-105.22644199999996,40.26148499999999],[-105.23731899999996,40.26189500000015],[-105.2600509999998,40.262384999999995],[-105.27344399999987,40.26248499999991],[-105.28944499999989,40.262785000000065],[-105.31331099999989,40.26218600000004],[-105.32674500000002,40.26198499999998],[-105.33624800000001,40.261685],[-105.34700299999992,40.261685],[-105.35474699999992,40.261784000000205],[-105.375226,40.261291000000085],[-105.39099599999997,40.257884000000274],[-105.4023489999999,40.257784000000015],[-105.41666599999996,40.25801200000001],[-105.43255199999993,40.25838400000026],[-105.44389999999999,40.25872300000009],[-105.46275199999991,40.25938400000007],[-105.48617899999999,40.259708000000046],[-105.49532099999993,40.259726000000114],[-105.50048099999992,40.26027700000003],[-105.52724999999998,40.26099299999993],[-105.53535399999998,40.26128400000022],[-105.55958499999991,40.26153100000005],[-105.58105499999999,40.261299000000065],[-105.60939499999978,40.26068300000003],[-105.61776099999992,40.26067300000011],[-105.635693,40.26048200000014],[-105.653321,40.26045700000003],[-105.65723500000001,40.262886000000094],[-105.66071999999997,40.26422800000012],[-105.66237799999993,40.26412400000004],[-105.66441399999991,40.265630999999985],[-105.66523599999988,40.266894000000036],[-105.6679089999999,40.267718],[-105.66918699999985,40.26756800000004],[-105.672055,40.268101000000115],[-105.67559,40.267267000000004],[-105.67818699999987,40.26812000000024],[-105.67890299999988,40.26986900000003],[-105.68242800000002,40.27280000000002],[-105.68034699999998,40.27466800000025],[-105.68124199999994,40.277495000000044],[-105.679262,40.27891200000005],[-105.68073200000003,40.28292700000003],[-105.68057899999997,40.28446000000008],[-105.685046,40.28640900000005],[-105.68634600000001,40.287485999999944],[-105.68640299999993,40.28891800000014],[-105.6853979999999,40.290079000000276],[-105.681848,40.29215500000009],[-105.68475799999999,40.29238299999997],[-105.68736799999994,40.29373300000009],[-105.68827199999987,40.29733400000015],[-105.68577199999987,40.30255400000004],[-105.68617699999999,40.30307499999998],[-105.68929499999985,40.30347900000004],[-105.69049499999994,40.30442100000016],[-105.6912779999999,40.30616400000014],[-105.69013999999987,40.30864800000012],[-105.69186299999996,40.30937300000011],[-105.69483399999984,40.309323000000006],[-105.69617,40.31051700000023],[-105.697764,40.31093899999996],[-105.70090899999997,40.31071500000007],[-105.70223499999992,40.31261000000006],[-105.70016699999996,40.316364000000135],[-105.6976479999999,40.31827299999998],[-105.69871899999998,40.3207800000003],[-105.70201999999989,40.32171100000005],[-105.70235000000002,40.32264900000007],[-105.70168000000001,40.324309000000255],[-105.70677299999994,40.32505100000003],[-105.71312699999999,40.325567999999976],[-105.72143799999992,40.33201800000006],[-105.72453300000001,40.33270000000016],[-105.727372,40.333979000000284],[-105.72908799999988,40.3352780000003],[-105.72878600000001,40.33793500000007],[-105.72994399999999,40.339624000000185],[-105.73310799999996,40.340774000000295],[-105.73544099999998,40.342984],[-105.736444,40.34490600000004],[-105.73634499999991,40.34661500000004],[-105.73961300000002,40.34710800000016],[-105.74156099999999,40.34816100000006],[-105.74244099999999,40.34926300000012],[-105.74500699999993,40.34815800000018],[-105.74775699999992,40.34750800000006],[-105.74866099999991,40.348086999999964],[-105.75238299999995,40.349093000000266],[-105.75287300000002,40.35084599999999],[-105.75711699999988,40.35165000000012],[-105.75927100000001,40.353307000000086],[-105.75976499999985,40.355778999999984],[-105.76302800000002,40.35711800000007],[-105.76444100000003,40.35926300000028],[-105.76769899999988,40.359763999999984],[-105.76832200000001,40.3620820000001],[-105.76946899999996,40.364597],[-105.77259600000002,40.364411000000075],[-105.77410500000002,40.365233000000046],[-105.77421900000002,40.366759],[-105.77643699999999,40.36954500000013],[-105.78014200000001,40.37210399999998],[-105.78194999999994,40.37375900000029],[-105.78146399999997,40.37555100000003],[-105.78181999999993,40.37699000000009],[-105.78499699999998,40.37971500000015],[-105.78724999999997,40.380872000000124],[-105.7885609999999,40.38428200000004],[-105.79072899999989,40.386262999999985],[-105.79123199999992,40.38786300000004],[-105.79054500000001,40.389639999999986],[-105.79103499999997,40.39278300000012],[-105.79166399999997,40.3943350000003],[-105.79095099999995,40.39864],[-105.79170899999997,40.39972300000005],[-105.79381499999994,40.400800000000174],[-105.79418199999986,40.40188200000017],[-105.79826500000001,40.405835000000025],[-105.80042300000002,40.40721800000006],[-105.80117000000001,40.40842200000003],[-105.80129399999998,40.410174999999924],[-105.80826499999989,40.41495400000031],[-105.811126,40.419399],[-105.81533999999988,40.421486000000016],[-105.81612799999988,40.42353800000018],[-105.8172659999999,40.4243230000003],[-105.81723099999994,40.426488000000006],[-105.814456,40.42852200000027],[-105.81433700000002,40.43133400000005],[-105.81636599999985,40.43551900000028],[-105.81578400000001,40.43743400000028],[-105.81301899999994,40.43916100000018],[-105.8117279999999,40.4428530000003],[-105.81039899999996,40.44323100000014],[-105.80655699999988,40.446201999999914],[-105.80568999999991,40.44788200000005],[-105.8072259999999,40.45066200000019],[-105.80761399999994,40.452262000000076],[-105.8057619999999,40.45688999999999],[-105.80444899999998,40.46296500000028],[-105.8044789999999,40.464224],[-105.80569099999991,40.46650000000005],[-105.80817999999988,40.46886800000004],[-105.81083699999999,40.47053099999994],[-105.80886499999997,40.47107800000009],[-105.80671000000001,40.47308100000026],[-105.8073409999999,40.474527000000194],[-105.81105300000002,40.47577300000006],[-105.812793,40.47585500000008],[-105.81542000000002,40.47533900000002],[-105.82011399999999,40.47597200000001],[-105.82472899999993,40.47764100000006],[-105.82795599999986,40.478289000000245],[-105.83060799999998,40.4795680000002],[-105.83460499999995,40.479869000000235],[-105.83782300000001,40.48303299999998],[-105.84108200000003,40.48361800000026],[-105.84306399999997,40.48299400000013],[-105.84696699999995,40.48279100000008],[-105.847308,40.48393100000004],[-105.85492599999998,40.48625200000009],[-105.85582199999988,40.48773800000009],[-105.85610500000001,40.49041199999999],[-105.8557869999999,40.49268200000006],[-105.85477300000002,40.49510600000008],[-105.85202600000002,40.49756900000011],[-105.85175499999991,40.50047500000022],[-105.85229699999996,40.503534],[-105.85230299999995,40.50655300000005],[-105.85473200000001,40.507789],[-105.857058,40.50785600000006],[-105.85820299999989,40.50962900000019],[-105.86062099999992,40.51154900000006],[-105.86189999999999,40.51444900000001],[-105.86310900000001,40.51552100000026],[-105.8663689999999,40.5172570000002],[-105.86751299999997,40.51971400000008],[-105.86909099999997,40.52073800000028],[-105.871805,40.52174800000017],[-105.87496199999998,40.52192900000023],[-105.87804799999998,40.52246500000001],[-105.87904099999997,40.52317400000021],[-105.88195399999995,40.52285300000011],[-105.88385199999988,40.52175100000005],[-105.89110399999993,40.52100300000018],[-105.89457599999997,40.520196],[-105.89911099999995,40.51822499999997],[-105.90417000000002,40.514443000000256],[-105.90482399999996,40.51421599999992],[-105.90785699999992,40.515493000000106],[-105.91077999999999,40.52017900000021],[-105.9111749999999,40.5221810000001],[-105.90976399999988,40.523452000000134],[-105.90786800000001,40.52621000000005],[-105.905914,40.530519000000254],[-105.90508999999986,40.535285000000044],[-105.90553699999998,40.5370200000001],[-105.90714999999994,40.540577999999925],[-105.90704900000003,40.54288800000012],[-105.90957700000001,40.544982000000005],[-105.9095099999999,40.54793000000001],[-105.911833,40.552688000000046],[-105.91081799999995,40.55671200000012],[-105.90991400000001,40.558806000000004],[-105.909784,40.562134000000015],[-105.91089199999993,40.56642000000028],[-105.91320999999988,40.568313000000046],[-105.91469399999994,40.570636000000036],[-105.91844399999997,40.570700000000215],[-105.92124799999999,40.57255900000018],[-105.92224799999985,40.57530300000002],[-105.9234249999999,40.57703700000013],[-105.92497699999996,40.580934000000184],[-105.92438799999991,40.58214600000008],[-105.92577499999999,40.58770700000014],[-105.92662399999989,40.589175000000125],[-105.92654299999987,40.59058900000002],[-105.92769699999997,40.59170900000015],[-105.92811599999999,40.59416299999998],[-105.92770499999989,40.594923000000165],[-105.92822099999995,40.59830500000021],[-105.9281299999999,40.60005799999993],[-105.92899999999986,40.601202000000285],[-105.92895299999998,40.603794000000164],[-105.930207,40.60618999999997],[-105.93471099999988,40.60642999999993],[-105.93868099999997,40.60881100000012],[-105.94206400000002,40.6090770000003],[-105.94453499999986,40.61209200000019],[-105.94474200000002,40.614231000000245],[-105.94658599999991,40.616028000000085],[-105.94700199999994,40.617367],[-105.94511,40.62119100000001],[-105.94549499999988,40.62274100000002],[-105.9471989999999,40.62374600000004],[-105.94926099999998,40.62732],[-105.94866699999994,40.63018500000027],[-105.94857300000001,40.63367800000026],[-105.948062,40.635986000000116],[-105.9483249999999,40.641745000000014],[-105.94915300000002,40.642776000000026],[-105.95081600000003,40.64328900000021],[-105.95107899999977,40.64587800000004],[-105.95384399999995,40.64697699999999],[-105.95461999999992,40.648072000000184],[-105.95476500000001,40.65020999999996],[-105.95598999999987,40.65259500000013],[-105.95668599999999,40.654964000000064],[-105.95588299999997,40.65846699999997],[-105.95783799999992,40.66087399999998],[-105.958639,40.663009999999986],[-105.95954099999994,40.66354899999999],[-105.96229199999993,40.666584000000114],[-105.96503899999993,40.66820700000011],[-105.96511399999991,40.671418000000074],[-105.96645499999994,40.67345499999999],[-105.96666399999992,40.67568100000017],[-105.96774899999997,40.67705000000012],[-105.96939199999997,40.67773999999997],[-105.97187299999996,40.68185600000021],[-105.9711089999999,40.684036000000276],[-105.97012699999993,40.68471600000004],[-105.9700259999999,40.68626499999999],[-105.97358399999996,40.68927400000024],[-105.97375999999997,40.691129000000046],[-105.97219599999977,40.69481100000007],[-105.97347300000001,40.69657400000028],[-105.97708499999999,40.702648000000124],[-105.97941699999996,40.70328000000001],[-105.98136099999994,40.70483200000001],[-105.98244799999992,40.70669200000026],[-105.98602099999988,40.70762700000017],[-105.98873899999995,40.70960500000024],[-105.98976299999987,40.71398199999999],[-105.98958099999999,40.71614500000027],[-105.99007699999999,40.71831300000008],[-105.98807899999997,40.72136800000004],[-105.98928399999994,40.72352000000012],[-105.99093900000003,40.72480100000024],[-105.99057999999997,40.72718000000026],[-105.991965,40.728919000000076],[-105.99626999999992,40.7331260000002],[-105.99867699999987,40.734222999999986],[-106.00056899999993,40.73433700000021],[-106.00402599999995,40.73679400000009],[-106.0060079999999,40.737618999999995],[-106.0083899999999,40.7400640000003],[-106.01069999999999,40.74104099999994],[-106.01035999999982,40.74240100000003],[-106.01221499999991,40.743946000000165],[-106.01268999999996,40.74549000000019],[-106.01397799999995,40.74664400000029],[-106.01435099999992,40.74860600000005],[-106.015447,40.7506030000003],[-106.01483100000002,40.7513780000001],[-106.01526599999994,40.75281600000005],[-106.01888799999995,40.75400300000024],[-106.02225499999992,40.754688999999985],[-106.02191500000004,40.75606200000004],[-106.02065099999999,40.75720099999995],[-106.02067299999987,40.75842900000015],[-106.02179799999988,40.75907499999994],[-106.0238799999999,40.76132800000005],[-106.02334999999994,40.76353400000011],[-106.02433799999989,40.76629600000001],[-106.02664299999998,40.76831500000009],[-106.02448800000002,40.77126400000003],[-106.02924699999988,40.773436000000174],[-106.02956599999999,40.77506800000003],[-106.02885499999996,40.77657800000026],[-106.03037499999988,40.77910600000001],[-106.02968499999997,40.78156200000001],[-106.03123599999998,40.782281000000125],[-106.03196999999994,40.78376700000001],[-106.03521099999989,40.78555100000017],[-106.03965199999999,40.78576000000004],[-106.0412579999998,40.78843399999994],[-106.04280999999997,40.79012999999998],[-106.044308,40.790581000000145],[-106.04537099999976,40.79185700000005],[-106.04600999999997,40.79428899999999],[-106.04597899999999,40.7963730000003],[-106.044896,40.797682000000066],[-106.046694,40.80079100000012],[-106.04647099999983,40.80244700000014],[-106.04797099999996,40.80464300000011],[-106.04958699999992,40.805507000000205],[-106.05165899999992,40.80543400000022],[-106.05150199999991,40.809140999999954],[-106.0532879999999,40.81041700000003],[-106.05377900000002,40.81266200000016],[-106.05509699999999,40.813631000000214],[-106.05636200000004,40.81350999999995],[-106.05779799999993,40.814437999999996],[-106.05771499999997,40.817120000000045],[-106.06253300000003,40.81950900000004],[-106.06483099999991,40.8215670000003],[-106.06843500000002,40.822196000000304],[-106.07131600000002,40.82233100000008],[-106.07312499999995,40.82463299999995],[-106.07530600000001,40.8256090000001],[-106.07639199999988,40.82792500000005],[-106.07782799999995,40.828498000000195],[-106.08086099999991,40.830903000000035],[-106.08322099999998,40.83117699999997],[-106.08791399999996,40.833402999999976],[-106.08859599999988,40.836133000000245],[-106.09050200000001,40.83718100000027],[-106.09249399999999,40.83978100000007],[-106.0926859999999,40.840839000000244],[-106.09553699999998,40.84164500000014],[-106.09877999999998,40.841699000000176],[-106.10126100000002,40.84356300000002],[-106.10013800000002,40.846980000000144],[-106.09899099999996,40.84804700000012],[-106.09851500000002,40.849736000000064],[-106.10244199999994,40.8523340000001],[-106.10422899999992,40.851985000000184],[-106.10749399999997,40.85207899999995],[-106.10773999999998,40.853024000000175],[-106.10628399999996,40.85407500000008],[-106.10428099999996,40.859263],[-106.10469799999998,40.86032900000009],[-106.10776299999992,40.861368000000084],[-106.10840399999995,40.863258000000144],[-106.11319199999997,40.86427900000007],[-106.116536,40.866441000000066],[-106.11897099999993,40.865775999999926],[-106.12080299999991,40.867632000000015],[-106.12254899999994,40.86736400000024],[-106.12464699999998,40.86880800000017],[-106.12689299999994,40.869556000000046],[-106.12769099999997,40.8717850000001],[-106.1306709999999,40.87386800000013],[-106.13242499999996,40.87655300000006],[-106.12997999999999,40.8782130000003],[-106.12825099999992,40.882002000000284],[-106.12761799999998,40.88488999999993],[-106.12816399999986,40.88702100000023],[-106.12904300000002,40.88794200000024],[-106.128987,40.88928099999998],[-106.13142999999997,40.892865000000256],[-106.13073400000002,40.895493999999985],[-106.13125100000002,40.89660800000013],[-106.13410199999998,40.89870900000005],[-106.13622499999997,40.89922800000022],[-106.13701799999996,40.90051999999997],[-106.13662199999987,40.901859000000115],[-106.13928599999991,40.90240200000028],[-106.14177699999988,40.903962000000035],[-106.144069,40.9049],[-106.14788099999993,40.904638000000205],[-106.15069,40.90671500000008],[-106.15219799999988,40.906789000000174],[-106.15459299999998,40.90942100000018],[-106.15855099999999,40.91061900000017],[-106.1595769999999,40.912765999999976],[-106.15920199999994,40.91472599999997],[-106.16116599999992,40.91510700000026],[-106.16304399999996,40.91672699999998],[-106.16816999999998,40.91839800000008],[-106.1694849999999,40.920315000000016],[-106.16908399999994,40.92286800000005],[-106.17002799999989,40.923804000000246],[-106.17343499999987,40.92478300000005],[-106.17589299999986,40.92825400000004],[-106.17872799999986,40.93025700000004],[-106.17994499999992,40.931521000000146],[-106.181624,40.932018000000085],[-106.18530699999991,40.93397299999998],[-106.18617,40.9367270000003],[-106.18759899999986,40.93774500000001],[-106.18716599999999,40.940825000000075],[-106.18929300000002,40.94589300000001],[-106.18765099999979,40.95016600000025],[-106.185629,40.952248000000054],[-106.18422599999991,40.95520700000026],[-106.18682899999999,40.957425000000285],[-106.18739099999988,40.95872000000014],[-106.18683799999997,40.95988800000015],[-106.18525299999999,40.9610320000001],[-106.18552699999992,40.96193599999998],[-106.184844,40.963831000000255],[-106.18770099999995,40.96490600000004],[-106.18675100000002,40.968052000000114],[-106.187097,40.971167000000264],[-106.18972999999994,40.9730890000003],[-106.19040499999994,40.976098000000036],[-106.19004000000001,40.97864000000004],[-106.19213899999988,40.982836000000304],[-106.19055300000002,40.9842550000003],[-106.191124,40.986030000000255],[-106.19493999999992,40.98693600000007],[-106.19537199999985,40.98993800000011],[-106.19423099999995,40.99197000000021],[-106.1923569999999,40.99368800000025],[-106.19155599999988,40.996285],[-106.19055400000002,40.99757800000003],[-106.16092900000001,40.99752200000023],[-106.15007400000002,40.99736400000029],[-106.12554699999993,40.997340000000065],[-106.12067899999994,40.99723599999999],[-106.08873299999993,40.99721199999999],[-106.06118099999998,40.9969990000003],[-106.03713599999992,40.99708499999997],[-106.0166299999999,40.997224000000074],[-106.00056199999995,40.997251000000176],[-105.93205999999998,40.99714000000023],[-105.86041899999992,40.99702600000006],[-105.84340199999997,40.99704700000018],[-105.79328699999996,40.99696100000017],[-105.72644099999997,40.99683599999997],[-105.724829,40.99690600000008],[-105.69687799999997,40.99697400000002],[-105.68161999999995,40.99705499999993],[-105.62547399999994,40.997149999999976],[-105.58268599999985,40.997298],[-105.50054699999998,40.99759500000022],[-105.44942100000003,40.99768500000005],[-105.43391799999989,40.997751000000164],[-105.37554199999994,40.99788700000022],[-105.36354999999998,40.997874000000024],[-105.34141499999987,40.99797199999995],[-105.25053700000001,40.99825699999991],[-105.24882200000002,40.99844000000013],[-105.15885199999997,40.99831599999999],[-105.15505399999995,40.99824700000016],[-105.07701399999996,40.99833300000017],[-105.07480900000002,40.99829799999992],[-105.00052599999992,40.99827600000026],[-104.96257800000001,40.9982030000001],[-104.94337100000001,40.99821300000019]]]},"properties":{"OBJECTID":36,"COUNTYFP":"069","Shape_Leng":3.89982359156,"Shape_Area":0.72653049692,"low":83810,"lowmod":134340,"LMMI":194105,"lowmoduniv":309125,"lowmod_pct":0.434581479984,"NAMELSAD10":"Larimer County"}},
{"type":"Feature","geometry":{"type":"Polygon","coordinates":[[[-103.40433899999994,37.64357600000005],[-103.38676999999996,37.64351499999998],[-103.375493,37.64353500000004],[-103.36126000000002,37.64333499999998],[-103.34180400000002,37.643513000000155],[-103.32690899999989,37.64376800000014],[-103.30813599999993,37.64393999999993],[-103.29057899999998,37.64385100000004],[-103.2675999999999,37.64414500000004],[-103.25882199999995,37.643980000000056],[-103.22204099999993,37.64401900000007],[-103.20549099999988,37.64408600000013],[-103.18749200000002,37.643669999999986],[-103.17517299999986,37.64351499999998],[-103.16323299999993,37.643599000000165],[-103.14667400000002,37.6435920000003],[-103.13148799999988,37.64366200000006],[-103.12548500000003,37.64358100000027],[-103.11764099999999,37.64329900000001],[-103.114734,37.64341500000006],[-103.07593799999995,37.643420000000276],[-103.07574199999999,37.62501099999997],[-103.075581,37.624601000000155],[-103.07538299999987,37.55663200000015],[-103.07522099999989,37.50158900000008],[-103.07527499999998,37.500037000000134],[-103.07514100000003,37.48448900000028],[-103.07515699999999,37.460738000000106],[-103.07500099999987,37.45574899999997],[-103.07514200000003,37.45094300000022],[-103.07502099999994,37.436776000000066],[-103.075243,37.415738999999974],[-103.07480499999991,37.398437000000115],[-103.07466299999999,37.38175400000017],[-103.082537,37.38161300000007],[-103.08278200000001,37.3663360000001],[-103.08276599999994,37.35839900000008],[-103.0830509999999,37.35173900000024],[-103.08302499999996,37.346745000000055],[-103.08335999999997,37.33719100000019],[-103.08323199999995,37.32270900000009],[-103.08367299999992,37.30806799999999],[-103.08352699999995,37.29429100000016],[-103.083777,37.28439300000008],[-103.08510699999994,37.236590999999976],[-103.08607599999999,37.200359000000276],[-103.08664299999992,37.180689000000086],[-103.08655799999991,37.176128000000006],[-103.08632,37.14342599999998],[-103.0869689999999,37.13508700000011],[-103.08727599999997,37.102678000000026],[-103.08653400000003,37.03905900000001],[-103.08610599999997,37.00026399999996],[-103.12328400000001,37.000293000000056],[-103.12548900000002,37.000174000000015],[-103.15592200000003,37.00023199999998],[-103.156002,37.00001900000012],[-103.17321900000002,37.00001900000012],[-103.17752299999995,36.9999480000003],[-103.22644700000001,36.999694999999974],[-103.25230599999986,36.999572000000285],[-103.33673599999992,36.99955600000021],[-103.349133,36.99949900000013],[-103.39270699999986,36.999396000000104],[-103.40623800000003,36.99944399999998],[-103.46254399999998,36.99925600000006],[-103.49996799999997,36.99918500000024],[-103.54977699999995,36.99884400000008],[-103.5990789999999,36.99862600000017],[-103.62550699999997,36.99862900000005],[-103.67427299999997,36.99833999999993],[-103.68010699999996,36.99870699999997],[-103.68574100000001,36.99884500000002],[-103.70171199999993,36.998202000000106],[-103.72085400000003,36.99813500000022],[-103.74052599999993,36.99796800000007],[-103.7505119999999,36.99797500000011],[-103.76800200000002,36.99776300000002],[-103.794894,36.99759000000006],[-103.81780299999997,36.99733900000007],[-103.84064299999989,36.99731300000025],[-103.868946,36.997225999999955],[-103.8920369999999,36.99686300000013],[-103.90481999999986,36.99677900000029],[-103.97393599999998,36.99649500000004],[-104.00036499999987,36.996218000000056],[-104.007855,36.996239],[-104.04033199999998,36.99600700000019],[-104.12517600000001,36.995411000000104],[-104.12552999999991,36.995355000000245],[-104.16285299999993,36.99518500000022],[-104.21563399999985,36.99475400000006],[-104.2189019999999,36.994843],[-104.25053600000001,36.99464400000022],[-104.29969800000003,36.99406900000008],[-104.32197100000002,36.993887000000086],[-104.33883300000002,36.99353500000029],[-104.36891300000002,36.993446000000006],[-104.375538,36.993524000000264],[-104.40831300000002,36.99347400000022],[-104.41352799999993,36.9935940000002],[-104.42982099999995,36.99375100000003],[-104.45054399999987,36.99372100000022],[-104.46197699999993,36.99383400000016],[-104.486985,36.99386700000002],[-104.49227200000001,36.99365799999998],[-104.49339499999996,36.99387700000017],[-104.51919499999991,36.99376600000022],[-104.53704800000003,36.99397600000003],[-104.56327299999998,36.993823000000134],[-104.56991399999987,36.99387700000017],[-104.57475399999993,36.994118000000014],[-104.58952199999999,36.994092000000194],[-104.61481600000002,36.994129000000044],[-104.62455499999987,36.99437500000016],[-104.62554399999999,36.99359800000002],[-104.63144799999998,36.993592000000206],[-104.64499699999999,36.993379000000175],[-104.70745399999993,36.99349700000016],[-104.73222499999991,36.993446000000006],[-104.75054699999987,36.99352499999998],[-104.78846199999981,36.99345500000004],[-104.84360399999997,36.99335500000012],[-104.87507499999987,36.99329800000004],[-104.87554999999998,36.99325300000004],[-104.90258699999998,36.993307999999956],[-104.93013999999977,36.99329499999999],[-105.00055099999997,36.99326200000007],[-105.02922599999994,36.99272700000023],[-105.05378300000001,36.99370899999997],[-105.06645999999989,36.993868000000305],[-105.08540999999991,36.99446100000017],[-105.12080000000003,36.99542800000006],[-105.1255569999999,36.99521199999998],[-105.13644699999998,36.995168999999976],[-105.15504199999992,36.995262000000025],[-105.15506099999999,37.000023999999996],[-105.15490799999998,37.02206500000011],[-105.15495499999986,37.07718600000021],[-105.154988,37.11112000000003],[-105.15506299999998,37.12501900000001],[-105.15493399999997,37.16296200000011],[-105.154898,37.164156000000105],[-105.15468799999996,37.21413899999999],[-105.15458299999989,37.25003200000009],[-105.15417599999995,37.29312900000008],[-105.15200500000003,37.29300000000006],[-105.15049899999991,37.293785000000014],[-105.14848699999999,37.29609100000022],[-105.14595999999989,37.29582600000015],[-105.14304199999987,37.2959350000001],[-105.13992499999989,37.29706299999998],[-105.13783899999987,37.298223000000235],[-105.13440700000001,37.29919100000001],[-105.1315909999999,37.301114000000155],[-105.13019199999997,37.29789599999998],[-105.12808000000001,37.296822000000304],[-105.11989799999992,37.29599700000006],[-105.11542500000002,37.29629399999999],[-105.11259100000001,37.29841400000004],[-105.11052799999993,37.299507999999946],[-105.10395499999998,37.30131100000011],[-105.09797100000003,37.304151000000275],[-105.09700900000001,37.30586099999999],[-105.09485599999988,37.30822100000023],[-105.09481800000003,37.309496000000195],[-105.09617700000001,37.311816000000306],[-105.09424799999994,37.314703000000065],[-105.09215999999992,37.31535200000013],[-105.088348,37.315106000000014],[-105.08553099999989,37.31525600000003],[-105.08419300000003,37.31495300000017],[-105.08155599999998,37.31579899999997],[-105.07766100000003,37.316545000000076],[-105.07512599999995,37.31666500000006],[-105.07409799999999,37.31732499999998],[-105.07371899999998,37.31976600000013],[-105.07146699999981,37.32197200000007],[-105.06804599999992,37.32604599999996],[-105.06101799999999,37.328283],[-105.05626099999995,37.329386000000284],[-105.05445099999986,37.33028799999994],[-105.05059399999993,37.331049000000064],[-105.046945,37.33344199999999],[-105.04522399999996,37.335789000000034],[-105.04363299999989,37.3362420000002],[-105.04062799999986,37.33807600000023],[-105.03968899999995,37.339454000000046],[-105.03787999999986,37.34027600000019],[-105.03641500000003,37.341657000000225],[-105.03382899999997,37.34038099999998],[-105.03025999999988,37.33947400000011],[-105.02771699999988,37.339416000000256],[-105.02559899999989,37.33996800000028],[-105.02347399999991,37.341649000000075],[-105.02348599999993,37.34350499999999],[-105.02452999999997,37.34719499999994],[-105.02476000000001,37.34998100000007],[-105.02358900000002,37.35460100000006],[-105.02368499999989,37.35626400000018],[-105.021454,37.35624100000024],[-105.01767599999988,37.35701800000004],[-105.01614099999989,37.35835800000007],[-105.01285899999993,37.35839000000004],[-105.01061099999993,37.35948000000019],[-105.00872400000003,37.361333],[-105.00663299999997,37.36269200000004],[-105.00478799999996,37.36317300000002],[-105.00368700000001,37.36580000000015],[-105.00227799999993,37.366859000000034],[-105.00122999999991,37.368551000000025],[-104.99550499999998,37.375550999999916],[-104.99473599999988,37.37582900000001],[-104.98990100000003,37.37596800000017],[-104.98833499999995,37.37555400000019],[-104.98681299999998,37.37432000000024],[-104.98352699999987,37.373388000000205],[-104.98003299999988,37.37424100000027],[-104.97686899999991,37.37642900000009],[-104.97578199999987,37.37674700000031],[-104.971228,37.37624299999999],[-104.96928500000001,37.37778300000008],[-104.96703300000001,37.37872300000021],[-104.96243399999997,37.38191400000011],[-104.9588819999999,37.38498900000013],[-104.9567459999999,37.38737100000003],[-104.95263099999988,37.389715000000194],[-104.94831899999997,37.39068400000025],[-104.94673599999999,37.39144600000009],[-104.944906,37.390499000000034],[-104.94062899999989,37.38925000000006],[-104.93886099999997,37.38920900000005],[-104.93715199999997,37.38997199999994],[-104.93527099999994,37.39248500000008],[-104.93255699999992,37.395412000000306],[-104.93055300000003,37.39618000000024],[-104.92560699999996,37.39705699999996],[-104.92504300000002,37.396259000000214],[-104.92218200000002,37.394318],[-104.91918099999987,37.393688999999995],[-104.91763000000003,37.39397900000006],[-104.91597899999988,37.393533000000104],[-104.91332799999998,37.39353700000004],[-104.91114900000002,37.39393700000011],[-104.90787499999999,37.392337999999995],[-104.90606100000002,37.39011300000027],[-104.90294899999992,37.390190000000075],[-104.90099799999996,37.38944500000008],[-104.89867900000002,37.38920900000005],[-104.89291799999995,37.389424000000076],[-104.88691099999994,37.387861999999984],[-104.88425999999998,37.38581700000026],[-104.87837000000002,37.38788000000005],[-104.87723199999999,37.387762000000066],[-104.87394799999987,37.38867800000003],[-104.872975,37.390405999999984],[-104.871644,37.391232000000116],[-104.868785,37.3916440000001],[-104.86633399999977,37.391377000000034],[-104.86497199999997,37.39079300000003],[-104.86225899999994,37.392042000000174],[-104.860658,37.39209900000009],[-104.85883699999988,37.39363800000024],[-104.85375699999992,37.394586000000004],[-104.849986,37.39568299999996],[-104.84720199999987,37.39569],[-104.84351899999996,37.39691900000008],[-104.84189699999996,37.397951000000035],[-104.83828699999998,37.39867500000008],[-104.83527499999997,37.39784499999996],[-104.83475499999986,37.39526500000022],[-104.83282700000001,37.395797000000016],[-104.83146899999991,37.39541300000002],[-104.82666799999998,37.39565099999999],[-104.82528499999995,37.39604000000003],[-104.82426299999986,37.397567000000095],[-104.82208299999985,37.398168999999996],[-104.81970099999995,37.39719600000029],[-104.81726200000003,37.397500000000036],[-104.81688400000002,37.399487000000306],[-104.81546900000001,37.40038100000021],[-104.81251500000002,37.40025600000001],[-104.81104599999998,37.39863000000008],[-104.80853899999994,37.39967399999995],[-104.80715999999995,37.402214000000015],[-104.803718,37.40293500000013],[-104.80062299999997,37.40025700000007],[-104.79910499999988,37.400245000000155],[-104.79756500000002,37.40117400000008],[-104.79187699999994,37.401388000000225],[-104.79076099999997,37.40049700000003],[-104.78742199999994,37.400107000000276],[-104.78200799999996,37.40129200000007],[-104.78073699999987,37.40302300000013],[-104.77931999999998,37.403782000000035],[-104.77683699999994,37.40408600000001],[-104.77459799999997,37.40388199999995],[-104.77346299999994,37.40604799999994],[-104.77186899999987,37.40647300000012],[-104.76993899999985,37.404920999999945],[-104.76758699999993,37.40539000000024],[-104.7607559999999,37.40545099999997],[-104.75842699999993,37.40699600000005],[-104.75628,37.40719400000012],[-104.75436400000001,37.40634699999998],[-104.75213899999994,37.40684800000025],[-104.74893799999995,37.40702900000031],[-104.748447,37.40743700000013],[-104.74942199999987,37.40972399999998],[-104.749075,37.410437],[-104.74955899999992,37.413277000000164],[-104.74723899999998,37.41571000000022],[-104.74315399999983,37.417887000000064],[-104.74242600000002,37.41979000000009],[-104.73997199999991,37.42166800000024],[-104.73711199999997,37.42300100000023],[-104.73679900000002,37.42494899999991],[-104.73340299999995,37.425367999999935],[-104.73221000000001,37.42665500000004],[-104.73058199999986,37.42724199999992],[-104.72679,37.42653599999994],[-104.72542399999998,37.42745800000023],[-104.72513600000002,37.429989000000205],[-104.7237649999999,37.42945700000024],[-104.72115499999995,37.42941600000023],[-104.72039799999993,37.431166000000246],[-104.71809899999988,37.431825999999944],[-104.7157749999999,37.434713999999985],[-104.71254399999998,37.436649999999986],[-104.71047399999986,37.437501000000054],[-104.70896900000002,37.436638000000016],[-104.705716,37.43772000000001],[-104.70237199999997,37.43815499999994],[-104.70190799999995,37.43882900000011],[-104.69830200000001,37.43988300000029],[-104.69689800000003,37.43894000000006],[-104.69507499999992,37.4399150000001],[-104.69561099999987,37.442188999999985],[-104.69726500000002,37.44481300000018],[-104.69835,37.44544599999995],[-104.69829199999992,37.44808499999999],[-104.70027800000003,37.45078500000011],[-104.70045599999992,37.45354900000001],[-104.69952999999992,37.45447500000006],[-104.69948299999987,37.45725900000002],[-104.69838599999997,37.45897000000002],[-104.69932499999993,37.460794000000305],[-104.69937699999997,37.46280100000013],[-104.69998599999991,37.46373999999997],[-104.69894899999991,37.464766],[-104.69965300000001,37.46594900000014],[-104.69888800000001,37.467933000000016],[-104.69993099999999,37.46983800000004],[-104.69723099999999,37.47180700000007],[-104.69621899999999,37.47350999999992],[-104.69575599999996,37.47545200000002],[-104.69401800000003,37.47578199999998],[-104.69176399999998,37.47903100000008],[-104.69344799999988,37.481846000000075],[-104.69482199999976,37.482123000000115],[-104.69614100000001,37.48434999999995],[-104.69611099999992,37.48542500000008],[-104.69184099999995,37.48662400000018],[-104.69199399999991,37.48775400000005],[-104.69093499999997,37.489239],[-104.69210900000002,37.49122699999998],[-104.6897019999999,37.49441899999999],[-104.68574000000001,37.495992000000115],[-104.681017,37.496026000000086],[-104.67015300000003,37.494987000000094],[-104.66647699999999,37.494792000000075],[-104.64922899999993,37.495320000000106],[-104.63291900000002,37.507204],[-104.62765299999995,37.510902000000044],[-104.62553799999984,37.51611900000006],[-104.620338,37.530053000000294],[-104.58584499999995,37.55152899999996],[-104.54865199999995,37.57470000000029],[-104.54841699999986,37.609424000000104],[-104.5373009999999,37.62501200000008],[-104.51428199999992,37.65788800000007],[-104.51169699999991,37.661280999999974],[-104.50053200000002,37.67674899999997],[-104.48313999999999,37.701188],[-104.45492999999993,37.740838999999994],[-104.45672299999995,37.74788000000029],[-104.45311900000002,37.75001100000003],[-104.41587599999997,37.773325000000284],[-104.37552699999986,37.79858799999994],[-104.372863,37.80040500000001],[-104.35110899999995,37.817488000000196],[-104.312678,37.80659700000007],[-104.25052499999993,37.78898500000014],[-104.24790599999994,37.7881560000003],[-104.16939299999996,37.765795000000026],[-104.12614899999988,37.75347099999999],[-104.12476300000003,37.75317100000001],[-104.10308499999996,37.74682199999995],[-104.06113199999987,37.73470400000025],[-104.05929000000003,37.734172000000285],[-104.05871699999989,37.688198000000284],[-104.05819599999995,37.64404900000011],[-104.03283299999998,37.64402900000022],[-104.000516,37.64405999999991],[-103.99206700000002,37.64385800000008],[-103.9536829999999,37.64405700000003],[-103.875498,37.64445500000011],[-103.857124,37.64482600000025],[-103.85524299999997,37.644738000000075],[-103.83338300000003,37.64477299999993],[-103.78465699999987,37.64413000000002],[-103.75081299999994,37.643688999999995],[-103.75050699999986,37.643557999999985],[-103.67808500000001,37.64385199999998],[-103.62550199999998,37.64397300000002],[-103.56227899999982,37.643538999999976],[-103.55498899999998,37.64344100000005],[-103.51265799999987,37.64312200000012],[-103.50049799999988,37.64293600000019],[-103.485323,37.643502000000126],[-103.451799,37.64378799999997],[-103.42708799999997,37.643658000000244],[-103.40433899999994,37.64357600000005]]]},"properties":{"OBJECTID":37,"COUNTYFP":"071","Shape_Leng":5.53786029629,"Shape_Area":1.25722755835,"low":3820,"lowmod":5880,"LMMI":9335,"lowmoduniv":13775,"lowmod_pct":0.426860254083,"NAMELSAD10":"Las Animas County"}},
{"type":"Feature","geometry":{"type":"Polygon","coordinates":[[[-103.15437600000001,39.565654000000166],[-103.154586,39.51047799999992],[-103.15475600000002,39.49106500000005],[-103.15465899999998,39.4668710000002],[-103.15482999999995,39.43297100000001],[-103.15490199999994,39.40398600000003],[-103.15523599999995,39.38897000000014],[-103.15511799999996,39.388499999999965],[-103.15514599999989,39.3777970000001],[-103.15498499999995,39.367982000000154],[-103.15507099999996,39.33433600000018],[-103.155009,39.33138300000007],[-103.1551839999999,39.288686000000155],[-103.15519299999994,39.27312800000004],[-103.15511999999995,39.25820500000003],[-103.15515199999993,39.2439500000001],[-103.15492699999987,39.22420300000027],[-103.15512999999999,39.16701699999999],[-103.15508899999998,39.13347999999996],[-103.15525300000002,39.12798399999997],[-103.16307199999994,39.127991000000065],[-103.1630199999999,39.12499200000025],[-103.16317300000003,39.11026000000004],[-103.16306199999991,39.10630900000018],[-103.163209,39.09557000000018],[-103.16303699999997,39.085113000000035],[-103.16318499999988,39.08060600000016],[-103.16312099999999,39.05591900000019],[-103.16302499999995,39.03760999999997],[-103.16298699999993,39.00814900000006],[-103.16308500000002,38.99999399999996],[-103.163253,38.935704999999984],[-103.16335599999991,38.927080000000274],[-103.16345999999999,38.87496100000021],[-103.163745,38.86492700000008],[-103.16373099999993,38.8495640000001],[-103.163636,38.84595300000029],[-103.16370799999999,38.83666900000003],[-103.163835,38.83524299999999],[-103.16380300000003,38.79136200000016],[-103.16378499999996,38.749998000000005],[-103.16406099999989,38.72555600000004],[-103.16415899999998,38.69013300000012],[-103.17247399999991,38.69005700000008],[-103.17261199999996,38.62891600000029],[-103.17294299999998,38.612450000000024],[-103.17304899999999,38.58323700000017],[-103.17304200000001,38.56489799999997],[-103.17287799999986,38.52531499999992],[-103.19109100000003,38.52509399999997],[-103.19341500000002,38.524999000000264],[-103.22779299999996,38.524372000000255],[-103.23796900000002,38.524043000000006],[-103.246127,38.52392200000014],[-103.25056099999989,38.523657999999955],[-103.28255499999995,38.52364299999999],[-103.30348500000002,38.52359000000001],[-103.35242,38.52370000000025],[-103.37130899999994,38.52341700000028],[-103.38193099999995,38.52332700000011],[-103.42662299999995,38.52348500000022],[-103.45541499999985,38.52381700000001],[-103.462785,38.52367800000002],[-103.49244499999998,38.523893000000044],[-103.50467399999997,38.52366400000011],[-103.50466599999993,38.51640900000024],[-103.56753099999997,38.51726100000002],[-103.61298299999999,38.51787899999999],[-103.61298999999991,38.52254800000014],[-103.64861399999995,38.52261699999997],[-103.67791799999986,38.522210000000086],[-103.68208600000003,38.52243199999998],[-103.70572499999997,38.52224400000023],[-103.70652799999988,38.521939999999915],[-103.71345599999995,38.521817000000055],[-103.71547599999997,38.52156800000006],[-103.71902899999992,38.52179600000028],[-103.73354199999994,38.52193300000005],[-103.75088800000003,38.522231000000204],[-103.76080699999989,38.52210900000006],[-103.77002899999997,38.52215200000006],[-103.77915300000001,38.52177100000017],[-103.85170699999998,38.52202100000005],[-103.87260899999995,38.52211700000004],[-103.87550799999991,38.5227720000002],[-103.88614999999999,38.5226330000001],[-103.89643999999998,38.52289500000006],[-103.90625599999981,38.52289400000018],[-103.91051599999986,38.52332999999999],[-103.91913199999999,38.52311300000002],[-103.93145199999992,38.5231740000001],[-103.96225099999987,38.523212],[-103.96255899999994,38.52313700000002],[-103.98503800000003,38.52277500000008],[-104.0172849999999,38.522594000000026],[-104.03446999999989,38.52230800000018],[-104.03722800000003,38.52247899999992],[-104.041339,38.52234200000015],[-104.05392099999989,38.522393000000136],[-104.054127,38.523086999999975],[-104.05426299999988,38.533718000000135],[-104.05430799999993,38.54515400000025],[-104.05421100000001,38.55280300000021],[-104.05427499999996,38.57362200000006],[-104.05403099999995,38.580183999999974],[-104.05423299999995,38.59259000000026],[-104.05438300000003,38.59530400000028],[-104.05423399999995,38.615195000000085],[-104.05409299999991,38.62497600000006],[-104.05408299999999,38.638667999999996],[-104.05372799999992,38.65366],[-104.05333999999988,38.68163700000025],[-104.05298199999993,38.69434100000012],[-104.05194199999994,38.69434700000028],[-104.0524749999999,38.709235000000206],[-104.05317100000002,38.73110699999995],[-104.05328499999996,38.738332000000014],[-104.05397600000003,38.759877000000074],[-104.05428999999987,38.77647500000012],[-104.0541389999999,38.778841000000114],[-104.05450300000001,38.78266899999994],[-104.054825,38.81115100000005],[-104.05526999999995,38.85270700000012],[-104.05552799999998,38.86886800000002],[-104.00829499999998,38.86877000000027],[-103.98137099999985,38.8687520000002],[-103.98071199999993,38.8686910000003],[-103.940473,38.86822599999999],[-103.87881299999998,38.86751499999997],[-103.85202899999996,38.866919999999936],[-103.83860500000003,38.8669670000001],[-103.77709400000003,38.866729000000134],[-103.76509199999987,38.86646300000001],[-103.75746800000002,38.86655900000011],[-103.75038799999987,38.86643600000025],[-103.7196639999998,38.86682700000006],[-103.71941799999996,38.87499400000007],[-103.72019299999988,38.905647000000044],[-103.7206119999999,38.93243800000022],[-103.72134499999999,38.98657700000007],[-103.72142299999996,38.999990000000025],[-103.72093899999987,39.011095000000296],[-103.72089399999987,39.042463999999995],[-103.7209519999999,39.05517999999995],[-103.72178699999995,39.079755999999975],[-103.72200699999996,39.08408500000024],[-103.72082399999994,39.09874200000007],[-103.72074999999995,39.11093000000005],[-103.720801,39.11989199999999],[-103.7211519999999,39.12830700000006],[-103.71525399999996,39.12824599999999],[-103.71534099999997,39.135289000000284],[-103.71528499999994,39.15763600000008],[-103.7150769999999,39.171722999999986],[-103.71511599999991,39.17814900000002],[-103.71491999999995,39.179022000000145],[-103.71517099999994,39.20136499999995],[-103.7150739999999,39.22974200000027],[-103.71489400000002,39.244187000000124],[-103.71504599999997,39.24998700000009],[-103.71488899999997,39.27739200000002],[-103.71489200000002,39.28515399999992],[-103.71545799999996,39.3023400000003],[-103.7150769999999,39.30988500000001],[-103.71479599999998,39.344540000000165],[-103.71484900000002,39.37388300000026],[-103.71525199999996,39.374986999999976],[-103.71528799999993,39.39463800000016],[-103.71520199999992,39.418088000000125],[-103.714765,39.46201200000013],[-103.71449699999994,39.483903000000055],[-103.71449699999994,39.490616000000216],[-103.71429099999995,39.49999000000008],[-103.71439700000002,39.53393999999997],[-103.71411699999993,39.548748000000046],[-103.71435500000001,39.56633100000022],[-103.70707199999987,39.56629500000008],[-103.699117,39.56625800000006],[-103.68385899999998,39.56635100000028],[-103.63777299999992,39.566222000000266],[-103.63510100000002,39.56576000000007],[-103.63181999999989,39.56575900000013],[-103.62932699999993,39.566220999999985],[-103.59843299999983,39.56612300000006],[-103.58281199999982,39.566134999999974],[-103.56378099999995,39.56606900000003],[-103.52961899999991,39.566102000000285],[-103.492032,39.56599100000017],[-103.48961600000001,39.565742],[-103.48107399999998,39.565996999999925],[-103.47213099999988,39.56575200000009],[-103.469336,39.56590500000016],[-103.45351499999987,39.565885000000264],[-103.44574499999982,39.56598100000002],[-103.42805699999997,39.56598800000006],[-103.38749999999987,39.566174000000046],[-103.34994699999993,39.56630800000028],[-103.34817900000002,39.566396999999995],[-103.31938099999996,39.56641400000001],[-103.31815699999987,39.56655300000011],[-103.263576,39.56644100000011],[-103.21196899999995,39.5661750000001],[-103.18206799999996,39.56614200000001],[-103.15495499999997,39.565949000000046],[-103.15437600000001,39.565654000000166]]]},"properties":{"OBJECTID":38,"COUNTYFP":"073","Shape_Leng":3.90096955163,"Shape_Area":0.696476726637,"low":1135,"lowmod":1820,"LMMI":2470,"lowmoduniv":3760,"lowmod_pct":0.484042553191,"NAMELSAD10":"Lincoln County"}},
{"type":"Feature","geometry":{"type":"Polygon","coordinates":[[[-102.65346299999999,41.00233200000014],[-102.65348799999992,40.999989000000085],[-102.65306499999991,40.96242800000016],[-102.65259199999997,40.920300000000225],[-102.652422,40.918791000000056],[-102.652264,40.909647000000064],[-102.65210200000001,40.88976100000008],[-102.65211099999993,40.88325200000014],[-102.65181399999989,40.86544500000008],[-102.6514479999999,40.79295900000011],[-102.65130999999985,40.74940200000003],[-102.651095,40.730995000000064],[-102.65087999999997,40.70533999999992],[-102.6646889999999,40.705189000000246],[-102.6647309999999,40.686059],[-102.66510399999993,40.6445750000002],[-102.66557599999993,40.613394000000255],[-102.66550000000001,40.58410700000002],[-102.66516199999995,40.558742999999936],[-102.66504700000002,40.52177599999999],[-102.66513700000002,40.512644999999964],[-102.66499999999996,40.487532999999985],[-102.665007,40.47288500000002],[-102.66486499999996,40.45673699999992],[-102.664804,40.43847300000027],[-102.682636,40.438500000000204],[-102.70167000000004,40.438646000000006],[-102.70242899999994,40.438541000000214],[-102.74855300000002,40.43853900000005],[-102.76274699999999,40.43858900000009],[-102.77996000000002,40.43844600000017],[-102.79391599999985,40.438507999999956],[-102.80605699999995,40.438481000000024],[-102.84905200000003,40.43827799999997],[-102.88262600000002,40.43831499999999],[-102.89315699999997,40.43827700000003],[-102.94897900000001,40.43787400000008],[-102.95083999999997,40.43777100000028],[-102.97363599999989,40.437656000000175],[-102.98660299999995,40.43751600000013],[-103.00851999999998,40.437432000000115],[-103.062252,40.43694099999999],[-103.08322099999992,40.43679500000002],[-103.09031499999998,40.43684500000006],[-103.13211000000001,40.43661800000007],[-103.13547800000003,40.436779000000115],[-103.15034000000003,40.43678999999992],[-103.15110800000002,40.436710999999946],[-103.1913429999999,40.436458000000016],[-103.22641299999992,40.4361780000001],[-103.22655800000001,40.436087000000214],[-103.25049199999995,40.43603700000017],[-103.32150299999995,40.43558100000007],[-103.36720099999997,40.43529399999994],[-103.37496199999987,40.4352980000001],[-103.375496,40.435471000000234],[-103.38706499999995,40.43543799999998],[-103.38766399999986,40.43557000000004],[-103.40704999999997,40.436360000000036],[-103.46537699999999,40.43634600000013],[-103.46524699999998,40.45750300000026],[-103.46589999999992,40.45842900000014],[-103.46606199999991,40.46542700000009],[-103.4658859999999,40.47259000000014],[-103.46630800000003,40.479820000000075],[-103.46656699999994,40.49430000000018],[-103.46719999999999,40.52325300000018],[-103.49086599999998,40.52317700000009],[-103.49299699999989,40.52309800000012],[-103.50271699999996,40.52328600000004],[-103.543181,40.52339400000011],[-103.58157299999999,40.52318400000013],[-103.58184399999993,40.52338300000008],[-103.58184799999992,40.53787200000005],[-103.58171299999992,40.56698600000021],[-103.58161999999987,40.595976000000064],[-103.58151999999995,40.60438599999992],[-103.58160799999996,40.61376900000005],[-103.58139999999997,40.61594400000001],[-103.58151099999992,40.62498300000027],[-103.58135799999997,40.63074999999992],[-103.58102199999996,40.65403999999995],[-103.58049,40.68313400000005],[-103.58037100000001,40.70202300000011],[-103.57430099999988,40.70201800000024],[-103.57400799999988,40.731036000000074],[-103.57326499999994,40.78191500000008],[-103.57321599999989,40.78916600000031],[-103.57337100000001,40.83280200000013],[-103.57354399999986,40.84726500000011],[-103.57360899999998,40.861834999999985],[-103.57396599999976,40.876187000000186],[-103.57375000000002,40.88363600000014],[-103.57372099999992,40.89305400000006],[-103.573489,40.8985780000001],[-103.57369899999998,40.90188699999993],[-103.57367099999988,40.91721000000018],[-103.57382099999995,40.92305399999998],[-103.57356899999996,40.925208000000055],[-103.57376399999993,40.94809299999997],[-103.57394799999997,40.963296000000014],[-103.57333699999987,40.978533000000255],[-103.57344599999999,40.99267800000018],[-103.57377399999996,41.00171599999993],[-103.53488600000003,41.00142100000005],[-103.50046999999995,41.001734],[-103.48669699999999,41.00191400000017],[-103.4479429999999,41.00176000000022],[-103.428676,41.00197000000003],[-103.41339499999998,41.001912000000004],[-103.39698900000002,41.002557999999965],[-103.38249200000001,41.002252000000055],[-103.37548699999996,41.00229900000005],[-103.36531399999996,41.00184600000023],[-103.32387099999994,41.00182000000001],[-103.26827899999995,41.002080000000035],[-103.25048300000003,41.00177600000012],[-103.24900600000001,41.00220999999999],[-103.2362829999999,41.00202400000006],[-103.23007699999994,41.00223799999998],[-103.21941499999991,41.00201200000009],[-103.21074099999987,41.00224700000001],[-103.19179799999989,41.002072999999996],[-103.17243699999995,41.00216800000004],[-103.16526499999992,41.002084000000025],[-103.143641,41.002312000000245],[-103.12548900000002,41.00215800000029],[-103.11500399999994,41.002342000000056],[-103.07780300000002,41.002297999999996],[-103.07678699999997,41.00222700000012],[-103.05962599999998,41.00237000000004],[-103.04344400000002,41.00234400000022],[-103.03870399999994,41.00225100000017],[-103.01953000000003,41.00244700000002],[-103.0020879999999,41.00248600000009],[-103.00096699999995,41.002312000000245],[-102.98271399999987,41.00227200000012],[-102.98148199999997,41.00211200000001],[-102.96366899999998,41.00218600000011],[-102.95510599999989,41.002159000000006],[-102.94482899999997,41.00230300000021],[-102.94315899999992,41.002050000000054],[-102.92508899999996,41.00228300000015],[-102.92397299999999,41.00214200000005],[-102.90635800000001,41.00231300000013],[-102.90485999999999,41.0021990000003],[-102.88728099999997,41.00231400000007],[-102.88558999999987,41.00213400000007],[-102.87555099999986,41.00233300000002],[-102.866083,41.00231700000012],[-102.86576999999988,41.00198899999998],[-102.84654299999994,41.00241100000011],[-102.84379899999999,41.002271000000064],[-102.83030299999996,41.002350000000035],[-102.82723699999985,41.0021430000001],[-102.77356099999992,41.00241900000003],[-102.76660600000002,41.00227600000028],[-102.75458599999996,41.002360000000124],[-102.743133,41.002211999999986],[-102.70865900000001,41.00219500000014],[-102.68791099999999,41.00233400000013],[-102.65346299999999,41.00233200000014]]]},"properties":{"OBJECTID":39,"COUNTYFP":"075","Shape_Leng":2.98706397772,"Shape_Area":0.509316372898,"low":5825,"lowmod":9695,"LMMI":14010,"lowmoduniv":20795,"lowmod_pct":0.466217840827,"NAMELSAD10":"Logan County"}},
{"type":"Feature","geometry":{"type":"Polygon","coordinates":[[[-107.43094899999977,39.366178000000104],[-107.4298169999999,39.36400599999996],[-107.42845799999992,39.362480000000005],[-107.42910499999988,39.359883999999965],[-107.42877799999991,39.35743600000012],[-107.42913399999998,39.355383000000074],[-107.43031799999989,39.354281000000015],[-107.43016599999993,39.349597000000244],[-107.43051400000002,39.348221999999964],[-107.43227100000001,39.347210000000075],[-107.435589,39.34694000000013],[-107.43676199999999,39.34638599999994],[-107.43887299999994,39.34328199999999],[-107.43853899999993,39.34101899999996],[-107.4415699999999,39.33991300000031],[-107.44341299999996,39.33850500000017],[-107.44551100000001,39.33526299999994],[-107.449998,39.33266400000002],[-107.45081899999991,39.33003400000001],[-107.45347500000003,39.32884300000006],[-107.45545799999996,39.329393000000096],[-107.46188999999976,39.327570000000264],[-107.463414,39.32427000000007],[-107.465957,39.322119999999984],[-107.46411699999999,39.32034300000004],[-107.46412700000002,39.31874499999998],[-107.46323899999993,39.31685200000021],[-107.46505599999989,39.31474200000025],[-107.46269199999995,39.31379300000009],[-107.46101899999991,39.312617999999986],[-107.45922899999994,39.309686000000056],[-107.45693699999993,39.30923599999994],[-107.456456,39.30692700000003],[-107.454498,39.306545000000085],[-107.45166699999999,39.30452799999995],[-107.44965499999995,39.30455900000004],[-107.44855499999989,39.30567600000029],[-107.44720799999999,39.30578800000012],[-107.44527800000003,39.308520000000044],[-107.44023199999975,39.308430000000044],[-107.43880000000001,39.30901899999992],[-107.43766499999992,39.30667600000004],[-107.43803799999995,39.305034999999975],[-107.43973399999976,39.30249100000003],[-107.43787999999995,39.30100400000026],[-107.43747899999988,39.29956399999992],[-107.43349499999994,39.29852699999992],[-107.43116899999995,39.29900500000008],[-107.42116699999991,39.29685600000005],[-107.41857600000003,39.29483499999998],[-107.414987,39.29378600000024],[-107.41260599999998,39.291877000000056],[-107.40855899999997,39.294098000000304],[-107.40677599999987,39.29303700000008],[-107.4038349999999,39.29278600000009],[-107.39851299999992,39.29511000000019],[-107.39665400000001,39.293912000000034],[-107.39768399999997,39.29182000000009],[-107.39614799999998,39.289513],[-107.3963389999999,39.28637300000008],[-107.39696100000003,39.28356000000002],[-107.39565199999998,39.28230700000029],[-107.393958,39.28159200000027],[-107.39407299999999,39.27938799999998],[-107.39017399999989,39.278921999999966],[-107.387157,39.27788099999998],[-107.383285,39.27550199999996],[-107.38153599999998,39.27512599999994],[-107.37983500000001,39.275739999999985],[-107.378376,39.27480400000013],[-107.37747899999994,39.27220800000009],[-107.37917500000003,39.26910100000009],[-107.38094299999995,39.2682030000002],[-107.38351799999992,39.26777300000015],[-107.38712299999986,39.265815000000146],[-107.39146499999993,39.263881000000026],[-107.391708,39.26016800000008],[-107.39369899999997,39.25902600000029],[-107.39448599999997,39.25629900000007],[-107.39791600000001,39.25373000000013],[-107.40029699999991,39.25294600000029],[-107.403279,39.253850000000114],[-107.40716999999978,39.25341200000014],[-107.40931599999999,39.25444600000003],[-107.41150099999993,39.254277000000286],[-107.4148229999999,39.25559599999997],[-107.41814599999998,39.25584200000009],[-107.41968199999997,39.255334000000175],[-107.42194699999988,39.25557500000002],[-107.42440199999987,39.25649200000004],[-107.42438099999998,39.257596999999976],[-107.42691399999995,39.25772999999998],[-107.4295839999998,39.25728600000002],[-107.430521,39.25650500000006],[-107.42988700000001,39.255326000000025],[-107.43010400000003,39.25254100000018],[-107.42940299999987,39.250588999999934],[-107.43041599999992,39.248999000000026],[-107.43054399999994,39.24664300000012],[-107.42920000000004,39.24223000000006],[-107.43026099999997,39.24054100000012],[-107.43187699999982,39.23992000000004],[-107.43581099999989,39.23914600000012],[-107.43672900000001,39.238297000000046],[-107.43951500000003,39.23694599999999],[-107.44284199999998,39.23646800000006],[-107.44374699999997,39.23478100000028],[-107.44639499999988,39.233262000000195],[-107.44978499999996,39.23225100000002],[-107.45107499999995,39.230673000000024],[-107.45442300000002,39.23024299999997],[-107.45778599999994,39.22940200000005],[-107.4607729999999,39.230313999999964],[-107.46348599999993,39.22841399999999],[-107.46571599999987,39.228649000000246],[-107.46903499999996,39.22835599999996],[-107.47131699999994,39.2292030000001],[-107.47470099999998,39.22904600000004],[-107.47743699999995,39.230460000000164],[-107.47874100000001,39.23355099999992],[-107.48050799999987,39.23389499999996],[-107.48287700000003,39.2356200000001],[-107.48439299999995,39.23521699999992],[-107.48503499999987,39.23213700000019],[-107.48576699999995,39.23130700000007],[-107.48581299999995,39.22911300000027],[-107.48775299999988,39.22560900000025],[-107.492819,39.221466000000305],[-107.49454899999989,39.21842300000003],[-107.49755599999997,39.21746300000001],[-107.500606,39.217916],[-107.50240600000001,39.21739999999994],[-107.5047899999999,39.21733900000004],[-107.50640299999998,39.21645799999999],[-107.50729599999994,39.21682600000025],[-107.50983499999978,39.21655500000003],[-107.51179499999995,39.21567500000003],[-107.51383199999992,39.214],[-107.51664199999999,39.210327000000234],[-107.52067799999998,39.20628400000004],[-107.52367299999997,39.205516999999986],[-107.52448900000002,39.20468799999992],[-107.52750499999996,39.203884000000016],[-107.52883799999995,39.202936000000136],[-107.528348,39.20188400000029],[-107.52607099999989,39.200708000000134],[-107.524768,39.198916000000054],[-107.52642900000001,39.198717999999985],[-107.53116599999993,39.19910200000015],[-107.53482099999991,39.19867099999999],[-107.53758399999998,39.19789900000006],[-107.54157599999996,39.19623500000006],[-107.54345499999994,39.1944010000002],[-107.54287599999998,39.190522000000044],[-107.54377699999992,39.18693500000029],[-107.54339700000003,39.18541099999993],[-107.54171199999996,39.184565000000134],[-107.54235099999988,39.18272200000001],[-107.54336399999994,39.18192499999998],[-107.54895099999993,39.18116400000008],[-107.55215099999992,39.179688],[-107.55692899999985,39.17834000000005],[-107.56082099999998,39.17842600000006],[-107.56389399999989,39.18016],[-107.56742300000002,39.181397000000004],[-107.56983100000002,39.180146000000036],[-107.570491,39.17823100000004],[-107.572992,39.176347000000135],[-107.57702499999994,39.1748970000001],[-107.58128099999999,39.174504999999954],[-107.58362599999987,39.17373300000003],[-107.58473499999997,39.17178899999993],[-107.5884559999999,39.169112000000155],[-107.58895999999987,39.16833500000001],[-107.59281599999997,39.16585299999997],[-107.59387700000002,39.164820000000304],[-107.59585599999997,39.16480300000012],[-107.59805599999993,39.16422200000005],[-107.60102699999999,39.16412000000014],[-107.605325,39.16206],[-107.60745999999995,39.16157100000004],[-107.60976699999998,39.15887900000007],[-107.61105599999996,39.15649900000017],[-107.61467499999998,39.15453500000001],[-107.61694499999987,39.155529],[-107.61910799999998,39.15488600000009],[-107.61994800000002,39.15384000000006],[-107.62012899999996,39.152169000000185],[-107.6212789999999,39.15103500000015],[-107.62455699999998,39.150204000000144],[-107.62624399999993,39.149184000000105],[-107.62784499999998,39.14900399999999],[-107.63010300000002,39.1465750000001],[-107.63081199999999,39.14384699999994],[-107.6318,39.141797],[-107.63306699999987,39.140715],[-107.634726,39.14085699999998],[-107.63789300000002,39.142020000000116],[-107.64004499999999,39.140458000000024],[-107.64140199999991,39.138094000000194],[-107.64453699999996,39.13559900000001],[-107.64820099999997,39.13121800000005],[-107.6513139999999,39.13093099999992],[-107.65362499999998,39.12656300000003],[-107.65376000000003,39.12400200000002],[-107.6566029999999,39.12269399999997],[-107.65844899999996,39.11982300000011],[-107.66173399999997,39.11767500000002],[-107.66267399999987,39.116239000000235],[-107.66236699999996,39.114823000000115],[-107.66030899999993,39.1117710000002],[-107.66460399999994,39.1094470000001],[-107.66985799999998,39.10862400000008],[-107.67143899999996,39.107633000000305],[-107.67277999999999,39.10546700000015],[-107.67383999999987,39.101091999999994],[-107.67634499999991,39.1003850000003],[-107.68081499999994,39.100256000000286],[-107.68294499999985,39.09270200000003],[-107.68529399999989,39.090918000000045],[-107.68688299999991,39.08862899999997],[-107.68955599999993,39.087171000000126],[-107.69284199999993,39.08757300000019],[-107.69411100000002,39.08688599999999],[-107.69411400000001,39.08513800000014],[-107.695694,39.085038000000225],[-107.70000699999991,39.08385700000002],[-107.70387599999992,39.084993000000054],[-107.70778000000001,39.08369499999998],[-107.71020999999979,39.08224200000018],[-107.71084199999996,39.081254000000115],[-107.71326699999997,39.08027900000013],[-107.71304699999996,39.07906800000012],[-107.716273,39.074281000000155],[-107.71756499999987,39.073899000000154],[-107.718684,39.07173800000004],[-107.7207899999998,39.0700040000001],[-107.72305499999999,39.065184999999985],[-107.729149,39.06619600000016],[-107.73075499999999,39.06588800000026],[-107.732216,39.063007000000084],[-107.73296900000003,39.063052000000084],[-107.7350229999999,39.0662750000003],[-107.73700399999996,39.06551600000006],[-107.73798799999992,39.066160000000025],[-107.74188099999998,39.064023000000134],[-107.74439899999993,39.062189000000274],[-107.74488099999996,39.05972700000012],[-107.74248599999993,39.05934200000007],[-107.74388999999985,39.05770600000005],[-107.74556299999989,39.057648000000086],[-107.74739099999988,39.055929000000106],[-107.75059799999991,39.053989],[-107.75165299999992,39.05439600000011],[-107.75560899999994,39.05389800000012],[-107.75640099999993,39.05229300000019],[-107.75607699999983,39.04989500000005],[-107.75782199999998,39.049314000000265],[-107.75869099999994,39.047879000000194],[-107.76304399999987,39.04722599999997],[-107.76505899999978,39.04468300000008],[-107.76549,39.04329400000006],[-107.76721499999991,39.04398300000025],[-107.77108499999986,39.043693000000076],[-107.77396799999991,39.047084000000154],[-107.77621599999992,39.04785900000013],[-107.78009699999996,39.05254999999994],[-107.78337899999991,39.053733000000136],[-107.78634099999994,39.056194000000005],[-107.78894400000001,39.05627600000008],[-107.78987699999976,39.05795599999999],[-107.79227700000001,39.06051099999996],[-107.79374699999988,39.06144900000015],[-107.79745200000002,39.06280200000003],[-107.80133299999989,39.06361800000002],[-107.80212999999992,39.06478800000019],[-107.80671199999995,39.06551600000006],[-107.80904999999996,39.06648700000011],[-107.81346099999996,39.06726299999997],[-107.81897500000002,39.0696250000002],[-107.81933299999992,39.07078799999999],[-107.82243900000003,39.07042400000023],[-107.82583699999998,39.071576000000164],[-107.82815600000004,39.071762999999976],[-107.82973500000003,39.07283300000006],[-107.83212599999996,39.07336000000015],[-107.83553499999994,39.075368000000196],[-107.84001,39.077168000000086],[-107.841793,39.07666599999993],[-107.84589900000003,39.077313000000004],[-107.84734300000002,39.078143000000296],[-107.84890199999995,39.077825000000075],[-107.85210799999993,39.07852200000002],[-107.85335999999995,39.078310999999985],[-107.85532699999999,39.07665000000003],[-107.85640399999994,39.07726300000013],[-107.85720600000002,39.07938800000028],[-107.85950000000003,39.07981800000016],[-107.86185499999993,39.07772400000027],[-107.862618,39.07887600000004],[-107.86523099999994,39.07661400000006],[-107.86647499999992,39.07623200000012],[-107.86707099999995,39.074677000000065],[-107.86905599999994,39.0743130000003],[-107.86998199999994,39.074929999999995],[-107.87325199999987,39.0734800000003],[-107.87481399999996,39.072387000000106],[-107.87562600000001,39.07076400000011],[-107.88017000000002,39.071101999999996],[-107.88308899999993,39.06956100000008],[-107.88621599999993,39.068285000000174],[-107.88817799999993,39.06681300000008],[-107.89313999999996,39.065798000000086],[-107.89729499999993,39.06621800000005],[-107.90066399999989,39.065521000000274],[-107.90352999999988,39.06299899999999],[-107.906811,39.060729000000265],[-107.90904399999994,39.06054699999993],[-107.90954699999998,39.06151599999998],[-107.91167199999995,39.06259799999998],[-107.91340400000001,39.06256400000001],[-107.91669200000001,39.063833000000045],[-107.92488099999997,39.06317700000011],[-107.92732899999993,39.062593000000106],[-107.93320199999994,39.06209400000023],[-107.93687499999999,39.06206500000013],[-107.94179299999996,39.06084200000021],[-107.949501,39.06030400000009],[-107.95122100000003,39.059702000000016],[-107.95684199999988,39.059821000000284],[-107.95922899999994,39.06025600000021],[-107.96429999999987,39.05965900000001],[-107.97261500000002,39.06121200000007],[-107.98217299999993,39.0598780000002],[-107.98382600000002,39.05824999999999],[-107.98720400000002,39.05287599999991],[-107.991019,39.05046299999992],[-107.99516699999992,39.048563000000115],[-108.00063799999998,39.04544100000027],[-108.00230099999982,39.04323600000026],[-108.004278,39.04191500000002],[-108.00948800000003,39.039743999999985],[-108.01227699999987,39.03823100000022],[-108.01311699999991,39.03734400000019],[-108.01592999999986,39.03611799999999],[-108.01936499999994,39.03351600000019],[-108.02070599999996,39.0332650000002],[-108.02302999999989,39.03088900000006],[-108.02575399999984,39.029765],[-108.02808499999998,39.029704000000265],[-108.030127,39.02884000000017],[-108.03200399999997,39.02736800000025],[-108.03294999999991,39.0246410000002],[-108.034179,39.023393],[-108.03580299999999,39.02060300000005],[-108.03611899999981,39.01718000000017],[-108.038297,39.0157270000002],[-108.04332099999993,39.013866000000235],[-108.04445699999997,39.01209800000015],[-108.04679599999997,39.01224400000012],[-108.04780099999988,39.01145199999996],[-108.04949799999997,39.014651000000015],[-108.05036099999995,39.015133000000105],[-108.05377899999996,39.01405200000016],[-108.05597299999988,39.010572000000025],[-108.05965199999997,39.00671800000015],[-108.06142699999992,39.00530300000031],[-108.06450599999988,39.004149999999925],[-108.06626899999992,39.00085100000001],[-108.06726999999995,38.99998699999992],[-108.06996300000003,38.99659200000008],[-108.07299,38.99503500000003],[-108.07616399999989,38.99261800000022],[-108.07971199999992,38.99112800000023],[-108.08173899999997,38.989408000000196],[-108.08288199999993,38.98728700000021],[-108.08565199999998,38.985160000000064],[-108.08691899999991,38.98298200000016],[-108.08981599999998,38.98113100000012],[-108.09343999999999,38.98058400000002],[-108.09478599999989,38.979684000000134],[-108.09204799999992,38.97618000000011],[-108.09110099999998,38.974312999999995],[-108.09037199999989,38.971612000000164],[-108.088574,38.970074000000125],[-108.08686,38.96780400000023],[-108.08752600000003,38.964814000000274],[-108.08730499999996,38.96318000000025],[-108.08876399999997,38.96223199999997],[-108.08977199999998,38.96020499999997],[-108.0914659999998,38.95936700000004],[-108.09344899999985,38.957458999999915],[-108.09556399999997,38.95676300000008],[-108.09546699999993,38.95453200000003],[-108.09763999999996,38.953564000000256],[-108.10227099999997,38.94918000000007],[-108.10307699999993,38.947806999999955],[-108.10797099999996,38.941010000000006],[-108.11078499999991,38.93755700000008],[-108.113268,38.93578800000029],[-108.11753599999992,38.9318550000001],[-108.11894599999977,38.92902400000003],[-108.12079299999994,38.92720900000029],[-108.12557899999996,38.92405400000018],[-108.12741399999993,38.923834000000284],[-108.13065699999993,38.92230500000005],[-108.13259700000003,38.92290200000008],[-108.1333939999999,38.92396200000002],[-108.13774099999995,38.92605200000014],[-108.14048099999991,38.92515800000024],[-108.14092199999999,38.92417300000011],[-108.14028299999995,38.92146100000002],[-108.14061599999997,38.92062299999998],[-108.14327699999996,38.91952500000008],[-108.14439799999991,38.91853000000003],[-108.144834,38.91640199999995],[-108.14420199999995,38.91488700000002],[-108.1418109999999,38.91192500000011],[-108.14032299999997,38.908767000000125],[-108.14072299999987,38.906846000000144],[-108.14176299999986,38.90637700000025],[-108.14477399999993,38.90747600000003],[-108.146953,38.90747100000016],[-108.14909399999988,38.90556600000008],[-108.15069499999993,38.90266800000012],[-108.15247799999992,38.9012680000003],[-108.15548499999994,38.90148400000004],[-108.15942000000001,38.901053000000275],[-108.16322599999995,38.90213],[-108.16630999999995,38.90224500000011],[-108.16861799999987,38.900750000000244],[-108.17052399999983,38.89831400000014],[-108.17420900000002,38.8981560000002],[-108.176152,38.89727499999998],[-108.176988,38.89531499999998],[-108.18309699999998,38.893238999999994],[-108.21042299999999,38.884184000000005],[-108.23610699999978,38.875906999999984],[-108.25135699999993,38.8711770000001],[-108.28628100000003,38.859587000000204],[-108.32081299999987,38.848239999999976],[-108.33432299999993,38.84374700000001],[-108.37621999999993,38.83017400000017],[-108.37895300000002,38.829108000000076],[-108.37884799999995,38.823026000000084],[-108.37890400000003,38.79805399999998],[-108.37899900000002,38.77661000000012],[-108.37876799999998,38.765274999999974],[-108.3787099999999,38.754647999999975],[-108.37880699999994,38.734037000000285],[-108.37875999999994,38.692616000000044],[-108.37869799999999,38.6680970000001],[-108.3787109999999,38.593795000000114],[-108.37863099999993,38.5854800000003],[-108.37864399999995,38.552592000000004],[-108.3786649999999,38.50219700000008],[-108.37913599999996,38.49998700000003],[-108.41970899999995,38.49998700000003],[-108.50064299999997,38.49998599999998],[-108.53666699999997,38.49998700000003],[-108.60599899999994,38.49998799999992],[-108.6566749999999,38.49998799999992],[-108.71010100000001,38.49998700000003],[-108.75910199999981,38.49998700000003],[-108.8150639999999,38.49998700000003],[-108.89661000000001,38.49998700000003],[-108.93294299999997,38.49998799999992],[-108.98935900000004,38.49998500000004],[-109.06011899999999,38.4999890000002],[-109.0601279999999,38.51060500000017],[-109.06000599999993,38.53590800000012],[-109.05996599999992,38.565266000000236],[-109.05985599999997,38.57939500000009],[-109.05969600000003,38.61916500000012],[-109.059731,38.63360499999993],[-109.05968899999999,38.66684500000008],[-109.05982399999988,38.71474600000005],[-109.05894999999998,38.75340499999999],[-109.05847799999998,38.76578300000011],[-109.057365,38.79754800000006],[-109.05612999999977,38.82646500000004],[-109.05411900000001,38.873532000000125],[-109.05402199999997,38.877161],[-109.05372399999993,38.90835400000003],[-109.05344099999996,38.93460500000009],[-109.0527479999999,38.99071700000013],[-109.05231499999996,39.0335570000002],[-109.05200000000002,39.06770100000017],[-109.05193699999995,39.06942499999997],[-109.0516659999999,39.105097000000114],[-109.05153100000001,39.111697000000106],[-109.05116599999985,39.142603000000236],[-109.05131799999998,39.18661300000008],[-109.05102999999997,39.227126],[-109.05109799999991,39.286620000000084],[-109.05109899999997,39.307326000000046],[-109.05102899999997,39.330962000000056],[-109.05101599999995,39.3500700000003],[-109.050701,39.360945000000015],[-109.05076499999996,39.36667699999998],[-109.00066299999992,39.36637700000023],[-108.98127099999999,39.36632199999997],[-108.97373500000003,39.36659900000012],[-108.951527,39.366525000000024],[-108.92527899999993,39.36651000000006],[-108.90859999999986,39.366690000000006],[-108.9052749999999,39.36652400000031],[-108.87565899999987,39.36653100000018],[-108.87503500000003,39.36627300000015],[-108.86585299999996,39.36624800000021],[-108.84618599999993,39.36631900000003],[-108.77864799999998,39.36642999999998],[-108.750655,39.36648600000001],[-108.74738899999994,39.366608000000156],[-108.73813399999989,39.36659200000025],[-108.73675700000001,39.36646700000006],[-108.69368800000001,39.366472999999985],[-108.63135099999988,39.366481000000306],[-108.5701709999999,39.366488000000174],[-108.50064700000001,39.366016],[-108.49178499999988,39.36610900000011],[-108.48542999999995,39.36598500000031],[-108.46601599999997,39.365950999999995],[-108.42853799999995,39.36605300000025],[-108.40960699999988,39.36598300000031],[-108.40824299999991,39.3664540000002],[-108.402353,39.366358000000105],[-108.39200499999993,39.36654300000009],[-108.38280199999997,39.36639600000001],[-108.37564299999997,39.36663500000026],[-108.36774000000003,39.366122000000075],[-108.29494999999991,39.36626899999999],[-108.26629899999989,39.36632900000018],[-108.25835999999998,39.3662070000002],[-108.25063699999987,39.366361000000154],[-108.24874599999998,39.36661400000014],[-108.21223199999986,39.36617199999995],[-108.20223699999985,39.3661620000002],[-108.17016899999987,39.366265],[-108.13571400000001,39.36619200000001],[-108.11387399999995,39.366234000000304],[-108.10601500000001,39.366151],[-108.0383139999999,39.36620000000016],[-108.03071499999999,39.366160000000036],[-108.00686999999994,39.366174000000115],[-108.00062699999995,39.366268000000275],[-107.99041,39.36613100000011],[-107.98518499999994,39.36618700000014],[-107.95408500000002,39.366254000000026],[-107.9097819999999,39.36619600000017],[-107.88413799999995,39.36619500000012],[-107.86997499999995,39.36626699999999],[-107.81093099999998,39.366253000000086],[-107.75061599999998,39.36624400000005],[-107.71333799999996,39.366130000000226],[-107.67391800000001,39.36623700000001],[-107.62561299999976,39.36637300000007],[-107.60725699999995,39.36614900000001],[-107.55956899999995,39.366130000000226],[-107.55070699999999,39.36604600000021],[-107.500609,39.36611000000016],[-107.49162699999988,39.36626300000006],[-107.43094899999977,39.366178000000104]]]},"properties":{"OBJECTID":40,"COUNTYFP":"077","Shape_Leng":5.03655959413,"Shape_Area":0.900045535632,"low":33935,"lowmod":57215,"LMMI":83840,"lowmoduniv":144125,"lowmod_pct":0.396981786644,"NAMELSAD10":"Mesa County"}},
{"type":"Feature","geometry":{"type":"Polygon","coordinates":[[[-107.00060199999996,37.95604100000003],[-106.99932799999999,37.95702099999994],[-106.997749,37.9573190000001],[-106.99686600000001,37.96039200000001],[-106.99443599999995,37.96077199999996],[-106.98980699999993,37.96103199999999],[-106.98832599999997,37.962072999999975],[-106.98762499999998,37.96373699999998],[-106.98748399999994,37.96795000000009],[-106.987368,37.96917899999994],[-106.98584800000003,37.969042],[-106.97792200000004,37.969088000000056],[-106.97259500000001,37.97198800000001],[-106.969921,37.97313500000007],[-106.9679809999999,37.97503900000004],[-106.96401300000002,37.97252199999997],[-106.96245299999998,37.97231700000009],[-106.96190899999993,37.96894099999997],[-106.96135500000003,37.967952000000025],[-106.95536899999996,37.965017000000046],[-106.95304199999998,37.962808999999936],[-106.95244400000001,37.961853000000076],[-106.95236499999993,37.95744700000006],[-106.94909100000001,37.95669499999997],[-106.94816099999991,37.9555630000001],[-106.94631700000002,37.954004999999995],[-106.94446399999998,37.95372500000008],[-106.94151899999997,37.954259000000036],[-106.93735600000002,37.95664700000009],[-106.935406,37.958785999999975],[-106.93525799999998,37.960363000000086],[-106.93379799999997,37.960866000000124],[-106.93014099999999,37.963298000000066],[-106.92418700000002,37.96486900000002],[-106.92237799999992,37.96559799999994],[-106.92073399999998,37.96643299999994],[-106.92031199999997,37.96743100000009],[-106.91674999999992,37.966436000000044],[-106.91157900000002,37.96647800000011],[-106.91018699999995,37.9652670000001],[-106.90866800000003,37.964053000000035],[-106.90519899999998,37.96390100000008],[-106.90224699999999,37.96564500000011],[-106.89948900000002,37.96678800000001],[-106.89783399999999,37.96789099999995],[-106.89455999999996,37.96710400000006],[-106.89293699999996,37.96534899999995],[-106.88960299999991,37.961870999999974],[-106.88275099999998,37.96153400000014],[-106.88136699999995,37.96224000000012],[-106.87834099999998,37.95850000000013],[-106.87914699999993,37.95640099999997],[-106.87988000000001,37.94986100000011],[-106.87868000000003,37.94915900000012],[-106.87713600000001,37.94858199999999],[-106.87559899999991,37.94582299999996],[-106.87460099999993,37.94331200000005],[-106.87435199999993,37.94268500000004],[-106.87199199999998,37.94205200000005],[-106.87140299999993,37.94189399999999],[-106.86975999999999,37.941454000000135],[-106.86868099999998,37.93793699999992],[-106.86625299999997,37.93409100000002],[-106.86419799999993,37.933245000000056],[-106.85924799999998,37.92857800000007],[-106.85289699999998,37.92388399999999],[-106.84726999999998,37.919170000000065],[-106.84150299999993,37.91889100000003],[-106.83398399999999,37.91891400000003],[-106.82993999999997,37.91789399999999],[-106.82452,37.91873000000004],[-106.81842699999993,37.91403900000006],[-106.80793699999998,37.91538100000008],[-106.80684999999994,37.914822000000015],[-106.80102599999998,37.91369900000001],[-106.79542500000002,37.90962100000007],[-106.78343699999994,37.89953200000002],[-106.77853699999997,37.89486199999999],[-106.77306599999991,37.88557500000002],[-106.7558229999999,37.88062400000001],[-106.75241699999998,37.88118800000012],[-106.75059699999997,37.88149099999998],[-106.7366659999999,37.88788100000005],[-106.72290299999997,37.874999],[-106.72109499999993,37.87402800000012],[-106.71986300000003,37.873385999999925],[-106.71673099999998,37.87228600000003],[-106.71467799999994,37.87190099999998],[-106.71244099999996,37.87199400000003],[-106.710286,37.872568000000115],[-106.70797599999997,37.87346999999994],[-106.706615,37.8747810000001],[-106.70448399999998,37.875844000000086],[-106.70154000000002,37.87635400000005],[-106.697161,37.87611099999998],[-106.69454400000001,37.87436300000013],[-106.69352299999997,37.8735190000001],[-106.69368699999995,37.871679000000086],[-106.69369199999994,37.864279000000124],[-106.69369999999998,37.85590600000012],[-106.693712,37.85396400000002],[-106.69374099999999,37.84973599999995],[-106.69379700000002,37.84172700000005],[-106.69380599999994,37.840430000000026],[-106.69406899999996,37.817703999999935],[-106.69419799999997,37.800877000000014],[-106.69424099999998,37.79474700000014],[-106.69430899999992,37.784519999999986],[-106.69432099999995,37.782876999999985],[-106.6944309999999,37.78035100000005],[-106.69443000000001,37.77000599999991],[-106.69460399999991,37.7570300000001],[-106.69469199999997,37.751318000000026],[-106.69525099999998,37.750001999999995],[-106.69556899999998,37.72252700000007],[-106.69577399999997,37.6906800000001],[-106.69577099999992,37.69017500000007],[-106.69574499999999,37.68507899999997],[-106.6956429999999,37.665369],[-106.69563800000003,37.6642710000001],[-106.71162400000003,37.664273000000094],[-106.71168599999999,37.65674000000013],[-106.71181199999995,37.64155900000014],[-106.71177599999999,37.64111100000002],[-106.71183500000001,37.63229100000012],[-106.71160299999997,37.62500499999999],[-106.71126799999996,37.61528299999992],[-106.71109699999994,37.61011000000002],[-106.71109499999994,37.6088400000001],[-106.71109999999993,37.60873900000013],[-106.71110799999997,37.608573999999976],[-106.71115800000001,37.60759400000006],[-106.71120000000002,37.60056700000001],[-106.7111339999999,37.59281900000002],[-106.711116,37.58862600000003],[-106.71101999999996,37.56505900000002],[-106.71099500000003,37.560963000000015],[-106.71098599999999,37.559808999999916],[-106.710914,37.55107300000009],[-106.71083699999991,37.53145800000004],[-106.71079199999991,37.524373000000026],[-106.71076199999993,37.50997799999999],[-106.71087799999992,37.49771199999992],[-106.71087699999987,37.49406299999998],[-106.71087299999999,37.48673400000007],[-106.71086700000001,37.48161099999999],[-106.71086400000002,37.47933800000004],[-106.71086200000002,37.47743900000012],[-106.71085599999998,37.47244300000011],[-106.710825,37.44823500000001],[-106.71080099999995,37.42535900000013],[-106.71078999999986,37.416168000000084],[-106.71078499999999,37.41218500000008],[-106.710781,37.40942299999995],[-106.710778,37.40723700000012],[-106.71077399999996,37.40431799999999],[-106.71077499999996,37.40422799999999],[-106.71077299999996,37.40262900000005],[-106.71077099999997,37.399782000000016],[-106.71076699999998,37.396238000000096],[-106.72094500000003,37.39613300000008],[-106.72936900000002,37.39604600000001],[-106.75045099999994,37.395974999999964],[-106.773125,37.395538000000045],[-106.77658400000001,37.3954720000001],[-106.79042400000003,37.39520800000014],[-106.801196,37.394975000000045],[-106.85217899999998,37.394010000000094],[-106.87118299999992,37.393665],[-106.87559499999986,37.39402300000012],[-106.89836700000001,37.39386300000001],[-106.89976399999995,37.393853999999976],[-106.94340399999987,37.3935570000001],[-106.960961,37.39343800000006],[-106.96389599999992,37.39341200000001],[-106.98503699999998,37.39324599999998],[-107.00059699999991,37.393136000000084],[-107.02242599999994,37.39270699999997],[-107.02717999999999,37.39262200000002],[-107.04085600000002,37.392381],[-107.04699299999993,37.39230599999996],[-107.04777899999993,37.39230500000008],[-107.04936799999996,37.392304000000024],[-107.05816099999993,37.39232900000013],[-107.06698399999993,37.39235700000012],[-107.07197899999994,37.392379000000005],[-107.07752099999993,37.392403],[-107.08919900000001,37.39245500000004],[-107.09117200000003,37.39242500000006],[-107.09869099999997,37.392434000000094],[-107.11358699999994,37.39249100000001],[-107.12559999999996,37.39248600000013],[-107.128737,37.39240899999999],[-107.12871399999995,37.40394900000001],[-107.12868199999997,37.421783000000005],[-107.12867999999997,37.42294199999998],[-107.12862499999994,37.45354900000001],[-107.12869899999987,37.47333600000013],[-107.12781799999999,37.50000400000005],[-107.12693999999999,37.539743000000044],[-107.12560400000001,37.60049200000003],[-107.12448,37.612088000000085],[-107.12373400000001,37.62500200000011],[-107.12386800000002,37.67502900000011],[-107.12560500000001,37.675196000000085],[-107.14513799999997,37.67549700000012],[-107.14274899999992,37.71680800000013],[-107.14070699999996,37.75],[-107.13994000000002,37.757197000000076],[-107.13735600000001,37.82878800000009],[-107.13860699999998,37.865860000000055],[-107.13816999999995,37.87499800000012],[-107.138191,37.93884800000001],[-107.13567699999999,37.93852700000008],[-107.13497899999999,37.93783500000001],[-107.13132000000002,37.937603000000024],[-107.12962699999991,37.93783800000011],[-107.125606,37.93915600000008],[-107.12196799999998,37.94086700000008],[-107.11576699999995,37.942004],[-107.11066900000003,37.943344000000025],[-107.10841600000003,37.944618000000105],[-107.10611699999998,37.94713400000012],[-107.10560199999998,37.94951600000002],[-107.10321199999993,37.95505300000002],[-107.102105,37.95481399999994],[-107.10147599999993,37.952904000000046],[-107.100077,37.95189600000009],[-107.09731999999997,37.95078600000011],[-107.09460599999994,37.94930699999992],[-107.09214399999996,37.94916999999998],[-107.08836199999996,37.95040899999998],[-107.08670000000001,37.952690000000075],[-107.085552,37.95336099999997],[-107.08015699999999,37.95429600000011],[-107.07565599999998,37.953922000000034],[-107.072001,37.95536199999998],[-107.06923199999994,37.95844699999998],[-107.06846100000001,37.960762000000045],[-107.06544899999994,37.95975199999998],[-107.06141499999995,37.95974799999999],[-107.05976399999997,37.957765999999936],[-107.05938800000001,37.9556510000001],[-107.05638399999998,37.952256999999975],[-107.05523700000003,37.95215500000006],[-107.05285700000002,37.95312600000011],[-107.05115899999993,37.952956000000086],[-107.04942,37.95214900000008],[-107.044716,37.94932300000005],[-107.04136299999988,37.94868100000002],[-107.03609599999999,37.94938000000013],[-107.03320599999995,37.95008100000001],[-107.03154699999993,37.95105200000006],[-107.03056400000003,37.95306500000004],[-107.02835199999998,37.954136000000005],[-107.02550599999995,37.95450099999999],[-107.02291799999995,37.953792000000135],[-107.02084299999996,37.95231299999995],[-107.01754199999993,37.949018000000024],[-107.01118099999997,37.94696099999999],[-107.00897299999997,37.94652200000007],[-107.007654,37.947292000000004],[-107.00675599999994,37.94920300000007],[-107.00669499999998,37.95410400000003],[-107.00626299999993,37.95621799999998],[-107.00468899999998,37.95692100000002],[-107.00231499999995,37.955977000000075],[-107.00060199999996,37.95604100000003]]]},"properties":{"OBJECTID":41,"COUNTYFP":"079","Shape_Leng":2.04506648495,"Shape_Area":0.23216854074,"low":145,"lowmod":295,"LMMI":425,"lowmoduniv":705,"lowmod_pct":0.418439716312,"NAMELSAD10":"Mineral County"}},
{"type":"Feature","geometry":{"type":"Polygon","coordinates":[[[-107.31779399999994,41.002920000000074],[-107.31786299999999,40.9999650000002],[-107.31741099999988,40.95741200000009],[-107.31688599999995,40.908011000000215],[-107.31653900000003,40.874950000000126],[-107.31608799999992,40.84484500000008],[-107.31532300000003,40.797995000000014],[-107.31509899999998,40.786572999999976],[-107.3145409999999,40.74996900000008],[-107.31356599999987,40.694074],[-107.31336499999986,40.686553000000174],[-107.31428800000003,40.62497099999996],[-107.31415300000003,40.60028100000028],[-107.34855500000003,40.60026600000009],[-107.37148699999995,40.600302000000056],[-107.37141500000001,40.59625300000005],[-107.371328,40.54239900000016],[-107.37561599999998,40.54241000000019],[-107.42881299999999,40.542207000000076],[-107.42869999999994,40.51750099999998],[-107.42852399999987,40.49997300000007],[-107.428516,40.484947999999974],[-107.42838799999998,40.48021],[-107.42850499999997,40.46976000000029],[-107.42910699999987,40.44453399999992],[-107.42934500000001,40.440770999999984],[-107.43123600000001,40.37578000000002],[-107.43107199999992,40.37496900000002],[-107.43152499999991,40.355424000000255],[-107.4320679999999,40.334878999999944],[-107.44259399999999,40.3347550000002],[-107.4425179999999,40.30843900000025],[-107.4401089999999,40.249976000000004],[-107.439506,40.247327999999925],[-107.43938600000001,40.22337900000008],[-107.43937199999993,40.22045400000002],[-107.45031899999992,40.22040600000014],[-107.49561799999987,40.21999100000028],[-107.50061499999987,40.220507],[-107.50666799999993,40.220507],[-107.52502400000003,40.22116700000004],[-107.56186599999995,40.22216500000002],[-107.58371099999988,40.222532],[-107.60333999999995,40.223116000000005],[-107.61209399999996,40.223651000000075],[-107.63404499999996,40.224131],[-107.65394099999992,40.22427499999998],[-107.65398699999992,40.22345200000029],[-107.66876000000002,40.223396000000264],[-107.67939799999982,40.22358500000007],[-107.71938599999999,40.223605000000134],[-107.74540499999995,40.223433000000114],[-107.75360899999993,40.22386200000028],[-107.764545,40.22370100000012],[-107.764544,40.2226150000003],[-107.78053599999998,40.222507000000064],[-107.78854599999988,40.22270600000002],[-107.80817599999995,40.22275100000019],[-107.80961899999988,40.222577],[-107.83977600000003,40.222442],[-107.8415589999999,40.222541999999976],[-107.86025399999988,40.22235100000012],[-107.875629,40.22239300000024],[-107.91999499999991,40.22213100000005],[-107.94239899999991,40.22209000000021],[-107.965419,40.22226600000022],[-107.99129099999976,40.22239100000024],[-107.99509499999994,40.22220800000002],[-108.00063399999993,40.22230399999995],[-108.01191999999998,40.2220760000003],[-108.06375200000002,40.22186700000003],[-108.113179,40.221670000000074],[-108.12556199999989,40.221705000000156],[-108.15966099999986,40.22167300000001],[-108.21387599999997,40.221757000000196],[-108.26889899999998,40.221847000000025],[-108.32479099999995,40.22193699999997],[-108.37564799999996,40.222019000000046],[-108.42459500000001,40.22215500000027],[-108.50065199999989,40.222369000000015],[-108.53541899999999,40.22236600000031],[-108.62565199999995,40.22236100000009],[-108.6879189999998,40.222465],[-108.70727799999992,40.22266500000018],[-108.75066200000003,40.22282200000001],[-108.82414499999993,40.22279000000003],[-108.87566600000002,40.222769000000255],[-108.90247499999998,40.22270800000018],[-108.96508599999993,40.22258699999992],[-109.00067099999995,40.2225160000001],[-109.00833799999998,40.222629999999924],[-109.0269409999998,40.222715999999934],[-109.02771799999988,40.2226150000003],[-109.050949,40.22264700000011],[-109.05099200000001,40.23736600000012],[-109.05094299999996,40.25240000000002],[-109.05102599999998,40.267495000000224],[-109.05093199999993,40.30303400000014],[-109.05086,40.369411999999954],[-109.05080099999992,40.37496700000003],[-109.05090999999987,40.428183000000104],[-109.051245,40.44159900000028],[-109.05083799999989,40.47190000000006],[-109.05009199999989,40.52919800000001],[-109.04900299999991,40.560427000000004],[-109.04800999999992,40.58097700000019],[-109.04808500000001,40.59510900000009],[-109.04776699999996,40.609879000000035],[-109.04827999999998,40.63973200000004],[-109.048629,40.65360200000015],[-109.048944,40.66260100000005],[-109.04843199999993,40.66260100000005],[-109.048993,40.68438500000002],[-109.04908799999987,40.71450299999992],[-109.048786,40.75143300000002],[-109.04870700000004,40.771538000000135],[-109.04873299999997,40.78269400000022],[-109.04845499999999,40.826081000000045],[-109.04850899999991,40.832084000000066],[-109.04884099999992,40.83782100000013],[-109.04880899999995,40.84264700000011],[-109.04896899999994,40.87300800000003],[-109.04887699999989,40.88776100000018],[-109.04901399999994,40.89190600000012],[-109.04927699999996,40.908645999999976],[-109.04953899999998,40.942730000000154],[-109.04994199999999,40.994908000000294],[-109.050026,41.000691000000074],[-109.02497499999993,41.000459000000035],[-109.00053199999991,41.00055800000024],[-108.94352799999996,41.00033000000002],[-108.88413799999984,41.00009400000022],[-108.87567099999995,41.00016600000015],[-108.84970900000002,41.00011400000028],[-108.82286699999997,41.000153000000125],[-108.81748899999991,41.00010500000025],[-108.79244399999988,41.000201000000004],[-108.77662099999992,41.00010000000003],[-108.75066700000002,41.000187000000096],[-108.74002599999994,41.00006900000028],[-108.68327399999998,41.000113],[-108.63110799999993,41.000156000000004],[-108.62566299999997,40.99996199999998],[-108.56185399999998,40.99996199999998],[-108.557658,40.999622000000045],[-108.538795,40.99972100000008],[-108.52602899999994,40.999592000000064],[-108.5196049999999,40.99996199999998],[-108.50065899999993,41.000112000000286],[-108.44955899999991,41.000071000000275],[-108.375653,41.00001300000008],[-108.30499799999996,41.000068],[-108.25064899999995,41.00011400000028],[-108.24972000000002,41.000361999999996],[-108.22868099999988,41.00031899999999],[-108.22219299999989,41.00039599999997],[-108.20964400000003,41.000329000000306],[-108.20646099999993,41.00041200000004],[-108.18122699999998,41.000455000000045],[-108.16939499999995,41.00059500000009],[-108.14932099999982,41.00096200000007],[-108.126215,41.00127600000013],[-108.12261899999993,41.00119800000027],[-108.10530999999986,41.00139500000023],[-108.08950799999991,41.00123300000013],[-108.08922399999994,41.00157800000011],[-108.046539,41.00206400000013],[-108.02423699999991,41.00206400000013],[-108.00063599999999,41.00181400000025],[-107.97032999999999,41.00201200000009],[-107.90079899999995,41.00200000000018],[-107.87563199999988,41.00202300000012],[-107.8666389999999,41.00193200000024],[-107.85194299999995,41.00192600000008],[-107.79164699999984,41.00196699999992],[-107.78479199999987,41.00192900000002],[-107.75062700000001,41.00197100000008],[-107.7203659999999,41.00212700000003],[-107.68936300000001,41.00216000000012],[-107.6695959999999,41.00213400000007],[-107.6256489999999,41.00212300000004],[-107.573624,41.00225100000017],[-107.54099899999994,41.002296],[-107.52663799999999,41.002356000000134],[-107.50062000000003,41.002349000000095],[-107.47870899999998,41.002670000000194],[-107.42380700000001,41.00285600000012],[-107.38607999999999,41.00302900000008],[-107.36754100000002,41.00307300000014],[-107.35377699999992,41.0029890000003],[-107.31779399999994,41.002920000000074]]]},"properties":{"OBJECTID":42,"COUNTYFP":"081","Shape_Leng":5.03260887132,"Shape_Area":1.30950040782,"low":3295,"lowmod":5245,"LMMI":8135,"lowmoduniv":12970,"lowmod_pct":0.404394757132,"NAMELSAD10":"Moffat County"}},
{"type":"Feature","geometry":{"type":"Polygon","coordinates":[[[-108.37920300000002,36.999513000000206],[-108.40373899999992,36.99949800000002],[-108.42190099999993,36.99943900000011],[-108.48110299999996,36.99931299999997],[-108.50017699999984,36.99916600000006],[-108.50064199999997,36.99927600000012],[-108.55706999999995,36.999269000000254],[-108.59082699999988,36.99921100000006],[-108.61906299999987,36.999202000000025],[-108.6256459999999,36.999108999999976],[-108.64713899999992,36.99910000000011],[-108.68209299999995,36.99896899999993],[-108.70146199999982,36.99897300000009],[-108.73706799999997,36.9988680000003],[-108.75064999999995,36.99889800000011],[-108.76676399999997,36.99884899999995],[-108.77876399999997,36.998902999999984],[-108.79206599999986,36.99884400000008],[-108.81031999999993,36.99894000000006],[-108.8264329999999,36.99896400000006],[-108.85688099999993,36.99891700000006],[-108.875317,36.9988390000002],[-108.91657499999997,36.99886400000014],[-108.961071,36.998918000000174],[-109.00065799999993,36.99893499999996],[-109.03042499999987,36.99900999999994],[-109.04522299999991,36.99908400000004],[-109.0455159999999,36.99999700000012],[-109.04523099999994,37.02355500000027],[-109.045166,37.07274200000006],[-109.04505799999993,37.07466099999999],[-109.04499499999997,37.08642900000007],[-109.04524600000002,37.09331900000012],[-109.04515599999985,37.112063999999975],[-109.045995,37.177279000000055],[-109.04597799999993,37.20183100000003],[-109.04548699999998,37.210844000000236],[-109.04558400000002,37.24935100000005],[-109.04603900000001,37.249993000000245],[-109.04596300000003,37.285930000000064],[-109.04587900000001,37.32509700000014],[-109.0458079999998,37.359725000000196],[-109.04580999999996,37.37499300000013],[-109.04486099999997,37.42244200000016],[-109.04456499999998,37.43410700000004],[-109.04362300000003,37.47832599999998],[-109.04405699999995,37.48080600000003],[-109.04348799999997,37.48468400000024],[-109.04113599999977,37.487262000000214],[-109.03912000000003,37.4888260000003],[-109.02926899999994,37.49992400000002],[-109.02494499999995,37.504545000000064],[-109.00849499999993,37.52519699999999],[-109.00066399999997,37.53421700000001],[-108.97593899999998,37.56298900000013],[-108.92902999999995,37.617859000000124],[-108.92284999999976,37.624989000000085],[-108.917081,37.63190199999991],[-108.87565999999987,37.632202000000234],[-108.85712599999988,37.63193500000017],[-108.8373939999999,37.63184600000011],[-108.79727200000002,37.63193600000005],[-108.77382999999992,37.63178600000009],[-108.75807700000001,37.6318],[-108.74361799999997,37.63167299999998],[-108.72576400000003,37.63159700000028],[-108.69320299999993,37.63151100000027],[-108.66605999999996,37.63158299999998],[-108.66605599999997,37.633195],[-108.64649800000001,37.63269300000019],[-108.62564999999978,37.63261000000006],[-108.60990099999992,37.63271900000001],[-108.56441499999988,37.6326380000001],[-108.50064700000001,37.632526999999925],[-108.48342899999994,37.63246800000002],[-108.47451099999995,37.63252200000005],[-108.45766100000003,37.632498000000055],[-108.44454999999988,37.632834000000116],[-108.42400499999997,37.632666000000256],[-108.41719199999989,37.63248100000004],[-108.40829299999996,37.63188600000001],[-108.39176199999997,37.63207],[-108.37564099999997,37.63210700000019],[-108.36353400000002,37.631930000000295],[-108.335642,37.632150000000195],[-108.30925699999983,37.63229700000028],[-108.30394100000001,37.6323890000001],[-108.25063499999987,37.63284700000014],[-108.22679699999998,37.6332460000001],[-108.20325899999995,37.633736],[-108.19448399999999,37.63397000000015],[-108.17917199999994,37.63462700000014],[-108.16114199999993,37.635265000000004],[-108.1404389999999,37.636827000000096],[-108.125632,37.638104999999996],[-108.1079959999999,37.63875300000012],[-108.09115600000001,37.638863000000015],[-108.06643399999996,37.63895700000012],[-108.05821800000001,37.639094000000114],[-108.04897399999976,37.639049000000284],[-108.00742599999995,37.63926700000002],[-108.0006249999999,37.639378000000136],[-107.97008599999987,37.639587000000006],[-107.970531,37.63847599999997],[-107.96992299999988,37.63717000000008],[-107.97212000000002,37.63432100000006],[-107.97244899999998,37.631641],[-107.97208699999987,37.629816000000176],[-107.97987799999987,37.62887500000011],[-107.98195299999986,37.62717200000026],[-107.98591599999992,37.625181],[-107.9872209999998,37.625004000000104],[-107.98921899999993,37.62224099999992],[-107.98972199999997,37.62070700000004],[-107.99171299999995,37.619651999999974],[-107.993153,37.61585900000017],[-107.99488699999995,37.61485700000003],[-108.0001999999999,37.614325000000065],[-108.00074799999993,37.611180000000275],[-108.00264199999998,37.60967400000004],[-108.00461000000001,37.60731700000002],[-108.00467899999995,37.606246000000056],[-108.00766199999993,37.605671000000086],[-108.00983199999979,37.60577700000016],[-108.01184799999999,37.60485800000015],[-108.01318600000002,37.60479000000021],[-108.01429799999994,37.603594000000044],[-108.01553000000001,37.600851999999975],[-108.01410399999997,37.59902800000003],[-108.01394399999987,37.596469999999954],[-108.01688999999999,37.595398999999986],[-108.01644599999992,37.59389900000008],[-108.01738899999998,37.59179399999999],[-108.02243900000002,37.59142600000007],[-108.02300599999984,37.59099900000007],[-108.02120599999995,37.58820700000018],[-108.01971099999997,37.584947000000284],[-108.02053399999994,37.58249000000001],[-108.01974299999995,37.58090400000026],[-108.02016099999992,37.578284],[-108.01947999999987,37.575624000000005],[-108.01950599999998,37.57402700000006],[-108.02085,37.57144500000027],[-108.01922400000001,37.56831999999997],[-108.0182789999999,37.567140999999936],[-108.02174599999995,37.56282399999998],[-108.02119800000003,37.55941400000029],[-108.0196959999999,37.558524000000034],[-108.0205299999999,37.55642300000011],[-108.02025299999997,37.553780000000245],[-108.02190699999994,37.55127900000008],[-108.02153999999996,37.54866700000014],[-108.02353499999992,37.54659400000003],[-108.02423699999991,37.54494300000022],[-108.02411999999993,37.54292100000009],[-108.0254959999998,37.541106000000184],[-108.02543700000001,37.53971899999999],[-108.02890799999994,37.53625199999999],[-108.03489699999989,37.532043999999985],[-108.03452400000003,37.529527000000144],[-108.03374599999995,37.527461000000244],[-108.03380800000002,37.525743000000205],[-108.03463799999986,37.52322600000031],[-108.03358800000001,37.5199990000001],[-108.03214300000002,37.51804599999997],[-108.03369099999992,37.51632100000006],[-108.03452099999976,37.51313500000026],[-108.03405700000002,37.511650000000145],[-108.03275400000001,37.50991499999992],[-108.03147000000001,37.50905899999998],[-108.03407899999996,37.508407000000034],[-108.03727400000002,37.505269999999996],[-108.0379319999999,37.503189000000134],[-108.03632099999993,37.49948599999999],[-108.03460499999989,37.49717300000009],[-108.03517999999985,37.496354],[-108.035345,37.49397499999998],[-108.03655600000002,37.49112700000006],[-108.03654899999987,37.48775499999999],[-108.035932,37.48573299999998],[-108.03376100000003,37.484498000000144],[-108.0336529999999,37.48009899999994],[-108.032644,37.478913000000034],[-108.03451399999989,37.47682900000029],[-108.03470399999998,37.47503900000015],[-108.0359899999998,37.47270900000012],[-108.03638999999987,37.471178000000066],[-108.03314599999987,37.464939999999956],[-108.03400999999997,37.46347000000003],[-108.03475600000002,37.460602999999935],[-108.03746899999999,37.45906599999995],[-108.03915799999993,37.455875000000276],[-108.03914999999978,37.454616000000044],[-108.03826200000003,37.45245399999999],[-108.03987799999999,37.45044900000005],[-108.04240499999992,37.448353],[-108.04328299999997,37.44434900000016],[-108.04450099999997,37.442704000000106],[-108.04844299999996,37.44256400000006],[-108.05302799999993,37.43752100000006],[-108.05516899999998,37.437545000000114],[-108.05739699999992,37.43811800000009],[-108.06141400000001,37.43744600000008],[-108.06374499999998,37.43774499999995],[-108.065406,37.43849100000023],[-108.06880699999994,37.43787000000003],[-108.07341300000002,37.43896100000006],[-108.07804199999987,37.43888099999998],[-108.07904599999989,37.437306000000035],[-108.07954999999993,37.43530900000002],[-108.08062199999989,37.43409000000008],[-108.08105199999994,37.43241400000011],[-108.08004099999988,37.4291870000003],[-108.0828709999999,37.4278920000001],[-108.08603199999988,37.42805900000002],[-108.08728699999989,37.42776900000001],[-108.08754299999993,37.42514000000028],[-108.08950899999996,37.42122400000028],[-108.09514199999984,37.41931900000003],[-108.0977949999999,37.417473000000086],[-108.0991679999999,37.41544100000016],[-108.09954599999992,37.412627000000214],[-108.10126399999996,37.41034700000006],[-108.10341799999992,37.408861],[-108.10533599999997,37.40557300000012],[-108.10569199999986,37.401863000000105],[-108.10630800000001,37.4007380000001],[-108.10665399999988,37.398134000000084],[-108.10755599999999,37.3943230000001],[-108.1074549999999,37.39208600000006],[-108.1056539999999,37.38912000000016],[-108.10625999999996,37.38457299999999],[-108.10585700000001,37.38206800000006],[-108.10917399999994,37.381542000000024],[-108.11361599999992,37.3803460000002],[-108.11649099999994,37.37828300000001],[-108.11723199999989,37.37674700000031],[-108.12048099999987,37.37471500000004],[-108.13234899999986,37.37467800000019],[-108.13373000000001,37.373725000000206],[-108.13718299999988,37.37309900000008],[-108.13982099999993,37.37303800000001],[-108.14119999999997,37.37196700000004],[-108.14219599999996,37.369741999999974],[-108.14564899999993,37.367363000000125],[-108.147041,37.3653790000003],[-108.15001099999989,37.363550000000146],[-108.15247299999993,37.362589000000014],[-108.1581059999998,37.36389400000019],[-108.16170399999999,37.36222499999991],[-108.16429900000003,37.36234300000012],[-108.16463199999987,37.36345700000027],[-108.16658599999988,37.365174000000025],[-108.17303199999986,37.36638800000014],[-108.17646200000001,37.364892],[-108.17913899999996,37.36403000000024],[-108.18359899999996,37.363109000000236],[-108.18446299999994,37.36239600000022],[-108.18894599999999,37.36016600000028],[-108.19042899999994,37.36015100000009],[-108.19268799999998,37.35907000000003],[-108.19566899999995,37.35607300000004],[-108.19777599999998,37.354939],[-108.19824199999988,37.35149999999999],[-108.20198099999988,37.34869099999992],[-108.20325299999996,37.3472440000001],[-108.20483599999994,37.34407999999996],[-108.20498499999985,37.34158900000028],[-108.20576,37.33856600000007],[-108.20659599999999,37.337630000000274],[-108.20772799999997,37.33507700000007],[-108.20792099999994,37.33307200000007],[-108.2067019999999,37.33224100000007],[-108.204811,37.32954600000005],[-108.204679,37.32812500000023],[-108.20303599999988,37.32631600000013],[-108.20521499999995,37.32445699999994],[-108.20794599999988,37.32325600000024],[-108.21048200000001,37.321744000000194],[-108.21142299999997,37.320512000000065],[-108.21202499999987,37.31747300000012],[-108.21382099999988,37.3145100000001],[-108.21361799999994,37.31316199999992],[-108.21647699999988,37.31161500000002],[-108.21649399999995,37.30622900000026],[-108.216094,37.305488000000025],[-108.21308499999998,37.30329699999993],[-108.21250399999997,37.300151000000085],[-108.21377100000001,37.298118000000045],[-108.21798899999993,37.29725000000019],[-108.21937599999995,37.29380400000002],[-108.21896800000002,37.29215099999999],[-108.2197049999998,37.29075800000021],[-108.21968800000002,37.28741500000001],[-108.2242819999999,37.286950000000274],[-108.22569099999993,37.28627300000005],[-108.22802999999993,37.281949],[-108.22687599999995,37.28050200000001],[-108.22731199999987,37.278166],[-108.22867600000001,37.27696400000002],[-108.23352999999992,37.27573500000011],[-108.23483799999997,37.27476800000022],[-108.23528099999993,37.27261500000026],[-108.23715399999992,37.270433000000196],[-108.23740999999995,37.266976],[-108.23858899999999,37.262667000000135],[-108.23993999999993,37.26184600000005],[-108.24246199999999,37.26142800000014],[-108.24429099999998,37.25801400000006],[-108.245046,37.25747300000006],[-108.24660099999977,37.25492900000012],[-108.246757,37.252312000000074],[-108.24731199999991,37.24989000000005],[-108.24962099999999,37.24955100000011],[-108.25063999999992,37.246313999999984],[-108.25249600000001,37.24561],[-108.25596999999976,37.243695],[-108.25613699999991,37.24264900000014],[-108.25767100000002,37.240352000000314],[-108.26007900000002,37.239305],[-108.26126999999997,37.23675700000007],[-108.26144799999997,37.23503100000005],[-108.267536,37.233087000000125],[-108.26770099999999,37.231966000000114],[-108.26890600000002,37.230159000000185],[-108.27078699999987,37.22920700000009],[-108.27691300000004,37.22870400000022],[-108.27884499999993,37.22724400000021],[-108.27974199999989,37.228571999999986],[-108.28168299999999,37.2285590000003],[-108.28465,37.226393000000144],[-108.28627699999987,37.22478800000022],[-108.289354,37.22353000000004],[-108.29042299999992,37.22358400000007],[-108.29167599999994,37.22200300000003],[-108.2915349999999,37.22028699999993],[-108.29379599999993,37.21737000000019],[-108.29459599999996,37.215363000000025],[-108.29728299999988,37.213233],[-108.29746699999998,37.208429999999964],[-108.2971609999999,37.207557000000065],[-108.29485699999998,37.20560200000011],[-108.29380399999997,37.20333699999992],[-108.29288299999996,37.202462000000196],[-108.293273,37.199993000000006],[-108.29262999999992,37.19831900000008],[-108.29290000000003,37.194323],[-108.29378999999977,37.192518000000234],[-108.29572099999996,37.19007100000027],[-108.29592499999995,37.186874000000046],[-108.2946,37.18186300000025],[-108.29462199999989,37.18019199999998],[-108.29540099999986,37.175310000000195],[-108.29730799999999,37.17210399999993],[-108.29741499999994,37.171159999999986],[-108.29588199999995,37.167084999999986],[-108.29404499999993,37.16476000000017],[-108.29387999999994,37.16239400000018],[-108.29576199999997,37.159696000000224],[-108.29607299999986,37.158422000000144],[-108.29440999999991,37.15717200000029],[-108.29188999999997,37.156032000000096],[-108.29016899999993,37.154201000000114],[-108.29021399999993,37.152206999999976],[-108.29124100000001,37.14932799999997],[-108.29014999999987,37.146959000000265],[-108.29045899999994,37.14597500000002],[-108.29422699999998,37.14444700000007],[-108.30089299999997,37.13878700000021],[-108.301513,37.136440000000164],[-108.30410799999987,37.135432000000094],[-108.30724399999991,37.13182599999993],[-108.309842,37.129968000000076],[-108.31047999999998,37.128461999999956],[-108.30977100000001,37.12730200000027],[-108.3096579999999,37.12496699999997],[-108.31099299999994,37.12415200000004],[-108.31112299999995,37.12312600000007],[-108.30961999999994,37.11983900000024],[-108.31179899999995,37.11682400000018],[-108.31141799999995,37.11416500000007],[-108.309845,37.11281300000013],[-108.30972199999997,37.111154],[-108.311173,37.10984199999996],[-108.31028999999995,37.107331999999985],[-108.31059099999999,37.10462200000006],[-108.31460099999987,37.10403600000012],[-108.31662299999994,37.102363000000025],[-108.316261,37.099834999999985],[-108.317341,37.09753500000011],[-108.32106499999992,37.097099999999955],[-108.32061899999997,37.095444999999984],[-108.32207399999993,37.09343500000011],[-108.32307000000003,37.09071499999999],[-108.32241899999997,37.08876099999992],[-108.32331199999987,37.087814000000094],[-108.32360099999988,37.08520900000002],[-108.325087,37.08337800000004],[-108.32410600000003,37.081345999999996],[-108.32567499999999,37.0795270000001],[-108.32570099999992,37.07615699999997],[-108.32681400000001,37.0754340000002],[-108.32833499999998,37.075660000000084],[-108.33015099999994,37.07445400000029],[-108.32773099999991,37.07124699999997],[-108.32826599999987,37.06912100000028],[-108.33045299999998,37.0678760000003],[-108.33317699999998,37.067443000000026],[-108.33346499999999,37.06561500000009],[-108.3343129999999,37.06456100000008],[-108.33445999999998,37.06275900000003],[-108.33338700000002,37.06079800000015],[-108.334719,37.059815000000185],[-108.33421900000002,37.05585700000006],[-108.334792,37.05422000000016],[-108.33736699999997,37.05232100000006],[-108.33922199999989,37.05195800000024],[-108.34115299999996,37.05101500000001],[-108.34304399999996,37.048794999999984],[-108.34572400000002,37.04846900000001],[-108.34700299999997,37.047772000000066],[-108.34718799999979,37.045912000000044],[-108.34985399999994,37.042603000000156],[-108.35071199999987,37.04070200000007],[-108.35454099999987,37.038356000000135],[-108.355118,37.03557000000018],[-108.35286299999996,37.03329000000002],[-108.355031,37.031158000000005],[-108.35643999999996,37.028431000000126],[-108.35555899999991,37.025555],[-108.35902299999998,37.02593800000011],[-108.36019099999999,37.02676300000013],[-108.36183900000003,37.026828000000194],[-108.36582699999997,37.02827900000017],[-108.36868099999992,37.028875000000085],[-108.3717999999999,37.0291050000003],[-108.372319,37.028481],[-108.37250599999993,37.025756000000115],[-108.37334399999992,37.0246930000003],[-108.37178,37.02123900000015],[-108.37321299999991,37.01873600000016],[-108.374323,37.01792300000022],[-108.37405999999999,37.016590000000235],[-108.37509699999993,37.01556000000028],[-108.37478899999991,37.01365200000015],[-108.3766779999998,37.01183800000001],[-108.37728199999987,37.00923399999999],[-108.37643099999997,37.006620000000055],[-108.37814499999996,37.005641999999966],[-108.37961200000001,37.004177000000084],[-108.38106900000002,37.00146799999993],[-108.37920300000002,36.999513000000206]]]},"properties":{"OBJECTID":43,"COUNTYFP":"083","Shape_Leng":3.29301347557,"Shape_Area":0.537266147491,"low":7095,"lowmod":12150,"LMMI":16785,"lowmoduniv":25445,"lowmod_pct":0.477500491256,"NAMELSAD10":"Montezuma County"}},
{"type":"Feature","geometry":{"type":"Polygon","coordinates":[[[-107.500655,38.66855300000003],[-107.50061099999999,38.66515199999998],[-107.50061199999999,38.608052999999984],[-107.50061399999987,38.5363330000003],[-107.50061299999987,38.476414000000034],[-107.50061299999987,38.43318699999992],[-107.50040200000001,38.43265299999996],[-107.5003779999999,38.40419500000007],[-107.50030099999992,38.380843000000084],[-107.5001279999999,38.36562100000003],[-107.50001999999995,38.30192599999998],[-107.5584409999999,38.30192900000009],[-107.57516099999998,38.30195900000024],[-107.60407499999985,38.30190500000003],[-107.63503999999995,38.301896],[-107.63499299999995,38.303992999999934],[-107.63372300000003,38.306467],[-107.6329879999999,38.309518000000025],[-107.63532499999991,38.312144000000046],[-107.63577199999986,38.314714000000265],[-107.63649699999996,38.31615400000021],[-107.63574499999993,38.318269999999984],[-107.63374699999997,38.320480999999916],[-107.63366599999995,38.32212700000025],[-107.63096199999995,38.32567900000015],[-107.63234,38.32686600000011],[-107.63362699999999,38.329884000000106],[-107.635423,38.33092400000015],[-107.63569699999994,38.33214800000019],[-107.71729799999997,38.33214700000008],[-107.72767099999999,38.332325999999966],[-107.73643800000002,38.33221200000014],[-107.77077499999996,38.33223799999996],[-107.81268299999999,38.332221000000175],[-107.81763199999995,38.33224600000011],[-107.8665969999999,38.33220500000027],[-107.87561800000003,38.332778000000076],[-107.93103399999995,38.33263800000003],[-107.99693500000001,38.332477999999924],[-108.00062200000002,38.33249500000011],[-108.01168100000001,38.33196099999998],[-108.059325,38.33191599999998],[-108.07267899999994,38.33201600000007],[-108.08606399999996,38.33197500000006],[-108.10533799999996,38.33206900000022],[-108.12562799999984,38.33165500000024],[-108.13238000000001,38.33157300000005],[-108.13174799999996,38.32075600000013],[-108.12368300000003,38.32092800000015],[-108.12326899999988,38.31363499999998],[-108.10481900000002,38.31371800000028],[-108.10416099999992,38.28417900000011],[-108.08634999999998,38.284334],[-108.08721799999995,38.26255300000008],[-108.08660399999997,38.25520200000028],[-108.01153099999988,38.25526100000019],[-108.01153099999988,38.248203000000274],[-107.98408399999988,38.24838599999998],[-107.98418399999997,38.2339360000002],[-107.965419,38.23401500000017],[-107.96540799999997,38.226509000000306],[-107.94714199999987,38.22647700000016],[-107.9470199999999,38.21894700000007],[-107.937588,38.21899200000007],[-107.9381449999999,38.17393599999997],[-107.94774399999994,38.173767000000225],[-107.94696299999998,38.152491],[-107.96578899999997,38.15232800000001],[-107.96963499999993,38.15229500000015],[-107.99186299999985,38.152491],[-108.00062200000002,38.152480000000025],[-108.011305,38.152702999999974],[-108.02103799999992,38.15261600000008],[-108.038882,38.15266700000001],[-108.06150500000001,38.15246400000012],[-108.11304899999999,38.15222800000009],[-108.118223,38.15226899999993],[-108.12562799999984,38.152702999999974],[-108.13606799999991,38.152670000000114],[-108.17011500000001,38.15225399999997],[-108.1772489999999,38.152304000000186],[-108.22759100000002,38.1524330000002],[-108.24981099999991,38.15253799999999],[-108.25063499999987,38.152793000000145],[-108.26093000000003,38.15267800000004],[-108.28624899999988,38.15254500000003],[-108.31495099999995,38.15247400000021],[-108.32563299999998,38.15249699999998],[-108.37563999999998,38.15284600000007],[-108.38760199999984,38.153043000000025],[-108.41482899999994,38.153267000000085],[-108.437096,38.15320000000008],[-108.44548999999995,38.15330000000017],[-108.50064500000002,38.15319899999997],[-108.52155199999987,38.15330200000017],[-108.53959199999986,38.15264900000011],[-108.57361800000001,38.152818999999965],[-108.60588099999995,38.15300700000006],[-108.6256489999999,38.15329800000018],[-108.63157799999999,38.153095000000064],[-108.66825499999993,38.15310100000022],[-108.68665299999986,38.153053],[-108.70512399999996,38.15323500000011],[-108.723503,38.15313200000014],[-108.74185599999993,38.15316000000013],[-108.750654,38.15342399999997],[-108.77827400000001,38.15326499999992],[-108.7922929999998,38.15343400000006],[-108.82151399999987,38.15363100000002],[-108.83628099999999,38.15386800000027],[-108.85583099999985,38.153909999999996],[-108.85902699999997,38.15384800000021],[-108.87565799999987,38.15415100000024],[-108.88067199999995,38.15402400000005],[-108.90022999999985,38.15478100000013],[-108.90674899999982,38.154959000000304],[-108.90674899999982,38.15329200000002],[-108.91675900000001,38.15328800000009],[-108.92938700000002,38.15358300000031],[-108.94214199999993,38.15371000000016],[-108.94940500000001,38.153503000000114],[-108.96598,38.153477000000066],[-108.97957799999989,38.15366200000028],[-108.98849799999994,38.153447000000256],[-108.9984849999999,38.153473000000076],[-109.01820900000001,38.15322899999995],[-109.04166700000002,38.153021000000024],[-109.04206199999993,38.155489999999986],[-109.04176199999989,38.164690000000064],[-109.050659,38.22008700000026],[-109.05467099999993,38.245080000000144],[-109.05612599999989,38.253672000000165],[-109.06006199999996,38.27548899999999],[-109.06012499999991,38.345994000000076],[-109.06016999999991,38.38810000000012],[-109.06002999999998,38.435584000000006],[-109.06017599999996,38.4623000000002],[-109.05993899999999,38.474274000000264],[-109.06011899999999,38.4999890000002],[-108.98935900000004,38.49998500000004],[-108.93294299999997,38.49998799999992],[-108.89661000000001,38.49998700000003],[-108.8150639999999,38.49998700000003],[-108.75910199999981,38.49998700000003],[-108.71010100000001,38.49998700000003],[-108.6566749999999,38.49998799999992],[-108.60599899999994,38.49998799999992],[-108.53666699999997,38.49998700000003],[-108.50064299999997,38.49998599999998],[-108.41970899999995,38.49998700000003],[-108.37913599999996,38.49998700000003],[-108.3786649999999,38.50219700000008],[-108.37864399999995,38.552592000000004],[-108.37863099999993,38.5854800000003],[-108.3787109999999,38.593795000000114],[-108.37869799999999,38.6680970000001],[-108.37627499999996,38.66812200000004],[-108.30350399999998,38.66831400000012],[-108.26107099999996,38.66842100000008],[-108.18716199999989,38.668442000000084],[-108.15012899999999,38.668510000000026],[-108.13296400000002,38.66839900000008],[-108.11350599999997,38.66852900000015],[-108.07733999999994,38.66848500000009],[-108.05960799999997,38.66839199999998],[-108.00402400000002,38.6684020000003],[-107.99888899999996,38.668476000000055],[-107.92980799999998,38.668304000000035],[-107.89265,38.66814400000027],[-107.85308299999997,38.668165999999985],[-107.83378499999992,38.66810399999997],[-107.77811500000001,38.66798],[-107.75119199999995,38.66805100000022],[-107.74917700000003,38.66826000000009],[-107.72244599999988,38.66828000000015],[-107.66082299999994,38.668378000000075],[-107.61010499999998,38.66843899999998],[-107.53608499999996,38.66851800000012],[-107.52087099999994,38.66856799999999],[-107.500655,38.66855300000003]]]},"properties":{"OBJECTID":44,"COUNTYFP":"085","Shape_Leng":4.58934792671,"Shape_Area":0.599005197875,"low":12235,"lowmod":19890,"LMMI":27540,"lowmoduniv":40250,"lowmod_pct":0.494161490683,"NAMELSAD10":"Montrose County"}},
{"type":"Feature","geometry":{"type":"Polygon","coordinates":[[[-103.58184399999993,40.52338300000008],[-103.58157299999999,40.52318400000013],[-103.543181,40.52339400000011],[-103.50271699999996,40.52328600000004],[-103.49299699999989,40.52309800000012],[-103.49086599999998,40.52317700000009],[-103.46719999999999,40.52325300000018],[-103.46656699999994,40.49430000000018],[-103.46630800000003,40.479820000000075],[-103.4658859999999,40.47259000000014],[-103.46606199999991,40.46542700000009],[-103.46589999999992,40.45842900000014],[-103.46524699999998,40.45750300000026],[-103.46537699999999,40.43634600000013],[-103.46448499999997,40.406242000000134],[-103.464537,40.39304000000027],[-103.46477299999992,40.36409400000014],[-103.46478299999995,40.354934000000014],[-103.46462500000001,40.34992000000011],[-103.47098199999994,40.34975900000012],[-103.47120999999987,40.3333990000001],[-103.471004,40.33270500000003],[-103.47139699999997,40.29177900000025],[-103.47180300000002,40.24997900000028],[-103.472331,40.24508000000009],[-103.47227099999992,40.24235700000003],[-103.47259799999978,40.23862800000006],[-103.4725949999999,40.23315900000017],[-103.47232099999997,40.22718700000007],[-103.472624,40.21713700000021],[-103.47317699999991,40.193846000000065],[-103.4733849999999,40.18224800000007],[-103.47348399999998,40.166652],[-103.47328199999998,40.156769999999995],[-103.47348099999994,40.15177800000015],[-103.47320100000002,40.146760000000256],[-103.47234400000002,40.10345500000028],[-103.47209599999991,40.088481],[-103.47226099999989,40.05961100000013],[-103.47210199999995,40.04296100000022],[-103.47217999999992,40.0306580000003],[-103.47199,40.001502000000016],[-103.47822999999994,40.001408000000254],[-103.50729499999994,40.00139100000007],[-103.53552999999994,40.001427000000035],[-103.5664319999999,40.00139799999994],[-103.56695500000001,40.00119100000023],[-103.57636099999996,40.0015480000003],[-103.59610599999996,40.001338000000146],[-103.62962499999998,40.00138500000014],[-103.62981200000002,40.00112200000001],[-103.65839999999992,40.001129000000105],[-103.67917899999992,40.001207000000136],[-103.70570099999992,40.00136600000013],[-103.71679699999999,40.00140699999997],[-103.729803,40.00121200000001],[-103.73379299999999,40.00133600000015],[-103.78762399999988,40.00132000000025],[-103.86146699999989,40.001215000000116],[-103.86364399999991,40.00147000000004],[-103.86774300000002,40.001198999999986],[-103.89091000000002,40.001221000000044],[-103.94653499999998,40.00117200000011],[-103.95460099999991,40.001132000000155],[-103.99767799999995,40.00115100000011],[-104.0005049999998,40.001059],[-104.03714999999994,40.001125000000286],[-104.03905399999985,40.00106700000009],[-104.07557600000001,40.001031000000125],[-104.077024,40.000974999999926],[-104.15033199999999,40.000856000000226],[-104.15029199999987,40.04424000000017],[-104.1501429999999,40.116737],[-104.14990799999998,40.16012000000006],[-104.14988299999987,40.172302000000286],[-104.14995499999998,40.203588000000025],[-104.15005799999977,40.217968999999925],[-104.15008899999987,40.232768000000135],[-104.14997499999993,40.249979999999994],[-104.15026199999988,40.318661000000304],[-104.15049399999998,40.3502450000002],[-104.147536,40.3502450000002],[-104.14729499999999,40.37392200000028],[-104.14738999999992,40.37976500000002],[-104.14715199999995,40.40069800000026],[-104.14690699999994,40.40925600000014],[-104.14652799999993,40.43043500000027],[-104.14674500000001,40.43766599999992],[-104.1468979999998,40.452303000000256],[-104.14730999999978,40.48112100000026],[-104.14766700000001,40.50996200000003],[-104.14771400000001,40.52438900000004],[-104.09430299999997,40.524419000000194],[-104.07063999999991,40.52434700000009],[-104.04663600000003,40.52438700000005],[-104.01295099999993,40.52432799999997],[-103.99371999999994,40.52417900000023],[-103.93664499999988,40.52395000000013],[-103.89838999999995,40.523874000000035],[-103.86028599999997,40.523941000000264],[-103.80336499999987,40.52417500000007],[-103.71488699999998,40.522387000000094],[-103.6962509999999,40.52202400000027],[-103.68674499999986,40.52208899999994],[-103.6579769999999,40.52246800000023],[-103.6198,40.52286700000002],[-103.5818369999999,40.5231600000003],[-103.58184399999993,40.52338300000008]]]},"properties":{"OBJECTID":45,"COUNTYFP":"087","Shape_Leng":2.41528112635,"Shape_Area":0.354778317765,"low":7350,"lowmod":13100,"LMMI":19145,"lowmoduniv":27825,"lowmod_pct":0.470799640611,"NAMELSAD10":"Morgan County"}},
{"type":"Feature","geometry":{"type":"Polygon","coordinates":[[[-103.39993799999996,38.26543499999997],[-103.39987699999989,38.25000400000005],[-103.39959999999996,38.24114899999995],[-103.39923499999998,38.18820200000016],[-103.39877899999999,38.125010000000145],[-103.39897199999996,38.121727000000305],[-103.39857999999987,38.114491000000214],[-103.39874399999985,38.10146700000013],[-103.39885699999985,38.098945000000015],[-103.39865299999997,38.082745000000045],[-103.3985459999999,38.04626600000017],[-103.39866599999988,38.038392999999985],[-103.39867499999991,38.02239500000002],[-103.39882399999999,38.013904000000196],[-103.39884399999994,37.99430600000011],[-103.39894199999998,37.98952200000008],[-103.39833399999992,37.98175099999992],[-103.39837299999988,37.979783],[-103.39810799999998,37.960627000000045],[-103.398236,37.959118000000274],[-103.39911999999987,37.94014900000002],[-103.39978100000002,37.89961600000004],[-103.3998739999999,37.875063000000125],[-103.39983599999994,37.86652700000013],[-103.399854,37.82281200000017],[-103.40368199999995,37.822792000000106],[-103.40349600000002,37.75907400000011],[-103.40343399999995,37.75001100000003],[-103.40365199999997,37.73433799999998],[-103.40379599999994,37.70166500000005],[-103.40438499999993,37.68689000000023],[-103.40391699999992,37.673168000000146],[-103.40396299999992,37.66296300000005],[-103.40442599999994,37.65820300000007],[-103.40433899999994,37.64357600000005],[-103.42708799999997,37.643658000000244],[-103.451799,37.64378799999997],[-103.485323,37.643502000000126],[-103.50049799999988,37.64293600000019],[-103.51265799999987,37.64312200000012],[-103.55498899999998,37.64344100000005],[-103.56227899999982,37.643538999999976],[-103.62550199999998,37.64397300000002],[-103.67808500000001,37.64385199999998],[-103.75050699999986,37.643557999999985],[-103.75081299999994,37.643688999999995],[-103.78465699999987,37.64413000000002],[-103.83338300000003,37.64477299999993],[-103.85524299999997,37.644738000000075],[-103.857124,37.64482600000025],[-103.875498,37.64445500000011],[-103.9536829999999,37.64405700000003],[-103.99206700000002,37.64385800000008],[-104.000516,37.64405999999991],[-104.03283299999998,37.64402900000022],[-104.05819599999995,37.64404900000011],[-104.05871699999989,37.688198000000284],[-104.05929000000003,37.734172000000285],[-104.06113199999987,37.73470400000025],[-104.06172699999996,37.75001100000003],[-104.06168599999995,37.7630410000001],[-104.062073,37.79212300000006],[-104.06189099999995,37.82271500000013],[-104.06006899999983,37.823030000000074],[-104.05947200000003,37.87090100000006],[-104.05937399999988,37.87500899999992],[-104.05937399999988,37.90010000000012],[-104.059323,37.93019700000008],[-104.05935699999998,37.94573000000008],[-104.05931599999997,37.95863900000006],[-104.059459,37.979810999999984],[-104.05938300000003,37.990769],[-104.05950100000001,37.996583000000214],[-104.05939199999995,38.054326],[-104.05908499999998,38.07628999999997],[-104.05904699999991,38.103723000000286],[-104.05880999999994,38.10544300000015],[-104.05868799999996,38.1148290000001],[-104.057995,38.141468000000145],[-104.058242,38.14649199999997],[-104.05595399999999,38.14601900000008],[-104.05040599999995,38.14654900000005],[-104.047913,38.14523800000012],[-104.04704500000003,38.141389000000174],[-104.04398399999991,38.141594000000055],[-104.03621599999997,38.14139699999993],[-104.03312799999998,38.14057700000012],[-104.02719000000002,38.140489000000116],[-104.02215899999993,38.14095099999997],[-104.01943499999987,38.14185800000007],[-104.015061,38.14280900000006],[-104.01022699999999,38.145737000000054],[-104.00775099999993,38.145547000000306],[-104.00510600000001,38.143541000000084],[-104.00256100000001,38.14277000000021],[-104.00096799999994,38.14333199999999],[-103.99922800000002,38.14578300000011],[-103.99828699999989,38.146256000000164],[-103.99469599999986,38.14665400000024],[-103.99120299999987,38.14543300000008],[-103.98759899999999,38.142776000000026],[-103.9842129999999,38.14091400000012],[-103.98070399999989,38.140579000000116],[-103.9777529999999,38.141068000000075],[-103.97499099999999,38.140336000000275],[-103.9741279999999,38.13862700000027],[-103.97695699999986,38.134348000000045],[-103.976407,38.13298999999995],[-103.97437799999989,38.132119000000046],[-103.97003000000001,38.13161700000023],[-103.96678500000002,38.131674000000146],[-103.96015499999999,38.130401000000006],[-103.95494699999995,38.13081200000005],[-103.95126999999991,38.13020400000005],[-103.94823199999979,38.12815800000021],[-103.94458600000002,38.126687000000175],[-103.93398899999994,38.12441700000011],[-103.93011999999993,38.12500500000027],[-103.92904099999998,38.12732800000009],[-103.92593,38.12964600000004],[-103.92339900000002,38.13042400000029],[-103.92015200000003,38.12927400000001],[-103.91990299999992,38.127531000000204],[-103.91868699999986,38.12554200000011],[-103.91630199999997,38.12472900000017],[-103.91439600000001,38.124760000000094],[-103.91098,38.12656300000003],[-103.91087399999992,38.12794500000001],[-103.91173300000003,38.129885000000286],[-103.91352499999988,38.13162400000027],[-103.91250699999995,38.13434600000005],[-103.90619400000003,38.13485999999995],[-103.902064,38.133975000000135],[-103.90023899999989,38.13301000000001],[-103.89851199999998,38.13113200000009],[-103.89904599999994,38.128473000000156],[-103.90028899999993,38.127281000000096],[-103.9006619999999,38.125409000000104],[-103.895059,38.1247400000002],[-103.89252999999985,38.12524300000024],[-103.88684599999993,38.12527799999992],[-103.88368199999996,38.124577000000045],[-103.87800099999987,38.12464899999992],[-103.87324899999999,38.12629900000002],[-103.87094099999996,38.127808000000016],[-103.86867199999989,38.12880900000005],[-103.86626199999989,38.128760000000284],[-103.86348999999996,38.1280720000002],[-103.86028599999997,38.1261320000001],[-103.85650599999985,38.125185999999985],[-103.85508299999987,38.12550399999998],[-103.85033499999997,38.12504800000028],[-103.84621099999998,38.12340500000022],[-103.84459200000003,38.12079399999999],[-103.84168699999992,38.117346],[-103.83770299999998,38.114163000000076],[-103.83561599999996,38.113339999999994],[-103.83144799999991,38.11420500000003],[-103.82870999999983,38.11441000000008],[-103.82590799999991,38.11306200000013],[-103.82186599999994,38.113516000000004],[-103.81859099999997,38.114835000000255],[-103.81376499999999,38.117418999999984],[-103.8119099999999,38.1178450000001],[-103.8098179999999,38.117508999999984],[-103.80731600000001,38.116400000000056],[-103.805028,38.11457799999994],[-103.80303300000003,38.11253300000021],[-103.79126699999995,38.112437000000114],[-103.77033,38.11292500000013],[-103.750674,38.113214000000255],[-103.747591,38.113074999999924],[-103.70132699999988,38.11315300000018],[-103.67373099999998,38.11290600000018],[-103.62549999999993,38.11333500000012],[-103.61917199999993,38.11335500000001],[-103.619011,38.143152000000214],[-103.61928299999988,38.17145500000004],[-103.60107499999987,38.17171900000022],[-103.58393499999994,38.17271500000021],[-103.56033300000001,38.17276400000003],[-103.55947300000003,38.172672000000205],[-103.53579099999996,38.17265500000002],[-103.509364,38.17251400000009],[-103.5094219999998,38.19295699999992],[-103.50955199999999,38.204075000000046],[-103.50955299999998,38.21621299999998],[-103.50971099999992,38.232119000000125],[-103.50987899999996,38.26409100000001],[-103.51006000000001,38.265169000000185],[-103.50176399999998,38.265015999999946],[-103.43730600000004,38.265031000000135],[-103.43012799999997,38.26516400000031],[-103.41835600000002,38.265162000000146],[-103.40981999999985,38.265442000000064],[-103.39993799999996,38.26543499999997]]]},"properties":{"OBJECTID":46,"COUNTYFP":"089","Shape_Leng":2.65281250003,"Shape_Area":0.33687737305,"low":6990,"lowmod":10705,"LMMI":14130,"lowmoduniv":18060,"lowmod_pct":0.592746400886,"NAMELSAD10":"Otero County"}},
{"type":"Feature","geometry":{"type":"Polygon","coordinates":[[[-107.63503999999995,38.301896],[-107.63398599999977,38.30003100000005],[-107.63169899999991,38.297259000000054],[-107.631464,38.29563200000024],[-107.62962499999998,38.29415700000004],[-107.63008299999996,38.29229200000009],[-107.62938500000001,38.29032700000005],[-107.62923599999988,38.287861000000305],[-107.62655699999999,38.28596999999996],[-107.625269,38.28303200000005],[-107.62447700000001,38.27971100000008],[-107.62316499999991,38.279028000000096],[-107.619937,38.27527199999997],[-107.61713699999996,38.27545000000015],[-107.61705099999995,38.27250700000002],[-107.61472499999985,38.270106999999996],[-107.61535500000002,38.26897100000002],[-107.61422099999999,38.26714599999997],[-107.61270899999994,38.26358700000003],[-107.61271699999998,38.26216000000011],[-107.61157000000003,38.26082700000029],[-107.60699,38.25786400000027],[-107.60206,38.25549600000011],[-107.60015900000002,38.25389499999994],[-107.5974139999999,38.253394999999955],[-107.593951,38.25352300000009],[-107.59070099999985,38.25204600000012],[-107.588821,38.24958800000013],[-107.588619,38.24676099999999],[-107.58664999999996,38.243029999999976],[-107.58747999999997,38.240228],[-107.58490799999998,38.23625500000003],[-107.58268199999992,38.23460299999999],[-107.58211299999999,38.23113900000004],[-107.580285,38.23037199999999],[-107.57986799999986,38.22785099999999],[-107.58037200000001,38.22496799999999],[-107.57901499999991,38.22266300000024],[-107.57953900000001,38.22028800000004],[-107.58087,38.218090000000075],[-107.57994299999996,38.215930000000014],[-107.57996500000002,38.214125999999965],[-107.58097799999996,38.212685000000306],[-107.57871,38.21200699999997],[-107.57739399999991,38.2104350000003],[-107.57557600000001,38.209797000000265],[-107.57508000000001,38.205625000000225],[-107.57583999999991,38.202725999999984],[-107.57535299999995,38.19947100000013],[-107.574364,38.19835900000015],[-107.57432499999999,38.196772000000294],[-107.573283,38.196175000000096],[-107.57350000000002,38.194773999999995],[-107.57261199999999,38.194064000000026],[-107.57207499999998,38.19014099999998],[-107.57246499999991,38.188852999999995],[-107.57189599999992,38.18611400000003],[-107.57079599999986,38.18396300000006],[-107.57025799999991,38.181353000000115],[-107.5680109999999,38.18081100000006],[-107.56804399999999,38.179538000000036],[-107.56656599999997,38.17664500000012],[-107.567993,38.17146800000006],[-107.56652299999996,38.170741000000305],[-107.56631099999993,38.168728000000044],[-107.56429200000002,38.16666399999997],[-107.56329800000003,38.162120000000016],[-107.56218899999993,38.158792000000005],[-107.56291799999997,38.15562700000015],[-107.56320799999986,38.15289700000022],[-107.56473299999993,38.14982199999997],[-107.56861399999997,38.14877000000001],[-107.56862000000001,38.147267],[-107.56869799999998,38.147167000000024],[-107.56873599999994,38.147126000000014],[-107.56923999999998,38.146596000000045],[-107.57000700000003,38.145421],[-107.57110099999994,38.14389100000005],[-107.57178799999997,38.14181400000001],[-107.571911,38.141427000000135],[-107.57249300000001,38.140872],[-107.572587,38.13988999999998],[-107.57272099999989,38.13885900000008],[-107.57298699999996,38.13805300000001],[-107.57341600000001,38.13793300000003],[-107.57386600000001,38.137329000000136],[-107.57403999999997,38.136330999999984],[-107.57352199999997,38.13495300000005],[-107.57228600000002,38.135073000000034],[-107.56944999999996,38.13430500000004],[-107.56920500000001,38.13404700000001],[-107.56966499999999,38.13328199999995],[-107.56951300000003,38.1328630000001],[-107.56883700000003,38.13199199999997],[-107.568667,38.131598000000054],[-107.56756699999994,38.130187000000035],[-107.5669759999999,38.129679000000124],[-107.56518099999988,38.12834100000009],[-107.56456000000003,38.1273020000001],[-107.56433700000002,38.125851000000125],[-107.563581,38.124219000000096],[-107.56320499999998,38.12321200000008],[-107.56363499999986,38.12182800000011],[-107.56396999999987,38.11963400000013],[-107.56423799999993,38.11906400000004],[-107.5639799999999,38.117808000000025],[-107.56375500000001,38.11645500000003],[-107.56174099999998,38.11390399999999],[-107.56138199999998,38.112497000000076],[-107.56102799999996,38.111005999999975],[-107.56134299999985,38.10831900000005],[-107.56131399999992,38.107357000000036],[-107.56222999999994,38.105187000000114],[-107.56251399999991,38.1048110000001],[-107.562501,38.103566000000114],[-107.56248099999993,38.10263700000013],[-107.562252,38.10193099999998],[-107.56275700000003,38.10082200000005],[-107.56307199999998,38.10043000000013],[-107.564164,38.10001599999998],[-107.56474600000001,38.099555000000066],[-107.56539399999991,38.09886800000004],[-107.56531299999995,38.09838199999996],[-107.564752,38.097453999999914],[-107.56464999999992,38.09699200000006],[-107.56369199999995,38.095978],[-107.56288,38.095337000000086],[-107.56151599999993,38.09378400000003],[-107.56101000000001,38.09313100000003],[-107.56046200000003,38.09207300000003],[-107.55994699999997,38.091346999999985],[-107.56017300000002,38.08899900000006],[-107.55954999999989,38.08720499999998],[-107.55784900000003,38.086148000000094],[-107.55653699999999,38.083625999999924],[-107.555838,38.08250999999996],[-107.55519599999997,38.08154799999994],[-107.55454500000002,38.08006899999998],[-107.55436499999996,38.07985200000013],[-107.553585,38.079200000000014],[-107.55285100000003,38.07913400000007],[-107.55128699999995,38.078815000000134],[-107.55028899999996,38.078236000000004],[-107.54825599999998,38.07874300000003],[-107.54725099999996,38.079425000000015],[-107.54481699999991,38.080526000000134],[-107.54437299999995,38.08073000000013],[-107.54290699999996,38.08013700000009],[-107.54245799999995,38.080115000000035],[-107.54154799999998,38.07974800000005],[-107.53739000000002,38.0794350000001],[-107.53658199999995,38.07951299999996],[-107.536022,38.07944100000009],[-107.5339489999999,38.07992300000001],[-107.532648,38.07924300000002],[-107.53243199999997,38.07885199999998],[-107.531227,38.07798000000014],[-107.530259,38.07700499999993],[-107.52974399999994,38.07676400000008],[-107.52913999999998,38.07683700000007],[-107.528188,38.076081000000045],[-107.52727900000002,38.07566500000013],[-107.52646599999997,38.075920999999994],[-107.52574099999993,38.07506300000006],[-107.52494999999993,38.073945000000094],[-107.52332999999993,38.07204900000005],[-107.52208199999995,38.07170100000002],[-107.521522,38.07120100000003],[-107.52053999999993,38.07042000000007],[-107.519139,38.07006200000012],[-107.51649099999992,38.06939699999998],[-107.51567899999998,38.068287],[-107.51527199999998,38.0669400000001],[-107.51511600000003,38.06609700000001],[-107.51500399999992,38.06525499999998],[-107.51388099999997,38.063535000000286],[-107.51255399999997,38.06264500000003],[-107.51087999999993,38.06087700000012],[-107.51129899999995,38.06025200000005],[-107.511819,38.05998499999998],[-107.51334699999995,38.05924500000009],[-107.51481999999999,38.058667000000014],[-107.51529799999997,38.05826899999994],[-107.51639999999998,38.05785499999996],[-107.51781199999999,38.05810100000008],[-107.51895899999994,38.05817200000013],[-107.519768,38.05784399999999],[-107.52082599999989,38.05750200000011],[-107.52182399999992,38.05719200000004],[-107.52289899999988,38.05613200000005],[-107.52245199999993,38.05471100000011],[-107.521728,38.05334399999998],[-107.521071,38.05218000000008],[-107.52180499999997,38.05044399999997],[-107.52220799999992,38.04933399999999],[-107.52256599999998,38.04840199999995],[-107.52262699999994,38.047957999999994],[-107.520985,38.046627],[-107.52077400000002,38.0455740000001],[-107.52158199999991,38.045067000000074],[-107.52233699999994,38.04416400000014],[-107.522152,38.04376600000006],[-107.52195499999993,38.04256700000002],[-107.52223200000003,38.041930000000036],[-107.52298000000002,38.041045],[-107.52449399999989,38.03953799999999],[-107.52576299999998,38.039353000000006],[-107.52702699999986,38.03987800000027],[-107.53055699999999,38.03898799999996],[-107.53112899999996,38.038963000000024],[-107.53179599999999,38.036992000000055],[-107.53349000000003,38.03564000000006],[-107.53449799999993,38.03439300000014],[-107.535055,38.034101000000135],[-107.53674999999998,38.03316200000006],[-107.5372799999999,38.03270800000013],[-107.53750199999996,38.03215199999994],[-107.53747900000002,38.03204800000009],[-107.53694899999994,38.031112000000064],[-107.53685200000001,38.02868600000005],[-107.538882,38.028235999999936],[-107.53969599999994,38.0269540000001],[-107.53940299999994,38.02594000000005],[-107.53824599999996,38.02479299999999],[-107.53737799999999,38.02436299999994],[-107.53658899999999,38.0227440000001],[-107.53699699999993,38.021399999999915],[-107.53756599999991,38.01970199999994],[-107.53871700000002,38.01888500000001],[-107.53942699999999,38.01771500000007],[-107.539556,38.01607599999994],[-107.539353,38.014707000000044],[-107.53827899999993,38.01392600000008],[-107.53791799999999,38.01230500000008],[-107.53806099999986,38.01098100000013],[-107.53787799999998,38.009596999999985],[-107.53873299999992,38.00833899999998],[-107.53938499999998,38.007919999999956],[-107.54070599999994,38.00723400000004],[-107.54274499999997,38.00635600000004],[-107.54376200000002,38.00510800000001],[-107.5450259999999,38.00334700000013],[-107.54538300000002,38.00289100000009],[-107.54607699999997,38.00149399999992],[-107.54631699999993,38.000826000000075],[-107.54717299999999,37.99999500000007],[-107.548588,37.998852],[-107.55058600000001,37.998356],[-107.55361999999997,37.999348],[-107.555792,37.9991040000001],[-107.55650600000001,37.99916200000007],[-107.55724599999996,37.99947199999997],[-107.55965799999996,38.00098400000002],[-107.559842,38.00111600000014],[-107.561466,38.00277700000004],[-107.56286899999998,38.003417999999954],[-107.56389799999994,38.00391599999995],[-107.56546700000001,38.004898000000026],[-107.56600100000003,38.00519500000013],[-107.56793799999991,38.005203999999935],[-107.572904,38.00614000000013],[-107.57426899999996,38.006190000000004],[-107.57432599999999,38.00592400000005],[-107.57430599999992,38.00499400000001],[-107.57465399999995,38.00448200000011],[-107.57576799999993,38.003898000000106],[-107.57681599999995,38.00309600000014],[-107.57747799999993,38.00265200000001],[-107.57805799999994,38.00226400000008],[-107.57993299999993,38.00092999999998],[-107.580421,38.00054800000004],[-107.58095500000002,38.000375000000076],[-107.58148799999998,38.000255000000095],[-107.582111,38.000221000000124],[-107.58268199999992,38.00020500000005],[-107.58380599999992,38.00013300000012],[-107.58419299999997,38.00011100000012],[-107.58692999999994,37.99896800000005],[-107.58667899999995,37.99824999999993],[-107.58658999999989,37.997176999999965],[-107.58666,37.99412799999993],[-107.58746799999994,37.99344000000002],[-107.59136699999999,37.99225200000001],[-107.58980799999995,37.98962900000021],[-107.58973900000001,37.988444000000015],[-107.58890600000001,37.98465500000003],[-107.58756199999993,37.982038999999986],[-107.58690999999999,37.98059000000001],[-107.58583099999998,37.979457000000025],[-107.58512300000001,37.97878200000031],[-107.58416499999998,37.97601900000012],[-107.58571399999988,37.974092999999925],[-107.58643099999995,37.97319800000014],[-107.58775500000002,37.97122200000007],[-107.58822099999992,37.97031200000009],[-107.58700399999992,37.96847800000023],[-107.58463499999993,37.967997000000025],[-107.58298399999995,37.967922999999985],[-107.58029599999986,37.968474999999955],[-107.57655699999992,37.967520000000036],[-107.576209,37.96742400000005],[-107.57494399999996,37.96714599999996],[-107.5736399999999,37.96714599999996],[-107.57268699999992,37.96716600000002],[-107.57167400000003,37.96644700000007],[-107.57073399999996,37.96602700000011],[-107.56903499999999,37.96603900000002],[-107.56931599999996,37.96501399999994],[-107.5706429999999,37.965013000000056],[-107.57124199999998,37.965012],[-107.57187299999998,37.965012],[-107.59996899999993,37.96498800000012],[-107.60562400000003,37.96498400000013],[-107.61296099999993,37.964979000000085],[-107.62228999999991,37.964970999999935],[-107.62407899999994,37.96497000000005],[-107.62540100000001,37.964981999999964],[-107.64158800000001,37.965009000000066],[-107.64168999999993,37.96489599999995],[-107.64126699999986,37.9631930000001],[-107.64041799999995,37.96201600000006],[-107.64089100000001,37.95927000000023],[-107.639928,37.95785900000004],[-107.64007999999995,37.957479000000035],[-107.640875,37.95702600000004],[-107.64147400000002,37.955839000000026],[-107.64211499999993,37.95520800000003],[-107.64288499999998,37.953134000000034],[-107.64273300000002,37.94974400000007],[-107.64091099999996,37.94733200000013],[-107.64099099999993,37.94667800000002],[-107.64103099999994,37.94650900000005],[-107.64167900000001,37.944667000000095],[-107.641839,37.94332700000007],[-107.64115200000003,37.94195600000012],[-107.64102899999995,37.941633000000024],[-107.640513,37.93939800000021],[-107.641886,37.93796000000009],[-107.64213000000001,37.937499000000116],[-107.64284199999997,37.9366110000002],[-107.64252799999991,37.93321200000014],[-107.64102100000002,37.93053400000008],[-107.64000900000002,37.9295340000001],[-107.63925499999999,37.92587000000003],[-107.63784299999998,37.924346000000014],[-107.63746399999997,37.92356400000017],[-107.637926,37.92085900000018],[-107.63895499999995,37.920220000000086],[-107.64028899999994,37.91961300000008],[-107.64108299999998,37.918990000000065],[-107.64230600000002,37.91837499999997],[-107.64590399999992,37.917410000000075],[-107.64679899999999,37.91665000000006],[-107.64641799999998,37.91531900000007],[-107.64794199999994,37.91317700000013],[-107.648799,37.91300700000011],[-107.649092,37.91212700000011],[-107.64930000000004,37.9115020000001],[-107.64974199999995,37.91059200000012],[-107.65162599999996,37.90971800000011],[-107.651881,37.909589000000096],[-107.654091,37.908682],[-107.65669099999991,37.90799199999992],[-107.65776199999988,37.908071000000064],[-107.66080499999987,37.909398000000124],[-107.66111099999995,37.909503000000086],[-107.66159600000003,37.90855699999997],[-107.66222600000003,37.90773400000012],[-107.66227499999997,37.90719300000012],[-107.66192899999987,37.90700300000003],[-107.66170299999999,37.90692799999999],[-107.66155400000002,37.90643],[-107.66281299999991,37.905471999999975],[-107.66365399999995,37.90380800000014],[-107.66375600000003,37.90351700000008],[-107.66539699999993,37.90344200000004],[-107.66787999999985,37.90301599999998],[-107.66794399999998,37.903019000000086],[-107.67160499999994,37.90288600000025],[-107.672911,37.903279000000055],[-107.67373799999996,37.903543999999954],[-107.67606299999994,37.903638000000115],[-107.67902199999997,37.90426300000013],[-107.68035799999996,37.904124000000024],[-107.68330399999991,37.9009200000001],[-107.684823,37.90085500000009],[-107.688606,37.901769],[-107.689484,37.901203000000066],[-107.68959199999995,37.89979800000003],[-107.69051399999995,37.89824699999991],[-107.69085999999999,37.89804500000008],[-107.69225599999993,37.89772800000014],[-107.69252699999998,37.89654900000011],[-107.69315399999999,37.894803999999965],[-107.69338799999997,37.89453700000007],[-107.69396699999993,37.893712999999934],[-107.6956649999999,37.892410000000154],[-107.69671599999992,37.89274000000012],[-107.69734599999992,37.892776000000026],[-107.70063899999997,37.892007000000035],[-107.70134399999995,37.892223000000115],[-107.70169299999998,37.89294100000001],[-107.70509099999992,37.89396899999997],[-107.70715999999987,37.89363399999996],[-107.70761500000003,37.895514000000105],[-107.70815900000002,37.896918000000085],[-107.70761500000003,37.89861400000012],[-107.70777900000002,37.89885600000002],[-107.70910500000002,37.899249000000054],[-107.70957499999992,37.89940100000001],[-107.71193099999999,37.89886799999999],[-107.71221499999996,37.89888600000006],[-107.71342599999997,37.8983990000001],[-107.714137,37.89808100000005],[-107.71576800000003,37.89774299999999],[-107.71663299999994,37.89708799999994],[-107.71816299999995,37.897360000000106],[-107.71879399999995,37.89719700000012],[-107.71924599999994,37.897721000000104],[-107.71984599999996,37.897898],[-107.72086399999989,37.89742000000007],[-107.72325599999999,37.89882800000004],[-107.72368399999999,37.898762999999974],[-107.724674,37.89909100000011],[-107.72590000000002,37.89982299999997],[-107.72642199999996,37.900347000000124],[-107.72741399999995,37.90111200000001],[-107.72956199999999,37.90267300000005],[-107.73162599999995,37.90370200000007],[-107.7342369999999,37.90436799999992],[-107.73727099999991,37.904764],[-107.73828299999991,37.905432000000076],[-107.73667999999998,37.907728000000304],[-107.73637099999996,37.90950500000025],[-107.73512299999993,37.91129100000023],[-107.73695099999992,37.91456500000004],[-107.73707299999995,37.91707500000007],[-107.73582399999992,37.91846600000008],[-107.73315899999994,37.920320000000174],[-107.73331799999988,37.92206299999998],[-107.73513300000002,37.92444900000021],[-107.73330699999985,37.92694699999993],[-107.73278799999997,37.92997500000001],[-107.73605900000001,37.932279999999935],[-107.73832699999997,37.93529100000006],[-107.73857299999992,37.93821300000002],[-107.73914799999989,37.939261000000045],[-107.74240699999996,37.94036799999998],[-107.7438929999999,37.942044000000294],[-107.74431600000003,37.94579700000014],[-107.74585899999994,37.948780000000056],[-107.747344,37.95023000000009],[-107.74889699999977,37.953116000000136],[-107.75575400000002,37.95464600000008],[-107.75889099999989,37.958360000000255],[-107.76085499999994,37.95963200000017],[-107.76360099999994,37.95920900000016],[-107.765783,37.96095100000008],[-107.76930099999976,37.96198100000004],[-107.77134199999989,37.96167100000014],[-107.77606199999997,37.96201100000019],[-107.77867500000002,37.96094799999997],[-107.78041200000001,37.961879000000124],[-107.78157199999998,37.96321400000005],[-107.78244799999999,37.965734000000054],[-107.78460199999995,37.96861400000029],[-107.785368,37.97101200000009],[-107.78685299999995,37.97303700000015],[-107.78917999999993,37.97527300000024],[-107.78708199999988,37.977205000000026],[-107.78684799999996,37.97851000000003],[-107.788862,37.980614],[-107.79172699999998,37.982354999999984],[-107.79330899999997,37.985333000000196],[-107.79398099999997,37.987278],[-107.79546899999997,37.98852000000028],[-107.79963999999995,37.98826400000024],[-107.80162599999994,37.98694400000011],[-107.80446199999994,37.988652],[-107.80806799999988,37.9892780000003],[-107.80979699999995,37.99014300000027],[-107.81114300000002,37.99001100000004],[-107.8143209999999,37.99067300000007],[-107.81560400000001,37.98939400000012],[-107.81690200000003,37.98919700000016],[-107.819481,37.98972400000025],[-107.82194300000003,37.98860200000013],[-107.82483299999996,37.98816300000004],[-107.82600599999984,37.988674999999944],[-107.82978099999991,37.98896300000001],[-107.83267799999999,37.99072700000028],[-107.83604700000001,37.992212999999936],[-107.83817399999998,37.992363000000296],[-107.84310499999992,37.991653000000156],[-107.84907999999996,37.9926080000003],[-107.84936799999997,37.99442699999997],[-107.8476819999999,37.99751800000013],[-107.84482499999996,37.998498999999924],[-107.842378,37.999993000000075],[-107.84149799999983,38.00113000000022],[-107.84143099999989,38.00288599999999],[-107.84229399999998,38.00582300000019],[-107.84445999999997,38.00590000000017],[-107.849131,38.008599000000004],[-107.85180300000002,38.008238000000176],[-107.85578399999997,38.00872099999998],[-107.857418,38.008357000000046],[-107.86099300000001,38.00876300000027],[-107.86241899999987,38.00943000000001],[-107.86593999999997,38.00801200000012],[-107.87356799999998,38.00681900000018],[-107.8749709999999,38.00689700000004],[-107.87865199999999,38.00823200000002],[-107.8832999999999,38.007371000000205],[-107.88639399999983,38.009250000000236],[-107.88971599999991,38.011615000000006],[-107.89104399999991,38.01353100000006],[-107.8893349999999,38.01761499999992],[-107.89138399999996,38.02253700000023],[-107.8907989999999,38.02353700000003],[-107.89136100000002,38.02515200000005],[-107.89561300000003,38.02573800000022],[-107.89919700000002,38.02571799999998],[-107.900193,38.026249000000234],[-107.89875099999978,38.029424000000006],[-107.89669200000003,38.031598000000145],[-107.89656400000001,38.03475300000008],[-107.89554699999991,38.03638500000011],[-107.89315599999998,38.03920000000011],[-107.89333199999987,38.041600000000074],[-107.89182800000003,38.04356400000006],[-107.88845700000002,38.04482200000024],[-107.8882079999999,38.04581200000007],[-107.89135599999997,38.04870699999998],[-107.89198399999998,38.05135500000023],[-107.89199899999988,38.05783700000012],[-107.89377400000001,38.057742000000076],[-107.893529,38.06070100000028],[-107.89292599999993,38.07247000000007],[-107.89671399999992,38.08682900000014],[-107.89205199999992,38.087011000000246],[-107.89253899999989,38.101346999999976],[-107.89152899999999,38.10136399999999],[-107.89161999999988,38.10796000000005],[-107.89146899999992,38.11595700000004],[-107.93113,38.115881000000115],[-107.95157399999994,38.11612500000024],[-107.95153099999993,38.12499000000008],[-107.95171700000003,38.144982000000084],[-107.96545300000002,38.14490300000011],[-107.96578899999997,38.15232800000001],[-107.94696299999998,38.152491],[-107.94774399999994,38.173767000000225],[-107.9381449999999,38.17393599999997],[-107.937588,38.21899200000007],[-107.9470199999999,38.21894700000007],[-107.94714199999987,38.22647700000016],[-107.96540799999997,38.226509000000306],[-107.965419,38.23401500000017],[-107.98418399999997,38.2339360000002],[-107.98408399999988,38.24838599999998],[-108.01153099999988,38.248203000000274],[-108.01153099999988,38.25526100000019],[-108.08660399999997,38.25520200000028],[-108.08721799999995,38.26255300000008],[-108.08634999999998,38.284334],[-108.10416099999992,38.28417900000011],[-108.10481900000002,38.31371800000028],[-108.12326899999988,38.31363499999998],[-108.12368300000003,38.32092800000015],[-108.13174799999996,38.32075600000013],[-108.13238000000001,38.33157300000005],[-108.12562799999984,38.33165500000024],[-108.10533799999996,38.33206900000022],[-108.08606399999996,38.33197500000006],[-108.07267899999994,38.33201600000007],[-108.059325,38.33191599999998],[-108.01168100000001,38.33196099999998],[-108.00062200000002,38.33249500000011],[-107.99693500000001,38.332477999999924],[-107.93103399999995,38.33263800000003],[-107.87561800000003,38.332778000000076],[-107.8665969999999,38.33220500000027],[-107.81763199999995,38.33224600000011],[-107.81268299999999,38.332221000000175],[-107.77077499999996,38.33223799999996],[-107.73643800000002,38.33221200000014],[-107.72767099999999,38.332325999999966],[-107.71729799999997,38.33214700000008],[-107.63569699999994,38.33214800000019],[-107.635423,38.33092400000015],[-107.63362699999999,38.329884000000106],[-107.63234,38.32686600000011],[-107.63096199999995,38.32567900000015],[-107.63366599999995,38.32212700000025],[-107.63374699999997,38.320480999999916],[-107.63574499999993,38.318269999999984],[-107.63649699999996,38.31615400000021],[-107.63577199999986,38.314714000000265],[-107.63532499999991,38.312144000000046],[-107.6329879999999,38.309518000000025],[-107.63372300000003,38.306467],[-107.63499299999995,38.303992999999934],[-107.63503999999995,38.301896]]]},"properties":{"OBJECTID":47,"COUNTYFP":"091","Shape_Leng":2.22545694614,"Shape_Area":0.144347883504,"low":1110,"lowmod":1725,"LMMI":2740,"lowmoduniv":4580,"lowmod_pct":0.376637554585,"NAMELSAD10":"Ouray County"}},
{"type":"Feature","geometry":{"type":"Polygon","coordinates":[[[-105.39894899999996,39.566056],[-105.398751,39.54118799999998],[-105.39885099999987,39.535588000000075],[-105.398551,39.52688800000021],[-105.39825099999996,39.51078799999999],[-105.39835099999988,39.50478900000013],[-105.39805100000001,39.48408800000027],[-105.39813900000001,39.46872500000029],[-105.39789799999988,39.46695700000021],[-105.398552,39.46348800000004],[-105.39851899999991,39.46011599999997],[-105.39806399999992,39.45225100000005],[-105.398052,39.41193299999992],[-105.3981569999998,39.39899500000024],[-105.39794999999992,39.38879100000025],[-105.398552,39.378089999999986],[-105.39845299999996,39.363088999999945],[-105.39814699999988,39.35069700000014],[-105.39845299999996,39.340289000000155],[-105.39865199999991,39.33707500000014],[-105.39825300000001,39.32718900000003],[-105.39865199999991,39.32618900000023],[-105.398052,39.3146040000002],[-105.39813900000001,39.305295],[-105.39804800000002,39.298427000000004],[-105.39671199999998,39.26488300000011],[-105.39789999999994,39.24999000000031],[-105.39783299999999,39.21113200000002],[-105.39788799999985,39.200616000000025],[-105.39772299999987,39.16151600000012],[-105.397875,39.12956399999996],[-105.37555199999997,39.12954300000001],[-105.33044399999994,39.12968500000022],[-105.33058900000003,39.11837800000029],[-105.33007199999997,39.05869400000029],[-105.32954799999987,38.99999100000008],[-105.32959700000004,38.9987440000001],[-105.32926299999991,38.96645899999993],[-105.32905799999992,38.95018500000009],[-105.32840799999997,38.892663000000084],[-105.32870099999985,38.88946900000013],[-105.32815299999999,38.875011000000256],[-105.32909499999977,38.86901999999998],[-105.328915,38.86767100000009],[-105.32892600000002,38.85772900000012],[-105.32878499999998,38.815963000000295],[-105.32898999999998,38.811216000000286],[-105.329159,38.79291500000022],[-105.329161,38.76733200000018],[-105.32906599999995,38.74988999999994],[-105.32879999999989,38.74455800000021],[-105.3291339999999,38.69720499999994],[-105.37554499999993,38.69751600000029],[-105.43429000000003,38.69755299999997],[-105.44838799999997,38.69758099999996],[-105.49824599999982,38.69712400000003],[-105.50054799999998,38.696641],[-105.51041599999996,38.69649500000003],[-105.51445799999999,38.69659000000024],[-105.53154699999988,38.69593500000002],[-105.57085699999988,38.69584300000008],[-105.58039200000002,38.695896000000005],[-105.613586,38.69534599999997],[-105.61617899999987,38.69541899999996],[-105.63423499999999,38.69515899999993],[-105.6494209999999,38.69505300000026],[-105.66739699999988,38.69482000000016],[-105.71345199999996,38.694675000000075],[-105.7268479999999,38.6945620000003],[-105.74609599999997,38.69459100000006],[-105.75058000000001,38.69469100000015],[-105.75957599999992,38.694589000000065],[-105.77730099999997,38.69460400000003],[-105.81383900000003,38.69450699999999],[-105.85085900000001,38.693924000000266],[-105.85950599999995,38.69399000000004],[-105.87556699999988,38.6943060000001],[-105.88157399999994,38.694118],[-105.88891199999995,38.695153000000175],[-105.89838999999989,38.69376900000003],[-105.92093799999998,38.693483000000015],[-105.94448799999998,38.6930890000001],[-105.96974999999998,38.69355100000013],[-105.96998599999989,38.69490300000007],[-105.96899599999995,38.69718600000016],[-105.96772299999998,38.697135],[-105.96537499999994,38.69897900000001],[-105.96364699999987,38.7013290000001],[-105.96182299999992,38.70605499999999],[-105.9572839999999,38.70897800000006],[-105.95544699999988,38.71066700000006],[-105.95303899999993,38.71365500000002],[-105.95016399999992,38.714375000000246],[-105.949614,38.71501300000011],[-105.94618499999984,38.716724000000056],[-105.947565,38.72018900000006],[-105.94610599999999,38.7216440000002],[-105.944434,38.72174699999999],[-105.94346899999994,38.72314399999999],[-105.94610799999998,38.72743500000007],[-105.94483000000002,38.72836500000011],[-105.94433300000003,38.730533000000264],[-105.94349299999988,38.73081800000023],[-105.943017,38.73453200000006],[-105.94475199999994,38.73576600000001],[-105.94426799999997,38.73706500000003],[-105.947947,38.74168899999995],[-105.94785999999999,38.743823000000134],[-105.94210699999996,38.74478800000003],[-105.94133699999992,38.745604999999955],[-105.94143299999996,38.748687000000245],[-105.93830599999995,38.753685000000246],[-105.937994,38.75603600000011],[-105.93875799999995,38.76302200000026],[-105.93970200000001,38.76476399999996],[-105.9445399999999,38.76810499999999],[-105.94251099999997,38.7685120000001],[-105.93989299999998,38.77067500000021],[-105.93974899999989,38.77191100000016],[-105.9372919999999,38.772088000000224],[-105.936287,38.771622000000036],[-105.93421199999989,38.77184300000022],[-105.93142799999993,38.7727490000002],[-105.92928899999987,38.7745470000001],[-105.92964199999994,38.777236000000016],[-105.93054999999993,38.77854100000002],[-105.92953499999999,38.77972199999999],[-105.92956299999997,38.78138999999999],[-105.92770599999989,38.782256000000245],[-105.925161,38.78266699999995],[-105.92529200000001,38.783787000000075],[-105.9267089999999,38.78550800000005],[-105.92368599999998,38.786246000000006],[-105.92163700000003,38.788105000000144],[-105.92112299999985,38.789360000000045],[-105.91804999999988,38.790245000000084],[-105.9151389999999,38.790437],[-105.913704,38.79245899999995],[-105.9144639999999,38.79373100000004],[-105.91290799999985,38.795513000000085],[-105.91268000000002,38.797354000000155],[-105.91179399999999,38.799878999999976],[-105.91055199999988,38.80197200000015],[-105.91149300000001,38.802931000000285],[-105.91045400000002,38.804876999999976],[-105.906587,38.806962],[-105.91068699999994,38.81048100000004],[-105.91072699999995,38.81236800000022],[-105.91258399999987,38.81389900000005],[-105.913997,38.81606300000004],[-105.91723199999996,38.81752500000022],[-105.92067299999991,38.82163000000003],[-105.92424599999987,38.82419300000004],[-105.92601299999995,38.82649100000026],[-105.928067,38.828125000000284],[-105.9322489999999,38.832731000000024],[-105.93505399999992,38.83430100000004],[-105.936398,38.836571000000106],[-105.939662,38.83913600000005],[-105.94162499999993,38.84017400000016],[-105.94184899999999,38.841268999999954],[-105.940156,38.84352799999999],[-105.93914699999999,38.84604600000017],[-105.939685,38.84811400000001],[-105.94056899999998,38.8492270000001],[-105.94288699999993,38.850503000000174],[-105.94317399999994,38.85230700000022],[-105.93951499999997,38.85659300000009],[-105.93711400000001,38.858407000000284],[-105.93501499999996,38.85910400000006],[-105.93300699999992,38.8610510000002],[-105.93205399999994,38.86323900000002],[-105.93251799999996,38.86450100000019],[-105.93144799999999,38.8665270000003],[-105.92913199999992,38.865945000000295],[-105.927819,38.86884500000025],[-105.92841099999998,38.869751000000065],[-105.92709000000002,38.8714820000003],[-105.92512299999999,38.87217799999996],[-105.92684599999995,38.874235],[-105.93115899999998,38.87470899999994],[-105.93297499999994,38.878219000000115],[-105.93602599999997,38.88144900000003],[-105.93887499999988,38.88332300000019],[-105.941238,38.885591999999974],[-105.94277299999999,38.885505999999964],[-105.94796399999996,38.88327100000015],[-105.94913899999995,38.883104],[-105.95381500000002,38.88419799999991],[-105.95564300000001,38.88369100000011],[-105.959969,38.88440600000007],[-105.96356000000003,38.885855000000106],[-105.96496499999995,38.885977000000025],[-105.97063700000001,38.88319600000017],[-105.97429699999998,38.88245800000021],[-105.97510799999992,38.883740999999986],[-105.9771439999999,38.88514600000002],[-105.97823999999991,38.887905000000046],[-105.979175,38.888912999999945],[-105.97974499999981,38.891728],[-105.97941799999995,38.89453300000008],[-105.97407299999992,38.89668000000029],[-105.97240699999992,38.89925699999998],[-105.96866099999994,38.90072000000009],[-105.96807499999994,38.9037830000002],[-105.96744699999988,38.90493900000007],[-105.96577000000002,38.90587599999998],[-105.96775699999984,38.906693000000075],[-105.96946399999996,38.90860600000008],[-105.97188599999998,38.908923000000016],[-105.97272900000002,38.90956100000005],[-105.97664399999985,38.9106220000001],[-105.97603299999992,38.9118390000001],[-105.97686499999992,38.912968000000035],[-105.97653099999991,38.91441900000001],[-105.97844799999979,38.91465500000004],[-105.98381599999993,38.91652300000021],[-105.98610000000002,38.91810600000008],[-105.99444899999997,38.914857999999924],[-105.99817000000002,38.91515899999996],[-105.9993429999999,38.91633700000028],[-105.99918300000002,38.91840900000011],[-106.00639999999993,38.92408399999999],[-106.00821899999988,38.9264800000002],[-106.01047399999993,38.92781900000011],[-106.0125999999999,38.929665000000284],[-106.0150099999999,38.93271200000015],[-106.01806299999981,38.93445099999997],[-106.02052900000001,38.93462800000003],[-106.02461700000003,38.93883000000028],[-106.02627099999995,38.94109000000003],[-106.02813799999996,38.94284900000008],[-106.0307969999999,38.94250799999992],[-106.032287,38.941599000000224],[-106.03683000000001,38.940646999999956],[-106.03887299999991,38.940792000000044],[-106.04378999999989,38.94192599999991],[-106.04577899999987,38.94154800000007],[-106.04861299999988,38.94282700000002],[-106.05170099999992,38.94345200000009],[-106.05824999999993,38.941805000000215],[-106.05844899999988,38.93915099999998],[-106.06014900000002,38.93715300000025],[-106.0635319999999,38.93568000000005],[-106.06475099999994,38.93443300000007],[-106.06496699999997,38.93269000000009],[-106.06590999999992,38.93189599999994],[-106.06782499999991,38.93287200000009],[-106.07047999999986,38.93211099999996],[-106.07148099999989,38.93377600000025],[-106.07396299999999,38.93475799999999],[-106.07618899999989,38.934555000000046],[-106.07862799999998,38.93525500000004],[-106.08358699999997,38.937935000000095],[-106.08531999999991,38.93946000000017],[-106.09065299999992,38.93654300000003],[-106.09354999999988,38.93795800000004],[-106.09486199999992,38.93677500000024],[-106.09646099999998,38.938243],[-106.09823399999993,38.938171000000125],[-106.1009709999999,38.93869999999998],[-106.10575799999987,38.94036399999999],[-106.110995,38.94009900000009],[-106.11266499999999,38.94136600000013],[-106.113114,38.9450240000001],[-106.11204299999986,38.95090800000003],[-106.11343999999997,38.95278800000017],[-106.11375699999991,38.955327000000125],[-106.11105999999995,38.95766900000007],[-106.11219299999999,38.96018800000013],[-106.11061699999999,38.96203400000013],[-106.10828499999985,38.96346300000022],[-106.10558300000002,38.96941800000002],[-106.10496799999987,38.97002600000002],[-106.104379,38.974140999999975],[-106.10558900000001,38.976046000000224],[-106.10648999999984,38.976494],[-106.10760699999997,38.9797470000002],[-106.10947899999996,38.9816330000001],[-106.10950999999989,38.98593900000003],[-106.10895299999999,38.98737700000004],[-106.11187100000001,38.986953000000085],[-106.11323899999996,38.98781000000014],[-106.11677099999997,38.98808400000024],[-106.12246799999991,38.990816000000166],[-106.12751799999995,38.994017000000156],[-106.13171699999992,38.9949180000001],[-106.13256799999999,38.995519000000115],[-106.13777800000003,38.994642],[-106.1412509999999,38.99496599999998],[-106.14265899999987,38.995428000000004],[-106.1454139999999,38.99789900000019],[-106.14956000000001,38.99702600000006],[-106.15066599999994,38.99709300000029],[-106.15407800000003,38.99616700000024],[-106.157804,38.99426399999999],[-106.16051399999998,38.994808000000035],[-106.16341999999997,38.993908000000204],[-106.16566099999994,38.99243900000016],[-106.16853399999991,38.991249000000096],[-106.169804,38.99253299999992],[-106.17006700000002,38.994644000000164],[-106.17316499999998,38.99740300000019],[-106.17491599999994,38.99959600000011],[-106.17591799999997,38.99964700000004],[-106.176962,39.00194900000008],[-106.177009,39.005341999999985],[-106.17802499999993,39.0076890000002],[-106.17792299999996,39.00910099999999],[-106.17946999999987,39.01124900000025],[-106.18305199999998,39.011835000000076],[-106.18494899999996,39.012581000000125],[-106.18607699999995,39.0155950000003],[-106.18825299999986,39.01538099999999],[-106.191416,39.016549],[-106.19295399999999,39.01770900000008],[-106.19310499999995,39.01968099999999],[-106.18947300000002,39.02127900000005],[-106.18980899999991,39.02311300000025],[-106.19127699999996,39.02583300000009],[-106.19207699999998,39.02928700000001],[-106.19032699999997,39.030570000000125],[-106.18872499999992,39.03323500000005],[-106.18690699999996,39.033195999999975],[-106.185879,39.03403800000001],[-106.18497000000002,39.036235999999974],[-106.18356199999988,39.03703100000007],[-106.182501,39.03855100000004],[-106.1827819999998,39.04011600000001],[-106.1853529999999,39.0440200000001],[-106.18570199999982,39.04676100000006],[-106.18492800000001,39.04911000000027],[-106.1848149999999,39.050837],[-106.18636199999992,39.052201000000025],[-106.19009299999988,39.05452400000007],[-106.19100600000002,39.05607300000014],[-106.19093499999991,39.05748599999998],[-106.19173299999994,39.05921700000022],[-106.19069599999995,39.061944000000096],[-106.19048099999992,39.06398300000001],[-106.19160799999992,39.067499],[-106.19124599999998,39.06844800000016],[-106.18849199999994,39.07004000000006],[-106.1865239999999,39.07163800000012],[-106.18678899999998,39.07312900000022],[-106.18610899999993,39.074987000000306],[-106.18358899999998,39.07647500000013],[-106.18032799999997,39.07724500000006],[-106.17569000000003,39.082939000000124],[-106.17279499999995,39.08396700000026],[-106.17110200000002,39.08610900000002],[-106.16966299999996,39.0864170000001],[-106.16928299999995,39.08778600000022],[-106.1698639999999,39.0887230000003],[-106.16986699999995,39.090731000000176],[-106.17152399999992,39.09518200000002],[-106.17296099999999,39.097278000000074],[-106.175389,39.09880500000014],[-106.17740000000003,39.101312000000064],[-106.17811099999994,39.10295800000006],[-106.17921799999999,39.10346300000026],[-106.18327199999993,39.10340300000007],[-106.18785999999994,39.10373600000008],[-106.19042200000001,39.102752000000066],[-106.19292999999988,39.10305100000028],[-106.19576799999993,39.104472000000044],[-106.19988799999987,39.10436600000003],[-106.20339899999993,39.102732000000174],[-106.20519199999995,39.103255000000274],[-106.20785199999995,39.103423000000134],[-106.20953500000002,39.1030750000001],[-106.21017299999988,39.10391800000025],[-106.20770900000002,39.10807099999994],[-106.20736499999998,39.1095370000001],[-106.20629999999989,39.11043600000005],[-106.20389499999993,39.11128800000006],[-106.20339499999994,39.111962000000005],[-106.20348299999995,39.11408900000015],[-106.19949099999997,39.11550800000015],[-106.19146999999987,39.12086899999997],[-106.189302,39.120574999999974],[-106.18826200000001,39.12142899999998],[-106.18763100000001,39.12459700000005],[-106.18584099999998,39.12683900000013],[-106.18579999999997,39.129249000000016],[-106.18439899999993,39.12929900000006],[-106.18229600000001,39.13029],[-106.181919,39.13148099999995],[-106.17841999999996,39.134769999999946],[-106.17384700000002,39.13707599999998],[-106.17294599999991,39.13876500000009],[-106.17482099999978,39.14142299999992],[-106.17348600000003,39.14359500000006],[-106.17346099999997,39.144795999999985],[-106.17481299999992,39.14801800000009],[-106.17429599999997,39.149364000000105],[-106.17554199999995,39.152918],[-106.17534899999998,39.15629800000022],[-106.17651000000001,39.15788599999996],[-106.17685299999988,39.16031600000014],[-106.17886199999992,39.16333500000019],[-106.17944699999993,39.16503600000004],[-106.1785589999999,39.16596800000025],[-106.17812799999984,39.168884000000105],[-106.17840999999993,39.174579999999935],[-106.17917499999999,39.17598800000013],[-106.18281599999989,39.18040300000018],[-106.18332599999997,39.182443000000035],[-106.18552599999992,39.18658700000026],[-106.1840969999999,39.18826800000011],[-106.18221699999987,39.18967299999997],[-106.1793199999999,39.19349300000022],[-106.1797699999999,39.19490600000006],[-106.179506,39.19999800000005],[-106.18132399999996,39.20475299999998],[-106.18348699999996,39.20671400000009],[-106.18474399999997,39.209308000000135],[-106.182051,39.20994300000007],[-106.17840899999993,39.21297800000019],[-106.17783299999996,39.21634],[-106.17718100000002,39.21814699999993],[-106.17821600000002,39.219132000000116],[-106.17427299999991,39.22072700000007],[-106.1701579999999,39.222994000000085],[-106.16965799999991,39.22562200000027],[-106.16991999999999,39.22810100000021],[-106.16891199999992,39.22876000000019],[-106.16830399999986,39.23072000000019],[-106.16949799999998,39.23375800000019],[-106.17293499999988,39.23680500000006],[-106.17683999999997,39.23855600000002],[-106.17993000000001,39.239446999999984],[-106.18278699999996,39.239607999999976],[-106.18398499999995,39.24047800000005],[-106.18511999999998,39.242694000000256],[-106.18348299999991,39.24439400000006],[-106.18106599999993,39.24816100000015],[-106.18174599999998,39.25064299999997],[-106.18095099999994,39.25244500000002],[-106.18159100000003,39.25700500000022],[-106.1806489999999,39.25868600000007],[-106.18048199999993,39.26109700000006],[-106.18164300000001,39.2632930000002],[-106.18145299999998,39.26536599999997],[-106.1841419999999,39.26886000000002],[-106.18566499999997,39.27328599999993],[-106.18574799999999,39.27580899999998],[-106.18675899999988,39.27985100000029],[-106.18493899999993,39.28339100000028],[-106.18545799999998,39.28575100000012],[-106.18499699999995,39.28661500000021],[-106.18188700000002,39.28996799999999],[-106.18257799999998,39.293060000000025],[-106.18091099999998,39.29880000000014],[-106.18305099999998,39.30111199999993],[-106.18303799999995,39.30327199999999],[-106.18397199999987,39.30935399999993],[-106.18370899999996,39.310283000000254],[-106.18183599999998,39.3112270000002],[-106.18003799999997,39.311195000000225],[-106.17642799999999,39.31207500000005],[-106.17505399999988,39.31441800000027],[-106.17476399999998,39.317112000000236],[-106.17418099999998,39.31827600000008],[-106.16947399999987,39.32223200000021],[-106.16780499999999,39.32164299999994],[-106.1623259999999,39.321628000000146],[-106.15357299999988,39.32041800000002],[-106.1512459999999,39.32069400000012],[-106.150485,39.324349999999924],[-106.14895299999989,39.32646899999992],[-106.147153,39.32760499999995],[-106.14715999999987,39.32852399999996],[-106.14380199999994,39.333867999999995],[-106.14219300000002,39.336890000000096],[-106.13992599999995,39.33955300000002],[-106.14341699999994,39.343124999999986],[-106.144407,39.34477300000026],[-106.14458000000002,39.346852000000126],[-106.14636899999988,39.34828700000003],[-106.14721399999996,39.349904000000095],[-106.14718099999999,39.353628000000185],[-106.14826699999998,39.35684800000013],[-106.14844800000003,39.35861300000016],[-106.15067199999993,39.35973200000001],[-106.15294299999988,39.36276099999998],[-106.14879499999995,39.36835000000008],[-106.14672199999995,39.367972000000066],[-106.144361,39.368120000000204],[-106.14165000000003,39.36996999999997],[-106.1412939999999,39.370914000000084],[-106.13850499999995,39.37224099999992],[-106.138441,39.37498800000003],[-106.13655199999994,39.37884000000025],[-106.13552900000002,39.37954600000023],[-106.13143299999996,39.37951099999998],[-106.12949400000002,39.378767000000096],[-106.12744700000002,39.3788770000001],[-106.12396100000001,39.37802100000016],[-106.12030799999997,39.37659000000025],[-106.11481299999997,39.37649600000009],[-106.11197999999996,39.37705299999999],[-106.10729000000003,39.37745399999994],[-106.10603800000001,39.37793099999999],[-106.10194299999995,39.37805400000008],[-106.09940099999994,39.37773300000015],[-106.096791,39.377896000000305],[-106.09431199999977,39.37651100000028],[-106.08842800000002,39.372000999999955],[-106.08732399999997,39.37082700000019],[-106.08444599999996,39.369282999999996],[-106.08126700000003,39.36914500000029],[-106.08076599999987,39.370228],[-106.07419199999993,39.36863000000011],[-106.07285699999989,39.36791700000009],[-106.06847099999999,39.36442899999997],[-106.06407899999994,39.36267699999996],[-106.06255899999996,39.361786000000166],[-106.05833499999994,39.36005200000022],[-106.05609599999997,39.35843100000005],[-106.05092199999996,39.358061000000134],[-106.04631699999987,39.357434000000126],[-106.04392599999994,39.357377999999926],[-106.03746999999993,39.358465000000024],[-106.03471699999994,39.35837600000008],[-106.03219899999999,39.35894200000001],[-106.02864099999994,39.360224000000244],[-106.02671999999995,39.361333],[-106.02433000000002,39.36114700000002],[-106.02196600000002,39.36171299999995],[-106.01761899999997,39.36550300000022],[-106.01747299999994,39.3733830000001],[-106.016794,39.37498800000003],[-106.01745299999999,39.379204000000016],[-106.01838299999991,39.381403000000034],[-106.01699400000001,39.38291100000009],[-106.01636400000001,39.38439000000028],[-106.01385099999993,39.386897999999974],[-106.00840099999994,39.38735600000018],[-106.00453699999997,39.38979000000012],[-106.00333899999998,39.39176600000002],[-105.99952899999994,39.39382500000022],[-105.99784499999987,39.394010000000094],[-105.99592799999988,39.39500099999998],[-105.99413199999987,39.39782399999996],[-105.9928559999999,39.40185900000006],[-105.99052799999993,39.402202000000045],[-105.98693000000003,39.401507000000095],[-105.98501799999991,39.401666999999975],[-105.98362099999997,39.40244100000007],[-105.98302499999988,39.404431000000045],[-105.981923,39.405974000000185],[-105.98065099999991,39.40668599999992],[-105.97796699999998,39.40726900000004],[-105.97686699999991,39.408570000000225],[-105.97320599999995,39.40918900000008],[-105.96872000000002,39.41067800000002],[-105.96280999999999,39.411991000000114],[-105.9592439999999,39.412473000000034],[-105.95232499999992,39.41410599999995],[-105.95169799999991,39.41461900000013],[-105.95365200000003,39.41838100000001],[-105.95387499999993,39.42047900000023],[-105.95509199999998,39.421551000000136],[-105.95539400000001,39.42285100000021],[-105.95483699999994,39.42622000000006],[-105.95630999999986,39.42784100000006],[-105.96072600000002,39.4290180000001],[-105.965013,39.433436000000086],[-105.96678599999979,39.4380350000003],[-105.96439499999985,39.439531999999986],[-105.96175199999993,39.44009100000005],[-105.95819199999988,39.439677000000074],[-105.956436,39.438401],[-105.953014,39.438867000000016],[-105.94665099999997,39.44062400000013],[-105.94415500000002,39.440910000000144],[-105.94401799999997,39.44509600000009],[-105.94369799999987,39.44618299999996],[-105.94185299999998,39.448246000000154],[-105.94249999999988,39.45048400000002],[-105.94217000000003,39.45277300000009],[-105.94061199999999,39.456524000000286],[-105.93866300000002,39.45849800000019],[-105.93465700000002,39.45788799999997],[-105.929823,39.45933800000023],[-105.92667399999993,39.4594170000002],[-105.92195200000003,39.458455000000185],[-105.91707199999996,39.457969000000105],[-105.91431599999981,39.45820300000014],[-105.91163699999993,39.456736000000035],[-105.91051099999976,39.458376000000214],[-105.90835099999993,39.45942800000023],[-105.905959,39.45965700000016],[-105.90391,39.46221000000003],[-105.901835,39.464248000000055],[-105.90125799999998,39.46721300000024],[-105.90161199999994,39.46961900000019],[-105.89909,39.47150000000022],[-105.89848099999995,39.47340900000006],[-105.89672599999989,39.47565700000007],[-105.89732699999996,39.47713699999997],[-105.89472499999988,39.47751700000015],[-105.89219899999995,39.48104400000011],[-105.89224999999999,39.48356100000001],[-105.88918499999988,39.483350000000144],[-105.88467700000001,39.484138000000144],[-105.87908699999997,39.483767],[-105.87917299999998,39.48682400000001],[-105.87834800000002,39.489618000000235],[-105.87696599999998,39.49060199999997],[-105.87471399999998,39.493521000000044],[-105.87241399999994,39.49480800000015],[-105.86990500000002,39.49495300000024],[-105.86752100000001,39.494147],[-105.86658399999993,39.494351000000165],[-105.86656499999987,39.496663000000126],[-105.86593099999999,39.498082000000124],[-105.86421399999995,39.49998600000009],[-105.86358399999995,39.50230499999998],[-105.86039499999987,39.50470999999999],[-105.85639199999997,39.50513000000029],[-105.85411699999992,39.50452000000007],[-105.85477200000003,39.506401000000096],[-105.85701699999998,39.50706000000008],[-105.8594569999999,39.509225000000185],[-105.85994700000003,39.51214800000008],[-105.86180999999999,39.51306000000005],[-105.86688900000001,39.51670200000012],[-105.86804899999998,39.5198620000001],[-105.86470099999991,39.522114999999985],[-105.86352499999987,39.523955],[-105.86289599999998,39.527307000000064],[-105.85911399999986,39.53181300000006],[-105.857732,39.532058000000234],[-105.85456199999999,39.527868000000296],[-105.8512849999999,39.52612099999999],[-105.8472109999999,39.52484000000004],[-105.84537699999993,39.52375000000029],[-105.84432900000002,39.525991999999974],[-105.838571,39.52782100000013],[-105.83605699999981,39.5292850000003],[-105.83001599999994,39.532043000000044],[-105.8239309999999,39.530560000000094],[-105.82280899999995,39.53080200000005],[-105.82123399999995,39.53246800000005],[-105.8192679999998,39.537572000000296],[-105.817631,39.53831000000025],[-105.81906599999996,39.544064000000276],[-105.81892800000003,39.545815000000005],[-105.81608699999987,39.54890999999998],[-105.81580699999978,39.55197599999997],[-105.81872399999992,39.55207400000012],[-105.82233699999989,39.55301200000025],[-105.82577099999997,39.55567800000006],[-105.82606099999981,39.55744200000015],[-105.82890600000002,39.56154699999996],[-105.82852099999997,39.563452999999924],[-105.82966199999998,39.564865000000054],[-105.77498000000003,39.56492300000025],[-105.750561,39.56489700000003],[-105.72405399999991,39.56537500000002],[-105.7216009999998,39.565269000000285],[-105.6951489999999,39.56564400000002],[-105.65795000000003,39.56635],[-105.63587499999988,39.566707000000065],[-105.623875,39.56703600000009],[-105.58798599999989,39.568197000000055],[-105.54777999999999,39.56721600000026],[-105.50782199999998,39.56625200000008],[-105.500292,39.566134999999974],[-105.48511699999995,39.566188000000295],[-105.46795299999991,39.566088000000036],[-105.45920899999993,39.56594200000001],[-105.4449219999999,39.56598800000006],[-105.42815999999999,39.56592800000027],[-105.42725100000001,39.56598800000006],[-105.401051,39.56598800000006],[-105.39894899999996,39.566056]]]},"properties":{"OBJECTID":48,"COUNTYFP":"093","Shape_Leng":3.79218939883,"Shape_Area":0.596376269293,"low":3770,"lowmod":7100,"LMMI":10865,"lowmoduniv":16075,"lowmod_pct":0.44167962675,"NAMELSAD10":"Park County"}},
{"type":"Feature","geometry":{"type":"Polygon","coordinates":[[[-102.05141599999996,40.749585999999965],[-102.05144399999989,40.69984500000021],[-102.05149899999992,40.63948300000004],[-102.05147899999997,40.616153999999995],[-102.0515289999999,40.56715400000007],[-102.051513,40.5199600000002],[-102.05155400000001,40.50002300000011],[-102.05127400000003,40.486269999999934],[-102.05150899999995,40.465508],[-102.05146499999995,40.44000800000009],[-102.08993199999992,40.43972600000001],[-102.12545899999992,40.43972000000025],[-102.16670499999998,40.439669000000094],[-102.21496300000001,40.439610000000016],[-102.21768499999996,40.43986100000001],[-102.22169600000001,40.43963099999996],[-102.22409899999997,40.43976399999997],[-102.25046799999996,40.43948500000016],[-102.30118099999993,40.439572000000055],[-102.3232559999999,40.43951700000014],[-102.33973000000003,40.4395530000001],[-102.36419099999989,40.43968699999999],[-102.36856,40.43960099999998],[-102.38843199999991,40.43963500000012],[-102.397874,40.43971600000009],[-102.41574500000002,40.439621000000045],[-102.41625899999991,40.43992400000025],[-102.45465899999994,40.43947200000014],[-102.4546969999999,40.43930899999998],[-102.50749699999989,40.438889000000245],[-102.5512829999999,40.43862200000001],[-102.56405499999988,40.43861500000014],[-102.58521899999988,40.438514999999995],[-102.6073429999999,40.43846800000006],[-102.6249969999999,40.438601000000006],[-102.664804,40.43847300000027],[-102.66486499999996,40.45673699999992],[-102.665007,40.47288500000002],[-102.66499999999996,40.487532999999985],[-102.66513700000002,40.512644999999964],[-102.66504700000002,40.52177599999999],[-102.66516199999995,40.558742999999936],[-102.66550000000001,40.58410700000002],[-102.66557599999993,40.613394000000255],[-102.66510399999993,40.6445750000002],[-102.6647309999999,40.686059],[-102.6646889999999,40.705189000000246],[-102.65087999999997,40.70533999999992],[-102.651095,40.730995000000064],[-102.65130999999985,40.74940200000003],[-102.59780199999989,40.74952500000006],[-102.56886599999996,40.74945100000002],[-102.52906200000001,40.74952900000005],[-102.50421699999993,40.749388000000295],[-102.45875799999999,40.74925099999996],[-102.41960899999998,40.749153000000035],[-102.40063499999991,40.749324],[-102.38280299999991,40.749366000000066],[-102.33602100000002,40.749246000000085],[-102.30242199999986,40.74910300000016],[-102.28690799999998,40.749108999999976],[-102.276928,40.7489800000003],[-102.24478099999988,40.74892000000011],[-102.22960799999998,40.74898600000006],[-102.1908049999999,40.748896000000116],[-102.16853699999996,40.74903399999994],[-102.11275899999993,40.74946000000023],[-102.06339400000002,40.74982700000021],[-102.05141599999996,40.749585999999965]]]},"properties":{"OBJECTID":49,"COUNTYFP":"095","Shape_Leng":1.84809771727,"Shape_Area":0.189577891307,"low":1155,"lowmod":1845,"LMMI":2735,"lowmoduniv":4310,"lowmod_pct":0.42807424594,"NAMELSAD10":"Phillips County"}},
{"type":"Feature","geometry":{"type":"Polygon","coordinates":[[[-106.42648700000001,39.361871000000065],[-106.42897099999993,39.35997500000008],[-106.42942799999997,39.35803200000004],[-106.43078500000001,39.35711400000031],[-106.43347999999997,39.35417400000023],[-106.4336689999999,39.351113999999995],[-106.43510399999985,39.34727499999997],[-106.43536,39.343972000000235],[-106.43493099999995,39.34109900000021],[-106.43354799999992,39.337492],[-106.43228199999999,39.335956000000124],[-106.42867599999994,39.33540200000027],[-106.429438,39.33438899999993],[-106.42866300000003,39.33255500000007],[-106.42870299999987,39.32818200000008],[-106.42694699999987,39.32335500000022],[-106.42801699999995,39.32145000000014],[-106.43042199999991,39.31937800000031],[-106.43245999999994,39.316352999999935],[-106.43451399999998,39.31710199999992],[-106.438223,39.316854000000205],[-106.43977100000001,39.317321000000106],[-106.4425399999999,39.315291],[-106.44582500000001,39.315495],[-106.44847199999998,39.3166940000001],[-106.45065199999988,39.31669800000026],[-106.45238899999998,39.315669000000014],[-106.45274599999993,39.31429200000014],[-106.45421899999997,39.312259000000154],[-106.4558649999999,39.31213500000001],[-106.45907599999992,39.313041],[-106.46148199999988,39.31261400000005],[-106.46338199999997,39.311494000000266],[-106.46551199999999,39.311578999999995],[-106.46680900000001,39.31041200000027],[-106.46696399999996,39.30924800000025],[-106.46872300000001,39.307896000000085],[-106.47072600000001,39.30788700000005],[-106.47287799999998,39.308423000000005],[-106.476205,39.312498000000005],[-106.476224,39.31474800000001],[-106.48512099999999,39.31256599999995],[-106.48639899999995,39.31147099999998],[-106.48743100000002,39.30897000000016],[-106.48814799999991,39.30901400000005],[-106.49184999999989,39.306620000000066],[-106.49260399999991,39.30573700000002],[-106.49251500000003,39.303995999999984],[-106.49423399999995,39.30307899999997],[-106.49617799999987,39.30284500000016],[-106.49989799999997,39.301483000000076],[-106.50164699999993,39.29981800000019],[-106.50268999999997,39.29806700000006],[-106.50356099999993,39.295051000000285],[-106.5035969999999,39.293609000000174],[-106.50286699999998,39.289993000000095],[-106.50458100000003,39.2873620000002],[-106.504031,39.28476100000029],[-106.50080999999994,39.28203100000019],[-106.4999259999999,39.28167300000007],[-106.49854399999987,39.279324999999915],[-106.49532199999999,39.27870200000007],[-106.49322899999993,39.27763199999998],[-106.49277699999993,39.27627600000005],[-106.48982899999999,39.273140000000126],[-106.483292,39.272669000000235],[-106.48372199999989,39.271702000000005],[-106.48341699999997,39.268852000000265],[-106.4840699999998,39.265396000000294],[-106.48327199999994,39.264272000000005],[-106.48300599999988,39.26213000000007],[-106.48122599999988,39.25916700000005],[-106.48297699999989,39.25801799999999],[-106.48312699999997,39.25510300000008],[-106.48615399999994,39.25390700000003],[-106.48825099999999,39.249264000000096],[-106.49397199999993,39.24495200000001],[-106.49677799999995,39.243321000000265],[-106.49764899999997,39.24202200000025],[-106.49874,39.234931000000074],[-106.49725699999988,39.232977999999946],[-106.49509399999988,39.23191800000001],[-106.4927209999999,39.22940799999998],[-106.48974900000002,39.22771799999998],[-106.49181799999997,39.226102000000196],[-106.49225999999987,39.2251950000001],[-106.49125800000002,39.22279300000031],[-106.49195099999997,39.22069799999997],[-106.49339399999991,39.22010299999994],[-106.49387000000002,39.218728000000056],[-106.49294299999997,39.21585800000008],[-106.49485399999992,39.210091000000034],[-106.494954,39.20854100000008],[-106.49377899999996,39.206367000000114],[-106.49459999999993,39.20014800000018],[-106.49364899999995,39.198326000000236],[-106.49583599999988,39.19441000000023],[-106.49773499999998,39.19322199999999],[-106.49831699999993,39.19201400000003],[-106.49899099999999,39.186217],[-106.49960699999997,39.183848000000296],[-106.50264899999996,39.180419000000086],[-106.50480299999992,39.17902000000015],[-106.50672599999996,39.17881500000004],[-106.50821599999989,39.17732300000006],[-106.50971700000002,39.17487700000004],[-106.50951099999992,39.171928000000264],[-106.50874399999998,39.169993000000034],[-106.50861399999997,39.16668100000027],[-106.511077,39.163532000000146],[-106.51386200000002,39.162258000000236],[-106.51637399999987,39.1603360000002],[-106.518913,39.15969800000016],[-106.51929899999993,39.15860200000026],[-106.52094199999999,39.157353000000114],[-106.52401999999995,39.158754000000215],[-106.527357,39.161021000000005],[-106.53128800000002,39.16122400000012],[-106.53263399999997,39.16246800000022],[-106.53513799999996,39.16314599999998],[-106.53577699999988,39.16258499999992],[-106.536112,39.15943700000025],[-106.53852599999988,39.15789400000011],[-106.54288399999996,39.15888199999995],[-106.54502200000002,39.15887200000003],[-106.54862599999996,39.160549999999944],[-106.55167699999998,39.16146500000002],[-106.55636400000003,39.15794900000003],[-106.557186,39.1567100000002],[-106.56126499999988,39.155677000000026],[-106.56263100000001,39.15194700000029],[-106.56108499999999,39.14996700000006],[-106.56104499999998,39.14776999999998],[-106.56025099999982,39.1453420000002],[-106.55867899999998,39.14384699999994],[-106.55964299999988,39.140881000000036],[-106.55949999999996,39.1396840000001],[-106.56053599999996,39.13782400000008],[-106.56424399999992,39.13423700000027],[-106.56457399999994,39.13242200000002],[-106.56372599999986,39.131199000000265],[-106.56041399999992,39.12844799999999],[-106.560179,39.1271220000001],[-106.55832499999991,39.12423700000011],[-106.55817099999996,39.12022300000018],[-106.56066699999997,39.117898000000196],[-106.563895,39.11574800000028],[-106.56498599999992,39.1140070000003],[-106.56453799999997,39.11197900000019],[-106.56469599999991,39.10908000000012],[-106.56349599999993,39.10824299999996],[-106.56398899999994,39.10604100000006],[-106.56698299999977,39.10506099999998],[-106.56843900000001,39.105205000000126],[-106.56893300000002,39.1040090000003],[-106.57334400000002,39.104423999999995],[-106.57658600000002,39.10400099999998],[-106.58031099999977,39.10216400000024],[-106.58378099999993,39.0979040000002],[-106.58615699999996,39.09404999999998],[-106.58633800000001,39.087743000000216],[-106.58491600000002,39.08483000000007],[-106.58323799999994,39.08295899999996],[-106.58010699999994,39.081751000000224],[-106.57707599999998,39.0794370000001],[-106.579456,39.07753600000001],[-106.57952399999994,39.07557400000019],[-106.577587,39.07275800000025],[-106.57722999999993,39.06615400000004],[-106.57686199999995,39.06336900000002],[-106.57763,39.060598000000255],[-106.577966,39.05790800000011],[-106.57755099999986,39.053210000000036],[-106.5807299999999,39.04871700000007],[-106.58424499999984,39.04718700000012],[-106.58913299999989,39.045582000000024],[-106.594964,39.04539699999998],[-106.59723299999996,39.04482100000013],[-106.59779700000001,39.03924900000004],[-106.59893599999992,39.03676799999994],[-106.59951599999994,39.0337880000003],[-106.59794099999993,39.03166600000003],[-106.59437700000001,39.02995900000002],[-106.5925959999999,39.027626999999995],[-106.59169199999997,39.024923000000115],[-106.594921,39.02080900000004],[-106.59517900000003,39.01685999999995],[-106.59495399999986,39.01375200000001],[-106.59516200000002,39.01030500000013],[-106.59445499999998,39.00418500000018],[-106.59542999999991,39.00213200000013],[-106.5992139999999,38.99799400000023],[-106.601114,38.99799900000011],[-106.60309799999993,38.996818000000076],[-106.60769699999997,38.99594100000019],[-106.61159999999995,38.99543400000016],[-106.61350199999976,38.99631799999992],[-106.61395500000003,38.998892000000296],[-106.6157179999999,38.99999200000002],[-106.61755199999993,39.001741000000095],[-106.61931099999998,39.002593000000275],[-106.62301499999995,39.003157000000215],[-106.62417399999993,39.003635000000145],[-106.62474999999989,39.00605300000001],[-106.62763299999995,39.00790200000006],[-106.62971299999992,39.01050600000002],[-106.63004099999989,39.01220999999998],[-106.63324,39.01210500000019],[-106.63788699999998,39.01470400000011],[-106.63833399999993,39.01922300000007],[-106.64115700000002,39.02018800000019],[-106.6441099999999,39.020741999999984],[-106.6444939999999,39.0212930000003],[-106.648392,39.02215199999995],[-106.64895999999993,39.02445500000027],[-106.65004999999996,39.02633300000002],[-106.64988199999999,39.02735599999994],[-106.64709699999997,39.02981100000022],[-106.64652699999988,39.030846999999994],[-106.64684599999998,39.03328500000009],[-106.648234,39.03604500000017],[-106.65027399999985,39.037036000000285],[-106.65222,39.03891300000015],[-106.651927,39.040838000000065],[-106.65266699999995,39.041899000000114],[-106.65549199999992,39.04442000000017],[-106.6589479999999,39.04359000000005],[-106.66285900000003,39.04190100000011],[-106.66543699999988,39.04196000000002],[-106.66807699999993,39.043689000000256],[-106.67085999999995,39.043838000000164],[-106.673182,39.044660000000306],[-106.67405400000001,39.04581099999996],[-106.67649999999998,39.04576400000002],[-106.6790959999999,39.045186000000115],[-106.681465,39.04553300000009],[-106.68445099999997,39.04513500000019],[-106.68563799999976,39.045893000000035],[-106.68614999999994,39.04789700000026],[-106.68796299999985,39.049796000000185],[-106.69084299999992,39.05134900000024],[-106.69154400000002,39.05310300000025],[-106.692996,39.05506500000007],[-106.69496900000001,39.055666999999914],[-106.69865099999998,39.055806000000246],[-106.70209099999988,39.05546700000008],[-106.70304499999986,39.05591700000019],[-106.70567899999992,39.054790000000025],[-106.70785099999995,39.05345799999998],[-106.70939399999997,39.050396999999975],[-106.71031499999998,39.047573000000284],[-106.713325,39.04860100000025],[-106.71721799999995,39.04829699999999],[-106.72129699999994,39.04622800000004],[-106.72266100000002,39.04515700000002],[-106.7245779999999,39.04532300000028],[-106.72949599999993,39.04647700000004],[-106.73208599999992,39.04636499999998],[-106.73460999999986,39.04491500000012],[-106.73713599999996,39.04503600000015],[-106.73897999999997,39.046105000000125],[-106.7406949999999,39.045906],[-106.74214,39.04399700000016],[-106.744642,39.043521000000055],[-106.74662199999995,39.04266300000029],[-106.74935700000003,39.04252800000012],[-106.747183,39.040020000000084],[-106.74661100000003,39.035535000000095],[-106.74584599999997,39.032015],[-106.74496599999998,39.02964600000007],[-106.7452869999999,39.02681800000005],[-106.74701800000003,39.02696700000013],[-106.75134299999985,39.02579400000002],[-106.754547,39.02259300000003],[-106.75508299999996,39.021142999999995],[-106.75736699999993,39.01971000000009],[-106.7619029999999,39.019301000000155],[-106.76160299999987,39.01618400000018],[-106.76241899999997,39.014897000000076],[-106.765266,39.01206999999994],[-106.76817499999987,39.0102310000002],[-106.76859400000001,39.00816900000012],[-106.76972699999988,39.00742800000012],[-106.773551,39.00681300000019],[-106.77501099999989,39.00612500000011],[-106.77811899999995,39.001278000000184],[-106.782082,38.998879999999986],[-106.78372000000002,38.995814000000166],[-106.783592,38.99395100000021],[-106.78257499999995,38.99242400000014],[-106.78221400000001,38.99074300000001],[-106.78398899999996,38.986916000000235],[-106.78764999999993,38.98432900000006],[-106.79068899999993,38.98413100000005],[-106.79460599999993,38.98482500000006],[-106.79659100000003,38.98416600000007],[-106.79742699999991,38.98333100000008],[-106.80024600000002,38.982176000000266],[-106.79968899999994,38.97965000000016],[-106.801874,38.97830300000027],[-106.80827699999998,38.97899200000006],[-106.81068399999998,38.97868199999999],[-106.81244199999998,38.97970600000019],[-106.81527499999999,38.98034999999999],[-106.81692999999996,38.98138300000005],[-106.81936300000001,38.98006400000014],[-106.82362999999992,38.97947599999998],[-106.82579299999992,38.978095999999994],[-106.82724799999977,38.97771999999998],[-106.82960199999985,38.978605000000016],[-106.83206999999987,38.980772000000115],[-106.834269,38.98084399999999],[-106.8354129999999,38.982045000000085],[-106.83591699999994,38.98357900000019],[-106.83967799999999,38.98461299999991],[-106.840848,38.985612000000174],[-106.84386299999989,38.98570500000022],[-106.84689800000001,38.98700700000012],[-106.84897899999999,38.989333000000215],[-106.84969299999989,38.991468999999995],[-106.85103899999996,38.9937240000001],[-106.85288199999997,38.995601000000306],[-106.85483499999998,38.996912000000236],[-106.85694499999994,38.99999100000008],[-106.85682099999991,39.00113100000027],[-106.85833299999996,39.00294200000019],[-106.85954699999985,39.00572900000003],[-106.859128,39.006482000000005],[-106.85982299999995,39.00829399999998],[-106.8616869999999,39.00943900000004],[-106.86515599999996,39.008371000000125],[-106.867752,39.00685900000008],[-106.868968,39.00726800000018],[-106.87052199999994,39.00655000000029],[-106.87648999999982,39.00694000000004],[-106.87808999999987,39.00590900000003],[-106.87867199999988,39.002861999999936],[-106.87663399999985,38.99999100000008],[-106.87681499999991,38.99731400000013],[-106.87761499999982,38.99645799999996],[-106.878041,38.99427500000019],[-106.88158599999997,38.99276000000009],[-106.88369899999998,38.99136499999992],[-106.88435299999992,38.99040600000001],[-106.88689299999993,38.99103100000019],[-106.89119799999997,38.99102200000016],[-106.89313300000003,38.98952500000013],[-106.89589899999999,38.990106000000026],[-106.900328,38.9924910000002],[-106.90272599999997,38.992053],[-106.90819599999998,38.992579000000205],[-106.90763099999992,38.995085000000245],[-106.91007300000001,38.99999100000008],[-106.91411900000003,38.99986499999994],[-106.91787499999992,38.99918400000013],[-106.9215329999999,39.001230000000135],[-106.92332599999986,39.001925000000085],[-106.92839199999997,39.00455900000003],[-106.93284699999998,39.005298000000266],[-106.93506500000001,39.006078000000116],[-106.93576199999995,39.00741800000014],[-106.937815,39.00825500000013],[-106.94070699999992,39.01111600000024],[-106.944638,39.014427000000296],[-106.94833599999998,39.01472000000001],[-106.9509799999999,39.01659999999998],[-106.95519000000002,39.01663100000002],[-106.95810699999993,39.01551200000017],[-106.96469499999995,39.017544999999984],[-106.96502199999992,39.01892800000002],[-106.96413000000001,39.02106099999992],[-106.9652309999999,39.023324000000116],[-106.96394299999992,39.02851299999992],[-106.96430700000002,39.02958100000001],[-106.96690299999995,39.030801000000054],[-106.9680679999999,39.029921000000115],[-106.97114399999992,39.028985999999975],[-106.97378499999996,39.02888400000006],[-106.97762699999998,39.02993199999992],[-106.97952599999996,39.03076200000004],[-106.98198799999994,39.030347000000006],[-106.98801000000003,39.03079700000012],[-106.99144099999995,39.03270900000001],[-106.99614499999996,39.034175000000175],[-106.99689799999999,39.03566700000016],[-106.9993859999999,39.0388760000003],[-107.001622,39.042272999999966],[-106.99967400000003,39.04277200000024],[-106.99772599999994,39.04623500000025],[-106.99781099999996,39.048055000000204],[-106.99683600000003,39.04934300000002],[-106.993248,39.05132200000014],[-106.98975599999989,39.05198000000007],[-106.989326,39.05547899999999],[-106.99105799999995,39.058524999999975],[-106.99084999999991,39.06004100000018],[-106.98639999999995,39.062470000000246],[-106.98627799999997,39.06427500000001],[-106.98682099999996,39.06716200000017],[-106.98897199999993,39.07071200000007],[-106.99424599999998,39.069645000000264],[-106.99695799999989,39.06934799999999],[-107.00060099999996,39.07190400000013],[-107.00146799999993,39.073130000000106],[-107.00425399999989,39.074497000000065],[-107.00570500000003,39.07675400000011],[-107.00928199999998,39.0804830000003],[-107.01093900000001,39.08304800000025],[-107.01248599999997,39.084380000000124],[-107.01525300000003,39.085681000000136],[-107.01796699999994,39.087619000000075],[-107.019565,39.08937000000003],[-107.026229,39.090910000000065],[-107.029266,39.09191300000009],[-107.03007899999989,39.0934290000003],[-107.03173300000003,39.093472000000304],[-107.03632999999991,39.09492799999998],[-107.04072500000001,39.09908400000012],[-107.04475399999995,39.10084499999999],[-107.04859299999998,39.10428300000024],[-107.05076899999989,39.10678299999995],[-107.05174699999998,39.10908899999998],[-107.05323299999992,39.110573000000045],[-107.05539399999986,39.11080900000019],[-107.06019000000003,39.11195800000007],[-107.06292400000001,39.11540100000002],[-107.06317000000001,39.11642500000016],[-107.06631999999996,39.118805000000066],[-107.07925899999987,39.118876000000284],[-107.12560400000001,39.118787],[-107.18032799999997,39.11878899999999],[-107.21625699999993,39.11879099999999],[-107.26751300000001,39.11870699999997],[-107.28249,39.11871800000017],[-107.28153500000002,39.11932000000024],[-107.28186899999986,39.120879000000286],[-107.28303900000003,39.1228760000003],[-107.28273399999995,39.12405400000006],[-107.285372,39.12498600000009],[-107.28482199999996,39.12639300000001],[-107.28372999999993,39.12696200000016],[-107.28358100000003,39.128293000000156],[-107.28550899999976,39.13094600000011],[-107.28629999999993,39.13370000000009],[-107.28591399999988,39.13481299999995],[-107.2865559999999,39.13692300000014],[-107.28768899999994,39.13789500000024],[-107.28819499999997,39.139764000000014],[-107.290708,39.141673000000196],[-107.29212599999988,39.14390900000029],[-107.29383599999989,39.144709000000034],[-107.29440899999997,39.147261000000015],[-107.29536300000001,39.149435000000096],[-107.29568699999999,39.15364999999997],[-107.29628100000002,39.15433300000018],[-107.30039799999997,39.15658200000007],[-107.3042989999999,39.15555100000023],[-107.30517399999991,39.15665200000001],[-107.30763799999994,39.1571340000001],[-107.31132300000002,39.159490000000005],[-107.311939,39.160431000000244],[-107.31416100000001,39.161977999999976],[-107.315785,39.163842000000045],[-107.31874099999987,39.16531300000008],[-107.32177300000001,39.16576800000007],[-107.32484399999998,39.16579600000023],[-107.32826299999999,39.165339000000074],[-107.33414199999987,39.1664430000003],[-107.33613500000001,39.16766800000022],[-107.33707499999991,39.169067000000155],[-107.34282999999994,39.168453],[-107.34823899999992,39.17030799999998],[-107.35365699999994,39.17332399999998],[-107.35411999999997,39.17608700000011],[-107.355503,39.17719300000016],[-107.35870699999998,39.181565000000035],[-107.35964999999993,39.18483399999997],[-107.36120299999999,39.18510900000018],[-107.3635279999998,39.18632400000001],[-107.36511299999995,39.188244000000225],[-107.36689799999994,39.19204099999996],[-107.370116,39.19376000000028],[-107.37303299999996,39.19671299999999],[-107.37560799999994,39.19793100000004],[-107.37677899999994,39.199024000000065],[-107.37765499999983,39.20082200000002],[-107.37709399999989,39.204134000000295],[-107.37845099999998,39.20626600000014],[-107.37839699999995,39.208146000000056],[-107.37947999999994,39.208578999999986],[-107.38062300000001,39.21022900000008],[-107.37987499999986,39.212068000000215],[-107.37800199999987,39.2148600000001],[-107.37684300000001,39.220364000000245],[-107.37683700000002,39.22338000000002],[-107.37783100000001,39.2241130000001],[-107.3785979999999,39.22754900000018],[-107.37797399999994,39.22875699999997],[-107.37931499999996,39.23109099999999],[-107.37781599999994,39.234016000000224],[-107.37572799999992,39.237004000000184],[-107.37149799999997,39.23881000000023],[-107.37189799999993,39.23981600000013],[-107.373829,39.24058100000002],[-107.37583799999987,39.24289100000021],[-107.37597099999977,39.24551800000012],[-107.37801000000002,39.24680500000022],[-107.37969499999997,39.24728900000014],[-107.38438699999978,39.24792000000008],[-107.38639399999994,39.25030300000009],[-107.39042199999989,39.25059099999993],[-107.39177099999995,39.252311000000134],[-107.39219399999985,39.255009000000086],[-107.39448599999997,39.25629900000007],[-107.39369899999997,39.25902600000029],[-107.391708,39.26016800000008],[-107.39146499999993,39.263881000000026],[-107.38712299999986,39.265815000000146],[-107.38351799999992,39.26777300000015],[-107.38094299999995,39.2682030000002],[-107.37917500000003,39.26910100000009],[-107.37747899999994,39.27220800000009],[-107.378376,39.27480400000013],[-107.37983500000001,39.275739999999985],[-107.38153599999998,39.27512599999994],[-107.383285,39.27550199999996],[-107.387157,39.27788099999998],[-107.39017399999989,39.278921999999966],[-107.39407299999999,39.27938799999998],[-107.393958,39.28159200000027],[-107.39565199999998,39.28230700000029],[-107.39696100000003,39.28356000000002],[-107.3963389999999,39.28637300000008],[-107.39614799999998,39.289513],[-107.39768399999997,39.29182000000009],[-107.39665400000001,39.293912000000034],[-107.39851299999992,39.29511000000019],[-107.4038349999999,39.29278600000009],[-107.40677599999987,39.29303700000008],[-107.40855899999997,39.294098000000304],[-107.41260599999998,39.291877000000056],[-107.414987,39.29378600000024],[-107.41857600000003,39.29483499999998],[-107.42116699999991,39.29685600000005],[-107.43116899999995,39.29900500000008],[-107.43349499999994,39.29852699999992],[-107.43747899999988,39.29956399999992],[-107.43787999999995,39.30100400000026],[-107.43973399999976,39.30249100000003],[-107.43803799999995,39.305034999999975],[-107.43766499999992,39.30667600000004],[-107.43880000000001,39.30901899999992],[-107.44023199999975,39.308430000000044],[-107.44527800000003,39.308520000000044],[-107.44720799999999,39.30578800000012],[-107.44855499999989,39.30567600000029],[-107.44965499999995,39.30455900000004],[-107.45166699999999,39.30452799999995],[-107.454498,39.306545000000085],[-107.456456,39.30692700000003],[-107.45693699999993,39.30923599999994],[-107.45922899999994,39.309686000000056],[-107.46101899999991,39.312617999999986],[-107.46269199999995,39.31379300000009],[-107.46505599999989,39.31474200000025],[-107.46323899999993,39.31685200000021],[-107.46412700000002,39.31874499999998],[-107.46411699999999,39.32034300000004],[-107.465957,39.322119999999984],[-107.463414,39.32427000000007],[-107.46188999999976,39.327570000000264],[-107.45545799999996,39.329393000000096],[-107.45347500000003,39.32884300000006],[-107.45081899999991,39.33003400000001],[-107.449998,39.33266400000002],[-107.44551100000001,39.33526299999994],[-107.44341299999996,39.33850500000017],[-107.4415699999999,39.33991300000031],[-107.43853899999993,39.34101899999996],[-107.43887299999994,39.34328199999999],[-107.43676199999999,39.34638599999994],[-107.435589,39.34694000000013],[-107.43227100000001,39.347210000000075],[-107.43051400000002,39.348221999999964],[-107.43016599999993,39.349597000000244],[-107.43031799999989,39.354281000000015],[-107.42913399999998,39.355383000000074],[-107.42877799999991,39.35743600000012],[-107.42910499999988,39.359883999999965],[-107.42845799999992,39.362480000000005],[-107.4298169999999,39.36400599999996],[-107.43094899999977,39.366178000000104],[-107.40809799999982,39.36618299999998],[-107.37560799999994,39.36602700000003],[-107.35572400000001,39.36612800000023],[-107.3189099999999,39.365859],[-107.30730599999998,39.366126000000065],[-107.291316,39.36609400000026],[-107.25039400000003,39.366186000000084],[-107.24880100000001,39.366002000000094],[-107.21479499999992,39.36602800000031],[-107.202609,39.36596399999996],[-107.12560400000001,39.366103000000294],[-107.11344600000001,39.3660660000001],[-107.08330299999989,39.366100000000074],[-107.0619759999999,39.366261000000236],[-107.05951400000004,39.36603800000006],[-107.05634800000001,39.366300999999964],[-107.04818999999986,39.36615400000005],[-107.02297499999997,39.366167000000075],[-106.97340299999996,39.36564300000009],[-106.90858999999989,39.364901000000145],[-106.87559499999986,39.364535000000046],[-106.84653299999991,39.364299000000244],[-106.81827900000002,39.36373100000014],[-106.77502200000004,39.363018000000125],[-106.75059199999998,39.36267900000013],[-106.74111099999988,39.36242900000025],[-106.740141,39.36219900000003],[-106.73254499999996,39.362599999999986],[-106.72846399999992,39.36248999999998],[-106.72418899999985,39.36263800000012],[-106.71282999999988,39.36200699999995],[-106.70829000000003,39.362009000000114],[-106.70236899999998,39.36237800000009],[-106.68404900000002,39.36242599999997],[-106.67266899999998,39.362081000000046],[-106.65868599999999,39.36228300000005],[-106.64809399999984,39.36228100000005],[-106.62558799999994,39.362107000000094],[-106.54699399999993,39.362088000000085],[-106.54458499999993,39.3622170000001],[-106.500587,39.362319000000014],[-106.48936199999997,39.36181500000009],[-106.42648700000001,39.361871000000065]]]},"properties":{"OBJECTID":50,"COUNTYFP":"097","Shape_Leng":2.9975731985,"Shape_Area":0.262895419335,"low":4980,"lowmod":7605,"LMMI":10210,"lowmoduniv":17300,"lowmod_pct":0.439595375723,"NAMELSAD10":"Pitkin County"}},
{"type":"Feature","geometry":{"type":"Polygon","coordinates":[[[-102.04447499999998,38.26874900000013],[-102.04446399999995,38.262412000000154],[-102.04483299999993,38.26128299999999],[-102.04479000000003,38.24902800000012],[-102.04444999999987,38.24469500000009],[-102.04448200000002,38.22465],[-102.04436299999986,38.210824],[-102.04429999999996,38.18743999999998],[-102.04437599999994,38.16966700000006],[-102.04440699999986,38.14619400000004],[-102.04425099999992,38.141778000000045],[-102.04425799999996,38.11294600000008],[-102.04442899999998,38.07505500000008],[-102.0447079999999,38.041257000000144],[-102.04458299999993,38.02694600000001],[-102.04451499999999,37.98457500000001],[-102.04411399999987,37.95103599999999],[-102.04369199999985,37.91400399999998],[-102.043185,37.87502000000012],[-102.04316099999988,37.844658000000095],[-102.04292299999997,37.82427300000006],[-102.042824,37.79711299999997],[-102.04248899999988,37.779405999999994],[-102.04219499999999,37.75003100000009],[-102.04234099999996,37.74820699999998],[-102.04187799999994,37.72396500000025],[-102.04162599999995,37.696699000000024],[-102.04168800000002,37.68728400000015],[-102.04169300000001,37.66598300000004],[-102.041609,37.660077],[-102.04158499999994,37.64428200000003],[-102.05576300000001,37.64408200000014],[-102.07368799999989,37.644204000000116],[-102.09089299999994,37.64424600000007],[-102.148864,37.64420200000012],[-102.15634499999999,37.644424000000015],[-102.15835900000002,37.644240999999965],[-102.16491299999996,37.64428700000025],[-102.18333100000001,37.644062000000076],[-102.201281,37.644181000000174],[-102.21955399999996,37.64416200000005],[-102.22271799999993,37.64424000000008],[-102.23794199999998,37.644113999999945],[-102.26151800000002,37.644178000000124],[-102.27395199999995,37.643961000000274],[-102.300997,37.6441040000002],[-102.317656,37.64412800000008],[-102.32867599999997,37.64421800000002],[-102.35503099999994,37.644165999999984],[-102.36317300000002,37.6443690000001],[-102.374008,37.64413400000001],[-102.39621899999992,37.64411700000005],[-102.41940199999999,37.643912000000114],[-102.46166499999998,37.643669999999986],[-102.49928699999998,37.643857000000196],[-102.5004689999999,37.6438280000001],[-102.54892699999999,37.64371299999999],[-102.55954700000001,37.64360900000008],[-102.56484799999998,37.64377899999994],[-102.580667,37.64362299999999],[-102.62051699999995,37.64348600000022],[-102.62547199999995,37.64362100000005],[-102.65845099999996,37.64368000000013],[-102.69447899999994,37.643595000000005],[-102.71266600000001,37.6435900000003],[-102.74761499999994,37.64364200000017],[-102.747683,37.67481299999997],[-102.74766399999993,37.687216000000205],[-102.74774499999995,37.70136400000001],[-102.74779999999998,37.72563000000014],[-102.74774399999995,37.73287900000025],[-102.747839,37.76721299999997],[-102.74783299999996,37.80387400000001],[-102.7479879999999,37.823216],[-102.74770799999993,37.82345100000009],[-102.74103000000002,37.823534999999936],[-102.74046399999997,37.82406400000002],[-102.74034599999999,37.851853000000006],[-102.74035700000002,37.863177000000064],[-102.7406509999999,37.883501000000194],[-102.74090399999989,37.954938000000084],[-102.74096099999997,37.98289100000011],[-102.74104799999998,37.99789900000002],[-102.741308,38.00000700000004],[-102.74136199999992,38.012517],[-102.74140999999992,38.04282800000027],[-102.74149999999997,38.08596800000009],[-102.74167599999998,38.09656000000001],[-102.741847,38.12500799999998],[-102.7423649999999,38.15746800000005],[-102.742434,38.17214000000024],[-102.74241499999994,38.18678000000011],[-102.74229699999995,38.20157400000005],[-102.7424059999999,38.215886000000296],[-102.74238500000001,38.235377000000085],[-102.74218899999988,38.23902500000008],[-102.74189799999999,38.250000000000114],[-102.74232999999992,38.26697100000007],[-102.72382399999987,38.26725000000005],[-102.72213899999997,38.26710099999997],[-102.68547799999993,38.26727299999999],[-102.65492999999987,38.26731900000027],[-102.63704399999995,38.26749800000016],[-102.6183779999999,38.26749500000011],[-102.59685599999995,38.267313000000115],[-102.57296999999988,38.267227000000105],[-102.52652699999999,38.267634000000214],[-102.50045899999986,38.26774500000016],[-102.48534399999988,38.26796900000022],[-102.46767499999999,38.26763700000009],[-102.43425599999995,38.26791400000013],[-102.432481,38.26786800000002],[-102.40006599999998,38.267943],[-102.36054399999995,38.26809899999995],[-102.32253700000001,38.26807000000008],[-102.31956600000001,38.26795800000002],[-102.30531099999996,38.267967000000056],[-102.25046699999996,38.26817100000005],[-102.2291699999999,38.268277000000126],[-102.21660399999996,38.268147000000056],[-102.20418599999994,38.26825600000018],[-102.19835099999995,38.26840500000009],[-102.18578299999996,38.26852800000029],[-102.18113599999987,38.2682660000001],[-102.17696999999998,38.268354999999985],[-102.15740099999988,38.26842299999993],[-102.14909699999998,38.26831700000025],[-102.13687999999996,38.268456000000015],[-102.12546199999991,38.268452000000025],[-102.12034899999998,38.26860000000022],[-102.11882300000002,38.26842099999999],[-102.11015900000001,38.268550000000005],[-102.08802599999996,38.268543000000136],[-102.08395999999999,38.268673000000206],[-102.066348,38.268643000000225],[-102.06417799999997,38.268774000000064],[-102.04447499999998,38.26874900000013]]]},"properties":{"OBJECTID":51,"COUNTYFP":"099","Shape_Leng":2.65950342287,"Shape_Area":0.436577933813,"low":3720,"lowmod":6680,"LMMI":8455,"lowmoduniv":11830,"lowmod_pct":0.564666103128,"NAMELSAD10":"Prowers County"}},
{"type":"Feature","geometry":{"type":"Polygon","coordinates":[[[-104.05392099999989,38.522393000000136],[-104.05419299999994,38.51961099999994],[-104.05465399999997,38.50254200000023],[-104.054712,38.491898000000106],[-104.0550169999999,38.484117000000026],[-104.05523499999998,38.47333200000014],[-104.05557699999997,38.46448600000008],[-104.05631399999993,38.4386820000002],[-104.0563489999999,38.43569000000008],[-104.05613700000004,38.41346500000009],[-104.05593699999991,38.40302200000002],[-104.05593599999992,38.392679999999984],[-104.055699,38.35692699999993],[-104.05548399999998,38.33614000000006],[-104.05542400000002,38.323657000000026],[-104.05500399999988,38.289355000000285],[-104.0547249999999,38.261889000000224],[-104.05756399999996,38.26164799999998],[-104.05840199999994,38.2608410000002],[-104.05848799999995,38.25220500000029],[-104.05829799999987,38.22582800000009],[-104.05845099999999,38.19996100000003],[-104.05834899999991,38.178716000000065],[-104.05822999999992,38.17407900000012],[-104.05837500000001,38.16561899999999],[-104.058242,38.14649199999997],[-104.057995,38.141468000000145],[-104.05868799999996,38.1148290000001],[-104.05880999999994,38.10544300000015],[-104.05904699999991,38.103723000000286],[-104.05908499999998,38.07628999999997],[-104.05939199999995,38.054326],[-104.05950100000001,37.996583000000214],[-104.05938300000003,37.990769],[-104.059459,37.979810999999984],[-104.05931599999997,37.95863900000006],[-104.05935699999998,37.94573000000008],[-104.059323,37.93019700000008],[-104.05937399999988,37.90010000000012],[-104.05937399999988,37.87500899999992],[-104.05947200000003,37.87090100000006],[-104.06006899999983,37.823030000000074],[-104.06189099999995,37.82271500000013],[-104.062073,37.79212300000006],[-104.06168599999995,37.7630410000001],[-104.06172699999996,37.75001100000003],[-104.06113199999987,37.73470400000025],[-104.10308499999996,37.74682199999995],[-104.12476300000003,37.75317100000001],[-104.12614899999988,37.75347099999999],[-104.16939299999996,37.765795000000026],[-104.24790599999994,37.7881560000003],[-104.25052499999993,37.78898500000014],[-104.312678,37.80659700000007],[-104.35110899999995,37.817488000000196],[-104.37520299999989,37.82427000000001],[-104.43998599999998,37.84257000000002],[-104.500269,37.85958600000015],[-104.52681899999993,37.86709799999994],[-104.58280099999996,37.88266299999998],[-104.59752500000002,37.88670000000002],[-104.62553399999996,37.894561000000124],[-104.64638300000001,37.900527000000125],[-104.70021199999996,37.89777800000002],[-104.75041899999991,37.89529500000009],[-104.78906499999988,37.893237],[-104.84623199999999,37.890199000000166],[-104.86834399999981,37.88918500000011],[-104.87554199999994,37.88868099999996],[-104.92122099999989,37.88635799999997],[-104.94577699999996,37.88506200000006],[-104.9482139999999,37.8850060000002],[-104.98774799999995,37.88284099999993],[-105.0137289999999,37.881271000000254],[-105.01376299999987,37.88348900000028],[-105.01575099999985,37.885509999999954],[-105.01961299999994,37.886967000000084],[-105.021748,37.887491000000296],[-105.0304349999999,37.89206300000001],[-105.03362600000003,37.893560000000264],[-105.0348029999999,37.89458300000001],[-105.03457499999985,37.89611200000002],[-105.03157299999998,37.90414700000031],[-105.03295400000002,37.90532200000018],[-105.03598999999997,37.90411500000016],[-105.03866499999987,37.90444400000001],[-105.039826,37.908124000000214],[-105.043905,37.909539999999936],[-105.04456099999993,37.91165600000005],[-105.04991699999988,37.915479000000005],[-105.05012399999987,37.984590000000026],[-105.04997299999997,37.98532400000016],[-105.05018999999999,38.0000060000001],[-105.04998099999983,38.006885000000125],[-105.04950999999994,38.03990500000003],[-105.05067799999995,38.11382400000008],[-105.05016499999999,38.11736100000019],[-105.04828700000002,38.12777299999999],[-105.04846099999992,38.15870000000001],[-105.04848599999997,38.17593800000026],[-105.04867899999994,38.22201800000016],[-105.04873099999998,38.239697999999976],[-105.04922299999993,38.25000300000016],[-105.04921499999989,38.25797300000022],[-105.02759400000002,38.25791700000002],[-105.00054099999994,38.25798099999997],[-104.9939819999999,38.258046000000036],[-104.94036599999998,38.25827499999997],[-104.94048299999997,38.28478899999999],[-104.94075099999986,38.338365000000124],[-104.94091300000002,38.37041200000016],[-104.94100400000002,38.37790800000005],[-104.94093299999992,38.38993599999998],[-104.94088599999992,38.43295300000011],[-104.94096100000002,38.45820300000031],[-104.94108499999987,38.46133800000018],[-104.9412989999999,38.47593400000011],[-104.94131199999993,38.499998999999946],[-104.941415,38.50315800000021],[-104.94153299999999,38.51956900000022],[-104.93294899999984,38.520286000000056],[-104.89568300000002,38.520300000000304],[-104.88834299999996,38.5199090000001],[-104.88197099999996,38.52008799999999],[-104.86667399999993,38.520017000000166],[-104.86615,38.51985700000006],[-104.84474699999998,38.51990600000022],[-104.83616699999988,38.51997999999992],[-104.82316499999985,38.5197790000002],[-104.81597599999986,38.52002900000008],[-104.79812199999992,38.51998400000008],[-104.79610400000001,38.520165000000134],[-104.78721699999994,38.52011900000025],[-104.77057899999988,38.51983899999999],[-104.75052999999997,38.519870999999966],[-104.7413499999999,38.51965200000001],[-104.72334000000001,38.519566],[-104.70694899999995,38.51954699999999],[-104.6843219999999,38.5194400000002],[-104.66037599999987,38.51975400000026],[-104.62565699999993,38.51994300000007],[-104.60410799999988,38.52029800000014],[-104.5909969999999,38.52000399999997],[-104.57371899999993,38.51993900000008],[-104.5670869999999,38.52000100000026],[-104.53263700000002,38.51984300000015],[-104.50052299999987,38.51989800000007],[-104.47999199999987,38.52028999999999],[-104.45648599999998,38.52032900000006],[-104.436511,38.520452999999975],[-104.42855299999985,38.52032500000024],[-104.41311799999994,38.520461000000125],[-104.410798,38.52061700000024],[-104.40023100000002,38.52061400000002],[-104.38180699999998,38.520733000000064],[-104.37552099999999,38.52060699999993],[-104.35418399999992,38.52080400000011],[-104.32649699999996,38.52087700000027],[-104.29851500000001,38.520991000000095],[-104.25051899999994,38.52130699999998],[-104.19352200000003,38.52183400000007],[-104.17450899999994,38.52218699999992],[-104.16528499999987,38.52227099999999],[-104.15250100000003,38.52190200000001],[-104.15015199999993,38.52173199999993],[-104.12877200000003,38.521832000000245],[-104.124347,38.521746000000235],[-104.09017899999998,38.522004000000265],[-104.08346799999993,38.522433999999976],[-104.05392099999989,38.522393000000136]]]},"properties":{"OBJECTID":52,"COUNTYFP":"101","Shape_Leng":3.4311983253,"Shape_Area":0.638492549717,"low":45835,"lowmod":74315,"LMMI":102805,"lowmoduniv":156710,"lowmod_pct":0.474219896624,"NAMELSAD10":"Pueblo County"}},
{"type":"Feature","geometry":{"type":"Polygon","coordinates":[[[-107.03736300000003,40.09153800000013],[-107.07419299999992,40.09152299999994],[-107.10183599999988,40.09146300000015],[-107.125607,40.09109000000001],[-107.16700300000002,40.09098300000022],[-107.21877899999987,40.090846000000056],[-107.25060999999994,40.090858000000026],[-107.31699300000002,40.090764999999976],[-107.317025,40.059552],[-107.31709499999982,40.003762000000165],[-107.3192469999999,40.003765000000044],[-107.31930599999998,39.999980999999934],[-107.31886099999986,39.93868700000013],[-107.3187509999999,39.91496799999999],[-107.32896199999993,39.9152490000003],[-107.38904000000002,39.916698999999994],[-107.43166499999995,39.91772500000002],[-107.431601,39.874980000000164],[-107.4317479999998,39.85465100000016],[-107.43177000000003,39.82837700000016],[-107.44751799999989,39.82831100000004],[-107.47653199999996,39.828267000000096],[-107.50061199999999,39.828308000000106],[-107.54470199999997,39.82808300000016],[-107.60915499999999,39.829342000000054],[-107.62561700000003,39.8295720000001],[-107.66395,39.8297850000003],[-107.691149,39.82999900000004],[-107.69422899999995,39.83010700000011],[-107.7279069999999,39.82826400000022],[-107.75062199999996,39.827146000000255],[-107.78720599999986,39.825046999999984],[-107.8387699999999,39.82490300000012],[-107.87167299999987,39.824816000000055],[-107.87562600000001,39.824952999999994],[-107.93850199999997,39.824996],[-107.93857999999994,39.79172899999992],[-107.93868199999986,39.74997500000006],[-107.93851599999988,39.73212600000005],[-107.93877199999991,39.70531300000022],[-107.93857099999991,39.70161099999996],[-107.93854599999997,39.69446399999998],[-107.94478499999997,39.6942830000001],[-107.96904399999988,39.69437399999998],[-107.98412999999994,39.69429500000001],[-108.00062899999995,39.69442600000002],[-108.00292699999994,39.69486299999994],[-108.01337699999993,39.69522400000017],[-108.03268800000001,39.69454400000001],[-108.05116099999998,39.695231000000206],[-108.06027399999994,39.69533999999999],[-108.069864,39.69521099999997],[-108.10754099999991,39.69510200000019],[-108.120971,39.695152000000064],[-108.12630999999999,39.695274999999924],[-108.16415799999999,39.69507500000009],[-108.17873900000001,39.695148000000074],[-108.217896,39.695137000000045],[-108.21791199999996,39.69385500000004],[-108.25493699999993,39.69404099999997],[-108.27281299999999,39.69436499999995],[-108.29255699999999,39.69425799999999],[-108.31131099999988,39.69471400000009],[-108.32819499999994,39.694330000000264],[-108.35062399999987,39.69415300000003],[-108.37564399999997,39.69420800000012],[-108.38704799999994,39.694300999999996],[-108.3959129999999,39.69424400000025],[-108.42204099999998,39.694222000000025],[-108.49509199999994,39.693963999999994],[-108.51728899999995,39.69416200000023],[-108.52856799999995,39.693958000000066],[-108.55472199999991,39.693640000000016],[-108.55462699999987,39.650164000000075],[-108.60153699999995,39.64988000000022],[-108.61607899999996,39.64999400000005],[-108.64379400000001,39.64978100000002],[-108.68142499999993,39.64971200000002],[-108.71536299999991,39.649780000000305],[-108.73030499999987,39.64998000000014],[-108.73614299999991,39.64987899999994],[-108.75065599999999,39.65006900000003],[-108.78069399999987,39.65007400000013],[-108.78844600000002,39.65015900000003],[-108.81980399999986,39.65015400000016],[-108.8519649999999,39.65032700000006],[-108.858247,39.65109600000011],[-108.87571899999995,39.650198000000046],[-108.88604199999992,39.650473999999974],[-108.8886599999999,39.65032200000002],[-108.89572800000002,39.650337000000206],[-108.89570399999991,39.66048700000016],[-108.95567599999993,39.66036800000006],[-108.99281400000001,39.66029300000008],[-109.01023099999992,39.660392000000286],[-109.05122699999993,39.66046600000004],[-109.05131599999999,39.6760290000002],[-109.05109899999997,39.72644400000013],[-109.05104299999988,39.75560300000029],[-109.05105299999991,39.787843000000294],[-109.05095599999987,39.821031000000005],[-109.05034099999989,39.84935400000029],[-109.05042300000002,39.88316600000013],[-109.05049700000001,39.89864400000005],[-109.05057999999985,39.9319900000001],[-109.05068799999998,39.95784700000007],[-109.05115699999999,40.00823800000012],[-109.05088299999989,40.03498300000001],[-109.05100099999987,40.04692799999998],[-109.05086399999999,40.06382999999994],[-109.05076299999979,40.09260400000005],[-109.05102099999982,40.119884000000184],[-109.05093899999997,40.124385000000075],[-109.05098299999997,40.14802000000009],[-109.05095299999988,40.173397000000136],[-109.05088699999993,40.194350999999926],[-109.05097599999993,40.209204],[-109.050949,40.22264700000011],[-109.02771799999988,40.2226150000003],[-109.0269409999998,40.222715999999934],[-109.00833799999998,40.222629999999924],[-109.00067099999995,40.2225160000001],[-108.96508599999993,40.22258699999992],[-108.90247499999998,40.22270800000018],[-108.87566600000002,40.222769000000255],[-108.82414499999993,40.22279000000003],[-108.75066200000003,40.22282200000001],[-108.70727799999992,40.22266500000018],[-108.6879189999998,40.222465],[-108.62565199999995,40.22236100000009],[-108.53541899999999,40.22236600000031],[-108.50065199999989,40.222369000000015],[-108.42459500000001,40.22215500000027],[-108.37564799999996,40.222019000000046],[-108.32479099999995,40.22193699999997],[-108.26889899999998,40.221847000000025],[-108.21387599999997,40.221757000000196],[-108.15966099999986,40.22167300000001],[-108.12556199999989,40.221705000000156],[-108.113179,40.221670000000074],[-108.06375200000002,40.22186700000003],[-108.01191999999998,40.2220760000003],[-108.00063399999993,40.22230399999995],[-107.99509499999994,40.22220800000002],[-107.99129099999976,40.22239100000024],[-107.965419,40.22226600000022],[-107.94239899999991,40.22209000000021],[-107.91999499999991,40.22213100000005],[-107.875629,40.22239300000024],[-107.86025399999988,40.22235100000012],[-107.8415589999999,40.222541999999976],[-107.83977600000003,40.222442],[-107.80961899999988,40.222577],[-107.80817599999995,40.22275100000019],[-107.78854599999988,40.22270600000002],[-107.78053599999998,40.222507000000064],[-107.764544,40.2226150000003],[-107.764545,40.22370100000012],[-107.75360899999993,40.22386200000028],[-107.74540499999995,40.223433000000114],[-107.71938599999999,40.223605000000134],[-107.67939799999982,40.22358500000007],[-107.66876000000002,40.223396000000264],[-107.65398699999992,40.22345200000029],[-107.65394099999992,40.22427499999998],[-107.63404499999996,40.224131],[-107.61209399999996,40.223651000000075],[-107.60333999999995,40.223116000000005],[-107.58371099999988,40.222532],[-107.56186599999995,40.22216500000002],[-107.52502400000003,40.22116700000004],[-107.50666799999993,40.220507],[-107.50061499999987,40.220507],[-107.49561799999987,40.21999100000028],[-107.45031899999992,40.22040600000014],[-107.43937199999993,40.22045400000002],[-107.43938600000001,40.22337900000008],[-107.41488399999997,40.22363400000006],[-107.40019000000001,40.22341300000005],[-107.36941300000001,40.22334600000022],[-107.35254099999997,40.223464000000206],[-107.31305099999997,40.22390300000029],[-107.267788,40.224154999999996],[-107.25060999999994,40.224199999999996],[-107.24715300000003,40.22428400000001],[-107.18755299999992,40.22472700000026],[-107.12414799999999,40.22516500000006],[-107.09606199999996,40.22513600000002],[-107.09608800000001,40.22606200000007],[-107.07655199999988,40.22534700000023],[-107.05972300000002,40.225305000000105],[-107.05603499999995,40.225173999999924],[-107.04714899999988,40.225219000000095],[-107.04038800000001,40.22545700000006],[-107.03792799999997,40.22537200000016],[-107.03778899999992,40.21040599999998],[-107.03799100000003,40.20284400000014],[-107.03824999999995,40.17327499999999],[-107.03840100000002,40.165645000000154],[-107.03768600000001,40.16561699999994],[-107.03763299999997,40.12503800000013],[-107.03741599999995,40.11101600000006],[-107.03736300000003,40.09153800000013]]]},"properties":{"OBJECTID":53,"COUNTYFP":"103","Shape_Leng":5.18126912625,"Shape_Area":0.88009231474,"low":1610,"lowmod":2595,"LMMI":4145,"lowmoduniv":6155,"lowmod_pct":0.421608448416,"NAMELSAD10":"Rio Blanco County"}},
{"type":"Feature","geometry":{"type":"Polygon","coordinates":[[[-106.03841299999988,37.74835300000018],[-106.03857099999988,37.7242940000001],[-106.03886799999987,37.675849000000085],[-106.03908899999993,37.64548599999995],[-106.03916500000003,37.61783800000012],[-106.03921699999995,37.58367100000004],[-106.03931499999999,37.57482200000027],[-106.03927499999998,37.564571000000115],[-106.03926899999999,37.52922100000001],[-106.03928699999989,37.48452300000025],[-106.03934399999997,37.464533000000074],[-106.03934199999992,37.4314950000001],[-106.03939299999996,37.41350800000009],[-106.03933099999989,37.40085200000027],[-106.06113299999998,37.40106000000026],[-106.07809600000002,37.401033000000155],[-106.09406599999994,37.401112000000296],[-106.14886599999988,37.401099999999985],[-106.162599,37.40104999999994],[-106.17390599999993,37.40108399999991],[-106.17632000000003,37.400938999999994],[-106.24478499999992,37.40090100000009],[-106.25058699999994,37.40095400000018],[-106.288002,37.40106900000029],[-106.35024399999992,37.40127799999999],[-106.35087499999992,37.401044000000184],[-106.36918400000002,37.400847000000226],[-106.37558999999999,37.400667999999996],[-106.42359499999992,37.400092000000086],[-106.48796800000002,37.39934999999997],[-106.48797000000002,37.40158500000018],[-106.52188999999981,37.402206000000035],[-106.54458199999993,37.402262000000235],[-106.56366699999995,37.402247000000045],[-106.57610699999992,37.40233500000005],[-106.5884779999999,37.4027200000001],[-106.61435099999994,37.402221000000054],[-106.62659399999995,37.402415000000076],[-106.64698099999998,37.40325400000006],[-106.64781099999988,37.40315500000003],[-106.67837299999997,37.403663000000165],[-106.67837299999997,37.40359600000011],[-106.70586800000001,37.404171000000076],[-106.71077499999984,37.40422799999999],[-106.71077399999996,37.40431799999999],[-106.710778,37.40723700000012],[-106.710781,37.40942299999995],[-106.71078499999999,37.41218500000008],[-106.71078999999986,37.416168000000084],[-106.71080099999995,37.42535900000013],[-106.710825,37.44823500000001],[-106.71085599999998,37.47244300000011],[-106.71086200000002,37.47743900000012],[-106.71086400000002,37.47933800000004],[-106.71086700000001,37.48161099999999],[-106.71087299999999,37.48673400000007],[-106.71087699999987,37.49406299999998],[-106.71087799999992,37.49771199999992],[-106.71076199999993,37.50997799999999],[-106.71079199999991,37.524373000000026],[-106.71083699999991,37.53145800000004],[-106.710914,37.55107300000009],[-106.71098599999999,37.559808999999916],[-106.71099500000003,37.560963000000015],[-106.71101999999996,37.56505900000002],[-106.711116,37.58862600000003],[-106.7111339999999,37.59281900000002],[-106.71120000000002,37.60056700000001],[-106.71115800000001,37.60759400000006],[-106.71110799999997,37.608573999999976],[-106.71109999999993,37.60873900000013],[-106.71109499999994,37.6088400000001],[-106.71109699999994,37.61011000000002],[-106.71126799999996,37.61528299999992],[-106.71160299999997,37.62500499999999],[-106.7118349999999,37.63229100000012],[-106.71177599999999,37.64111100000002],[-106.71181199999995,37.64155900000014],[-106.71168599999999,37.65674000000013],[-106.71162400000003,37.664273000000094],[-106.69563800000003,37.6642710000001],[-106.6956429999999,37.665369],[-106.69574499999999,37.68507899999997],[-106.69577099999992,37.69017500000007],[-106.69577399999997,37.6906800000001],[-106.69556899999998,37.72252700000007],[-106.69525099999998,37.750001999999995],[-106.69469199999997,37.751318000000026],[-106.69460399999991,37.7570300000001],[-106.6944299999999,37.77000599999991],[-106.6944309999999,37.78035100000022],[-106.69432099999995,37.782876999999985],[-106.69430899999992,37.784519999999986],[-106.69424099999998,37.79474700000014],[-106.69419799999997,37.800877000000014],[-106.69406899999996,37.817703999999935],[-106.6938449999999,37.8352440000001],[-106.62559599999997,37.834494000000234],[-106.61911099999998,37.83476600000017],[-106.60447599999992,37.83466499999997],[-106.58713899999992,37.83442600000029],[-106.58717799999988,37.79225800000023],[-106.58703199999997,37.78610100000026],[-106.58645199999984,37.77623400000027],[-106.58656799999994,37.75000100000028],[-106.58637399999998,37.747809000000075],[-106.546426,37.748289],[-106.53228699999988,37.74839900000006],[-106.52721099999991,37.74830800000018],[-106.486674,37.748071000000095],[-106.47784300000001,37.74831500000022],[-106.42315999999988,37.74828600000012],[-106.40503999999999,37.74836000000022],[-106.35085300000003,37.74840700000004],[-106.33255199999996,37.74823200000009],[-106.29134399999987,37.74817800000005],[-106.287665,37.748303000000305],[-106.25649299999992,37.74815400000023],[-106.23555299999981,37.74808899999999],[-106.23175399999991,37.74813600000016],[-106.20480999999995,37.74807700000025],[-106.18779899999987,37.74815500000011],[-106.136685,37.74815600000022],[-106.10240299999998,37.748212000000024],[-106.03841299999988,37.74835300000018]]]},"properties":{"OBJECTID":54,"COUNTYFP":"105","Shape_Leng":2.21073141503,"Shape_Area":0.241033047455,"low":3585,"lowmod":5945,"LMMI":8630,"lowmoduniv":11520,"lowmod_pct":0.516059027778,"NAMELSAD10":"Rio Grande County"}},
{"type":"Feature","geometry":{"type":"Polygon","coordinates":[[[-106.8577729999999,41.002663000000155],[-106.86145399999998,40.99996800000008],[-106.86160799999993,40.996802],[-106.85985499999975,40.9953040000002],[-106.85709500000002,40.99385500000005],[-106.85086499999994,40.99127900000008],[-106.85073599999993,40.990247000000295],[-106.854222,40.9883750000003],[-106.85542900000002,40.98708500000015],[-106.85581599999989,40.983948999999996],[-106.85740199999992,40.98127999999997],[-106.85936799999996,40.98319700000013],[-106.86230699999999,40.983703000000276],[-106.86703499999982,40.98264400000022],[-106.87058400000001,40.98122300000006],[-106.871374,40.9780870000003],[-106.87282199999993,40.975877000000025],[-106.87321999999989,40.973032000000046],[-106.87246299999987,40.97108200000014],[-106.87274599999984,40.96938100000011],[-106.87245399999995,40.96656000000007],[-106.87012499999997,40.96260400000017],[-106.8695239999999,40.959001000000114],[-106.86989099999988,40.95672000000002],[-106.86831899999993,40.955125000000066],[-106.86462899999998,40.95419199999992],[-106.86399199999994,40.953701000000024],[-106.86301200000003,40.95086500000002],[-106.86219999999992,40.94727800000004],[-106.86211999999995,40.944329000000096],[-106.86246599999998,40.941927000000135],[-106.86210399999987,40.94092800000027],[-106.859376,40.938423000000114],[-106.85644899999994,40.93758599999995],[-106.855887,40.936982000000114],[-106.85611599999993,40.93497500000029],[-106.85224699999992,40.933792000000096],[-106.85238800000002,40.92972900000001],[-106.850797,40.928537000000006],[-106.84876999999989,40.926306000000125],[-106.84999799999997,40.925451000000066],[-106.85000599999984,40.92420100000004],[-106.84560999999991,40.92505800000026],[-106.840555,40.924730000000295],[-106.83718399999998,40.92365900000016],[-106.83384699999999,40.92420900000002],[-106.83120799999995,40.92392000000007],[-106.82857100000001,40.92466200000018],[-106.82640200000003,40.92707300000018],[-106.82608500000003,40.92801600000007],[-106.823534,40.93028199999998],[-106.82272599999999,40.92809800000026],[-106.81997899999999,40.92530999999997],[-106.81859799999995,40.922134000000085],[-106.8182569999999,40.92025599999994],[-106.81900699999994,40.91863600000022],[-106.81529799999993,40.9173390000002],[-106.81189499999999,40.91645300000005],[-106.81058999999993,40.91417300000006],[-106.80983500000002,40.91140100000001],[-106.80845499999987,40.908547],[-106.80633499999988,40.90664499999997],[-106.80592899999994,40.90303399999999],[-106.80534499999993,40.90197800000004],[-106.807526,40.900551000000064],[-106.807324,40.89910800000007],[-106.80489599999999,40.89731100000023],[-106.80206799999996,40.896908999999994],[-106.80063799999994,40.895901000000265],[-106.797821,40.89646599999992],[-106.795365,40.89791700000012],[-106.79425500000002,40.90056100000004],[-106.79200900000001,40.89995600000003],[-106.78949899999998,40.900779000000284],[-106.7878889999999,40.90056100000004],[-106.788589,40.90284200000008],[-106.78697899999992,40.90290700000014],[-106.78314599999993,40.90613100000007],[-106.780911,40.90704200000016],[-106.77867600000002,40.90642900000023],[-106.77780699999988,40.905437000000234],[-106.77636599999994,40.905743000000314],[-106.77315599999997,40.905283999999995],[-106.77117499999997,40.90428400000002],[-106.769004,40.904138000000216],[-106.76774199999994,40.90216300000003],[-106.76754,40.89864099999994],[-106.77007099999992,40.896997],[-106.76674499999996,40.89619800000014],[-106.76493399999987,40.89636700000028],[-106.76327100000003,40.895714000000055],[-106.76163999999989,40.89417400000019],[-106.75647100000003,40.89384200000006],[-106.75148200000001,40.894018000000074],[-106.74678899999992,40.892979999999966],[-106.74356,40.891994000000295],[-106.74085300000002,40.88979999999998],[-106.744644,40.88457100000005],[-106.7468199999999,40.88233200000002],[-106.74587099999991,40.87980000000016],[-106.74420999999995,40.87911400000024],[-106.7422019999999,40.877105000000086],[-106.74263199999996,40.87497100000013],[-106.74094100000002,40.8723260000001],[-106.73934099999997,40.87084300000015],[-106.73898799999989,40.87269599999996],[-106.73659199999997,40.87310000000019],[-106.7283329999999,40.871241000000055],[-106.72686899999991,40.871375000000285],[-106.72185400000001,40.87295599999999],[-106.71359799999993,40.87399600000026],[-106.7057559999999,40.87600400000014],[-106.70454699999988,40.87665600000025],[-106.70355699999993,40.878686000000016],[-106.70346799999993,40.88043499999992],[-106.70018899999997,40.882648000000074],[-106.700376,40.88418799999994],[-106.69926099999992,40.88537200000002],[-106.69415900000001,40.88801799999993],[-106.69166899999988,40.88819200000012],[-106.68758600000001,40.88630800000021],[-106.68588299999999,40.88582200000013],[-106.685293,40.88460400000008],[-106.68600700000002,40.88291200000026],[-106.68616199999997,40.88061500000026],[-106.6847699999999,40.87866200000013],[-106.68269699999996,40.877636000000166],[-106.68220500000001,40.87578900000028],[-106.68030299999987,40.874303000000054],[-106.68053599999996,40.87227600000023],[-106.6794789999999,40.86955500000016],[-106.67798699999992,40.86786999999998],[-106.675475,40.86697300000003],[-106.67350399999998,40.867057000000045],[-106.66816099999994,40.86514400000004],[-106.6679769999999,40.86279600000029],[-106.66709900000001,40.86096299999997],[-106.66736299999991,40.85979099999997],[-106.66596800000002,40.858233999999925],[-106.66448100000002,40.859082000000114],[-106.66405500000002,40.85682500000007],[-106.66159500000003,40.85571800000014],[-106.65998000000002,40.854395000000125],[-106.65755100000001,40.8532390000002],[-106.65454099999994,40.849117000000206],[-106.654472,40.84554300000025],[-106.65889700000002,40.84397600000011],[-106.662644,40.84334100000012],[-106.66130399999997,40.84090400000014],[-106.66270499999996,40.8379510000002],[-106.66350299999999,40.837211999999965],[-106.66404099999983,40.83496600000012],[-106.66289,40.83231700000016],[-106.66399199999995,40.82819600000005],[-106.66003399999994,40.826653000000306],[-106.65845899999982,40.82392699999997],[-106.65503699999994,40.82309100000009],[-106.65294199999988,40.8234020000001],[-106.65130699999997,40.82306399999999],[-106.64713499999999,40.821296000000075],[-106.64472799999999,40.82110799999998],[-106.64300399999996,40.82036700000015],[-106.63839300000001,40.817004000000054],[-106.63932399999987,40.81582900000018],[-106.639227,40.81452200000007],[-106.638102,40.813303000000076],[-106.63675199999994,40.81288299999994],[-106.63547599999998,40.81136500000031],[-106.63713899999999,40.80920900000007],[-106.63675399999994,40.80562800000007],[-106.63553799999988,40.804756000000054],[-106.63649999999978,40.80345900000003],[-106.6358239999999,40.802616000000285],[-106.63688499999995,40.79959200000019],[-106.63651699999997,40.7966650000003],[-106.63689399999998,40.79415300000011],[-106.63596099999995,40.79193099999992],[-106.63692199999997,40.78948900000006],[-106.63927100000001,40.78818499999994],[-106.63992699999994,40.78705100000013],[-106.64254799999998,40.78550300000029],[-106.64270099999993,40.78486500000025],[-106.64749699999993,40.782651000000214],[-106.64928299999991,40.780099000000064],[-106.6497839999999,40.777312999999936],[-106.64916099999994,40.77538800000002],[-106.65238299999999,40.77562000000006],[-106.65460599999989,40.77455000000015],[-106.6592369999999,40.772861000000034],[-106.66206199999988,40.77057700000023],[-106.66348899999991,40.76834200000002],[-106.66332699999992,40.76697900000005],[-106.66479399999997,40.76553400000023],[-106.66892899999988,40.76291199999997],[-106.6699809999999,40.76030800000001],[-106.66949799999998,40.75965699999995],[-106.66607399999992,40.75828100000001],[-106.6642349999999,40.756451000000084],[-106.66343699999987,40.754076000000225],[-106.66187099999996,40.75408500000026],[-106.66199099999994,40.75239799999997],[-106.66417799999999,40.750965000000065],[-106.66518899999988,40.748857000000044],[-106.66428499999995,40.748118000000204],[-106.662556,40.74476900000002],[-106.66295199999996,40.74301800000029],[-106.66499699999997,40.74214000000029],[-106.66832599999981,40.739120000000185],[-106.67114500000002,40.73865900000004],[-106.67474299999998,40.73667800000027],[-106.67669399999994,40.7366310000001],[-106.676425,40.7355510000001],[-106.67382599999996,40.73510900000031],[-106.67052799999993,40.73216700000006],[-106.66952000000003,40.728791],[-106.67017299999998,40.72768400000007],[-106.67244699999992,40.72618400000027],[-106.67380700000001,40.72402499999998],[-106.67331499999989,40.72297000000009],[-106.6719359999999,40.72281399999997],[-106.67037299999998,40.72139900000013],[-106.66978399999994,40.71978800000005],[-106.6711929999999,40.716379000000074],[-106.67046399999998,40.714131000000066],[-106.66887599999995,40.711725000000115],[-106.668924,40.71025700000018],[-106.6724789999999,40.70847800000024],[-106.67253699999998,40.70691300000004],[-106.67190299999993,40.70451100000008],[-106.67099699999994,40.70306300000004],[-106.6707669999999,40.70104700000019],[-106.66856499999989,40.698618000000295],[-106.66795299999995,40.69630500000005],[-106.67022899999989,40.69284499999992],[-106.67256099999992,40.690143000000205],[-106.67286199999995,40.688336000000106],[-106.67517900000001,40.68721500000004],[-106.67841900000002,40.686535000000276],[-106.67947600000002,40.685918000000015],[-106.67967099999998,40.68422299999992],[-106.68145800000002,40.68324899999999],[-106.68449599999997,40.68267400000002],[-106.68734599999988,40.682679000000064],[-106.68707299999994,40.68060799999995],[-106.68467599999985,40.679616000000124],[-106.68352599999992,40.67823300000009],[-106.68239099999988,40.677898000000084],[-106.68105700000001,40.675167999999985],[-106.686083,40.67370700000026],[-106.68723099999994,40.671944000000224],[-106.6860989999999,40.666673],[-106.68576499999989,40.662326000000235],[-106.68850499999985,40.661187000000155],[-106.69050199999987,40.659906000000205],[-106.69182599999999,40.659958000000245],[-106.69208200000003,40.65774600000003],[-106.69126799999992,40.655441999999994],[-106.68865599999998,40.650773000000015],[-106.68773099999993,40.64987299999996],[-106.68518799999993,40.64944600000018],[-106.68206399999991,40.649675000000116],[-106.68065399999995,40.64928400000008],[-106.68014199999976,40.64803600000022],[-106.68284800000004,40.64686499999999],[-106.68348599999996,40.64481400000011],[-106.68601599999988,40.64144100000004],[-106.68865599999998,40.639649000000304],[-106.688875,40.63877800000017],[-106.687117,40.635775000000024],[-106.69116600000001,40.634446000000025],[-106.694209,40.632902],[-106.69716199999999,40.62993200000011],[-106.696958,40.62920600000007],[-106.69826399999994,40.627482999999984],[-106.70284899999996,40.62770900000021],[-106.70101099999994,40.6259570000002],[-106.70395699999989,40.62183800000008],[-106.70379000000003,40.62013000000002],[-106.70614899999998,40.61659700000024],[-106.70600000000002,40.614399000000276],[-106.70374099999998,40.61055300000021],[-106.70365800000002,40.608620000000144],[-106.70434799999992,40.60738600000002],[-106.70234499999992,40.60599600000012],[-106.701862,40.60440199999999],[-106.69872299999992,40.602039000000104],[-106.69826699999993,40.599390000000085],[-106.6952609999999,40.598194999999976],[-106.69414799999993,40.59524099999999],[-106.69403499999999,40.59355699999992],[-106.6947659999999,40.59200900000013],[-106.69800299999986,40.5887640000002],[-106.6997,40.586022000000014],[-106.69864199999995,40.58349400000026],[-106.69992200000002,40.58196100000009],[-106.70014599999996,40.580269000000044],[-106.70234099999976,40.578025000000025],[-106.69889599999993,40.57514800000001],[-106.69713100000001,40.57266199999998],[-106.696867,40.57054400000027],[-106.69417799999991,40.56691500000005],[-106.69218799999993,40.56353700000011],[-106.69180699999993,40.56116900000012],[-106.69285999999994,40.55858100000023],[-106.6928509999999,40.5566390000003],[-106.68770999999987,40.552105000000154],[-106.68621799999988,40.55132700000007],[-106.68473999999998,40.54892100000012],[-106.683378,40.54453200000006],[-106.67911099999992,40.544489000000056],[-106.67736299999996,40.54246500000011],[-106.67732999999998,40.539734000000294],[-106.6766889999999,40.538035000000036],[-106.67468899999977,40.53693499999997],[-106.67168800000002,40.53584400000011],[-106.66984500000001,40.534446000000116],[-106.66948799999994,40.53248100000002],[-106.67135599999989,40.530157000000145],[-106.67328199999997,40.52693100000005],[-106.67369999999994,40.52368500000006],[-106.67206399999992,40.521214000000214],[-106.66947500000003,40.520139000000086],[-106.667101,40.520143000000246],[-106.66177700000003,40.515963000000056],[-106.658974,40.51427600000028],[-106.66079299999996,40.51295900000002],[-106.66110500000002,40.511944000000085],[-106.65928499999995,40.510577000000296],[-106.65580799999998,40.50914099999994],[-106.65545199999991,40.507103000000086],[-106.65558299999998,40.50341300000031],[-106.65631499999989,40.501768000000084],[-106.65604199999996,40.49997799999994],[-106.65481799999992,40.49902900000012],[-106.65460999999988,40.49644300000023],[-106.6531129999999,40.494732000000226],[-106.65392499999996,40.49243900000022],[-106.65761899999995,40.491209000000026],[-106.65931,40.49019800000025],[-106.65788299999997,40.48913900000019],[-106.65729399999998,40.48702799999995],[-106.65618499999988,40.48560800000007],[-106.65623299999987,40.48440800000026],[-106.65742999999998,40.483154999999954],[-106.66328099999993,40.480272000000014],[-106.66387099999997,40.47816300000011],[-106.66477199999991,40.47711799999996],[-106.66706299999987,40.47606100000007],[-106.66728099999995,40.47264700000005],[-106.665301,40.47052400000007],[-106.66348899999991,40.470582999999976],[-106.65960699999988,40.47174899999999],[-106.65788099999997,40.47155900000013],[-106.65844699999991,40.466593999999986],[-106.65745999999996,40.4624300000001],[-106.65895699999982,40.45937599999996],[-106.65649499999995,40.45604300000025],[-106.65641199999993,40.45149500000002],[-106.65580599999998,40.45019100000013],[-106.65385500000002,40.44951100000014],[-106.65152499999988,40.449377000000084],[-106.65211199999987,40.445231000000035],[-106.65218700000003,40.44520700000021],[-106.652601,40.403019000000086],[-106.65295699999996,40.36440499999992],[-106.65283199999993,40.341562000000295],[-106.64987199999996,40.34163799999999],[-106.63225699999992,40.341559000000075],[-106.6323779999999,40.32877500000029],[-106.63224999999994,40.32318000000009],[-106.63247999999999,40.3111370000002],[-106.63232399999987,40.30975700000005],[-106.632477,40.28355600000003],[-106.63248499999997,40.26697700000017],[-106.63272099999995,40.22994299999999],[-106.63298700000001,40.203864000000124],[-106.6331209999999,40.1789960000001],[-106.6369949999999,40.17898500000007],[-106.63710900000001,40.12498499999998],[-106.63697400000001,40.12357800000029],[-106.63700299999994,40.058168999999964],[-106.63704699999994,40.03635700000001],[-106.63703099999987,40.00210600000008],[-106.62653,40.001870999999994],[-106.62611200000003,39.999986000000035],[-106.62657300000001,39.95199900000006],[-106.62656900000002,39.9247850000001],[-106.62655499999994,39.91867100000013],[-106.653257,39.91996999999992],[-106.6536529999999,39.91992700000009],[-106.71298300000001,39.920618000000104],[-106.72531099999998,39.92060800000013],[-106.74851100000001,39.92012699999998],[-106.75058999999999,39.92002300000024],[-106.76631099999992,39.91983400000004],[-106.77423299999992,39.9199280000002],[-106.78554999999989,39.919811000000095],[-106.79236499999996,39.9200000000003],[-106.79856399999994,39.919825],[-106.8331649999999,39.91940599999998],[-106.836974,39.91919700000028],[-106.86030299999987,39.91917000000018],[-106.87607199999997,39.918814000000225],[-106.90945199999999,39.91885199999996],[-106.91506599999985,39.918911000000094],[-106.93617199999994,39.91874100000001],[-106.95635500000003,39.91881600000005],[-106.96506599999992,39.9187850000003],[-106.98359099999988,39.91886900000014],[-106.99356999999998,39.91879700000004],[-107.00045599999999,39.91904800000003],[-107.03396799999996,39.91891300000003],[-107.03217099999995,39.93966900000021],[-107.032648,39.96372500000001],[-107.03177199999988,39.963727000000006],[-107.031971,39.983894000000134],[-107.03204999999997,40.00293900000008],[-107.03735899999987,40.00308100000029],[-107.03724799999992,40.053607],[-107.03729499999997,40.05656099999999],[-107.03736300000003,40.09153800000013],[-107.03741599999995,40.11101600000006],[-107.03763299999997,40.12503800000013],[-107.03768600000001,40.16561699999994],[-107.03840100000002,40.165645000000154],[-107.03824999999995,40.17327499999999],[-107.03799100000003,40.20284400000014],[-107.03778899999992,40.21040599999998],[-107.03792799999997,40.22537200000016],[-107.04038800000001,40.22545700000006],[-107.04714899999988,40.225219000000095],[-107.05603499999995,40.225173999999924],[-107.05972300000002,40.225305000000105],[-107.07655199999988,40.22534700000023],[-107.09608800000001,40.22606200000007],[-107.09606199999996,40.22513600000002],[-107.12414799999999,40.22516500000006],[-107.18755299999992,40.22472700000026],[-107.24715300000003,40.22428400000001],[-107.25060999999994,40.224199999999996],[-107.267788,40.224154999999996],[-107.31305099999997,40.22390300000029],[-107.35254099999997,40.223464000000206],[-107.36941300000001,40.22334600000022],[-107.40019000000001,40.22341300000005],[-107.41488399999997,40.22363400000006],[-107.43938600000001,40.22337900000008],[-107.439506,40.247327999999925],[-107.4401089999999,40.249976000000004],[-107.4425179999999,40.30843900000025],[-107.44259399999999,40.3347550000002],[-107.4320679999999,40.334878999999944],[-107.43152499999991,40.355424000000255],[-107.43107199999992,40.37496900000002],[-107.43123600000001,40.37578000000002],[-107.42934500000001,40.440770999999984],[-107.42910699999987,40.44453399999992],[-107.42850499999997,40.46976000000029],[-107.42838799999998,40.48021],[-107.428516,40.484947999999974],[-107.42852399999987,40.49997300000007],[-107.42869999999994,40.51750099999998],[-107.42881299999999,40.542207000000076],[-107.37561599999998,40.54241000000019],[-107.371328,40.54239900000016],[-107.37141500000001,40.59625300000005],[-107.37148699999995,40.600302000000056],[-107.34855500000003,40.60026600000009],[-107.31415300000003,40.60028100000028],[-107.31428800000003,40.62497099999996],[-107.31336499999986,40.686553000000174],[-107.31356599999987,40.694074],[-107.3145409999999,40.74996900000008],[-107.31509899999998,40.786572999999976],[-107.31532300000003,40.797995000000014],[-107.31608799999992,40.84484500000008],[-107.31653900000003,40.874950000000126],[-107.31688599999995,40.908011000000215],[-107.31741099999988,40.95741200000009],[-107.31786299999999,40.9999650000002],[-107.31779399999994,41.002920000000074],[-107.30717199999992,41.002949],[-107.27496400000001,41.00282900000002],[-107.25061199999993,41.00288100000006],[-107.2411939999999,41.00280400000008],[-107.21690699999999,41.002832000000126],[-107.20696899999996,41.00289800000007],[-107.17115799999999,41.00296100000014],[-107.12561399999987,41.003055000000074],[-107.10226199999994,41.00311999999997],[-107.09200099999987,41.00324500000016],[-107.05018100000001,41.00332700000001],[-107.00311899999986,41.003421],[-106.97415299999994,41.00337200000001],[-106.90130199999993,41.003093000000206],[-106.8755999999999,41.00299600000017],[-106.8577729999999,41.002663000000155]]]},"properties":{"OBJECTID":55,"COUNTYFP":"107","Shape_Leng":4.1050889935,"Shape_Area":0.651459288189,"low":5685,"lowmod":9950,"LMMI":14250,"lowmoduniv":23200,"lowmod_pct":0.428879310345,"NAMELSAD10":"Routt County"}},
{"type":"Feature","geometry":{"type":"Polygon","coordinates":[[[-106.01075100000003,38.446566000000246],[-106.01230399999992,38.44535700000023],[-106.0127,38.44412399999999],[-106.012789,38.44077699999997],[-106.01230699999991,38.4389450000001],[-106.01101599999993,38.437231000000224],[-106.00866200000002,38.43545899999998],[-106.00793399999992,38.43371500000012],[-106.0064769999999,38.43251800000019],[-106.00193000000002,38.431466],[-106.001038,38.42992300000003],[-106.00030300000003,38.4270200000002],[-106.00081399999993,38.425809000000186],[-105.99978799999997,38.42362200000025],[-105.997751,38.42203100000023],[-105.99676799999992,38.420303000000104],[-105.99393199999992,38.41937500000006],[-105.992008,38.41956900000008],[-105.98730799999987,38.41880200000003],[-105.98349999999999,38.419392000000016],[-105.98040899999995,38.41727700000001],[-105.97922199999988,38.41412700000012],[-105.97691899999995,38.41298000000006],[-105.974647,38.41111499999994],[-105.973389,38.409459000000254],[-105.97022600000003,38.41004000000004],[-105.96793300000002,38.408772000000056],[-105.964721,38.40626000000009],[-105.963391,38.404750000000035],[-105.96066899999994,38.40309500000001],[-105.95981999999987,38.40127800000016],[-105.95994499999989,38.39802299999997],[-105.95949499999978,38.39709500000026],[-105.95717199999996,38.395180000000266],[-105.95566799999995,38.394437000000096],[-105.95065399999999,38.39373300000011],[-105.94996899999995,38.39233599999994],[-105.94760600000001,38.39052600000031],[-105.947067,38.38549500000005],[-105.94543199999998,38.38274000000001],[-105.94463699999994,38.37863800000025],[-105.94043299999998,38.376972999999964],[-105.938536,38.37407400000029],[-105.93576999999999,38.37321900000023],[-105.93262400000003,38.37043200000005],[-105.93135999999998,38.369791000000134],[-105.92700999999994,38.36871999999994],[-105.92463799999979,38.368737000000294],[-105.92406999999997,38.3679360000001],[-105.91985099999994,38.36715600000002],[-105.918406,38.36387400000012],[-105.91696899999988,38.36294099999998],[-105.912489,38.361539000000164],[-105.91080399999993,38.35924100000028],[-105.91023599999994,38.35641300000003],[-105.90844699999997,38.35447800000003],[-105.90630699999991,38.35326800000007],[-105.90260199999989,38.3523850000002],[-105.899831,38.35072100000019],[-105.89764699999989,38.35005100000018],[-105.895423,38.34741200000013],[-105.8934779999999,38.346292000000005],[-105.89257399999997,38.34334200000018],[-105.89076499999987,38.34137400000009],[-105.88876600000003,38.3408110000002],[-105.88829099999998,38.33788600000014],[-105.88886599999995,38.33731499999993],[-105.88875999999988,38.33436000000023],[-105.88691999999998,38.33254499999998],[-105.88514199999986,38.32889000000006],[-105.88224699999995,38.327492000000234],[-105.88140399999992,38.324881000000005],[-105.880044,38.323222000000044],[-105.87929199999996,38.320628],[-105.87671799999998,38.31880200000029],[-105.87637599999994,38.3159510000001],[-105.87390999999991,38.31514200000015],[-105.87194399999987,38.31130200000001],[-105.87011199999989,38.309392000000116],[-105.86861199999987,38.30872500000015],[-105.86847399999999,38.30779600000005],[-105.86681699999986,38.3057],[-105.86498099999989,38.30465300000003],[-105.86162299999995,38.303422000000126],[-105.86073799999991,38.302498999999955],[-105.858315,38.29835900000012],[-105.85323299999988,38.29641199999992],[-105.85017599999998,38.29445600000008],[-105.8494189999999,38.29238700000013],[-105.84616900000003,38.29020400000019],[-105.84547900000001,38.29115400000012],[-105.84278,38.29322600000029],[-105.84082999999987,38.29245999999995],[-105.83720199999988,38.288410999999996],[-105.83282500000001,38.28674100000018],[-105.83076899999998,38.28506199999998],[-105.82824399999987,38.281130000000076],[-105.82712300000003,38.27856800000001],[-105.82147399999991,38.27611700000011],[-105.81897299999991,38.27359700000011],[-105.81786799999998,38.27391600000004],[-105.81460399999992,38.27266800000018],[-105.81087400000001,38.27249999999998],[-105.80843500000003,38.271837000000005],[-105.8066419999999,38.270917000000054],[-105.80401499999988,38.27089100000023],[-105.80001300000004,38.26992200000018],[-105.79767099999987,38.26675699999993],[-105.79689699999977,38.26504700000004],[-105.79566499999999,38.26013900000021],[-105.79274399999997,38.256294000000025],[-105.79218000000003,38.254815000000235],[-105.78914299999991,38.251824],[-105.78921099999997,38.24627799999996],[-105.789761,38.244360000000086],[-105.78788700000001,38.24205600000005],[-105.783928,38.23963300000031],[-105.780933,38.23914000000019],[-105.7754539999998,38.23584900000003],[-105.77106000000003,38.23483600000009],[-105.76823200000001,38.233763000000124],[-105.76720699999993,38.231815999999924],[-105.76560499999988,38.23080300000015],[-105.75949000000003,38.2284130000001],[-105.75653899999992,38.22496000000007],[-105.75588599999998,38.222030000000075],[-105.756482,38.218975000000114],[-105.75602600000002,38.218127000000266],[-105.75362899999993,38.21667500000001],[-105.750563,38.21421300000026],[-105.74850699999996,38.2141400000001],[-105.74480599999998,38.21460300000001],[-105.74442499999992,38.213701000000015],[-105.74069800000001,38.210283000000004],[-105.73631799999998,38.20923100000016],[-105.73580999999996,38.20720700000004],[-105.7342099999999,38.205057000000295],[-105.73333000000002,38.20174300000019],[-105.7325909999999,38.20067999999998],[-105.73107299999998,38.200911000000076],[-105.72675300000003,38.19947200000007],[-105.72204899999991,38.19344899999999],[-105.7212659999999,38.19155500000011],[-105.71883300000002,38.18983500000007],[-105.71733799999987,38.1881370000001],[-105.716001,38.185210999999924],[-105.71292399999987,38.183235000000025],[-105.70830999999993,38.18186000000014],[-105.70808,38.180352000000255],[-105.70684799999992,38.178152000000296],[-105.705374,38.176438000000076],[-105.70219499999996,38.174203000000034],[-105.70111699999995,38.17197900000025],[-105.70041399999997,38.1691800000001],[-105.69798000000003,38.16720200000003],[-105.69321500000001,38.16573800000026],[-105.69133199999999,38.166494000000284],[-105.68683799999997,38.166845000000194],[-105.6861599999998,38.165306000000214],[-105.68078199999991,38.16050200000012],[-105.67744700000003,38.15952599999997],[-105.67304199999995,38.15912600000013],[-105.67284099999995,38.156777999999974],[-105.67134299999998,38.15385200000003],[-105.67016899999993,38.15309700000006],[-105.67105900000001,38.15107700000027],[-105.6726129999999,38.14936200000011],[-105.67306599999989,38.14797299999992],[-105.67626999999987,38.14617300000003],[-105.6743219999999,38.14133800000002],[-105.67254699999995,38.140220000000056],[-105.67339500000003,38.137396000000194],[-105.67205200000001,38.13406700000007],[-105.66968099999997,38.13074100000023],[-105.67021499999998,38.12855400000012],[-105.66644100000002,38.12585200000018],[-105.66501399999999,38.12428300000022],[-105.6632899999999,38.12154499999997],[-105.66265399999986,38.1191580000002],[-105.660145,38.11806800000005],[-105.6576629999999,38.11745400000024],[-105.65849400000002,38.11518500000011],[-105.6599129999999,38.11324900000011],[-105.65983599999993,38.11242800000008],[-105.66133999999994,38.11149800000021],[-105.66122899999999,38.109781999999996],[-105.66224599999993,38.106845000000135],[-105.66148499999986,38.10638300000011],[-105.66222099999999,38.10434800000013],[-105.66146599999996,38.10313600000006],[-105.66147999999998,38.10091200000005],[-105.66030199999994,38.09886899999992],[-105.65882599999992,38.097276000000306],[-105.65557899999988,38.095490000000154],[-105.65509399999996,38.09358700000007],[-105.65585599999997,38.09229099999993],[-105.65529599999991,38.09134300000005],[-105.65364899999997,38.090880000000084],[-105.65196499999996,38.08939700000013],[-105.650105,38.08695300000005],[-105.64644900000002,38.08503700000017],[-105.64528399999989,38.08240400000011],[-105.64124099999987,38.078883000000076],[-105.63936899999999,38.07848100000007],[-105.636484,38.07558400000016],[-105.63727599999999,38.07279699999992],[-105.63415900000001,38.069439999999986],[-105.63139699999982,38.06811499999998],[-105.627206,38.067309999999964],[-105.62692799999991,38.064592000000175],[-105.62626299999994,38.06353300000029],[-105.6259619999999,38.06116100000014],[-105.62674900000002,38.059149000000104],[-105.62820599999998,38.05742099999998],[-105.62556299999994,38.05569299999996],[-105.61780699999991,38.05326000000014],[-105.61756399999996,38.04977300000007],[-105.617996,38.04726100000005],[-105.61720600000001,38.04592900000017],[-105.614957,38.04366199999998],[-105.61400300000003,38.03953700000011],[-105.613246,38.03808500000008],[-105.60885399999978,38.03495300000026],[-105.6085609999999,38.033708000000274],[-105.60937999999999,38.032338000000095],[-105.6065129999999,38.03160400000007],[-105.60352699999993,38.03126299999997],[-105.60311399999995,38.02714400000002],[-105.60080499999987,38.023746000000244],[-105.59869799999996,38.02264900000006],[-105.59795199999996,38.02001900000005],[-105.59832499999993,38.01633800000013],[-105.59711199999992,38.013638000000014],[-105.59727299999992,38.012664000000086],[-105.59887599999985,38.01124700000008],[-105.60435499999994,38.008745000000204],[-105.60486799999995,38.00738799999999],[-105.60463600000003,38.005014000000074],[-105.60390699999994,38.00373900000005],[-105.600596,38.002274],[-105.60024399999998,37.99999900000006],[-105.598413,37.99702100000002],[-105.59589199999988,37.996370000000184],[-105.59573899999992,37.995069],[-105.59366299999982,37.99277500000028],[-105.59207099999992,37.99214800000027],[-105.58989899999989,37.98743100000024],[-105.58815599999991,37.986689000000126],[-105.58715999999993,37.98450900000023],[-105.58590999999996,37.97968700000024],[-105.58319099999989,37.97997800000013],[-105.58185500000002,37.97797200000008],[-105.5815219999999,37.975224999999966],[-105.58396399999998,37.973008999999934],[-105.58505400000001,37.970451000000025],[-105.58415799999995,37.96908700000017],[-105.58446199999997,37.96785500000004],[-105.58261599999992,37.96593400000023],[-105.58128599999992,37.96618000000001],[-105.577676,37.965535999999986],[-105.57561999999979,37.96398399999998],[-105.573982,37.96175800000003],[-105.5732809999999,37.96158200000002],[-105.57069200000001,37.958265999999924],[-105.56719199999986,37.95714500000008],[-105.56235099999998,37.953744000000256],[-105.56031100000001,37.95188600000017],[-105.55635699999999,37.950980000000015],[-105.551826,37.950584000000276],[-105.55010099999998,37.95387100000028],[-105.54619099999985,37.95604800000012],[-105.54273699999999,37.95650999999998],[-105.54194899999987,37.95604300000002],[-105.53770299999985,37.95503600000001],[-105.53331499999996,37.95518200000021],[-105.53037799999987,37.95336299999997],[-105.52714699999996,37.94953099999998],[-105.525534,37.94837000000007],[-105.523572,37.948037],[-105.51572899999991,37.942015000000026],[-105.514591,37.939930000000174],[-105.51108599999998,37.939131000000145],[-105.50836399999997,37.936094000000026],[-105.50704200000001,37.93101999999999],[-105.50568699999991,37.927891000000045],[-105.50489299999992,37.92721300000028],[-105.50311599999998,37.92372500000016],[-105.50227299999978,37.922950000000014],[-105.49719699999991,37.920339000000126],[-105.49410899999992,37.917100000000005],[-105.4911449999999,37.915225000000305],[-105.48814500000003,37.90984500000013],[-105.48260399999987,37.90500399999996],[-105.47835099999998,37.90334899999999],[-105.47899999999993,37.902629000000104],[-105.47887499999996,37.90050500000024],[-105.47708299999994,37.899298000000044],[-105.47605599999997,37.89938900000027],[-105.47330699999998,37.896649000000025],[-105.4732039999999,37.89596999999998],[-105.47365899999988,37.894516000000124],[-105.475435,37.89237300000008],[-105.47705500000001,37.887961000000075],[-105.47621399999997,37.8832950000002],[-105.47758099999993,37.88044200000007],[-105.47191799999996,37.879687000000104],[-105.46915200000001,37.87875500000007],[-105.46530899999999,37.87953000000027],[-105.46454599999993,37.878974000000085],[-105.460106,37.88035500000012],[-105.45658799999995,37.87989399999998],[-105.4540849999999,37.87850100000003],[-105.45053699999988,37.87811199999999],[-105.44848200000001,37.876378999999986],[-105.44330899999994,37.870721999999944],[-105.440742,37.86961200000013],[-105.44059499999992,37.868353999999954],[-105.43924099999987,37.86620200000027],[-105.43946899999992,37.86324800000011],[-105.43894899999998,37.86127500000009],[-105.43721399999993,37.860231000000226],[-105.43677399999996,37.858202000000006],[-105.4334619999999,37.85751599999992],[-105.43188899999979,37.85482400000012],[-105.42915499999998,37.85236000000003],[-105.42889299999996,37.851733000000024],[-105.4305579999999,37.848653999999954],[-105.43091700000002,37.84736000000021],[-105.42864699999996,37.84477899999996],[-105.42867399999989,37.84366600000027],[-105.43096500000001,37.838753],[-105.43073099999998,37.837826000000234],[-105.43262300000004,37.8350690000002],[-105.43240700000001,37.83316700000006],[-105.4355829999999,37.82850500000012],[-105.43509299999994,37.82648300000017],[-105.43409699999995,37.824505999999985],[-105.434573,37.821929000000296],[-105.43673899999999,37.81921799999998],[-105.43552999999997,37.814637000000005],[-105.43466799999987,37.81373300000001],[-105.43591100000003,37.81179599999996],[-105.43552499999998,37.81082300000014],[-105.43582299999986,37.80782000000016],[-105.43505599999992,37.806301000000246],[-105.43517899999995,37.80476699999997],[-105.43689699999999,37.803641000000084],[-105.44074,37.80250700000022],[-105.44223699999998,37.802367000000174],[-105.44739199999998,37.80006100000014],[-105.450695,37.7976560000003],[-105.45368199999996,37.7968590000001],[-105.45491199999975,37.795696000000305],[-105.45480199999997,37.79328499999997],[-105.45518699999997,37.790814000000125],[-105.45245299999988,37.78829300000007],[-105.45166599999976,37.78689500000007],[-105.45142799999996,37.784759000000065],[-105.45170499999989,37.78263500000003],[-105.44976899999995,37.780754],[-105.44945399999989,37.77812700000004],[-105.45091599999989,37.77519500000028],[-105.45097099999981,37.77303300000028],[-105.45023299999986,37.77060999999998],[-105.45241599999991,37.76789900000006],[-105.45370300000002,37.76728400000013],[-105.45591100000001,37.76242700000006],[-105.45770400000004,37.76013000000012],[-105.4578689999999,37.75867699999998],[-105.45627300000001,37.75806800000004],[-105.455827,37.756498000000136],[-105.45957699999991,37.7536290000001],[-105.45978899999989,37.75276500000018],[-105.45725499999992,37.751464],[-105.49797899999987,37.751003000000026],[-105.50056599999994,37.75117799999998],[-105.51404600000001,37.7512730000002],[-105.596295,37.75209500000017],[-105.61726899999991,37.75230600000003],[-105.62556899999998,37.751888000000065],[-105.64356099999998,37.75174800000008],[-105.70149499999997,37.751458000000014],[-105.73682500000001,37.75128500000011],[-105.75057300000003,37.75092200000029],[-105.79026799999991,37.75063500000016],[-105.809327,37.750451000000055],[-105.81944499999997,37.7502970000001],[-105.872481,37.74959799999999],[-105.91008099999999,37.749135000000024],[-105.92791599999987,37.74884900000018],[-105.96253899999994,37.74878600000011],[-105.99258299999997,37.74868600000019],[-106.01959099999988,37.74841700000013],[-106.03841299999988,37.74835300000018],[-106.10240299999998,37.748212000000024],[-106.136685,37.74815600000022],[-106.18779899999987,37.74815500000011],[-106.20480999999995,37.74807700000025],[-106.23175399999991,37.74813600000016],[-106.23555299999981,37.74808899999999],[-106.25649299999992,37.74815400000023],[-106.287665,37.748303000000305],[-106.29134399999987,37.74817800000005],[-106.33255199999996,37.74823200000009],[-106.35085300000003,37.74840700000004],[-106.40503999999999,37.74836000000022],[-106.42315999999988,37.74828600000012],[-106.47784300000001,37.74831500000022],[-106.486674,37.748071000000095],[-106.52721099999991,37.74830800000018],[-106.53228699999988,37.74839900000006],[-106.546426,37.748289],[-106.58637399999998,37.747809000000075],[-106.58656799999994,37.75000100000028],[-106.58645199999984,37.77623400000027],[-106.58703199999997,37.78610100000026],[-106.58717799999988,37.79225800000023],[-106.58713899999992,37.83442600000029],[-106.60447599999992,37.83466499999997],[-106.61911099999998,37.83476600000017],[-106.62559599999997,37.834494000000234],[-106.6938449999999,37.8352440000001],[-106.69380599999994,37.840430000000026],[-106.69379700000002,37.84172700000005],[-106.69374099999999,37.84973599999995],[-106.693712,37.85396400000002],[-106.69369999999998,37.85590600000012],[-106.69369199999994,37.864279000000124],[-106.69368699999995,37.871679000000086],[-106.69352299999997,37.8735190000001],[-106.69454400000001,37.87436300000013],[-106.697161,37.87611099999998],[-106.70154000000002,37.87635400000005],[-106.70448399999998,37.875844000000086],[-106.706615,37.8747810000001],[-106.70797599999997,37.87346999999994],[-106.710286,37.872568000000115],[-106.71244099999984,37.87199400000003],[-106.71467799999994,37.87190099999998],[-106.71673099999998,37.87228600000003],[-106.71986300000003,37.873385999999925],[-106.72109499999993,37.87402800000012],[-106.72290299999997,37.87499900000017],[-106.7366659999999,37.88788100000022],[-106.75059699999997,37.88149100000015],[-106.75241699999998,37.88118800000012],[-106.7558229999999,37.88062400000018],[-106.77306599999991,37.88557500000002],[-106.77853699999997,37.89486200000016],[-106.78343699999994,37.89953200000002],[-106.79542500000002,37.90962100000007],[-106.80102599999998,37.91369900000018],[-106.80684999999994,37.914822000000186],[-106.80793699999998,37.91538100000025],[-106.81842699999993,37.91403900000023],[-106.82452,37.91873000000004],[-106.82993999999997,37.91789400000016],[-106.83398399999999,37.91891400000003],[-106.84150299999993,37.91889100000003],[-106.84726999999998,37.919170000000236],[-106.85289699999987,37.92388399999999],[-106.85924799999998,37.92857800000024],[-106.86419799999993,37.933245000000056],[-106.86625299999997,37.93409100000002],[-106.86868099999987,37.93793699999992],[-106.86975999999987,37.941454000000135],[-106.87140299999993,37.94189399999999],[-106.87199199999998,37.94205200000005],[-106.87435199999993,37.94268500000004],[-106.87460099999993,37.94331200000005],[-106.87559899999991,37.94582299999996],[-106.8771359999999,37.94858199999999],[-106.87868000000003,37.94915900000012],[-106.87988000000001,37.94986100000011],[-106.87914699999993,37.95640099999997],[-106.87834099999986,37.9585000000003],[-106.88136699999995,37.96224000000012],[-106.88275099999998,37.96153400000014],[-106.8896029999998,37.961871000000144],[-106.89293699999996,37.96534899999995],[-106.89455999999996,37.967104000000234],[-106.89783399999999,37.96789099999995],[-106.89948900000002,37.96678800000001],[-106.90224699999999,37.96564500000011],[-106.90519899999987,37.96390100000025],[-106.90866800000003,37.964053000000206],[-106.91018699999995,37.9652670000001],[-106.91157900000002,37.96647800000011],[-106.91674999999992,37.966436000000044],[-106.92031199999997,37.96743100000009],[-106.92073399999987,37.96643299999994],[-106.92237799999992,37.96559799999994],[-106.92418700000002,37.96486900000002],[-106.93014099999999,37.963298000000066],[-106.93379799999997,37.960866000000124],[-106.93525799999998,37.96036300000026],[-106.93540599999989,37.958785999999975],[-106.93735600000002,37.95664700000009],[-106.94151899999997,37.954259000000036],[-106.94446399999998,37.95372500000008],[-106.94631700000002,37.954005000000166],[-106.94816099999991,37.9555630000001],[-106.94909100000001,37.95669499999997],[-106.95236499999993,37.95744700000006],[-106.9524439999999,37.961853000000076],[-106.95304199999998,37.962808999999936],[-106.95536899999996,37.965017000000216],[-106.96135500000003,37.967952000000025],[-106.96190899999993,37.96894099999997],[-106.96245299999998,37.97231700000026],[-106.96401300000002,37.97252199999997],[-106.9679809999999,37.97503900000004],[-106.969921,37.97313500000007],[-106.9725949999999,37.97198800000001],[-106.97792200000004,37.969088000000056],[-106.98584800000003,37.96904200000017],[-106.987368,37.96917899999994],[-106.98748399999994,37.96795000000009],[-106.98762499999998,37.96373699999998],[-106.98832599999997,37.962072999999975],[-106.98980699999993,37.96103199999999],[-106.99443599999995,37.96077199999996],[-106.99686600000001,37.96039200000001],[-106.997749,37.9573190000001],[-106.99932799999999,37.95702099999994],[-107.00060199999996,37.95604100000003],[-107.00079599999992,37.96618199999995],[-107.00101799999993,37.97662400000013],[-107.00151399999993,37.99999600000001],[-107.00154099999986,38.00654800000012],[-107.00157100000001,38.01410600000003],[-107.00160899999997,38.02341700000005],[-107.00168799999994,38.042902000000026],[-107.00172499999996,38.05201200000005],[-107.00173699999999,38.05504500000001],[-107.001822,38.075776000000076],[-107.00184499999995,38.08153400000003],[-107.00188000000003,38.09006299999999],[-107.00188899999995,38.09290600000003],[-107.00177399999995,38.09932300000003],[-107.00131399999998,38.124994999999956],[-107.00120300000003,38.137512000000015],[-107.001192,38.138644000000056],[-107.00115299999999,38.14317600000004],[-107.00114599999995,38.143955000000005],[-107.00111900000002,38.14707199999998],[-107.001149,38.15394500000025],[-107.00112899999988,38.20577700000018],[-107.00096999999994,38.23395900000014],[-107.00078999999988,38.24999400000013],[-107.00069400000001,38.29452700000013],[-107.00060199999996,38.33201400000013],[-107.00060199999996,38.384372999999925],[-107.00060099999996,38.4256190000001],[-106.98254999999989,38.425357000000076],[-106.94782199999992,38.42529500000029],[-106.8756019999999,38.425166000000274],[-106.8715259999999,38.42526299999997],[-106.85174299999994,38.425297000000285],[-106.83081500000003,38.425144000000046],[-106.80380699999995,38.42513500000001],[-106.76247999999987,38.42507199999994],[-106.72389599999997,38.42487299999999],[-106.65282000000002,38.42446100000001],[-106.62557700000002,38.42435499999999],[-106.61936300000002,38.42447200000004],[-106.56912999999997,38.42444800000021],[-106.500585,38.42440199999993],[-106.48190599999981,38.42421500000029],[-106.43907200000001,38.4240880000001],[-106.39327499999996,38.42370699999998],[-106.38664599999993,38.42367800000011],[-106.37561699999992,38.423877000000005],[-106.319279,38.4233430000001],[-106.25059499999998,38.42291200000011],[-106.24694,38.42276800000013],[-106.22461599999997,38.422692999999924],[-106.17619699999995,38.4223860000003],[-106.17297500000001,38.422394000000054],[-106.12557099999998,38.422092000000305],[-106.08812999999998,38.42228000000023],[-106.08606799999978,38.424113000000034],[-106.08407799999998,38.422875999999974],[-106.08013699999987,38.423538000000235],[-106.07653499999992,38.42365200000006],[-106.07455500000003,38.424979000000064],[-106.07428199999993,38.42650300000008],[-106.07212999999996,38.42738700000001],[-106.07121199999995,38.429159000000254],[-106.06935399999986,38.43018000000018],[-106.07022599999988,38.432005000000004],[-106.06883199999993,38.43293700000021],[-106.06640199999975,38.43379600000003],[-106.06514400000003,38.43587400000001],[-106.06404599999996,38.43665400000009],[-106.06260199999997,38.439200000000255],[-106.06052599999987,38.441075000000126],[-106.05798199999987,38.444691000000034],[-106.05911100000003,38.44836399999997],[-106.06224399999991,38.45033000000029],[-106.06004299999995,38.45273700000013],[-106.06031000000002,38.45496400000019],[-106.05857100000003,38.45694500000013],[-106.05568399999999,38.45489000000009],[-106.05207399999989,38.45422100000019],[-106.04323399999998,38.457671000000175],[-106.04010499999998,38.45696300000003],[-106.03600999999992,38.45511500000009],[-106.030665,38.45353800000015],[-106.02354000000003,38.45391700000005],[-106.02148899999997,38.454469000000074],[-106.01904300000001,38.45442300000019],[-106.01691700000003,38.45179600000006],[-106.01767999999993,38.44869300000005],[-106.01597699999996,38.44762400000002],[-106.01075100000003,38.446566000000246]]]},"properties":{"OBJECTID":56,"COUNTYFP":"109","Shape_Leng":4.39184276373,"Shape_Area":0.843148723936,"low":2345,"lowmod":3750,"LMMI":4915,"lowmoduniv":6215,"lowmod_pct":0.603378921963,"NAMELSAD10":"Saguache County"}},
{"type":"Feature","geometry":{"type":"Polygon","coordinates":[[[-107.97505999999993,37.68364800000006],[-107.97384599999992,37.68477400000012],[-107.97176399999995,37.68694000000011],[-107.97030799999993,37.68767000000008],[-107.97042199999987,37.689162000000124],[-107.97031299999998,37.69120399999997],[-107.96903299999997,37.69349899999992],[-107.968955,37.695516000000055],[-107.96770400000003,37.697125000000085],[-107.96435499999995,37.69926399999997],[-107.96364399999993,37.700540000000046],[-107.96005999999994,37.70217100000002],[-107.95810499999993,37.703184000000135],[-107.95758699999999,37.704678000000115],[-107.95430899999991,37.70647700000012],[-107.952628,37.706788000000074],[-107.95061999999996,37.706389000000115],[-107.94963200000001,37.706414999999936],[-107.949184,37.70682000000005],[-107.94638099999997,37.706277000000114],[-107.94386299999996,37.706041000000084],[-107.94334200000003,37.70465400000012],[-107.94243499999988,37.704139999999995],[-107.94105400000001,37.70276999999999],[-107.939886,37.70289500000001],[-107.93720599999995,37.702562],[-107.93524199999996,37.70436600000005],[-107.93474299999997,37.705246000000045],[-107.93305299999997,37.70591200000007],[-107.93128199999995,37.70803799999993],[-107.92985599999992,37.709154000000126],[-107.92647399999998,37.71036400000003],[-107.92395799999997,37.71040100000005],[-107.921805,37.71015600000004],[-107.92051099999998,37.71037599999994],[-107.916967,37.71103699999992],[-107.91616199999993,37.711700000000064],[-107.91349200000002,37.71242500000005],[-107.91097500000001,37.71255900000011],[-107.91025299999995,37.71448099999998],[-107.91061000000002,37.715182000000084],[-107.91033599999992,37.71642500000007],[-107.90891899999991,37.71620200000007],[-107.907015,37.71766699999995],[-107.90341799999993,37.71839299999999],[-107.90134899999998,37.7190020000001],[-107.899474,37.718771000000004],[-107.897895,37.719186000000036],[-107.89673299999998,37.71917200000013],[-107.89498900000001,37.717875000000106],[-107.89271999999994,37.717808000000105],[-107.88939599999998,37.71844100000004],[-107.88551299999995,37.71785100000011],[-107.88158799999991,37.71841399999994],[-107.88039599999996,37.7175610000001],[-107.87719499999997,37.716309000000024],[-107.87562200000002,37.718142],[-107.87371000000002,37.720083000000045],[-107.87296600000002,37.72241599999995],[-107.87562200000002,37.725787000000025],[-107.87562200000002,37.726861999999926],[-107.87454400000001,37.72887900000006],[-107.87514499999997,37.732173999999986],[-107.87494900000002,37.73430400000001],[-107.87560199999996,37.73755700000004],[-107.87562200000002,37.73783600000007],[-107.87562200000002,37.73885100000001],[-107.872951,37.74164100000013],[-107.87168799999995,37.74354900000003],[-107.86992299999991,37.74526400000002],[-107.86736399999995,37.74654600000002],[-107.86590199999995,37.747268000000076],[-107.86603600000001,37.74801700000006],[-107.86456099999992,37.75055200000003],[-107.86461499999996,37.75261200000011],[-107.86202399999996,37.756169000000114],[-107.86309900000003,37.75682699999999],[-107.86307799999997,37.760016000000064],[-107.86570699999999,37.76171000000011],[-107.86709799999994,37.761890999999935],[-107.87048099999993,37.762762000000066],[-107.87043699999992,37.76347299999992],[-107.86863900000003,37.7669360000001],[-107.86819199999996,37.767322999999976],[-107.86739699999993,37.76767699999999],[-107.86660199999994,37.76794500000011],[-107.86489199999994,37.769093000000055],[-107.86540299999996,37.771058000000096],[-107.86521800000003,37.77170400000011],[-107.86370199999999,37.77241200000003],[-107.86315199999996,37.77266100000003],[-107.86315099999996,37.77296800000005],[-107.86193799999995,37.7741620000001],[-107.86084499999987,37.77551700000009],[-107.85710499999993,37.775776000000064],[-107.8511039999999,37.77625599999999],[-107.84975199999997,37.77588899999995],[-107.84587199999999,37.77466900000013],[-107.84455400000002,37.77364799999998],[-107.84277799999995,37.77363600000007],[-107.84181799999993,37.773915000000045],[-107.84085199999987,37.774138999999934],[-107.84000199999997,37.77599500000002],[-107.83948299999992,37.777251999999976],[-107.83905900000002,37.77843299999995],[-107.83866899999998,37.779418000000135],[-107.83661799999993,37.781659000000104],[-107.83579399999996,37.78288500000008],[-107.83275399999997,37.78430900000012],[-107.83120199999996,37.786317],[-107.82932599999992,37.78764500000011],[-107.82795899999996,37.789096000000086],[-107.82705199999998,37.78974900000014],[-107.82465000000002,37.79022800000013],[-107.82361200000003,37.793271000000004],[-107.8221999999999,37.795748],[-107.82428199999993,37.79669800000005],[-107.82846099999995,37.799624999999935],[-107.82788699999998,37.801086000000055],[-107.82613199999997,37.80264900000003],[-107.8246999999999,37.804852000000096],[-107.82616299999995,37.80537199999992],[-107.82895899999994,37.808626999999944],[-107.82917600000002,37.81066400000009],[-107.82758299999995,37.81231600000001],[-107.82415999999995,37.81395600000002],[-107.82182799999993,37.81452400000012],[-107.82102299999985,37.81503900000007],[-107.81903199999988,37.817006000000106],[-107.81774499999995,37.81847299999998],[-107.81602099999998,37.81957599999993],[-107.813556,37.82059600000014],[-107.81123500000001,37.82101800000004],[-107.81020599999994,37.821995999999956],[-107.80976099999998,37.823382000000095],[-107.80869000000001,37.82410600000014],[-107.80442399999993,37.82512100000008],[-107.80386699999991,37.82444900000007],[-107.80133599999994,37.82375600000012],[-107.79746399999993,37.82271099999997],[-107.796447,37.822725000000105],[-107.79570699999988,37.82458099999997],[-107.79508599999991,37.82561700000008],[-107.79506900000001,37.82641400000011],[-107.79459499999996,37.82805300000001],[-107.79457300000001,37.828384000000085],[-107.79447899999997,37.829094],[-107.79272299999997,37.830608999999924],[-107.79042599999991,37.83516000000009],[-107.79054599999989,37.835701000000086],[-107.790234,37.83737300000007],[-107.78785899999997,37.838255000000004],[-107.78684799999996,37.839085000000125],[-107.78292299999993,37.84093999999993],[-107.78249700000003,37.843241999999975],[-107.78266100000002,37.843794],[-107.78320500000001,37.84410800000006],[-107.78423800000002,37.845525000000066],[-107.78448600000002,37.847142000000076],[-107.78368899999998,37.847947999999974],[-107.78310599999998,37.848487000000034],[-107.78157299999998,37.848979999999926],[-107.78076799999997,37.84881000000007],[-107.779653,37.848839],[-107.77931799999999,37.85074099999997],[-107.77913899999993,37.85266899999999],[-107.77932099999998,37.85288700000007],[-107.780034,37.85379700000004],[-107.77990699999998,37.85401200000007],[-107.77985399999994,37.85491000000013],[-107.77956599999999,37.85818900000004],[-107.78060899999997,37.85959000000014],[-107.78046899999993,37.86130300000008],[-107.78026999999997,37.862700000000075],[-107.78288199999992,37.86633500000005],[-107.78274499999998,37.867514000000085],[-107.78245900000002,37.867772000000116],[-107.78216399999985,37.86771399999998],[-107.78079899999989,37.86774300000002],[-107.77682900000002,37.86789400000009],[-107.774902,37.868566999999985],[-107.77289799999994,37.870686000000035],[-107.772448,37.87111299999998],[-107.77171299999986,37.87162000000001],[-107.77044699999999,37.87254600000006],[-107.76909999999992,37.873431000000096],[-107.76735299999996,37.875992999999994],[-107.76605799999993,37.877465000000086],[-107.76249199999995,37.87724300000002],[-107.760535,37.87810999999999],[-107.75616400000001,37.8829070000001],[-107.75615099999993,37.88369],[-107.756505,37.885687000000075],[-107.75610999999992,37.88806499999998],[-107.75623499999995,37.889760999999964],[-107.75618999999995,37.890035000000125],[-107.75389799999994,37.89084200000008],[-107.7526499999999,37.891225000000134],[-107.75061699999998,37.89241300000003],[-107.749371,37.893527000000006],[-107.74821400000002,37.894861000000105],[-107.74821600000001,37.89545100000004],[-107.749011,37.89795099999998],[-107.74860399999989,37.897968000000105],[-107.74563799999999,37.898289000000034],[-107.74378399999995,37.89887399999992],[-107.74129499999998,37.90099500000008],[-107.74082199999992,37.90194800000006],[-107.74031400000001,37.903071000000125],[-107.73900299999997,37.90430900000001],[-107.73880099999997,37.90485000000001],[-107.73828299999991,37.905432000000076],[-107.73727099999991,37.904764],[-107.7342369999999,37.90436799999992],[-107.73162599999995,37.90370200000007],[-107.72956199999999,37.90267300000005],[-107.72741399999995,37.90111200000001],[-107.72642199999996,37.900347000000124],[-107.72590000000002,37.89982299999997],[-107.724674,37.89909100000011],[-107.72368399999999,37.898762999999974],[-107.72325599999999,37.89882800000004],[-107.720864,37.89742000000007],[-107.71984599999996,37.897898],[-107.71924599999994,37.897721000000104],[-107.71879399999995,37.89719700000012],[-107.71816299999995,37.897360000000106],[-107.71663299999994,37.89708799999994],[-107.71576800000003,37.89774299999999],[-107.714137,37.89808100000005],[-107.71342599999997,37.8983990000001],[-107.71221499999996,37.89888600000006],[-107.71193099999999,37.89886799999999],[-107.70957499999992,37.89940100000001],[-107.70910500000002,37.899249000000054],[-107.70777900000002,37.89885600000002],[-107.70761500000003,37.89861400000012],[-107.70815900000002,37.896918000000085],[-107.70761500000003,37.895514000000105],[-107.70715999999987,37.89363399999996],[-107.70509099999992,37.89396899999997],[-107.70169299999998,37.89294100000001],[-107.70134399999995,37.892223000000115],[-107.70063899999997,37.892007000000035],[-107.69734599999992,37.892776000000026],[-107.69671599999992,37.89274000000012],[-107.69566500000002,37.892409999999984],[-107.69396699999993,37.893712999999934],[-107.69338799999997,37.89453700000007],[-107.69315399999999,37.894803999999965],[-107.69252699999998,37.89654900000011],[-107.69225599999993,37.89772800000014],[-107.69085999999999,37.89804500000008],[-107.69051399999995,37.89824699999991],[-107.68959199999995,37.89979800000003],[-107.689484,37.901203000000066],[-107.688606,37.901769],[-107.684823,37.90085500000009],[-107.68330399999991,37.9009200000001],[-107.68035799999996,37.904124000000024],[-107.67902199999997,37.90426300000013],[-107.67606299999994,37.903638000000115],[-107.67373799999996,37.903543999999954],[-107.672911,37.903279000000055],[-107.67160499999994,37.90288600000008],[-107.66794399999998,37.903019000000086],[-107.66787999999985,37.90301599999998],[-107.66539699999993,37.90344200000004],[-107.66375600000003,37.90351700000008],[-107.66365399999995,37.90380800000014],[-107.66281300000003,37.905471999999975],[-107.66155400000002,37.90643],[-107.66170299999999,37.90692799999999],[-107.66192899999987,37.90700300000003],[-107.66227499999997,37.90719300000012],[-107.66222600000003,37.90773400000012],[-107.66159600000003,37.90855699999997],[-107.66111099999995,37.909503000000086],[-107.66080499999987,37.909398000000124],[-107.65776199999999,37.908071000000064],[-107.65669099999991,37.90799199999992],[-107.654091,37.908682],[-107.651881,37.909589000000096],[-107.65162599999996,37.90971800000011],[-107.64974199999995,37.91059200000012],[-107.64930000000004,37.9115020000001],[-107.649092,37.91212700000011],[-107.648799,37.91300700000011],[-107.64794199999994,37.91317700000013],[-107.64641799999998,37.91531900000007],[-107.64679899999999,37.91665000000006],[-107.64590399999992,37.917410000000075],[-107.64230600000002,37.91837499999997],[-107.64108299999998,37.918990000000065],[-107.64028899999994,37.91961300000008],[-107.63895499999995,37.920220000000086],[-107.637926,37.92085900000001],[-107.63746399999997,37.923564],[-107.63784299999998,37.924346000000014],[-107.63925499999999,37.92587000000003],[-107.64000900000002,37.9295340000001],[-107.64102100000002,37.93053400000008],[-107.64252799999991,37.93321200000014],[-107.64284199999997,37.93661100000003],[-107.64213000000001,37.937499000000116],[-107.641886,37.93796000000009],[-107.640513,37.93939800000004],[-107.64102899999995,37.941633000000024],[-107.64115200000003,37.94195600000012],[-107.641839,37.94332700000007],[-107.64167900000001,37.944667000000095],[-107.64103099999994,37.94650900000005],[-107.64099099999993,37.94667800000002],[-107.64091099999996,37.94733200000013],[-107.64273300000002,37.94974400000007],[-107.64288499999998,37.953134000000034],[-107.64211499999993,37.95520800000003],[-107.64147400000002,37.955839000000026],[-107.640875,37.95702600000004],[-107.64007999999995,37.957479000000035],[-107.639928,37.95785900000004],[-107.64089100000001,37.95927000000006],[-107.64041799999995,37.96201600000006],[-107.64126699999986,37.9631930000001],[-107.64168999999993,37.96489599999995],[-107.64158800000001,37.965009000000066],[-107.62540100000001,37.964981999999964],[-107.62407899999994,37.96497000000005],[-107.62228999999991,37.964970999999935],[-107.61296099999993,37.964979000000085],[-107.60562400000003,37.96498400000013],[-107.59996899999993,37.96498800000012],[-107.57187299999998,37.965012],[-107.57124199999998,37.965012],[-107.5706429999999,37.965013000000056],[-107.56931599999996,37.96501399999994],[-107.56887499999999,37.96501500000005],[-107.569005,37.963085999999976],[-107.56721900000002,37.9613270000001],[-107.56491399999999,37.96121599999998],[-107.56336899999997,37.95593699999995],[-107.559166,37.95369599999998],[-107.55656099999993,37.95291600000013],[-107.5528589999999,37.951076000000114],[-107.55104599999999,37.949686999999926],[-107.54423300000002,37.94871499999999],[-107.54006900000002,37.94788399999999],[-107.53734600000001,37.9476130000001],[-107.53427099999999,37.944931000000054],[-107.53259299999996,37.943905000000086],[-107.532644,37.94084700000002],[-107.53430900000001,37.938170000000014],[-107.53414900000001,37.93713800000012],[-107.53770099999991,37.93464399999999],[-107.53835100000003,37.929819000000066],[-107.53735799999987,37.926144000000136],[-107.53454599999998,37.9246710000001],[-107.536518,37.922412000000065],[-107.536652,37.92108800000011],[-107.53590199999991,37.920123000000046],[-107.53074699999996,37.91681900000003],[-107.52892399999996,37.91368700000004],[-107.52971599999995,37.911748000000046],[-107.52851499999997,37.90878399999991],[-107.52850499999994,37.907097000000135],[-107.527511,37.90619800000002],[-107.52480600000001,37.905345000000125],[-107.52463499999999,37.90405400000009],[-107.52016699999996,37.90131200000002],[-107.521638,37.900419000000056],[-107.52182099999999,37.89877200000001],[-107.52303599999993,37.89753300000001],[-107.52642900000001,37.896213000000046],[-107.52605599999998,37.89364899999998],[-107.52716199999992,37.89137699999992],[-107.529068,37.890022999999985],[-107.53128200000003,37.88915900000012],[-107.53310599999986,37.88936100000012],[-107.53344400000003,37.88807700000012],[-107.53248100000002,37.884047000000066],[-107.53450299999997,37.88178800000003],[-107.53646800000001,37.880110000000116],[-107.53624399999995,37.87859400000008],[-107.53532799999994,37.87711300000012],[-107.534967,37.874996000000124],[-107.53553899999997,37.874062000000094],[-107.53256599999986,37.87284800000003],[-107.52774099999993,37.870361000000116],[-107.52601399999998,37.86895200000009],[-107.52414199999998,37.86660500000005],[-107.52440200000001,37.86516300000011],[-107.52234499999997,37.863435000000095],[-107.52107999999993,37.86186100000009],[-107.52078699999993,37.85954800000002],[-107.518644,37.858086000000014],[-107.51785899999999,37.856978000000026],[-107.51817399999993,37.85594000000009],[-107.51859299999995,37.849185000000034],[-107.52106100000003,37.84735600000005],[-107.52096399999994,37.84489200000007],[-107.52178300000003,37.84463499999998],[-107.52177,37.8425840000001],[-107.51966699999991,37.84119400000003],[-107.52058599999992,37.83949300000012],[-107.51837399999994,37.83898199999993],[-107.51620400000002,37.834546999999986],[-107.51683499999996,37.83347200000003],[-107.51612999999992,37.83193799999998],[-107.51656700000001,37.829769999999996],[-107.51728700000001,37.829391000000044],[-107.51654699999995,37.82645100000002],[-107.51532299999991,37.82656600000007],[-107.51292199999995,37.82533400000011],[-107.50356099999999,37.82370900000012],[-107.49640799999997,37.8230860000001],[-107.49489299999993,37.824080000000095],[-107.49030999999997,37.82427500000006],[-107.48547699999995,37.82242100000013],[-107.48227699999995,37.81988500000011],[-107.4813069999999,37.818590000000086],[-107.47887600000001,37.81668500000001],[-107.47497199999992,37.815687000000025],[-107.472305,37.81233900000012],[-107.470955,37.81176199999999],[-107.46884299999999,37.809726000000126],[-107.46882899999997,37.80780400000009],[-107.46678499999996,37.80461400000013],[-107.46557299999995,37.80089100000009],[-107.46336499999995,37.7992660000001],[-107.46202899999997,37.798767],[-107.462173,37.79737200000005],[-107.46096899999998,37.796884000000034],[-107.46022099999999,37.79491200000007],[-107.459224,37.79408300000006],[-107.45962500000002,37.792039999999986],[-107.45937499999997,37.78972900000008],[-107.46071,37.78699899999998],[-107.45963699999999,37.78619800000013],[-107.45834200000002,37.78179899999992],[-107.46061099999991,37.7812770000001],[-107.46074199999998,37.77983800000004],[-107.46387499999997,37.77708100000001],[-107.46518200000003,37.77439499999997],[-107.46488399999998,37.773610000000076],[-107.46754599999997,37.76941399999998],[-107.46673499999997,37.767163000000096],[-107.46718499999992,37.76603699999998],[-107.47081599999996,37.76655199999999],[-107.47327899999999,37.767628],[-107.47476699999993,37.76767100000001],[-107.47842000000003,37.769267999999954],[-107.48089599999997,37.768080999999995],[-107.48224500000003,37.768353000000104],[-107.48206099999993,37.749998000000005],[-107.48220700000002,37.74388400000004],[-107.48218399999996,37.68837200000013],[-107.48217899999992,37.639500999999996],[-107.49909899999994,37.639478000000054],[-107.50061199999999,37.63962800000007],[-107.50320599999992,37.63962800000007],[-107.52941799999996,37.63962800000007],[-107.55132899999995,37.63963000000001],[-107.57629400000002,37.639632000000006],[-107.60139299999997,37.63962700000013],[-107.61692099999993,37.63962500000014],[-107.64196399999997,37.639610000000005],[-107.65369899999996,37.639602000000025],[-107.66195900000002,37.63959600000004],[-107.686914,37.63957999999997],[-107.69069999999999,37.63957799999997],[-107.691371,37.63957799999997],[-107.691528,37.63957700000009],[-107.69185199999998,37.63957700000009],[-107.74318700000003,37.63954700000011],[-107.74631899999991,37.63986999999997],[-107.75060999999994,37.640315000000044],[-107.75890399999997,37.64033000000006],[-107.780955,37.64024600000005],[-107.78959799999996,37.640212000000076],[-107.79584699999998,37.640184999999974],[-107.79655399999996,37.64018200000004],[-107.80142499999994,37.64016399999997],[-107.80343099999999,37.640154999999936],[-107.80568199999999,37.64014800000007],[-107.81023900000002,37.640131999999994],[-107.81081,37.64013],[-107.82874699999996,37.64006000000006],[-107.83216099999993,37.640045999999984],[-107.842265,37.64000800000002],[-107.84563400000002,37.63999600000011],[-107.86156199999994,37.64000699999997],[-107.87181699999996,37.63993599999998],[-107.88625699999994,37.639972000000114],[-107.90325199999995,37.63990300000006],[-107.92579699999999,37.63978599999996],[-107.926064,37.63978399999996],[-107.93324599999994,37.63974500000012],[-107.94573699999995,37.639677000000006],[-107.96025400000002,37.63963500000011],[-107.962083,37.639635999999996],[-107.96380499999992,37.63965900000011],[-107.96473199999997,37.63967200000013],[-107.96500400000002,37.6396610000001],[-107.97008599999998,37.639587000000006],[-107.96650899999992,37.641197000000034],[-107.96338899999989,37.64131400000008],[-107.96157499999998,37.64185299999991],[-107.96006099999994,37.642638000000034],[-107.95943199999994,37.64373599999993],[-107.95861399999995,37.647113000000104],[-107.95707799999997,37.64889400000004],[-107.95831099999992,37.65108200000009],[-107.96102699999994,37.657015],[-107.96167999999989,37.65809400000006],[-107.96132599999999,37.659628],[-107.96155299999992,37.661748999999986],[-107.96355199999988,37.66365600000006],[-107.96469399999995,37.66534000000013],[-107.96441199999998,37.66763300000014],[-107.96586099999996,37.66950900000006],[-107.96548599999994,37.67059900000004],[-107.96706899999998,37.673452],[-107.96823199999994,37.67495800000012],[-107.96941600000002,37.677134000000024],[-107.96934699999991,37.67866700000002],[-107.96942000000001,37.67923200000007],[-107.97197999999997,37.681856000000096],[-107.97505999999993,37.68364800000006]]]},"properties":{"OBJECTID":57,"COUNTYFP":"111","Shape_Leng":1.71664052921,"Shape_Area":0.102833681636,"low":185,"lowmod":345,"LMMI":465,"lowmoduniv":605,"lowmod_pct":0.570247933884,"NAMELSAD10":"San Juan County"}},
{"type":"Feature","geometry":{"type":"Polygon","coordinates":[[[-107.96578899999997,38.15232800000001],[-107.96545300000002,38.14490300000011],[-107.95171700000003,38.144982000000084],[-107.95153099999993,38.12499000000008],[-107.95157399999994,38.11612500000024],[-107.93113,38.115881000000115],[-107.89146899999992,38.11595700000004],[-107.89161999999988,38.10796000000005],[-107.89152899999999,38.10136399999999],[-107.89253899999989,38.101346999999976],[-107.89205199999992,38.087011000000246],[-107.89671399999992,38.08682900000014],[-107.89292599999993,38.07247000000007],[-107.893529,38.06070100000028],[-107.89377400000001,38.057742000000076],[-107.89199899999988,38.05783700000012],[-107.89198399999998,38.05135500000023],[-107.89135599999997,38.04870699999998],[-107.8882079999999,38.04581200000007],[-107.88845700000002,38.04482200000024],[-107.89182800000003,38.04356400000006],[-107.89333199999987,38.041600000000074],[-107.89315599999998,38.03920000000011],[-107.89554699999991,38.03638500000011],[-107.89656400000001,38.03475300000008],[-107.89669200000003,38.031598000000145],[-107.89875099999978,38.029424000000006],[-107.900193,38.026249000000234],[-107.89919700000002,38.02571799999998],[-107.89561300000003,38.02573800000022],[-107.89136100000002,38.02515200000005],[-107.8907989999999,38.02353700000003],[-107.89138399999996,38.02253700000023],[-107.8893349999999,38.01761499999992],[-107.89104399999991,38.01353100000006],[-107.88971599999991,38.011615000000006],[-107.88639399999983,38.009250000000236],[-107.8832999999999,38.007371000000205],[-107.87865199999999,38.00823200000002],[-107.8749709999999,38.00689700000004],[-107.87356799999998,38.00681900000018],[-107.86593999999997,38.00801200000012],[-107.86241899999987,38.00943000000001],[-107.86099300000001,38.00876300000027],[-107.857418,38.008357000000046],[-107.85578399999997,38.00872099999998],[-107.85180300000002,38.008238000000176],[-107.849131,38.008599000000004],[-107.84445999999997,38.00590000000017],[-107.84229399999998,38.00582300000019],[-107.84143099999989,38.00288599999999],[-107.84149799999983,38.00113000000022],[-107.842378,37.999993000000075],[-107.84482499999996,37.998498999999924],[-107.8476819999999,37.99751800000013],[-107.84936799999997,37.99442699999997],[-107.84907999999996,37.9926080000003],[-107.84310499999992,37.991653000000156],[-107.83817399999998,37.992363000000296],[-107.83604700000001,37.992212999999936],[-107.83267799999999,37.99072700000028],[-107.82978099999991,37.98896300000001],[-107.82600599999984,37.988674999999944],[-107.82483299999996,37.98816300000004],[-107.82194300000003,37.98860200000013],[-107.819481,37.98972400000025],[-107.81690200000003,37.98919700000016],[-107.81560400000001,37.98939400000012],[-107.8143209999999,37.99067300000007],[-107.81114300000002,37.99001100000004],[-107.80979699999995,37.99014300000027],[-107.80806799999988,37.9892780000003],[-107.80446199999994,37.988652],[-107.80162599999994,37.98694400000011],[-107.79963999999995,37.98826400000024],[-107.79546899999997,37.98852000000028],[-107.79398099999997,37.987278],[-107.79330899999997,37.985333000000196],[-107.79172699999998,37.982354999999984],[-107.788862,37.980614],[-107.78684799999996,37.97851000000003],[-107.78708199999988,37.977205000000026],[-107.78917999999993,37.97527300000024],[-107.78685299999995,37.97303700000015],[-107.785368,37.97101200000009],[-107.78460199999995,37.96861400000029],[-107.78244799999999,37.965734000000054],[-107.78157199999998,37.96321400000005],[-107.78041200000001,37.961879000000124],[-107.77867500000002,37.96094799999997],[-107.77606199999997,37.96201100000019],[-107.77134199999989,37.96167100000014],[-107.76930099999976,37.96198100000004],[-107.765783,37.96095100000008],[-107.76360099999994,37.95920900000016],[-107.76085499999994,37.95963200000017],[-107.75889099999989,37.958360000000255],[-107.75575400000002,37.95464600000008],[-107.74889699999977,37.953116000000136],[-107.747344,37.95023000000009],[-107.74585899999994,37.948780000000056],[-107.74431600000003,37.94579700000014],[-107.7438929999999,37.942044000000294],[-107.74240699999996,37.94036799999998],[-107.73914799999989,37.939261000000045],[-107.73857299999992,37.93821300000002],[-107.73832699999997,37.93529100000006],[-107.73605900000001,37.932279999999935],[-107.73278799999997,37.92997500000001],[-107.73330699999985,37.92694699999993],[-107.73513300000002,37.92444900000021],[-107.73331799999988,37.92206299999998],[-107.73315899999994,37.920320000000174],[-107.73582399999992,37.91846600000008],[-107.73707299999995,37.91707500000007],[-107.73695099999992,37.91456500000004],[-107.73512299999993,37.91129100000023],[-107.73637099999996,37.90950500000025],[-107.73667999999998,37.907728000000304],[-107.73828299999991,37.905432000000076],[-107.73880099999997,37.90485000000001],[-107.73900299999997,37.90430900000001],[-107.74031400000001,37.903071000000125],[-107.74082199999992,37.90194800000006],[-107.74129499999998,37.90099500000008],[-107.74378399999995,37.89887399999992],[-107.74563799999999,37.898289000000204],[-107.74860399999989,37.897968000000105],[-107.749011,37.89795099999998],[-107.74821600000001,37.89545100000004],[-107.74821400000002,37.894861000000105],[-107.749371,37.893527000000006],[-107.75061699999998,37.89241300000003],[-107.7526499999999,37.891225000000134],[-107.75389799999994,37.89084200000008],[-107.75618999999995,37.890035000000125],[-107.75623499999995,37.889760999999964],[-107.75610999999992,37.88806499999998],[-107.756505,37.885687000000075],[-107.75615099999993,37.88369],[-107.75616400000001,37.8829070000001],[-107.760535,37.87810999999999],[-107.76249199999995,37.87724300000002],[-107.76605799999993,37.877465000000086],[-107.76735299999996,37.875992999999994],[-107.76909999999992,37.873431000000096],[-107.77044699999999,37.87254600000006],[-107.77171299999986,37.87162000000001],[-107.772448,37.87111299999998],[-107.77289799999994,37.870686000000035],[-107.774902,37.868566999999985],[-107.77682900000002,37.86789400000009],[-107.78079899999989,37.86774300000002],[-107.78216399999985,37.86771399999998],[-107.78245900000002,37.867772000000116],[-107.78274499999998,37.867514000000085],[-107.78288199999992,37.86633500000005],[-107.78026999999997,37.862700000000075],[-107.78046899999993,37.86130300000008],[-107.78060899999997,37.85959000000014],[-107.77956599999999,37.85818900000004],[-107.77985399999994,37.85491000000013],[-107.77990699999998,37.85401200000007],[-107.780034,37.85379700000004],[-107.77932099999998,37.85288700000007],[-107.77913899999993,37.85266899999999],[-107.77931799999999,37.85074099999997],[-107.779653,37.848839],[-107.78076799999997,37.84881000000007],[-107.78157299999998,37.848979999999926],[-107.78310599999998,37.848487000000034],[-107.78368899999998,37.847947999999974],[-107.78448600000002,37.847142000000076],[-107.78423800000002,37.845525000000066],[-107.78320500000001,37.84410800000006],[-107.78266100000002,37.843794],[-107.78249700000003,37.843241999999975],[-107.78292299999993,37.84093999999993],[-107.78684799999996,37.839085000000125],[-107.78785899999997,37.838255000000004],[-107.790234,37.83737300000007],[-107.79054599999989,37.835701000000086],[-107.79042599999991,37.83516000000009],[-107.79272299999997,37.830608999999924],[-107.79447899999997,37.829094],[-107.79457300000001,37.828384000000085],[-107.79459499999996,37.82805300000001],[-107.79506900000001,37.82641400000011],[-107.79508599999991,37.82561700000008],[-107.79570699999988,37.82458099999997],[-107.796447,37.822725000000105],[-107.79746399999993,37.82271099999997],[-107.80133599999994,37.82375600000012],[-107.80386699999991,37.82444900000007],[-107.80442399999993,37.82512100000008],[-107.80869000000001,37.82410600000014],[-107.80976099999998,37.823382000000265],[-107.81020599999994,37.821995999999956],[-107.81123500000001,37.82101800000004],[-107.813556,37.82059600000014],[-107.81602099999998,37.81957599999993],[-107.81774499999995,37.81847299999998],[-107.81903199999988,37.817006000000106],[-107.82102299999985,37.81503900000007],[-107.82182799999993,37.81452400000012],[-107.82415999999995,37.81395600000002],[-107.82758299999995,37.81231600000001],[-107.82917600000002,37.81066400000009],[-107.82895899999994,37.808626999999944],[-107.82616299999995,37.80537199999992],[-107.8246999999999,37.804852000000096],[-107.82613199999997,37.80264900000003],[-107.82788699999986,37.801086000000225],[-107.82846099999995,37.799624999999935],[-107.82428199999993,37.79669800000005],[-107.8221999999999,37.795748],[-107.82361200000003,37.793271000000004],[-107.82465000000002,37.79022800000013],[-107.82705199999998,37.78974900000014],[-107.82795899999996,37.789096000000086],[-107.82932599999992,37.78764500000011],[-107.83120199999996,37.786317],[-107.83275399999997,37.78430900000012],[-107.83579399999996,37.78288500000008],[-107.83661799999993,37.781659000000104],[-107.83866899999998,37.779418000000135],[-107.83905900000002,37.77843299999995],[-107.83948299999992,37.777251999999976],[-107.84000199999997,37.77599500000002],[-107.84085199999987,37.774138999999934],[-107.84181799999993,37.773915000000045],[-107.84277799999995,37.77363600000007],[-107.84455400000002,37.77364799999998],[-107.84587199999999,37.77466900000013],[-107.84975199999997,37.77588899999995],[-107.8511039999999,37.77625599999999],[-107.85710499999993,37.775776000000064],[-107.86084499999976,37.77551700000009],[-107.86139000000003,37.776859000000115],[-107.86424999999997,37.77912700000019],[-107.86739999999998,37.77970800000014],[-107.870497,37.78145100000029],[-107.87000899999992,37.78430200000008],[-107.87318699999997,37.78623900000031],[-107.8806709999999,37.78597800000006],[-107.88249999999988,37.78509800000023],[-107.88625699999994,37.785454000000016],[-107.88723099999999,37.78664299999997],[-107.88992999999994,37.788632000000064],[-107.89087299999989,37.7909200000002],[-107.89061800000002,37.79241500000006],[-107.88874499999986,37.7961490000003],[-107.88897699999995,37.79808100000008],[-107.89030699999995,37.79905100000025],[-107.8925079999999,37.80265700000018],[-107.89644900000002,37.80357900000007],[-107.90012399999989,37.80555900000013],[-107.903435,37.80635900000004],[-107.90484699999996,37.80745899999994],[-107.90552600000001,37.81014200000027],[-107.90624000000003,37.81102800000002],[-107.90964999999989,37.811145000000295],[-107.91080699999998,37.81240600000018],[-107.91360999999989,37.814111000000025],[-107.91561999999993,37.81654400000025],[-107.91808799999995,37.818686000000014],[-107.91774199999992,37.81957500000021],[-107.91921499999995,37.820715000000064],[-107.92134699999997,37.823713999999995],[-107.92591099999993,37.82334900000001],[-107.9276989999999,37.82362400000022],[-107.93008699999996,37.823341000000255],[-107.9338659999998,37.823494000000096],[-107.93764599999992,37.822667999999965],[-107.938673,37.82300800000007],[-107.94053199999996,37.82521400000013],[-107.94273599999991,37.82629600000013],[-107.94473799999992,37.82808200000011],[-107.947248,37.828817000000186],[-107.94671900000003,37.83067599999998],[-107.94818199999992,37.83382000000006],[-107.95076299999988,37.83575900000028],[-107.95492000000002,37.83444799999995],[-107.95900499999988,37.83451900000017],[-107.96189200000003,37.83279600000026],[-107.96334499999989,37.83285200000006],[-107.96589499999993,37.8318240000001],[-107.96759199999991,37.83178300000009],[-107.97139299999992,37.83242799999999],[-107.97334399999994,37.8335910000003],[-107.97530599999993,37.835869000000116],[-107.9772989999999,37.83922200000012],[-107.9776349999999,37.84033000000011],[-107.97982100000002,37.84301200000016],[-107.98417199999977,37.84503699999999],[-107.98559499999993,37.84712100000013],[-107.98747699999996,37.8509590000001],[-107.98796699999991,37.85460400000022],[-107.9892079999999,37.85663999999991],[-107.99213499999996,37.85641200000009],[-107.99545999999992,37.85659499999991],[-107.998986,37.8558330000003],[-108.00267899999994,37.85709400000019],[-108.00400899999994,37.85621400000002],[-108.0072889999999,37.85506600000008],[-108.01031699999999,37.855950000000064],[-108.01124300000004,37.85690899999997],[-108.01294899999976,37.85704500000003],[-108.01529399999993,37.85613900000004],[-108.0189089999999,37.85551499999991],[-108.0239479999999,37.85593700000021],[-108.02569399999993,37.855226000000016],[-108.03086499999995,37.855495000000076],[-108.03183899999999,37.854631000000154],[-108.03477199999992,37.85315100000008],[-108.03583700000001,37.8521820000002],[-108.03958399999999,37.85183900000004],[-108.04122799999976,37.85128900000001],[-108.04540299999996,37.851139000000046],[-108.04767700000002,37.85072500000007],[-108.05294899999996,37.84542699999997],[-108.05545599999999,37.844658000000095],[-108.05574899999999,37.84389100000004],[-108.05468999999994,37.842215000000294],[-108.055298,37.841625000000136],[-108.05847599999993,37.84100000000012],[-108.05981899999995,37.84262799999999],[-108.06034199999988,37.84487700000028],[-108.06173299999995,37.84499599999998],[-108.06420500000002,37.846727000000044],[-108.06647899999996,37.85104800000016],[-108.06893699999995,37.85132699999997],[-108.07007299999998,37.85045400000007],[-108.07184599999994,37.84793500000001],[-108.07561099999998,37.84674400000006],[-108.07897799999995,37.84450500000003],[-108.081029,37.844250999999986],[-108.08457699999985,37.84161700000004],[-108.08724799999987,37.8416450000002],[-108.09052600000001,37.83999400000005],[-108.09329999999994,37.840837000000136],[-108.09500599999996,37.84085000000016],[-108.09547299999997,37.84486600000025],[-108.098275,37.84841100000011],[-108.100055,37.84940000000023],[-108.10162000000003,37.84959100000003],[-108.10410200000001,37.85115200000024],[-108.10595199999989,37.85159200000004],[-108.10823199999987,37.853645000000085],[-108.11169899999993,37.854672000000164],[-108.11252300000001,37.85523500000022],[-108.11737399999993,37.85344399999997],[-108.119731,37.85335099999992],[-108.12095099999993,37.85384100000016],[-108.12221699999998,37.852539999999976],[-108.12471199999999,37.851075000000264],[-108.12465399999991,37.84859100000011],[-108.12663099999997,37.8453430000003],[-108.12843499999985,37.84397799999999],[-108.13110699999993,37.84295700000007],[-108.1322679999999,37.841547000000105],[-108.137429,37.837890000000016],[-108.13766799999996,37.83707300000009],[-108.14007699999996,37.83487400000024],[-108.14013399999999,37.832813999999985],[-108.14152699999994,37.83245400000004],[-108.14364499999994,37.83553500000005],[-108.14448799999997,37.83831700000019],[-108.14590099999992,37.83789100000013],[-108.14724099999995,37.838852999999915],[-108.1489719999999,37.83736000000022],[-108.15017399999999,37.83432600000003],[-108.15314199999989,37.835103000000174],[-108.15641600000004,37.83427200000017],[-108.16037399999988,37.83380900000003],[-108.16222299999993,37.83472899999998],[-108.16385399999996,37.834923],[-108.16620499999999,37.83331300000003],[-108.16710199999994,37.831928000000005],[-108.16914199999997,37.831110000000194],[-108.17490599999996,37.83259200000009],[-108.17834799999997,37.83117800000031],[-108.1831709999999,37.82647900000018],[-108.19021799999996,37.82524000000001],[-108.19239099999999,37.82441400000005],[-108.19572799999997,37.82408200000009],[-108.19906300000002,37.824200000000076],[-108.20545499999992,37.82177899999999],[-108.209093,37.82093600000002],[-108.21032300000002,37.82204400000023],[-108.20962199999991,37.82348500000006],[-108.21073200000001,37.82483600000012],[-108.21099799999996,37.829049],[-108.212354,37.83203800000024],[-108.21464300000002,37.83313199999998],[-108.21568200000002,37.83449099999996],[-108.21842499999997,37.834904999999935],[-108.21807799999993,37.83616599999999],[-108.21852799999988,37.837755000000016],[-108.22218399999986,37.83907600000009],[-108.22195499999981,37.839731000000086],[-108.22364400000004,37.840933000000064],[-108.22403999999995,37.842070000000035],[-108.225796,37.84245599999997],[-108.22700499999996,37.84347500000007],[-108.22652199999987,37.84552900000023],[-108.2294829999999,37.85421800000006],[-108.2300239999999,37.85717000000011],[-108.22958599999998,37.861041000000114],[-108.23159900000002,37.86174100000028],[-108.23194999999993,37.86506600000024],[-108.23286299999995,37.86712],[-108.23506099999992,37.86931300000009],[-108.23598399999992,37.87261799999999],[-108.23783700000001,37.87639199999995],[-108.23926099999989,37.87639600000011],[-108.245252,37.87865499999998],[-108.24793399999993,37.87937999999991],[-108.25036299999982,37.88457199999999],[-108.25268399999993,37.8853630000001],[-108.25424299999997,37.88647600000013],[-108.25739199999992,37.88968100000011],[-108.25773099999992,37.89104500000019],[-108.25653699999998,37.89355699999999],[-108.25649799999997,37.89460700000001],[-108.30669699999987,37.894120999999984],[-108.336589,37.89383800000002],[-108.34787599999987,37.89398399999999],[-108.35744399999999,37.893638000000124],[-108.36516299999988,37.89386999999999],[-108.37428699999987,37.89347700000013],[-108.37564099999997,37.893635000000245],[-108.39089899999999,37.893508000000224],[-108.43765599999995,37.89389900000009],[-108.45486699999998,37.894195000000025],[-108.50064500000002,37.894259000000034],[-108.55417999999992,37.89458100000019],[-108.5871679999999,37.89477900000003],[-108.61708099999998,37.89510600000028],[-108.62422400000003,37.895333999999934],[-108.62564999999978,37.89522400000027],[-108.65905899999996,37.895263000000114],[-108.670234,37.89524200000017],[-108.72616199999999,37.89533000000017],[-108.74628799999999,37.89538900000025],[-108.750654,37.895312000000274],[-108.80031200000002,37.89545100000004],[-108.81671,37.89553799999999],[-108.81812399999995,37.89375300000006],[-108.81808899999999,37.891460999999936],[-108.81870399999997,37.889478],[-108.81769899999995,37.888087999999925],[-108.82020699999998,37.88669700000031],[-108.820515,37.88508200000007],[-108.81988999999987,37.88413100000008],[-108.82052199999993,37.88085100000012],[-108.82134799999989,37.880920000000174],[-108.85372999999998,37.880942000000005],[-108.88332500000001,37.881034],[-108.90083499999997,37.88100500000007],[-108.92543399999988,37.88107900000017],[-108.93164499999995,37.88104199999998],[-108.96572199999997,37.88100400000019],[-108.99526900000001,37.88106300000027],[-109.00066399999997,37.881208000000186],[-109.01440699999995,37.88110100000006],[-109.0415119999999,37.881167000000175],[-109.04105799999991,37.907236000000125],[-109.04161499999998,37.92364200000003],[-109.04184399999991,37.93907800000005],[-109.04281599999996,37.96235900000005],[-109.04313100000002,37.97465],[-109.04287199999999,37.97492199999999],[-109.04287299999999,37.999302000000114],[-109.04281399999996,37.99999200000019],[-109.04268899999994,38.02564400000006],[-109.04236700000001,38.092839000000026],[-109.04207400000001,38.153017000000204],[-109.04166700000002,38.153021000000024],[-109.01820900000001,38.15322899999995],[-108.9984849999999,38.153473000000076],[-108.98849799999994,38.153447000000256],[-108.97957799999989,38.15366200000028],[-108.96598,38.153477000000066],[-108.94940500000001,38.153503000000114],[-108.94214199999993,38.15371000000016],[-108.92938700000002,38.15358300000031],[-108.91675900000001,38.15328800000009],[-108.90674899999982,38.15329200000002],[-108.90674899999982,38.154959000000304],[-108.90022999999985,38.15478100000013],[-108.88067199999995,38.15402400000005],[-108.87565799999987,38.15415100000024],[-108.85902699999997,38.15384800000021],[-108.85583099999985,38.153909999999996],[-108.83628099999999,38.15386800000027],[-108.82151399999987,38.15363100000002],[-108.7922929999998,38.15343400000006],[-108.77827400000001,38.15326499999992],[-108.750654,38.15342399999997],[-108.74185599999993,38.15316000000013],[-108.723503,38.15313200000014],[-108.70512399999996,38.15323500000011],[-108.68665299999986,38.153053],[-108.66825499999993,38.15310100000022],[-108.63157799999999,38.153095000000064],[-108.6256489999999,38.15329800000018],[-108.60588099999995,38.15300700000006],[-108.57361800000001,38.152818999999965],[-108.53959199999986,38.15264900000011],[-108.52155199999987,38.15330200000017],[-108.50064500000002,38.15319899999997],[-108.44548999999995,38.15330000000017],[-108.437096,38.15320000000008],[-108.41482899999994,38.153267000000085],[-108.38760199999984,38.153043000000025],[-108.37563999999998,38.15284600000007],[-108.32563299999998,38.15249699999998],[-108.31495099999995,38.15247400000021],[-108.28624899999988,38.15254500000003],[-108.26093000000003,38.15267800000004],[-108.25063499999987,38.152793000000145],[-108.24981099999991,38.15253799999999],[-108.22759100000002,38.1524330000002],[-108.1772489999999,38.152304000000186],[-108.17011500000001,38.15225399999997],[-108.13606799999991,38.152670000000114],[-108.12562799999984,38.152702999999974],[-108.118223,38.15226899999993],[-108.11304899999999,38.15222800000009],[-108.06150500000001,38.15246400000012],[-108.038882,38.15266700000001],[-108.02103799999992,38.15261600000008],[-108.011305,38.152702999999974],[-108.00062200000002,38.152480000000025],[-107.99186299999985,38.152491],[-107.96963499999993,38.15229500000015],[-107.96578899999997,38.15232800000001]]]},"properties":{"OBJECTID":58,"COUNTYFP":"113","Shape_Leng":3.44896311297,"Shape_Area":0.342338435418,"low":2180,"lowmod":3625,"LMMI":5245,"lowmoduniv":7645,"lowmod_pct":0.474166121648,"NAMELSAD10":"San Miguel County"}},
{"type":"Feature","geometry":{"type":"Polygon","coordinates":[[[-102.05141599999996,40.749585999999965],[-102.06339400000002,40.74982700000021],[-102.11275899999993,40.74946000000023],[-102.16853699999996,40.74903399999994],[-102.1908049999999,40.748896000000116],[-102.22960799999998,40.74898600000006],[-102.24478099999988,40.74892000000011],[-102.276928,40.7489800000003],[-102.28690799999998,40.749108999999976],[-102.30242199999986,40.74910300000016],[-102.33602100000002,40.749246000000085],[-102.38280299999991,40.749366000000066],[-102.40063499999991,40.749324],[-102.41960899999998,40.749153000000035],[-102.45875799999999,40.74925099999996],[-102.50421699999993,40.749388000000295],[-102.52906200000001,40.74952900000005],[-102.56886599999996,40.74945100000002],[-102.59780199999989,40.74952500000006],[-102.65130999999985,40.74940200000003],[-102.6514479999999,40.79295900000011],[-102.65181399999989,40.86544500000008],[-102.65211099999993,40.88325200000014],[-102.65210200000001,40.88976100000008],[-102.652264,40.909647000000064],[-102.652422,40.918791000000056],[-102.65259199999997,40.920300000000225],[-102.65306499999991,40.96242800000016],[-102.65348799999992,40.999989000000085],[-102.65346299999999,41.00233200000014],[-102.62546799999996,41.00235399999997],[-102.62103300000001,41.00259700000004],[-102.57573799999994,41.00226799999996],[-102.55750499999999,41.002439000000265],[-102.55672099999987,41.002200000000016],[-102.5004659999999,41.00234000000006],[-102.48807599999998,41.002443000000085],[-102.472215,41.00245100000001],[-102.46033399999999,41.002383000000066],[-102.43065000000001,41.00244700000002],[-102.36065899999994,41.002428000000066],[-102.34487199999995,41.00218899999999],[-102.29135399999996,41.002207000000055],[-102.28818499999994,41.00232100000028],[-102.27210000000002,41.002245000000016],[-102.26792799999998,41.00242500000002],[-102.23338999999999,41.002350000000035],[-102.22899299999989,41.002274000000114],[-102.2122,41.00246200000004],[-102.18927299999996,41.00231000000025],[-102.18865099999994,41.00245100000001],[-102.15288499999997,41.00244500000008],[-102.12792299999995,41.002465000000086],[-102.1220469999999,41.002344999999934],[-102.07028399999996,41.00241600000015],[-102.051717,41.00235900000007],[-102.05156299999999,40.99912700000016],[-102.05156799999986,40.96913700000016],[-102.0515309999999,40.920072000000005],[-102.05154399999992,40.879954999999995],[-102.05149799999992,40.8417280000001],[-102.05144100000001,40.80588],[-102.05141599999996,40.749585999999965]]]},"properties":{"OBJECTID":59,"COUNTYFP":"115","Shape_Leng":1.70754546552,"Shape_Area":0.152011012501,"low":695,"lowmod":1140,"LMMI":1660,"lowmoduniv":2325,"lowmod_pct":0.490322580645,"NAMELSAD10":"Sedgwick County"}},
{"type":"Feature","geometry":{"type":"Polygon","coordinates":[[[-105.92461800000001,39.69897200000014],[-105.92004199999985,39.69388800000013],[-105.9204729999999,39.69253800000013],[-105.919581,39.69028700000024],[-105.92132900000001,39.68431000000004],[-105.92465299999998,39.68015600000007],[-105.92703,39.6767030000002],[-105.92629,39.67298500000004],[-105.92228499999999,39.67012300000022],[-105.91995500000002,39.669434000000024],[-105.91486599999996,39.66578000000004],[-105.910753,39.66229400000009],[-105.90369099999992,39.66443500000014],[-105.8968119999999,39.66466400000007],[-105.89394499999992,39.65941800000002],[-105.888215,39.658427000000245],[-105.88486099999994,39.65708399999994],[-105.87961299999978,39.663638000000276],[-105.87364000000002,39.66586100000018],[-105.86810500000001,39.66690599999998],[-105.86476700000003,39.66721799999999],[-105.86296799999997,39.664957000000015],[-105.86202600000001,39.66227600000002],[-105.85947499999997,39.65870200000006],[-105.85677299999998,39.65783000000005],[-105.8567809999999,39.65588800000012],[-105.8556559999999,39.652233000000024],[-105.85434099999998,39.649081000000194],[-105.85122799999999,39.646641999999986],[-105.84866499999993,39.64415600000001],[-105.84610199999997,39.64408700000001],[-105.84154599999994,39.64618400000012],[-105.83947399999994,39.64559500000024],[-105.838863,39.64457800000014],[-105.83669099999997,39.64295000000004],[-105.83395000000002,39.64222200000023],[-105.82873999999998,39.64194900000001],[-105.82626499999992,39.64092800000009],[-105.82269600000001,39.64232900000002],[-105.82099199999999,39.64256499999999],[-105.82017699999989,39.64053500000011],[-105.819163,39.63965900000011],[-105.81848399999996,39.63638700000007],[-105.81863599999991,39.634910000000104],[-105.81664599999993,39.633546000000194],[-105.81093899999996,39.63534800000008],[-105.80622700000004,39.634947000000295],[-105.80065699999994,39.635571000000084],[-105.79827499999993,39.63635399999998],[-105.79305799999986,39.63632800000016],[-105.79119100000003,39.634236999999985],[-105.78801699999985,39.632393000000036],[-105.78460299999989,39.63114700000011],[-105.78267399999999,39.62902400000013],[-105.78222800000003,39.6243300000001],[-105.78283699999997,39.62265600000012],[-105.78270599999996,39.61795699999999],[-105.78120599999994,39.61596000000014],[-105.77732699999996,39.61275700000016],[-105.77614299999988,39.61108000000013],[-105.77624099999991,39.60833800000012],[-105.77570199999991,39.60652800000008],[-105.77596899999998,39.605139000000065],[-105.77756699999992,39.60441700000001],[-105.78138799999999,39.60089699999992],[-105.7827449999998,39.600145000000055],[-105.79166399999997,39.59789100000006],[-105.79317300000002,39.59667800000011],[-105.794374,39.594830000000115],[-105.7961049999999,39.59393800000004],[-105.80043299999988,39.59321100000028],[-105.80137099999996,39.591244000000245],[-105.803222,39.58981899999992],[-105.80596200000002,39.58583800000008],[-105.80808300000001,39.58339500000005],[-105.81379300000003,39.582405999999935],[-105.81809099999987,39.58084000000008],[-105.82006299999995,39.5821390000001],[-105.82062300000001,39.58092499999998],[-105.82271899999989,39.580093999999974],[-105.82654400000001,39.58020000000022],[-105.82783399999988,39.57906900000006],[-105.83154200000001,39.57807999999994],[-105.8336779999998,39.576135000000136],[-105.83836899999989,39.572952000000214],[-105.83839799999998,39.57145300000002],[-105.83659899999998,39.56879000000009],[-105.83061499999997,39.566460000000234],[-105.82966199999998,39.564865000000054],[-105.82852099999997,39.563452999999924],[-105.82890600000002,39.56154699999996],[-105.82606099999981,39.55744200000015],[-105.82577099999997,39.55567800000006],[-105.82233699999989,39.55301200000025],[-105.81872399999992,39.55207400000012],[-105.81580699999978,39.55197599999997],[-105.81608699999987,39.54890999999998],[-105.81892800000003,39.545815000000005],[-105.81906599999996,39.544064000000276],[-105.817631,39.53831000000025],[-105.8192679999998,39.537572000000296],[-105.82123399999995,39.53246800000005],[-105.82280899999995,39.53080200000005],[-105.8239309999999,39.530560000000094],[-105.83001599999994,39.532043000000044],[-105.83605699999981,39.5292850000003],[-105.838571,39.52782100000013],[-105.84432900000002,39.525991999999974],[-105.84537699999993,39.52375000000029],[-105.8472109999999,39.52484000000004],[-105.8512849999999,39.52612099999999],[-105.85456199999999,39.527868000000296],[-105.857732,39.532058000000234],[-105.85911399999986,39.53181300000006],[-105.86289599999998,39.527307000000064],[-105.86352499999987,39.523955],[-105.86470099999991,39.522114999999985],[-105.86804899999998,39.5198620000001],[-105.86688900000001,39.51670200000012],[-105.86180999999999,39.51306000000005],[-105.85994700000003,39.51214800000008],[-105.8594569999999,39.509225000000185],[-105.85701699999998,39.50706000000008],[-105.85477200000003,39.506401000000096],[-105.85411699999992,39.50452000000007],[-105.85639199999997,39.50513000000029],[-105.86039499999987,39.50470999999999],[-105.86358399999995,39.50230499999998],[-105.86421399999995,39.49998600000009],[-105.86593099999999,39.498082000000124],[-105.86656499999987,39.496663000000126],[-105.86658399999993,39.494351000000165],[-105.86752100000001,39.494147],[-105.86990500000002,39.49495300000024],[-105.87241399999994,39.49480800000015],[-105.87471399999998,39.493521000000044],[-105.87696599999998,39.49060199999997],[-105.87834800000002,39.489618000000235],[-105.87917299999998,39.48682400000001],[-105.87908699999997,39.483767],[-105.88467700000001,39.484138000000144],[-105.88918499999988,39.483350000000144],[-105.89224999999999,39.48356100000001],[-105.89219899999995,39.48104400000011],[-105.89472499999988,39.47751700000015],[-105.89732699999996,39.47713699999997],[-105.89672599999989,39.47565700000007],[-105.89848099999995,39.47340900000006],[-105.89909,39.47150000000022],[-105.90161199999994,39.46961900000019],[-105.90125799999998,39.46721300000024],[-105.901835,39.464248000000055],[-105.90391,39.46221000000003],[-105.905959,39.45965700000016],[-105.90835099999993,39.45942800000023],[-105.91051099999976,39.458376000000214],[-105.91163699999993,39.456736000000035],[-105.91431599999981,39.45820300000014],[-105.91707199999996,39.457969000000105],[-105.92195200000003,39.458455000000185],[-105.92667399999993,39.4594170000002],[-105.929823,39.45933800000023],[-105.93465700000002,39.45788799999997],[-105.93866300000002,39.45849800000019],[-105.94061199999999,39.456524000000286],[-105.94217000000003,39.45277300000009],[-105.94249999999988,39.45048400000002],[-105.94185299999998,39.448246000000154],[-105.94369799999987,39.44618299999996],[-105.94401799999997,39.44509600000009],[-105.94415500000002,39.440910000000144],[-105.94665099999997,39.44062400000013],[-105.953014,39.438867000000016],[-105.956436,39.438401],[-105.95819199999988,39.439677000000074],[-105.96175199999993,39.44009100000005],[-105.96439499999985,39.439531999999986],[-105.96678599999979,39.4380350000003],[-105.965013,39.433436000000086],[-105.96072600000002,39.4290180000001],[-105.95630999999986,39.42784100000006],[-105.95483699999994,39.42622000000006],[-105.95539400000001,39.42285100000021],[-105.95509199999998,39.421551000000136],[-105.95387499999993,39.42047900000023],[-105.95365200000003,39.41838100000001],[-105.95169799999991,39.41461900000013],[-105.95232499999992,39.41410599999995],[-105.9592439999999,39.412473000000034],[-105.96280999999999,39.411991000000114],[-105.96872000000002,39.41067800000002],[-105.97320599999995,39.40918900000008],[-105.97686699999991,39.408570000000225],[-105.97796699999998,39.40726900000004],[-105.98065099999991,39.40668599999992],[-105.981923,39.405974000000185],[-105.98302499999988,39.404431000000045],[-105.98362099999997,39.40244100000007],[-105.98501799999991,39.401666999999975],[-105.98693000000003,39.401507000000095],[-105.99052799999993,39.402202000000045],[-105.9928559999999,39.40185900000006],[-105.99413199999987,39.39782399999996],[-105.99592799999988,39.39500099999998],[-105.99784499999987,39.394010000000094],[-105.99952899999994,39.39382500000022],[-106.00333899999998,39.39176600000002],[-106.00453699999997,39.38979000000012],[-106.00840099999994,39.38735600000018],[-106.01385099999993,39.386897999999974],[-106.01636400000001,39.38439000000028],[-106.01699400000001,39.38291100000009],[-106.01838299999991,39.381403000000034],[-106.01745299999999,39.379204000000016],[-106.016794,39.37498800000003],[-106.01747299999994,39.3733830000001],[-106.01761899999997,39.36550300000022],[-106.02196600000002,39.36171299999995],[-106.02433000000002,39.36114700000002],[-106.02671999999995,39.361333],[-106.02864099999994,39.360224000000244],[-106.03219899999999,39.35894200000001],[-106.03471699999994,39.35837600000008],[-106.03746999999993,39.358465000000024],[-106.04392599999994,39.357377999999926],[-106.04631699999987,39.357434000000126],[-106.05092199999996,39.358061000000134],[-106.05609599999997,39.35843100000005],[-106.05833499999994,39.36005200000022],[-106.06255899999996,39.361786000000166],[-106.06407899999994,39.36267699999996],[-106.06847099999999,39.36442899999997],[-106.07285699999989,39.36791700000009],[-106.07419199999993,39.36863000000011],[-106.08076599999987,39.370228],[-106.08126700000003,39.36914500000029],[-106.08444599999996,39.369282999999996],[-106.08732399999997,39.37082700000019],[-106.08842800000002,39.372000999999955],[-106.09431199999977,39.37651100000028],[-106.096791,39.377896000000305],[-106.09940099999994,39.37773300000015],[-106.10194299999995,39.37805400000008],[-106.10603800000001,39.37793099999999],[-106.10729000000003,39.37745399999994],[-106.11197999999996,39.37705299999999],[-106.11481299999997,39.37649600000009],[-106.12030799999997,39.37659000000025],[-106.12396100000001,39.37802100000016],[-106.12744700000002,39.3788770000001],[-106.12949400000002,39.378767000000096],[-106.13143299999996,39.37951099999998],[-106.13552900000002,39.37954600000023],[-106.1364119999999,39.37969400000003],[-106.19141199999996,39.379696000000195],[-106.20673199999993,39.379636000000005],[-106.20791300000002,39.38110500000005],[-106.210534,39.3827950000001],[-106.21039999999994,39.3841240000001],[-106.20821599999988,39.38630500000011],[-106.20678199999998,39.38702600000005],[-106.20304099999998,39.38973900000019],[-106.20104200000003,39.39481500000005],[-106.20237500000002,39.39713900000015],[-106.211588,39.403609000000074],[-106.213074,39.404274000000214],[-106.21350499999988,39.405970000000025],[-106.216321,39.407196000000226],[-106.21786099999997,39.408346000000165],[-106.21698499999997,39.40993400000008],[-106.21898299999998,39.412328000000116],[-106.22090500000002,39.41229600000014],[-106.2222129999999,39.41302399999995],[-106.22536100000002,39.41362300000014],[-106.22855499999997,39.41465000000022],[-106.23040900000001,39.41695400000026],[-106.23091,39.418867999999975],[-106.23001099999993,39.419892000000004],[-106.23005799999999,39.42226100000005],[-106.232687,39.4262480000001],[-106.22759899999994,39.42809900000009],[-106.2256769999999,39.430153000000246],[-106.22544299999998,39.43184300000007],[-106.22320999999988,39.43310500000024],[-106.223251,39.43380700000023],[-106.224985,39.43660100000011],[-106.22835399999997,39.43975699999993],[-106.23015499999997,39.4409],[-106.23185799999999,39.44097100000022],[-106.23406399999999,39.44421300000005],[-106.23599100000001,39.44596200000001],[-106.23892199999995,39.451361000000304],[-106.23811,39.45415800000012],[-106.23925899999995,39.45585200000028],[-106.24179200000003,39.45738000000023],[-106.24354099999994,39.45993199999998],[-106.24514899999997,39.461476000000005],[-106.24689899999998,39.461910000000046],[-106.249933,39.46334500000012],[-106.25163799999996,39.46503899999993],[-106.25152800000001,39.46752100000015],[-106.2505799999999,39.468218999999976],[-106.25040000000001,39.47093200000029],[-106.25167999999996,39.474300000000255],[-106.25148300000001,39.47671400000013],[-106.250313,39.47743400000002],[-106.24940900000001,39.48078600000008],[-106.25008699999995,39.48204100000004],[-106.24906099999998,39.48460300000005],[-106.24554699999993,39.48939200000001],[-106.24728500000003,39.49130000000014],[-106.25243499999999,39.49286200000006],[-106.25579399999992,39.493043000000114],[-106.25916399999994,39.49421500000028],[-106.25826599999999,39.49755300000004],[-106.25687699999997,39.49966100000006],[-106.25366399999979,39.50032500000009],[-106.25107999999994,39.502098000000046],[-106.24890199999987,39.50485999999995],[-106.24811399999987,39.508148999999946],[-106.24692299999992,39.51039899999995],[-106.24641599999978,39.51275200000015],[-106.24843900000002,39.51569900000027],[-106.2505809999999,39.517122000000086],[-106.25368800000001,39.51791900000012],[-106.25576699999999,39.51989699999996],[-106.25541299999992,39.524061000000074],[-106.2546099999999,39.52635699999996],[-106.25451800000002,39.52853399999998],[-106.25577399999986,39.532234000000244],[-106.2540259999999,39.53534700000023],[-106.2505809999999,39.53854400000006],[-106.24992799999995,39.54209400000002],[-106.24829499999993,39.54425099999992],[-106.24521599999997,39.54383300000018],[-106.24342399999995,39.54411900000002],[-106.24010599999997,39.546820000000025],[-106.23674800000003,39.548594000000094],[-106.23541,39.548261999999966],[-106.23539899999986,39.54454299999992],[-106.23427299999997,39.54278099999999],[-106.23297099999996,39.542191],[-106.228861,39.538405000000296],[-106.228523,39.53704599999992],[-106.22533299999986,39.53187000000031],[-106.22443699999997,39.53121900000008],[-106.22238899999996,39.53126600000007],[-106.21818999999999,39.53200700000008],[-106.21667999999994,39.53191900000013],[-106.21435499999995,39.53105700000003],[-106.21256499999998,39.53099700000001],[-106.21126799999996,39.53013200000004],[-106.21095500000001,39.533870000000036],[-106.21039599999995,39.534595000000195],[-106.21178500000002,39.53971000000007],[-106.21011099999998,39.5425120000001],[-106.2106819999999,39.54517600000008],[-106.20992599999988,39.54694300000028],[-106.20794099999995,39.54816699999992],[-106.20431399999995,39.54890900000004],[-106.2020139999999,39.54998300000028],[-106.19870700000001,39.55002000000013],[-106.19772199999989,39.5507570000002],[-106.197248,39.55265000000014],[-106.19764799999996,39.55506600000001],[-106.19645299999996,39.55734600000005],[-106.19874199999998,39.557885000000056],[-106.20376599999992,39.55779700000022],[-106.20571899999976,39.55821100000003],[-106.20864599999993,39.56163200000009],[-106.20946500000002,39.56358300000022],[-106.20914199999993,39.56464299999999],[-106.20992699999988,39.567047000000116],[-106.20982299999997,39.57033900000005],[-106.21160899999995,39.57336900000007],[-106.21453099999997,39.57432100000017],[-106.21502699999996,39.575186000000144],[-106.21364899999998,39.578776000000175],[-106.21395100000001,39.5820150000003],[-106.21086500000001,39.586803000000145],[-106.21166,39.58981100000017],[-106.2112239999999,39.593213000000276],[-106.20903699999997,39.59939600000007],[-106.20737400000002,39.601247000000114],[-106.20605599999993,39.60204500000009],[-106.20566300000002,39.603866000000096],[-106.20210499999996,39.60583600000001],[-106.20041800000001,39.60640400000011],[-106.19396499999988,39.605502000000286],[-106.19032299999998,39.60397900000021],[-106.18587399999984,39.60330399999998],[-106.18397400000003,39.60417300000029],[-106.1820679999999,39.60591400000027],[-106.17874999999998,39.60786400000012],[-106.17760399999992,39.60905600000018],[-106.17796699999997,39.61179500000031],[-106.17611499999992,39.614220999999986],[-106.17591999999996,39.616111000000046],[-106.17641299999997,39.619729000000234],[-106.176218,39.62391900000017],[-106.17746499999998,39.63167700000008],[-106.17615599999993,39.63332700000001],[-106.17646400000001,39.6357220000001],[-106.17820799999998,39.637140000000045],[-106.17956100000004,39.63960400000013],[-106.17934400000001,39.642503000000204],[-106.1813949999999,39.64591200000018],[-106.18205,39.647666000000186],[-106.18348999999995,39.6486120000003],[-106.1842319999999,39.65063600000025],[-106.18384000000003,39.652153],[-106.18920499999996,39.65334100000024],[-106.19147299999992,39.651861],[-106.19439399999993,39.65112300000004],[-106.19709899999992,39.65146400000003],[-106.19801499999988,39.65234200000003],[-106.20216099999999,39.654034000000024],[-106.20665399999996,39.6544430000003],[-106.20770099999999,39.65528800000004],[-106.20922699999994,39.65427800000015],[-106.21716500000002,39.65459200000004],[-106.21895499999988,39.65628000000004],[-106.2191749999999,39.657359000000156],[-106.21839299999994,39.66076300000026],[-106.21996499999989,39.66258600000009],[-106.22245199999998,39.66302700000017],[-106.224196,39.66259099999996],[-106.22646299999991,39.66094100000004],[-106.22951699999999,39.66114700000003],[-106.232485,39.662229000000025],[-106.23483999999979,39.66203000000013],[-106.23986300000001,39.66695700000008],[-106.240476,39.66931800000003],[-106.23930099999995,39.6709350000001],[-106.23882700000001,39.67599200000001],[-106.23909099999992,39.67774500000007],[-106.23874499999988,39.680274000000054],[-106.24035900000001,39.68024200000008],[-106.24284699999998,39.68091800000025],[-106.2505809999999,39.680210999999986],[-106.25199399999991,39.67903700000022],[-106.25558199999995,39.68003400000009],[-106.25824699999993,39.679804000000104],[-106.2613209999999,39.68271600000014],[-106.26161699999994,39.685874000000126],[-106.26399799999996,39.68705299999999],[-106.26563899999996,39.688529000000074],[-106.265897,39.68953600000009],[-106.26433099999997,39.691805000000045],[-106.26405499999998,39.6945750000001],[-106.26318900000001,39.69558100000006],[-106.26262199999996,39.69785100000007],[-106.26429099999996,39.7012840000001],[-106.26902599999988,39.702563000000055],[-106.27205200000003,39.70434799999998],[-106.27362599999998,39.702997000000266],[-106.27592199999998,39.70004399999999],[-106.27740099999994,39.69953900000013],[-106.28069899999997,39.70042100000029],[-106.28325799999988,39.70151100000004],[-106.28548499999994,39.70195699999999],[-106.28729499999997,39.7029500000001],[-106.28929299999999,39.703201999999976],[-106.29091599999998,39.70406600000007],[-106.29131999999993,39.70538699999997],[-106.29283800000002,39.7064840000001],[-106.2941879999999,39.70934500000004],[-106.29393299999992,39.7123170000001],[-106.2951789999999,39.71469400000012],[-106.29518699999994,39.716417000000035],[-106.29654900000003,39.71732900000001],[-106.29834599999992,39.72056900000007],[-106.29937699999988,39.720488999999986],[-106.3040069999999,39.72256400000009],[-106.30416300000002,39.72291900000016],[-106.30941899999988,39.72480100000007],[-106.31201199999992,39.72519000000011],[-106.31253199999998,39.72575400000005],[-106.316195,39.72717599999993],[-106.31849499999998,39.72865999999999],[-106.32048099999997,39.731674],[-106.32214599999998,39.73252100000025],[-106.32541500000002,39.733232999999984],[-106.32696299999992,39.73655200000002],[-106.32802499999985,39.73760000000004],[-106.32938899999994,39.73782700000015],[-106.33127200000001,39.74013100000002],[-106.33136300000001,39.74254799999994],[-106.332986,39.74407100000002],[-106.33387899999991,39.74655199999995],[-106.33562799999993,39.749526],[-106.33889599999998,39.750670000000014],[-106.342105,39.753030000000024],[-106.34333799999996,39.75529199999994],[-106.34355299999999,39.75904200000008],[-106.34591799999993,39.758111000000156],[-106.3507439999999,39.756879000000026],[-106.35210899999987,39.7555880000001],[-106.3539859999999,39.75255700000014],[-106.35913799999986,39.753152],[-106.36219800000003,39.75194699999997],[-106.36431299999987,39.75223000000028],[-106.368472,39.75558700000022],[-106.37053899999995,39.75661200000013],[-106.3733529999999,39.759549000000106],[-106.37419699999992,39.76221800000013],[-106.37921900000003,39.763036],[-106.37993299999988,39.764325999999926],[-106.38161799999995,39.76542700000027],[-106.38547800000003,39.76716400000009],[-106.38590299999993,39.76956200000029],[-106.391033,39.77694200000019],[-106.39092299999987,39.77791700000023],[-106.3888879999999,39.78422500000005],[-106.38856999999996,39.78654499999999],[-106.39031599999993,39.788725000000056],[-106.39430700000003,39.79056400000019],[-106.39686199999994,39.7900930000003],[-106.39664799999991,39.79170700000026],[-106.39579100000003,39.792707000000064],[-106.39642000000003,39.795781000000034],[-106.40073699999999,39.798736000000076],[-106.40127999999999,39.79985499999992],[-106.40090399999997,39.80281500000007],[-106.40109999999993,39.80564800000002],[-106.40188899999993,39.80754500000012],[-106.40618899999993,39.811240999999995],[-106.40716099999997,39.8128040000002],[-106.40946799999989,39.81920200000002],[-106.41071299999987,39.82036400000027],[-106.41119900000001,39.822089000000176],[-106.4106789999999,39.823698999999976],[-106.41111999999998,39.825491000000284],[-106.41222999999997,39.82678600000014],[-106.4120109999999,39.828670000000045],[-106.41510699999998,39.831440999999984],[-106.41872399999994,39.83627300000012],[-106.420612,39.840003000000195],[-106.42373900000001,39.84142500000007],[-106.42525499999994,39.844172000000015],[-106.42557499999998,39.84548800000016],[-106.423497,39.84805200000005],[-106.42362500000002,39.85010599999998],[-106.42578600000002,39.85536900000005],[-106.42708999999991,39.857675000000256],[-106.43038199999995,39.861526000000026],[-106.43268799999998,39.86455699999999],[-106.43273999999985,39.865968000000066],[-106.4339589999999,39.87026400000002],[-106.43441899999993,39.873001000000045],[-106.43379199999993,39.8749870000002],[-106.43396199999995,39.87737500000026],[-106.43109699999997,39.88010300000002],[-106.42810299999996,39.882164000000046],[-106.42947399999997,39.88711399999994],[-106.42868299999998,39.89510699999994],[-106.429237,39.89838700000013],[-106.43075999999996,39.901620000000264],[-106.43064900000002,39.90390099999996],[-106.42998999999986,39.90540300000009],[-106.43040000000002,39.90890600000006],[-106.43287499999991,39.91260700000004],[-106.43917899999997,39.91563200000019],[-106.44030099999998,39.91832300000027],[-106.43920200000002,39.919944999999984],[-106.43442399999992,39.92338700000022],[-106.43450799999994,39.924914000000115],[-106.4077099999999,39.92476600000009],[-106.39525700000002,39.924589000000196],[-106.39543300000003,39.91401600000006],[-106.38741099999999,39.91391199999998],[-106.38111499999997,39.914016999999944],[-106.375585,39.913636000000054],[-106.36748999999992,39.91382599999997],[-106.36235499999987,39.91373599999997],[-106.34351800000002,39.913798000000156],[-106.33845799999978,39.91370100000029],[-106.32628299999999,39.91374800000028],[-106.294624,39.91396000000003],[-106.29477699999995,39.91892000000013],[-106.289785,39.918851000000075],[-106.2716519999999,39.916685000000086],[-106.252027,39.91471200000012],[-106.2505809999999,39.91304800000006],[-106.2469099999999,39.912700000000086],[-106.2436239999999,39.90929],[-106.24111499999987,39.90726899999993],[-106.23929799999996,39.90681399999994],[-106.234824,39.90270800000002],[-106.23168799999996,39.900806000000046],[-106.23188099999993,39.899895000000015],[-106.23000899999994,39.89799600000026],[-106.22972399999998,39.89656000000008],[-106.228361,39.89351700000003],[-106.22751999999997,39.89250800000002],[-106.2241259999999,39.88996800000001],[-106.2230889999999,39.888304000000005],[-106.22078199999999,39.88768600000003],[-106.21937400000002,39.8853040000003],[-106.21932099999998,39.8836110000002],[-106.21686,39.88063000000028],[-106.2141739999999,39.878801000000294],[-106.21147999999994,39.87844000000007],[-106.207989,39.87839900000006],[-106.20649900000001,39.87770100000023],[-106.206368,39.87498500000004],[-106.20519199999995,39.873167999999964],[-106.20320099999998,39.87125800000007],[-106.2027619999999,39.86975600000011],[-106.20139099999977,39.86852399999998],[-106.20154500000001,39.86546499999997],[-106.20056599999992,39.86404200000004],[-106.195424,39.861084999999946],[-106.19428899999997,39.85968100000014],[-106.18567999999988,39.85542100000009],[-106.18330500000002,39.85352399999999],[-106.18044599999979,39.852456000000075],[-106.17769999999996,39.84925799999996],[-106.17474299999992,39.84794900000003],[-106.17251099999999,39.84791300000006],[-106.1725219999999,39.84639700000008],[-106.17114300000003,39.8452930000002],[-106.16890599999994,39.84502400000014],[-106.16760299999999,39.84554100000008],[-106.1651169999999,39.84382200000016],[-106.16161099999988,39.84366499999999],[-106.15631699999989,39.84220100000016],[-106.15165699999989,39.840687000000116],[-106.14950699999997,39.83956100000006],[-106.14340400000003,39.83934799999997],[-106.14243699999997,39.83996500000006],[-106.13941299999988,39.84301100000005],[-106.13791200000003,39.84353799999991],[-106.13452699999999,39.84367700000024],[-106.131483,39.84420000000017],[-106.12972699999995,39.84400399999993],[-106.12445300000002,39.84179800000021],[-106.12345399999987,39.84034500000024],[-106.1217669999998,39.83897700000006],[-106.12069399999996,39.83739500000024],[-106.12030999999996,39.83567199999993],[-106.11703999999986,39.8319920000003],[-106.11458700000003,39.83038600000009],[-106.11242700000003,39.82970600000016],[-106.10986299999996,39.82745700000004],[-106.10528399999998,39.82238100000018],[-106.10288500000001,39.82084000000003],[-106.09991600000001,39.81980399999992],[-106.09900299999987,39.81842999999992],[-106.09580299999988,39.816201999999976],[-106.09434399999986,39.8146230000001],[-106.09201599999989,39.81287100000009],[-106.08997399999987,39.809336000000144],[-106.08653199999992,39.80799400000012],[-106.08411499999994,39.80665499999998],[-106.083711,39.80596400000002],[-106.08394599999991,39.80332499999997],[-106.08235300000001,39.80070599999999],[-106.08216299999998,39.79633600000028],[-106.07948699999992,39.79415900000009],[-106.07806900000003,39.79341000000011],[-106.07919699999991,39.792266000000154],[-106.08120699999995,39.78739899999999],[-106.07898199999988,39.785356000000036],[-106.07765199999977,39.7828880000003],[-106.07516799999996,39.78096000000011],[-106.07300900000001,39.77713499999999],[-106.07103199999995,39.77267900000027],[-106.07104499999997,39.76974400000006],[-106.06956999999989,39.7662440000002],[-106.06752099999989,39.763934000000006],[-106.06555500000002,39.76262400000002],[-106.06482699999992,39.760733000000016],[-106.06501500000002,39.75855400000006],[-106.06573600000002,39.757558000000074],[-106.06589799999989,39.75379900000024],[-106.064055,39.75218100000012],[-106.06340399999988,39.75074200000006],[-106.06394699999993,39.74891600000001],[-106.0648149999999,39.748201000000165],[-106.06402600000001,39.74592400000023],[-106.06086399999992,39.741369000000304],[-106.05971899999997,39.74062100000003],[-106.05946899999998,39.73943200000025],[-106.06026700000001,39.73801700000024],[-106.05982099999989,39.73610500000012],[-106.06001700000002,39.73317600000007],[-106.05803899999995,39.72881200000012],[-106.05866100000003,39.72627200000011],[-106.05854799999997,39.72470099999998],[-106.056649,39.72301299999992],[-106.05595099999977,39.721404000000064],[-106.05126199999995,39.72148299999998],[-106.04863499999982,39.72107200000028],[-106.04792999999995,39.72001900000021],[-106.04466999999994,39.718924000000186],[-106.04406499999999,39.71767500000004],[-106.04158299999989,39.71602000000007],[-106.03810599999997,39.712058000000184],[-106.03826199999997,39.70959699999992],[-106.03788799999995,39.70757900000012],[-106.03675099999992,39.70552400000008],[-106.03430999999995,39.70295800000008],[-106.03364199999999,39.69676000000004],[-106.03188899999981,39.69417500000003],[-106.02633200000002,39.690751999999975],[-106.02668599999998,39.68952100000007],[-106.02378299999998,39.68842000000012],[-106.02118799999994,39.68840299999994],[-106.01943599999993,39.6875500000001],[-106.01560099999995,39.68711600000023],[-106.01298699999995,39.686341000000084],[-106.00881399999992,39.68777399999999],[-106.00648399999994,39.68795300000022],[-106.00460399999992,39.68734400000028],[-106.00289999999995,39.68583799999999],[-105.99633900000003,39.684122000000116],[-105.99097799999993,39.68752000000006],[-105.98857099999992,39.68700100000012],[-105.98540099999991,39.687376000000256],[-105.98095099999995,39.68976900000018],[-105.97818699999988,39.689762000000144],[-105.97235,39.691689999999994],[-105.97042699999997,39.6905720000002],[-105.96194100000002,39.690781999999956],[-105.95103599999976,39.695149000000015],[-105.94560099999984,39.688866000000075],[-105.94057099999998,39.69060200000018],[-105.93649499999987,39.69256999999993],[-105.93280099999993,39.69468200000006],[-105.9300899999999,39.69751800000006],[-105.92665999999997,39.69868700000018],[-105.92461800000001,39.69897200000014]]]},"properties":{"OBJECTID":60,"COUNTYFP":"117","Shape_Leng":2.59953732281,"Shape_Area":0.168272038762,"low":8835,"lowmod":14090,"LMMI":19645,"lowmoduniv":28580,"lowmod_pct":0.49300209937,"NAMELSAD10":"Summit County"}},
{"type":"Feature","geometry":{"type":"Polygon","coordinates":[[[-105.033544,39.12981900000011],[-105.03278499999999,39.1230690000001],[-105.03290799999996,39.119080999999994],[-105.03255699999994,39.11376500000017],[-105.03255799999994,39.11008099999998],[-105.03179499999999,39.10423500000002],[-105.031746,39.10170400000004],[-105.03122999999994,39.09790099999992],[-105.03141499999987,39.091922000000125],[-105.03056099999992,39.086398000000145],[-105.03050899999988,39.08036000000004],[-105.02954799999986,39.07097299999998],[-105.02926699999989,39.06452100000013],[-105.02868899999993,39.06078200000002],[-105.02848199999994,39.055499000000054],[-105.02809300000001,39.05089200000009],[-105.02809200000002,39.04651499999994],[-105.02718799999991,39.03651600000006],[-105.02694499999996,39.029831000000286],[-105.02731499999993,39.023081000000275],[-105.027626,39.02067500000015],[-105.0279339999999,39.013127],[-105.02862199999998,39.00771199999997],[-105.02914299999992,39.00557500000008],[-105.02899500000001,39.00407299999995],[-105.02982700000001,38.9994040000002],[-105.0272,38.985334999999964],[-105.02776999999992,38.94573300000013],[-105.02747299999993,38.93894499999999],[-105.02710799999994,38.93617400000005],[-105.02787799999999,38.93277400000011],[-105.02819499999998,38.914515000000165],[-105.02875499999988,38.87603700000005],[-105.02890299999996,38.868907000000036],[-105.06670000000003,38.86950800000022],[-105.06732999999991,38.86530500000009],[-105.067409,38.86215300000009],[-105.06795399999987,38.85968800000006],[-105.06765999999988,38.85899100000029],[-105.07163699999995,38.81944000000021],[-105.07252900000003,38.808400000000006],[-105.072184,38.799382000000264],[-105.05318399999982,38.79912900000011],[-105.030619,38.797132999999974],[-105.01531799999992,38.796947000000046],[-105.01103299999994,38.79698700000017],[-105.00535899999988,38.79739600000005],[-104.98576199999997,38.79763500000013],[-104.93899399999998,38.79688399999998],[-104.93897699999991,38.79229100000009],[-104.93920200000002,38.781811000000005],[-104.93926399999992,38.75715700000029],[-104.93942600000003,38.74998800000026],[-104.93900599999989,38.71052600000007],[-104.93955499999993,38.70364500000005],[-104.93974400000002,38.69614200000029],[-104.94223499999998,38.696199000000206],[-104.9424259999999,38.64988200000005],[-104.99491999999998,38.650874000000044],[-105.00053199999991,38.65118300000023],[-105.03565900000001,38.65139300000021],[-105.05392599999993,38.65309600000006],[-105.05755399999993,38.65169300000014],[-105.06694800000002,38.64976100000001],[-105.07152899999988,38.64855],[-105.09295399999996,38.64776599999993],[-105.10685399999988,38.64753600000029],[-105.11587600000001,38.64762700000017],[-105.125497,38.647860000000264],[-105.19589699999989,38.64770400000015],[-105.240362,38.647595000000194],[-105.24042999999995,38.659563000000105],[-105.23992399999992,38.677058999999986],[-105.23779200000001,38.696901000000025],[-105.24243199999995,38.69698100000005],[-105.25053500000001,38.69648400000017],[-105.279222,38.69674300000008],[-105.28544,38.69675200000012],[-105.31351299999989,38.69699899999995],[-105.3291339999999,38.69720499999994],[-105.32879999999989,38.74455800000021],[-105.32906599999995,38.74988999999994],[-105.329161,38.76733200000018],[-105.329159,38.79291500000022],[-105.32898999999998,38.811216000000286],[-105.32878499999998,38.815963000000295],[-105.32892600000002,38.85772900000012],[-105.328915,38.86767100000009],[-105.32909499999977,38.86901999999998],[-105.32815299999999,38.875011000000256],[-105.32870099999985,38.88946900000013],[-105.32840799999997,38.892663000000084],[-105.32905799999992,38.95018500000009],[-105.32926299999991,38.96645899999993],[-105.32959700000004,38.9987440000001],[-105.32954799999987,38.99999100000008],[-105.33007199999997,39.05869400000029],[-105.33058900000003,39.11837800000029],[-105.33044399999994,39.12968500000022],[-105.3292199999999,39.12968900000021],[-105.28566299999994,39.12967600000019],[-105.27549399999992,39.129525000000115],[-105.25054799999992,39.12954600000012],[-105.24622299999993,39.12961000000024],[-105.221676,39.129585000000304],[-105.20147500000002,39.12965100000025],[-105.19910400000003,39.12956500000024],[-105.17177700000002,39.12953900000002],[-105.16260999999997,39.12947900000023],[-105.12554399999999,39.12949400000008],[-105.10656999999992,39.129604000000086],[-105.033544,39.12981900000011]]]},"properties":{"OBJECTID":61,"COUNTYFP":"119","Shape_Leng":1.82234189026,"Shape_Area":0.150289879862,"low":4215,"lowmod":9375,"LMMI":13505,"lowmoduniv":23160,"lowmod_pct":0.404792746114,"NAMELSAD10":"Teller County"}},
{"type":"Feature","geometry":{"type":"Polygon","coordinates":[[[-102.77996000000002,40.43844600000017],[-102.78015599999998,40.4267930000002],[-102.78008699999998,40.417768999999964],[-102.78008799999998,40.381343000000015],[-102.78017999999992,40.35114400000015],[-102.79358200000001,40.351280999999915],[-102.79347299999995,40.317686000000094],[-102.79329799999988,40.29221600000017],[-102.79330099999993,40.260501999999974],[-102.79320000000001,40.23929600000008],[-102.79337800000002,40.21416500000015],[-102.79368899999997,40.19234999999992],[-102.79411399999998,40.17541200000005],[-102.79418199999992,40.16761600000001],[-102.79419299999995,40.143125000000055],[-102.79412200000002,40.13201800000013],[-102.794198,40.11826200000007],[-102.794082,40.08933500000029],[-102.79367599999989,40.06034900000009],[-102.793454,40.050774000000104],[-102.793455,40.02884300000022],[-102.79330500000003,40.00249300000013],[-102.80376699999994,40.00255400000003],[-102.80343799999997,39.98946100000029],[-102.80332699999985,39.977484000000004],[-102.80310199999997,39.972558000000106],[-102.80298399999998,39.95998800000007],[-102.80274500000002,39.95040400000005],[-102.80261100000001,39.94052000000028],[-102.802074,39.914821000000074],[-102.80113199999994,39.865351000000146],[-102.80041699999987,39.83083100000016],[-102.80054499999989,39.81609700000001],[-102.80095499999987,39.78666900000013],[-102.80111899999997,39.74998700000015],[-102.80190800000003,39.720515000000205],[-102.80152299999997,39.71932500000014],[-102.80152399999997,39.713881000000015],[-102.8011689999999,39.713538000000085],[-102.80149699999987,39.698658999999964],[-102.80156299999999,39.68572300000011],[-102.80190800000003,39.68086000000005],[-102.80235599999997,39.63617700000009],[-102.80254799999989,39.621739000000275],[-102.80271600000003,39.591274999999996],[-102.80293199999994,39.5678410000001],[-102.81615699999986,39.56797600000027],[-102.83131500000002,39.568033000000185],[-102.86478699999998,39.568504000000246],[-102.87547799999993,39.5682680000001],[-102.889861,39.56840599999998],[-102.92924700000003,39.56857700000006],[-102.95285499999989,39.568381000000045],[-102.95418799999987,39.568265],[-103.00049200000001,39.567892000000086],[-103.00949800000001,39.56786299999999],[-103.0192019999999,39.567611000000056],[-103.02819999999991,39.56759000000011],[-103.03390499999989,39.56773900000019],[-103.04343399999988,39.56757299999998],[-103.06813199999999,39.567275000000166],[-103.08613899999995,39.56685500000003],[-103.09036199999997,39.56690300000014],[-103.10793699999999,39.56660700000015],[-103.12412199999994,39.566263000000276],[-103.14348399999989,39.565922000000114],[-103.15399199999996,39.56595900000002],[-103.15437600000001,39.565654000000166],[-103.15495499999997,39.565949000000046],[-103.18206799999996,39.56614200000001],[-103.21196899999995,39.5661750000001],[-103.263576,39.56644100000011],[-103.31815699999987,39.56655300000011],[-103.31938099999996,39.56641400000001],[-103.34817900000002,39.566396999999995],[-103.34994699999993,39.56630800000028],[-103.38749999999987,39.566174000000046],[-103.42805699999997,39.56598800000006],[-103.44574499999982,39.56598100000002],[-103.45351499999987,39.565885000000264],[-103.469336,39.56590500000016],[-103.47213099999988,39.56575200000009],[-103.48107399999998,39.565996999999925],[-103.48961600000001,39.565742],[-103.492032,39.56599100000017],[-103.52961899999991,39.566102000000285],[-103.56378099999995,39.56606900000003],[-103.58281199999982,39.566134999999974],[-103.59843299999983,39.56612300000006],[-103.62932699999993,39.566220999999985],[-103.63181999999989,39.56575900000013],[-103.63510100000002,39.56576000000007],[-103.63777299999992,39.566222000000266],[-103.68385899999998,39.56635100000028],[-103.699117,39.56625800000006],[-103.70707199999987,39.56629500000008],[-103.70696399999997,39.58757400000002],[-103.70661299999989,39.624985000000265],[-103.70661699999988,39.65838900000011],[-103.70662299999992,39.697811],[-103.70667399999996,39.7254410000001],[-103.70654699999994,39.739894000000106],[-103.70657699999992,39.76555300000007],[-103.70651900000001,39.78349100000008],[-103.70637999999997,39.79605900000007],[-103.70626299999998,39.82854800000007],[-103.70630299999999,39.857679000000076],[-103.70629499999995,39.888999000000126],[-103.70610299999998,39.908538000000135],[-103.70617099999993,39.93264399999998],[-103.70617099999993,39.95397300000013],[-103.70624399999991,39.95888000000025],[-103.70601999999997,39.973374999999976],[-103.70569499999999,39.98511399999995],[-103.70570099999992,40.00136600000013],[-103.67917899999992,40.001207000000136],[-103.65839999999992,40.001129000000105],[-103.62981200000002,40.00112200000001],[-103.62962499999998,40.00138500000014],[-103.59610599999996,40.001338000000146],[-103.57636099999996,40.0015480000003],[-103.56695500000001,40.00119100000023],[-103.5664319999999,40.00139799999994],[-103.53552999999994,40.001427000000035],[-103.50729499999994,40.00139100000007],[-103.47822999999994,40.001408000000254],[-103.47199,40.001502000000016],[-103.47217999999992,40.0306580000003],[-103.47210199999995,40.04296100000022],[-103.47226099999989,40.05961100000013],[-103.47209599999991,40.088481],[-103.47234400000002,40.10345500000028],[-103.47320100000002,40.146760000000256],[-103.47348099999994,40.15177800000015],[-103.47328199999998,40.156769999999995],[-103.47348399999998,40.166652],[-103.4733849999999,40.18224800000007],[-103.47317699999991,40.193846000000065],[-103.472624,40.21713700000021],[-103.47232099999997,40.22718700000007],[-103.4725949999999,40.23315900000017],[-103.47259799999978,40.23862800000006],[-103.47227099999992,40.24235700000003],[-103.472331,40.24508000000009],[-103.47180300000002,40.24997900000028],[-103.47139699999997,40.29177900000025],[-103.471004,40.33270500000003],[-103.47120999999987,40.3333990000001],[-103.47098199999994,40.34975900000012],[-103.46462500000001,40.34992000000011],[-103.46478299999995,40.354934000000014],[-103.46477299999992,40.36409400000014],[-103.464537,40.39304000000027],[-103.46448499999997,40.406242000000134],[-103.46537699999999,40.43634600000013],[-103.40704999999997,40.436360000000036],[-103.38766399999986,40.43557000000004],[-103.38706499999995,40.43543799999998],[-103.375496,40.435471000000234],[-103.37496199999987,40.4352980000001],[-103.36720099999997,40.43529399999994],[-103.32150299999995,40.43558100000007],[-103.25049199999995,40.43603700000017],[-103.22655800000001,40.436087000000214],[-103.22641299999992,40.4361780000001],[-103.1913429999999,40.436458000000016],[-103.15110800000002,40.436710999999946],[-103.15034000000003,40.43678999999992],[-103.13547800000003,40.436779000000115],[-103.13211000000001,40.43661800000007],[-103.09031499999998,40.43684500000006],[-103.08322099999992,40.43679500000002],[-103.062252,40.43694099999999],[-103.00851999999998,40.437432000000115],[-102.98660299999995,40.43751600000013],[-102.97363599999989,40.437656000000175],[-102.95083999999997,40.43777100000028],[-102.94897900000001,40.43787400000008],[-102.89315699999997,40.43827700000003],[-102.88262600000002,40.43831499999999],[-102.84905200000003,40.43827799999997],[-102.80605699999995,40.438481000000024],[-102.79391599999985,40.438507999999956],[-102.77996000000002,40.43844600000017]]]},"properties":{"OBJECTID":62,"COUNTYFP":"121","Shape_Leng":3.59641099724,"Shape_Area":0.689155959914,"low":945,"lowmod":1995,"LMMI":2995,"lowmoduniv":4520,"lowmod_pct":0.441371681416,"NAMELSAD10":"Washington County"}},
{"type":"Feature","geometry":{"type":"MultiPolygon","coordinates":[[[[-104.97944499999988,40.033058000000096],[-104.97304999999994,40.03303700000015],[-104.973074,40.029511999999954],[-104.97991699999994,40.029486000000134],[-104.97944499999988,40.033058000000096]]],[[[-103.57377399999996,41.00171599999993],[-103.57344599999999,40.99267800000018],[-103.57333699999987,40.978533000000255],[-103.57394799999997,40.963296000000014],[-103.57376399999993,40.94809299999997],[-103.57356899999996,40.925208000000055],[-103.57382099999995,40.92305399999998],[-103.57367099999988,40.91721000000018],[-103.57369899999998,40.90188699999993],[-103.573489,40.8985780000001],[-103.57372099999992,40.89305400000006],[-103.57375000000002,40.88363600000014],[-103.57396599999976,40.876187000000186],[-103.57360899999998,40.861834999999985],[-103.57354399999986,40.84726500000011],[-103.57337100000001,40.83280200000013],[-103.57321599999989,40.78916600000031],[-103.57326499999994,40.78191500000008],[-103.57400799999988,40.731036000000074],[-103.57430099999988,40.70201800000024],[-103.58037100000001,40.70202300000011],[-103.58049,40.68313400000005],[-103.58102199999996,40.65403999999995],[-103.58135799999997,40.63074999999992],[-103.58151099999992,40.62498300000027],[-103.58139999999997,40.61594400000001],[-103.58160799999996,40.61376900000005],[-103.58151999999995,40.60438599999992],[-103.58161999999987,40.595976000000064],[-103.58171299999992,40.56698600000021],[-103.58184799999992,40.53787200000005],[-103.58184399999993,40.52338300000008],[-103.5818369999999,40.5231600000003],[-103.6198,40.52286700000002],[-103.6579769999999,40.52246800000023],[-103.68674499999986,40.52208899999994],[-103.6962509999999,40.52202400000027],[-103.71488699999998,40.522387000000094],[-103.80336499999987,40.52417500000007],[-103.86028599999997,40.523941000000264],[-103.89838999999995,40.523874000000035],[-103.93664499999988,40.52395000000013],[-103.99371999999994,40.52417900000023],[-104.01295099999993,40.52432799999997],[-104.04663600000003,40.52438700000005],[-104.07063999999991,40.52434700000009],[-104.09430299999997,40.524419000000194],[-104.14771400000001,40.52438900000004],[-104.14766700000001,40.50996200000003],[-104.14730999999978,40.48112100000026],[-104.1468979999998,40.452303000000256],[-104.14674500000001,40.43766599999992],[-104.14652799999993,40.43043500000027],[-104.14690699999994,40.40925600000014],[-104.14715199999995,40.40069800000026],[-104.14738999999992,40.37976500000002],[-104.14729499999999,40.37392200000028],[-104.147536,40.3502450000002],[-104.15049399999998,40.3502450000002],[-104.15026199999988,40.318661000000304],[-104.14997499999993,40.249979999999994],[-104.15008899999987,40.232768000000135],[-104.15005799999977,40.217968999999925],[-104.14995499999998,40.203588000000025],[-104.14988299999987,40.172302000000286],[-104.14990799999998,40.16012000000006],[-104.1501429999999,40.116737],[-104.15029199999987,40.04424000000017],[-104.15033199999999,40.000856000000226],[-104.16933399999988,40.000781000000245],[-104.21926500000001,40.000829000000124],[-104.26738599999993,40.00092100000029],[-104.30149599999993,40.000765],[-104.38137999999992,40.00069000000002],[-104.39537799999994,40.000822000000255],[-104.40365999999995,40.00074200000006],[-104.44010999999995,40.00071900000006],[-104.45226000000002,40.0006250000003],[-104.49997899999988,40.0006250000003],[-104.51110199999988,40.00058400000029],[-104.52724199999994,40.000628000000006],[-104.54679599999986,40.000602000000185],[-104.59234800000002,40.00059299999998],[-104.60185999999987,40.00052899999997],[-104.62440099999992,40.00057000000004],[-104.63155199999989,40.00053099999997],[-104.65098499999999,40.00058200000029],[-104.665437,40.00053099999997],[-104.71066099999996,40.0005000000001],[-104.74592899999993,40.000496999999996],[-104.78853100000003,40.00040700000005],[-104.8003169999999,40.000430000000165],[-104.84879799999987,40.00038300000017],[-104.90478899999994,40.00032299999998],[-104.961408,40.000337000000286],[-104.96153800000002,40.00040400000012],[-104.96148299999999,40.014979000000096],[-104.96132499999999,40.01515100000012],[-104.96124199999986,40.032687000000124],[-104.961071,40.044144000000074],[-104.97852599999999,40.04388799999998],[-104.97990500000003,40.042111000000034],[-104.98004099999991,40.04031600000002],[-104.98006299999997,40.02932500000014],[-104.98649999999992,40.02930900000024],[-104.98654299999993,40.02579900000006],[-104.999007,40.02577600000012],[-104.99900499999984,40.02350899999993],[-104.99674199999993,40.02350899999993],[-104.99678399999993,40.02082000000007],[-104.99617599999999,40.01943199999994],[-104.9990699999999,40.01613100000003],[-104.99906499999992,40.01488699999993],[-104.9896139999999,40.01489000000021],[-104.98964699999999,40.00767900000005],[-105.00799499999994,40.0075810000003],[-105.00790599999993,40.00048300000009],[-105.04623199999997,40.0004660000003],[-105.05282299999999,40.00026000000014],[-105.05564199999998,40.000249999999994],[-105.05540999999988,40.01771000000031],[-105.05536899999987,40.0291850000001],[-105.05550499999993,40.04399600000005],[-105.05541900000003,40.09018400000019],[-105.05543899999998,40.1118690000003],[-105.05535699999996,40.12751700000024],[-105.05540200000002,40.14087799999993],[-105.05534199999994,40.16036200000019],[-105.05519299999997,40.18106300000005],[-105.05519399999986,40.222852000000046],[-105.05507599999999,40.232778999999994],[-105.05484999999982,40.23971299999994],[-105.05510499999997,40.24901100000011],[-105.0550889999999,40.26179300000024],[-105.0551749999999,40.29088300000001],[-105.05510799999985,40.31015900000028],[-105.05537600000002,40.319080000000156],[-105.05522099999996,40.3210450000002],[-105.05513499999995,40.33518300000026],[-105.05671999999993,40.34928100000019],[-104.98408199999994,40.34918800000031],[-104.94555100000002,40.34909900000008],[-104.94518399999993,40.37352000000004],[-104.94472499999995,40.4001640000003],[-104.94436400000001,40.417587000000196],[-104.94422199999991,40.435894000000076],[-104.94427499999995,40.44568600000002],[-104.94415299999997,40.46005000000014],[-104.94407100000001,40.490342000000055],[-104.94425100000001,40.52283700000021],[-104.94424800000002,40.53173500000031],[-104.94412,40.55516700000004],[-104.94410299999993,40.598078999999984],[-104.94401199999993,40.61080000000021],[-104.94394899999998,40.63496000000009],[-104.94348299999996,40.65257099999991],[-104.94349,40.658139000000006],[-104.94310999999999,40.69649500000014],[-104.94354299999986,40.69650900000005],[-104.9433479999999,40.74670699999996],[-104.9434629999999,40.754965000000084],[-104.94346399999989,40.7649390000002],[-104.94334200000003,40.769253000000276],[-104.94335199999995,40.79798500000027],[-104.94345399999997,40.81965100000002],[-104.94354099999998,40.826876000000084],[-104.94349699999998,40.84133900000006],[-104.943242,40.86308600000012],[-104.94321100000002,40.88226400000008],[-104.94309099999992,40.88489500000003],[-104.94312999999988,40.90294799999998],[-104.94305199999991,40.91091400000005],[-104.94315099999994,40.93839800000001],[-104.94337100000001,40.99821300000019],[-104.88245199999989,40.99818600000009],[-104.87004999999994,40.9982990000002],[-104.85527299999995,40.99804800000004],[-104.83868899999993,40.99885699999999],[-104.81830999999988,40.999197000000095],[-104.81628399999994,40.99929300000002],[-104.79786699999994,40.99945700000012],[-104.77884699999998,40.99976899999996],[-104.75966499999987,40.99998199999999],[-104.75052299999999,40.99998199999999],[-104.73656199999994,41.00021900000007],[-104.67599899999993,41.00095700000003],[-104.62551899999988,41.00142800000009],[-104.57757900000001,41.0015590000001],[-104.55013899999994,41.0015350000001],[-104.51455900000002,41.00167499999992],[-104.48721699999982,41.00169299999999],[-104.46767199999994,41.00147300000009],[-104.45288299999976,41.00149400000004],[-104.41960199999988,41.00145500000002],[-104.37551099999996,41.001523000000134],[-104.35577499999982,41.00149300000015],[-104.34370099999984,41.00154399999991],[-104.33698399999992,41.00167000000005],[-104.31463099999996,41.001643000000115],[-104.29558999999995,41.00172000000009],[-104.25748999999996,41.00167900000008],[-104.25050699999986,41.00162],[-104.21144099999987,41.00158000000005],[-104.17343599999998,41.001769999999965],[-104.162579,41.001569000000075],[-104.12550499999998,41.00170500000013],[-104.1238229999999,41.00162500000005],[-104.10458999999997,41.00154300000003],[-104.07018699999998,41.00158200000004],[-104.06443899999994,41.00133100000005],[-104.05349899999999,41.00139100000024],[-104.05152899999996,41.001527000000124],[-104.03923799999995,41.00150200000019],[-104.02338299999997,41.00188700000007],[-104.01822300000003,41.001617000000294],[-104.00050299999992,41.001780999999994],[-103.97224,41.00166400000012],[-103.97125199999988,41.00153300000011],[-103.95326899999992,41.001662000000124],[-103.93318199999999,41.001696000000265],[-103.93199199999992,41.00181400000025],[-103.9120749999999,41.00162799999998],[-103.89562899999993,41.001717999999926],[-103.85844899999995,41.00174900000002],[-103.84273699999994,41.00168300000024],[-103.79608599999995,41.00176500000009],[-103.78316199999989,41.00157000000013],[-103.76333999999986,41.001468000000216],[-103.74937799999998,41.00156700000025],[-103.72618599999998,41.001603000000046],[-103.6877449999999,41.001780999999994],[-103.6690539999999,41.001442],[-103.63628199999994,41.00164900000027],[-103.62549399999995,41.00154199999997],[-103.62021599999997,41.00171700000021],[-103.57377399999996,41.00171599999993]]]]},"properties":{"OBJECTID":63,"COUNTYFP":"123","Shape_Leng":5.09976105551,"Shape_Area":1.10610835332,"low":56122,"lowmod":100164,"LMMI":150579,"lowmoduniv":264445,"lowmod_pct":0.378770632835,"NAMELSAD10":"Weld County"}},
{"type":"Feature","geometry":{"type":"Polygon","coordinates":[[[-102.05146499999995,40.44000800000009],[-102.0515299999999,40.418162000000166],[-102.05162999999993,40.41106199999996],[-102.05144399999989,40.393027000000245],[-102.05155500000001,40.37499500000024],[-102.05179799999996,40.36006900000018],[-102.051309,40.33838100000003],[-102.05159800000001,40.282093000000145],[-102.05172600000003,40.24256500000001],[-102.05149499999993,40.22924800000004],[-102.05177700000002,40.227355999999986],[-102.05161299999992,40.22399200000001],[-102.05193300000002,40.21742799999993],[-102.05170699999991,40.212553000000185],[-102.05197099999998,40.201521999999954],[-102.05183299999993,40.19161900000006],[-102.0518679999999,40.18540900000011],[-102.05165299999987,40.17887300000007],[-102.0519109999999,40.162602000000106],[-102.05172099999999,40.162220000000275],[-102.05176599999999,40.14868400000029],[-102.05197199999998,40.14834900000028],[-102.05183799999998,40.12499100000031],[-102.05185799999987,40.08852900000022],[-102.05179099999992,40.03962100000007],[-102.05171999999999,39.985153000000196],[-102.05175399999996,39.98486499999996],[-102.051602,39.94377099999997],[-102.05147299999999,39.908101000000215],[-102.05150399999997,39.90720700000014],[-102.05135699999988,39.87499100000002],[-102.05156899999986,39.84980500000012],[-102.05136299999987,39.84347100000025],[-102.05112799999995,39.79020299999996],[-102.05095199999994,39.74999300000013],[-102.05064199999998,39.71909000000005],[-102.05074499999989,39.707539],[-102.05050799999998,39.66093599999999],[-102.050095,39.65402900000015],[-102.05029699999994,39.640953000000025],[-102.0502459999999,39.611015000000066],[-102.05004299999996,39.593817],[-102.04992499999997,39.59047900000019],[-102.04980599999999,39.57405800000009],[-102.08209099999999,39.57366800000011],[-102.08799799999997,39.573647000000165],[-102.11035099999992,39.57328200000018],[-102.12544700000001,39.573262000000284],[-102.15821999999997,39.57300700000013],[-102.17287499999998,39.57296800000012],[-102.19106899999997,39.572792000000106],[-102.20345800000001,39.57257299999992],[-102.23545699999994,39.57215500000018],[-102.25252499999999,39.571970000000135],[-102.26033099999995,39.571750000000236],[-102.27351799999991,39.571535999999924],[-102.29131399999994,39.571544000000245],[-102.29708,39.57133600000009],[-102.30508399999985,39.5713750000001],[-102.3135519999999,39.571137000000135],[-102.32656800000001,39.57107899999994],[-102.36983199999997,39.57075400000025],[-102.37547299999989,39.57086199999998],[-102.38541499999997,39.57076599999999],[-102.41055799999992,39.570745000000215],[-102.42567799999995,39.57066400000008],[-102.44669399999987,39.570788999999934],[-102.45860500000003,39.570764],[-102.45893000000001,39.57085600000016],[-102.47448199999991,39.57082600000001],[-102.5,39.57069800000005],[-102.526611,39.57065700000004],[-102.54398200000003,39.57068700000002],[-102.5520279999999,39.57063900000014],[-102.57086199999992,39.56929400000024],[-102.59299799999997,39.5691030000001],[-102.612368,39.56885700000015],[-102.61494399999998,39.56890900000019],[-102.64960999999988,39.56861400000014],[-102.718032,39.568022000000155],[-102.740925,39.56798300000031],[-102.79398199999997,39.56774800000022],[-102.80293199999994,39.5678410000001],[-102.80271600000003,39.591274999999996],[-102.80254799999989,39.621739000000275],[-102.80235599999997,39.63617700000009],[-102.80190800000003,39.68086000000005],[-102.80156299999999,39.68572300000011],[-102.80149699999987,39.698658999999964],[-102.8011689999999,39.713538000000085],[-102.80152399999997,39.713881000000015],[-102.80152299999997,39.71932500000014],[-102.80190800000003,39.720515000000205],[-102.80111899999997,39.74998700000015],[-102.80095499999987,39.78666900000013],[-102.80054499999989,39.81609700000001],[-102.80041699999987,39.83083100000016],[-102.80113199999994,39.865351000000146],[-102.802074,39.914821000000074],[-102.80261100000001,39.94052000000028],[-102.80274500000002,39.95040400000005],[-102.80298399999998,39.95998800000007],[-102.80310199999997,39.972558000000106],[-102.80332699999985,39.977484000000004],[-102.80343799999997,39.98946100000029],[-102.80376699999994,40.00255400000003],[-102.79330500000003,40.00249300000013],[-102.793455,40.02884300000022],[-102.793454,40.050774000000104],[-102.79367599999989,40.06034900000009],[-102.794082,40.08933500000029],[-102.794198,40.11826200000007],[-102.79412200000002,40.13201800000013],[-102.79419299999995,40.143125000000055],[-102.79418199999992,40.16761600000001],[-102.79411399999998,40.17541200000005],[-102.79368899999997,40.19234999999992],[-102.79337800000002,40.21416500000015],[-102.79320000000001,40.23929600000008],[-102.79330099999993,40.260501999999974],[-102.79329799999988,40.29221600000017],[-102.79347299999995,40.317686000000094],[-102.79358200000001,40.351280999999915],[-102.78017999999992,40.35114400000015],[-102.78008799999998,40.381343000000015],[-102.78008699999998,40.417768999999964],[-102.78015599999998,40.4267930000002],[-102.77996000000002,40.43844600000017],[-102.76274699999999,40.43858900000009],[-102.74855300000002,40.43853900000005],[-102.70242899999994,40.438541000000214],[-102.70167000000004,40.438646000000006],[-102.682636,40.438500000000204],[-102.664804,40.43847300000027],[-102.6249969999999,40.438601000000006],[-102.6073429999999,40.43846800000006],[-102.58521899999988,40.438514999999995],[-102.56405499999988,40.43861500000014],[-102.5512829999999,40.43862200000001],[-102.50749699999989,40.438889000000245],[-102.4546969999999,40.43930899999998],[-102.45465899999994,40.43947200000014],[-102.41625899999991,40.43992400000025],[-102.41574500000002,40.439621000000045],[-102.397874,40.43971600000009],[-102.38843199999991,40.43963500000012],[-102.36856,40.43960099999998],[-102.36419099999989,40.43968699999999],[-102.33973000000003,40.4395530000001],[-102.3232559999999,40.43951700000014],[-102.30118099999993,40.439572000000055],[-102.25046799999996,40.43948500000016],[-102.22409899999997,40.43976399999997],[-102.22169600000001,40.43963099999996],[-102.21768499999996,40.43986100000001],[-102.21496300000001,40.439610000000016],[-102.16670499999998,40.439669000000094],[-102.12545899999992,40.43972000000025],[-102.08993199999992,40.43972600000001],[-102.05146499999995,40.44000800000009]]]},"properties":{"OBJECTID":64,"COUNTYFP":"125","Shape_Leng":3.24381143275,"Shape_Area":0.647059977928,"low":2610,"lowmod":4630,"LMMI":6935,"lowmoduniv":10055,"lowmod_pct":0.46046742914,"NAMELSAD10":"Yuma County"}}
]

In [77]:
geometry[0]

{'type': 'Feature',
 'geometry': {'type': 'Polygon',
  'coordinates': [[[-103.70570099999992, 40.00136600000013],
    [-103.70569499999999, 39.98511399999995],
    [-103.70601999999997, 39.973374999999976],
    [-103.70624399999991, 39.95888000000025],
    [-103.70617099999993, 39.95397300000013],
    [-103.70617099999993, 39.93264399999998],
    [-103.70610299999998, 39.908538000000135],
    [-103.70629499999995, 39.888999000000126],
    [-103.70630299999999, 39.857679000000076],
    [-103.70626299999998, 39.82854800000007],
    [-103.70637999999997, 39.79605900000007],
    [-103.70651900000001, 39.78349100000008],
    [-103.70657699999992, 39.76555300000007],
    [-103.70654699999994, 39.739894000000106],
    [-103.72393599999998, 39.739785000000154],
    [-103.79395699999992, 39.74004500000001],
    [-103.80019900000002, 39.7402010000003],
    [-103.828213, 39.74024600000013],
    [-103.86606299999994, 39.74023599999998],
    [-103.8755099999999, 39.74029799999994],
    [-103.907515

In [78]:
geometry[0]['properties']

{'OBJECTID': 1,
 'COUNTYFP': '001',
 'Shape_Leng': 4.32102125651,
 'Shape_Area': 0.322757892201,
 'low': 132545,
 'lowmod': 231255,
 'LMMI': 334950,
 'lowmoduniv': 467200,
 'lowmod_pct': 0.494980736301,
 'NAMELSAD10': 'Adams County'}

In [79]:
places =[]
for geo in geometry:
    place ={}
    place_dict = geo['geometry']
    place['type']=place_dict['type']
    place['coordinates'] = place_dict['coordinates']
    place['name']=geo['properties']['NAMELSAD10']
    places.append(place)

In [80]:
geometry_df = pd.DataFrame(places)

In [81]:
output = output.merge(geometry_df,left_on='properties.name',right_on='name',how='left')

In [82]:
output = output.rename(index=str, columns={"coordinates": "geometry.coordinates", "type": "geometry.type"})
output=output.drop(columns='name')
output.head(5)


,properties.name,properties.article,properties.headline,properties.group_name,properties.group_id,geometry.coordinates,geometry.type
0,Adams County,<div id='county'><P>Adams county has a Median ...,218 licensed marijuana businesses,Second Income Quartile,3,"[[[-103.70570099999992, 40.00136600000013], [-...",Polygon
1,Alamosa County,<div id='county'><P>Alamosa county has a Media...,4 licensed marijuana businesses,Bottom Income Quartile,1,"[[[-105.45725499999992, 37.751464], [-105.4569...",Polygon
2,Arapahoe County,<div id='county'><P>Arapahoe county has a Medi...,134 licensed marijuana businesses,Top Income Quartile,4,"[[[[-104.9294349999999, 39.67238700000013], [-...",MultiPolygon
3,Archuleta County,<div id='county'><P>Archuleta county has a Med...,27 licensed marijuana businesses,Second Income Quartile,3,"[[[-107.300318, 37.42302000000001], [-107.2803...",Polygon
4,Bent County,<div id='county'><P>Bent county has a Median H...,2 licensed marijuana businesses,Bottom Income Quartile,1,"[[[-102.74232999999992, 38.26697100000007], [-...",Polygon


In [84]:
df3 = output[['geometry.coordinates', 'geometry.type', 'properties.name', 'properties.headline', 'properties.article','properties.group_id','properties.group_name']]
ok_json = json.loads(df3.to_json(orient='records'))

In [85]:
def process_to_geojson(file):
    geo_data = {"type": "FeatureCollection", "features":[]}
    for row in file:
        this_dict = {"type": "Feature", "properties":{}, "geometry": {}}
        for key, value in row.items():
            key_names = key.split('.')
            if key_names[0] == 'geometry':
                this_dict['geometry'][key_names[1]] = value
            if str(key_names[0]) == 'properties':
                this_dict['properties'][key_names[1]] = value
        geo_data['features'].append(this_dict)
    return geo_data


In [86]:
geo_format = process_to_geojson(ok_json)

In [87]:
#Variable name
with open('geo-data.js', 'w') as outfile:
    outfile.write("var infoData = ")
#geojson output
with open('geo-data.js', 'a') as outfile:
    json.dump(geo_format, outfile)
